<a href="https://colab.research.google.com/github/gargile/Taiwan_Stock/blob/main/%E6%AF%8F%E6%97%A5%E5%BF%85%E8%B7%91%E7%A8%8B%E5%BC%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip

# 安裝資料處理和分析套件
!pip install pandas numpy -q

# 安裝視覺化套件
!pip install matplotlib seaborn mplfinance -q
!pip install chineseize-matplotlib -q

# 安裝資料獲取套件
!pip install yfinance requests lxml html5lib -q
!pip install twstock -q

# 安裝台灣股市相關套件欸
!pip install shioaji[speed] -q
!uv add shioaji --extra speed -q
!pip install pandas-market-calendars pytz -q

# 安裝異步和通知相關套件
!pip install python-telegram-bot aiohttp nest-asyncio -q
!pip install discord-webhook -q
!pip install python-dotenv -q

# 檢查已安裝的套件
get_ipython().system('pip list | grep -E "yfinance|mplfinance|pandas|numpy|matplotlib|plotly|shioaji|beautifulsoup4"')
# -*- coding: utf-8 -*-
!pip install mplfinance chineseize_matplotlib yfinance pandas numpy matplotlib plotly discord-webhook requests aiohttp python-telegram-bot nest-asyncio -q



"""
台股多股技術分析與篩選工具
=======================
功能：
- 自動獲取台股清單
- 批次下載歷史數據
- 技術指標分析
- 綜合評分排名
- 自動通知推送
- 圖表生成與報告匯出

作者：股票分析系統
版本：v2.1
更新日期：2025-08-08
"""

# ==============================================================================
# 1. 基礎 Python 標準庫
# ==============================================================================
import os
import sys
import json
import time
import traceback
import platform
import re
import glob
import asyncio
import warnings
from io import StringIO
from datetime import datetime, timedelta
from typing import List, Dict, Any, Optional, Tuple
import logging

# ==============================================================================
# 2. 第三方套件 - 數據處理與科學計算
# ==============================================================================
import pandas as pd
import numpy as np

# ==============================================================================
# 3. 第三方套件 - 網路請求與異步處理
# ==============================================================================
import requests
import aiohttp
import nest_asyncio

# 應用 nest_asyncio 以支援 Jupyter 環境
nest_asyncio.apply()

# ==============================================================================
# 4. 第三方套件 - 金融數據源
# ==============================================================================
import yfinance as yf

# ==============================================================================
# 5. 第三方套件 - 數據可視化
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import font_manager
import mplfinance as mpf
import plotly.graph_objects as go
from pathlib import Path

# ==============================================================================
# 6. 第三方套件 - 時區與通知
# ==============================================================================
import pytz
from discord_webhook import DiscordWebhook
import telegram
# ==============================================================================
# 7. 中文字體支援
# ==============================================================================
try:
    import chineseize_matplotlib
    chineseize_matplotlib.chineseize()
    print("✅ 已載入 chineseize_matplotlib 中文字體支援")
except ImportError:
    print("⚠️ 未安裝 chineseize_matplotlib，將使用自定義字體設定")

# ==============================================================================
# 8. 全域設定
# ==============================================================================
# 警告過濾
warnings.filterwarnings('ignore')

# 時區設定
taipei_tz = pytz.timezone('Asia/Taipei')


# --- 目錄設定 ---
BASE_DIR = os.getcwd() # <--- 新增這一行
CACHE_DIR = 'cache'
RESULTS_DIR = 'results'
CHARTS_DIR = os.path.join(RESULTS_DIR, 'charts')
STOCK_LIST_PATH = os.path.join(CACHE_DIR, 'stock_list.csv')
HISTORY_DATA_CACHE_DIR = os.path.join(CACHE_DIR, 'history_data')
ETF_LIST_PATH = os.path.join(CACHE_DIR, 'etf_list.csv')
# --- 初始化目錄 ---
# 確保目錄是相對於 BASE_DIR 建立的，這樣更穩健
for directory in [os.path.join(BASE_DIR, CACHE_DIR),
                  os.path.join(BASE_DIR, RESULTS_DIR),
                  os.path.join(BASE_DIR, CHARTS_DIR),
                  os.path.join(BASE_DIR, HISTORY_DATA_CACHE_DIR)]:
    os.makedirs(directory, exist_ok=True)

# ... 後續程式碼 ...

# 初始化目錄
for directory in [CACHE_DIR, RESULTS_DIR, CHARTS_DIR, HISTORY_DATA_CACHE_DIR]:
    os.makedirs(directory, exist_ok=True)

# ==============================================================================
# 9. 通訊與通知設定
# ==============================================================================
# 建議將來使用環境變數或 .env 檔案管理密鑰，以策安全
TELEGRAM_TOKEN = "7902318521:AAEYoDMqwfHabI7L1SRiE4z33aFay42-VGE"
TELEGRAM_CHAT_ID = [879781796, 8113868436]  # 支援多用戶通知
DISCORD_WEBHOOK_URL = "https://discord.com/api/webhooks/1362715080734802102/Jma7A3VhEQrrRxIX_JW2l6rATjAZXsGXGfnJuAMqmS1QvqG_2ptg3vr_nsVnuV_PlnBl"

# ==============================================================================
# 10. 日誌系統設定
# ==============================================================================
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=[
        logging.FileHandler("stock_analysis.log", encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# ==============================================================================
# 11. 字體與環境設定函式
# ==============================================================================
def set_chinese_font():
    """
    自動設定適合當前作業系統的中文字體，用於 Matplotlib 顯示。

    支援的作業系統：
    - Windows: Microsoft JhengHei, Microsoft YaHei, SimHei
    - macOS: PingFang HK, PingFang SC, Heiti TC
    - Linux: Noto Sans CJK / WenQuanYi Zen Hei
    """
    try:
        system = platform.system()
        font_name = ""

        if system == 'Windows':
            # Windows 系統字體設定（包含繁體中文優先）
            font_candidates = [
                'Microsoft JhengHei',  # 微軟正黑體（繁體中文）
                'Microsoft YaHei',     # 微軟雅黑（簡體中文）
                'Arial Unicode MS',    # 萬國碼字體
                'SimHei',              # 黑體
                'KaiTi'                # 楷體
            ]

            for font in font_candidates:
                try:
                    # 測試字體是否可用
                    test_fig, test_ax = plt.subplots(figsize=(1, 1))
                    test_ax.text(0.5, 0.5, '測試', fontname=font)
                    plt.close(test_fig)

                    plt.rcParams['font.sans-serif'] = [font]
                    font_name = font
                    break
                except:
                    continue

        elif system == 'Darwin':  # macOS
            # macOS 系統字體設定
            font_candidates = [
                'PingFang HK',      # 蘋方-香港
                'PingFang SC',      # 蘋方-簡體中文
                'PingFang TC',      # 蘋方-繁體中文
                'Heiti TC',         # 黑體-繁體中文
                'Heiti SC',         # 黑體-簡體中文
                'STHeiti'           # 華文黑體
            ]

            for font in font_candidates:
                try:
                    plt.rcParams['font.sans-serif'] = [font]
                    font_name = font
                    break
                except:
                    continue

        else:  # Linux 和其他系統
            # Linux 系統字體路徑
            font_paths = [
                '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc',
                '/usr/share/fonts/truetype/wqy/wqy-zenhei.ttc',
                '/usr/share/fonts/truetype/arphic/ukai.ttc',
                '/usr/share/fonts/truetype/arphic/uming.ttc',
                '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf'
            ]

            found_font_path = None
            for path in font_paths:
                if os.path.exists(path):
                    found_font_path = path
                    break

            if found_font_path:
                try:
                    font_manager.fontManager.addfont(found_font_path)
                    font_prop = font_manager.FontProperties(fname=found_font_path)
                    font_name = font_prop.get_name()
                    plt.rcParams['font.sans-serif'] = [font_name]
                except Exception as e:
                    logger.warning(f"載入字體失敗: {e}")
                    font_name = "DejaVu Sans"
            else:
                print("⚠️ 未在常見路徑找到中文字體，圖表中的中文可能無法正常顯示。")
                font_name = "DejaVu Sans"

        # 設定 matplotlib 參數
        plt.rcParams['axes.unicode_minus'] = False  # 正確顯示負號

        # 設定字體大小
        plt.rcParams['font.size'] = 10
        plt.rcParams['axes.titlesize'] = 12
        plt.rcParams['axes.labelsize'] = 10
        plt.rcParams['xtick.labelsize'] = 9
        plt.rcParams['ytick.labelsize'] = 9
        plt.rcParams['legend.fontsize'] = 9

        # 設定圖表品質
        plt.rcParams['figure.dpi'] = 100
        plt.rcParams['savefig.dpi'] = 150
        plt.rcParams['savefig.bbox'] = 'tight'

        if font_name and font_name != "DejaVu Sans":
            print(f"✅ 已設定圖表字體為: {font_name} ({system})")
        else:
            print(f"⚠️ 使用預設字體: {font_name}")
            plt.rcParams['font.sans-serif'] = ['DejaVu Sans']

    except Exception as e:
        print(f"❌ 字體配置錯誤: {e}。使用預設字體。")
        plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
        plt.rcParams['axes.unicode_minus'] = False

def check_environment():
    """
    檢查執行環境與相依套件
    """
    print("🔍 檢查執行環境...")
    print(f"Python 版本: {sys.version}")
    print(f"作業系統: {platform.system()} {platform.release()}")
    print(f"當前時區: {taipei_tz}")

    # 檢查重要套件版本
    required_packages = {
        'pandas': pd.__version__,
        'numpy': np.__version__,
        'matplotlib': plt.matplotlib.__version__,
        'requests': requests.__version__,
        'yfinance': getattr(yf, '__version__', 'Unknown'),
        'aiohttp': aiohttp.__version__,
        'pytz': pytz.__version__
    }

    print("\n📦 套件版本:")
    for package, version in required_packages.items():
        print(f"  {package}: {version}")

    # 檢查目錄權限
    print(f"\n📁 工作目錄: {os.getcwd()}")
    directories_status = {
        '快取目錄': (CACHE_DIR, os.access(CACHE_DIR, os.W_OK)),
        '結果目錄': (RESULTS_DIR, os.access(RESULTS_DIR, os.W_OK)),
        '圖表目錄': (CHARTS_DIR, os.access(CHARTS_DIR, os.W_OK))
    }

    for name, (path, writable) in directories_status.items():
        status = '✅' if writable else '❌'
        print(f"{name}: {path} {status}")

    # 檢查網路連線（簡單測試）
    try:
        response = requests.get('https://httpbin.org/status/200', timeout=5)
        network_status = '✅' if response.status_code == 200 else '❌'
    except:
        network_status = '❌'

    print(f"網路連線: {network_status}")
    print("=" * 50)

def get_current_time():
    """
    獲取當前台北時間
    """
    return datetime.now(taipei_tz)

def format_timestamp(dt=None):
    """
    格式化時間戳記
    """
    if dt is None:
        dt = get_current_time()
    return dt.strftime('%Y-%m-%d %H:%M:%S %Z')

# ==============================================================================
# 12. 程式初始化
# ==============================================================================
def initialize_application():
    """
    應用程式初始化
    """
    print("🚀 台股多股技術分析與篩選工具 v2.1")
    print("=" * 50)

    # 檢查環境
    check_environment()

    # 設定中文字體
    set_chinese_font()

    # 記錄啟動時間
    start_time = get_current_time()
    logger.info(f"應用程式啟動 - {format_timestamp(start_time)}")
    logger.info(f"工作目錄: {os.getcwd()}")

    print(f"✅ 初始化完成 - {format_timestamp(start_time)}\n")

# ==============================================================================
# 通知函數
# ==============================================================================
async def send_notification(session: aiohttp.ClientSession, message: str, files: List[str] = None):
    """發送通知到 Telegram 和 Discord"""
    # Telegram
    try:
        url_msg = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage"
        payload = {"chat_id": TELEGRAM_CHAT_ID, "text": message, "parse_mode": "Markdown"}
        await session.post(url_msg, json=payload, timeout=20)
        logger.info("Telegram 摘要發送成功。")

        if files:
            url_photo = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendPhoto"
            for file_path in files:
                if os.path.exists(file_path):
                    data = aiohttp.FormData()
                    data.add_field('chat_id', TELEGRAM_CHAT_ID)
                    with open(file_path, 'rb') as f:
                        data.add_field('photo', f, filename=os.path.basename(file_path))
                        await session.post(url_photo, data=data, timeout=60)
            logger.info(f"Telegram 圖檔發送成功 ({len(files)}個)。")
    except Exception as e:
        logger.error(f"發送 Telegram 通知時異常: {e}")

    # Discord
    if MESSAGING_AVAILABLE:
        try:
            webhook = DiscordWebhook(url=DISCORD_WEBHOOK_URL, content=f"```\n{message}\n```")
            if files:
                for file_path in files:
                    if os.path.exists(file_path):
                        with open(file_path, 'rb') as f:
                            webhook.add_file(file=f.read(), filename=os.path.basename(file_path))
            response = webhook.execute()
            if response.ok:
                logger.info("Discord 通知發送成功。")
            else:
                logger.error(f"Discord 通知失敗: {response.status_code} {response.content}")
        except Exception as e:
            logger.error(f"發送 Discord 通知時異常: {e}")
    else:
        logger.warning("Discord 通知功能未啟用（套件未安裝）")

def format_notification_message(filtered_results: Dict) -> str:
    """格式化通知訊息"""
    try:
        if taipei_tz:
            current_time = datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')
        else:
            current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        if not filtered_results:
            return f"""
🤖 台股技術分析報告
📅 分析時間: {current_time}

❌ 本次分析未找到符合條件的優質股票
💡 建議調整篩選條件或關注市場變化
"""

        message = f"""
🤖 台股技術分析報告
📅 分析時間: {current_time}
🎯 找到 {len(filtered_results)} 支優質股票

📊 TOP 5 推薦股票:
"""

        for rank, (stock_id, result) in enumerate(list(filtered_results.items())[:5], 1):
            trend = result.get('trend_analysis', {})
            tech = result.get('technical_analysis', {})
            recommendation = result.get('recommendation', {})

            message += f"""
{rank}. {result.get('stock_name', '')} ({stock_id})
   💰 價格: {result.get('current_price', 0):.2f} TWD
   📈 評分: {result.get('combined_score', 0):.1f}/100
   🎯 建議: {recommendation.get('action', '觀望')}
   📊 趨勢: {trend.get('trend', '盤整')}
   🔍 RSI: {tech.get('rsi_value', 50):.1f}
"""

        message += f"""
⚠️  投資提醒:
• 本分析僅供參考，投資有風險
• 建議結合基本面分析
• 請做好風險控制
"""

        return message.strip()

    except Exception as e:
        logger.error(f"格式化通知訊息時發生錯誤: {e}")
        return f"台股分析完成，但訊息格式化失敗: {str(e)}"

print("✅ 環境設定完成！")
print(f"📁 工作目錄: {BASE_DIR}")
print(f"📊 圖表目錄: {CHARTS_DIR}")
print(f"🔔 通知功能: {'已啟用' if MESSAGING_AVAILABLE else '部分啟用（僅 Telegram）'}")

上面是安裝程式碼

In [ ]:

import asyncio
import aiohttp
import pandas as pd
import numpy as np
import yfinance as yf
import logging
import traceback
from datetime import datetime, timezone, timedelta
from typing import Dict, List, Any, Optional
import json
import os
import time
import requests
import random
from io import StringIO

# 如果您在 Jupyter Notebook 中，請使用這個版本
import nest_asyncio

# 安裝 nest_asyncio（如果尚未安裝）
try:
    import nest_asyncio
except ImportError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "nest_asyncio"])
    import nest_asyncio

# 應用 nest_asyncio 以允許嵌套事件循環
nest_asyncio.apply()

# 設定日誌
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# 台北時區
taipei_tz = timezone(timedelta(hours=8))

# Telegram 設定
TELEGRAM_BOT_TOKEN = "7902318521:AAEYoDMqwfHabI7L1SRiE4z33aFay42-VGE"
TELEGRAM_CHAT_IDS = [879781796, 8113868436]

# Discord 設定 (可選)
DISCORD_WEBHOOK_URL = "https://discord.com/api/webhooks/1362715080734802102/Jma7A3VhEQrrRxIX_JW2l6rATjAZXsGXGfnJuAMqmS1QvqG_2ptg3vr_nsVnuV_PlnBl"
MESSAGING_AVAILABLE = False

try:
    from discord_webhook import DiscordWebhook
    MESSAGING_AVAILABLE = True
except ImportError:
    MESSAGING_AVAILABLE = False
    logger.info("Discord webhook 套件未安裝，將跳過 Discord 通知")

class StockAnalyzer:
    def __init__(self):
        """初始化股票分析器"""
        self.config = {
            'rsi_period': 14,
            'macd_fast': 12,
            'macd_slow': 26,
            'macd_signal': 9,
            'bb_period': 20,
            'bb_std': 2,
            'kd_period': 14,
            'volume_ma_period': 20,
            'min_volume_lots': 1000,  # 最小成交量（張）
            'trend_weights': {
                'price_trend': 0.3,
                'volume_trend': 0.2,
                'technical_score': 0.5
            }
        }
        self.taiwan_stocks = None
        self.stock_list_path = "taiwan_stocks_cache.csv"

    def get_taiwan_stocks(self, force_update=True):
        """獲取台灣股票清單"""
        if not force_update and os.path.exists(self.stock_list_path):
            if (time.time() - os.path.getmtime(self.stock_list_path)) < 86400:
                logger.info("從快取載入股票列表。")
                return pd.read_csv(self.stock_list_path, dtype={'stock_id': str})

        logger.info("從台灣證交所網站獲取最新股票清單...")
        urls = {
            "上市": "https://isin.twse.com.tw/isin/C_public.jsp?strMode=2",
            "上櫃": "https://isin.twse.com.tw/isin/C_public.jsp?strMode=4"
        }
        all_stocks_df = []
        headers = {'User-Agent': 'Mozilla/5.0'}

        for market_name, url in urls.items():
            try:
                res = requests.get(url, headers=headers, timeout=30)
                res.encoding = 'big5'
                html_dfs = pd.read_html(StringIO(res.text))
                df = html_dfs[0].copy()  # 修正：使用 copy() 避免警告
                df.columns = df.iloc[0]
                df = df.iloc[1:].copy()  # 修正：使用 copy()
                df.loc[:, 'market'] = market_name  # 修正：使用 .loc 設定值
                all_stocks_df.append(df)
            except Exception as e:
                logger.error(f"獲取 {market_name} 股票列表失敗: {e}")
                continue

        if not all_stocks_df:
            logger.error("無法從網站獲取任何股票數據，使用備用清單。")
            return self._get_backup_stock_list()

        try:
            df = pd.concat(all_stocks_df, ignore_index=True)
            df[['stock_id', 'stock_name']] = df['有價證券代號及名稱'].str.split(r'\s+', n=1, expand=True)
            df = df[df['stock_id'].str.match(r'^\d{4}$', na=False)].copy()
            exclude = ['ETF', 'ETN', 'TDR', '受益', '指數', '購', '牛', '熊', '存託憑證']
            df = df[~df['stock_name'].str.contains('|'.join(exclude), na=False)].copy()

            # 修正：使用 .loc 設定 yahoo_symbol
            df.loc[:, 'yahoo_symbol'] = df.apply(
                lambda row: f"{row['stock_id']}.TW" if '上市' in row['market'] else f"{row['stock_id']}.TWO", axis=1)

            final_df = df[['stock_id', 'stock_name', 'market', '產業別', 'yahoo_symbol']].rename(columns={'產業別': 'industry'})
            final_df = final_df.drop_duplicates(subset=['stock_id']).reset_index(drop=True)
            final_df.to_csv(self.stock_list_path, index=False)
            logger.info(f"成功獲取 {len(final_df)} 支股票清單並保存快取。")
            return final_df
        except Exception as e:
            logger.error(f"處理股票清單時發生錯誤: {e}")
            return self._get_backup_stock_list()

    def _get_backup_stock_list(self) -> pd.DataFrame:
        """備用股票清單"""
        try:
            logger.info("使用備用股票清單")

            # 擴展知名股票列表
            famous_stocks = [
                {'stock_id': '2330', 'stock_name': '台積電', 'market': '上市', 'yahoo_symbol': '2330.TW', 'industry': '半導體'},
                {'stock_id': '2317', 'stock_name': '鴻海', 'market': '上市', 'yahoo_symbol': '2317.TW', 'industry': '電子'},
                {'stock_id': '2454', 'stock_name': '聯發科', 'market': '上市', 'yahoo_symbol': '2454.TW', 'industry': '半導體'},
                {'stock_id': '2881', 'stock_name': '富邦金', 'market': '上市', 'yahoo_symbol': '2881.TW', 'industry': '金融'},
                {'stock_id': '2882', 'stock_name': '國泰金', 'market': '上市', 'yahoo_symbol': '2882.TW', 'industry': '金融'},
                {'stock_id': '2412', 'stock_name': '中華電', 'market': '上市', 'yahoo_symbol': '2412.TW', 'industry': '通信'},
                {'stock_id': '1301', 'stock_name': '台塑', 'market': '上市', 'yahoo_symbol': '1301.TW', 'industry': '塑膠'},
                {'stock_id': '1303', 'stock_name': '南亞', 'market': '上市', 'yahoo_symbol': '1303.TW', 'industry': '塑膠'},
                {'stock_id': '2308', 'stock_name': '台達電', 'market': '上市', 'yahoo_symbol': '2308.TW', 'industry': '電子'},
                {'stock_id': '2303', 'stock_name': '聯電', 'market': '上市', 'yahoo_symbol': '2303.TW', 'industry': '半導體'},
                {'stock_id': '2002', 'stock_name': '中鋼', 'market': '上市', 'yahoo_symbol': '2002.TW', 'industry': '鋼鐵'},
                {'stock_id': '2886', 'stock_name': '兆豐金', 'market': '上市', 'yahoo_symbol': '2886.TW', 'industry': '金融'},
                {'stock_id': '2891', 'stock_name': '中信金', 'market': '上市', 'yahoo_symbol': '2891.TW', 'industry': '金融'},
                {'stock_id': '3008', 'stock_name': '大立光', 'market': '上市', 'yahoo_symbol': '3008.TW', 'industry': '光學'},
                {'stock_id': '2357', 'stock_name': '華碩', 'market': '上市', 'yahoo_symbol': '2357.TW', 'industry': '電腦'},
                {'stock_id': '2382', 'stock_name': '廣達', 'market': '上市', 'yahoo_symbol': '2382.TW', 'industry': '電腦'},
                {'stock_id': '2395', 'stock_name': '研華', 'market': '上市', 'yahoo_symbol': '2395.TW', 'industry': '電腦'},
                {'stock_id': '3711', 'stock_name': '日月光投控', 'market': '上市', 'yahoo_symbol': '3711.TW', 'industry': '半導體'},
                {'stock_id': '2409', 'stock_name': '友達', 'market': '上市', 'yahoo_symbol': '2409.TW', 'industry': '面板'},
                {'stock_id': '2884', 'stock_name': '玉山金', 'market': '上市', 'yahoo_symbol': '2884.TW', 'industry': '金融'},
                # 新增更多股票
                {'stock_id': '2474', 'stock_name': '可成', 'market': '上市', 'yahoo_symbol': '2474.TW', 'industry': '電子'},
                {'stock_id': '2408', 'stock_name': '南亞科', 'market': '上市', 'yahoo_symbol': '2408.TW', 'industry': '半導體'},
                {'stock_id': '2301', 'stock_name': '光寶科', 'market': '上市', 'yahoo_symbol': '2301.TW', 'industry': '電子'},
                {'stock_id': '2207', 'stock_name': '和泰車', 'market': '上市', 'yahoo_symbol': '2207.TW', 'industry': '汽車'},
                {'stock_id': '1216', 'stock_name': '統一', 'market': '上市', 'yahoo_symbol': '1216.TW', 'industry': '食品'},
                {'stock_id': '1101', 'stock_name': '台泥', 'market': '上市', 'yahoo_symbol': '1101.TW', 'industry': '水泥'},
                {'stock_id': '2105', 'stock_name': '正新', 'market': '上市', 'yahoo_symbol': '2105.TW', 'industry': '橡膠'},
                {'stock_id': '2912', 'stock_name': '統一超', 'market': '上市', 'yahoo_symbol': '2912.TW', 'industry': '貿易百貨'},
                {'stock_id': '2885', 'stock_name': '元大金', 'market': '上市', 'yahoo_symbol': '2885.TW', 'industry': '金融'},
                {'stock_id': '2892', 'stock_name': '第一金', 'market': '上市', 'yahoo_symbol': '2892.TW', 'industry': '金融'},
            ]

            df = pd.DataFrame(famous_stocks)
            logger.info(f"備用股票清單包含 {len(df)} 支股票")
            return df

        except Exception as e:
            logger.error(f"生成備用股票清單時發生錯誤: {e}")
            return pd.DataFrame()

    def check_volume_filter(self, df: pd.DataFrame) -> bool:
        """檢查成交量是否符合篩選條件（每日至少1000張）"""
        try:
            if df.empty or 'Volume' not in df.columns:
                return False

            # 取最近20天的平均成交量
            recent_volume = df['Volume'].tail(20).mean()

            # 轉換為張數（1張 = 1000股）
            volume_lots = recent_volume / 1000

            # 檢查是否符合最小成交量要求
            meets_volume = volume_lots >= self.config['min_volume_lots']

            if meets_volume:
                logger.debug(f"成交量符合條件: {volume_lots:.0f} 張/日 (≥ {self.config['min_volume_lots']} 張)")
            else:
                logger.debug(f"成交量不符合條件: {volume_lots:.0f} 張/日 (< {self.config['min_volume_lots']} 張)")

            return meets_volume

        except Exception as e:
            logger.error(f"檢查成交量篩選時發生錯誤: {e}")
            return False

    def fetch_yfinance_data(self, stock_id: str, period: str = "1y", interval: str = "1d", retries: int = 3):
        """獲取股票數據（同步版本）"""
        for attempt in range(retries):
            try:
                df = yf.download(tickers=stock_id, period=period, interval=interval, progress=False, auto_adjust=True)
                if df.empty:
                    logger.warning(f"[{stock_id}] 無數據返回")
                    return pd.DataFrame()

                if isinstance(df.columns, pd.MultiIndex):
                    df.columns = df.columns.get_level_values(0)

                required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
                for col in required_cols:
                    if col in df.columns:
                        df[col] = pd.to_numeric(df[col], errors='coerce')

                df = df.dropna(subset=required_cols)
                return df.reset_index()

            except Exception as e:
                if attempt < retries - 1:
                    sleep_time = 0.5 * (2 ** attempt) + random.uniform(0, 1)
                    logger.warning(f"[{stock_id}] 第 {attempt + 1}/{retries} 次嘗試失敗: {e}。等待 {sleep_time:.2f} 秒後重試...")
                    time.sleep(sleep_time)
                else:
                    logger.error(f"[{stock_id}] 獲取數據失敗。")
                    return pd.DataFrame()
        return pd.DataFrame()

    def calculate_technical_indicators(self, df: pd.DataFrame) -> pd.DataFrame:
        """計算技術指標"""
        try:
            if df.empty:
                return df

            df = df.copy()

            # RSI
            df['RSI'] = self.calculate_rsi(df['Close'], self.config['rsi_period'])

            # MACD
            macd_line, macd_signal, macd_histogram = self.calculate_macd(
                df['Close'],
                self.config['macd_fast'],
                self.config['macd_slow'],
                self.config['macd_signal']
            )
            df['MACD'] = macd_line
            df['MACD_Signal'] = macd_signal
            df['MACD_Histogram'] = macd_histogram

            # 布林帶
            bb_upper, bb_middle, bb_lower = self.calculate_bollinger_bands(
                df['Close'],
                self.config['bb_period'],
                self.config['bb_std']
            )
            df['BB_Upper'] = bb_upper
            df['BB_Middle'] = bb_middle
            df['BB_Lower'] = bb_lower

            # KD指標
            k_percent, d_percent = self.calculate_kd(
                df['High'],
                df['Low'],
                df['Close'],
                self.config['kd_period']
            )
            df['K_Percent'] = k_percent
            df['D_Percent'] = d_percent

            # 移動平均線
            df['MA5'] = df['Close'].rolling(window=5).mean()
            df['MA10'] = df['Close'].rolling(window=10).mean()
            df['MA20'] = df['Close'].rolling(window=20).mean()

            # 成交量移動平均
            if 'Volume' in df.columns:
                df['Volume_MA'] = df['Volume'].rolling(window=self.config['volume_ma_period']).mean()

            return df

        except Exception as e:
            logger.error(f"計算技術指標時發生錯誤: {e}")
            return df

    def calculate_rsi(self, prices: pd.Series, period: int = 14) -> pd.Series:
        """計算RSI"""
        try:
            delta = prices.diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
            rs = gain / loss
            rsi = 100 - (100 / (1 + rs))
            return rsi
        except Exception as e:
            logger.error(f"計算RSI時發生錯誤: {e}")
            return pd.Series(index=prices.index, data=50)

    def calculate_macd(self, prices: pd.Series, fast: int = 12, slow: int = 26, signal: int = 9):
        """計算MACD"""
        try:
            ema_fast = prices.ewm(span=fast).mean()
            ema_slow = prices.ewm(span=slow).mean()
            macd_line = ema_fast - ema_slow
            macd_signal = macd_line.ewm(span=signal).mean()
            macd_histogram = macd_line - macd_signal
            return macd_line, macd_signal, macd_histogram
        except Exception as e:
            logger.error(f"計算MACD時發生錯誤: {e}")
            zero_series = pd.Series(index=prices.index, data=0)
            return zero_series, zero_series, zero_series

    def calculate_bollinger_bands(self, prices: pd.Series, period: int = 20, std_dev: int = 2):
        """計算布林帶"""
        try:
            middle = prices.rolling(window=period).mean()
            std = prices.rolling(window=period).std()
            upper = middle + (std * std_dev)
            lower = middle - (std * std_dev)
            return upper, middle, lower
        except Exception as e:
            logger.error(f"計算布林帶時發生錯誤: {e}")
            zero_series = pd.Series(index=prices.index, data=0)
            return zero_series, zero_series, zero_series

    def calculate_kd(self, high: pd.Series, low: pd.Series, close: pd.Series, period: int = 14):
        """計算KD指標"""
        try:
            lowest_low = low.rolling(window=period).min()
            highest_high = high.rolling(window=period).max()

            k_percent = 100 * ((close - lowest_low) / (highest_high - lowest_low))
            k_percent = k_percent.rolling(window=3).mean()
            d_percent = k_percent.rolling(window=3).mean()

            return k_percent, d_percent
        except Exception as e:
            logger.error(f"計算KD時發生錯誤: {e}")
            fifty_series = pd.Series(index=high.index, data=50)
            return fifty_series, fifty_series

    def analyze_trend(self, df: pd.DataFrame) -> Dict[str, Any]:
        """趨勢分析"""
        try:
            if df.empty or len(df) < 20:
                return {'trend': '盤整', 'strength': 0, 'price_change_5d': 0, 'price_change_20d': 0}

            current_price = float(df['Close'].iloc[-1])
            price_5d_ago = float(df['Close'].iloc[-6]) if len(df) >= 6 else current_price
            price_20d_ago = float(df['Close'].iloc[-21]) if len(df) >= 21 else current_price

            change_5d = ((current_price - price_5d_ago) / price_5d_ago * 100) if price_5d_ago != 0 else 0
            change_20d = ((current_price - price_20d_ago) / price_20d_ago * 100) if price_20d_ago != 0 else 0

            # 移動平均線趨勢
            ma5_current = float(df['MA5'].iloc[-1]) if 'MA5' in df.columns and not pd.isna(df['MA5'].iloc[-1]) else current_price
            ma20_current = float(df['MA20'].iloc[-1]) if 'MA20' in df.columns and not pd.isna(df['MA20'].iloc[-1]) else current_price

            # 判斷趨勢
            if change_5d > 3 and change_20d > 5 and current_price > ma5_current > ma20_current:
                trend = '強勢上漲'
                strength = 3
            elif change_5d > 1 and change_20d > 2 and current_price > ma5_current:
                trend = '上漲'
                strength = 2
            elif change_5d < -3 and change_20d < -5 and current_price < ma5_current < ma20_current:
                trend = '強勢下跌'
                strength = -3
            elif change_5d < -1 and change_20d < -2 and current_price < ma5_current:
                trend = '下跌'
                strength = -2
            else:
                trend = '盤整'
                strength = 0

            return {
                'trend': trend,
                'strength': strength,
                'price_change_5d': change_5d,
                'price_change_20d': change_20d,
                'ma5_position': ma5_current,
                'ma20_position': ma20_current
            }

        except Exception as e:
            logger.error(f"趨勢分析時發生錯誤: {e}")
            return {'trend': '盤整', 'strength': 0, 'price_change_5d': 0, 'price_change_20d': 0}

    def analyze_volume(self, df: pd.DataFrame) -> Dict[str, Any]:
        """成交量分析"""
        try:
            if df.empty or 'Volume' not in df.columns or len(df) < 20:
                return {'volume_trend': '正常', 'volume_ratio': 1.0, 'volume_signal': '中性', 'avg_volume_lots': 0}

            current_volume = float(df['Volume'].iloc[-1])
            avg_volume_20d = float(df['Volume'].rolling(window=20).mean().iloc[-1])

            volume_ratio = current_volume / avg_volume_20d if avg_volume_20d > 0 else 1.0
            avg_volume_lots = avg_volume_20d / 1000  # 轉換為張數

            if volume_ratio > 2.0:
                volume_trend = '爆量'
                volume_signal = '強烈'
            elif volume_ratio > 1.5:
                volume_trend = '放量'
                volume_signal = '積極'
            elif volume_ratio < 0.5:
                volume_trend = '縮量'
                volume_signal = '消極'
            else:
                volume_trend = '正常'
                volume_signal = '中性'

            return {
                'volume_trend': volume_trend,
                'volume_ratio': volume_ratio,
                'volume_signal': volume_signal,
                'avg_volume_lots': avg_volume_lots
            }

        except Exception as e:
            logger.error(f"成交量分析時發生錯誤: {e}")
            return {'volume_trend': '正常', 'volume_ratio': 1.0, 'volume_signal': '中性', 'avg_volume_lots': 0}

    def analyze_technical_indicators(self, df: pd.DataFrame) -> Dict[str, Any]:
        """技術指標分析"""
        try:
            if df.empty:
                return {
                    'rsi_signal': '中性', 'rsi_value': 50,
                    'macd_signal': '中性', 'macd_value': 0,
                    'kd_signal': '中性', 'k_value': 50, 'd_value': 50,
                    'bb_signal': '中性', 'bb_position': 0.5,
                    'score': 50
                }

            signals = {}
            score = 50

            # RSI 分析
            if 'RSI' in df.columns and not df['RSI'].empty:
                rsi_value = float(df['RSI'].iloc[-1]) if not pd.isna(df['RSI'].iloc[-1]) else 50
                signals['rsi_value'] = rsi_value

                if rsi_value > 80:
                    signals['rsi_signal'] = '超買'
                    score -= 15
                elif rsi_value > 70:
                    signals['rsi_signal'] = '偏高'
                    score -= 5
                elif rsi_value < 20:
                    signals['rsi_signal'] = '超賣'
                    score += 15
                elif rsi_value < 30:
                    signals['rsi_signal'] = '偏低'
                    score += 5
                else:
                    signals['rsi_signal'] = '中性'
            else:
                signals['rsi_signal'] = '中性'
                signals['rsi_value'] = 50

            # MACD 分析
            if all(col in df.columns for col in ['MACD', 'MACD_Signal']) and len(df) >= 2:
                macd_current = float(df['MACD'].iloc[-1]) if not pd.isna(df['MACD'].iloc[-1]) else 0
                macd_signal_current = float(df['MACD_Signal'].iloc[-1]) if not pd.isna(df['MACD_Signal'].iloc[-1]) else 0
                macd_prev = float(df['MACD'].iloc[-2]) if not pd.isna(df['MACD'].iloc[-2]) else 0
                macd_signal_prev = float(df['MACD_Signal'].iloc[-2]) if not pd.isna(df['MACD_Signal'].iloc[-2]) else 0

                signals['macd_value'] = macd_current

                if macd_prev <= macd_signal_prev and macd_current > macd_signal_current:
                    signals['macd_signal'] = '黃金交叉'
                    score += 20
                elif macd_prev >= macd_signal_prev and macd_current < macd_signal_current:
                    signals['macd_signal'] = '死亡交叉'
                    score -= 20
                elif macd_current > macd_signal_current:
                    signals['macd_signal'] = '多頭'
                    score += 5
                elif macd_current < macd_signal_current:
                    signals['macd_signal'] = '空頭'
                    score -= 5
                else:
                    signals['macd_signal'] = '中性'
            else:
                signals['macd_signal'] = '中性'
                signals['macd_value'] = 0

            # KD 分析
            if all(col in df.columns for col in ['K_Percent', 'D_Percent']):
                k_value = float(df['K_Percent'].iloc[-1]) if not pd.isna(df['K_Percent'].iloc[-1]) else 50
                d_value = float(df['D_Percent'].iloc[-1]) if not pd.isna(df['D_Percent'].iloc[-1]) else 50

                signals['k_value'] = k_value
                signals['d_value'] = d_value

                if k_value > 80 and d_value > 80:
                    signals['kd_signal'] = '超買'
                    score -= 10
                elif k_value < 20 and d_value < 20:
                    signals['kd_signal'] = '超賣'
                    score += 10
                elif k_value > d_value:
                    signals['kd_signal'] = '偏多'
                    score += 3
                elif k_value < d_value:
                    signals['kd_signal'] = '偏空'
                    score -= 3
                else:
                    signals['kd_signal'] = '中性'
            else:
                signals['kd_signal'] = '中性'
                signals['k_value'] = 50
                signals['d_value'] = 50

            # 布林帶分析
            if all(col in df.columns for col in ['BB_Upper', 'BB_Lower', 'Close']):
                current_price = float(df['Close'].iloc[-1])
                bb_upper = float(df['BB_Upper'].iloc[-1]) if not pd.isna(df['BB_Upper'].iloc[-1]) else current_price
                bb_lower = float(df['BB_Lower'].iloc[-1]) if not pd.isna(df['BB_Lower'].iloc[-1]) else current_price

                if bb_upper != bb_lower:
                    bb_position = (current_price - bb_lower) / (bb_upper - bb_lower)
                    signals['bb_position'] = bb_position

                    if bb_position > 0.8:
                        signals['bb_signal'] = '接近上軌'
                        score -= 5
                    elif bb_position < 0.2:
                        signals['bb_signal'] = '接近下軌'
                        score += 5
                    else:
                        signals['bb_signal'] = '中性'
                else:
                    signals['bb_signal'] = '中性'
                    signals['bb_position'] = 0.5
            else:
                signals['bb_signal'] = '中性'
                signals['bb_position'] = 0.5

            score = max(0, min(100, score))
            signals['score'] = score

            return signals

        except Exception as e:
            logger.error(f"技術指標分析時發生錯誤: {e}")
            return {
                'rsi_signal': '中性', 'rsi_value': 50,
                'macd_signal': '中性', 'macd_value': 0,
                'kd_signal': '中性', 'k_value': 50, 'd_value': 50,
                'bb_signal': '中性', 'bb_position': 0.5,
                'score': 50
            }

    def calculate_combined_score(self, trend_analysis: Dict, volume_analysis: Dict, technical_analysis: Dict) -> float:
        """計算綜合分數"""
        try:
            base_score = 50

            # 趨勢分數 (權重: 30%)
            trend_score = 0
            trend_strength = trend_analysis.get('strength', 0)
            if trend_strength > 0:
                trend_score = min(30, trend_strength * 10)
            elif trend_strength < 0:
                trend_score = max(-30, trend_strength * 10)

            # 成交量分數 (權重: 20%)
            volume_score = 0
            volume_ratio = volume_analysis.get('volume_ratio', 1.0)
            if volume_ratio > 1.5:
                volume_score = 10
            elif volume_ratio > 1.2:
                volume_score = 5
            elif volume_ratio < 0.7:
                volume_score = -5

            # 技術指標分數 (權重: 50%)
            tech_score = technical_analysis.get('score', 50) - 50

            # 計算加權總分
            total_score = base_score + (trend_score * 0.3) + (volume_score * 0.2) + (tech_score * 0.5)

            return max(0, min(100, total_score))

        except Exception as e:
            logger.error(f"計算綜合分數時發生錯誤: {e}")
            return 50.0

    def generate_recommendation(self, combined_score: float, trend_analysis: Dict, technical_analysis: Dict) -> Dict[str, Any]:
        """生成投資建議"""
        try:
            recommendation = "持有"
            confidence = "中等"
            reasons = []

            if combined_score >= 75:
                recommendation = "強力買進"
                confidence = "高"
            elif combined_score >= 60:
                recommendation = "買進"
                confidence = "中高"
            elif combined_score >= 40:
                recommendation = "持有"
                confidence = "中等"
            elif combined_score >= 25:
                recommendation = "賣出"
                confidence = "中高"
            else:
                recommendation = "強力賣出"
                confidence = "高"

            # 生成建議原因
            trend = trend_analysis.get('trend', '盤整')
            if '上漲' in trend:
                reasons.append(f"價格趨勢：{trend}")
            elif '下跌' in trend:
                reasons.append(f"價格趨勢：{trend}")

            rsi_signal = technical_analysis.get('rsi_signal', '中性')
            if rsi_signal in ['超賣', '偏低']:
                reasons.append(f"RSI指標：{rsi_signal}，可能反彈")
            elif rsi_signal in ['超買', '偏高']:
                reasons.append(f"RSI指標：{rsi_signal}，注意回調")

            macd_signal = technical_analysis.get('macd_signal', '中性')
            if macd_signal == '黃金交叉':
                reasons.append("MACD出現黃金交叉")
            elif macd_signal == '死亡交叉':
                reasons.append("MACD出現死亡交叉")

            return {
                'recommendation': recommendation,
                'confidence': confidence,
                'score': combined_score,
                'reasons': reasons
            }

        except Exception as e:
            logger.error(f"生成投資建議時發生錯誤: {e}")
            return {
                'recommendation': '持有',
                'confidence': '低',
                'score': 50,
                'reasons': ['分析過程出現錯誤']
            }

    async def analyze_stock_async(self, session: aiohttp.ClientSession, stock_info: Dict) -> Dict[str, Any]:
        """異步分析單支股票"""
        symbol = stock_info['yahoo_symbol']
        stock_name = stock_info['stock_name']

        try:
            logger.info(f"開始分析股票: {stock_name} ({symbol})")

            # 獲取股票數據
            df = self.fetch_yfinance_data(symbol)
            if df is None or df.empty or len(df) < 60:
                return {
                    'symbol': symbol,
                    'stock_name': stock_name,
                    'success': False,
                    'error': '無法獲取足夠的股票數據'
                }

            # 檢查成交量篩選條件
            if not self.check_volume_filter(df):
                return {
                    'symbol': symbol,
                    'stock_name': stock_name,
                    'success': False,
                    'error': f'成交量不符合條件（需≥{self.config["min_volume_lots"]}張/日）'
                }

            # 計算技術指標
            df_with_indicators = self.calculate_technical_indicators(df)

            # 進行各項分析
            trend_analysis = self.analyze_trend(df_with_indicators)
            volume_analysis = self.analyze_volume(df_with_indicators)
            technical_analysis = self.analyze_technical_indicators(df_with_indicators)

            # 計算綜合分數
            combined_score = self.calculate_combined_score(trend_analysis, volume_analysis, technical_analysis)

            # 生成投資建議
            recommendation = self.generate_recommendation(combined_score, trend_analysis, technical_analysis)

            # 獲取當前價格資訊
            current_price = float(df['Close'].iloc[-1])
            price_change = trend_analysis.get('price_change_5d', 0)

            result = {
                'symbol': symbol,
                'stock_name': stock_name,
                'stock_id': stock_info.get('stock_id', ''),
                'market': stock_info.get('market', ''),
                'industry': stock_info.get('industry', ''),
                'success': True,
                'current_price': current_price,
                'price_change_5d': price_change,
                'combined_score': combined_score,
                'trend_analysis': trend_analysis,
                'volume_analysis': volume_analysis,
                'technical_analysis': technical_analysis,
                'recommendation': recommendation,
                'analysis_time': datetime.now(taipei_tz).isoformat()
            }

            logger.info(f"完成分析: {stock_name} - 分數: {combined_score:.1f} - 建議: {recommendation['recommendation']} - 成交量: {volume_analysis.get('avg_volume_lots', 0):.0f}張/日")
            return result

        except Exception as e:
            logger.error(f"分析股票 {symbol} 時發生錯誤: {e}")
            return {
                'symbol': symbol,
                'stock_name': stock_name,
                'success': False,
                'error': str(e)
            }

    async def analyze_all_stocks(self) -> Dict[str, Any]:
        """分析所有股票（加入成交量篩選）"""
        try:
            # 獲取完整股票清單
            if self.taiwan_stocks is None:
                self.taiwan_stocks = self.get_taiwan_stocks()

            if self.taiwan_stocks.empty:
                logger.error("無法獲取股票清單")
                return {}

            logger.info(f"準備分析 {len(self.taiwan_stocks)} 支股票（包含成交量篩選）")

            results = {}
            failed_count = 0
            volume_filtered_count = 0

            # 使用 aiohttp 進行異步分析
            async with aiohttp.ClientSession() as session:
                # 創建分析任務
                tasks = []
                for _, stock_info in self.taiwan_stocks.iterrows():
                    task = self.analyze_stock_async(session, stock_info.to_dict())
                    tasks.append(task)

                # 分批執行任務以避免過載
                batch_size = 10
                for i in range(0, len(tasks), batch_size):
                    batch_tasks = tasks[i:i+batch_size]
                    batch_results = await asyncio.gather(*batch_tasks, return_exceptions=True)

                    # 處理批次結果
                    for result in batch_results:
                        if isinstance(result, Exception):
                            logger.error(f"分析任務異常: {result}")
                            failed_count += 1
                            continue

                        if isinstance(result, dict) and 'symbol' in result:
                            if result.get('success', False):
                                results[result['symbol']] = result
                            else:
                                error_msg = result.get('error', '')
                                if '成交量不符合條件' in error_msg:
                                    volume_filtered_count += 1
                                else:
                                    failed_count += 1

                    # 批次間暫停避免過載
                    if i + batch_size < len(tasks):
                        await asyncio.sleep(2)

            logger.info(f"分析完成統計：")
            logger.info(f"  - 成功分析: {len(results)} 支")
            logger.info(f"  - 成交量篩選淘汰: {volume_filtered_count} 支")
            logger.info(f"  - 其他失敗: {failed_count} 支")

            return results

        except Exception as e:
            logger.error(f"批量分析股票時發生錯誤: {e}")
            return {}

def format_analysis_message(results: Dict[str, Any], limit: int = 10) -> str:
    """格式化分析結果為訊息（顯示前十名）"""
    try:
        if not results:
            return "❌ 未能獲取任何分析結果"

        # 過濾成功的結果並按分數排序
        successful_results = [
            result for result in results.values()
            if result.get('success', False) and result.get('combined_score', 0) > 0
        ]

        successful_results.sort(key=lambda x: x.get('combined_score', 0), reverse=True)

        # 統計資訊
        total_analyzed = len(results)
        successful_count = len(successful_results)

        # 建立訊息
        message_parts = []
        message_parts.append("🏆 台股技術分析 - 前十名優質股票")
        message_parts.append("=" * 40)
        message_parts.append(f"📈 分析時間: {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}")
        message_parts.append(f"📊 成功分析: {successful_count} 支股票")
        message_parts.append(f"🔍 成交量篩選: ≥1000張/日")
        message_parts.append("")

        if not successful_results:
            message_parts.append("❌ 沒有符合條件的優質股票")
            return "\n".join(message_parts)

        # 顯示前十名優質股票
        top_stocks = successful_results[:limit]
        message_parts.append(f"🏆 前 {len(top_stocks)} 名優質股票:")
        message_parts.append("-" * 40)

        for i, result in enumerate(top_stocks, 1):
            stock_name = result.get('stock_name', 'Unknown')
            stock_id = result.get('stock_id', 'Unknown')
            score = result.get('combined_score', 0)
            recommendation = result.get('recommendation', {})
            rec_text = recommendation.get('recommendation', '持有')
            current_price = result.get('current_price', 0)
            price_change = result.get('price_change_5d', 0)
            volume_info = result.get('volume_analysis', {})
            avg_volume_lots = volume_info.get('avg_volume_lots', 0)

            # 價格變化符號
            change_symbol = "📈" if price_change > 0 else "📉" if price_change < 0 else "➡️"

            # 建議符號
            rec_symbol = "🚀" if "強力買" in rec_text else "📈" if "買" in rec_text else "📊" if "持有" in rec_text else "📉"

            message_parts.append(f"{i}. {stock_name} ({stock_id})")
            message_parts.append(f"   💰 價格: {current_price:.2f} ({change_symbol}{price_change:+.2f}%)")
            message_parts.append(f"   ⭐ 評分: {score:.1f}/100")
            message_parts.append(f"   {rec_symbol} 建議: {rec_text}")
            message_parts.append(f"   📊 成交量: {avg_volume_lots:.0f}張/日")

            # 添加主要技術指標
            tech = result.get('technical_analysis', {})
            rsi_value = tech.get('rsi_value', 50)
            macd_signal = tech.get('macd_signal', '中性')
            message_parts.append(f"   🔍 RSI: {rsi_value:.1f} | MACD: {macd_signal}")
            message_parts.append("")

        # 添加統計摘要
        if successful_results:
            avg_score = sum(r.get('combined_score', 0) for r in successful_results) / len(successful_results)
            high_score_count = len([r for r in successful_results if r.get('combined_score', 0) >= 70])

            message_parts.append("📊 統計摘要:")
            message_parts.append(f"   📊 平均分數: {avg_score:.1f}")
            message_parts.append(f"   🌟 高分股票 (≥70): {high_score_count}")

            # 建議分布
            recommendations = {}
            for result in top_stocks:
                rec = result.get('recommendation', {}).get('recommendation', '持有')
                recommendations[rec] = recommendations.get(rec, 0) + 1

            if recommendations:
                message_parts.append("   📈 前十名建議分布:")
                for rec, count in recommendations.items():
                    message_parts.append(f"      {rec}: {count}")

        message_parts.append("")
        message_parts.append("⚠️ 投資提醒:")
        message_parts.append("• 本分析僅供參考，投資有風險")
        message_parts.append("• 已篩選成交量≥1000張/日的活躍股票")
        message_parts.append("• 請結合基本面分析做最終決策")

        return "\n".join(message_parts)

    except Exception as e:
        logger.error(f"格式化訊息時發生錯誤: {e}")
        return f"❌ 格式化分析結果時發生錯誤: {str(e)}"

def display_terminal_results(results: Dict[str, Any], limit: int = 10):
    """在終端顯示分析結果"""
    try:
        if not results:
            print("❌ 未能獲取任何分析結果")
            return

        # 過濾成功的結果並按分數排序
        successful_results = [
            result for result in results.values()
            if result.get('success', False) and result.get('combined_score', 0) > 0
        ]

        successful_results.sort(key=lambda x: x.get('combined_score', 0), reverse=True)

        print("\n" + "=" * 80)
        print("🏆 台股技術分析結果 - 前十名優質股票".center(80))
        print("=" * 80)
        print(f"📈 分析時間: {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"📊 成功分析: {len(successful_results)} 支股票")
        print(f"🔍 成交量篩選條件: ≥1000張/日")
        print("=" * 80)

        if not successful_results:
            print("❌ 沒有符合條件的優質股票")
            return

        # 表格標題
        print(f"{'排名':<4}{'代碼':<8}{'股票名稱':<12}{'評分':<8}{'價格':<10}{'漲跌%':<8}{'建議':<10}{'成交量(張)':<12}")
        print("-" * 80)

        # 顯示前十名
        top_stocks = successful_results[:limit]
        for i, result in enumerate(top_stocks, 1):
            stock_name = result.get('stock_name', 'Unknown')[:10]  # 限制長度
            stock_id = result.get('stock_id', 'Unknown')
            score = result.get('combined_score', 0)
            recommendation = result.get('recommendation', {})
            rec_text = recommendation.get('recommendation', '持有')[:8]  # 限制長度
            current_price = result.get('current_price', 0)
            price_change = result.get('price_change_5d', 0)
            volume_info = result.get('volume_analysis', {})
            avg_volume_lots = volume_info.get('avg_volume_lots', 0)

            print(f"{i:<4}{stock_id:<8}{stock_name:<12}{score:<8.1f}{current_price:<10.2f}{price_change:<+8.2f}{rec_text:<10}{avg_volume_lots:<12.0f}")

        print("=" * 80)

        # 詳細分析前五名
        print("\n📊 詳細分析報告 (前五名):")
        print("=" * 80)

        for i, result in enumerate(top_stocks[:5], 1):
            stock_name = result.get('stock_name', 'Unknown')
            stock_id = result.get('stock_id', 'Unknown')
            score = result.get('combined_score', 0)
            recommendation = result.get('recommendation', {})
            trend_analysis = result.get('trend_analysis', {})
            technical_analysis = result.get('technical_analysis', {})
            volume_analysis = result.get('volume_analysis', {})

            print(f"\n{i}. {stock_name} ({stock_id}) - 評分: {score:.1f}")
            print("-" * 50)
            print(f"💰 當前價格: {result.get('current_price', 0):.2f}")
            print(f"📈 5日漲跌: {result.get('price_change_5d', 0):+.2f}%")
            print(f"🎯 投資建議: {recommendation.get('recommendation', '持有')}")
            print(f"📊 信心度: {recommendation.get('confidence', '中等')}")

            # 技術指標詳情
            print(f"🔍 技術指標:")
            print(f"   RSI: {technical_analysis.get('rsi_value', 50):.1f} ({technical_analysis.get('rsi_signal', '中性')})")
            print(f"   MACD: {technical_analysis.get('macd_signal', '中性')}")
            print(f"   KD: K={technical_analysis.get('k_value', 50):.1f}, D={technical_analysis.get('d_value', 50):.1f}")

            # 趨勢和成交量
            print(f"📊 趨勢分析: {trend_analysis.get('trend', '盤整')}")
            print(f"📈 成交量: {volume_analysis.get('avg_volume_lots', 0):.0f}張/日 ({volume_analysis.get('volume_trend', '正常')})")

            # 建議原因
            reasons = recommendation.get('reasons', [])
            if reasons:
                print(f"💡 建議原因: {', '.join(reasons[:2])}")  # 顯示前兩個原因

        print("\n" + "=" * 80)
        print("⚠️  投資提醒:")
        print("• 本分析僅供參考，投資一定有風險")
        print("• 已篩選成交量≥1000張/日的活躍股票")
        print("• 請務必結合基本面分析做最終投資決策")
        print("• 建議分散投資，控制風險")
        print("=" * 80)

    except Exception as e:
        logger.error(f"顯示終端結果時發生錯誤: {e}")
        print(f"❌ 顯示結果時發生錯誤: {str(e)}")

async def send_notification(session: aiohttp.ClientSession, message: str, files: List[str] = None):
    """發送通知到 Telegram 和 Discord"""

    # Telegram 通知
    try:
        # 檢查 Telegram 設定
        if not TELEGRAM_BOT_TOKEN or not TELEGRAM_CHAT_IDS or TELEGRAM_BOT_TOKEN == "YOUR_BOT_TOKEN_HERE":
            logger.warning("Telegram 設定未完成，跳過發送通知")
            print("📱 Telegram 設定未完成，請設定 TELEGRAM_BOT_TOKEN 和 TELEGRAM_CHAT_IDS")
        else:
            # 分割長訊息
            max_length = 4000
            if len(message) > max_length:
                parts = []
                current_part = ""

                for line in message.split('\n'):
                    if len(current_part + line + '\n') > max_length:
                        if current_part:
                            parts.append(current_part.strip())
                            current_part = line + '\n'
                        else:
                            # 如果單行太長，直接截斷
                            parts.append(line[:max_length])
                    else:
                        current_part += line + '\n'

                if current_part:
                    parts.append(current_part.strip())
            else:
                parts = [message]

            # 發送文字訊息給所有 chat_id
            telegram_url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"

            for chat_id in TELEGRAM_CHAT_IDS:
                for i, part in enumerate(parts):
                    if i > 0:
                        part = f"🏆 台股分析報告 (續 {i+1}/{len(parts)})\n\n" + part

                    payload = {
                        'chat_id': chat_id,
                        'text': part
                    }

                    try:
                        async with session.post(telegram_url, json=payload, timeout=20) as response:
                            if response.status == 200:
                                logger.info(f"成功發送 Telegram 通知到 {chat_id} (第 {i+1}/{len(parts)} 部分)")
                            else:
                                error_text = await response.text()
                                logger.error(f"發送 Telegram 通知失敗到 {chat_id} (第 {i+1} 部分): {response.status} - {error_text}")
                    except Exception as e:
                        logger.error(f"發送 Telegram 通知時發生網路錯誤到 {chat_id} (第 {i+1} 部分): {e}")

                    # 避免發送太快
                    if i < len(parts) - 1:
                        await asyncio.sleep(1)

    except Exception as e:
        logger.error(f"發送 Telegram 通知時異常: {e}")

    # Discord 通知 (如果可用)
    if MESSAGING_AVAILABLE and DISCORD_WEBHOOK_URL and DISCORD_WEBHOOK_URL != "YOUR_DISCORD_WEBHOOK_URL":
        try:
            webhook = DiscordWebhook(url=DISCORD_WEBHOOK_URL, content=f"```\n{message}\n```")
            response = webhook.execute()
            if response.ok:
                logger.info("Discord 通知發送成功")
            else:
                logger.error(f"Discord 通知失敗: {response.status_code} {response.content}")
        except Exception as e:
            logger.error(f"發送 Discord 通知時異常: {e}")

async def main():
    """主程式"""
    try:
        print("🚀 台股技術分析程式啟動")
        print(f"⏰ 開始時間: {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}")
        print("🔍 分析條件: 成交量≥1000張/日")
        print("🏆 目標: 找出前十名優質股票")
        print("=" * 60)

        # 初始化分析器
        analyzer = StockAnalyzer()

        # 執行全部股票分析
        print("📊 開始分析所有台股（包含成交量篩選）...")
        results = await analyzer.analyze_all_stocks()

        if not results:
            error_message = "❌ 分析失敗，未能獲取任何結果"
            print(error_message)

            # 發送錯誤通知
            async with aiohttp.ClientSession() as session:
                await send_notification(session, error_message)
            return

        # 在終端顯示結果
        display_terminal_results(results, limit=10)

        # 格式化通知訊息
        print("\n📱 正在準備發送通知...")
        message = format_analysis_message(results, limit=10)

        # 發送通知
        print("📱 正在發送通知到 Telegram 和 Discord...")
        async with aiohttp.ClientSession() as session:
            await send_notification(session, message)

        print(f"\n⏰ 程式執行完成: {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}")
        print("=" * 60)
        print("✅ 分析報告已完成並發送")
        print("🏆 前十名優質股票已顯示在上方")

    except Exception as e:
        error_message = f"❌ 主程式執行錯誤: {str(e)}\n\n詳細錯誤:\n{traceback.format_exc()}"
        logger.error(error_message)
        print(error_message)

        # 嘗試發送錯誤通知
        try:
            async with aiohttp.ClientSession() as session:
                await send_notification(session, f"❌ 程式執行錯誤: {str(e)}")
        except Exception as notify_error:
            logger.error(f"發送錯誤通知失敗: {notify_error}")

if __name__ == "__main__":
    # 執行主程式

    asyncio.run(main())

上面是台股選擇每日1000張，評分最優前十名

In [ ]:


import asyncio
import aiohttp
import pandas as pd
import numpy as np
import yfinance as yf
import logging
import traceback
from datetime import datetime, timezone, timedelta
from typing import Dict, List, Any, Optional
import json
import os
import time
import requests
import random
from io import StringIO

# 如果您在 Jupyter Notebook 中，請使用這個版本
import nest_asyncio

# 安裝 nest_asyncio（如果尚未安裝）
try:
    import nest_asyncio
except ImportError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "nest_asyncio"])
    import nest_asyncio

# 應用 nest_asyncio 以允許嵌套事件循環
nest_asyncio.apply()

# 設定日誌
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# 台北時區
taipei_tz = timezone(timedelta(hours=8))

# Telegram 設定
TELEGRAM_BOT_TOKEN = "7902318521:AAEYoDMqwfHabI7L1SRiE4z33aFay42-VGE"
TELEGRAM_CHAT_IDS = [879781796, 8113868436]

# Discord 設定 (可選)
DISCORD_WEBHOOK_URL = "https://discord.com/api/webhooks/1362715080734802102/Jma7A3VhEQrrRxIX_JW2l6rATjAZXsGXGfnJuAMqmS1QvqG_2ptg3vr_nsVnuV_PlnBl"
MESSAGING_AVAILABLE = False

try:
    from discord_webhook import DiscordWebhook
    MESSAGING_AVAILABLE = True
except ImportError:
    MESSAGING_AVAILABLE = False
    logger.info("Discord webhook 套件未安裝，將跳過 Discord 通知")

class StockAnalyzer:
    def __init__(self):
        """初始化股票分析器"""
        self.config = {
            'rsi_period': 14,
            'macd_fast': 12,
            'macd_slow': 26,
            'macd_signal': 9,
            'bb_period': 20,
            'bb_std': 2,
            'kd_period': 14,
            'volume_ma_period': 20,
            'min_volume_lots': 1000,  # 最小成交量（張）
            'trend_weights': {
                'price_trend': 0.3,
                'volume_trend': 0.2,
                'technical_score': 0.5
            }
        }
        self.taiwan_stocks = None
        self.etf_list_path = "taiwan_stocks_cache.csv"

    def get_taiwan_etfs(self, force_update=True):
        """獲取台灣ETF清單"""
        if not force_update and os.path.exists(self.etf_list_path):
            if (time.time() - os.path.getmtime(self.etf_list_path)) < 86400:
                logger.info("從快取載入ETF列表。")
                return pd.read_csv(self.etf_list_path, dtype={'etf_id': str})

        logger.info("從台灣證交所網站獲取最新ETF清單...")
        urls = {
            "上市": "https://isin.twse.com.tw/isin/C_public.jsp?strMode=2",
            "上櫃": "https://isin.twse.com.tw/isin/C_public.jsp?strMode=4"
        }
        all_etfs_df = []
        headers = {'User-Agent': 'Mozilla/5.0'}

        for market_name, url in urls.items():
            try:
                res = requests.get(url, headers=headers, timeout=30)
                res.encoding = 'big5'
                html_dfs = pd.read_html(StringIO(res.text))
                df = html_dfs[0].copy()
                df.columns = df.iloc[0]
                df = df.iloc[1:].copy()
                df.loc[:, 'market'] = market_name
                all_etfs_df.append(df)
            except Exception as e:
                logger.error(f"獲取 {market_name} ETF列表失敗: {e}")
                continue

        if not all_etfs_df:
            logger.error("無法從網站獲取任何ETF數據，使用備用清單。")
            return self._get_backup_etf_list()

        try:
            df = pd.concat(all_etfs_df, ignore_index=True)
            df[['etf_id', 'etf_name']] = df['有價證券代號及名稱'].str.split(r'\s+', n=1, expand=True)

            # 篩選ETF：只保留4位數字代碼且名稱包含ETF相關關鍵字的項目
            df = df[df['etf_id'].str.match(r'^\d{4}$', na=False)].copy()

            # 只保留ETF相關的證券
            etf_keywords = ['ETF', 'ETN', '指數', '基金', '傘型', '期貨', '反向', '槓桿']
            df = df[df['etf_name'].str.contains('|'.join(etf_keywords), na=False)].copy()

            # 排除一般股票和其他非ETF商品
            exclude_keywords = ['購', '牛', '熊', '認購', '認售', '權證', '存託憑證', 'TDR']
            df = df[~df['etf_name'].str.contains('|'.join(exclude_keywords), na=False)].copy()

            # 設定Yahoo Finance符號
            df.loc[:, 'yahoo_symbol'] = df.apply(
                lambda row: f"{row['etf_id']}.TW" if '上市' in row['market'] else f"{row['etf_id']}.TWO", axis=1)

            final_df = df[['etf_id', 'etf_name', 'market', '產業別', 'yahoo_symbol']].rename(
                columns={'產業別': 'category', 'etf_id': 'etf_id', 'etf_name': 'etf_name'})
            final_df = final_df.drop_duplicates(subset=['etf_id']).reset_index(drop=True)
            final_df.to_csv(self.etf_list_path, index=False)
            logger.info(f"成功獲取 {len(final_df)} 支ETF清單並保存快取。")
            return final_df
        except Exception as e:
            logger.error(f"處理ETF清單時發生錯誤: {e}")
            return self._get_backup_etf_list()

    def _get_backup_etf_list(self) -> pd.DataFrame:
        """備用ETF清單"""
        try:
            logger.info("使用備用ETF清單")

            # 台灣主要ETF列表
            famous_etfs = [
                # 市值型ETF
                {'etf_id': '0050', 'etf_name': '元大台灣50', 'market': '上市', 'yahoo_symbol': '0050.TW', 'category': '股票型ETF'},
                {'etf_id': '0056', 'etf_name': '元大高股息', 'market': '上市', 'yahoo_symbol': '0056.TW', 'category': '股票型ETF'},
                {'etf_id': '006208', 'etf_name': '富邦台50', 'market': '上市', 'yahoo_symbol': '006208.TW', 'category': '股票型ETF'},
                {'etf_id': '00878', 'etf_name': '國泰永續高股息', 'market': '上市', 'yahoo_symbol': '00878.TW', 'category': '股票型ETF'},
                {'etf_id': '00881', 'etf_name': '國泰台灣5G+', 'market': '上市', 'yahoo_symbol': '00881.TW', 'category': '股票型ETF'},

                # 高股息ETF
                {'etf_id': '00713', 'etf_name': '元大台灣高息低波', 'market': '上市', 'yahoo_symbol': '00713.TW', 'category': '股票型ETF'},
                {'etf_id': '00701', 'etf_name': '國泰低波動', 'market': '上市', 'yahoo_symbol': '00701.TW', 'category': '股票型ETF'},
                {'etf_id': '00692', 'etf_name': '富邦公司治理', 'market': '上市', 'yahoo_symbol': '00692.TW', 'category': '股票型ETF'},
                {'etf_id': '00900', 'etf_name': '富邦特選高股息30', 'market': '上市', 'yahoo_symbol': '00900.TW', 'category': '股票型ETF'},
                {'etf_id': '00919', 'etf_name': '群益台灣精選高息', 'market': '上市', 'yahoo_symbol': '00919.TW', 'category': '股票型ETF'},

                # 科技類ETF
                {'etf_id': '00757', 'etf_name': '統一FANG+', 'market': '上市', 'yahoo_symbol': '00757.TW', 'category': '股票型ETF'},
                {'etf_id': '00861', 'etf_name': '元大全球未來通訊', 'market': '上市', 'yahoo_symbol': '00861.TW', 'category': '股票型ETF'},
                {'etf_id': '00876', 'etf_name': '元大未來關鍵科技', 'market': '上市', 'yahoo_symbol': '00876.TW', 'category': '股票型ETF'},

                # 債券型ETF
                {'etf_id': '00679B', 'etf_name': '元大美債20年', 'market': '上市', 'yahoo_symbol': '00679B.TW', 'category': '債券型ETF'},
                {'etf_id': '00687B', 'etf_name': '國泰20年美債', 'market': '上市', 'yahoo_symbol': '00687B.TW', 'category': '債券型ETF'},
                {'etf_id': '00720B', 'etf_name': '元大投資級公司債', 'market': '上市', 'yahoo_symbol': '00720B.TW', 'category': '債券型ETF'},
                {'etf_id': '00751B', 'etf_name': '元大AAA至A公司債', 'market': '上市', 'yahoo_symbol': '00751B.TW', 'category': '債券型ETF'},

                # 國際股票ETF
                {'etf_id': '00646', 'etf_name': '元大S&P500', 'market': '上市', 'yahoo_symbol': '00646.TW', 'category': '股票型ETF'},
                {'etf_id': '00662', 'etf_name': '富邦NASDAQ', 'market': '上市', 'yahoo_symbol': '00662.TW', 'category': '股票型ETF'},
                {'etf_id': '00670L', 'etf_name': '富邦NASDAQ正2', 'market': '上市', 'yahoo_symbol': '00670L.TW', 'category': '槓桿型ETF'},
                {'etf_id': '00672L', 'etf_name': '元大S&P500正2', 'market': '上市', 'yahoo_symbol': '00672L.TW', 'category': '槓桿型ETF'},

                # 反向ETF
                {'etf_id': '00632R', 'etf_name': '元大台灣50反1', 'market': '上市', 'yahoo_symbol': '00632R.TW', 'category': '反向型ETF'},
                {'etf_id': '00673R', 'etf_name': '元大S&P500反1', 'market': '上市', 'yahoo_symbol': '00673R.TW', 'category': '反向型ETF'},

                # 中小型ETF
                {'etf_id': '0051', 'etf_name': '元大中型100', 'market': '上市', 'yahoo_symbol': '0051.TW', 'category': '股票型ETF'},
                {'etf_id': '006201', 'etf_name': '元大富櫃50', 'market': '上市', 'yahoo_symbol': '006201.TW', 'category': '股票型ETF'},

                # ESG相關ETF
                {'etf_id': '00850', 'etf_name': '元大台灣ESG永續', 'market': '上市', 'yahoo_symbol': '00850.TW', 'category': '股票型ETF'},
                {'etf_id': '00888', 'etf_name': '永豐台灣ESG', 'market': '上市', 'yahoo_symbol': '00888.TW', 'category': '股票型ETF'},

                # 產業型ETF
                {'etf_id': '00891', 'etf_name': '中信關鍵半導體', 'market': '上市', 'yahoo_symbol': '00891.TW', 'category': '股票型ETF'},
                {'etf_id': '00892', 'etf_name': '富邦台灣半導體', 'market': '上市', 'yahoo_symbol': '00892.TW', 'category': '股票型ETF'},
                {'etf_id': '00896', 'etf_name': '中信綠能及電動車', 'market': '上市', 'yahoo_symbol': '00896.TW', 'category': '股票型ETF'},
            ]

            df = pd.DataFrame(famous_etfs)
            logger.info(f"備用ETF清單包含 {len(df)} 支ETF")
            return df

        except Exception as e:
            logger.error(f"生成備用ETF清單時發生錯誤: {e}")
            return pd.DataFrame()


    def check_volume_filter(self, df: pd.DataFrame) -> bool:
        """檢查成交量是否符合篩選條件（每日至少1000張）"""
        try:
            if df.empty or 'Volume' not in df.columns:
                return False

            # 取最近20天的平均成交量
            recent_volume = df['Volume'].tail(20).mean()

            # 轉換為張數（1張 = 1000股）
            volume_lots = recent_volume / 1000

            # 檢查是否符合最小成交量要求
            meets_volume = volume_lots >= self.config['min_volume_lots']

            if meets_volume:
                logger.debug(f"成交量符合條件: {volume_lots:.0f} 張/日 (≥ {self.config['min_volume_lots']} 張)")
            else:
                logger.debug(f"成交量不符合條件: {volume_lots:.0f} 張/日 (< {self.config['min_volume_lots']} 張)")

            return meets_volume

        except Exception as e:
            logger.error(f"檢查成交量篩選時發生錯誤: {e}")
            return False

    def fetch_yfinance_data(self, etf_id: str, period: str = "1y", interval: str = "1d", retries: int = 3):
        """獲取股票數據（同步版本）"""
        for attempt in range(retries):
            try:
                df = yf.download(tickers=etf_id, period=period, interval=interval, progress=False, auto_adjust=True)
                if df.empty:
                    logger.warning(f"[{etf_id}] 無數據返回")
                    return pd.DataFrame()

                if isinstance(df.columns, pd.MultiIndex):
                    df.columns = df.columns.get_level_values(0)

                required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
                for col in required_cols:
                    if col in df.columns:
                        df[col] = pd.to_numeric(df[col], errors='coerce')

                df = df.dropna(subset=required_cols)
                return df.reset_index()

            except Exception as e:
                if attempt < retries - 1:
                    sleep_time = 0.5 * (2 ** attempt) + random.uniform(0, 1)
                    logger.warning(f"[{etf_id}] 第 {attempt + 1}/{retries} 次嘗試失敗: {e}。等待 {sleep_time:.2f} 秒後重試...")
                    time.sleep(sleep_time)
                else:
                    logger.error(f"[{etf_id}] 獲取數據失敗。")
                    return pd.DataFrame()
        return pd.DataFrame()

    def calculate_technical_indicators(self, df: pd.DataFrame) -> pd.DataFrame:
        """計算技術指標"""
        try:
            if df.empty:
                return df

            df = df.copy()

            # RSI
            df['RSI'] = self.calculate_rsi(df['Close'], self.config['rsi_period'])

            # MACD
            macd_line, macd_signal, macd_histogram = self.calculate_macd(
                df['Close'],
                self.config['macd_fast'],
                self.config['macd_slow'],
                self.config['macd_signal']
            )
            df['MACD'] = macd_line
            df['MACD_Signal'] = macd_signal
            df['MACD_Histogram'] = macd_histogram

            # 布林帶
            bb_upper, bb_middle, bb_lower = self.calculate_bollinger_bands(
                df['Close'],
                self.config['bb_period'],
                self.config['bb_std']
            )
            df['BB_Upper'] = bb_upper
            df['BB_Middle'] = bb_middle
            df['BB_Lower'] = bb_lower

            # KD指標
            k_percent, d_percent = self.calculate_kd(
                df['High'],
                df['Low'],
                df['Close'],
                self.config['kd_period']
            )
            df['K_Percent'] = k_percent
            df['D_Percent'] = d_percent

            # 移動平均線
            df['MA5'] = df['Close'].rolling(window=5).mean()
            df['MA10'] = df['Close'].rolling(window=10).mean()
            df['MA20'] = df['Close'].rolling(window=20).mean()

            # 成交量移動平均
            if 'Volume' in df.columns:
                df['Volume_MA'] = df['Volume'].rolling(window=self.config['volume_ma_period']).mean()

            return df

        except Exception as e:
            logger.error(f"計算技術指標時發生錯誤: {e}")
            return df

    def calculate_rsi(self, prices: pd.Series, period: int = 14) -> pd.Series:
        """計算RSI"""
        try:
            delta = prices.diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
            rs = gain / loss
            rsi = 100 - (100 / (1 + rs))
            return rsi
        except Exception as e:
            logger.error(f"計算RSI時發生錯誤: {e}")
            return pd.Series(index=prices.index, data=50)

    def calculate_macd(self, prices: pd.Series, fast: int = 12, slow: int = 26, signal: int = 9):
        """計算MACD"""
        try:
            ema_fast = prices.ewm(span=fast).mean()
            ema_slow = prices.ewm(span=slow).mean()
            macd_line = ema_fast - ema_slow
            macd_signal = macd_line.ewm(span=signal).mean()
            macd_histogram = macd_line - macd_signal
            return macd_line, macd_signal, macd_histogram
        except Exception as e:
            logger.error(f"計算MACD時發生錯誤: {e}")
            zero_series = pd.Series(index=prices.index, data=0)
            return zero_series, zero_series, zero_series

    def calculate_bollinger_bands(self, prices: pd.Series, period: int = 20, std_dev: int = 2):
        """計算布林帶"""
        try:
            middle = prices.rolling(window=period).mean()
            std = prices.rolling(window=period).std()
            upper = middle + (std * std_dev)
            lower = middle - (std * std_dev)
            return upper, middle, lower
        except Exception as e:
            logger.error(f"計算布林帶時發生錯誤: {e}")
            zero_series = pd.Series(index=prices.index, data=0)
            return zero_series, zero_series, zero_series

    def calculate_kd(self, high: pd.Series, low: pd.Series, close: pd.Series, period: int = 14):
        """計算KD指標"""
        try:
            lowest_low = low.rolling(window=period).min()
            highest_high = high.rolling(window=period).max()

            k_percent = 100 * ((close - lowest_low) / (highest_high - lowest_low))
            k_percent = k_percent.rolling(window=3).mean()
            d_percent = k_percent.rolling(window=3).mean()

            return k_percent, d_percent
        except Exception as e:
            logger.error(f"計算KD時發生錯誤: {e}")
            fifty_series = pd.Series(index=high.index, data=50)
            return fifty_series, fifty_series

    def analyze_trend(self, df: pd.DataFrame) -> Dict[str, Any]:
        """趨勢分析"""
        try:
            if df.empty or len(df) < 20:
                return {'trend': '盤整', 'strength': 0, 'price_change_5d': 0, 'price_change_20d': 0}

            current_price = float(df['Close'].iloc[-1])
            price_5d_ago = float(df['Close'].iloc[-6]) if len(df) >= 6 else current_price
            price_20d_ago = float(df['Close'].iloc[-21]) if len(df) >= 21 else current_price

            change_5d = ((current_price - price_5d_ago) / price_5d_ago * 100) if price_5d_ago != 0 else 0
            change_20d = ((current_price - price_20d_ago) / price_20d_ago * 100) if price_20d_ago != 0 else 0

            # 移動平均線趨勢
            ma5_current = float(df['MA5'].iloc[-1]) if 'MA5' in df.columns and not pd.isna(df['MA5'].iloc[-1]) else current_price
            ma20_current = float(df['MA20'].iloc[-1]) if 'MA20' in df.columns and not pd.isna(df['MA20'].iloc[-1]) else current_price

            # 判斷趨勢
            if change_5d > 3 and change_20d > 5 and current_price > ma5_current > ma20_current:
                trend = '強勢上漲'
                strength = 3
            elif change_5d > 1 and change_20d > 2 and current_price > ma5_current:
                trend = '上漲'
                strength = 2
            elif change_5d < -3 and change_20d < -5 and current_price < ma5_current < ma20_current:
                trend = '強勢下跌'
                strength = -3
            elif change_5d < -1 and change_20d < -2 and current_price < ma5_current:
                trend = '下跌'
                strength = -2
            else:
                trend = '盤整'
                strength = 0

            return {
                'trend': trend,
                'strength': strength,
                'price_change_5d': change_5d,
                'price_change_20d': change_20d,
                'ma5_position': ma5_current,
                'ma20_position': ma20_current
            }

        except Exception as e:
            logger.error(f"趨勢分析時發生錯誤: {e}")
            return {'trend': '盤整', 'strength': 0, 'price_change_5d': 0, 'price_change_20d': 0}

    def analyze_volume(self, df: pd.DataFrame) -> Dict[str, Any]:
        """成交量分析"""
        try:
            if df.empty or 'Volume' not in df.columns or len(df) < 20:
                return {'volume_trend': '正常', 'volume_ratio': 1.0, 'volume_signal': '中性', 'avg_volume_lots': 0}

            current_volume = float(df['Volume'].iloc[-1])
            avg_volume_20d = float(df['Volume'].rolling(window=20).mean().iloc[-1])

            volume_ratio = current_volume / avg_volume_20d if avg_volume_20d > 0 else 1.0
            avg_volume_lots = avg_volume_20d / 1000  # 轉換為張數

            if volume_ratio > 2.0:
                volume_trend = '爆量'
                volume_signal = '強烈'
            elif volume_ratio > 1.5:
                volume_trend = '放量'
                volume_signal = '積極'
            elif volume_ratio < 0.5:
                volume_trend = '縮量'
                volume_signal = '消極'
            else:
                volume_trend = '正常'
                volume_signal = '中性'

            return {
                'volume_trend': volume_trend,
                'volume_ratio': volume_ratio,
                'volume_signal': volume_signal,
                'avg_volume_lots': avg_volume_lots
            }

        except Exception as e:
            logger.error(f"成交量分析時發生錯誤: {e}")
            return {'volume_trend': '正常', 'volume_ratio': 1.0, 'volume_signal': '中性', 'avg_volume_lots': 0}

    def analyze_technical_indicators(self, df: pd.DataFrame) -> Dict[str, Any]:
        """技術指標分析"""
        try:
            if df.empty:
                return {
                    'rsi_signal': '中性', 'rsi_value': 50,
                    'macd_signal': '中性', 'macd_value': 0,
                    'kd_signal': '中性', 'k_value': 50, 'd_value': 50,
                    'bb_signal': '中性', 'bb_position': 0.5,
                    'score': 50
                }

            signals = {}
            score = 50

            # RSI 分析
            if 'RSI' in df.columns and not df['RSI'].empty:
                rsi_value = float(df['RSI'].iloc[-1]) if not pd.isna(df['RSI'].iloc[-1]) else 50
                signals['rsi_value'] = rsi_value

                if rsi_value > 80:
                    signals['rsi_signal'] = '超買'
                    score -= 15
                elif rsi_value > 70:
                    signals['rsi_signal'] = '偏高'
                    score -= 5
                elif rsi_value < 20:
                    signals['rsi_signal'] = '超賣'
                    score += 15
                elif rsi_value < 30:
                    signals['rsi_signal'] = '偏低'
                    score += 5
                else:
                    signals['rsi_signal'] = '中性'
            else:
                signals['rsi_signal'] = '中性'
                signals['rsi_value'] = 50

            # MACD 分析
            if all(col in df.columns for col in ['MACD', 'MACD_Signal']) and len(df) >= 2:
                macd_current = float(df['MACD'].iloc[-1]) if not pd.isna(df['MACD'].iloc[-1]) else 0
                macd_signal_current = float(df['MACD_Signal'].iloc[-1]) if not pd.isna(df['MACD_Signal'].iloc[-1]) else 0
                macd_prev = float(df['MACD'].iloc[-2]) if not pd.isna(df['MACD'].iloc[-2]) else 0
                macd_signal_prev = float(df['MACD_Signal'].iloc[-2]) if not pd.isna(df['MACD_Signal'].iloc[-2]) else 0

                signals['macd_value'] = macd_current

                if macd_prev <= macd_signal_prev and macd_current > macd_signal_current:
                    signals['macd_signal'] = '黃金交叉'
                    score += 20
                elif macd_prev >= macd_signal_prev and macd_current < macd_signal_current:
                    signals['macd_signal'] = '死亡交叉'
                    score -= 20
                elif macd_current > macd_signal_current:
                    signals['macd_signal'] = '多頭'
                    score += 5
                elif macd_current < macd_signal_current:
                    signals['macd_signal'] = '空頭'
                    score -= 5
                else:
                    signals['macd_signal'] = '中性'
            else:
                signals['macd_signal'] = '中性'
                signals['macd_value'] = 0

            # KD 分析
            if all(col in df.columns for col in ['K_Percent', 'D_Percent']):
                k_value = float(df['K_Percent'].iloc[-1]) if not pd.isna(df['K_Percent'].iloc[-1]) else 50
                d_value = float(df['D_Percent'].iloc[-1]) if not pd.isna(df['D_Percent'].iloc[-1]) else 50

                signals['k_value'] = k_value
                signals['d_value'] = d_value

                if k_value > 80 and d_value > 80:
                    signals['kd_signal'] = '超買'
                    score -= 10
                elif k_value < 20 and d_value < 20:
                    signals['kd_signal'] = '超賣'
                    score += 10
                elif k_value > d_value:
                    signals['kd_signal'] = '偏多'
                    score += 3
                elif k_value < d_value:
                    signals['kd_signal'] = '偏空'
                    score -= 3
                else:
                    signals['kd_signal'] = '中性'
            else:
                signals['kd_signal'] = '中性'
                signals['k_value'] = 50
                signals['d_value'] = 50

            # 布林帶分析
            if all(col in df.columns for col in ['BB_Upper', 'BB_Lower', 'Close']):
                current_price = float(df['Close'].iloc[-1])
                bb_upper = float(df['BB_Upper'].iloc[-1]) if not pd.isna(df['BB_Upper'].iloc[-1]) else current_price
                bb_lower = float(df['BB_Lower'].iloc[-1]) if not pd.isna(df['BB_Lower'].iloc[-1]) else current_price

                if bb_upper != bb_lower:
                    bb_position = (current_price - bb_lower) / (bb_upper - bb_lower)
                    signals['bb_position'] = bb_position

                    if bb_position > 0.8:
                        signals['bb_signal'] = '接近上軌'
                        score -= 5
                    elif bb_position < 0.2:
                        signals['bb_signal'] = '接近下軌'
                        score += 5
                    else:
                        signals['bb_signal'] = '中性'
                else:
                    signals['bb_signal'] = '中性'
                    signals['bb_position'] = 0.5
            else:
                signals['bb_signal'] = '中性'
                signals['bb_position'] = 0.5

            score = max(0, min(100, score))
            signals['score'] = score

            return signals

        except Exception as e:
            logger.error(f"技術指標分析時發生錯誤: {e}")
            return {
                'rsi_signal': '中性', 'rsi_value': 50,
                'macd_signal': '中性', 'macd_value': 0,
                'kd_signal': '中性', 'k_value': 50, 'd_value': 50,
                'bb_signal': '中性', 'bb_position': 0.5,
                'score': 50
            }

    def calculate_combined_score(self, trend_analysis: Dict, volume_analysis: Dict, technical_analysis: Dict) -> float:
        """計算綜合分數"""
        try:
            base_score = 50

            # 趨勢分數 (權重: 30%)
            trend_score = 0
            trend_strength = trend_analysis.get('strength', 0)
            if trend_strength > 0:
                trend_score = min(30, trend_strength * 10)
            elif trend_strength < 0:
                trend_score = max(-30, trend_strength * 10)

            # 成交量分數 (權重: 20%)
            volume_score = 0
            volume_ratio = volume_analysis.get('volume_ratio', 1.0)
            if volume_ratio > 1.5:
                volume_score = 10
            elif volume_ratio > 1.2:
                volume_score = 5
            elif volume_ratio < 0.7:
                volume_score = -5

            # 技術指標分數 (權重: 50%)
            tech_score = technical_analysis.get('score', 50) - 50

            # 計算加權總分
            total_score = base_score + (trend_score * 0.3) + (volume_score * 0.2) + (tech_score * 0.5)

            return max(0, min(100, total_score))

        except Exception as e:
            logger.error(f"計算綜合分數時發生錯誤: {e}")
            return 50.0

    def generate_recommendation(self, combined_score: float, trend_analysis: Dict, technical_analysis: Dict) -> Dict[str, Any]:
        """生成投資建議"""
        try:
            recommendation = "持有"
            confidence = "中等"
            reasons = []

            if combined_score >= 75:
                recommendation = "強力買進"
                confidence = "高"
            elif combined_score >= 60:
                recommendation = "買進"
                confidence = "中高"
            elif combined_score >= 40:
                recommendation = "持有"
                confidence = "中等"
            elif combined_score >= 25:
                recommendation = "賣出"
                confidence = "中高"
            else:
                recommendation = "強力賣出"
                confidence = "高"

            # 生成建議原因
            trend = trend_analysis.get('trend', '盤整')
            if '上漲' in trend:
                reasons.append(f"價格趨勢：{trend}")
            elif '下跌' in trend:
                reasons.append(f"價格趨勢：{trend}")

            rsi_signal = technical_analysis.get('rsi_signal', '中性')
            if rsi_signal in ['超賣', '偏低']:
                reasons.append(f"RSI指標：{rsi_signal}，可能反彈")
            elif rsi_signal in ['超買', '偏高']:
                reasons.append(f"RSI指標：{rsi_signal}，注意回調")

            macd_signal = technical_analysis.get('macd_signal', '中性')
            if macd_signal == '黃金交叉':
                reasons.append("MACD出現黃金交叉")
            elif macd_signal == '死亡交叉':
                reasons.append("MACD出現死亡交叉")

            return {
                'recommendation': recommendation,
                'confidence': confidence,
                'score': combined_score,
                'reasons': reasons
            }

        except Exception as e:
            logger.error(f"生成投資建議時發生錯誤: {e}")
            return {
                'recommendation': '持有',
                'confidence': '低',
                'score': 50,
                'reasons': ['分析過程出現錯誤']
            }

    async def analyze_stock_async(self, session: aiohttp.ClientSession, stock_info: Dict) -> Dict[str, Any]:
        """異步分析單支股票"""
        symbol = stock_info['yahoo_symbol']
        etf_name = stock_info['etf_name']

        try:
            logger.info(f"開始分析股票: {etf_name} ({symbol})")

            # 獲取股票數據
            df = self.fetch_yfinance_data(symbol)
            if df is None or df.empty or len(df) < 60:
                return {
                    'symbol': symbol,
                    'etf_name': etf_name,
                    'success': False,
                    'error': '無法獲取足夠的股票數據'
                }

            # 檢查成交量篩選條件
            if not self.check_volume_filter(df):
                return {
                    'symbol': symbol,
                    'etf_name': etf_name,
                    'success': False,
                    'error': f'成交量不符合條件（需≥{self.config["min_volume_lots"]}張/日）'
                }

            # 計算技術指標
            df_with_indicators = self.calculate_technical_indicators(df)

            # 進行各項分析
            trend_analysis = self.analyze_trend(df_with_indicators)
            volume_analysis = self.analyze_volume(df_with_indicators)
            technical_analysis = self.analyze_technical_indicators(df_with_indicators)

            # 計算綜合分數
            combined_score = self.calculate_combined_score(trend_analysis, volume_analysis, technical_analysis)

            # 生成投資建議
            recommendation = self.generate_recommendation(combined_score, trend_analysis, technical_analysis)

            # 獲取當前價格資訊
            current_price = float(df['Close'].iloc[-1])
            price_change = trend_analysis.get('price_change_5d', 0)

            result = {
                'symbol': symbol,
                'etf_name': etf_name,
                'etf_id': stock_info.get('etf_id', ''),
                'market': stock_info.get('market', ''),
                'industry': stock_info.get('industry', ''),
                'success': True,
                'current_price': current_price,
                'price_change_5d': price_change,
                'combined_score': combined_score,
                'trend_analysis': trend_analysis,
                'volume_analysis': volume_analysis,
                'technical_analysis': technical_analysis,
                'recommendation': recommendation,
                'analysis_time': datetime.now(taipei_tz).isoformat()
            }

            logger.info(f"完成分析: {etf_name} - 分數: {combined_score:.1f} - 建議: {recommendation['recommendation']} - 成交量: {volume_analysis.get('avg_volume_lots', 0):.0f}張/日")
            return result

        except Exception as e:
            logger.error(f"分析股票 {symbol} 時發生錯誤: {e}")
            return {
                'symbol': symbol,
                'etf_name': etf_name,
                'success': False,
                'error': str(e)
            }

    async def analyze_all_stocks(self) -> Dict[str, Any]:
        """分析所有股票（加入成交量篩選）"""
        try:
            # 獲取完整股票清單
            if self.taiwan_stocks is None:
                self.taiwan_stocks = self.get_taiwan_etfs()

            if self.taiwan_stocks.empty:
                logger.error("無法獲取股票清單")
                return {}

            logger.info(f"準備分析 {len(self.taiwan_stocks)} 支股票（包含成交量篩選）")

            results = {}
            failed_count = 0
            volume_filtered_count = 0

            # 使用 aiohttp 進行異步分析
            async with aiohttp.ClientSession() as session:
                # 創建分析任務
                tasks = []
                for _, stock_info in self.taiwan_stocks.iterrows():
                    task = self.analyze_stock_async(session, stock_info.to_dict())
                    tasks.append(task)

                # 分批執行任務以避免過載
                batch_size = 10
                for i in range(0, len(tasks), batch_size):
                    batch_tasks = tasks[i:i+batch_size]
                    batch_results = await asyncio.gather(*batch_tasks, return_exceptions=True)

                    # 處理批次結果
                    for result in batch_results:
                        if isinstance(result, Exception):
                            logger.error(f"分析任務異常: {result}")
                            failed_count += 1
                            continue

                        if isinstance(result, dict) and 'symbol' in result:
                            if result.get('success', False):
                                results[result['symbol']] = result
                            else:
                                error_msg = result.get('error', '')
                                if '成交量不符合條件' in error_msg:
                                    volume_filtered_count += 1
                                else:
                                    failed_count += 1

                    # 批次間暫停避免過載
                    if i + batch_size < len(tasks):
                        await asyncio.sleep(2)

            logger.info(f"分析完成統計：")
            logger.info(f"  - 成功分析: {len(results)} 支")
            logger.info(f"  - 成交量篩選淘汰: {volume_filtered_count} 支")
            logger.info(f"  - 其他失敗: {failed_count} 支")

            return results

        except Exception as e:
            logger.error(f"批量分析股票時發生錯誤: {e}")
            return {}

def format_analysis_message(results: Dict[str, Any], limit: int = 10) -> str:
    """格式化分析結果為訊息（顯示前十名）"""
    try:
        if not results:
            return "❌ 未能獲取任何分析結果"

        # 過濾成功的結果並按分數排序
        successful_results = [
            result for result in results.values()
            if result.get('success', False) and result.get('combined_score', 0) > 0
        ]

        successful_results.sort(key=lambda x: x.get('combined_score', 0), reverse=True)

        # 統計資訊
        total_analyzed = len(results)
        successful_count = len(successful_results)

        # 建立訊息
        message_parts = []
        message_parts.append("🏆 台股ETF技術分析 - 前十名優質股票")
        message_parts.append("=" * 40)
        message_parts.append(f"📈 分析時間: {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}")
        message_parts.append(f"📊 成功分析: {successful_count} 支股票")
        message_parts.append(f"🔍 成交量篩選: ≥1000張/日")
        message_parts.append("")

        if not successful_results:
            message_parts.append("❌ 沒有符合條件的優質股票")
            return "\n".join(message_parts)

        # 顯示前十名優質股票
        top_stocks = successful_results[:limit]
        message_parts.append(f"🏆 前 {len(top_stocks)} 名優質股票:")
        message_parts.append("-" * 40)

        for i, result in enumerate(top_stocks, 1):
            etf_name = result.get('etf_name', 'Unknown')
            etf_id = result.get('etf_id', 'Unknown')
            score = result.get('combined_score', 0)
            recommendation = result.get('recommendation', {})
            rec_text = recommendation.get('recommendation', '持有')
            current_price = result.get('current_price', 0)
            price_change = result.get('price_change_5d', 0)
            volume_info = result.get('volume_analysis', {})
            avg_volume_lots = volume_info.get('avg_volume_lots', 0)

            # 價格變化符號
            change_symbol = "📈" if price_change > 0 else "📉" if price_change < 0 else "➡️"

            # 建議符號
            rec_symbol = "🚀" if "強力買" in rec_text else "📈" if "買" in rec_text else "📊" if "持有" in rec_text else "📉"

            message_parts.append(f"{i}. {etf_name} ({etf_id})")
            message_parts.append(f"   💰 價格: {current_price:.2f} ({change_symbol}{price_change:+.2f}%)")
            message_parts.append(f"   ⭐ 評分: {score:.1f}/100")
            message_parts.append(f"   {rec_symbol} 建議: {rec_text}")
            message_parts.append(f"   📊 成交量: {avg_volume_lots:.0f}張/日")

            # 添加主要技術指標
            tech = result.get('technical_analysis', {})
            rsi_value = tech.get('rsi_value', 50)
            macd_signal = tech.get('macd_signal', '中性')
            message_parts.append(f"   🔍 RSI: {rsi_value:.1f} | MACD: {macd_signal}")
            message_parts.append("")

        # 添加統計摘要
        if successful_results:
            avg_score = sum(r.get('combined_score', 0) for r in successful_results) / len(successful_results)
            high_score_count = len([r for r in successful_results if r.get('combined_score', 0) >= 70])

            message_parts.append("📊 統計摘要:")
            message_parts.append(f"   📊 平均分數: {avg_score:.1f}")
            message_parts.append(f"   🌟 高分股票 (≥70): {high_score_count}")

            # 建議分布
            recommendations = {}
            for result in top_stocks:
                rec = result.get('recommendation', {}).get('recommendation', '持有')
                recommendations[rec] = recommendations.get(rec, 0) + 1

            if recommendations:
                message_parts.append("   📈 前十名建議分布:")
                for rec, count in recommendations.items():
                    message_parts.append(f"      {rec}: {count}")

        message_parts.append("")
        message_parts.append("⚠️ 投資提醒:")
        message_parts.append("• 本分析僅供參考，投資有風險")
        message_parts.append("• 已篩選成交量≥1000張/日的活躍股票")
        message_parts.append("• 請結合基本面分析做最終決策")

        return "\n".join(message_parts)

    except Exception as e:
        logger.error(f"格式化訊息時發生錯誤: {e}")
        return f"❌ 格式化分析結果時發生錯誤: {str(e)}"

def display_terminal_results(results: Dict[str, Any], limit: int = 10):
    """在終端顯示分析結果"""
    try:
        if not results:
            print("❌ 未能獲取任何分析結果")
            return

        # 過濾成功的結果並按分數排序
        successful_results = [
            result for result in results.values()
            if result.get('success', False) and result.get('combined_score', 0) > 0
        ]

        successful_results.sort(key=lambda x: x.get('combined_score', 0), reverse=True)

        print("\n" + "=" * 80)
        print("🏆 台股ETF技術分析結果 - 前十名優質股票".center(80))
        print("=" * 80)
        print(f"📈 分析時間: {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"📊 成功分析: {len(successful_results)} 支股票")
        print(f"🔍 成交量篩選條件: ≥1000張/日")
        print("=" * 80)

        if not successful_results:
            print("❌ 沒有符合條件的優質股票")
            return

        # 表格標題
        print(f"{'排名':<4}{'代碼':<8}{'股票名稱':<12}{'評分':<8}{'價格':<10}{'漲跌%':<8}{'建議':<10}{'成交量(張)':<12}")
        print("-" * 80)

        # 顯示前十名
        top_stocks = successful_results[:limit]
        for i, result in enumerate(top_stocks, 1):
            etf_name = result.get('etf_name', 'Unknown')[:10]  # 限制長度
            etf_id = result.get('etf_id', 'Unknown')
            score = result.get('combined_score', 0)
            recommendation = result.get('recommendation', {})
            rec_text = recommendation.get('recommendation', '持有')[:8]  # 限制長度
            current_price = result.get('current_price', 0)
            price_change = result.get('price_change_5d', 0)
            volume_info = result.get('volume_analysis', {})
            avg_volume_lots = volume_info.get('avg_volume_lots', 0)

            print(f"{i:<4}{etf_id:<8}{etf_name:<12}{score:<8.1f}{current_price:<10.2f}{price_change:<+8.2f}{rec_text:<10}{avg_volume_lots:<12.0f}")

        print("=" * 80)

        # 詳細分析前五名
        print("\n📊 詳細分析報告 (前五名):")
        print("=" * 80)

        for i, result in enumerate(top_stocks[:5], 1):
            etf_name = result.get('etf_name', 'Unknown')
            etf_id = result.get('etf_id', 'Unknown')
            score = result.get('combined_score', 0)
            recommendation = result.get('recommendation', {})
            trend_analysis = result.get('trend_analysis', {})
            technical_analysis = result.get('technical_analysis', {})
            volume_analysis = result.get('volume_analysis', {})

            print(f"\n{i}. {etf_name} ({etf_id}) - 評分: {score:.1f}")
            print("-" * 50)
            print(f"💰 當前價格: {result.get('current_price', 0):.2f}")
            print(f"📈 5日漲跌: {result.get('price_change_5d', 0):+.2f}%")
            print(f"🎯 投資建議: {recommendation.get('recommendation', '持有')}")
            print(f"📊 信心度: {recommendation.get('confidence', '中等')}")

            # 技術指標詳情
            print(f"🔍 技術指標:")
            print(f"   RSI: {technical_analysis.get('rsi_value', 50):.1f} ({technical_analysis.get('rsi_signal', '中性')})")
            print(f"   MACD: {technical_analysis.get('macd_signal', '中性')}")
            print(f"   KD: K={technical_analysis.get('k_value', 50):.1f}, D={technical_analysis.get('d_value', 50):.1f}")

            # 趨勢和成交量
            print(f"📊 趨勢分析: {trend_analysis.get('trend', '盤整')}")
            print(f"📈 成交量: {volume_analysis.get('avg_volume_lots', 0):.0f}張/日 ({volume_analysis.get('volume_trend', '正常')})")

            # 建議原因
            reasons = recommendation.get('reasons', [])
            if reasons:
                print(f"💡 建議原因: {', '.join(reasons[:2])}")  # 顯示前兩個原因

        print("\n" + "=" * 80)
        print("⚠️  投資提醒:")
        print("• 本分析僅供參考，投資一定有風險")
        print("• 已篩選成交量≥1000張/日的活躍股票")
        print("• 請務必結合基本面分析做最終投資決策")
        print("• 建議分散投資，控制風險")
        print("=" * 80)

    except Exception as e:
        logger.error(f"顯示終端結果時發生錯誤: {e}")
        print(f"❌ 顯示結果時發生錯誤: {str(e)}")

async def send_notification(session: aiohttp.ClientSession, message: str, files: List[str] = None):
    """發送通知到 Telegram 和 Discord"""

    # Telegram 通知
    try:
        # 檢查 Telegram 設定
        if not TELEGRAM_BOT_TOKEN or not TELEGRAM_CHAT_IDS or TELEGRAM_BOT_TOKEN == "YOUR_BOT_TOKEN_HERE":
            logger.warning("Telegram 設定未完成，跳過發送通知")
            print("📱 Telegram 設定未完成，請設定 TELEGRAM_BOT_TOKEN 和 TELEGRAM_CHAT_IDS")
        else:
            # 分割長訊息
            max_length = 4000
            if len(message) > max_length:
                parts = []
                current_part = ""

                for line in message.split('\n'):
                    if len(current_part + line + '\n') > max_length:
                        if current_part:
                            parts.append(current_part.strip())
                            current_part = line + '\n'
                        else:
                            # 如果單行太長，直接截斷
                            parts.append(line[:max_length])
                    else:
                        current_part += line + '\n'

                if current_part:
                    parts.append(current_part.strip())
            else:
                parts = [message]

            # 發送文字訊息給所有 chat_id
            telegram_url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"

            for chat_id in TELEGRAM_CHAT_IDS:
                for i, part in enumerate(parts):
                    if i > 0:
                        part = f"🏆 台股ETF分析報告 (續 {i+1}/{len(parts)})\n\n" + part

                    payload = {
                        'chat_id': chat_id,
                        'text': part
                    }

                    try:
                        async with session.post(telegram_url, json=payload, timeout=20) as response:
                            if response.status == 200:
                                logger.info(f"成功發送 Telegram 通知到 {chat_id} (第 {i+1}/{len(parts)} 部分)")
                            else:
                                error_text = await response.text()
                                logger.error(f"發送 Telegram 通知失敗到 {chat_id} (第 {i+1} 部分): {response.status} - {error_text}")
                    except Exception as e:
                        logger.error(f"發送 Telegram 通知時發生網路錯誤到 {chat_id} (第 {i+1} 部分): {e}")

                    # 避免發送太快
                    if i < len(parts) - 1:
                        await asyncio.sleep(1)

    except Exception as e:
        logger.error(f"發送 Telegram 通知時異常: {e}")

    # Discord 通知 (如果可用)
    if MESSAGING_AVAILABLE and DISCORD_WEBHOOK_URL and DISCORD_WEBHOOK_URL != "YOUR_DISCORD_WEBHOOK_URL":
        try:
            webhook = DiscordWebhook(url=DISCORD_WEBHOOK_URL, content=f"```\n{message}\n```")
            response = webhook.execute()
            if response.ok:
                logger.info("Discord 通知發送成功")
            else:
                logger.error(f"Discord 通知失敗: {response.status_code} {response.content}")
        except Exception as e:
            logger.error(f"發送 Discord 通知時異常: {e}")

async def main():
    """主程式"""
    try:
        print("🚀 台股ETF技術分析程式啟動")
        print(f"⏰ 開始時間: {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}")
        print("🔍 分析條件: 成交量≥1000張/日")
        print("🏆 目標: 找出前十名優質ETF")
        print("=" * 60)

        # 初始化分析器
        analyzer = StockAnalyzer()

        # 執行全部股票分析
        print("📊 開始分析所有ETF（包含成交量篩選）...")
        results = await analyzer.analyze_all_stocks()

        if not results:
            error_message = "❌ 分析失敗，未能獲取任何結果"
            print(error_message)

            # 發送錯誤通知
            async with aiohttp.ClientSession() as session:
                await send_notification(session, error_message)
            return

        # 在終端顯示結果
        display_terminal_results(results, limit=10)

        # 格式化通知訊息
        print("\n📱 正在準備發送通知...")
        message = format_analysis_message(results, limit=10)

        # 發送通知
        print("📱 正在發送通知到 Telegram 和 Discord...")
        async with aiohttp.ClientSession() as session:
            await send_notification(session, message)

        print(f"\n⏰ 程式執行完成: {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}")
        print("=" * 60)
        print("✅ 分析報告已完成並發送")
        print("🏆 前十名優質股票已顯示在上方")

    except Exception as e:
        error_message = f"❌ 主程式執行錯誤: {str(e)}\n\n詳細錯誤:\n{traceback.format_exc()}"
        logger.error(error_message)
        print(error_message)

        # 嘗試發送錯誤通知
        try:
            async with aiohttp.ClientSession() as session:
                await send_notification(session, f"❌ 程式執行錯誤: {str(e)}")
        except Exception as notify_error:
            logger.error(f"發送錯誤通知失敗: {notify_error}")

if __name__ == "__main__":
    # 執行主程式

    asyncio.run(main())

上面是台灣的ETF選股

In [ ]:

import asyncio
import aiohttp
import pandas as pd
import numpy as np
import yfinance as yf
import logging
import traceback
from datetime import datetime, timezone, timedelta
from typing import Dict, List, Any, Optional
import json
import os
import time
import requests
import random
from io import StringIO

# 如果您在 Jupyter Notebook 中，請使用這個版本
import nest_asyncio

# 安裝 nest_asyncio（如果尚未安裝）
try:
    import nest_asyncio
except ImportError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "nest_asyncio"])
    import nest_asyncio

# 應用 nest_asyncio 以允許嵌套事件循環
nest_asyncio.apply()

# 設定日誌
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# 台北時區
taipei_tz = timezone(timedelta(hours=8))

# Telegram 設定
TELEGRAM_BOT_TOKEN = "7902318521:AAEYoDMqwfHabI7L1SRiE4z33aFay42-VGE"
TELEGRAM_CHAT_IDS = [879781796, 8113868436]

# Discord 設定 (可選)
DISCORD_WEBHOOK_URL = "https://discord.com/api/webhooks/1362715080734802102/Jma7A3VhEQrrRxIX_JW2l6rATjAZXsGXGfnJuAMqmS1QvqG_2ptg3vr_nsVnuV_PlnBl"
MESSAGING_AVAILABLE = False

try:
    from discord_webhook import DiscordWebhook
    MESSAGING_AVAILABLE = True
except ImportError:
    MESSAGING_AVAILABLE = False
    logger.info("Discord webhook 套件未安裝，將跳過 Discord 通知")

class StockAnalyzer:
    def __init__(self):
        """初始化股票分析器"""
        self.config = {
            'rsi_period': 14,
            'macd_fast': 12,
            'macd_slow': 26,
            'macd_signal': 9,
            'bb_period': 20,
            'bb_std': 2,
            'kd_period': 14,
            'volume_ma_period': 20,
            'min_volume_lots': 1000,  # 最小成交量（張）
            'trend_weights': {
                'price_trend': 0.3,
                'volume_trend': 0.2,
                'technical_score': 0.5
            }
        }
        self.taiwan_stocks = None
        self.stock_list_path = "taiwan_stocks_cache.csv"

    def get_us_stocks(self, force_update=True):
        """獲取美股股票清單"""
        if not force_update and os.path.exists(self.stock_list_path):
            if (time.time() - os.path.getmtime(self.stock_list_path)) < 86400:
                logger.info("從快取載入美股股票列表。")
                return pd.read_csv(self.stock_list_path, dtype={'stock_id': str})

        logger.info("從網路獲取最新美股清單...")
        all_stocks_df = []

        # 使用多個數據源獲取美股清單
        try:
            # 方法1: 使用 yfinance 獲取 S&P 500 成分股
            import yfinance as yf

            # 獲取 S&P 500 成分股
            sp500_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
            sp500_df = pd.read_html(sp500_url)[0]
            sp500_df = sp500_df.rename(columns={
                'Symbol': 'stock_id',
                'Security': 'stock_name',
                'GICS Sector': 'industry'
            })
            sp500_df['market'] = 'NYSE/NASDAQ'
            sp500_df['yahoo_symbol'] = sp500_df['stock_id']
            all_stocks_df.append(sp500_df[['stock_id', 'stock_name', 'market', 'industry', 'yahoo_symbol']])

            # 獲取 NASDAQ 100 成分股
            nasdaq100_url = "https://en.wikipedia.org/wiki/NASDAQ-100"
            nasdaq100_df = pd.read_html(nasdaq100_url)[4]  # 通常是第4個表格
            nasdaq100_df = nasdaq100_df.rename(columns={
                'Ticker': 'stock_id',
                'Company': 'stock_name',
                'GICS Sector': 'industry'
            })
            nasdaq100_df['market'] = 'NASDAQ'
            nasdaq100_df['yahoo_symbol'] = nasdaq100_df['stock_id']
            all_stocks_df.append(nasdaq100_df[['stock_id', 'stock_name', 'market', 'industry', 'yahoo_symbol']])

        except Exception as e:
            logger.error(f"從網路獲取美股清單失敗: {e}")
            logger.info("使用備用美股清單")
            return self._get_backup_us_stock_list()

        if not all_stocks_df:
            logger.error("無法從網站獲取任何美股數據，使用備用清單。")
            return self._get_backup_us_stock_list()

        try:
            df = pd.concat(all_stocks_df, ignore_index=True)

            # 清理數據
            df = df.dropna(subset=['stock_id', 'stock_name']).copy()

            # 過濾美股股票代碼（通常是1-5個字母）
            df = df[df['stock_id'].str.match(r'^[A-Z]{1,5}$', na=False)].copy()

            # 排除ETF、基金等非股票商品
            exclude_keywords = ['ETF', 'Fund', 'Trust', 'LP', 'Inc.', 'Corp.', 'Ltd.']
            # 但保留公司名稱中常見的 Inc., Corp. 等
            exclude_pattern = r'\b(ETF|Fund|Trust|LP)\b'
            df = df[~df['stock_name'].str.contains(exclude_pattern, case=False, na=False)].copy()

            # 去重並整理
            df = df.drop_duplicates(subset=['stock_id']).reset_index(drop=True)

            # 填充缺失的行業資訊
            df['industry'] = df['industry'].fillna('其他')

            final_df = df[['stock_id', 'stock_name', 'market', 'industry', 'yahoo_symbol']]
            final_df.to_csv(self.stock_list_path, index=False)
            logger.info(f"成功獲取 {len(final_df)} 支美股清單並保存快取。")
            return final_df

        except Exception as e:
            logger.error(f"處理美股清單時發生錯誤: {e}")
            return self._get_backup_us_stock_list()

    def _get_backup_us_stock_list(self) -> pd.DataFrame:
        """備用美股清單"""
        try:
            logger.info("使用備用美股清單")

            # 美國知名股票列表
            famous_us_stocks = [
            # 科技股 (FAANG + 其他)
            {'stock_id': 'AAPL', 'stock_name': 'Apple Inc.', 'market': 'NASDAQ', 'yahoo_symbol': 'AAPL', 'industry': 'Technology'},
            {'stock_id': 'MSFT', 'stock_name': 'Microsoft Corporation', 'market': 'NASDAQ', 'yahoo_symbol': 'MSFT', 'industry': 'Technology'},
            {'stock_id': 'GOOGL', 'stock_name': 'Alphabet Inc. Class A', 'market': 'NASDAQ', 'yahoo_symbol': 'GOOGL', 'industry': 'Technology'},
            {'stock_id': 'AMZN', 'stock_name': 'Amazon.com Inc.', 'market': 'NASDAQ', 'yahoo_symbol': 'AMZN', 'industry': 'Consumer Discretionary'},
            {'stock_id': 'META', 'stock_name': 'Meta Platforms Inc.', 'market': 'NASDAQ', 'yahoo_symbol': 'META', 'industry': 'Technology'},
            {'stock_id': 'TSLA', 'stock_name': 'Tesla Inc.', 'market': 'NASDAQ', 'yahoo_symbol': 'TSLA', 'industry': 'Consumer Discretionary'},
            {'stock_id': 'NVDA', 'stock_name': 'NVIDIA Corporation', 'market': 'NASDAQ', 'yahoo_symbol': 'NVDA', 'industry': 'Technology'},
            {'stock_id': 'NFLX', 'stock_name': 'Netflix Inc.', 'market': 'NASDAQ', 'yahoo_symbol': 'NFLX', 'industry': 'Communication Services'},

            # 傳統大型股
            {'stock_id': 'BRK.B', 'stock_name': 'Berkshire Hathaway Inc. Class B', 'market': 'NYSE', 'yahoo_symbol': 'BRK-B', 'industry': 'Financial Services'},
            {'stock_id': 'JPM', 'stock_name': 'JPMorgan Chase & Co.', 'market': 'NYSE', 'yahoo_symbol': 'JPM', 'industry': 'Financial Services'},
            {'stock_id': 'JNJ', 'stock_name': 'Johnson & Johnson', 'market': 'NYSE', 'yahoo_symbol': 'JNJ', 'industry': 'Healthcare'},
            {'stock_id': 'V', 'stock_name': 'Visa Inc.', 'market': 'NYSE', 'yahoo_symbol': 'V', 'industry': 'Financial Services'},
            {'stock_id': 'PG', 'stock_name': 'Procter & Gamble Co.', 'market': 'NYSE', 'yahoo_symbol': 'PG', 'industry': 'Consumer Staples'},
            {'stock_id': 'UNH', 'stock_name': 'UnitedHealth Group Inc.', 'market': 'NYSE', 'yahoo_symbol': 'UNH', 'industry': 'Healthcare'},
            {'stock_id': 'HD', 'stock_name': 'Home Depot Inc.', 'market': 'NYSE', 'yahoo_symbol': 'HD', 'industry': 'Consumer Discretionary'},
            {'stock_id': 'MA', 'stock_name': 'Mastercard Inc.', 'market': 'NYSE', 'yahoo_symbol': 'MA', 'industry': 'Financial Services'},

            # 工業股
            {'stock_id': 'BA', 'stock_name': 'Boeing Co.', 'market': 'NYSE', 'yahoo_symbol': 'BA', 'industry': 'Industrials'},
            {'stock_id': 'CAT', 'stock_name': 'Caterpillar Inc.', 'market': 'NYSE', 'yahoo_symbol': 'CAT', 'industry': 'Industrials'},
            {'stock_id': 'MMM', 'stock_name': '3M Co.', 'market': 'NYSE', 'yahoo_symbol': 'MMM', 'industry': 'Industrials'},
            {'stock_id': 'GE', 'stock_name': 'General Electric Co.', 'market': 'NYSE', 'yahoo_symbol': 'GE', 'industry': 'Industrials'},

            # 能源股
            {'stock_id': 'XOM', 'stock_name': 'Exxon Mobil Corporation', 'market': 'NYSE', 'yahoo_symbol': 'XOM', 'industry': 'Energy'},
            {'stock_id': 'CVX', 'stock_name': 'Chevron Corporation', 'market': 'NYSE', 'yahoo_symbol': 'CVX', 'industry': 'Energy'},

            # 金融股
            {'stock_id': 'BAC', 'stock_name': 'Bank of America Corp.', 'market': 'NYSE', 'yahoo_symbol': 'BAC', 'industry': 'Financial Services'},
            {'stock_id': 'WFC', 'stock_name': 'Wells Fargo & Co.', 'market': 'NYSE', 'yahoo_symbol': 'WFC', 'industry': 'Financial Services'},
            {'stock_id': 'GS', 'stock_name': 'Goldman Sachs Group Inc.', 'market': 'NYSE', 'yahoo_symbol': 'GS', 'industry': 'Financial Services'},
            {'stock_id': 'MS', 'stock_name': 'Morgan Stanley', 'market': 'NYSE', 'yahoo_symbol': 'MS', 'industry': 'Financial Services'},

            # 消費股
            {'stock_id': 'KO', 'stock_name': 'Coca-Cola Co.', 'market': 'NYSE', 'yahoo_symbol': 'KO', 'industry': 'Consumer Staples'},
            {'stock_id': 'PEP', 'stock_name': 'PepsiCo Inc.', 'market': 'NASDAQ', 'yahoo_symbol': 'PEP', 'industry': 'Consumer Staples'},
            {'stock_id': 'WMT', 'stock_name': 'Walmart Inc.', 'market': 'NYSE', 'yahoo_symbol': 'WMT', 'industry': 'Consumer Staples'},
            {'stock_id': 'MCD', 'stock_name': 'McDonald\'s Corp.', 'market': 'NYSE', 'yahoo_symbol': 'MCD', 'industry': 'Consumer Discretionary'},

            # 醫療保健股
            {'stock_id': 'PFE', 'stock_name': 'Pfizer Inc.', 'market': 'NYSE', 'yahoo_symbol': 'PFE', 'industry': 'Healthcare'},
            {'stock_id': 'ABBV', 'stock_name': 'AbbVie Inc.', 'market': 'NYSE', 'yahoo_symbol': 'ABBV', 'industry': 'Healthcare'},
            {'stock_id': 'MRK', 'stock_name': 'Merck & Co. Inc.', 'market': 'NYSE', 'yahoo_symbol': 'MRK', 'industry': 'Healthcare'},
            {'stock_id': 'LLY', 'stock_name': 'Eli Lilly and Co.', 'market': 'NYSE', 'yahoo_symbol': 'LLY', 'industry': 'Healthcare'},

            # 通信股
            {'stock_id': 'T', 'stock_name': 'AT&T Inc.', 'market': 'NYSE', 'yahoo_symbol': 'T', 'industry': 'Communication Services'},
            {'stock_id': 'VZ', 'stock_name': 'Verizon Communications Inc.', 'market': 'NYSE', 'yahoo_symbol': 'VZ', 'industry': 'Communication Services'},

            # 半導體股
            {'stock_id': 'AMD', 'stock_name': 'Advanced Micro Devices Inc.', 'market': 'NASDAQ', 'yahoo_symbol': 'AMD', 'industry': 'Technology'},
            {'stock_id': 'INTC', 'stock_name': 'Intel Corporation', 'market': 'NASDAQ', 'yahoo_symbol': 'INTC', 'industry': 'Technology'},
            {'stock_id': 'QCOM', 'stock_name': 'Qualcomm Inc.', 'market': 'NASDAQ', 'yahoo_symbol': 'QCOM', 'industry': 'Technology'},

            # 其他知名科技股
            {'stock_id': 'CRM', 'stock_name': 'Salesforce Inc.', 'market': 'NYSE', 'yahoo_symbol': 'CRM', 'industry': 'Technology'},
            {'stock_id': 'ORCL', 'stock_name': 'Oracle Corporation', 'market': 'NYSE', 'yahoo_symbol': 'ORCL', 'industry': 'Technology'},
            {'stock_id': 'ADBE', 'stock_name': 'Adobe Inc.', 'market': 'NASDAQ', 'yahoo_symbol': 'ADBE', 'industry': 'Technology'},
            {'stock_id': 'PYPL', 'stock_name': 'PayPal Holdings Inc.', 'market': 'NASDAQ', 'yahoo_symbol': 'PYPL', 'industry': 'Financial Services'},

            # 電動車相關
            {'stock_id': 'NIO', 'stock_name': 'NIO Inc.', 'market': 'NYSE', 'yahoo_symbol': 'NIO', 'industry': 'Consumer Discretionary'},
            {'stock_id': 'RIVN', 'stock_name': 'Rivian Automotive Inc.', 'market': 'NASDAQ', 'yahoo_symbol': 'RIVN', 'industry': 'Consumer Discretionary'},

            # 新興科技股
            {'stock_id': 'PLTR', 'stock_name': 'Palantir Technologies Inc.', 'market': 'NYSE', 'yahoo_symbol': 'PLTR', 'industry': 'Technology'},
            {'stock_id': 'SNOW', 'stock_name': 'Snowflake Inc.', 'market': 'NYSE', 'yahoo_symbol': 'SNOW', 'industry': 'Technology'},
            ]

            df = pd.DataFrame(famous_us_stocks)
            logger.info(f"備用美股清單包含 {len(df)} 支股票")
            return df

        except Exception as e:
            logger.error(f"生成備用美股清單時發生錯誤: {e}")
            return pd.DataFrame()


    def check_volume_filter(self, df: pd.DataFrame) -> bool:
        """檢查成交量是否符合篩選條件（每日至少1000張）"""
        try:
            if df.empty or 'Volume' not in df.columns:
                return False

            # 取最近20天的平均成交量
            recent_volume = df['Volume'].tail(20).mean()

            # 轉換為張數（1張 = 1000股）
            volume_lots = recent_volume / 1000

            # 檢查是否符合最小成交量要求
            meets_volume = volume_lots >= self.config['min_volume_lots']

            if meets_volume:
                logger.debug(f"成交量符合條件: {volume_lots:.0f} 張/日 (≥ {self.config['min_volume_lots']} 張)")
            else:
                logger.debug(f"成交量不符合條件: {volume_lots:.0f} 張/日 (< {self.config['min_volume_lots']} 張)")

            return meets_volume

        except Exception as e:
            logger.error(f"檢查成交量篩選時發生錯誤: {e}")
            return False

    def fetch_yfinance_data(self, stock_id: str, period: str = "1y", interval: str = "1d", retries: int = 3):
        """獲取股票數據（同步版本）"""
        for attempt in range(retries):
            try:
                df = yf.download(tickers=stock_id, period=period, interval=interval, progress=False, auto_adjust=True)
                if df.empty:
                    logger.warning(f"[{stock_id}] 無數據返回")
                    return pd.DataFrame()

                if isinstance(df.columns, pd.MultiIndex):
                    df.columns = df.columns.get_level_values(0)

                required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
                for col in required_cols:
                    if col in df.columns:
                        df[col] = pd.to_numeric(df[col], errors='coerce')

                df = df.dropna(subset=required_cols)
                return df.reset_index()

            except Exception as e:
                if attempt < retries - 1:
                    sleep_time = 0.5 * (2 ** attempt) + random.uniform(0, 1)
                    logger.warning(f"[{stock_id}] 第 {attempt + 1}/{retries} 次嘗試失敗: {e}。等待 {sleep_time:.2f} 秒後重試...")
                    time.sleep(sleep_time)
                else:
                    logger.error(f"[{stock_id}] 獲取數據失敗。")
                    return pd.DataFrame()
        return pd.DataFrame()

    def calculate_technical_indicators(self, df: pd.DataFrame) -> pd.DataFrame:
        """計算技術指標"""
        try:
            if df.empty:
                return df

            df = df.copy()

            # RSI
            df['RSI'] = self.calculate_rsi(df['Close'], self.config['rsi_period'])

            # MACD
            macd_line, macd_signal, macd_histogram = self.calculate_macd(
                df['Close'],
                self.config['macd_fast'],
                self.config['macd_slow'],
                self.config['macd_signal']
            )
            df['MACD'] = macd_line
            df['MACD_Signal'] = macd_signal
            df['MACD_Histogram'] = macd_histogram

            # 布林帶
            bb_upper, bb_middle, bb_lower = self.calculate_bollinger_bands(
                df['Close'],
                self.config['bb_period'],
                self.config['bb_std']
            )
            df['BB_Upper'] = bb_upper
            df['BB_Middle'] = bb_middle
            df['BB_Lower'] = bb_lower

            # KD指標
            k_percent, d_percent = self.calculate_kd(
                df['High'],
                df['Low'],
                df['Close'],
                self.config['kd_period']
            )
            df['K_Percent'] = k_percent
            df['D_Percent'] = d_percent

            # 移動平均線
            df['MA5'] = df['Close'].rolling(window=5).mean()
            df['MA10'] = df['Close'].rolling(window=10).mean()
            df['MA20'] = df['Close'].rolling(window=20).mean()

            # 成交量移動平均
            if 'Volume' in df.columns:
                df['Volume_MA'] = df['Volume'].rolling(window=self.config['volume_ma_period']).mean()

            return df

        except Exception as e:
            logger.error(f"計算技術指標時發生錯誤: {e}")
            return df

    def calculate_rsi(self, prices: pd.Series, period: int = 14) -> pd.Series:
        """計算RSI"""
        try:
            delta = prices.diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
            rs = gain / loss
            rsi = 100 - (100 / (1 + rs))
            return rsi
        except Exception as e:
            logger.error(f"計算RSI時發生錯誤: {e}")
            return pd.Series(index=prices.index, data=50)

    def calculate_macd(self, prices: pd.Series, fast: int = 12, slow: int = 26, signal: int = 9):
        """計算MACD"""
        try:
            ema_fast = prices.ewm(span=fast).mean()
            ema_slow = prices.ewm(span=slow).mean()
            macd_line = ema_fast - ema_slow
            macd_signal = macd_line.ewm(span=signal).mean()
            macd_histogram = macd_line - macd_signal
            return macd_line, macd_signal, macd_histogram
        except Exception as e:
            logger.error(f"計算MACD時發生錯誤: {e}")
            zero_series = pd.Series(index=prices.index, data=0)
            return zero_series, zero_series, zero_series

    def calculate_bollinger_bands(self, prices: pd.Series, period: int = 20, std_dev: int = 2):
        """計算布林帶"""
        try:
            middle = prices.rolling(window=period).mean()
            std = prices.rolling(window=period).std()
            upper = middle + (std * std_dev)
            lower = middle - (std * std_dev)
            return upper, middle, lower
        except Exception as e:
            logger.error(f"計算布林帶時發生錯誤: {e}")
            zero_series = pd.Series(index=prices.index, data=0)
            return zero_series, zero_series, zero_series

    def calculate_kd(self, high: pd.Series, low: pd.Series, close: pd.Series, period: int = 14):
        """計算KD指標"""
        try:
            lowest_low = low.rolling(window=period).min()
            highest_high = high.rolling(window=period).max()

            k_percent = 100 * ((close - lowest_low) / (highest_high - lowest_low))
            k_percent = k_percent.rolling(window=3).mean()
            d_percent = k_percent.rolling(window=3).mean()

            return k_percent, d_percent
        except Exception as e:
            logger.error(f"計算KD時發生錯誤: {e}")
            fifty_series = pd.Series(index=high.index, data=50)
            return fifty_series, fifty_series

    def analyze_trend(self, df: pd.DataFrame) -> Dict[str, Any]:
        """趨勢分析"""
        try:
            if df.empty or len(df) < 20:
                return {'trend': '盤整', 'strength': 0, 'price_change_5d': 0, 'price_change_20d': 0}

            current_price = float(df['Close'].iloc[-1])
            price_5d_ago = float(df['Close'].iloc[-6]) if len(df) >= 6 else current_price
            price_20d_ago = float(df['Close'].iloc[-21]) if len(df) >= 21 else current_price

            change_5d = ((current_price - price_5d_ago) / price_5d_ago * 100) if price_5d_ago != 0 else 0
            change_20d = ((current_price - price_20d_ago) / price_20d_ago * 100) if price_20d_ago != 0 else 0

            # 移動平均線趨勢
            ma5_current = float(df['MA5'].iloc[-1]) if 'MA5' in df.columns and not pd.isna(df['MA5'].iloc[-1]) else current_price
            ma20_current = float(df['MA20'].iloc[-1]) if 'MA20' in df.columns and not pd.isna(df['MA20'].iloc[-1]) else current_price

            # 判斷趨勢
            if change_5d > 3 and change_20d > 5 and current_price > ma5_current > ma20_current:
                trend = '強勢上漲'
                strength = 3
            elif change_5d > 1 and change_20d > 2 and current_price > ma5_current:
                trend = '上漲'
                strength = 2
            elif change_5d < -3 and change_20d < -5 and current_price < ma5_current < ma20_current:
                trend = '強勢下跌'
                strength = -3
            elif change_5d < -1 and change_20d < -2 and current_price < ma5_current:
                trend = '下跌'
                strength = -2
            else:
                trend = '盤整'
                strength = 0

            return {
                'trend': trend,
                'strength': strength,
                'price_change_5d': change_5d,
                'price_change_20d': change_20d,
                'ma5_position': ma5_current,
                'ma20_position': ma20_current
            }

        except Exception as e:
            logger.error(f"趨勢分析時發生錯誤: {e}")
            return {'trend': '盤整', 'strength': 0, 'price_change_5d': 0, 'price_change_20d': 0}

    def analyze_volume(self, df: pd.DataFrame) -> Dict[str, Any]:
        """成交量分析"""
        try:
            if df.empty or 'Volume' not in df.columns or len(df) < 20:
                return {'volume_trend': '正常', 'volume_ratio': 1.0, 'volume_signal': '中性', 'avg_volume_lots': 0}

            current_volume = float(df['Volume'].iloc[-1])
            avg_volume_20d = float(df['Volume'].rolling(window=20).mean().iloc[-1])

            volume_ratio = current_volume / avg_volume_20d if avg_volume_20d > 0 else 1.0
            avg_volume_lots = avg_volume_20d / 1000  # 轉換為張數

            if volume_ratio > 2.0:
                volume_trend = '爆量'
                volume_signal = '強烈'
            elif volume_ratio > 1.5:
                volume_trend = '放量'
                volume_signal = '積極'
            elif volume_ratio < 0.5:
                volume_trend = '縮量'
                volume_signal = '消極'
            else:
                volume_trend = '正常'
                volume_signal = '中性'

            return {
                'volume_trend': volume_trend,
                'volume_ratio': volume_ratio,
                'volume_signal': volume_signal,
                'avg_volume_lots': avg_volume_lots
            }

        except Exception as e:
            logger.error(f"成交量分析時發生錯誤: {e}")
            return {'volume_trend': '正常', 'volume_ratio': 1.0, 'volume_signal': '中性', 'avg_volume_lots': 0}

    def analyze_technical_indicators(self, df: pd.DataFrame) -> Dict[str, Any]:
        """技術指標分析"""
        try:
            if df.empty:
                return {
                    'rsi_signal': '中性', 'rsi_value': 50,
                    'macd_signal': '中性', 'macd_value': 0,
                    'kd_signal': '中性', 'k_value': 50, 'd_value': 50,
                    'bb_signal': '中性', 'bb_position': 0.5,
                    'score': 50
                }

            signals = {}
            score = 50

            # RSI 分析
            if 'RSI' in df.columns and not df['RSI'].empty:
                rsi_value = float(df['RSI'].iloc[-1]) if not pd.isna(df['RSI'].iloc[-1]) else 50
                signals['rsi_value'] = rsi_value

                if rsi_value > 80:
                    signals['rsi_signal'] = '超買'
                    score -= 15
                elif rsi_value > 70:
                    signals['rsi_signal'] = '偏高'
                    score -= 5
                elif rsi_value < 20:
                    signals['rsi_signal'] = '超賣'
                    score += 15
                elif rsi_value < 30:
                    signals['rsi_signal'] = '偏低'
                    score += 5
                else:
                    signals['rsi_signal'] = '中性'
            else:
                signals['rsi_signal'] = '中性'
                signals['rsi_value'] = 50

            # MACD 分析
            if all(col in df.columns for col in ['MACD', 'MACD_Signal']) and len(df) >= 2:
                macd_current = float(df['MACD'].iloc[-1]) if not pd.isna(df['MACD'].iloc[-1]) else 0
                macd_signal_current = float(df['MACD_Signal'].iloc[-1]) if not pd.isna(df['MACD_Signal'].iloc[-1]) else 0
                macd_prev = float(df['MACD'].iloc[-2]) if not pd.isna(df['MACD'].iloc[-2]) else 0
                macd_signal_prev = float(df['MACD_Signal'].iloc[-2]) if not pd.isna(df['MACD_Signal'].iloc[-2]) else 0

                signals['macd_value'] = macd_current

                if macd_prev <= macd_signal_prev and macd_current > macd_signal_current:
                    signals['macd_signal'] = '黃金交叉'
                    score += 20
                elif macd_prev >= macd_signal_prev and macd_current < macd_signal_current:
                    signals['macd_signal'] = '死亡交叉'
                    score -= 20
                elif macd_current > macd_signal_current:
                    signals['macd_signal'] = '多頭'
                    score += 5
                elif macd_current < macd_signal_current:
                    signals['macd_signal'] = '空頭'
                    score -= 5
                else:
                    signals['macd_signal'] = '中性'
            else:
                signals['macd_signal'] = '中性'
                signals['macd_value'] = 0

            # KD 分析
            if all(col in df.columns for col in ['K_Percent', 'D_Percent']):
                k_value = float(df['K_Percent'].iloc[-1]) if not pd.isna(df['K_Percent'].iloc[-1]) else 50
                d_value = float(df['D_Percent'].iloc[-1]) if not pd.isna(df['D_Percent'].iloc[-1]) else 50

                signals['k_value'] = k_value
                signals['d_value'] = d_value

                if k_value > 80 and d_value > 80:
                    signals['kd_signal'] = '超買'
                    score -= 10
                elif k_value < 20 and d_value < 20:
                    signals['kd_signal'] = '超賣'
                    score += 10
                elif k_value > d_value:
                    signals['kd_signal'] = '偏多'
                    score += 3
                elif k_value < d_value:
                    signals['kd_signal'] = '偏空'
                    score -= 3
                else:
                    signals['kd_signal'] = '中性'
            else:
                signals['kd_signal'] = '中性'
                signals['k_value'] = 50
                signals['d_value'] = 50

            # 布林帶分析
            if all(col in df.columns for col in ['BB_Upper', 'BB_Lower', 'Close']):
                current_price = float(df['Close'].iloc[-1])
                bb_upper = float(df['BB_Upper'].iloc[-1]) if not pd.isna(df['BB_Upper'].iloc[-1]) else current_price
                bb_lower = float(df['BB_Lower'].iloc[-1]) if not pd.isna(df['BB_Lower'].iloc[-1]) else current_price

                if bb_upper != bb_lower:
                    bb_position = (current_price - bb_lower) / (bb_upper - bb_lower)
                    signals['bb_position'] = bb_position

                    if bb_position > 0.8:
                        signals['bb_signal'] = '接近上軌'
                        score -= 5
                    elif bb_position < 0.2:
                        signals['bb_signal'] = '接近下軌'
                        score += 5
                    else:
                        signals['bb_signal'] = '中性'
                else:
                    signals['bb_signal'] = '中性'
                    signals['bb_position'] = 0.5
            else:
                signals['bb_signal'] = '中性'
                signals['bb_position'] = 0.5

            score = max(0, min(100, score))
            signals['score'] = score

            return signals

        except Exception as e:
            logger.error(f"技術指標分析時發生錯誤: {e}")
            return {
                'rsi_signal': '中性', 'rsi_value': 50,
                'macd_signal': '中性', 'macd_value': 0,
                'kd_signal': '中性', 'k_value': 50, 'd_value': 50,
                'bb_signal': '中性', 'bb_position': 0.5,
                'score': 50
            }

    def calculate_combined_score(self, trend_analysis: Dict, volume_analysis: Dict, technical_analysis: Dict) -> float:
        """計算綜合分數"""
        try:
            base_score = 50

            # 趨勢分數 (權重: 30%)
            trend_score = 0
            trend_strength = trend_analysis.get('strength', 0)
            if trend_strength > 0:
                trend_score = min(30, trend_strength * 10)
            elif trend_strength < 0:
                trend_score = max(-30, trend_strength * 10)

            # 成交量分數 (權重: 20%)
            volume_score = 0
            volume_ratio = volume_analysis.get('volume_ratio', 1.0)
            if volume_ratio > 1.5:
                volume_score = 10
            elif volume_ratio > 1.2:
                volume_score = 5
            elif volume_ratio < 0.7:
                volume_score = -5

            # 技術指標分數 (權重: 50%)
            tech_score = technical_analysis.get('score', 50) - 50

            # 計算加權總分
            total_score = base_score + (trend_score * 0.3) + (volume_score * 0.2) + (tech_score * 0.5)

            return max(0, min(100, total_score))

        except Exception as e:
            logger.error(f"計算綜合分數時發生錯誤: {e}")
            return 50.0

    def generate_recommendation(self, combined_score: float, trend_analysis: Dict, technical_analysis: Dict) -> Dict[str, Any]:
        """生成投資建議"""
        try:
            recommendation = "持有"
            confidence = "中等"
            reasons = []

            if combined_score >= 75:
                recommendation = "強力買進"
                confidence = "高"
            elif combined_score >= 60:
                recommendation = "買進"
                confidence = "中高"
            elif combined_score >= 40:
                recommendation = "持有"
                confidence = "中等"
            elif combined_score >= 25:
                recommendation = "賣出"
                confidence = "中高"
            else:
                recommendation = "強力賣出"
                confidence = "高"

            # 生成建議原因
            trend = trend_analysis.get('trend', '盤整')
            if '上漲' in trend:
                reasons.append(f"價格趨勢：{trend}")
            elif '下跌' in trend:
                reasons.append(f"價格趨勢：{trend}")

            rsi_signal = technical_analysis.get('rsi_signal', '中性')
            if rsi_signal in ['超賣', '偏低']:
                reasons.append(f"RSI指標：{rsi_signal}，可能反彈")
            elif rsi_signal in ['超買', '偏高']:
                reasons.append(f"RSI指標：{rsi_signal}，注意回調")

            macd_signal = technical_analysis.get('macd_signal', '中性')
            if macd_signal == '黃金交叉':
                reasons.append("MACD出現黃金交叉")
            elif macd_signal == '死亡交叉':
                reasons.append("MACD出現死亡交叉")

            return {
                'recommendation': recommendation,
                'confidence': confidence,
                'score': combined_score,
                'reasons': reasons
            }

        except Exception as e:
            logger.error(f"生成投資建議時發生錯誤: {e}")
            return {
                'recommendation': '持有',
                'confidence': '低',
                'score': 50,
                'reasons': ['分析過程出現錯誤']
            }

    async def analyze_stock_async(self, session: aiohttp.ClientSession, stock_info: Dict) -> Dict[str, Any]:
        """異步分析單支股票"""
        symbol = stock_info['yahoo_symbol']
        stock_name = stock_info['stock_name']

        try:
            logger.info(f"開始分析股票: {stock_name} ({symbol})")

            # 獲取股票數據
            df = self.fetch_yfinance_data(symbol)
            if df is None or df.empty or len(df) < 60:
                return {
                    'symbol': symbol,
                    'stock_name': stock_name,
                    'success': False,
                    'error': '無法獲取足夠的股票數據'
                }

            # 檢查成交量篩選條件
            if not self.check_volume_filter(df):
                return {
                    'symbol': symbol,
                    'stock_name': stock_name,
                    'success': False,
                    'error': f'成交量不符合條件（需≥{self.config["min_volume_lots"]}張/日）'
                }

            # 計算技術指標
            df_with_indicators = self.calculate_technical_indicators(df)

            # 進行各項分析
            trend_analysis = self.analyze_trend(df_with_indicators)
            volume_analysis = self.analyze_volume(df_with_indicators)
            technical_analysis = self.analyze_technical_indicators(df_with_indicators)

            # 計算綜合分數
            combined_score = self.calculate_combined_score(trend_analysis, volume_analysis, technical_analysis)

            # 生成投資建議
            recommendation = self.generate_recommendation(combined_score, trend_analysis, technical_analysis)

            # 獲取當前價格資訊
            current_price = float(df['Close'].iloc[-1])
            price_change = trend_analysis.get('price_change_5d', 0)

            result = {
                'symbol': symbol,
                'stock_name': stock_name,
                'stock_id': stock_info.get('stock_id', ''),
                'market': stock_info.get('market', ''),
                'industry': stock_info.get('industry', ''),
                'success': True,
                'current_price': current_price,
                'price_change_5d': price_change,
                'combined_score': combined_score,
                'trend_analysis': trend_analysis,
                'volume_analysis': volume_analysis,
                'technical_analysis': technical_analysis,
                'recommendation': recommendation,
                'analysis_time': datetime.now(taipei_tz).isoformat()
            }

            logger.info(f"完成分析: {stock_name} - 分數: {combined_score:.1f} - 建議: {recommendation['recommendation']} - 成交量: {volume_analysis.get('avg_volume_lots', 0):.0f}張/日")
            return result

        except Exception as e:
            logger.error(f"分析股票 {symbol} 時發生錯誤: {e}")
            return {
                'symbol': symbol,
                'stock_name': stock_name,
                'success': False,
                'error': str(e)
            }

    async def analyze_all_stocks(self) -> Dict[str, Any]:
        """分析所有股票（加入成交量篩選）"""
        try:
            # 獲取完整股票清單
            if self.taiwan_stocks is None:
                self.taiwan_stocks = self.get_us_stocks()

            if self.taiwan_stocks.empty:
                logger.error("無法獲取股票清單")
                return {}

            logger.info(f"準備分析 {len(self.taiwan_stocks)} 支股票（包含成交量篩選）")

            results = {}
            failed_count = 0
            volume_filtered_count = 0

            # 使用 aiohttp 進行異步分析
            async with aiohttp.ClientSession() as session:
                # 創建分析任務
                tasks = []
                for _, stock_info in self.taiwan_stocks.iterrows():
                    task = self.analyze_stock_async(session, stock_info.to_dict())
                    tasks.append(task)

                # 分批執行任務以避免過載
                batch_size = 10
                for i in range(0, len(tasks), batch_size):
                    batch_tasks = tasks[i:i+batch_size]
                    batch_results = await asyncio.gather(*batch_tasks, return_exceptions=True)

                    # 處理批次結果
                    for result in batch_results:
                        if isinstance(result, Exception):
                            logger.error(f"分析任務異常: {result}")
                            failed_count += 1
                            continue

                        if isinstance(result, dict) and 'symbol' in result:
                            if result.get('success', False):
                                results[result['symbol']] = result
                            else:
                                error_msg = result.get('error', '')
                                if '成交量不符合條件' in error_msg:
                                    volume_filtered_count += 1
                                else:
                                    failed_count += 1

                    # 批次間暫停避免過載
                    if i + batch_size < len(tasks):
                        await asyncio.sleep(2)

            logger.info(f"分析完成統計：")
            logger.info(f"  - 成功分析: {len(results)} 支")
            logger.info(f"  - 成交量篩選淘汰: {volume_filtered_count} 支")
            logger.info(f"  - 其他失敗: {failed_count} 支")

            return results

        except Exception as e:
            logger.error(f"批量分析股票時發生錯誤: {e}")
            return {}

def format_analysis_message(results: Dict[str, Any], limit: int = 10) -> str:
    """格式化分析結果為訊息（顯示前十名）"""
    try:
        if not results:
            return "❌ 未能獲取任何分析結果"

        # 過濾成功的結果並按分數排序
        successful_results = [
            result for result in results.values()
            if result.get('success', False) and result.get('combined_score', 0) > 0
        ]

        successful_results.sort(key=lambda x: x.get('combined_score', 0), reverse=True)

        # 統計資訊
        total_analyzed = len(results)
        successful_count = len(successful_results)

        # 建立訊息
        message_parts = []
        message_parts.append("🏆 美股技術分析 - 前十名優質股票")
        message_parts.append("=" * 40)
        message_parts.append(f"📈 分析時間: {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}")
        message_parts.append(f"📊 成功分析: {successful_count} 支股票")
        message_parts.append(f"🔍 成交量篩選: ≥1000張/日")
        message_parts.append("")

        if not successful_results:
            message_parts.append("❌ 沒有符合條件的優質股票")
            return "\n".join(message_parts)

        # 顯示前十名優質股票
        top_stocks = successful_results[:limit]
        message_parts.append(f"🏆 前 {len(top_stocks)} 名優質股票:")
        message_parts.append("-" * 40)

        for i, result in enumerate(top_stocks, 1):
            stock_name = result.get('stock_name', 'Unknown')
            stock_id = result.get('stock_id', 'Unknown')
            score = result.get('combined_score', 0)
            recommendation = result.get('recommendation', {})
            rec_text = recommendation.get('recommendation', '持有')
            current_price = result.get('current_price', 0)
            price_change = result.get('price_change_5d', 0)
            volume_info = result.get('volume_analysis', {})
            avg_volume_lots = volume_info.get('avg_volume_lots', 0)

            # 價格變化符號
            change_symbol = "📈" if price_change > 0 else "📉" if price_change < 0 else "➡️"

            # 建議符號
            rec_symbol = "🚀" if "強力買" in rec_text else "📈" if "買" in rec_text else "📊" if "持有" in rec_text else "📉"

            message_parts.append(f"{i}. {stock_name} ({stock_id})")
            message_parts.append(f"   💰 價格: {current_price:.2f} ({change_symbol}{price_change:+.2f}%)")
            message_parts.append(f"   ⭐ 評分: {score:.1f}/100")
            message_parts.append(f"   {rec_symbol} 建議: {rec_text}")
            message_parts.append(f"   📊 成交量: {avg_volume_lots:.0f}張/日")

            # 添加主要技術指標
            tech = result.get('technical_analysis', {})
            rsi_value = tech.get('rsi_value', 50)
            macd_signal = tech.get('macd_signal', '中性')
            message_parts.append(f"   🔍 RSI: {rsi_value:.1f} | MACD: {macd_signal}")
            message_parts.append("")

        # 添加統計摘要
        if successful_results:
            avg_score = sum(r.get('combined_score', 0) for r in successful_results) / len(successful_results)
            high_score_count = len([r for r in successful_results if r.get('combined_score', 0) >= 70])

            message_parts.append("📊 統計摘要:")
            message_parts.append(f"   📊 平均分數: {avg_score:.1f}")
            message_parts.append(f"   🌟 高分股票 (≥70): {high_score_count}")

            # 建議分布
            recommendations = {}
            for result in top_stocks:
                rec = result.get('recommendation', {}).get('recommendation', '持有')
                recommendations[rec] = recommendations.get(rec, 0) + 1

            if recommendations:
                message_parts.append("   📈 前十名建議分布:")
                for rec, count in recommendations.items():
                    message_parts.append(f"      {rec}: {count}")

        message_parts.append("")
        message_parts.append("⚠️ 投資提醒:")
        message_parts.append("• 本分析僅供參考，投資有風險")
        message_parts.append("• 已篩選成交量≥1000張/日的活躍股票")
        message_parts.append("• 請結合基本面分析做最終決策")

        return "\n".join(message_parts)

    except Exception as e:
        logger.error(f"格式化訊息時發生錯誤: {e}")
        return f"❌ 格式化分析結果時發生錯誤: {str(e)}"

def display_terminal_results(results: Dict[str, Any], limit: int = 10):
    """在終端顯示分析結果"""
    try:
        if not results:
            print("❌ 未能獲取任何分析結果")
            return

        # 過濾成功的結果並按分數排序
        successful_results = [
            result for result in results.values()
            if result.get('success', False) and result.get('combined_score', 0) > 0
        ]

        successful_results.sort(key=lambda x: x.get('combined_score', 0), reverse=True)

        print("\n" + "=" * 80)
        print("🏆 美股技術分析結果 - 前十名優質股票".center(80))
        print("=" * 80)
        print(f"📈 分析時間: {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"📊 成功分析: {len(successful_results)} 支股票")
        print(f"🔍 成交量篩選條件: ≥1000張/日")
        print("=" * 80)

        if not successful_results:
            print("❌ 沒有符合條件的優質股票")
            return

        # 表格標題
        print(f"{'排名':<4}{'代碼':<8}{'股票名稱':<12}{'評分':<8}{'價格':<10}{'漲跌%':<8}{'建議':<10}{'成交量(張)':<12}")
        print("-" * 80)

        # 顯示前十名
        top_stocks = successful_results[:limit]
        for i, result in enumerate(top_stocks, 1):
            stock_name = result.get('stock_name', 'Unknown')[:10]  # 限制長度
            stock_id = result.get('stock_id', 'Unknown')
            score = result.get('combined_score', 0)
            recommendation = result.get('recommendation', {})
            rec_text = recommendation.get('recommendation', '持有')[:8]  # 限制長度
            current_price = result.get('current_price', 0)
            price_change = result.get('price_change_5d', 0)
            volume_info = result.get('volume_analysis', {})
            avg_volume_lots = volume_info.get('avg_volume_lots', 0)

            print(f"{i:<4}{stock_id:<8}{stock_name:<12}{score:<8.1f}{current_price:<10.2f}{price_change:<+8.2f}{rec_text:<10}{avg_volume_lots:<12.0f}")

        print("=" * 80)

        # 詳細分析前五名
        print("\n📊 詳細分析報告 (前五名):")
        print("=" * 80)

        for i, result in enumerate(top_stocks[:5], 1):
            stock_name = result.get('stock_name', 'Unknown')
            stock_id = result.get('stock_id', 'Unknown')
            score = result.get('combined_score', 0)
            recommendation = result.get('recommendation', {})
            trend_analysis = result.get('trend_analysis', {})
            technical_analysis = result.get('technical_analysis', {})
            volume_analysis = result.get('volume_analysis', {})

            print(f"\n{i}. {stock_name} ({stock_id}) - 評分: {score:.1f}")
            print("-" * 50)
            print(f"💰 當前價格: {result.get('current_price', 0):.2f}")
            print(f"📈 5日漲跌: {result.get('price_change_5d', 0):+.2f}%")
            print(f"🎯 投資建議: {recommendation.get('recommendation', '持有')}")
            print(f"📊 信心度: {recommendation.get('confidence', '中等')}")

            # 技術指標詳情
            print(f"🔍 技術指標:")
            print(f"   RSI: {technical_analysis.get('rsi_value', 50):.1f} ({technical_analysis.get('rsi_signal', '中性')})")
            print(f"   MACD: {technical_analysis.get('macd_signal', '中性')}")
            print(f"   KD: K={technical_analysis.get('k_value', 50):.1f}, D={technical_analysis.get('d_value', 50):.1f}")

            # 趨勢和成交量
            print(f"📊 趨勢分析: {trend_analysis.get('trend', '盤整')}")
            print(f"📈 成交量: {volume_analysis.get('avg_volume_lots', 0):.0f}張/日 ({volume_analysis.get('volume_trend', '正常')})")

            # 建議原因
            reasons = recommendation.get('reasons', [])
            if reasons:
                print(f"💡 建議原因: {', '.join(reasons[:2])}")  # 顯示前兩個原因

        print("\n" + "=" * 80)
        print("⚠️  投資提醒:")
        print("• 本分析僅供參考，投資一定有風險")
        print("• 已篩選成交量≥1000張/日的活躍股票")
        print("• 請務必結合基本面分析做最終投資決策")
        print("• 建議分散投資，控制風險")
        print("=" * 80)

    except Exception as e:
        logger.error(f"顯示終端結果時發生錯誤: {e}")
        print(f"❌ 顯示結果時發生錯誤: {str(e)}")

async def send_notification(session: aiohttp.ClientSession, message: str, files: List[str] = None):
    """發送通知到 Telegram 和 Discord"""

    # Telegram 通知
    try:
        # 檢查 Telegram 設定
        if not TELEGRAM_BOT_TOKEN or not TELEGRAM_CHAT_IDS or TELEGRAM_BOT_TOKEN == "YOUR_BOT_TOKEN_HERE":
            logger.warning("Telegram 設定未完成，跳過發送通知")
            print("📱 Telegram 設定未完成，請設定 TELEGRAM_BOT_TOKEN 和 TELEGRAM_CHAT_IDS")
        else:
            # 分割長訊息
            max_length = 4000
            if len(message) > max_length:
                parts = []
                current_part = ""

                for line in message.split('\n'):
                    if len(current_part + line + '\n') > max_length:
                        if current_part:
                            parts.append(current_part.strip())
                            current_part = line + '\n'
                        else:
                            # 如果單行太長，直接截斷
                            parts.append(line[:max_length])
                    else:
                        current_part += line + '\n'

                if current_part:
                    parts.append(current_part.strip())
            else:
                parts = [message]

            # 發送文字訊息給所有 chat_id
            telegram_url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"

            for chat_id in TELEGRAM_CHAT_IDS:
                for i, part in enumerate(parts):
                    if i > 0:
                        part = f"🏆 美股分析報告 (續 {i+1}/{len(parts)})\n\n" + part

                    payload = {
                        'chat_id': chat_id,
                        'text': part
                    }

                    try:
                        async with session.post(telegram_url, json=payload, timeout=20) as response:
                            if response.status == 200:
                                logger.info(f"成功發送 Telegram 通知到 {chat_id} (第 {i+1}/{len(parts)} 部分)")
                            else:
                                error_text = await response.text()
                                logger.error(f"發送 Telegram 通知失敗到 {chat_id} (第 {i+1} 部分): {response.status} - {error_text}")
                    except Exception as e:
                        logger.error(f"發送 Telegram 通知時發生網路錯誤到 {chat_id} (第 {i+1} 部分): {e}")

                    # 避免發送太快
                    if i < len(parts) - 1:
                        await asyncio.sleep(1)

    except Exception as e:
        logger.error(f"發送 Telegram 通知時異常: {e}")

    # Discord 通知 (如果可用)
    if MESSAGING_AVAILABLE and DISCORD_WEBHOOK_URL and DISCORD_WEBHOOK_URL != "YOUR_DISCORD_WEBHOOK_URL":
        try:
            webhook = DiscordWebhook(url=DISCORD_WEBHOOK_URL, content=f"```\n{message}\n```")
            response = webhook.execute()
            if response.ok:
                logger.info("Discord 通知發送成功")
            else:
                logger.error(f"Discord 通知失敗: {response.status_code} {response.content}")
        except Exception as e:
            logger.error(f"發送 Discord 通知時異常: {e}")

async def main():
    """主程式"""
    try:
        print("🚀 美股技術分析程式啟動")
        print(f"⏰ 開始時間: {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}")
        print("🔍 分析條件: 成交量≥1000張/日")
        print("🏆 目標: 找出前十名優質股票")
        print("=" * 60)

        # 初始化分析器
        analyzer = StockAnalyzer()

        # 執行全部股票分析
        print("📊 開始分析所有美股（包含成交量篩選）...")
        results = await analyzer.analyze_all_stocks()

        if not results:
            error_message = "❌ 分析失敗，未能獲取任何結果"
            print(error_message)

            # 發送錯誤通知
            async with aiohttp.ClientSession() as session:
                await send_notification(session, error_message)
            return

        # 在終端顯示結果
        display_terminal_results(results, limit=10)

        # 格式化通知訊息
        print("\n📱 正在準備發送通知...")
        message = format_analysis_message(results, limit=10)

        # 發送通知
        print("📱 正在發送通知到 Telegram 和 Discord...")
        async with aiohttp.ClientSession() as session:
            await send_notification(session, message)

        print(f"\n⏰ 程式執行完成: {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}")
        print("=" * 60)
        print("✅ 分析報告已完成並發送")
        print("🏆 前十名優質股票已顯示在上方")

    except Exception as e:
        error_message = f"❌ 主程式執行錯誤: {str(e)}\n\n詳細錯誤:\n{traceback.format_exc()}"
        logger.error(error_message)
        print(error_message)

        # 嘗試發送錯誤通知
        try:
            async with aiohttp.ClientSession() as session:
                await send_notification(session, f"❌ 程式執行錯誤: {str(e)}")
        except Exception as notify_error:
            logger.error(f"發送錯誤通知失敗: {notify_error}")

if __name__ == "__main__":
    # 執行主程式

    asyncio.run(main())

上面是美股篩選

In [ ]:

import asyncio
import aiohttp
import pandas as pd
import numpy as np
import yfinance as yf
import logging
import traceback
from datetime import datetime, timezone, timedelta
from typing import Dict, List, Any, Optional
import json
import os
import time
import requests
import random
from io import StringIO
from concurrent.futures import ThreadPoolExecutor, as_completed

# 如果您在 Jupyter Notebook 中，請使用這個版本
try:
    import nest_asyncio
    nest_asyncio.apply()
except ImportError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "nest_asyncio"])
    import nest_asyncio
    nest_asyncio.apply()

# 設定日誌
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# 台北時區
taipei_tz = timezone(timedelta(hours=8))

# Telegram 設定
TELEGRAM_BOT_TOKEN = "7902318521:AAEYoDMqwfHabI7L1SRiE4z33aFay42-VGE"
TELEGRAM_CHAT_IDS = [879781796, 8113868436]

# Discord 設定 (可選)
DISCORD_WEBHOOK_URL = "https://discord.com/api/webhooks/1362715080734802102/Jma7A3VhEQrrRxIX_JW2l6rATjAZXsGXGfnJuAMqmS1QvqG_2ptg3vr_nsVnuV_PlnBl"
MESSAGING_AVAILABLE = False

try:
    from discord_webhook import DiscordWebhook
    MESSAGING_AVAILABLE = True
except ImportError:
    MESSAGING_AVAILABLE = False
    logger.info("Discord webhook 套件未安裝，將跳過 Discord 通知")

class USETFAnalyzer:
    def __init__(self):
        """初始化美股ETF分析器"""
        self.config = {
            'rsi_period': 14,
            'macd_fast': 12,
            'macd_slow': 26,
            'macd_signal': 9,
            'bb_period': 20,
            'bb_std': 2,
            'kd_period': 14,
            'volume_ma_period': 20,
            'min_volume_lots': 1000,  # 最小成交量（張）
            'trend_weights': {
                'price_trend': 0.3,
                'volume_trend': 0.2,
                'technical_score': 0.5
            }
        }
        self.us_etfs = None
        self.etf_list_path = "us_etfs_cache.csv"

    def _get_comprehensive_us_etf_list(self) -> pd.DataFrame:
        """獲取完整美股ETF清單 - 包含各類型ETF和期貨"""
        try:
            logger.info("建立完整美股ETF/期貨/貴金屬清單...")

            # 完整美股ETF和期貨列表
            us_etfs = [
                # 大盤指數ETF
                {'etf_id': 'SPY', 'etf_name': 'SPDR S&P 500 ETF Trust', 'market': 'NYSE Arca', 'yahoo_symbol': 'SPY', 'category': '大盤指數ETF'},
                {'etf_id': 'QQQ', 'etf_name': 'Invesco QQQ Trust', 'market': 'NASDAQ', 'yahoo_symbol': 'QQQ', 'category': '科技指數ETF'},
                {'etf_id': 'IWM', 'etf_name': 'iShares Russell 2000 ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'IWM', 'category': '小型股ETF'},
                {'etf_id': 'VTI', 'etf_name': 'Vanguard Total Stock Market ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'VTI', 'category': '全市場ETF'},
                {'etf_id': 'VOO', 'etf_name': 'Vanguard S&P 500 ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'VOO', 'category': '大盤指數ETF'},
                {'etf_id': 'DIA', 'etf_name': 'SPDR Dow Jones Industrial Average ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'DIA', 'category': '道瓊指數ETF'},
                {'etf_id': 'VEA', 'etf_name': 'Vanguard FTSE Developed Markets ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'VEA', 'category': '已開發市場ETF'},
                {'etf_id': 'VWO', 'etf_name': 'Vanguard FTSE Emerging Markets ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'VWO', 'category': '新興市場ETF'},

                # 槓桿ETF
                {'etf_id': 'TQQQ', 'etf_name': 'ProShares UltraPro QQQ', 'market': 'NASDAQ', 'yahoo_symbol': 'TQQQ', 'category': '3倍槓桿ETF'},
                {'etf_id': 'UPRO', 'etf_name': 'ProShares UltraPro S&P500', 'market': 'NYSE Arca', 'yahoo_symbol': 'UPRO', 'category': '3倍槓桿ETF'},
                {'etf_id': 'SPXL', 'etf_name': 'Direxion Daily S&P 500 Bull 3X Shares', 'market': 'NYSE Arca', 'yahoo_symbol': 'SPXL', 'category': '3倍槓桿ETF'},
                {'etf_id': 'TNA', 'etf_name': 'Direxion Daily Small Cap Bull 3X Shares', 'market': 'NYSE Arca', 'yahoo_symbol': 'TNA', 'category': '3倍槓桿ETF'},
                {'etf_id': 'TECL', 'etf_name': 'Direxion Daily Technology Bull 3X Shares', 'market': 'NYSE Arca', 'yahoo_symbol': 'TECL', 'category': '3倍槓桿ETF'},
                {'etf_id': 'SOXL', 'etf_name': 'Direxion Daily Semiconductor Bull 3X Shares', 'market': 'NYSE Arca', 'yahoo_symbol': 'SOXL', 'category': '3倍槓桿ETF'},
                {'etf_id': 'LABU', 'etf_name': 'Direxion Daily S&P Biotech Bull 3X Shares', 'market': 'NYSE Arca', 'yahoo_symbol': 'LABU', 'category': '3倍槓桿ETF'},
                {'etf_id': 'FNGU', 'etf_name': 'MicroSectors FANG+ Index 3X Leveraged ETN', 'market': 'NYSE Arca', 'yahoo_symbol': 'FNGU', 'category': '3倍槓桿ETF'},

                # 反向ETF
                {'etf_id': 'SQQQ', 'etf_name': 'ProShares UltraPro Short QQQ', 'market': 'NASDAQ', 'yahoo_symbol': 'SQQQ', 'category': '3倍反向ETF'},
                {'etf_id': 'SPXS', 'etf_name': 'Direxion Daily S&P 500 Bear 3X Shares', 'market': 'NYSE Arca', 'yahoo_symbol': 'SPXS', 'category': '3倍反向ETF'},
                {'etf_id': 'TZA', 'etf_name': 'Direxion Daily Small Cap Bear 3X Shares', 'market': 'NYSE Arca', 'yahoo_symbol': 'TZA', 'category': '3倍反向ETF'},
                {'etf_id': 'SOXS', 'etf_name': 'Direxion Daily Semiconductor Bear 3X Shares', 'market': 'NYSE Arca', 'yahoo_symbol': 'SOXS', 'category': '3倍反向ETF'},
                {'etf_id': 'LABD', 'etf_name': 'Direxion Daily S&P Biotech Bear 3X Shares', 'market': 'NYSE Arca', 'yahoo_symbol': 'LABD', 'category': '3倍反向ETF'},
                {'etf_id': 'FNGD', 'etf_name': 'MicroSectors FANG+ Index -3X Inverse Leveraged ETN', 'market': 'NYSE Arca', 'yahoo_symbol': 'FNGD', 'category': '3倍反向ETF'},

                # 波動率ETF
                {'etf_id': 'UVXY', 'etf_name': 'ProShares Ultra VIX Short-Term Futures ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'UVXY', 'category': '波動率ETF'},
                {'etf_id': 'VXX', 'etf_name': 'iPath Series B S&P 500 VIX Short-Term Futures ETN', 'market': 'NYSE Arca', 'yahoo_symbol': 'VXX', 'category': '波動率ETF'},
                {'etf_id': 'SVXY', 'etf_name': 'ProShares Short VIX Short-Term Futures ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'SVXY', 'category': '反向波動率ETF'},
                {'etf_id': 'VIXY', 'etf_name': 'ProShares VIX Short-Term Futures ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'VIXY', 'category': '波動率ETF'},

                # 債券ETF
                {'etf_id': 'TLT', 'etf_name': 'iShares 20+ Year Treasury Bond ETF', 'market': 'NASDAQ', 'yahoo_symbol': 'TLT', 'category': '長期國債ETF'},
                {'etf_id': 'IEF', 'etf_name': 'iShares 7-10 Year Treasury Bond ETF', 'market': 'NASDAQ', 'yahoo_symbol': 'IEF', 'category': '中期國債ETF'},
                {'etf_id': 'SHY', 'etf_name': 'iShares 1-3 Year Treasury Bond ETF', 'market': 'NASDAQ', 'yahoo_symbol': 'SHY', 'category': '短期國債ETF'},
                {'etf_id': 'HYG', 'etf_name': 'iShares iBoxx $ High Yield Corporate Bond ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'HYG', 'category': '高收益債券ETF'},
                {'etf_id': 'LQD', 'etf_name': 'iShares iBoxx $ Investment Grade Corporate Bond ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'LQD', 'category': '投資級債券ETF'},
                {'etf_id': 'AGG', 'etf_name': 'iShares Core U.S. Aggregate Bond ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'AGG', 'category': '綜合債券ETF'},
                {'etf_id': 'TMF', 'etf_name': 'Direxion Daily 20+ Year Treasury Bull 3X Shares', 'market': 'NYSE Arca', 'yahoo_symbol': 'TMF', 'category': '3倍槓桿債券ETF'},
                {'etf_id': 'TMV', 'etf_name': 'Direxion Daily 20+ Year Treasury Bear 3X Shares', 'market': 'NYSE Arca', 'yahoo_symbol': 'TMV', 'category': '3倍反向債券ETF'},

                # 貴金屬ETF
                {'etf_id': 'GLD', 'etf_name': 'SPDR Gold Trust', 'market': 'NYSE Arca', 'yahoo_symbol': 'GLD', 'category': '黃金ETF'},
                {'etf_id': 'SLV', 'etf_name': 'iShares Silver Trust', 'market': 'NYSE Arca', 'yahoo_symbol': 'SLV', 'category': '白銀ETF'},
                {'etf_id': 'PPLT', 'etf_name': 'abrdn Physical Platinum Shares ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'PPLT', 'category': '鉑金ETF'},
                {'etf_id': 'PALL', 'etf_name': 'abrdn Physical Palladium Shares ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'PALL', 'category': '鈀金ETF'},
                {'etf_id': 'IAU', 'etf_name': 'iShares Gold Trust', 'market': 'NYSE Arca', 'yahoo_symbol': 'IAU', 'category': '黃金ETF'},
                {'etf_id': 'UGLD', 'etf_name': 'VelocityShares 3x Long Gold ETN', 'market': 'NYSE Arca', 'yahoo_symbol': 'UGLD', 'category': '3倍槓桿黃金ETF'},
                {'etf_id': 'DGLD', 'etf_name': 'VelocityShares 3x Inverse Gold ETN', 'market': 'NYSE Arca', 'yahoo_symbol': 'DGLD', 'category': '3倍反向黃金ETF'},

                # 貴金屬期貨
                {'etf_id': 'GC=F', 'etf_name': 'Gold Futures', 'market': 'COMEX', 'yahoo_symbol': 'GC=F', 'category': '黃金期貨'},
                {'etf_id': 'SI=F', 'etf_name': 'Silver Futures', 'market': 'COMEX', 'yahoo_symbol': 'SI=F', 'category': '白銀期貨'},
                {'etf_id': 'PL=F', 'etf_name': 'Platinum Futures', 'market': 'NYMEX', 'yahoo_symbol': 'PL=F', 'category': '鉑金期貨'},
                {'etf_id': 'PA=F', 'etf_name': 'Palladium Futures', 'market': 'NYMEX', 'yahoo_symbol': 'PA=F', 'category': '鈀金期貨'},

                # 能源期貨
                {'etf_id': 'CL=F', 'etf_name': 'Crude Oil Futures', 'market': 'NYMEX', 'yahoo_symbol': 'CL=F', 'category': '原油期貨'},
                {'etf_id': 'NG=F', 'etf_name': 'Natural Gas Futures', 'market': 'NYMEX', 'yahoo_symbol': 'NG=F', 'category': '天然氣期貨'},
                {'etf_id': 'RB=F', 'etf_name': 'Gasoline Futures', 'market': 'NYMEX', 'yahoo_symbol': 'RB=F', 'category': '汽油期貨'},
                {'etf_id': 'HO=F', 'etf_name': 'Heating Oil Futures', 'market': 'NYMEX', 'yahoo_symbol': 'HO=F', 'category': '燃油期貨'},

                # 原物料期貨ETF
                {'etf_id': 'USO', 'etf_name': 'United States Oil Fund', 'market': 'NYSE Arca', 'yahoo_symbol': 'USO', 'category': '原油期貨ETF'},
                {'etf_id': 'UNG', 'etf_name': 'United States Natural Gas Fund', 'market': 'NYSE Arca', 'yahoo_symbol': 'UNG', 'category': '天然氣期貨ETF'},
                {'etf_id': 'DBA', 'etf_name': 'Invesco DB Agriculture Fund', 'market': 'NYSE Arca', 'yahoo_symbol': 'DBA', 'category': '農產品期貨ETF'},
                {'etf_id': 'DBC', 'etf_name': 'Invesco DB Commodity Index Tracking Fund', 'market': 'NYSE Arca', 'yahoo_symbol': 'DBC', 'category': '商品期貨ETF'},
                {'etf_id': 'DBB', 'etf_name': 'Invesco DB Base Metals Fund', 'market': 'NYSE Arca', 'yahoo_symbol': 'DBB', 'category': '基本金屬期貨ETF'},
                {'etf_id': 'CORN', 'etf_name': 'Teucrium Corn Fund', 'market': 'NYSE Arca', 'yahoo_symbol': 'CORN', 'category': '玉米期貨ETF'},
                {'etf_id': 'WEAT', 'etf_name': 'Teucrium Wheat Fund', 'market': 'NYSE Arca', 'yahoo_symbol': 'WEAT', 'category': '小麥期貨ETF'},
                {'etf_id': 'SOYB', 'etf_name': 'Teucrium Soybean Fund', 'market': 'NYSE Arca', 'yahoo_symbol': 'SOYB', 'category': '大豆期貨ETF'},
                {'etf_id': 'UCO', 'etf_name': 'ProShares Ultra Bloomberg Crude Oil', 'market': 'NYSE Arca', 'yahoo_symbol': 'UCO', 'category': '2倍槓桿原油ETF'},
                {'etf_id': 'SCO', 'etf_name': 'ProShares UltraShort Bloomberg Crude Oil', 'market': 'NYSE Arca', 'yahoo_symbol': 'SCO', 'category': '2倍反向原油ETF'},

                # 農產品期貨
                {'etf_id': 'ZC=F', 'etf_name': 'Corn Futures', 'market': 'CBOT', 'yahoo_symbol': 'ZC=F', 'category': '玉米期貨'},
                {'etf_id': 'ZS=F', 'etf_name': 'Soybean Futures', 'market': 'CBOT', 'yahoo_symbol': 'ZS=F', 'category': '大豆期貨'},
                {'etf_id': 'ZW=F', 'etf_name': 'Wheat Futures', 'market': 'CBOT', 'yahoo_symbol': 'ZW=F', 'category': '小麥期貨'},
                {'etf_id': 'CT=F', 'etf_name': 'Cotton Futures', 'market': 'ICE', 'yahoo_symbol': 'CT=F', 'category': '棉花期貨'},
                {'etf_id': 'SB=F', 'etf_name': 'Sugar Futures', 'market': 'ICE', 'yahoo_symbol': 'SB=F', 'category': '糖期貨'},
                {'etf_id': 'KC=F', 'etf_name': 'Coffee Futures', 'market': 'ICE', 'yahoo_symbol': 'KC=F', 'category': '咖啡期貨'},
                {'etf_id': 'CC=F', 'etf_name': 'Cocoa Futures', 'market': 'ICE', 'yahoo_symbol': 'CC=F', 'category': '可可期貨'},

                # 工業金屬期貨
                {'etf_id': 'HG=F', 'etf_name': 'Copper Futures', 'market': 'COMEX', 'yahoo_symbol': 'HG=F', 'category': '銅期貨'},

                # 產業類ETF
                {'etf_id': 'XLF', 'etf_name': 'Financial Select Sector SPDR Fund', 'market': 'NYSE Arca', 'yahoo_symbol': 'XLF', 'category': '金融業ETF'},
                {'etf_id': 'XLE', 'etf_name': 'Energy Select Sector SPDR Fund', 'market': 'NYSE Arca', 'yahoo_symbol': 'XLE', 'category': '能源業ETF'},
                {'etf_id': 'XLK', 'etf_name': 'Technology Select Sector SPDR Fund', 'market': 'NYSE Arca', 'yahoo_symbol': 'XLK', 'category': '科技業ETF'},
                {'etf_id': 'XLV', 'etf_name': 'Health Care Select Sector SPDR Fund', 'market': 'NYSE Arca', 'yahoo_symbol': 'XLV', 'category': '醫療保健ETF'},
                {'etf_id': 'XLI', 'etf_name': 'Industrial Select Sector SPDR Fund', 'market': 'NYSE Arca', 'yahoo_symbol': 'XLI', 'category': '工業ETF'},
                {'etf_id': 'XLY', 'etf_name': 'Consumer Discretionary Select Sector SPDR Fund', 'market': 'NYSE Arca', 'yahoo_symbol': 'XLY', 'category': '非必需消費ETF'},
                {'etf_id': 'XLP', 'etf_name': 'Consumer Staples Select Sector SPDR Fund', 'market': 'NYSE Arca', 'yahoo_symbol': 'XLP', 'category': '必需消費ETF'},
                {'etf_id': 'XLU', 'etf_name': 'Utilities Select Sector SPDR Fund', 'market': 'NYSE Arca', 'yahoo_symbol': 'XLU', 'category': '公用事業ETF'},
                {'etf_id': 'XLB', 'etf_name': 'Materials Select Sector SPDR Fund', 'market': 'NYSE Arca', 'yahoo_symbol': 'XLB', 'category': '原材料ETF'},
                {'etf_id': 'XLRE', 'etf_name': 'Real Estate Select Sector SPDR Fund', 'market': 'NYSE Arca', 'yahoo_symbol': 'XLRE', 'category': '房地產ETF'},
                {'etf_id': 'SOXX', 'etf_name': 'iShares Semiconductor ETF', 'market': 'NASDAQ', 'yahoo_symbol': 'SOXX', 'category': '半導體ETF'},

                # 國際市場ETF
                {'etf_id': 'EEM', 'etf_name': 'iShares MSCI Emerging Markets ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'EEM', 'category': '新興市場ETF'},
                {'etf_id': 'EFA', 'etf_name': 'iShares MSCI EAFE ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'EFA', 'category': '歐澳遠東ETF'},
                {'etf_id': 'FXI', 'etf_name': 'iShares China Large-Cap ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'FXI', 'category': '中國大型股ETF'},
                {'etf_id': 'EWJ', 'etf_name': 'iShares MSCI Japan ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'EWJ', 'category': '日本ETF'},
                {'etf_id': 'EWZ', 'etf_name': 'iShares MSCI Brazil ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'EWZ', 'category': '巴西ETF'},
                {'etf_id': 'INDA', 'etf_name': 'iShares MSCI India ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'INDA', 'category': '印度ETF'},
                {'etf_id': 'RSX', 'etf_name': 'VanEck Russia ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'RSX', 'category': '俄羅斯ETF'},

                # 房地產ETF
                {'etf_id': 'VNQ', 'etf_name': 'Vanguard Real Estate ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'VNQ', 'category': '房地產ETF'},
                {'etf_id': 'IYR', 'etf_name': 'iShares U.S. Real Estate ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'IYR', 'category': '房地產ETF'},

                # 加密貨幣相關ETF
                {'etf_id': 'BITO', 'etf_name': 'ProShares Bitcoin Strategy ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'BITO', 'category': '比特幣期貨ETF'},
                {'etf_id': 'BITI', 'etf_name': 'ProShares Short Bitcoin Strategy ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'BITI', 'category': '反向比特幣期貨ETF'},

                # 創新科技ETF
                {'etf_id': 'ARKK', 'etf_name': 'ARK Innovation ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'ARKK', 'category': '創新科技ETF'},
                {'etf_id': 'ARKQ', 'etf_name': 'ARK Autonomous Technology & Robotics ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'ARKQ', 'category': '自動化科技ETF'},
                {'etf_id': 'ARKW', 'etf_name': 'ARK Next Generation Internet ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'ARKW', 'category': '網路科技ETF'},
                {'etf_id': 'ARKG', 'etf_name': 'ARK Genomic Revolution ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'ARKG', 'category': '基因科技ETF'},

                # 波動率指數
                {'etf_id': 'VIX', 'etf_name': 'CBOE Volatility Index', 'market': 'CBOE', 'yahoo_symbol': '^VIX', 'category': '波動率指數'},

                # 外匯ETF
                {'etf_id': 'UUP', 'etf_name': 'Invesco DB US Dollar Index Bullish Fund', 'market': 'NYSE Arca', 'yahoo_symbol': 'UUP', 'category': '美元指數ETF'},
                {'etf_id': 'UDN', 'etf_name': 'Invesco DB US Dollar Index Bearish Fund', 'market': 'NYSE Arca', 'yahoo_symbol': 'UDN', 'category': '反向美元指數ETF'},
                {'etf_id': 'FXE', 'etf_name': 'Invesco CurrencyShares Euro Trust', 'market': 'NYSE Arca', 'yahoo_symbol': 'FXE', 'category': '歐元ETF'},
                {'etf_id': 'FXY', 'etf_name': 'Invesco CurrencyShares Japanese Yen Trust', 'market': 'NYSE Arca', 'yahoo_symbol': 'FXY', 'category': '日圓ETF'},

                # 其他熱門ETF
                {'etf_id': 'JETS', 'etf_name': 'U.S. Global Jets ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'JETS', 'category': '航空業ETF'},
                {'etf_id': 'ICLN', 'etf_name': 'iShares Global Clean Energy ETF', 'market': 'NASDAQ', 'yahoo_symbol': 'ICLN', 'category': '清潔能源ETF'},
                                {'etf_id': 'BOTZ', 'etf_name': 'Global X Robotics & Artificial Intelligence ETF', 'market': 'NASDAQ', 'yahoo_symbol': 'BOTZ', 'category': '機器人AI ETF'},
                {'etf_id': 'CLOU', 'etf_name': 'Global X Cloud Computing ETF', 'market': 'NASDAQ', 'yahoo_symbol': 'CLOU', 'category': '雲端運算ETF'},
                {'etf_id': 'ESPO', 'etf_name': 'VanEck Video Gaming and eSports ETF', 'market': 'NASDAQ', 'yahoo_symbol': 'ESPO', 'category': '電競遊戲ETF'},
                {'etf_id': 'LIT', 'etf_name': 'Global X Lithium & Battery Tech ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'LIT', 'category': '鋰電池ETF'},
                {'etf_id': 'KWEB', 'etf_name': 'KraneShares CSI China Internet ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'KWEB', 'category': '中國網路ETF'},
                {'etf_id': 'MCHI', 'etf_name': 'iShares MSCI China ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'MCHI', 'category': '中國ETF'},
                {'etf_id': 'GDX', 'etf_name': 'VanEck Gold Miners ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'GDX', 'category': '黃金礦業ETF'},
                {'etf_id': 'GDXJ', 'etf_name': 'VanEck Junior Gold Miners ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'GDXJ', 'category': '小型黃金礦業ETF'},
                {'etf_id': 'SIL', 'etf_name': 'Global X Silver Miners ETF', 'market': 'NYSE Arca', 'yahoo_symbol': 'SIL', 'category': '白銀礦業ETF'},
            ]

            # 驗證每個標的是否可以從 yfinance 獲取數據
            validated_etfs = self._validate_symbols_with_yfinance(us_etfs)

            df = pd.DataFrame(validated_etfs)
            logger.info(f"完整美股ETF清單包含 {len(df)} 支已驗證的ETF/期貨/貴金屬")
            return df

        except Exception as e:
            logger.error(f"生成美股ETF清單時發生錯誤: {e}")
            return pd.DataFrame()

    def _validate_symbols_with_yfinance(self, etf_list: List[Dict]) -> List[Dict]:
        """使用 yfinance 驗證標的是否有效"""
        logger.info("開始驗證標的有效性...")
        validated_etfs = []

        def validate_single_symbol(etf_info):
            """驗證單一標的"""
            try:
                symbol = etf_info['yahoo_symbol']
                ticker = yf.Ticker(symbol)

                # 嘗試獲取最近5天的數據來驗證標的是否有效
                hist = ticker.history(period="5d")

                if not hist.empty and len(hist) > 0:
                    # 獲取基本信息
                    info = ticker.info

                    # 更新標的名稱（如果 yfinance 提供了更準確的名稱）
                    if 'longName' in info and info['longName']:
                        etf_info['etf_name'] = info['longName']
                    elif 'shortName' in info and info['shortName']:
                        etf_info['etf_name'] = info['shortName']

                    # 添加額外信息
                    etf_info['last_price'] = float(hist['Close'].iloc[-1]) if not hist['Close'].empty else None
                    etf_info['volume'] = int(hist['Volume'].iloc[-1]) if not hist['Volume'].empty else None
                    etf_info['validated'] = True
                    etf_info['validation_date'] = pd.Timestamp.now().strftime('%Y-%m-%d')

                    logger.debug(f"✓ {symbol} 驗證成功")
                    return etf_info
                else:
                    logger.warning(f"✗ {symbol} 無數據")
                    return None

            except Exception as e:
                logger.warning(f"✗ {etf_info['yahoo_symbol']} 驗證失敗: {e}")
                return None

        # 使用多線程加速驗證過程
        with ThreadPoolExecutor(max_workers=10) as executor:
            future_to_etf = {executor.submit(validate_single_symbol, etf): etf for etf in etf_list}

            for future in as_completed(future_to_etf):
                result = future.result()
                if result is not None:
                    validated_etfs.append(result)

        logger.info(f"驗證完成: {len(validated_etfs)}/{len(etf_list)} 個標的有效")
        return validated_etfs

    def get_us_etfs(self, force_update=False):
        """獲取美股ETF清單 - 使用真實數據"""
        if not force_update and os.path.exists(self.etf_list_path):
            if (time.time() - os.path.getmtime(self.etf_list_path)) < 86400:  # 24小時快取
                logger.info("從快取載入美股ETF列表")
                return pd.read_csv(self.etf_list_path, dtype={'etf_id': str})

        logger.info("獲取美股ETF清單 - 使用 yfinance 真實數據驗證...")

        try:
            # 獲取並驗證ETF清單
            df = self._get_comprehensive_us_etf_list()

            if not df.empty:
                # 保存到快取
                df.to_csv(self.etf_list_path, index=False)
                logger.info(f"成功獲取並驗證 {len(df)} 支美股ETF清單並保存快取")

                # 顯示統計信息
                category_counts = df['category'].value_counts()
                logger.info("標的類別統計:")
                for category, count in category_counts.items():
                    logger.info(f"  {category}: {count} 支")

            return df

        except Exception as e:
            logger.error(f"獲取美股ETF清單時發生錯誤: {e}")
            # 如果驗證失敗，返回基本清單
            return pd.DataFrame([
                {'etf_id': 'SPY', 'etf_name': 'SPDR S&P 500 ETF Trust', 'market': 'NYSE Arca', 'yahoo_symbol': 'SPY', 'category': '大盤指數ETF'},
                {'etf_id': 'QQQ', 'etf_name': 'Invesco QQQ Trust', 'market': 'NASDAQ', 'yahoo_symbol': 'QQQ', 'category': '科技指數ETF'},
                {'etf_id': 'GLD', 'etf_name': 'SPDR Gold Trust', 'market': 'NYSE Arca', 'yahoo_symbol': 'GLD', 'category': '黃金ETF'},
            ])

    def check_volume_filter(self, df: pd.DataFrame) -> bool:
        """檢查成交量是否符合篩選條件（每日至少1000張）"""
        try:
            if df.empty or 'Volume' not in df.columns:
                return False

            # 取最近20天的平均成交量
            recent_volume = df['Volume'].tail(20).mean()

            # 轉換為張數（1張 = 1000股）
            volume_lots = recent_volume / 1000

            # 檢查是否符合最小成交量要求
            meets_volume = volume_lots >= self.config['min_volume_lots']

            if meets_volume:
                logger.debug(f"成交量符合條件: {volume_lots:.0f} 張/日 (≥ {self.config['min_volume_lots']} 張)")
            else:
                logger.debug(f"成交量不符合條件: {volume_lots:.0f} 張/日 (< {self.config['min_volume_lots']} 張)")

            return meets_volume

        except Exception as e:
            logger.error(f"檢查成交量篩選時發生錯誤: {e}")
            return False

    def fetch_yfinance_data(self, etf_id: str, period: str = "1y", interval: str = "1d", retries: int = 3):
        """獲取ETF數據（同步版本）"""
        for attempt in range(retries):
            try:
                df = yf.download(tickers=etf_id, period=period, interval=interval, progress=False, auto_adjust=True)
                if df.empty:
                    logger.warning(f"[{etf_id}] 無數據返回")
                    return pd.DataFrame()

                if isinstance(df.columns, pd.MultiIndex):
                    df.columns = df.columns.get_level_values(0)

                required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
                for col in required_cols:
                    if col in df.columns:
                        df[col] = pd.to_numeric(df[col], errors='coerce')

                df = df.dropna(subset=required_cols)
                return df.reset_index()

            except Exception as e:
                if attempt < retries - 1:
                    sleep_time = 0.5 * (2 ** attempt) + random.uniform(0, 1)
                    logger.warning(f"[{etf_id}] 第 {attempt + 1}/{retries} 次嘗試失敗: {e}。等待 {sleep_time:.2f} 秒後重試...")
                    time.sleep(sleep_time)
                else:
                    logger.error(f"[{etf_id}] 獲取數據失敗")
                    return pd.DataFrame()
        return pd.DataFrame()

    def calculate_technical_indicators(self, df: pd.DataFrame) -> pd.DataFrame:
        """計算技術指標"""
        try:
            if df.empty:
                return df

            df = df.copy()

            # RSI
            df['RSI'] = self.calculate_rsi(df['Close'], self.config['rsi_period'])

            # MACD
            macd_line, macd_signal, macd_histogram = self.calculate_macd(
                df['Close'],
                self.config['macd_fast'],
                self.config['macd_slow'],
                self.config['macd_signal']
            )
            df['MACD'] = macd_line
            df['MACD_Signal'] = macd_signal
            df['MACD_Histogram'] = macd_histogram

            # 布林帶
            bb_upper, bb_middle, bb_lower = self.calculate_bollinger_bands(
                df['Close'],
                self.config['bb_period'],
                self.config['bb_std']
            )
            df['BB_Upper'] = bb_upper
            df['BB_Middle'] = bb_middle
            df['BB_Lower'] = bb_lower

            # KD指標
            k_percent, d_percent = self.calculate_kd(
                df['High'],
                df['Low'],
                df['Close'],
                self.config['kd_period']
            )
            df['K_Percent'] = k_percent
            df['D_Percent'] = d_percent

            # 移動平均線
            df['MA5'] = df['Close'].rolling(window=5).mean()
            df['MA10'] = df['Close'].rolling(window=10).mean()
            df['MA20'] = df['Close'].rolling(window=20).mean()
            df['MA60'] = df['Close'].rolling(window=60).mean()

            # 成交量移動平均
            if 'Volume' in df.columns:
                df['Volume_MA'] = df['Volume'].rolling(window=self.config['volume_ma_period']).mean()

            return df

        except Exception as e:
            logger.error(f"計算技術指標時發生錯誤: {e}")
            return df

    def calculate_rsi(self, prices: pd.Series, period: int = 14) -> pd.Series:
        """計算RSI"""
        try:
            delta = prices.diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
            rs = gain / loss
            rsi = 100 - (100 / (1 + rs))
            return rsi
        except Exception as e:
            logger.error(f"計算RSI時發生錯誤: {e}")
            return pd.Series(index=prices.index, data=50)

    def calculate_macd(self, prices: pd.Series, fast: int = 12, slow: int = 26, signal: int = 9):
        """計算MACD"""
        try:
            ema_fast = prices.ewm(span=fast).mean()
            ema_slow = prices.ewm(span=slow).mean()
            macd_line = ema_fast - ema_slow
            macd_signal = macd_line.ewm(span=signal).mean()
            macd_histogram = macd_line - macd_signal
            return macd_line, macd_signal, macd_histogram
        except Exception as e:
            logger.error(f"計算MACD時發生錯誤: {e}")
            zero_series = pd.Series(index=prices.index, data=0)
            return zero_series, zero_series, zero_series

    def calculate_bollinger_bands(self, prices: pd.Series, period: int = 20, std_dev: int = 2):
        """計算布林帶"""
        try:
            middle = prices.rolling(window=period).mean()
            std = prices.rolling(window=period).std()
            upper = middle + (std * std_dev)
            lower = middle - (std * std_dev)
            return upper, middle, lower
        except Exception as e:
            logger.error(f"計算布林帶時發生錯誤: {e}")
            zero_series = pd.Series(index=prices.index, data=0)
            return zero_series, zero_series, zero_series

    def calculate_kd(self, high: pd.Series, low: pd.Series, close: pd.Series, period: int = 14):
        """計算KD指標"""
        try:
            lowest_low = low.rolling(window=period).min()
            highest_high = high.rolling(window=period).max()

            k_percent = 100 * ((close - lowest_low) / (highest_high - lowest_low))
            k_percent = k_percent.rolling(window=3).mean()
            d_percent = k_percent.rolling(window=3).mean()

            return k_percent, d_percent
        except Exception as e:
            logger.error(f"計算KD時發生錯誤: {e}")
            fifty_series = pd.Series(index=high.index, data=50)
            return fifty_series, fifty_series

    def analyze_trend(self, df: pd.DataFrame) -> Dict[str, Any]:
        """趨勢分析"""
        try:
            if df.empty or len(df) < 20:
                return {'trend': '盤整', 'strength': 0, 'price_change_5d': 0, 'price_change_20d': 0}

            current_price = float(df['Close'].iloc[-1])
            price_5d_ago = float(df['Close'].iloc[-6]) if len(df) >= 6 else current_price
            price_20d_ago = float(df['Close'].iloc[-21]) if len(df) >= 21 else current_price

            change_5d = ((current_price - price_5d_ago) / price_5d_ago * 100) if price_5d_ago != 0 else 0
            change_20d = ((current_price - price_20d_ago) / price_20d_ago * 100) if price_20d_ago != 0 else 0

            # 移動平均線趨勢
            ma5_current = float(df['MA5'].iloc[-1]) if 'MA5' in df.columns and not pd.isna(df['MA5'].iloc[-1]) else current_price
            ma20_current = float(df['MA20'].iloc[-1]) if 'MA20' in df.columns and not pd.isna(df['MA20'].iloc[-1]) else current_price

            # 判斷趨勢
            if change_5d > 3 and change_20d > 5 and current_price > ma5_current > ma20_current:
                trend = '強勢上漲'
                strength = 3
            elif change_5d > 1 and change_20d > 2 and current_price > ma5_current:
                trend = '上漲'
                strength = 2
            elif change_5d < -3 and change_20d < -5 and current_price < ma5_current < ma20_current:
                trend = '強勢下跌'
                strength = -3
            elif change_5d < -1 and change_20d < -2 and current_price < ma5_current:
                trend = '下跌'
                strength = -2
            else:
                trend = '盤整'
                strength = 0

            return {
                'trend': trend,
                'strength': strength,
                'price_change_5d': change_5d,
                'price_change_20d': change_20d,
                'ma5_position': ma5_current,
                'ma20_position': ma20_current
            }

        except Exception as e:
            logger.error(f"趨勢分析時發生錯誤: {e}")
            return {'trend': '盤整', 'strength': 0, 'price_change_5d': 0, 'price_change_20d': 0}

    def analyze_volume(self, df: pd.DataFrame) -> Dict[str, Any]:
        """成交量分析"""
        try:
            if df.empty or 'Volume' not in df.columns or len(df) < 20:
                return {'volume_trend': '正常', 'volume_ratio': 1.0, 'volume_signal': '中性', 'avg_volume_lots': 0}

            current_volume = float(df['Volume'].iloc[-1])
            avg_volume_20d = float(df['Volume'].rolling(window=20).mean().iloc[-1])

            volume_ratio = current_volume / avg_volume_20d if avg_volume_20d > 0 else 1.0
            avg_volume_lots = avg_volume_20d / 1000  # 轉換為張數

            if volume_ratio > 2.0:
                volume_trend = '爆量'
                volume_signal = '強烈'
            elif volume_ratio > 1.5:
                volume_trend = '放量'
                volume_signal = '積極'
            elif volume_ratio < 0.5:
                volume_trend = '縮量'
                volume_signal = '消極'
            else:
                volume_trend = '正常'
                volume_signal = '中性'

            return {
                'volume_trend': volume_trend,
                'volume_ratio': volume_ratio,
                'volume_signal': volume_signal,
                'avg_volume_lots': avg_volume_lots
            }

        except Exception as e:
            logger.error(f"成交量分析時發生錯誤: {e}")
            return {'volume_trend': '正常', 'volume_ratio': 1.0, 'volume_signal': '中性', 'avg_volume_lots': 0}

    def analyze_technical_indicators(self, df: pd.DataFrame) -> Dict[str, Any]:
        """技術指標分析"""
        try:
            if df.empty:
                return {
                    'rsi_signal': '中性', 'rsi_value': 50,
                    'macd_signal': '中性', 'macd_value': 0,
                    'kd_signal': '中性', 'k_value': 50, 'd_value': 50,
                    'bb_signal': '中性', 'bb_position': 0.5,
                    'score': 50
                }

            signals = {}
            score = 50

            # RSI 分析
            if 'RSI' in df.columns and not df['RSI'].empty:
                rsi_value = float(df['RSI'].iloc[-1]) if not pd.isna(df['RSI'].iloc[-1]) else 50
                signals['rsi_value'] = rsi_value

                if rsi_value > 80:
                    signals['rsi_signal'] = '超買'
                    score -= 15
                elif rsi_value > 70:
                    signals['rsi_signal'] = '偏高'
                    score -= 5
                elif rsi_value < 20:
                    signals['rsi_signal'] = '超賣'
                    score += 15
                elif rsi_value < 30:
                    signals['rsi_signal'] = '偏低'
                    score += 5
                else:
                    signals['rsi_signal'] = '中性'
            else:
                signals['rsi_signal'] = '中性'
                signals['rsi_value'] = 50

            # MACD 分析
            if all(col in df.columns for col in ['MACD', 'MACD_Signal']) and len(df) >= 2:
                macd_current = float(df['MACD'].iloc[-1]) if not pd.isna(df['MACD'].iloc[-1]) else 0
                macd_signal_current = float(df['MACD_Signal'].iloc[-1]) if not pd.isna(df['MACD_Signal'].iloc[-1]) else 0
                macd_prev = float(df['MACD'].iloc[-2]) if not pd.isna(df['MACD'].iloc[-2]) else 0
                macd_signal_prev = float(df['MACD_Signal'].iloc[-2]) if not pd.isna(df['MACD_Signal'].iloc[-2]) else 0

                signals['macd_value'] = macd_current

                if macd_prev <= macd_signal_prev and macd_current > macd_signal_current:
                    signals['macd_signal'] = '黃金交叉'
                    score += 20
                elif macd_prev >= macd_signal_prev and macd_current < macd_signal_current:
                    signals['macd_signal'] = '死亡交叉'
                    score -= 20
                elif macd_current > macd_signal_current:
                    signals['macd_signal'] = '多頭'
                    score += 5
                elif macd_current < macd_signal_current:
                    signals['macd_signal'] = '空頭'
                    score -= 5
                else:
                    signals['macd_signal'] = '中性'
            else:
                signals['macd_signal'] = '中性'
                signals['macd_value'] = 0

            # KD 分析
            if all(col in df.columns for col in ['K_Percent', 'D_Percent']):
                k_value = float(df['K_Percent'].iloc[-1]) if not pd.isna(df['K_Percent'].iloc[-1]) else 50
                d_value = float(df['D_Percent'].iloc[-1]) if not pd.isna(df['D_Percent'].iloc[-1]) else 50

                signals['k_value'] = k_value
                signals['d_value'] = d_value

                if k_value > 80 and d_value > 80:
                    signals['kd_signal'] = '超買'
                    score -= 10
                elif k_value < 20 and d_value < 20:
                    signals['kd_signal'] = '超賣'
                    score += 10
                elif k_value > d_value:
                    signals['kd_signal'] = '偏多'
                    score += 3
                elif k_value < d_value:
                    signals['kd_signal'] = '偏空'
                    score -= 3
                else:
                    signals['kd_signal'] = '中性'
            else:
                signals['kd_signal'] = '中性'
                signals['k_value'] = 50
                signals['d_value'] = 50

            # 布林帶分析
            if all(col in df.columns for col in ['BB_Upper', 'BB_Lower', 'Close']):
                current_price = float(df['Close'].iloc[-1])
                bb_upper = float(df['BB_Upper'].iloc[-1]) if not pd.isna(df['BB_Upper'].iloc[-1]) else current_price
                bb_lower = float(df['BB_Lower'].iloc[-1]) if not pd.isna(df['BB_Lower'].iloc[-1]) else current_price

                if bb_upper != bb_lower:
                    bb_position = (current_price - bb_lower) / (bb_upper - bb_lower)
                    signals['bb_position'] = bb_position

                    if bb_position > 0.8:
                        signals['bb_signal'] = '接近上軌'
                        score -= 5
                    elif bb_position < 0.2:
                        signals['bb_signal'] = '接近下軌'
                        score += 5
                    else:
                        signals['bb_signal'] = '中性'
                else:
                    signals['bb_signal'] = '中性'
                    signals['bb_position'] = 0.5
            else:
                signals['bb_signal'] = '中性'
                signals['bb_position'] = 0.5

            score = max(0, min(100, score))
            signals['score'] = score

            return signals

        except Exception as e:
            logger.error(f"技術指標分析時發生錯誤: {e}")
            return {
                'rsi_signal': '中性', 'rsi_value': 50,
                'macd_signal': '中性', 'macd_value': 0,
                'kd_signal': '中性', 'k_value': 50, 'd_value': 50,
                'bb_signal': '中性', 'bb_position': 0.5,
                'score': 50
            }

    def calculate_combined_score(self, trend_analysis: Dict, volume_analysis: Dict, technical_analysis: Dict) -> float:
        """計算綜合分數"""
        try:
            base_score = 50

            # 趨勢分數 (權重: 30%)
            trend_score = 0
            trend_strength = trend_analysis.get('strength', 0)
            if trend_strength > 0:
                trend_score = min(30, trend_strength * 10)
            elif trend_strength < 0:
                trend_score = max(-30, trend_strength * 10)

            # 成交量分數 (權重: 20%)
            volume_score = 0
            volume_ratio = volume_analysis.get('volume_ratio', 1.0)
            if volume_ratio > 1.5:
                volume_score = 10
            elif volume_ratio > 1.2:
                volume_score = 5
            elif volume_ratio < 0.7:
                volume_score = -5

            # 技術指標分數 (權重: 50%)
            tech_score = technical_analysis.get('score', 50) - 50

            # 計算加權總分
            total_score = base_score + (trend_score * 0.3) + (volume_score * 0.2) + (tech_score * 0.5)

            return max(0, min(100, total_score))

        except Exception as e:
            logger.error(f"計算綜合分數時發生錯誤: {e}")
            return 50.0

    def generate_recommendation(self, combined_score: float, trend_analysis: Dict, technical_analysis: Dict) -> Dict[str, Any]:
        """生成投資建議"""
        try:
            recommendation = "持有"
            confidence = "中等"
            reasons = []

            if combined_score >= 75:
                recommendation = "強力買進"
                confidence = "高"
            elif combined_score >= 60:
                recommendation = "買進"
                confidence = "中高"
            elif combined_score >= 40:
                recommendation = "持有"
                confidence = "中等"
            elif combined_score >= 25:
                recommendation = "賣出"
                confidence = "中高"
            else:
                recommendation = "強力賣出"
                confidence = "高"

            # 生成建議原因
            trend = trend_analysis.get('trend', '盤整')
            if '上漲' in trend:
                reasons.append(f"價格趨勢：{trend}")
            elif '下跌' in trend:
                reasons.append(f"價格趨勢：{trend}")

            rsi_signal = technical_analysis.get('rsi_signal', '中性')
            if rsi_signal in ['超賣', '偏低']:
                reasons.append(f"RSI指標：{rsi_signal}，可能反彈")
            elif rsi_signal in ['超買', '偏高']:
                reasons.append(f"RSI指標：{rsi_signal}，注意回調")

            macd_signal = technical_analysis.get('macd_signal', '中性')
            if macd_signal == '黃金交叉':
                reasons.append("MACD出現黃金交叉")
            elif macd_signal == '死亡交叉':
                reasons.append("MACD出現死亡交叉")

            return {
                'recommendation': recommendation,
                'confidence': confidence,
                'score': combined_score,
                'reasons': reasons
            }

        except Exception as e:
            logger.error(f"生成投資建議時發生錯誤: {e}")
            return {
                'recommendation': '持有',
                'confidence': '低',
                'score': 50,
                'reasons': ['分析過程出現錯誤']
            }

    async def analyze_etf_async(self, session: aiohttp.ClientSession, etf_info: Dict) -> Dict[str, Any]:
        """異步分析單支ETF"""
        symbol = etf_info['yahoo_symbol']
        etf_name = etf_info['etf_name']

        try:
            logger.info(f"開始分析ETF: {etf_name} ({symbol})")

            # 獲取ETF數據
            df = self.fetch_yfinance_data(symbol)
            if df is None or df.empty or len(df) < 60:
                return {
                    'symbol': symbol,
                    'etf_name': etf_name,
                    'success': False,
                    'error': '無法獲取足夠的ETF數據'
                }

            # 檢查成交量篩選條件
            if not self.check_volume_filter(df):
                return {
                    'symbol': symbol,
                    'etf_name': etf_name,
                    'success': False,
                    'error': f'成交量不符合條件（需≥{self.config["min_volume_lots"]}張/日）'
                }

            # 計算技術指標
            df_with_indicators = self.calculate_technical_indicators(df)

            # 進行各項分析
            trend_analysis = self.analyze_trend(df_with_indicators)
            volume_analysis = self.analyze_volume(df_with_indicators)
            technical_analysis = self.analyze_technical_indicators(df_with_indicators)

            # 計算綜合分數
            combined_score = self.calculate_combined_score(trend_analysis, volume_analysis, technical_analysis)

            # 生成投資建議
            recommendation = self.generate_recommendation(combined_score, trend_analysis, technical_analysis)

            # 獲取當前價格資訊
            current_price = float(df['Close'].iloc[-1])
            price_change = trend_analysis.get('price_change_5d', 0)

            result = {
                'symbol': symbol,
                'etf_name': etf_name,
                'etf_id': etf_info.get('etf_id', ''),
                'market': etf_info.get('market', ''),
                'category': etf_info.get('category', ''),
                'success': True,
                'current_price': current_price,
                'price_change_5d': price_change,
                'combined_score': combined_score,
                'trend_analysis': trend_analysis,
                'volume_analysis': volume_analysis,
                'technical_analysis': technical_analysis,
                'recommendation': recommendation,
                'analysis_time': datetime.now(taipei_tz).isoformat()
            }

            logger.info(f"完成分析: {etf_name} - 分數: {combined_score:.1f} - 建議: {recommendation['recommendation']} - 成交量: {volume_analysis.get('avg_volume_lots', 0):.0f}張/日")
            return result

        except Exception as e:
            logger.error(f"分析ETF {symbol} 時發生錯誤: {e}")
            return {
                'symbol': symbol,
                'etf_name': etf_name,
                'success': False,
                'error': str(e)
            }

    async def analyze_all_etfs(self) -> Dict[str, Any]:
        """分析所有ETF（加入成交量篩選）"""
        try:
            # 獲取完整ETF清單
            if self.us_etfs is None:
                self.us_etfs = self.get_us_etfs()

            if self.us_etfs.empty:
                logger.error("無法獲取ETF清單")
                return {}

            logger.info(f"準備分析 {len(self.us_etfs)} 支ETF（包含成交量篩選）")

            results = {}
            failed_count = 0
            volume_filtered_count = 0

            # 使用 aiohttp 進行異步分析
            async with aiohttp.ClientSession() as session:
                # 創建分析任務
                tasks = []
                for _, etf_info in self.us_etfs.iterrows():
                    task = self.analyze_etf_async(session, etf_info.to_dict())
                    tasks.append(task)

                # 分批執行任務以避免過載
                batch_size = 10
                for i in range(0, len(tasks), batch_size):
                    batch_tasks = tasks[i:i+batch_size]
                    batch_results = await asyncio.gather(*batch_tasks, return_exceptions=True)

                    # 處理批次結果
                    for result in batch_results:
                        if isinstance(result, Exception):
                            logger.error(f"分析任務異常: {result}")
                            failed_count += 1
                            continue

                        if isinstance(result, dict) and 'symbol' in result:
                            if result.get('success', False):
                                results[result['symbol']] = result
                            else:
                                error_msg = result.get('error', '')
                                if '成交量不符合條件' in error_msg:
                                    volume_filtered_count += 1
                                else:
                                    failed_count += 1

                    # 批次間暫停避免過載
                    if i + batch_size < len(tasks):
                        await asyncio.sleep(2)

            logger.info(f"分析完成統計：")
            logger.info(f"  - 成功分析: {len(results)} 支")
            logger.info(f"  - 成交量篩選淘汰: {volume_filtered_count} 支")
            logger.info(f"  - 其他失敗: {failed_count} 支")

            return results

        except Exception as e:
            logger.error(f"批量分析ETF時發生錯誤: {e}")
            return {}

def format_analysis_message(results: Dict[str, Any], limit: int = 10) -> str:
    """格式化分析結果為訊息（顯示前十名）"""
    try:
        if not results:
            return "❌ 未能獲取任何分析結果"

        # 過濾成功的結果並按分數排序
        successful_results = [
            result for result in results.values()
            if result.get('success', False) and result.get('combined_score', 0) > 0
        ]

        successful_results.sort(key=lambda x: x.get('combined_score', 0), reverse=True)

        # 統計資訊
        total_analyzed = len(results)
        successful_count = len(successful_results)

        # 建立訊息
        message_parts = []
        message_parts.append("🏆 美股ETF技術分析 - 前十名優質標的")
        message_parts.append("=" * 40)
        message_parts.append(f"📈 分析時間: {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}")
        message_parts.append(f"📊 成功分析: {successful_count} 支ETF/期貨/貴金屬")
        message_parts.append(f"🔍 成交量篩選: ≥1000張/日")
        message_parts.append("")

        if not successful_results:
            message_parts.append("❌ 沒有符合條件的優質標的")
            return "\n".join(message_parts)

        # 顯示前十名優質標的
        top_etfs = successful_results[:limit]
        message_parts.append(f"🏆 前 {len(top_etfs)} 名優質標的:")
        message_parts.append("-" * 40)

        for i, result in enumerate(top_etfs, 1):
            etf_name = result.get('etf_name', 'Unknown')
            etf_id = result.get('etf_id', 'Unknown')
            category = result.get('category', 'Unknown')
            score = result.get('combined_score', 0)
            recommendation = result.get('recommendation', {})
            rec_text = recommendation.get('recommendation', '持有')
            current_price = result.get('current_price', 0)
            price_change = result.get('price_change_5d', 0)
            volume_info = result.get('volume_analysis', {})
            avg_volume_lots = volume_info.get('avg_volume_lots', 0)

            # 價格變化符號
            change_symbol = "📈" if price_change > 0 else "📉" if price_change < 0 else "➡️"

            # 建議符號
            rec_symbol = "🚀" if "強力買" in rec_text else "📈" if "買" in rec_text else "📊" if "持有" in rec_text else "📉"

            # 類別符號
            category_symbol = "🏛️" if "指數" in category else "⚡" if "槓桿" in category else "🔄" if "反向" in category else "🥇" if "黃金" in category else "🛢️" if "原油" in category else "🌾" if "農產品" in category else "📊"

            message_parts.append(f"{i}. {etf_name[:25]} ({etf_id})")
            message_parts.append(f"   {category_symbol} 類別: {category}")
            message_parts.append(f"   💰 價格: ${current_price:.2f} ({change_symbol}{price_change:+.2f}%)")
            message_parts.append(f"   ⭐ 評分: {score:.1f}/100")
            message_parts.append(f"   {rec_symbol} 建議: {rec_text}")
            message_parts.append(f"   📊 成交量: {avg_volume_lots:.0f}張/日")

            # 添加主要技術指標
            tech = result.get('technical_analysis', {})
            rsi_value = tech.get('rsi_value', 50)
            macd_signal = tech.get('macd_signal', '中性')
            message_parts.append(f"   🔍 RSI: {rsi_value:.1f} | MACD: {macd_signal}")
            message_parts.append("")

        # 添加統計摘要
        if successful_results:
            avg_score = sum(r.get('combined_score', 0) for r in successful_results) / len(successful_results)
            high_score_count = len([r for r in successful_results if r.get('combined_score', 0) >= 70])

            message_parts.append("📊 統計摘要:")
            message_parts.append(f"   📊 平均分數: {avg_score:.1f}")
            message_parts.append(f"   🌟 高分標的 (≥70): {high_score_count}")

            # 建議分布
            recommendations = {}
            for result in top_etfs:
                rec = result.get('recommendation', {}).get('recommendation', '持有')
                recommendations[rec] = recommendations.get(rec, 0) + 1

            if recommendations:
                message_parts.append("   📈 前十名建議分布:")
                for rec, count in recommendations.items():
                    message_parts.append(f"      {rec}: {count}")

            # 類別分布
            categories = {}
            for result in top_etfs:
                cat = result.get('category', 'Unknown')
                categories[cat] = categories.get(cat, 0) + 1

            if categories:
                message_parts.append("   🏷️ 前十名類別分布:")
                for cat, count in sorted(categories.items(), key=lambda x: x[1], reverse=True):
                    message_parts.append(f"      {cat}: {count}")

        message_parts.append("")
        message_parts.append("⚠️ 投資提醒:")
        message_parts.append("• 本分析僅供參考，投資有風險")
        message_parts.append("• 已篩選成交量≥1000張/日的活躍標的")
        message_parts.append("• 包含ETF、期貨、貴金屬等多元化標的")
        message_parts.append("• 請結合基本面分析做最終決策")
        message_parts.append("• 槓桿和反向ETF風險較高，請謹慎操作")

        return "\n".join(message_parts)

    except Exception as e:
        logger.error(f"格式化訊息時發生錯誤: {e}")
        return f"❌ 格式化分析結果時發生錯誤: {str(e)}"

def display_terminal_results(results: Dict[str, Any], limit: int = 10):
    """在終端顯示分析結果"""
    try:
        if not results:
            print("❌ 未能獲取任何分析結果")
            return

        # 過濾成功的結果並按分數排序
        successful_results = [
            result for result in results.values()
            if result.get('success', False) and result.get('combined_score', 0) > 0
        ]

        successful_results.sort(key=lambda x: x.get('combined_score', 0), reverse=True)

        print("\n" + "=" * 100)
        print("🏆 美股ETF技術分析結果 - 前十名優質標的".center(100))
        print("=" * 100)
        print(f"📈 分析時間: {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"📊 成功分析: {len(successful_results)} 支ETF/期貨/貴金屬")
        print(f"🔍 成交量篩選條件: ≥1000張/日")
        print("=" * 100)

        if not successful_results:
            print("❌ 沒有符合條件的優質標的")
            return

        # 表格標題
        print(f"{'排名':<4}{'代碼':<8}{'標的名稱':<20}{'類別':<15}{'評分':<8}{'價格':<10}{'漲跌%':<8}{'建議':<10}{'成交量(張)':<12}")
        print("-" * 100)

        # 顯示前十名
        top_etfs = successful_results[:limit]
        for i, result in enumerate(top_etfs, 1):
            etf_name = result.get('etf_name', 'Unknown')[:18]  # 限制長度
            etf_id = result.get('etf_id', 'Unknown')
            category = result.get('category', 'Unknown')[:13]  # 限制長度
            score = result.get('combined_score', 0)
            recommendation = result.get('recommendation', {})
            rec_text = recommendation.get('recommendation', '持有')[:8]  # 限制長度
            current_price = result.get('current_price', 0)
            price_change = result.get('price_change_5d', 0)
            volume_info = result.get('volume_analysis', {})
            avg_volume_lots = volume_info.get('avg_volume_lots', 0)

            print(f"{i:<4}{etf_id:<8}{etf_name:<20}{category:<15}{score:<8.1f}${current_price:<9.2f}{price_change:<+8.2f}{rec_text:<10}{avg_volume_lots:<12.0f}")

        print("=" * 100)

        # 詳細分析前五名
        print("\n📊 詳細分析報告 (前五名):")
        print("=" * 100)

        for i, result in enumerate(top_etfs[:5], 1):
            etf_name = result.get('etf_name', 'Unknown')
            etf_id = result.get('etf_id', 'Unknown')
            category = result.get('category', 'Unknown')
            score = result.get('combined_score', 0)
            recommendation = result.get('recommendation', {})
            trend_analysis = result.get('trend_analysis', {})
            technical_analysis = result.get('technical_analysis', {})
            volume_analysis = result.get('volume_analysis', {})

            print(f"\n{i}. {etf_name} ({etf_id}) - 評分: {score:.1f}")
            print(f"   🏷️ 類別: {category}")
            print("-" * 60)
            print(f"💰 當前價格: ${result.get('current_price', 0):.2f}")
            print(f"📈 5日漲跌: {result.get('price_change_5d', 0):+.2f}%")
            print(f"🎯 投資建議: {recommendation.get('recommendation', '持有')}")
            print(f"📊 信心度: {recommendation.get('confidence', '中等')}")

            # 技術指標詳情
            print(f"🔍 技術指標:")
            print(f"   RSI: {technical_analysis.get('rsi_value', 50):.1f} ({technical_analysis.get('rsi_signal', '中性')})")
            print(f"   MACD: {technical_analysis.get('macd_signal', '中性')}")
            print(f"   KD: K={technical_analysis.get('k_value', 50):.1f}, D={technical_analysis.get('d_value', 50):.1f}")

            # 趨勢和成交量
            print(f"📊 趨勢分析: {trend_analysis.get('trend', '盤整')}")
            print(f"📈 成交量: {volume_analysis.get('avg_volume_lots', 0):.0f}張/日 ({volume_analysis.get('volume_trend', '正常')})")

            # 建議原因
            reasons = recommendation.get('reasons', [])
            if reasons:
                print(f"💡 建議原因: {', '.join(reasons[:2])}")  # 顯示前兩個原因

        # 類別統計
        print("\n" + "=" * 100)
        print("📊 類別統計分析:")
        categories = {}
        for result in successful_results:
            cat = result.get('category', 'Unknown')
            if cat not in categories:
                categories[cat] = {'count': 0, 'avg_score': 0, 'scores': []}
            categories[cat]['count'] += 1
            categories[cat]['scores'].append(result.get('combined_score', 0))

        for cat, data in categories.items():
            data['avg_score'] = sum(data['scores']) / len(data['scores'])

        sorted_categories = sorted(categories.items(), key=lambda x: x[1]['avg_score'], reverse=True)

        print(f"{'類別':<20}{'數量':<8}{'平均分數':<12}{'最高分':<10}")
        print("-" * 50)
        for cat, data in sorted_categories:
            print(f"{cat:<20}{data['count']:<8}{data['avg_score']:<12.1f}{max(data['scores']):<10.1f}")

        print("\n" + "=" * 100)
        print("⚠️  投資提醒:")
        print("• 本分析僅供參考，投資一定有風險")
        print("• 已篩選成交量≥1000張/日的活躍標的")
        print("• 包含ETF、期貨、貴金屬等多元化投資工具")
        print("• 槓桿ETF和反向ETF具有較高風險，請謹慎操作")
        print("• 期貨商品波動較大，適合有經驗的投資者")
        print("• 請務必結合基本面分析做最終投資決策")
        print("• 建議分散投資，控制風險")
        print("=" * 100)

    except Exception as e:
        logger.error(f"顯示終端結果時發生錯誤: {e}")
        print(f"❌ 顯示結果時發生錯誤: {str(e)}")

async def send_notification(session: aiohttp.ClientSession, message: str, files: List[str] = None):
    """發送通知到 Telegram 和 Discord"""

    # Telegram 通知
    try:
        # 檢查 Telegram 設定
        if not TELEGRAM_BOT_TOKEN or not TELEGRAM_CHAT_IDS or TELEGRAM_BOT_TOKEN == "YOUR_BOT_TOKEN_HERE":
            logger.warning("Telegram 設定未完成，跳過發送通知")
            print("📱 Telegram 設定未完成，請設定 TELEGRAM_BOT_TOKEN 和 TELEGRAM_CHAT_IDS")
        else:
            # 分割長訊息
            max_length = 4000
            if len(message) > max_length:
                parts = []
                current_part = ""

                for line in message.split('\n'):
                    if len(current_part + line + '\n') > max_length:
                        if current_part:
                            parts.append(current_part.strip())
                            current_part = line + '\n'
                        else:
                            # 如果單行太長，直接截斷
                            parts.append(line[:max_length])
                    else:
                        current_part += line + '\n'

                if current_part:
                    parts.append(current_part.strip())
            else:
                parts = [message]

            # 發送文字訊息給所有 chat_id
            telegram_url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"

            for chat_id in TELEGRAM_CHAT_IDS:
                for i, part in enumerate(parts):
                    if i > 0:
                        part = f"🏆 美股ETF分析報告 (續 {i+1}/{len(parts)})\n\n" + part

                    payload = {
                        'chat_id': chat_id,
                        'text': part
                    }

                    try:
                        async with session.post(telegram_url, json=payload, timeout=20) as response:
                            if response.status == 200:
                                logger.info(f"成功發送 Telegram 通知到 {chat_id} (第 {i+1}/{len(parts)} 部分)")
                            else:
                                error_text = await response.text()
                                logger.error(f"發送 Telegram 通知失敗到 {chat_id} (第 {i+1} 部分): {response.status} - {error_text}")
                    except Exception as e:
                        logger.error(f"發送 Telegram 通知時發生網路錯誤到 {chat_id} (第 {i+1} 部分): {e}")

                    # 避免發送太快
                    if i < len(parts) - 1:
                        await asyncio.sleep(1)

    except Exception as e:
        logger.error(f"發送 Telegram 通知時異常: {e}")

    # Discord 通知 - 修改這部分
    if MESSAGING_AVAILABLE and DISCORD_WEBHOOK_URL and DISCORD_WEBHOOK_URL != "YOUR_DISCORD_WEBHOOK_URL":
        try:
            # Discord 限制 2000 字符，需要分割訊息
            max_discord_length = 1900  # 留一些緩衝空間

            if len(message) <= max_discord_length:
                # 訊息不長，直接發送
                webhook = DiscordWebhook(url=DISCORD_WEBHOOK_URL, content=f"```\n{message}\n```")
                response = webhook.execute()
                if response.ok:
                    logger.info("Discord 通知發送成功")
                else:
                    logger.error(f"Discord 通知失敗: {response.status_code} {response.content}")
            else:
                # 訊息太長，分割發送
                lines = message.split('\n')
                current_chunk = ""
                chunk_count = 1

                for line in lines:
                    # 檢查加入這行後是否超過限制
                    test_chunk = current_chunk + line + '\n'
                    if len(f"```\n{test_chunk}\n```") > max_discord_length:
                        # 發送當前塊
                        if current_chunk.strip():
                            header = f"🏆 美股ETF分析報告 (第{chunk_count}部分)\n\n"
                            chunk_content = header + current_chunk.strip()
                            webhook = DiscordWebhook(url=DISCORD_WEBHOOK_URL, content=f"```\n{chunk_content}\n```")
                            response = webhook.execute()
                            if response.ok:
                                logger.info(f"Discord 通知第{chunk_count}部分發送成功")
                            else:
                                logger.error(f"Discord 通知第{chunk_count}部分失敗: {response.status_code}")
                            chunk_count += 1
                            await asyncio.sleep(1)  # 避免發送太快

                        # 開始新塊
                        current_chunk = line + '\n'
                    else:
                        current_chunk = test_chunk

                # 發送最後一塊
                if current_chunk.strip():
                    header = f"🏆 美股ETF分析報告 (第{chunk_count}部分)\n\n"
                    chunk_content = header + current_chunk.strip()
                    webhook = DiscordWebhook(url=DISCORD_WEBHOOK_URL, content=f"```\n{chunk_content}\n```")
                    response = webhook.execute()
                    if response.ok:
                        logger.info(f"Discord 通知第{chunk_count}部分發送成功")
                    else:
                        logger.error(f"Discord 通知第{chunk_count}部分失敗: {response.status_code}")

        except Exception as e:
            logger.error(f"發送 Discord 通知時異常: {e}")
async def main():
    """主程式"""
    try:
        print("🚀 美股ETF技術分析程式啟動")
        print(f"⏰ 開始時間: {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}")
        print("🔍 分析條件: 成交量≥1000張/日")
        print("🏆 目標: 找出前十名優質美股ETF/期貨/貴金屬")
        print("📊 涵蓋: ETF、槓桿ETF、反向ETF、期貨、貴金屬等")
        print("=" * 70)

        # 初始化分析器
        analyzer = USETFAnalyzer()

        # 執行全部ETF分析
        print("📊 開始分析所有美股ETF/期貨/貴金屬（包含成交量篩選）...")
        results = await analyzer.analyze_all_etfs()

        if not results:
            error_message = "❌ 分析失敗，未能獲取任何結果"
            print(error_message)

            # 發送錯誤通知
            async with aiohttp.ClientSession() as session:
                await send_notification(session, error_message)
            return

        # 在終端顯示結果
        display_terminal_results(results, limit=10)

        # 格式化通知訊息
        print("\n📱 正在準備發送通知...")
        message = format_analysis_message(results, limit=10)

        # 發送通知
        print("📱 正在發送通知到 Telegram 和 Discord...")
        async with aiohttp.ClientSession() as session:
            await send_notification(session, message)

        print(f"\n⏰ 程式執行完成: {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}")
        print("=" * 70)
        print("✅ 美股ETF分析報告已完成並發送")
        print("🏆 前十名優質標的已顯示在上方")
        print("📊 涵蓋多種投資工具：ETF、期貨、貴金屬等")

    except Exception as e:
        error_message = f"❌ 主程式執行錯誤: {str(e)}\n\n詳細錯誤:\n{traceback.format_exc()}"
        logger.error(error_message)
        print(error_message)

        # 嘗試發送錯誤通知
        try:
            async with aiohttp.ClientSession() as session:
                await send_notification(session, f"❌ 美股ETF分析程式執行錯誤: {str(e)}")
        except Exception as notify_error:
            logger.error(f"發送錯誤通知失敗: {notify_error}")

if __name__ == "__main__":
    # 執行主程式
    asyncio.run(main())

In [ ]:
import asyncio
import aiohttp
import pandas as pd
import numpy as np
import yfinance as yf
import logging
import traceback
from datetime import datetime, timezone, timedelta
from typing import Dict, List, Any, Optional
import json
import os
import time
import requests
import random
from io import StringIO

# 如果您在 Jupyter Notebook 中，請使用這個版本
import nest_asyncio

# 安裝 nest_asyncio（如果尚未安裝）
try:
    import nest_asyncio
except ImportError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "nest_asyncio"])
    import nest_asyncio

# 應用 nest_asyncio 以允許嵌套事件循環
nest_asyncio.apply()

# 設定日誌
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# 台北時區
taipei_tz = timezone(timedelta(hours=8))

# Telegram 設定
TELEGRAM_BOT_TOKEN = "7902318521:AAEYoDMqwfHabI7L1SRiE4z33aFay42-VGE"
TELEGRAM_CHAT_IDS = [879781796, 8113868436]

# Discord 設定 (可選)
DISCORD_WEBHOOK_URL = "https://discord.com/api/webhooks/1362715080734802102/Jma7A3VhEQrrRxIX_JW2l6rATjAZXsGXGfnJuAMqmS1QvqG_2ptg3vr_nsVnuV_PlnBl"
MESSAGING_AVAILABLE = False

try:
    from discord_webhook import DiscordWebhook
    MESSAGING_AVAILABLE = True
except ImportError:
    MESSAGING_AVAILABLE = False
    logger.info("Discord webhook 套件未安裝，將跳過 Discord 通知")

def stock_selection_formula(stock_data: Dict[str, Any]) -> Dict[str, Any]:
    """
    股票選股公式評分系統
    根據漲幅、量比、價格合理性、流動性進行評分
    """
    try:
        score = 0
        details = {
            'total_score': 0,
            'criteria_scores': {},
            'meets_threshold': False,
            'selection_reasons': []
        }

        # 獲取數據
        price_change_5d = stock_data.get('price_change_5d', 0)
        current_price = stock_data.get('current_price', 0)
        volume_analysis = stock_data.get('volume_analysis', {})
        volume_ratio = volume_analysis.get('volume_ratio', 1.0)
        avg_volume_lots = volume_analysis.get('avg_volume_lots', 0)

        # 1. 漲幅權重 (40%)
        price_score = 0
        if price_change_5d > 9.5:
            price_score = 40
            details['selection_reasons'].append(f"5日漲幅達 {price_change_5d:.2f}% (>9.5%)")
        elif price_change_5d > 7.0:
            price_score = 30
            details['selection_reasons'].append(f"5日漲幅 {price_change_5d:.2f}% (>7.0%)")
        elif price_change_5d > 5.0:
            price_score = 20
            details['selection_reasons'].append(f"5日漲幅 {price_change_5d:.2f}% (>5.0%)")
        elif price_change_5d > 3.0:
            price_score = 10

        details['criteria_scores']['漲幅評分'] = price_score
        score += price_score

        # 2. 量比權重 (30%)
        volume_score = 0
        if volume_ratio > 2.0:
            volume_score = 30
            details['selection_reasons'].append(f"量比達 {volume_ratio:.2f} (爆量)")
        elif volume_ratio > 1.5:
            volume_score = 30
            details['selection_reasons'].append(f"量比達 {volume_ratio:.2f} (放量)")
        elif volume_ratio > 1.2:
            volume_score = 20
            details['selection_reasons'].append(f"量比 {volume_ratio:.2f} (溫和放量)")
        elif volume_ratio > 1.0:
            volume_score = 20
        elif volume_ratio > 0.8:
            volume_score = 10

        details['criteria_scores']['量比評分'] = volume_score
        score += volume_score

        # 3. 價格合理性 (20%)
        price_score_reasonable = 0
        if 20 <= current_price <= 500:
            price_score_reasonable = 20
            details['selection_reasons'].append(f"價格合理 {current_price:.2f}元 (20-500元)")
        elif 10 <= current_price < 20:
            price_score_reasonable = 15
            details['selection_reasons'].append(f"低價股 {current_price:.2f}元")
        elif 500 < current_price <= 1000:
            price_score_reasonable = 15
            details['selection_reasons'].append(f"中高價股 {current_price:.2f}元")
        elif 5 <= current_price < 10:
            price_score_reasonable = 10
        elif 1000 < current_price <= 2000:
            price_score_reasonable = 10

        details['criteria_scores']['價格合理性'] = price_score_reasonable
        score += price_score_reasonable

        # 4. 流動性 (10%)
        liquidity_score = 0
        if avg_volume_lots > 5000:
            liquidity_score = 10
            details['selection_reasons'].append(f"高流動性 {avg_volume_lots:.0f}張/日")
        elif avg_volume_lots > 2000:
            liquidity_score = 10
            details['selection_reasons'].append(f"良好流動性 {avg_volume_lots:.0f}張/日")
        elif avg_volume_lots > 1000:
            liquidity_score = 10
            details['selection_reasons'].append(f"基本流動性 {avg_volume_lots:.0f}張/日")
        elif avg_volume_lots > 500:
            liquidity_score = 5

        details['criteria_scores']['流動性評分'] = liquidity_score
        score += liquidity_score

        # 總分和是否符合門檻
        details['total_score'] = score
        details['meets_threshold'] = score >= 70

        # 額外加分條件
        bonus_score = 0
        bonus_reasons = []

        # 技術指標加分
        technical_analysis = stock_data.get('technical_analysis', {})
        rsi_value = technical_analysis.get('rsi_value', 50)
        macd_signal = technical_analysis.get('macd_signal', '中性')

        if 30 <= rsi_value <= 70 and macd_signal in ['黃金交叉', '多頭']:
            bonus_score += 5
            bonus_reasons.append("技術指標良好")

        # 趨勢加分
        trend_analysis = stock_data.get('trend_analysis', {})
        trend = trend_analysis.get('trend', '盤整')
        if '強勢上漲' in trend:
            bonus_score += 5
            bonus_reasons.append("強勢上漲趨勢")
        elif '上漲' in trend:
            bonus_score += 3
            bonus_reasons.append("上漲趨勢")

        details['bonus_score'] = bonus_score
        details['bonus_reasons'] = bonus_reasons
        details['final_score'] = score + bonus_score
        details['meets_threshold'] = (score + bonus_score) >= 70

        return details

    except Exception as e:
        logger.error(f"選股公式計算錯誤: {e}")
        return {
            'total_score': 0,
            'final_score': 0,
            'criteria_scores': {},
            'meets_threshold': False,
            'selection_reasons': ['計算錯誤'],
            'bonus_score': 0,
            'bonus_reasons': []
        }

class StockAnalyzer:
    def __init__(self):
        """初始化股票分析器"""
        self.config = {
            'rsi_period': 14,
            'macd_fast': 12,
            'macd_slow': 26,
            'macd_signal': 9,
            'bb_period': 20,
            'bb_std': 2,
            'kd_period': 14,
            'volume_ma_period': 20,
            'min_volume_lots': 1000,  # 最小成交量（張）
            'selection_threshold': 70,  # 選股門檻分數
            'trend_weights': {
                'price_trend': 0.3,
                'volume_trend': 0.2,
                'technical_score': 0.5
            }
        }
        self.taiwan_stocks = None
        self.stock_list_path = "taiwan_stocks_cache.csv"

    def get_taiwan_stocks(self, force_update=True):
        """獲取台灣股票清單"""
        if not force_update and os.path.exists(self.stock_list_path):
            if (time.time() - os.path.getmtime(self.stock_list_path)) < 86400:
                logger.info("從快取載入股票列表。")
                return pd.read_csv(self.stock_list_path, dtype={'stock_id': str})

        logger.info("從台灣證交所網站獲取最新股票清單...")
        urls = {
            "上市": "https://isin.twse.com.tw/isin/C_public.jsp?strMode=2",
            "上櫃": "https://isin.twse.com.tw/isin/C_public.jsp?strMode=4"
        }
        all_stocks_df = []
        headers = {'User-Agent': 'Mozilla/5.0'}

        for market_name, url in urls.items():
            try:
                res = requests.get(url, headers=headers, timeout=30)
                res.encoding = 'big5'
                html_dfs = pd.read_html(StringIO(res.text))
                df = html_dfs[0].copy()  # 修正：使用 copy() 避免警告
                df.columns = df.iloc[0]
                df = df.iloc[1:].copy()  # 修正：使用 copy()
                df.loc[:, 'market'] = market_name  # 修正：使用 .loc 設定值
                all_stocks_df.append(df)
            except Exception as e:
                logger.error(f"獲取 {market_name} 股票列表失敗: {e}")
                continue

        if not all_stocks_df:
            logger.error("無法從網站獲取任何股票數據，使用備用清單。")
            return self._get_backup_stock_list()

        try:
            df = pd.concat(all_stocks_df, ignore_index=True)
            df[['stock_id', 'stock_name']] = df['有價證券代號及名稱'].str.split(r'\s+', n=1, expand=True)
            df = df[df['stock_id'].str.match(r'^\d{4}$', na=False)].copy()
            exclude = ['ETF', 'ETN', 'TDR', '受益', '指數', '購', '牛', '熊', '存託憑證']
            df = df[~df['stock_name'].str.contains('|'.join(exclude), na=False)].copy()

            # 修正：使用 .loc 設定 yahoo_symbol
            df.loc[:, 'yahoo_symbol'] = df.apply(
                lambda row: f"{row['stock_id']}.TW" if '上市' in row['market'] else f"{row['stock_id']}.TWO", axis=1)

            final_df = df[['stock_id', 'stock_name', 'market', '產業別', 'yahoo_symbol']].rename(columns={'產業別': 'industry'})
            final_df = final_df.drop_duplicates(subset=['stock_id']).reset_index(drop=True)
            final_df.to_csv(self.stock_list_path, index=False)
            logger.info(f"成功獲取 {len(final_df)} 支股票清單並保存快取。")
            return final_df
        except Exception as e:
            logger.error(f"處理股票清單時發生錯誤: {e}")
            return self._get_backup_stock_list()

    def _get_backup_stock_list(self) -> pd.DataFrame:
        """備用股票清單"""
        try:
            logger.info("使用備用股票清單")

            # 擴展知名股票列表
            famous_stocks = [
                {'stock_id': '2330', 'stock_name': '台積電', 'market': '上市', 'yahoo_symbol': '2330.TW', 'industry': '半導體'},
                {'stock_id': '2317', 'stock_name': '鴻海', 'market': '上市', 'yahoo_symbol': '2317.TW', 'industry': '電子'},
                {'stock_id': '2454', 'stock_name': '聯發科', 'market': '上市', 'yahoo_symbol': '2454.TW', 'industry': '半導體'},
                {'stock_id': '2881', 'stock_name': '富邦金', 'market': '上市', 'yahoo_symbol': '2881.TW', 'industry': '金融'},
                {'stock_id': '2882', 'stock_name': '國泰金', 'market': '上市', 'yahoo_symbol': '2882.TW', 'industry': '金融'},
                {'stock_id': '2412', 'stock_name': '中華電', 'market': '上市', 'yahoo_symbol': '2412.TW', 'industry': '通信'},
                {'stock_id': '1301', 'stock_name': '台塑', 'market': '上市', 'yahoo_symbol': '1301.TW', 'industry': '塑膠'},
                {'stock_id': '1303', 'stock_name': '南亞', 'market': '上市', 'yahoo_symbol': '1303.TW', 'industry': '塑膠'},
                {'stock_id': '2308', 'stock_name': '台達電', 'market': '上市', 'yahoo_symbol': '2308.TW', 'industry': '電子'},
                {'stock_id': '2303', 'stock_name': '聯電', 'market': '上市', 'yahoo_symbol': '2303.TW', 'industry': '半導體'},
                {'stock_id': '2002', 'stock_name': '中鋼', 'market': '上市', 'yahoo_symbol': '2002.TW', 'industry': '鋼鐵'},
                {'stock_id': '2886', 'stock_name': '兆豐金', 'market': '上市', 'yahoo_symbol': '2886.TW', 'industry': '金融'},
                {'stock_id': '2891', 'stock_name': '中信金', 'market': '上市', 'yahoo_symbol': '2891.TW', 'industry': '金融'},
                {'stock_id': '3008', 'stock_name': '大立光', 'market': '上市', 'yahoo_symbol': '3008.TW', 'industry': '光學'},
                {'stock_id': '2357', 'stock_name': '華碩', 'market': '上市', 'yahoo_symbol': '2357.TW', 'industry': '電腦'},
                {'stock_id': '2382', 'stock_name': '廣達', 'market': '上市', 'yahoo_symbol': '2382.TW', 'industry': '電腦'},
                {'stock_id': '2395', 'stock_name': '研華', 'market': '上市', 'yahoo_symbol': '2395.TW', 'industry': '電腦'},
                {'stock_id': '3711', 'stock_name': '日月光投控', 'market': '上市', 'yahoo_symbol': '3711.TW', 'industry': '半導體'},
                {'stock_id': '2409', 'stock_name': '友達', 'market': '上市', 'yahoo_symbol': '2409.TW', 'industry': '面板'},
                {'stock_id': '2884', 'stock_name': '玉山金', 'market': '上市', 'yahoo_symbol': '2884.TW', 'industry': '金融'},
                # 新增更多股票
                {'stock_id': '2474', 'stock_name': '可成', 'market': '上市', 'yahoo_symbol': '2474.TW', 'industry': '電子'},
                {'stock_id': '2408', 'stock_name': '南亞科', 'market': '上市', 'yahoo_symbol': '2408.TW', 'industry': '半導體'},
                {'stock_id': '2301', 'stock_name': '光寶科', 'market': '上市', 'yahoo_symbol': '2301.TW', 'industry': '電子'},
                {'stock_id': '2207', 'stock_name': '和泰車', 'market': '上市', 'yahoo_symbol': '2207.TW', 'industry': '汽車'},
                {'stock_id': '1216', 'stock_name': '統一', 'market': '上市', 'yahoo_symbol': '1216.TW', 'industry': '食品'},
                {'stock_id': '1101', 'stock_name': '台泥', 'market': '上市', 'yahoo_symbol': '1101.TW', 'industry': '水泥'},
                {'stock_id': '2105', 'stock_name': '正新', 'market': '上市', 'yahoo_symbol': '2105.TW', 'industry': '橡膠'},
                {'stock_id': '2912', 'stock_name': '統一超', 'market': '上市', 'yahoo_symbol': '2912.TW', 'industry': '貿易百貨'},
                {'stock_id': '2885', 'stock_name': '元大金', 'market': '上市', 'yahoo_symbol': '2885.TW', 'industry': '金融'},
                {'stock_id': '2892', 'stock_name': '第一金', 'market': '上市', 'yahoo_symbol': '2892.TW', 'industry': '金融'},
            ]

            df = pd.DataFrame(famous_stocks)
            logger.info(f"備用股票清單包含 {len(df)} 支股票")
            return df

        except Exception as e:
            logger.error(f"生成備用股票清單時發生錯誤: {e}")
            return pd.DataFrame()

    def check_volume_filter(self, df: pd.DataFrame) -> bool:
        """檢查成交量是否符合篩選條件（每日至少1000張）"""
        try:
            if df.empty or 'Volume' not in df.columns:
                return False

            # 取最近20天的平均成交量
            recent_volume = df['Volume'].tail(20).mean()

            # 轉換為張數（1張 = 1000股）
            volume_lots = recent_volume / 1000

            # 檢查是否符合最小成交量要求
            meets_volume = volume_lots >= self.config['min_volume_lots']

            if meets_volume:
                logger.debug(f"成交量符合條件: {volume_lots:.0f} 張/日 (≥ {self.config['min_volume_lots']} 張)")
            else:
                logger.debug(f"成交量不符合條件: {volume_lots:.0f} 張/日 (< {self.config['min_volume_lots']} 張)")

            return meets_volume

        except Exception as e:
            logger.error(f"檢查成交量篩選時發生錯誤: {e}")
            return False

    def fetch_yfinance_data(self, stock_id: str, period: str = "1y", interval: str = "1d", retries: int = 3):
        """獲取股票數據（同步版本）"""
        for attempt in range(retries):
            try:
                df = yf.download(tickers=stock_id, period=period, interval=interval, progress=False, auto_adjust=True)
                if df.empty:
                    logger.warning(f"[{stock_id}] 無數據返回")
                    return pd.DataFrame()

                if isinstance(df.columns, pd.MultiIndex):
                    df.columns = df.columns.get_level_values(0)

                required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
                for col in required_cols:
                    if col in df.columns:
                        df[col] = pd.to_numeric(df[col], errors='coerce')

                df = df.dropna(subset=required_cols)
                return df.reset_index()

            except Exception as e:
                if attempt < retries - 1:
                    sleep_time = 0.5 * (2 ** attempt) + random.uniform(0, 1)
                    logger.warning(f"[{stock_id}] 第 {attempt + 1}/{retries} 次嘗試失敗: {e}。等待 {sleep_time:.2f} 秒後重試...")
                    time.sleep(sleep_time)
                else:
                    logger.error(f"[{stock_id}] 獲取數據失敗。")
                    return pd.DataFrame()
        return pd.DataFrame()

    def calculate_technical_indicators(self, df: pd.DataFrame) -> pd.DataFrame:
        """計算技術指標"""
        try:
            if df.empty:
                return df

            df = df.copy()

            # RSI
            df['RSI'] = self.calculate_rsi(df['Close'], self.config['rsi_period'])

            # MACD
            macd_line, macd_signal, macd_histogram = self.calculate_macd(
                df['Close'],
                self.config['macd_fast'],
                self.config['macd_slow'],
                self.config['macd_signal']
            )
            df['MACD'] = macd_line
            df['MACD_Signal'] = macd_signal
            df['MACD_Histogram'] = macd_histogram

            # 布林帶
            bb_upper, bb_middle, bb_lower = self.calculate_bollinger_bands(
                df['Close'],
                self.config['bb_period'],
                self.config['bb_std']
            )
            df['BB_Upper'] = bb_upper
            df['BB_Middle'] = bb_middle
            df['BB_Lower'] = bb_lower

            # KD指標
            k_percent, d_percent = self.calculate_kd(
                df['High'],
                df['Low'],
                df['Close'],
                self.config['kd_period']
            )
            df['K_Percent'] = k_percent
            df['D_Percent'] = d_percent

            # 移動平均線
            df['MA5'] = df['Close'].rolling(window=5).mean()
            df['MA10'] = df['Close'].rolling(window=10).mean()
            df['MA20'] = df['Close'].rolling(window=20).mean()

            # 成交量移動平均
            if 'Volume' in df.columns:
                df['Volume_MA'] = df['Volume'].rolling(window=self.config['volume_ma_period']).mean()

            return df

        except Exception as e:
            logger.error(f"計算技術指標時發生錯誤: {e}")
            return df

    def calculate_rsi(self, prices: pd.Series, period: int = 14) -> pd.Series:
        """計算RSI"""
        try:
            delta = prices.diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
            rs = gain / loss
            rsi = 100 - (100 / (1 + rs))
            return rsi
        except Exception as e:
            logger.error(f"計算RSI時發生錯誤: {e}")
            return pd.Series(index=prices.index, data=50)

    def calculate_macd(self, prices: pd.Series, fast: int = 12, slow: int = 26, signal: int = 9):
        """計算MACD"""
        try:
            ema_fast = prices.ewm(span=fast).mean()
            ema_slow = prices.ewm(span=slow).mean()
            macd_line = ema_fast - ema_slow
            macd_signal = macd_line.ewm(span=signal).mean()
            macd_histogram = macd_line - macd_signal
            return macd_line, macd_signal, macd_histogram
        except Exception as e:
            logger.error(f"計算MACD時發生錯誤: {e}")
            zero_series = pd.Series(index=prices.index, data=0)
            return zero_series, zero_series, zero_series

    def calculate_bollinger_bands(self, prices: pd.Series, period: int = 20, std_dev: int = 2):
        """計算布林帶"""
        try:
            middle = prices.rolling(window=period).mean()
            std = prices.rolling(window=period).std()
            upper = middle + (std * std_dev)
            lower = middle - (std * std_dev)
            return upper, middle, lower
        except Exception as e:
            logger.error(f"計算布林帶時發生錯誤: {e}")
            zero_series = pd.Series(index=prices.index, data=0)
            return zero_series, zero_series, zero_series

    def calculate_kd(self, high: pd.Series, low: pd.Series, close: pd.Series, period: int = 14):
        """計算KD指標"""
        try:
            lowest_low = low.rolling(window=period).min()
            highest_high = high.rolling(window=period).max()

            k_percent = 100 * ((close - lowest_low) / (highest_high - lowest_low))
            k_percent = k_percent.rolling(window=3).mean()
            d_percent = k_percent.rolling(window=3).mean()

            return k_percent, d_percent
        except Exception as e:
            logger.error(f"計算KD時發生錯誤: {e}")
            fifty_series = pd.Series(index=high.index, data=50)
            return fifty_series, fifty_series

    def analyze_trend(self, df: pd.DataFrame) -> Dict[str, Any]:
        """趨勢分析"""
        try:
            if df.empty or len(df) < 20:
                return {'trend': '盤整', 'strength': 0, 'price_change_5d': 0, 'price_change_20d': 0}

            current_price = float(df['Close'].iloc[-1])
            price_5d_ago = float(df['Close'].iloc[-6]) if len(df) >= 6 else current_price
            price_20d_ago = float(df['Close'].iloc[-21]) if len(df) >= 21 else current_price

            change_5d = ((current_price - price_5d_ago) / price_5d_ago * 100) if price_5d_ago != 0 else 0
            change_20d = ((current_price - price_20d_ago) / price_20d_ago * 100) if price_20d_ago != 0 else 0

            # 移動平均線趨勢
            ma5_current = float(df['MA5'].iloc[-1]) if 'MA5' in df.columns and not pd.isna(df['MA5'].iloc[-1]) else current_price
            ma20_current = float(df['MA20'].iloc[-1]) if 'MA20' in df.columns and not pd.isna(df['MA20'].iloc[-1]) else current_price

            # 判斷趨勢
            if change_5d > 3 and change_20d > 5 and current_price > ma5_current > ma20_current:
                trend = '強勢上漲'
                strength = 3
            elif change_5d > 1 and change_20d > 2 and current_price > ma5_current:
                trend = '上漲'
                strength = 2
            elif change_5d < -3 and change_20d < -5 and current_price < ma5_current < ma20_current:
                trend = '強勢下跌'
                strength = -3
            elif change_5d < -1 and change_20d < -2 and current_price < ma5_current:
                trend = '下跌'
                strength = -2
            else:
                trend = '盤整'
                strength = 0

            return {'trend': trend,
                'strength': strength,
                'price_change_5d': change_5d,
                'price_change_20d': change_20d,
                'ma5_position': ma5_current,
                'ma20_position': ma20_current
            }

        except Exception as e:
            logger.error(f"趨勢分析時發生錯誤: {e}")
            return {'trend': '盤整', 'strength': 0, 'price_change_5d': 0, 'price_change_20d': 0}

    def analyze_volume(self, df: pd.DataFrame) -> Dict[str, Any]:
        """成交量分析"""
        try:
            if df.empty or 'Volume' not in df.columns or len(df) < 20:
                return {'volume_trend': '正常', 'volume_ratio': 1.0, 'volume_signal': '中性', 'avg_volume_lots': 0}

            current_volume = float(df['Volume'].iloc[-1])
            avg_volume_20d = float(df['Volume'].rolling(window=20).mean().iloc[-1])

            volume_ratio = current_volume / avg_volume_20d if avg_volume_20d > 0 else 1.0
            avg_volume_lots = avg_volume_20d / 1000  # 轉換為張數

            if volume_ratio > 2.0:
                volume_trend = '爆量'
                volume_signal = '強烈'
            elif volume_ratio > 1.5:
                volume_trend = '放量'
                volume_signal = '積極'
            elif volume_ratio < 0.5:
                volume_trend = '縮量'
                volume_signal = '消極'
            else:
                volume_trend = '正常'
                volume_signal = '中性'

            return {
                'volume_trend': volume_trend,
                'volume_ratio': volume_ratio,
                'volume_signal': volume_signal,
                'avg_volume_lots': avg_volume_lots
            }

        except Exception as e:
            logger.error(f"成交量分析時發生錯誤: {e}")
            return {'volume_trend': '正常', 'volume_ratio': 1.0, 'volume_signal': '中性', 'avg_volume_lots': 0}

    def analyze_technical_indicators(self, df: pd.DataFrame) -> Dict[str, Any]:
        """技術指標分析"""
        try:
            if df.empty:
                return {
                    'rsi_signal': '中性', 'rsi_value': 50,
                    'macd_signal': '中性', 'macd_value': 0,
                    'kd_signal': '中性', 'k_value': 50, 'd_value': 50,
                    'bb_signal': '中性', 'bb_position': 0.5,
                    'score': 50
                }

            signals = {}
            score = 50

            # RSI 分析
            if 'RSI' in df.columns and not df['RSI'].empty:
                rsi_value = float(df['RSI'].iloc[-1]) if not pd.isna(df['RSI'].iloc[-1]) else 50
                signals['rsi_value'] = rsi_value

                if rsi_value > 80:
                    signals['rsi_signal'] = '超買'
                    score -= 15
                elif rsi_value > 70:
                    signals['rsi_signal'] = '偏高'
                    score -= 5
                elif rsi_value < 20:
                    signals['rsi_signal'] = '超賣'
                    score += 15
                elif rsi_value < 30:
                    signals['rsi_signal'] = '偏低'
                    score += 5
                else:
                    signals['rsi_signal'] = '中性'
            else:
                signals['rsi_signal'] = '中性'
                signals['rsi_value'] = 50

            # MACD 分析
            if all(col in df.columns for col in ['MACD', 'MACD_Signal']) and len(df) >= 2:
                macd_current = float(df['MACD'].iloc[-1]) if not pd.isna(df['MACD'].iloc[-1]) else 0
                macd_signal_current = float(df['MACD_Signal'].iloc[-1]) if not pd.isna(df['MACD_Signal'].iloc[-1]) else 0
                macd_prev = float(df['MACD'].iloc[-2]) if not pd.isna(df['MACD'].iloc[-2]) else 0
                macd_signal_prev = float(df['MACD_Signal'].iloc[-2]) if not pd.isna(df['MACD_Signal'].iloc[-2]) else 0

                signals['macd_value'] = macd_current

                if macd_prev <= macd_signal_prev and macd_current > macd_signal_current:
                    signals['macd_signal'] = '黃金交叉'
                    score += 20
                elif macd_prev >= macd_signal_prev and macd_current < macd_signal_current:
                    signals['macd_signal'] = '死亡交叉'
                    score -= 20
                elif macd_current > macd_signal_current:
                    signals['macd_signal'] = '多頭'
                    score += 5
                elif macd_current < macd_signal_current:
                    signals['macd_signal'] = '空頭'
                    score -= 5
                else:
                    signals['macd_signal'] = '中性'
            else:
                signals['macd_signal'] = '中性'
                signals['macd_value'] = 0

            # KD 分析
            if all(col in df.columns for col in ['K_Percent', 'D_Percent']):
                k_value = float(df['K_Percent'].iloc[-1]) if not pd.isna(df['K_Percent'].iloc[-1]) else 50
                d_value = float(df['D_Percent'].iloc[-1]) if not pd.isna(df['D_Percent'].iloc[-1]) else 50

                signals['k_value'] = k_value
                signals['d_value'] = d_value

                if k_value > 80 and d_value > 80:
                    signals['kd_signal'] = '超買'
                    score -= 10
                elif k_value < 20 and d_value < 20:
                    signals['kd_signal'] = '超賣'
                    score += 10
                elif k_value > d_value:
                    signals['kd_signal'] = '偏多'
                    score += 3
                elif k_value < d_value:
                    signals['kd_signal'] = '偏空'
                    score -= 3
                else:
                    signals['kd_signal'] = '中性'
            else:
                signals['kd_signal'] = '中性'
                signals['k_value'] = 50
                signals['d_value'] = 50

            # 布林帶分析
            if all(col in df.columns for col in ['BB_Upper', 'BB_Lower', 'Close']):
                current_price = float(df['Close'].iloc[-1])
                bb_upper = float(df['BB_Upper'].iloc[-1]) if not pd.isna(df['BB_Upper'].iloc[-1]) else current_price
                bb_lower = float(df['BB_Lower'].iloc[-1]) if not pd.isna(df['BB_Lower'].iloc[-1]) else current_price

                if bb_upper != bb_lower:
                    bb_position = (current_price - bb_lower) / (bb_upper - bb_lower)
                    signals['bb_position'] = bb_position

                    if bb_position > 0.8:
                        signals['bb_signal'] = '接近上軌'
                        score -= 5
                    elif bb_position < 0.2:
                        signals['bb_signal'] = '接近下軌'
                        score += 5
                    else:
                        signals['bb_signal'] = '中性'
                else:
                    signals['bb_signal'] = '中性'
                    signals['bb_position'] = 0.5
            else:
                signals['bb_signal'] = '中性'
                signals['bb_position'] = 0.5

            score = max(0, min(100, score))
            signals['score'] = score

            return signals

        except Exception as e:
            logger.error(f"技術指標分析時發生錯誤: {e}")
            return {
                'rsi_signal': '中性', 'rsi_value': 50,
                'macd_signal': '中性', 'macd_value': 0,
                'kd_signal': '中性', 'k_value': 50, 'd_value': 50,
                'bb_signal': '中性', 'bb_position': 0.5,
                'score': 50
            }

    def calculate_combined_score(self, trend_analysis: Dict, volume_analysis: Dict, technical_analysis: Dict) -> float:
        """計算綜合分數"""
        try:
            base_score = 50

            # 趨勢分數 (權重: 30%)
            trend_score = 0
            trend_strength = trend_analysis.get('strength', 0)
            if trend_strength > 0:
                trend_score = min(30, trend_strength * 10)
            elif trend_strength < 0:
                trend_score = max(-30, trend_strength * 10)

            # 成交量分數 (權重: 20%)
            volume_score = 0
            volume_ratio = volume_analysis.get('volume_ratio', 1.0)
            if volume_ratio > 1.5:
                volume_score = 10
            elif volume_ratio > 1.2:
                volume_score = 5
            elif volume_ratio < 0.7:
                volume_score = -5

            # 技術指標分數 (權重: 50%)
            tech_score = technical_analysis.get('score', 50) - 50

            # 計算加權總分
            total_score = base_score + (trend_score * 0.3) + (volume_score * 0.2) + (tech_score * 0.5)

            return max(0, min(100, total_score))

        except Exception as e:
            logger.error(f"計算綜合分數時發生錯誤: {e}")
            return 50.0

    def generate_recommendation(self, combined_score: float, trend_analysis: Dict, technical_analysis: Dict) -> Dict[str, Any]:
        """生成投資建議"""
        try:
            recommendation = "持有"
            confidence = "中等"
            reasons = []

            if combined_score >= 75:
                recommendation = "強力買進"
                confidence = "高"
            elif combined_score >= 60:
                recommendation = "買進"
                confidence = "中高"
            elif combined_score >= 40:
                recommendation = "持有"
                confidence = "中等"
            elif combined_score >= 25:
                recommendation = "賣出"
                confidence = "中高"
            else:
                recommendation = "強力賣出"
                confidence = "高"

            # 生成建議原因
            trend = trend_analysis.get('trend', '盤整')
            if '上漲' in trend:
                reasons.append(f"價格趨勢：{trend}")
            elif '下跌' in trend:
                reasons.append(f"價格趨勢：{trend}")

            rsi_signal = technical_analysis.get('rsi_signal', '中性')
            if rsi_signal in ['超賣', '偏低']:
                reasons.append(f"RSI指標：{rsi_signal}，可能反彈")
            elif rsi_signal in ['超買', '偏高']:
                reasons.append(f"RSI指標：{rsi_signal}，注意回調")

            macd_signal = technical_analysis.get('macd_signal', '中性')
            if macd_signal == '黃金交叉':
                reasons.append("MACD出現黃金交叉")
            elif macd_signal == '死亡交叉':
                reasons.append("MACD出現死亡交叉")

            return {
                'recommendation': recommendation,
                'confidence': confidence,
                'score': combined_score,
                'reasons': reasons
            }

        except Exception as e:
            logger.error(f"生成投資建議時發生錯誤: {e}")
            return {
                'recommendation': '持有',
                'confidence': '低',
                'score': 50,
                'reasons': ['分析過程出現錯誤']
            }

    async def analyze_stock_async(self, session: aiohttp.ClientSession, stock_info: Dict) -> Dict[str, Any]:
        """異步分析單支股票（加入選股公式）"""
        symbol = stock_info['yahoo_symbol']
        stock_name = stock_info['stock_name']

        try:
            logger.info(f"開始分析股票: {stock_name} ({symbol})")

            # 獲取股票數據
            df = self.fetch_yfinance_data(symbol)
            if df is None or df.empty or len(df) < 60:
                return {
                    'symbol': symbol,
                    'stock_name': stock_name,
                    'success': False,
                    'error': '無法獲取足夠的股票數據'
                }

            # 檢查成交量篩選條件
            if not self.check_volume_filter(df):
                return {
                    'symbol': symbol,
                    'stock_name': stock_name,
                    'success': False,
                    'error': f'成交量不符合條件（需≥{self.config["min_volume_lots"]}張/日）'
                }

            # 計算技術指標
            df_with_indicators = self.calculate_technical_indicators(df)

            # 進行各項分析
            trend_analysis = self.analyze_trend(df_with_indicators)
            volume_analysis = self.analyze_volume(df_with_indicators)
            technical_analysis = self.analyze_technical_indicators(df_with_indicators)

            # 計算綜合分數
            combined_score = self.calculate_combined_score(trend_analysis, volume_analysis, technical_analysis)

            # 生成投資建議
            recommendation = self.generate_recommendation(combined_score, trend_analysis, technical_analysis)

            # 獲取當前價格資訊
            current_price = float(df['Close'].iloc[-1])
            price_change = trend_analysis.get('price_change_5d', 0)

            # 建立股票數據字典用於選股公式
            stock_data = {
                'price_change_5d': price_change,
                'current_price': current_price,
                'volume_analysis': volume_analysis,
                'technical_analysis': technical_analysis,
                'trend_analysis': trend_analysis
            }

            # 應用選股公式
            selection_result = stock_selection_formula(stock_data)

            result = {
                'symbol': symbol,
                'stock_name': stock_name,
                'stock_id': stock_info.get('stock_id', ''),
                'market': stock_info.get('market', ''),
                'industry': stock_info.get('industry', ''),
                'success': True,
                'current_price': current_price,
                'price_change_5d': price_change,
                'combined_score': combined_score,
                'trend_analysis': trend_analysis,
                'volume_analysis': volume_analysis,
                'technical_analysis': technical_analysis,
                'recommendation': recommendation,
                'selection_result': selection_result,  # 新增選股公式結果
                'analysis_time': datetime.now(taipei_tz).isoformat()
            }

            # 記錄是否符合選股條件
            selection_status = "✅ 符合" if selection_result['meets_threshold'] else "❌ 不符合"
            logger.info(f"完成分析: {stock_name} - 分數: {combined_score:.1f} - 建議: {recommendation['recommendation']} - 選股公式: {selection_status} ({selection_result['final_score']}/100)")

            return result

        except Exception as e:
            logger.error(f"分析股票 {symbol} 時發生錯誤: {e}")
            return {
                'symbol': symbol,
                'stock_name': stock_name,
                'success': False,
                'error': str(e)
            }

    async def analyze_all_stocks(self) -> Dict[str, Any]:
        """分析所有股票（加入選股公式篩選）"""
        try:
            # 獲取完整股票清單
            if self.taiwan_stocks is None:
                self.taiwan_stocks = self.get_taiwan_stocks()

            if self.taiwan_stocks.empty:
                logger.error("無法獲取股票清單")
                return {}

            logger.info(f"準備分析 {len(self.taiwan_stocks)} 支股票（包含成交量篩選和選股公式）")

            results = {}
            failed_count = 0
            volume_filtered_count = 0
            selection_filtered_count = 0

            # 使用 aiohttp 進行異步分析
            async with aiohttp.ClientSession() as session:
                # 創建分析任務
                tasks = []
                for _, stock_info in self.taiwan_stocks.iterrows():
                    task = self.analyze_stock_async(session, stock_info.to_dict())
                    tasks.append(task)

                # 分批執行任務以避免過載
                batch_size = 10
                for i in range(0, len(tasks), batch_size):
                    batch_tasks = tasks[i:i+batch_size]
                    batch_results = await asyncio.gather(*batch_tasks, return_exceptions=True)

                    # 處理批次結果
                    for result in batch_results:
                        if isinstance(result, Exception):
                            logger.error(f"分析任務異常: {result}")
                            failed_count += 1
                            continue

                        if isinstance(result, dict) and 'symbol' in result:
                            if result.get('success', False):
                                # 檢查是否符合選股公式條件
                                selection_result = result.get('selection_result', {})
                                if selection_result.get('meets_threshold', False):
                                    results[result['symbol']] = result
                                else:
                                    selection_filtered_count += 1
                            else:
                                error_msg = result.get('error', '')
                                if '成交量不符合條件' in error_msg:
                                    volume_filtered_count += 1
                                else:
                                    failed_count += 1

                    # 批次間暫停避免過載
                    if i + batch_size < len(tasks):
                        await asyncio.sleep(2)

            logger.info(f"分析完成統計：")
            logger.info(f"  - 符合選股公式: {len(results)} 支")
            logger.info(f"  - 選股公式淘汰: {selection_filtered_count} 支")
            logger.info(f"  - 成交量篩選淘汰: {volume_filtered_count} 支")
            logger.info(f"  - 其他失敗: {failed_count} 支")

            return results

        except Exception as e:
            logger.error(f"批量分析股票時發生錯誤: {e}")
            return {}

def format_analysis_message(results: Dict[str, Any], limit: int = 10) -> str:
    """格式化分析結果為訊息（顯示符合選股公式的股票）"""
    try:
        if not results:
            return "❌ 未能獲取任何符合選股公式的分析結果"

        # 過濾成功的結果並按選股公式分數排序
        successful_results = [
            result for result in results.values()
            if result.get('success', False) and
               result.get('selection_result', {}).get('meets_threshold', False)
        ]

        # 按選股公式最終分數排序
        successful_results.sort(key=lambda x: x.get('selection_result', {}).get('final_score', 0), reverse=True)

        # 統計資訊
        total_analyzed = len(results)
        successful_count = len(successful_results)

        # 建立訊息
        message_parts = []
        message_parts.append("🎯 台股選股公式篩選結果")
        message_parts.append("=" * 40)
        message_parts.append(f"📈 分析時間: {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}")
        message_parts.append(f"🏆 符合條件: {successful_count} 支股票")
        message_parts.append(f"📊 選股條件:")
        message_parts.append(f"   • 5日漲幅 >9.5% (40分)")
        message_parts.append(f"   • 量比 >1.5 (30分)")
        message_parts.append(f"   • 價格 20-500元 (20分)")
        message_parts.append(f"   • 成交量 >1000張 (10分)")
        message_parts.append(f"   • 門檻分數: ≥70分")
        message_parts.append("")

        if not successful_results:
            message_parts.append("❌ 沒有符合選股公式條件的股票")
            message_parts.append("💡 建議調整篩選條件或等待更好的市場時機")
            return "\n".join(message_parts)

        # 顯示符合條件的優質股票
        top_stocks = successful_results[:limit]
        message_parts.append(f"🏆 符合選股公式的優質股票 (前{len(top_stocks)}名):")
        message_parts.append("-" * 40)

        for i, result in enumerate(top_stocks, 1):
            stock_name = result.get('stock_name', 'Unknown')
            stock_id = result.get('stock_id', 'Unknown')
            selection_result = result.get('selection_result', {})
            final_score = selection_result.get('final_score', 0)
            selection_reasons = selection_result.get('selection_reasons', [])
            current_price = result.get('current_price', 0)
            price_change = result.get('price_change_5d', 0)
            volume_info = result.get('volume_analysis', {})
            volume_ratio = volume_info.get('volume_ratio', 1.0)
            avg_volume_lots = volume_info.get('avg_volume_lots', 0)

            # 價格變化符號
            change_symbol = "🚀" if price_change > 15 else "📈" if price_change > 0 else "📉"

            message_parts.append(f"{i}. {stock_name} ({stock_id})")
            message_parts.append(f"   🎯 選股分數: {final_score}/100")
            message_parts.append(f"   💰 價格: {current_price:.2f}元 ({change_symbol}{price_change:+.2f}%)")
            message_parts.append(f"   📊 量比: {volume_ratio:.2f} | 成交量: {avg_volume_lots:.0f}張/日")

            # 顯示選股原因（前3個）
            if selection_reasons:
                reasons_text = " | ".join(selection_reasons[:3])
                message_parts.append(f"   ✅ 符合條件: {reasons_text}")

            # 詳細評分
            criteria_scores = selection_result.get('criteria_scores', {})
            message_parts.append(f"   📋 評分明細: 漲幅{criteria_scores.get('漲幅評分', 0)} | 量比{criteria_scores.get('量比評分', 0)} | 價格{criteria_scores.get('價格合理性', 0)} | 流動性{criteria_scores.get('流動性評分', 0)}")

            # 加分項目
            bonus_score = selection_result.get('bonus_score', 0)
            if bonus_score > 0:
                bonus_reasons = selection_result.get('bonus_reasons', [])
                message_parts.append(f"   ⭐ 加分: +{bonus_score}分 ({', '.join(bonus_reasons)})")

            message_parts.append("")

        # 添加統計摘要
        if successful_results:
            avg_selection_score = sum(r.get('selection_result', {}).get('final_score', 0) for r in successful_results) / len(successful_results)
            high_score_count = len([r for r in successful_results if r.get('selection_result', {}).get('final_score', 0) >= 80])

            message_parts.append("📊 統計摘要:")
            message_parts.append(f"   🎯 平均選股分數: {avg_selection_score:.1f}")
            message_parts.append(f"   🌟 高分股票 (≥80): {high_score_count}")

            # 漲幅分布
            price_changes = [r.get('price_change_5d', 0) for r in top_stocks]
            avg_change = sum(price_changes) / len(price_changes) if price_changes else 0
            max_change = max(price_changes) if price_changes else 0

            message_parts.append(f"   📈 平均漲幅: {avg_change:.2f}%")
            message_parts.append(f"   🚀 最高漲幅: {max_change:.2f}%")

        message_parts.append("")
        message_parts.append("🎯 選股公式說明:")
        message_parts.append("• 漲幅權重40%: 5日漲幅>9.5%獲滿分")
        message_parts.append("• 量比權重30%: 量比>1.5獲滿分")
        message_parts.append("• 價格合理性20%: 20-500元獲滿分")
        message_parts.append("• 流動性10%: >1000張/日獲滿分")
        message_parts.append("• 技術指標和趨勢可額外加分")
        message_parts.append("")
        message_parts.append("⚠️ 投資提醒:")
        message_parts.append("• 本選股公式僅供參考，投資有風險")
        message_parts.append("• 已篩選出符合多重條件的強勢股")
        message_parts.append("• 請注意風險控制，適當分散投資")
        message_parts.append("• 建議結合基本面分析做最終決策")

        return "\n".join(message_parts)

    except Exception as e:
        logger.error(f"格式化訊息時發生錯誤: {e}")
        return f"❌ 格式化分析結果時發生錯誤: {str(e)}"

def display_terminal_results(results: Dict[str, Any], limit: int = 10):
    """在終端顯示分析結果（選股公式版本）"""
    try:
        if not results:
            print("❌ 未能獲取任何符合選股公式的分析結果")
            return

        # 過濾符合選股公式的結果並按分數排序
        successful_results = [
            result for result in results.values()
            if result.get('success', False) and
               result.get('selection_result', {}).get('meets_threshold', False)
        ]

        successful_results.sort(key=lambda x: x.get('selection_result', {}).get('final_score', 0), reverse=True)

        print("\n" + "=" * 90)
        print("🎯 台股選股公式篩選結果".center(90))
        print("=" * 90)
        print(f"📈 分析時間: {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"🏆 符合選股公式條件: {len(successful_results)} 支股票")
        print(f"📊 選股條件: 漲幅>9.5%(40分) + 量比>1.5(30分) + 價格20-500元(20分) + 成交量>1000張(10分) ≥70分")
        print("=" * 90)

        if not successful_results:
            print("❌ 沒有符合選股公式條件的股票")
            print("💡 建議調整篩選條件或等待更好的市場時機")
            return

        # 表格標題
        print(f"{'排名':<4}{'代碼':<8}{'股票名稱':<12}{'選股分數':<8}{'價格':<10}{'漲跌%':<10}{'量比':<8}{'成交量(張)':<12}")
        print("-" * 90)

        # 顯示符合條件的股票
        top_stocks = successful_results[:limit]
        for i, result in enumerate(top_stocks, 1):
            stock_name = result.get('stock_name', 'Unknown')[:10]  # 限制長度

            stock_id = result.get('stock_id', 'Unknown')
            selection_result = result.get('selection_result', {})
            final_score = selection_result.get('final_score', 0)
            current_price = result.get('current_price', 0)
            price_change = result.get('price_change_5d', 0)
            volume_info = result.get('volume_analysis', {})
            volume_ratio = volume_info.get('volume_ratio', 1.0)
            avg_volume_lots = volume_info.get('avg_volume_lots', 0)

            print(f"{i:<4}{stock_id:<8}{stock_name:<12}{final_score:<8.0f}{current_price:<10.2f}{price_change:<+10.2f}{volume_ratio:<8.2f}{avg_volume_lots:<12.0f}")

        print("=" * 90)

        # 詳細分析前五名
        print("\n📊 詳細選股分析報告 (前五名):")
        print("=" * 90)

        for i, result in enumerate(top_stocks[:5], 1):
            stock_name = result.get('stock_name', 'Unknown')
            stock_id = result.get('stock_id', 'Unknown')
            selection_result = result.get('selection_result', {})
            final_score = selection_result.get('final_score', 0)
            criteria_scores = selection_result.get('criteria_scores', {})
            selection_reasons = selection_result.get('selection_reasons', [])
            bonus_score = selection_result.get('bonus_score', 0)
            bonus_reasons = selection_result.get('bonus_reasons', [])

            trend_analysis = result.get('trend_analysis', {})
            technical_analysis = result.get('technical_analysis', {})
            volume_analysis = result.get('volume_analysis', {})

            print(f"\n{i}. {stock_name} ({stock_id}) - 選股分數: {final_score}/100")
            print("-" * 60)
            print(f"💰 當前價格: {result.get('current_price', 0):.2f}元")
            print(f"🚀 5日漲幅: {result.get('price_change_5d', 0):+.2f}%")
            print(f"📊 量比: {volume_analysis.get('volume_ratio', 1.0):.2f}")
            print(f"📈 成交量: {volume_analysis.get('avg_volume_lots', 0):.0f}張/日")

            # 選股公式評分明細
            print(f"🎯 選股公式評分明細:")
            print(f"   漲幅評分: {criteria_scores.get('漲幅評分', 0)}/40")
            print(f"   量比評分: {criteria_scores.get('量比評分', 0)}/30")
            print(f"   價格合理性: {criteria_scores.get('價格合理性', 0)}/20")
            print(f"   流動性評分: {criteria_scores.get('流動性評分', 0)}/10")
            if bonus_score > 0:
                print(f"   加分項目: +{bonus_score} ({', '.join(bonus_reasons)})")

            # 符合條件說明
            if selection_reasons:
                print(f"✅ 符合條件: {', '.join(selection_reasons[:3])}")

            # 技術指標詳情
            print(f"🔍 技術指標:")
            print(f"   RSI: {technical_analysis.get('rsi_value', 50):.1f} ({technical_analysis.get('rsi_signal', '中性')})")
            print(f"   MACD: {technical_analysis.get('macd_signal', '中性')}")
            print(f"   趨勢: {trend_analysis.get('trend', '盤整')}")

        print("\n" + "=" * 90)
        print("🎯 選股公式說明:")
        print("• 漲幅權重40%: 5日漲幅>9.5%獲40分，>7%獲30分，>5%獲20分，>3%獲10分")
        print("• 量比權重30%: 量比>2.0或>1.5獲30分，>1.2獲20分，>1.0獲20分")
        print("• 價格合理性20%: 20-500元獲20分，其他價位按合理性給分")
        print("• 流動性10%: 成交量>1000張/日獲10分")
        print("• 技術指標和趨勢分析可額外加分")
        print("• 總分≥70分才符合選股條件")
        print("\n⚠️ 投資提醒:")
        print("• 本選股公式專門篩選強勢上漲且有量配合的股票")
        print("• 已符合多重技術條件，但投資仍有風險")
        print("• 建議分散投資，設定停損點")
        print("• 請結合基本面分析做最終投資決策")
        print("=" * 90)

    except Exception as e:
        logger.error(f"顯示終端結果時發生錯誤: {e}")
        print(f"❌ 顯示結果時發生錯誤: {str(e)}")

async def send_notification(session: aiohttp.ClientSession, message: str, files: List[str] = None):
    """發送通知到 Telegram 和 Discord"""

    # Telegram 通知
    try:
        # 檢查 Telegram 設定
        if not TELEGRAM_BOT_TOKEN or not TELEGRAM_CHAT_IDS or TELEGRAM_BOT_TOKEN == "YOUR_BOT_TOKEN_HERE":
            logger.warning("Telegram 設定未完成，跳過發送通知")
            print("📱 Telegram 設定未完成，請設定 TELEGRAM_BOT_TOKEN 和 TELEGRAM_CHAT_IDS")
        else:
            # 分割長訊息
            max_length = 4000
            if len(message) > max_length:
                parts = []
                current_part = ""

                for line in message.split('\n'):
                    if len(current_part + line + '\n') > max_length:
                        if current_part:
                            parts.append(current_part.strip())
                            current_part = line + '\n'
                        else:
                            # 如果單行太長，直接截斷
                            parts.append(line[:max_length])
                    else:
                        current_part += line + '\n'

                if current_part:
                    parts.append(current_part.strip())
            else:
                parts = [message]

            # 發送文字訊息給所有 chat_id
            telegram_url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"

            for chat_id in TELEGRAM_CHAT_IDS:
                for i, part in enumerate(parts):
                    if i > 0:
                        part = f"🎯 選股公式篩選結果 (續 {i+1}/{len(parts)})\n\n" + part

                    payload = {
                        'chat_id': chat_id,
                        'text': part
                    }

                    try:
                        async with session.post(telegram_url, json=payload, timeout=20) as response:
                            if response.status == 200:
                                logger.info(f"成功發送 Telegram 通知到 {chat_id} (第 {i+1}/{len(parts)} 部分)")
                            else:
                                error_text = await response.text()
                                logger.error(f"發送 Telegram 通知失敗到 {chat_id} (第 {i+1} 部分): {response.status} - {error_text}")
                    except Exception as e:
                        logger.error(f"發送 Telegram 通知時發生網路錯誤到 {chat_id} (第 {i+1} 部分): {e}")

                    # 避免發送太快
                    if i < len(parts) - 1:
                        await asyncio.sleep(1)

    except Exception as e:
        logger.error(f"發送 Telegram 通知時異常: {e}")

    # Discord 通知 (如果可用)
    if MESSAGING_AVAILABLE and DISCORD_WEBHOOK_URL and DISCORD_WEBHOOK_URL != "YOUR_DISCORD_WEBHOOK_URL":
        try:
            webhook = DiscordWebhook(url=DISCORD_WEBHOOK_URL, content=f"```\n{message}\n```")
            response = webhook.execute()
            if response.ok:
                logger.info("Discord 通知發送成功")
            else:
                logger.error(f"Discord 通知失敗: {response.status_code} {response.content}")
        except Exception as e:
            logger.error(f"發送 Discord 通知時異常: {e}")

async def main():
    """主程式"""
    try:
        print("🎯 台股選股公式分析程式啟動")
        print(f"⏰ 開始時間: {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}")
        print("📊 選股條件:")
        print("   • 5日漲幅 >9.5% (40分)")
        print("   • 量比 >1.5 (30分)")
        print("   • 價格 20-500元 (20分)")
        print("   • 成交量 >1000張/日 (10分)")
        print("   • 門檻分數: ≥70分")
        print("🎯 目標: 找出符合選股公式的強勢股票")
        print("=" * 70)

        # 初始化分析器
        analyzer = StockAnalyzer()

        # 執行全部股票分析
        print("📊 開始分析所有台股（應用選股公式篩選）...")
        results = await analyzer.analyze_all_stocks()

        if not results:
            error_message = "❌ 分析失敗，未找到符合選股公式條件的股票"
            print(error_message)
            print("💡 可能原因：")
            print("   • 當前市場沒有符合條件的強勢股")
            print("   • 選股條件過於嚴格")
            print("   • 建議調整篩選參數或等待更好的市場時機")

            # 發送錯誤通知
            async with aiohttp.ClientSession() as session:
                await send_notification(session, error_message + "\n\n💡 建議等待更好的市場時機或調整選股條件")
            return

        # 在終端顯示結果
        display_terminal_results(results, limit=10)

        # 格式化通知訊息
        print("\n📱 正在準備發送通知...")
        message = format_analysis_message(results, limit=10)

        # 發送通知
        print("📱 正在發送選股結果到 Telegram 和 Discord...")
        async with aiohttp.ClientSession() as session:
            await send_notification(session, message)

        print(f"\n⏰ 程式執行完成: {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}")
        print("=" * 70)
        print("✅ 選股公式分析報告已完成並發送")
        print(f"🎯 找到 {len(results)} 支符合條件的強勢股票")
        print("🏆 符合選股公式的優質股票已顯示在上方")

    except Exception as e:
        error_message = f"❌ 主程式執行錯誤: {str(e)}\n\n詳細錯誤:\n{traceback.format_exc()}"
        logger.error(error_message)
        print(error_message)

        # 嘗試發送錯誤通知
        try:
            async with aiohttp.ClientSession() as session:
                await send_notification(session, f"❌ 選股程式執行錯誤: {str(e)}")
        except Exception as notify_error:
            logger.error(f"發送錯誤通知失敗: {notify_error}")

if __name__ == "__main__":
    # 執行主程式
    asyncio.run(main())

上面可傳訊可發報告

In [ ]:

import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import mplfinance as mpf
import warnings
import logging
import os
import time
import asyncio
import aiohttp
import glob
import sys
import traceback
import requests
from io import StringIO
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional, Tuple
import pytz
import nest_asyncio
from discord_webhook import DiscordWebhook
import telegram

# 應用 nest_asyncio 以支援 Jupyter 環境
nest_asyncio.apply()

warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 設定時區
taipei_tz = pytz.timezone('Asia/Taipei')

# 設定日誌
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# 建立圖表目錄
CHARTS_DIR = "charts"
os.makedirs(CHARTS_DIR, exist_ok=True)

# 股票清單快取路徑
STOCK_LIST_PATH = "taiwan_stocks.csv"

# ==============================================================================
# 全域常數與密鑰管理
# ==============================================================================
TELEGRAM_TOKEN = "7902318521:AAEYoDMqwfHabI7L1SRiE4z33aFay42-VGE"
TELEGRAM_CHAT_ID = [879781796, 8113868436]
DISCORD_WEBHOOK_URL = "https://discord.com/api/webhooks/1362715080734802102/Jma7A3VhEQrrRxIX_JW2l6rATjAZXsGXGfnJuAMqmS1QvqG_2ptg3vr_nsVnuV_PlnBl"

# ==============================================================================
# 通知函數
# ==============================================================================
async def send_notification(session: aiohttp.ClientSession, message: str, files: List[str] = None):
    """發送通知到 Telegram 和 Discord"""
    # Telegram
    try:
        # 遍歷所有 Telegram Chat ID
        for chat_id in TELEGRAM_CHAT_ID:
            # 發送文字訊息
            url_msg = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage"
            payload = {"chat_id": chat_id, "text": message, "parse_mode": "Markdown"}
            await session.post(url_msg, json=payload, timeout=20)
            logger.info(f"Telegram 摘要已成功發送到 chat_id: {chat_id}。")

            # 如果有檔案，也遍歷發送
            if files:
                url_photo = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendPhoto"
                for file_path in files:
                    if os.path.exists(file_path):
                        data = aiohttp.FormData()
                        data.add_field('chat_id', str(chat_id))
                        with open(file_path, 'rb') as f:
                            data.add_field('photo', f, filename=os.path.basename(file_path))
                            await session.post(url_photo, data=data, timeout=60)
                logger.info(f"Telegram 圖檔已成功發送到 chat_id: {chat_id} ({len(files)}個)。")
    except Exception as e:
        logger.error(f"發送 Telegram 通知時異常: {e}")
        traceback.print_exc()

    # Discord
    try:
        webhook = DiscordWebhook(url=DISCORD_WEBHOOK_URL, content=f"```\n{message}\n```")
        if files:
            for file_path in files:
                if os.path.exists(file_path):
                    with open(file_path, 'rb') as f:
                        webhook.add_file(file=f.read(), filename=os.path.basename(file_path))
        response = webhook.execute()
        if response.ok:
            logger.info("Discord 通知發送成功。")
        else:
            logger.error(f"Discord 通知失敗: {response.status_code} {response.content}")
    except Exception as e:
        logger.error(f"發送 Discord 通知時異常: {e}")

def format_notification_message(filtered_results: Dict) -> str:
    """格式化通知訊息"""
    try:
        current_time = datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')

        if not filtered_results:
            return f"""
🤖 台股技術分析報告
📅 分析時間: {current_time}

❌ 本次分析未找到符合條件的優質股票
💡 建議調整篩選條件或關注市場變化
"""

        message = f"""
🤖 台股技術分析報告
📅 分析時間: {current_time}
🎯 找到 {len(filtered_results)} 支優質股票

📊 TOP 5 推薦股票:
"""

        for rank, (stock_id, result) in enumerate(list(filtered_results.items())[:5], 1):
            trend = result.get('trend_analysis', {})
            tech = result.get('technical_analysis', {})
            recommendation = result.get('recommendation', {})

            message += f"""
{rank}. {result.get('stock_name', '')} ({stock_id})
   💰 價格: {result.get('current_price', 0):.2f} TWD
   📈 評分: {result.get('combined_score', 0):.1f}/100
   🎯 建議: {recommendation.get('action', '觀望')}
   📊 趨勢: {trend.get('trend', '盤整')}
   🔍 RSI: {tech.get('rsi_value', 50):.1f}
"""

        message += f"""
⚠️  投資提醒:
• 本分析僅供參考，投資有風險
• 建議結合基本面分析
• 請做好風險控制
"""

        return message.strip()

    except Exception as e:
        logger.error(f"格式化通知訊息時發生錯誤: {e}")
        return f"台股分析完成，但訊息格式化失敗: {str(e)}"

def get_stock_symbols() -> List[str]:
    """獲取股票代碼列表"""
    taiwan_stocks = StockAnalyzer().get_taiwan_stocks()
    return taiwan_stocks['yahoo_symbol'].tolist()

class TechnicalIndicators:
    """純 pandas/numpy 技術指標計算類"""

    @staticmethod
    def sma(data: pd.Series, window: int) -> pd.Series:
        """簡單移動平均線"""
        return data.rolling(window=window, min_periods=1).mean()

    @staticmethod
    def ema(data: pd.Series, window: int) -> pd.Series:
        """指數移動平均線"""
        return data.ewm(span=window, adjust=False).mean()

    @staticmethod
    def rsi(data: pd.Series, window: int = 14) -> pd.Series:
        """相對強弱指標"""
        try:
            delta = data.diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=window, min_periods=1).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=window, min_periods=1).mean()

            # 避免除零錯誤
            rs = gain / loss.replace(0, np.nan)
            rsi = 100 - (100 / (1 + rs))
            return rsi.fillna(50)  # 填充 NaN 為中性值 50
        except Exception as e:
            logger.warning(f"計算 RSI 時發生錯誤: {e}")
            return pd.Series([50] * len(data), index=data.index)

    @staticmethod
    def macd(data: pd.Series, fast: int = 12, slow: int = 26, signal: int = 9) -> Tuple[pd.Series, pd.Series, pd.Series]:
        """MACD 指標"""
        try:
            ema_fast = TechnicalIndicators.ema(data, fast)
            ema_slow = TechnicalIndicators.ema(data, slow)
            macd_line = ema_fast - ema_slow
            signal_line = TechnicalIndicators.ema(macd_line, signal)
            histogram = macd_line - signal_line
            return macd_line, signal_line, histogram
        except Exception as e:
            logger.warning(f"計算 MACD 時發生錯誤: {e}")
            zeros = pd.Series([0] * len(data), index=data.index)
            return zeros, zeros, zeros

    @staticmethod
    def bollinger_bands(data: pd.Series, window: int = 20, num_std: float = 2) -> Tuple[pd.Series, pd.Series, pd.Series]:
        """布林帶"""
        try:
            sma = TechnicalIndicators.sma(data, window)
            std = data.rolling(window=window, min_periods=1).std()
            upper_band = sma + (std * num_std)
            lower_band = sma - (std * num_std)
            return upper_band, sma, lower_band
        except Exception as e:
            logger.warning(f"計算布林帶時發生錯誤: {e}")
            sma = TechnicalIndicators.sma(data, window)
            return sma, sma, sma

    @staticmethod
    def stochastic(high: pd.Series, low: pd.Series, close: pd.Series,
                   k_window: int = 14, d_window: int = 3) -> Tuple[pd.Series, pd.Series]:
        """KD 隨機指標"""
        try:
            lowest_low = low.rolling(window=k_window, min_periods=1).min()
            highest_high = high.rolling(window=k_window, min_periods=1).max()

            # 避免除零錯誤
            k_percent = 100 * ((close - lowest_low) / (highest_high - lowest_low).replace(0, np.nan))
            k_percent = k_percent.fillna(50)

            # 平滑化 K 值
            k_smooth = k_percent.rolling(window=d_window, min_periods=1).mean()
            d_smooth = k_smooth.rolling(window=d_window, min_periods=1).mean()

            return k_smooth, d_smooth
        except Exception as e:
            logger.warning(f"計算 KD 指標時發生錯誤: {e}")
            default_series = pd.Series([50] * len(close), index=close.index)
            return default_series, default_series

    @staticmethod
    def williams_r(high: pd.Series, low: pd.Series, close: pd.Series, window: int = 14) -> pd.Series:
        """威廉指標"""
        try:
            highest_high = high.rolling(window=window, min_periods=1).max()
            lowest_low = low.rolling(window=window, min_periods=1).min()
            wr = -100 * (highest_high - close) / (highest_high - lowest_low).replace(0, np.nan)
            return wr.fillna(-50)
        except Exception as e:
            logger.warning(f"計算威廉指標時發生錯誤: {e}")
            return pd.Series([-50] * len(close), index=close.index)

class StockAnalyzer:
    def __init__(self):
        self.taiwan_stocks = None
        self.tech_indicators = TechnicalIndicators()
        self.stock_data = {}
        self.technical_indicators = {}
        self.config = {
            'top_n': 10,
            'score_weights': {
                'rsi': 0.3,
                'macd': 0.3,
                'volume': 0.2,
                'price_trend': 0.2
            }
        }
        self.stocks_df = pd.DataFrame()

    def get_stock_symbols(self) -> List[str]:
        """獲取股票代碼列表"""
        taiwan_stocks = self.get_taiwan_stocks()
        return taiwan_stocks['yahoo_symbol'].tolist()

    def standardize_columns(self, df: pd.DataFrame) -> pd.DataFrame:
        if not isinstance(df, pd.DataFrame) or df.empty:
            return pd.DataFrame()
        df = df.copy()
        if df.index.name is not None and 'date' in df.index.name.lower():
            df = df.reset_index()
        column_mapping = {
            'Date': 'Date', 'date': 'Date', '日期': 'Date',
            'Open': 'Open', 'open': 'Open', '開盤價': 'Open',
            'High': 'High', 'high': 'High', '最高價': 'High',
            'Low': 'Low', 'low': 'Low', '最低價': 'Low',
            'Close': 'Close', 'close': 'Close', '收盤價': 'Close', 'Adj Close': 'Close',
            'Volume': 'Volume', 'volume': 'Volume', '成交量': 'Volume',
        }
        rename_dict = {col: column_mapping.get(col, col) for col in df.columns}
        df.rename(columns=rename_dict, inplace=True)
        return df

    def get_taiwan_stocks(self, force_update=False):
        if not force_update and os.path.exists(STOCK_LIST_PATH):
            if (time.time() - os.path.getmtime(STOCK_LIST_PATH)) < 86400:
                logger.info("從快取載入股票列表。")
                return pd.read_csv(STOCK_LIST_PATH, dtype={'stock_id': str})

        logger.info("從台灣證交所網站獲取最新股票清單...")
        urls = {
            "上市": "https://isin.twse.com.tw/isin/C_public.jsp?strMode=2",
            "上櫃": "https://isin.twse.com.tw/isin/C_public.jsp?strMode=4"
        }
        all_stocks_df = []
        headers = {'User-Agent': 'Mozilla/5.0'}

        for market_name, url in urls.items():
            try:
                res = requests.get(url, headers=headers, timeout=30)
                res.encoding = 'big5'
                html_dfs = pd.read_html(StringIO(res.text))
                df = html_dfs[0]
                df.columns = df.iloc[0]
                df = df.iloc[1:]
                df['market'] = market_name
                all_stocks_df.append(df)
            except Exception as e:
                logger.error(f"獲取 {market_name} 股票列表失敗: {e}")
                continue

        if not all_stocks_df:
            logger.error("無法從網站獲取任何股票數據，程序終止。")
            return pd.DataFrame()

        df = pd.concat(all_stocks_df, ignore_index=True)
        df[['stock_id', 'stock_name']] = df['有價證券代號及名稱'].str.split(r'\s+', n=1, expand=True)
        df = df[df['stock_id'].str.match(r'^\d{4}$')].copy()
        exclude = ['ETF', 'ETN', 'TDR', '受益', '指數', '購', '牛', '熊', '存託憑證']
        df = df[~df['stock_name'].str.contains('|'.join(exclude), na=False)]
        df['yahoo_symbol'] = df.apply(
            lambda row: f"{row['stock_id']}.TW" if '上市' in row['market'] else f"{row['stock_id']}.TWO", axis=1)

        final_df = df[['stock_id', 'stock_name', 'market', '產業別', 'yahoo_symbol']].rename(columns={'產業別': 'industry'})
        final_df = final_df.drop_duplicates(subset=['stock_id']).reset_index(drop=True)
        final_df.to_csv(STOCK_LIST_PATH, index=False)
        logger.info(f"成功獲取 {len(final_df)} 支股票清單並保存快取。")
        return final_df

    def fetch_yfinance_data(self, symbol: str, period: str = '1y', interval: str = '1d') -> Optional[pd.DataFrame]:
        """獲取 Yahoo Finance 數據"""
        try:
            ticker = yf.Ticker(symbol)
            df = ticker.history(period=period, interval=interval)

            if df.empty:
                logger.warning(f"無法獲取 {symbol} 的數據")
                return None

            # 確保數據類型正確
            numeric_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
            for col in numeric_columns:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col], errors='coerce')

            # 移除 NaN 值
            df = df.dropna()

            if len(df) < 20:  # 至少需要20天數據
                logger.warning(f"{symbol} 數據不足，只有 {len(df)} 天")
                return None

            logger.info(f"成功獲取 {symbol} 數據，共 {len(df)} 筆記錄")
            return df

        except Exception as e:
            logger.error(f"獲取 {symbol} 數據時發生錯誤: {e}")
            return None

    def calculate_technical_indicators(self, df: pd.DataFrame) -> pd.DataFrame:
        """計算技術指標 - 純 pandas 版本"""
        try:
            df = df.copy()

            # 確保數據類型正確
            for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col], errors='coerce')

            # 移除 NaN 值
            df = df.dropna()

            if len(df) < 20:  # 需要足夠的數據計算指標
                logger.warning("數據不足，無法計算所有技術指標")
                return df

            # 計算移動平均線
            df['MA5'] = self.tech_indicators.sma(df['Close'], 5)
            df['MA10'] = self.tech_indicators.sma(df['Close'], 10)
            df['MA20'] = self.tech_indicators.sma(df['Close'], 20)
            df['MA60'] = self.tech_indicators.sma(df['Close'], 60)

            # 計算 EMA
            df['EMA12'] = self.tech_indicators.ema(df['Close'], 12)
            df['EMA26'] = self.tech_indicators.ema(df['Close'], 26)

            # 計算 RSI
            df['RSI'] = self.tech_indicators.rsi(df['Close'], 14)

            # 計算 MACD
            macd, macd_signal, macd_hist = self.tech_indicators.macd(df['Close'])
            df['MACD'] = macd
            df['MACD_signal'] = macd_signal
            df['Histogram'] = macd_hist

            # 計算布林帶
            bb_upper, bb_middle, bb_lower = self.tech_indicators.bollinger_bands(df['Close'])
            df['BB_upper'] = bb_upper
            df['BB_middle'] = bb_middle
            df['BB_lower'] = bb_lower

            # 計算 KD 指標
            k_percent, d_percent = self.tech_indicators.stochastic(
                df['High'], df['Low'], df['Close']
            )
            df['K'] = k_percent
            df['D'] = d_percent

            # 計算威廉指標
            df['Williams_R'] = self.tech_indicators.williams_r(
                df['High'], df['Low'], df['Close']
            )

            # 計算成交量移動平均
            df['Volume_MA5'] = self.tech_indicators.sma(df['Volume'], 5)
            df['Volume_MA20'] = self.tech_indicators.sma(df['Volume'], 20)

            # 計算價格變化率
            df['Price_Change'] = df['Close'].pct_change()
            df['Price_Change_5d'] = df['Close'].pct_change(periods=5)

            logger.info("技術指標計算完成")
            return df

        except Exception as e:
            logger.error(f"計算技術指標時發生錯誤: {e}")
            traceback.print_exc()
            return df

    def analyze_trend(self, df: pd.DataFrame) -> Dict[str, Any]:
        """趨勢分析 - 完全修復版本"""
        try:
            if df.empty or len(df) < 20:
                return {'trend': '無法判斷', 'strength': 0, 'score': 50}

            # 確保必要的欄位存在
            required_columns = ['Close', 'MA5', 'MA20']
            missing_columns = [col for col in required_columns if col not in df.columns]
            if missing_columns:
                logger.warning(f"缺少必要的列進行趨勢分析: {missing_columns}")
                return {'trend': '無法判斷', 'strength': 0, 'score': 50}

            # 獲取最新數據，確保沒有 NaN
            latest_data = df.dropna().tail(10)
            if latest_data.empty:
                return {'trend': '無法判斷', 'strength': 0, 'score': 50}

            # 使用 .iloc[-1] 獲取標量值，避免 Series 布林值問題
            current_price = float(latest_data['Close'].iloc[-1])
            ma5_current = float(latest_data['MA5'].iloc[-1]) if not pd.isna(latest_data['MA5'].iloc[-1]) else current_price
            ma20_current = float(latest_data['MA20'].iloc[-1]) if not pd.isna(latest_data['MA20'].iloc[-1]) else current_price

            # 計算價格變化 - 使用標量運算
            price_change_5d = 0
            price_change_10d = 0

            if len(latest_data) >= 6:
                old_price_5d = float(latest_data['Close'].iloc[-6])
                price_change_5d = (current_price - old_price_5d) / old_price_5d * 100

            if len(df) >= 11:
                old_price_10d = float(df['Close'].iloc[-11])
                price_change_10d = (current_price - old_price_10d) / old_price_10d * 100

            # 趨勢判斷邏輯 - 使用標量比較
            trend_score = 50  # 中性起始分數
            trend = '盤整'

            # MA 排列判斷
            if current_price > ma5_current and ma5_current > ma20_current:
                trend_score += 15
                trend = '上漲'
            elif current_price < ma5_current and ma5_current < ma20_current:
                trend_score -= 15
                trend = '下跌'

            # 價格變化判斷
            if price_change_5d > 3:
                trend_score += 10
            elif price_change_5d < -3:
                trend_score -= 10

            if price_change_10d > 5:
                trend_score += 10
            elif price_change_10d < -5:
                trend_score -= 10

            # 計算趨勢強度
            if 'MA60' in df.columns and not pd.isna(latest_data['MA60'].iloc[-1]):
                ma60_current = float(latest_data['MA60'].iloc[-1])
                if current_price > ma60_current:
                    trend_score += 5
                else:
                    trend_score -= 5

            # 限制分數範圍
            trend_score = max(0, min(100, trend_score))

            # 計算強度
            strength = abs(price_change_5d) + abs(price_change_10d)

            return {
                'trend': trend,
                'strength': strength,
                'score': trend_score,
                'price_change_5d': price_change_5d,
                'price_change_10d': price_change_10d,
                'current_price': current_price,
                'ma5': ma5_current,
                'ma20': ma20_current
            }

        except Exception as e:
            logger.error(f"趨勢分析時發生錯誤: {e}")
            traceback.print_exc()
            return {'trend': '無法判斷', 'strength': 0, 'score': 50}

    def analyze_volume(self, df: pd.DataFrame) -> Dict[str, Any]:
        """成交量分析 - 完全修復版本"""
        try:
            if df.empty or len(df) < 20:
                return {'volume_trend': '無法判斷', 'score': 50}

            # 確保 Volume 欄位存在且為數值型
            if 'Volume' not in df.columns:
                logger.warning("缺少 Volume 欄位")
                return {'volume_trend': '無法判斷', 'score': 50}

            # 清理數據
            df_clean = df.dropna(subset=['Volume', 'Close']).copy()
            if len(df_clean) < 10:
                return {'volume_trend': '無法判斷', 'score': 50}

            # 轉換為數值型並移除異常值
            df_clean['Volume'] = pd.to_numeric(df_clean['Volume'], errors='coerce')
            df_clean['Close'] = pd.to_numeric(df_clean['Close'], errors='coerce')
            df_clean = df_clean.dropna(subset=['Volume', 'Close'])

            if df_clean.empty:
                return {'volume_trend': '無法判斷', 'score': 50}

            # 計算成交量移動平均
            df_clean['Volume_MA5'] = df_clean['Volume'].rolling(window=5, min_periods=1).mean()
            df_clean['Volume_MA20'] = df_clean['Volume'].rolling(window=min(20, len(df_clean)), min_periods=1).mean()

            # 獲取最新數據 - 使用標量值
            recent_data = df_clean.tail(10)
            current_volume = float(recent_data['Volume'].iloc[-1])
            volume_ma5 = float(recent_data['Volume_MA5'].iloc[-1])
            volume_ma20 = float(recent_data['Volume_MA20'].iloc[-1])

            # 計算價格變化 - 使用標量值
            price_changes = recent_data['Close'].pct_change().fillna(0)
            latest_price_change = float(price_changes.iloc[-1]) if len(price_changes) > 0 else 0

            # 價量配合分析 - 使用標量比較
            score = 50  # 中性分數
            volume_trend = '正常'

            try:
                # 成交量趨勢判斷
                if current_volume > volume_ma5 * 1.2:  # 放量
                    if latest_price_change > 0.02:  # 價漲量增
                        score += 20
                        volume_trend = '價漲量增'
                    elif latest_price_change < -0.02:  # 價跌量增
                        score -= 15
                        volume_trend = '價跌量增'
                    else:
                        score += 5
                        volume_trend = '放量'

                elif current_volume < volume_ma5 * 0.8:  # 縮量
                    if latest_price_change > 0.02:  # 價漲量縮
                        score -= 10
                        volume_trend = '價漲量縮'
                    elif latest_price_change < -0.02:  # 價跌量縮
                        score += 10
                        volume_trend = '價跌量縮'
                    else:
                        volume_trend = '縮量'

                # 檢查連續放量 - 使用 numpy 陣列避免 Series 問題
                recent_volumes = recent_data['Volume'].tail(3).values
                if len(recent_volumes) >= 3:
                    high_volume_count = np.sum(recent_volumes > volume_ma20 * 1.5)
                    if high_volume_count >= 2:
                        score += 10

                # 限制分數範圍
                score = max(0, min(100, score))

            except Exception as e:
                logger.warning(f"價量配合分析時發生錯誤: {e}")
                score = 50
                volume_trend = '無法判斷'

            return {
                'volume_trend': volume_trend,
                'score': score,
                'current_volume': current_volume,
                'volume_ma5': volume_ma5,
                'volume_ma20': volume_ma20,

                'volume_ratio': current_volume / volume_ma20 if volume_ma20 > 0 else 1
            }

        except Exception as e:
            logger.error(f"成交量分析時發生錯誤: {e}")
            traceback.print_exc()
            return {'volume_trend': '無法判斷', 'score': 50}

    def analyze_technical_indicators(self, df: pd.DataFrame) -> Dict[str, Any]:
        """技術指標分析 - 完全修復版本"""
        try:
            if df.empty:
                return {'rsi_signal': '中性', 'macd_signal': '中性', 'kd_signal': '中性', 'bb_signal': '中性', 'score': 50}

            # 獲取最新數據
            latest = df.dropna().tail(1)
            if latest.empty:
                return {'rsi_signal': '中性', 'macd_signal': '中性', 'kd_signal': '中性', 'bb_signal': '中性', 'score': 50}

            score = 50
            signals = {}

            # RSI 分析 - 使用標量值
            if 'RSI' in latest.columns and not pd.isna(latest['RSI'].iloc[0]):
                rsi_value = float(latest['RSI'].iloc[0])
                if rsi_value < 30:
                    signals['rsi_signal'] = '超賣'
                    score += 15
                elif rsi_value > 70:
                    signals['rsi_signal'] = '超買'
                    score -= 15
                elif rsi_value < 50:
                    signals['rsi_signal'] = '偏空'
                    score -= 5
                elif rsi_value > 50:
                    signals['rsi_signal'] = '偏多'
                    score += 5
                else:
                    signals['rsi_signal'] = '中性'
                signals['rsi_value'] = rsi_value
            else:
                signals['rsi_signal'] = '無數據'
                signals['rsi_value'] = 50

            # MACD 分析 - 使用標量值
            if all(col in latest.columns for col in ['MACD', 'MACD_signal']) and \
               not pd.isna(latest['MACD'].iloc[0]) and not pd.isna(latest['MACD_signal'].iloc[0]):
                macd_value = float(latest['MACD'].iloc[0])
                signal_value = float(latest['MACD_signal'].iloc[0])

                if macd_value > signal_value and macd_value > 0:
                    signals['macd_signal'] = '強力買進'
                    score += 15
                elif macd_value > signal_value:
                    signals['macd_signal'] = '買進'
                    score += 10
                elif macd_value < signal_value and macd_value < 0:
                    signals['macd_signal'] = '強力賣出'
                    score -= 15
                else:
                    signals['macd_signal'] = '賣出'
                    score -= 10

                signals['macd_value'] = macd_value
                signals['macd_signal_value'] = signal_value
            else:
                signals['macd_signal'] = '無數據'
                signals['macd_value'] = 0
                signals['macd_signal_value'] = 0

            # KD 分析 - 使用標量值
            if all(col in latest.columns for col in ['K', 'D']) and \
               not pd.isna(latest['K'].iloc[0]) and not pd.isna(latest['D'].iloc[0]):
                k_value = float(latest['K'].iloc[0])
                d_value = float(latest['D'].iloc[0])

                if k_value < 20 and d_value < 20:
                    signals['kd_signal'] = '超賣'
                    score += 12
                elif k_value > 80 and d_value > 80:
                    signals['kd_signal'] = '超買'
                    score -= 12
                elif k_value > d_value:
                    signals['kd_signal'] = '買進'
                    score += 8
                else:
                    signals['kd_signal'] = '賣出'
                    score -= 8

                signals['k_value'] = k_value
                signals['d_value'] = d_value
            else:
                signals['kd_signal'] = '無數據'
                signals['k_value'] = 50
                signals['d_value'] = 50

            # 布林帶分析 - 使用標量值
            if all(col in latest.columns for col in ['Close', 'BB_upper', 'BB_lower', 'BB_middle']) and \
               all(not pd.isna(latest[col].iloc[0]) for col in ['Close', 'BB_upper', 'BB_lower', 'BB_middle']):
                close_price = float(latest['Close'].iloc[0])
                bb_upper = float(latest['BB_upper'].iloc[0])
                bb_lower = float(latest['BB_lower'].iloc[0])
                bb_middle = float(latest['BB_middle'].iloc[0])

                if close_price <= bb_lower:
                    signals['bb_signal'] = '超賣'
                    score += 10
                elif close_price >= bb_upper:
                    signals['bb_signal'] = '超買'
                    score -= 10
                elif close_price > bb_middle:
                    signals['bb_signal'] = '偏多'
                    score += 5
                else:
                    signals['bb_signal'] = '偏空'
                    score -= 5

                signals['bb_position'] = (close_price - bb_lower) / (bb_upper - bb_lower) if (bb_upper - bb_lower) > 0 else 0.5
            else:
                signals['bb_signal'] = '無數據'
                signals['bb_position'] = 0.5

            # 威廉指標分析
            if 'Williams_R' in latest.columns and not pd.isna(latest['Williams_R'].iloc[0]):
                wr_value = float(latest['Williams_R'].iloc[0])
                if wr_value < -80:
                    score += 8
                elif wr_value > -20:
                    score -= 8
                signals['williams_r'] = wr_value
            else:
                signals['williams_r'] = -50

            # 限制分數範圍
            score = max(0, min(100, score))

            return {
                **signals,
                'score': score
            }

        except Exception as e:
            logger.error(f"技術指標分析時發生錯誤: {e}")
            traceback.print_exc()
            return {'rsi_signal': '中性', 'macd_signal': '中性', 'kd_signal': '中性', 'bb_signal': '中性', 'score': 50}

    def analyze_stock(self, stock_info: Dict) -> Dict[str, Any]:
        """完整股票分析"""
        try:
            stock_id = stock_info.get('stock_id', '')
            stock_name = stock_info.get('stock_name', '')
            df = stock_info.get('data')

            if df is None or df.empty:
                logger.warning(f"{stock_name} 無數據可分析")
                return {'success': False, 'error': '無數據'}

            # 計算技術指標
            df_with_indicators = self.calculate_technical_indicators(df)

            # 各項分析
            trend_analysis = self.analyze_trend(df_with_indicators)
            volume_analysis = self.analyze_volume(df_with_indicators)
            technical_analysis = self.analyze_technical_indicators(df_with_indicators)

            # 綜合評分計算 - 使用標量運算
            trend_score = float(trend_analysis.get('score', 50))
            volume_score = float(volume_analysis.get('score', 50))
            technical_score = float(technical_analysis.get('score', 50))

            # 加權計算綜合分數
            combined_score = (trend_score * 0.4 + volume_score * 0.3 + technical_score * 0.3)

            # 獲取當前價格和基本資訊
            latest_data = df_with_indicators.dropna().tail(1)
            if not latest_data.empty:
                current_price = float(latest_data['Close'].iloc[0])
                current_volume = float(latest_data['Volume'].iloc[0]) if 'Volume' in latest_data.columns else 0
            else:
                current_price = 0
                current_volume = 0

            # 生成投資建議
            recommendation = self.generate_recommendation(combined_score, trend_analysis, technical_analysis)

            result = {
                'success': True,
                'stock_id': stock_id,
                'stock_name': stock_name,
                'current_price': current_price,
                'current_volume': current_volume,
                'combined_score': combined_score,
                'trend_analysis': trend_analysis,
                'volume_analysis': volume_analysis,
                'technical_analysis': technical_analysis,
                'recommendation': recommendation,
                'data_df': df_with_indicators,
                'industry': stock_info.get('industry', ''),
                'market': stock_info.get('market', '')
            }

            return result

        except Exception as e:
            logger.error(f"分析股票 {stock_info.get('stock_name', '')} 時發生錯誤: {e}")
            traceback.print_exc()
            return {'success': False, 'error': str(e)}

    def generate_recommendation(self, score: float, trend_analysis: Dict, technical_analysis: Dict) -> Dict[str, str]:
        """生成投資建議"""
        try:
            # 基於綜合分數的建議
            if score >= 75:
                action = "強力買入"
                reason = "多項指標顯示強勁上漲趨勢"
            elif score >= 65:
                action = "買入"
                reason = "技術面偏多，建議逢低買入"
            elif score >= 55:
                action = "持有"
                reason = "盤整格局，可持有觀望"
            elif score >= 45:
                action = "觀望"
                reason = "趨勢不明確，建議觀望"
            elif score >= 35:
                action = "減碼"
                reason = "技術面轉弱，建議減碼"
            else:
                action = "賣出"
                reason = "多項指標顯示下跌趨勢"

            # 加入特殊情況判斷
            rsi_signal = technical_analysis.get('rsi_signal', '中性')
            trend = trend_analysis.get('trend', '盤整')

            if rsi_signal == '超賣' and trend != '下跌':
                action = "逢低買入"
                reason += "，RSI顯示超賣"
            elif rsi_signal == '超買' and score < 70:
                action = "獲利了結"
                reason += "，RSI顯示超買"

            return {
                'action': action,
                'reason': reason,
                'confidence': min(100, abs(score - 50) * 2)  # 信心度
            }

        except Exception as e:
            logger.error(f"生成建議時發生錯誤: {e}")
            return {'action': '觀望', 'reason': '分析異常', 'confidence': 0}

    def advanced_multi_criteria_filter(self, analysis_results: Dict) -> Dict:
        """進階多重條件篩選"""
        try:
            if not analysis_results:
                logger.warning("沒有分析結果可供篩選")
                return {}

            filtered_stocks = {}

            for stock_id, result in analysis_results.items():
                try:
                    # 基本條件檢查
                    if not result.get('success', False):
                        continue

                    score = float(result.get('combined_score', 0))
                    trend_analysis = result.get('trend_analysis', {})
                    technical_analysis = result.get('technical_analysis', {})
                    volume_analysis = result.get('volume_analysis', {})

                    # 篩選條件
                    conditions_met = 0
                    total_conditions = 6

                    # 1. 綜合分數條件
                    if score >= 55:
                        conditions_met += 1

                    # 2. 趨勢條件
                    trend = trend_analysis.get('trend', '')
                    if trend in ['上漲', '盤整']:
                        conditions_met += 1

                    # 3. RSI 條件 - 避免極端超買
                    rsi_value = technical_analysis.get('rsi_value', 50)
                    if 20 <= rsi_value <= 75:  # 不要極端超買或超賣
                        conditions_met += 1

                    # 4. MACD 條件
                    macd_signal = technical_analysis.get('macd_signal', '')
                    if macd_signal in ['買進', '強力買進']:
                        conditions_met += 1

                    # 5. 成交量條件
                    volume_trend = volume_analysis.get('volume_trend', '')
                    if volume_trend in ['價漲量增', '放量', '正常']:
                        conditions_met += 1

                    # 6. 價格趨勢條件
                    price_change_5d = trend_analysis.get('price_change_5d', 0)
                    if price_change_5d >= -5:  # 近期跌幅不超過5%
                        conditions_met += 1

                    # 至少滿足 4/6 條件才納入
                    if conditions_met >= 4:
                        filtered_stocks[stock_id] = result
                        logger.info(f"{result.get('stock_name', '')} 通過篩選 ({conditions_met}/{total_conditions})")

                except Exception as e:
                    logger.error(f"篩選股票 {stock_id} 時發生錯誤: {e}")
                    continue

            # 按綜合分數排序
            sorted_stocks = dict(sorted(
                filtered_stocks.items(),
                key=lambda x: x[1].get('combined_score', 0),
                reverse=True
            ))

            logger.info(f"篩選完成，共 {len(sorted_stocks)} 支股票通過條件")
            return sorted_stocks

        except Exception as e:
            logger.error(f"進階篩選時發生錯誤: {e}")
            return {}

    def print_advanced_filtered_stocks(self, filtered_results):
        """輸出篩選結果"""
        try:
            # 先檢查是否有資料
            if not filtered_results:
                print("\n".join([
                    "❌ 未找到符合條件的股票",
                    "💡 建議調整篩選條件或增加分析股票數量",
                    "📱 空結果通知已發送"
                ]))
                return

            print("\n" + "="*80)
            print("🎯 最終篩選優質股票清單 🎯".center(80))
            print("="*80)
            print(f"{'排名':<4}{'代碼':<7}{'名稱':<12}{'分數':<7}{'價格':<10}{'建議':<10}{'產業':<15}")
            print("-" * 80)

            # 輸出每支股票的資訊
            for i, (stock_id, stock) in enumerate(list(filtered_results.items())[:10], 1):
                print(
                    f"{i:<4}"
                    f"{stock_id:<7}"
                    f"{stock['stock_name'][:10]:<12}"
                    f"{stock['combined_score']:<7.1f}"
                    f"{stock['current_price']:<10.2f}"
                    f"{stock['recommendation']['action'][:8]:<10}"
                    f"{stock.get('industry', '')[:12]:<15}"
                )
            print("=" * 80)

            # 輸出詳細報告
            print("\n" + "="*80)
            print("📊 詳細分析報告 📊".center(80))
            print("="*80)

            for i, (stock_id, stock) in enumerate(list(filtered_results.items())[:5], 1):
                print(f"\n報告 #{i}: {stock_id} - {stock['stock_name']}")
                print("-" * 50)
                print(f"綜合評分: {stock['combined_score']:.1f}")
                print(f"最新收盤價: {stock['current_price']:.2f} 元")
                print(f"產業類別: {stock.get('industry', '未分類')}")
                print(f"推薦行動: {stock['recommendation']['action']}")
                print(f"信心度: {stock['recommendation']['confidence']:.1f}%")
                print(f"推薦理由: {stock['recommendation']['reason']}")

                # 技術指標詳情
                tech = stock['technical_analysis']
                trend = stock['trend_analysis']
                print(f"趨勢: {trend.get('trend', '盤整')} (5日漲跌: {trend.get('price_change_5d', 0):.2f}%)")
                print(f"RSI: {tech.get('rsi_value', 50):.1f} ({tech.get('rsi_signal', '中性')})")
                print(f"MACD: {tech.get('macd_signal', '中性')}")
                print(f"KD: K={tech.get('k_value', 50):.1f}, D={tech.get('d_value', 50):.1f}")
                print("-" * 50)

            print("=" * 80)
            print("\n📱 分析結果已發送到 Telegram 和 Discord")

        except Exception as e:
            logger.error(f"輸出結果時發生錯誤: {e}")
            print("\n".join([
                "❌ 未找到符合條件的股票",
                "💡 建議調整篩選條件或增加分析股票數量",
                "📱 空結果通知已發送"
            ]))

    def create_stock_chart(self, stock_data: Dict, save_path: str = None) -> str:
        """創建股票圖表 - 增強版本"""
        try:
            df = stock_data['data_df'].copy()
            stock_name = stock_data['stock_name']
            stock_id = stock_data['stock_id']

            if df.empty or len(df) < 20:
                logger.warning(f"數據不足，無法繪製 {stock_name} 圖表")
                return None

            # 準備數據
            df = df.dropna().tail(60)  # 取最近60天
            df.index = pd.to_datetime(df.index)

            # 設定圖表樣式
            mc = mpf.make_marketcolors(up='r', down='g', inherit=True)
            s = mpf.make_mpf_style(marketcolors=mc, gridstyle='-', y_on_right=False)

            # 添加技術指標
            apds = []

            # 添加移動平均線
            if all(col in df.columns for col in ['MA5', 'MA20', 'MA60']):
                apds.append(mpf.make_addplot(df['MA5'], color='orange', width=1.5, alpha=0.8))
                apds.append(mpf.make_addplot(df['MA20'], color='blue', width=1.5, alpha=0.8))
                apds.append(mpf.make_addplot(df['MA60'], color='purple', width=1, alpha=0.7))

            # 添加布林帶
            if all(col in df.columns for col in ['BB_upper', 'BB_lower']):
                apds.append(mpf.make_addplot(df['BB_upper'], color='gray', width=0.8, alpha=0.5, linestyle='--'))
                apds.append(mpf.make_addplot(df['BB_lower'], color='gray', width=0.8, alpha=0.5, linestyle='--'))

            # 添加成交量
            if 'Volume' in df.columns:
                apds.append(mpf.make_addplot(df['Volume'], panel=1, type='bar', alpha=0.7, color='lightblue'))
                if 'Volume_MA20' in df.columns:
                    apds.append(mpf.make_addplot(df['Volume_MA20'], panel=1, color='red', width=1))

            # 添加 RSI
            if 'RSI' in df.columns:
                apds.append(mpf.make_addplot(df['RSI'], panel=2, color='purple', width=1.2))
                # RSI 超買超賣線
                rsi_70 = pd.Series([70] * len(df), index=df.index)
                rsi_30 = pd.Series([30] * len(df), index=df.index)
                apds.append(mpf.make_addplot(rsi_70, panel=2, color='red', width=0.8, linestyle='--', alpha=0.7))
                apds.append(mpf.make_addplot(rsi_30, panel=2, color='green', width=0.8, linestyle='--', alpha=0.7))

            # 設定檔案路徑
            if save_path is None:
                timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
                save_path = os.path.join(CHARTS_DIR, f"{stock_id}_{stock_name}_{timestamp}.png")

            # 獲取分析結果用於標題
            current_price = stock_data.get('current_price', 0)
            combined_score = stock_data.get('combined_score', 0)
            recommendation = stock_data.get('recommendation', {}).get('action', '觀望')

            # 繪製圖表
            mpf.plot(
                df,
                type='candle',
                style=s,
                addplot=apds,
                volume=False,  # 我們手動添加成交量
                title=f"{stock_name} ({stock_id}) - 價格: {current_price:.2f} | 評分: {combined_score:.1f} | 建議: {recommendation}",
                ylabel='價格 (TWD)',
                ylabel_lower='成交量',
                figsize=(14, 10),
                savefig=dict(fname=save_path, dpi=150, bbox_inches='tight'),
                tight_layout=True,
                panel_ratios=(3, 1, 1)  # 主圖:成交量:RSI = 3:1:1
            )

            logger.info(f"成功創建 {stock_name} 圖表: {save_path}")
            return save_path

        except Exception as e:
            logger.error(f"創建圖表時發生錯誤: {e}")
            traceback.print_exc()
            return None

    async def run_analysis(self):
        """執行完整分析流程 - 整合通知功能"""
        try:
            logger.info("開始股票分析流程...")

            # 獲取股票清單
            stocks_df = self.get_taiwan_stocks()
            if stocks_df.empty:
                logger.error("無法獲取股票清單")
                return

            # 限制分析數量
            stocks_to_analyze = stocks_df.head(2000)  # 分析前50支股票
            logger.info(f"將分析 {len(stocks_to_analyze)} 支股票")

            # 獲取股票數據
            analysis_results = {}
            for _, stock in stocks_to_analyze.iterrows():
                try:
                    symbol = stock['yahoo_symbol']
                    logger.info(f"正在獲取 {stock['stock_name']} ({symbol}) 數據...")

                    df = self.fetch_yfinance_data(symbol)
                    if df is not None and not df.empty:
                        stock_info = stock.to_dict()
                        stock_info['data'] = df

                        # 分析股票
                        result = self.analyze_stock(stock_info)
                        if result.get('success', False):
                            analysis_results[stock['stock_id']] = result
                            logger.info(f"完成 {stock['stock_name']} 分析，評分: {result.get('combined_score', 0):.1f}")

                    # 避免請求過於頻繁
                    await asyncio.sleep(0.5)

                except Exception as e:
                    logger.error(f"分析 {stock['stock_name']} 時發生錯誤: {e}")
                    continue

            # 進階篩選
            filtered_results = self.advanced_multi_criteria_filter(analysis_results)

            # 輸出結果
            self.print_advanced_filtered_stocks(filtered_results)

            # 生成圖表
            chart_files = []
            for stock_id, result in list(filtered_results.items())[:5]:  # 只為前5名生成圖表
                try:
                    chart_path = self.create_stock_chart(result)
                    if chart_path and os.path.exists(chart_path):
                        chart_files.append(chart_path)
                except Exception as e:
                    logger.error(f"生成 {result.get('stock_name', '')} 圖表時發生錯誤: {e}")

            # 發送通知
            try:
                async with aiohttp.ClientSession() as session:
                    # 格式化通知訊息
                    message = format_notification_message(filtered_results)

                    # 發送通知和圖表
                    await send_notification(session, message, chart_files)
                    logger.info("通知發送完成")

            except Exception as e:
                logger.error(f"發送通知時發生錯誤: {e}")

            if chart_files:
                logger.info(f"成功生成 {len(chart_files)} 個圖表檔案")
                print(f"\n📊 圖表檔案已保存至: {CHARTS_DIR}")
                for chart_file in chart_files:
                    print(f"   - {os.path.basename(chart_file)}")
            else:
                logger.warning("未生成任何圖表檔案")

            return filtered_results, chart_files

        except Exception as e:
            logger.error(f"執行分析時發生錯誤: {e}")
            traceback.print_exc()
            return {}, []

# 主程式
async def main():
    """主程式 - 整合通知功能"""
    try:
        print("🚀 開始台股技術分析...")
        print("📊 使用純 pandas/numpy 計算技術指標")
        print("📱 將自動發送分析結果到 Telegram 和 Discord")
        print("-" * 50)

        analyzer = StockAnalyzer()
        results, charts = await analyzer.run_analysis()

        if results:
            print(f"\n✅ 分析完成！找到 {len(results)} 支優質股票")
            print(f"📊 生成了 {len(charts)} 個圖表檔案")
            print(f"📱 通知已發送到 Telegram 和 Discord")

            # 顯示詳細分析結果
            print("\n" + "="*60)
            print("📈 詳細分析結果")
            print("="*60)

            for i, (stock_id, result) in enumerate(list(results.items())[:3], 1):
                print(f"\n{i}. {result['stock_name']} ({stock_id})")
                print(f"   綜合評分: {result['combined_score']:.1f}")
                print(f"   當前價格: {result['current_price']:.2f}")
                print(f"   投資建議: {result['recommendation']['action']}")
                print(f"   建議理由: {result['recommendation']['reason']}")

                # 技術指標詳情
                tech = result['technical_analysis']
                trend = result['trend_analysis']
                print(f"   趨勢: {trend.get('trend', '盤整')} (5日漲跌: {trend.get('price_change_5d', 0):.2f}%)")
                print(f"   RSI: {tech.get('rsi_value', 50):.1f} ({tech.get('rsi_signal', '中性')})")
                print(f"   MACD: {tech.get('macd_signal', '中性')}")
                print(f"   KD: K={tech.get('k_value', 50):.1f}, D={tech.get('d_value', 50):.1f}")

        else:
            print("\n❌ 未找到符合條件的股票")
            print("💡 建議調整篩選條件或增加分析股票數量")

            # 即使沒有結果也發送通知
            try:
                async with aiohttp.ClientSession() as session:
                    message = format_notification_message({})
                    await send_notification(session, message)
                    print("📱 空結果通知已發送")
            except Exception as e:
                logger.error(f"發送空結果通知時發生錯誤: {e}")

    except Exception as e:
        logger.error(f"主程式執行錯誤: {e}")
        traceback.print_exc()
        # 發送錯誤通知
        try:
            async with aiohttp.ClientSession() as session:
                error_message = f"""
🚨 台股分析程式執行錯誤

❌ 錯誤訊息: {str(e)}
⏰ 時間: {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}

請檢查程式狀態並重新執行。
"""
                await send_notification(session, error_message)
                logger.info("錯誤通知已發送")
        except Exception as notify_error:
            logger.error(f"發送錯誤通知失敗: {notify_error}")

# 執行程式
if __name__ == "__main__":
    asyncio.run(main())

上面可發訊discord有訊息且有圖，telgram有訊息且有圖，終端有報告

In [ ]:
!pip install mplfinance chineseize_matplotlib yfinance pandas numpy matplotlib plotly discord-webhook requests aiohttp python-telegram-bot nest-asyncio -q

In [ ]:

!pip install mplfinance chineseize_matplotlib yfinance pandas numpy matplotlib plotly discord-webhook requests aiohttp python-telegram-bot nest-asyncio -q
# -*- coding: utf-8 -*-
"""
台股多股技術分析與篩選工具
=======================
功能：
- 自動獲取台股清單
- 批次下載歷史數據
- 技術指標分析
- 綜合評分排名
- 自動通知推送
- 圖表生成與報告匯出

作者：股票分析系統
版本：v2.1
更新日期：2025-08-08
"""

# ==============================================================================
# 1. 基礎 Python 標準庫
# ==============================================================================
import os
import sys
import json
import time
import traceback
import platform
import re
import glob
import asyncio
import warnings
from io import StringIO
from datetime import datetime, timedelta
from typing import List, Dict, Any, Optional, Tuple
import logging
import random
# ==============================================================================
# 2. 第三方套件 - 數據處理與科學計算
# ==============================================================================
import pandas as pd
import numpy as np

# ==============================================================================
# 3. 第三方套件 - 網路請求與異步處理
# ==============================================================================
import requests
import aiohttp
import nest_asyncio

# 應用 nest_asyncio 以支援 Jupyter 環境
nest_asyncio.apply()

# ==============================================================================
# 4. 第三方套件 - 金融數據源
# ==============================================================================
import yfinance as yf

# ==============================================================================
# 5. 第三方套件 - 數據可視化
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import font_manager
import mplfinance as mpf
import plotly.graph_objects as go
from pathlib import Path

# ==============================================================================
# 6. 第三方套件 - 時區與通知
# ==============================================================================
import pytz
from discord_webhook import DiscordWebhook
import telegram
# ==============================================================================
# 7. 中文字體支援
# ==============================================================================
try:
    import chineseize_matplotlib
    chineseize_matplotlib.chineseize()
    print("✅ 已載入 chineseize_matplotlib 中文字體支援")
except ImportError:
    print("⚠️ 未安裝 chineseize_matplotlib，將使用自定義字體設定")

# ==============================================================================
# 8. 全域設定
# ==============================================================================
# 警告過濾
warnings.filterwarnings('ignore')

# 時區設定
taipei_tz = pytz.timezone('Asia/Taipei')

# 目錄結構設定
CACHE_DIR = 'cache'
RESULTS_DIR = 'results'
CHARTS_DIR = os.path.join(RESULTS_DIR, 'charts')
STOCK_LIST_PATH = os.path.join(CACHE_DIR, 'stock_list.csv')
HISTORY_DATA_CACHE_DIR = os.path.join(CACHE_DIR, 'history_data')

# 初始化目錄
for directory in [CACHE_DIR, RESULTS_DIR, CHARTS_DIR, HISTORY_DATA_CACHE_DIR]:
    os.makedirs(directory, exist_ok=True)

# ==============================================================================
# 9. 通訊與通知設定
# ==============================================================================
# 建議將來使用環境變數或 .env 檔案管理密鑰，以策安全
TELEGRAM_TOKEN = "7902318521:AAEYoDMqwfHabI7L1SRiE4z33aFay42-VGE"
TELEGRAM_CHAT_ID = [879781796, 8113868436]  # 支援多用戶通知
DISCORD_WEBHOOK_URL = "https://discord.com/api/webhooks/1362715080734802102/Jma7A3VhEQrrRxIX_JW2l6rATjAZXsGXGfnJuAMqmS1QvqG_2ptg3vr_nsVnuV_PlnBl"
# 新增此行：全域通知開關
# 設定為 True 以啟用 Telegram 和 Discord 通知，設定為 False 則停用所有通知

MESSAGING_AVAILABLE = True
# ==============================================================================
# 10. 日誌系統設定
# ==============================================================================
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=[
        logging.FileHandler("stock_analysis.log", encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# ==============================================================================
# 11. 字體與環境設定函式
# ==============================================================================
def set_chinese_font():
    """
    自動設定適合當前作業系統的中文字體，用於 Matplotlib 顯示。

    支援的作業系統：
    - Windows: Microsoft JhengHei, Microsoft YaHei, SimHei
    - macOS: PingFang HK, PingFang SC, Heiti TC
    - Linux: Noto Sans CJK / WenQuanYi Zen Hei
    """
    try:
        system = platform.system()
        font_name = ""

        if system == 'Windows':
            # Windows 系統字體設定（包含繁體中文優先）
            font_candidates = [
                'Microsoft JhengHei',  # 微軟正黑體（繁體中文）
                'Microsoft YaHei',     # 微軟雅黑（簡體中文）
                'Arial Unicode MS',    # 萬國碼字體
                'SimHei',              # 黑體
                'KaiTi'                # 楷體
            ]

            for font in font_candidates:
                try:
                    # 測試字體是否可用
                    test_fig, test_ax = plt.subplots(figsize=(1, 1))
                    test_ax.text(0.5, 0.5, '測試', fontname=font)
                    plt.close(test_fig)

                    plt.rcParams['font.sans-serif'] = [font]
                    font_name = font
                    break
                except:
                    continue

        elif system == 'Darwin':  # macOS
            # macOS 系統字體設定
            font_candidates = [
                'PingFang HK',      # 蘋方-香港
                'PingFang SC',      # 蘋方-簡體中文
                'PingFang TC',      # 蘋方-繁體中文
                'Heiti TC',         # 黑體-繁體中文
                'Heiti SC',         # 黑體-簡體中文
                'STHeiti'           # 華文黑體
            ]

            for font in font_candidates:
                try:
                    plt.rcParams['font.sans-serif'] = [font]
                    font_name = font
                    break
                except:
                    continue

        else:  # Linux 和其他系統
            # Linux 系統字體路徑
            font_paths = [
                '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc',
                '/usr/share/fonts/truetype/wqy/wqy-zenhei.ttc',
                '/usr/share/fonts/truetype/arphic/ukai.ttc',
                '/usr/share/fonts/truetype/arphic/uming.ttc',
                '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf'
            ]

            found_font_path = None
            for path in font_paths:
                if os.path.exists(path):
                    found_font_path = path
                    break

            if found_font_path:
                try:
                    font_manager.fontManager.addfont(found_font_path)
                    font_prop = font_manager.FontProperties(fname=found_font_path)
                    font_name = font_prop.get_name()
                    plt.rcParams['font.sans-serif'] = [font_name]
                except Exception as e:
                    logger.warning(f"載入字體失敗: {e}")
                    font_name = "DejaVu Sans"
            else:
                print("⚠️ 未在常見路徑找到中文字體，圖表中的中文可能無法正常顯示。")
                font_name = "DejaVu Sans"

        # 設定 matplotlib 參數
        plt.rcParams['axes.unicode_minus'] = False  # 正確顯示負號

        # 設定字體大小
        plt.rcParams['font.size'] = 10
        plt.rcParams['axes.titlesize'] = 12
        plt.rcParams['axes.labelsize'] = 10
        plt.rcParams['xtick.labelsize'] = 9
        plt.rcParams['ytick.labelsize'] = 9
        plt.rcParams['legend.fontsize'] = 9

        # 設定圖表品質
        plt.rcParams['figure.dpi'] = 100
        plt.rcParams['savefig.dpi'] = 150
        plt.rcParams['savefig.bbox'] = 'tight'

        if font_name and font_name != "DejaVu Sans":
            print(f"✅ 已設定圖表字體為: {font_name} ({system})")
        else:
            print(f"⚠️ 使用預設字體: {font_name}")
            plt.rcParams['font.sans-serif'] = ['DejaVu Sans']

    except Exception as e:
        print(f"❌ 字體配置錯誤: {e}。使用預設字體。")
        plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
        plt.rcParams['axes.unicode_minus'] = False

def check_environment():
    """
    檢查執行環境與相依套件
    """
    print("🔍 檢查執行環境...")
    print(f"Python 版本: {sys.version}")
    print(f"作業系統: {platform.system()} {platform.release()}")
    print(f"當前時區: {taipei_tz}")

    # 檢查重要套件版本
    required_packages = {
        'pandas': pd.__version__,
        'numpy': np.__version__,
        'matplotlib': plt.matplotlib.__version__,
        'requests': requests.__version__,
        'yfinance': getattr(yf, '__version__', 'Unknown'),
        'aiohttp': aiohttp.__version__,
        'pytz': pytz.__version__
    }

    print("\n📦 套件版本:")
    for package, version in required_packages.items():
        print(f"  {package}: {version}")

    # 檢查目錄權限
    print(f"\n📁 工作目錄: {os.getcwd()}")
    directories_status = {
        '快取目錄': (CACHE_DIR, os.access(CACHE_DIR, os.W_OK)),
        '結果目錄': (RESULTS_DIR, os.access(RESULTS_DIR, os.W_OK)),
        '圖表目錄': (CHARTS_DIR, os.access(CHARTS_DIR, os.W_OK))
    }

    for name, (path, writable) in directories_status.items():
        status = '✅' if writable else '❌'
        print(f"{name}: {path} {status}")

    # 檢查網路連線（簡單測試）
    try:
        response = requests.get('https://httpbin.org/status/200', timeout=5)
        network_status = '✅' if response.status_code == 200 else '❌'
    except:
        network_status = '❌'

    print(f"網路連線: {network_status}")
    print("=" * 50)

def get_current_time():
    """
    獲取當前台北時間
    """
    return datetime.now(taipei_tz)

def format_timestamp(dt=None):
    """
    格式化時間戳記
    """
    if dt is None:
        dt = get_current_time()
    return dt.strftime('%Y-%m-%d %H:%M:%S %Z')

# ==============================================================================
# 12. 程式初始化
# ==============================================================================
def initialize_application():
    """
    應用程式初始化
    """
    print("🚀 台股多股技術分析與篩選工具 v2.1")
    print("=" * 50)

    # 檢查環境
    check_environment()

    # 設定中文字體
    set_chinese_font()

    # 記錄啟動時間
    start_time = get_current_time()
    logger.info(f"應用程式啟動 - {format_timestamp(start_time)}")
    logger.info(f"工作目錄: {os.getcwd()}")

    print(f"✅ 初始化完成 - {format_timestamp(start_time)}\n")



# ==============================================================================
# StockAnalyzer 類定義 (與之前相同，無需變更)
# ==============================================================================
class StockAnalyzer:
    def __init__(self):
        self.stocks_df = pd.DataFrame()

    def standardize_columns(self, df: pd.DataFrame) -> pd.DataFrame:
        if not isinstance(df, pd.DataFrame) or df.empty:
            return pd.DataFrame()
        df = df.copy()
        if df.index.name is not None and 'date' in df.index.name.lower():
            df = df.reset_index()
        column_mapping = {
            'Date': 'Date', 'date': 'Date', '日期': 'Date',
            'Open': 'Open', 'open': 'Open', '開盤價': 'Open',
            'High': 'High', 'high': 'High', '最高價': 'High',
            'Low': 'Low', 'low': 'Low', '最低價': 'Low',
            'Close': 'Close', 'close': 'Close', '收盤價': 'Close', 'Adj Close': 'Close',
            'Volume': 'Volume', 'volume': 'Volume', '成交量': 'Volume',
        }
        rename_dict = {col: column_mapping.get(col, col) for col in df.columns}
        df.rename(columns=rename_dict, inplace=True)
        return df

    def get_taiwan_stocks(self, force_update=False):
        if not force_update and os.path.exists(STOCK_LIST_PATH):
            if (time.time() - os.path.getmtime(STOCK_LIST_PATH)) < 86400:
                logger.info("從快取載入股票列表。")
                return pd.read_csv(STOCK_LIST_PATH, dtype={'stock_id': str})

        logger.info("從台灣證交所網站獲取最新股票清單...")
        urls = {
            "上市": "https://isin.twse.com.tw/isin/C_public.jsp?strMode=2",
            "上櫃": "https://isin.twse.com.tw/isin/C_public.jsp?strMode=4"
        }
        all_stocks_df = []
        headers = {'User-Agent': 'Mozilla/5.0'}

        for market_name, url in urls.items():
            try:
                res = requests.get(url, headers=headers, timeout=30)
                res.encoding = 'big5'
                html_dfs = pd.read_html(StringIO(res.text))
                df = html_dfs[0]
                df.columns = df.iloc[0]
                df = df.iloc[1:]
                df['market'] = market_name
                all_stocks_df.append(df)
            except Exception as e:
                logger.error(f"獲取 {market_name} 股票列表失敗: {e}")
                continue

        if not all_stocks_df:
            logger.error("無法從網站獲取任何股票數據，程序終止。")
            return pd.DataFrame()

        df = pd.concat(all_stocks_df, ignore_index=True)
        df[['stock_id', 'stock_name']] = df['有價證券代號及名稱'].str.split(r'\s+', n=1, expand=True)
        df = df[df['stock_id'].str.match(r'^\d{4}$')].copy()
        exclude = ['ETF', 'ETN', 'TDR', '受益', '指數', '購', '牛', '熊', '存託憑證']
        df = df[~df['stock_name'].str.contains('|'.join(exclude), na=False)]
        df['yahoo_symbol'] = df.apply(
            lambda row: f"{row['stock_id']}.TW" if '上市' in row['market'] else f"{row['stock_id']}.TWO", axis=1)

        final_df = df[['stock_id', 'stock_name', 'market', '產業別', 'yahoo_symbol']].rename(columns={'產業別': 'industry'})
        final_df = final_df.drop_duplicates(subset=['stock_id']).reset_index(drop=True)
        final_df.to_csv(STOCK_LIST_PATH, index=False)
        logger.info(f"成功獲取 {len(final_df)} 支股票清單並保存快取。")
        return final_df

    async def fetch_yfinance_data_async(self, stock_id: str, period: str = "1y", interval: str = "1d", retries: int = 3, backoff_factor: float = 0.5):
        for attempt in range(retries):
            try:
                df = yf.download(tickers=stock_id, period=period, interval=interval, progress=False, auto_adjust=True)
                if df.empty:
                    logger.warning(f"[{stock_id}] 無數據返回")
                    return pd.DataFrame()
                if isinstance(df.columns, pd.MultiIndex):
                    df.columns = df.columns.get_level_values(0)
                required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
                for col in required_cols:
                    if col in df.columns:
                        df[col] = pd.to_numeric(df[col], errors='coerce')
                        if df[col].isna().all():
                            logger.warning(f"[{stock_id}] {col} 數據全為 NaN")
                            return pd.DataFrame()
                df = df.dropna(subset=required_cols)
                return df
            except Exception as e:
                if attempt < retries - 1:
                    sleep_time = backoff_factor * (2 ** attempt) + random.uniform(0, 1)
                    logger.warning(f"[{stock_id}] 第 {attempt + 1}/{retries} 次嘗試時發生異常: {e}。等待 {sleep_time:.2f} 秒後重試...")
                    await asyncio.sleep(sleep_time)
                else:
                    logger.error(f"[{stock_id}] 因異常，在 {retries} 次嘗試後獲取數據失敗。")
                    return pd.DataFrame()
        return pd.DataFrame()

    async def fetch_all_stocks(self, stocks_df):
        tasks = [self.fetch_yfinance_data_async(row['yahoo_symbol']) for _, row in stocks_df.iterrows()]
        results = await analyzer.fetch_all_stocks(stocks_df)
        for stock, data_df in zip(stocks_df.itertuples(), results):
            stock_id = stock.stock_id
            if isinstance(data_df, pd.DataFrame) and not data_df.empty and len(data_df) >= 60:
                data_df = analyzer.standardize_columns(data_df.reset_index())
                # Proceed with volume checks and analysis
            else:
                logger.warning(f"[{stock_id}] 數據不足或獲取失敗，跳過。")
                failed_stocks.append(stock_id)
                continue
        return await asyncio.gather(*tasks, return_exceptions=True)

    def fetch_yfinance_data_single_robust(self, stock_id: str, period: str = "1y", interval: str = "1d", retries: int = 3, backoff_factor: float = 0.5, request_timeout: int = 30):
        for attempt in range(retries):
            try:
                df = yf.download(tickers=stock_id, period=period, interval=interval, progress=False, auto_adjust=True, timeout=request_timeout)
                if df.empty:
                    logger.warning(f"[{stock_id}] 無數據返回")
                    return pd.DataFrame()
                # Handle multi-level columns if present
                if isinstance(df.columns, pd.MultiIndex):
                    df.columns = df.columns.get_level_values(0)  # Flatten to first level
                # Ensure all required columns are numeric
                required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
                for col in required_cols:
                    if col in df.columns:
                        df[col] = pd.to_numeric(df[col], errors='coerce')
                        if df[col].isna().all():
                            logger.warning(f"[{stock_id}] {col} 數據全為 NaN")
                            return pd.DataFrame()
                # Drop rows with any NaN in required columns
                df = df.dropna(subset=required_cols)
                if df.empty:
                    logger.warning(f"[{stock_id}] 數據清理後為空")
                    return pd.DataFrame()
                return df
            except Exception as e:
                if attempt < retries - 1:
                    sleep_time = backoff_factor * (2 ** attempt) + random.uniform(0, 1)
                    logger.warning(f"[{stock_id}] 第 {attempt + 1}/{retries} 次嘗試時發生異常: {e}。等待 {sleep_time:.2f} 秒後重試...")
                    time.sleep(sleep_time)
                else:
                    logger.error(f"[{stock_id}] 因異常，在 {retries} 次嘗試後獲取數據失敗。")
                    return pd.DataFrame()
        return pd.DataFrame()


    def calculate_all_indicators(self, df: pd.DataFrame) -> pd.DataFrame:
        if df.empty or 'Close' not in df.columns or len(df) < 20:
            return df
        df = df.copy()
        for days in [5, 10, 20, 60]:
            df[f'MA{days}'] = df['Close'].rolling(window=days, min_periods=1).mean()
        delta = df['Close'].diff()
        gain = delta.where(delta > 0, 0).rolling(window=14).mean()
        loss = -delta.where(delta < 0, 0).rolling(window=14).mean()
        rs = gain / (loss + 1e-9)
        df['RSI'] = 100 - (100 / (1 + rs))
        exp1 = df['Close'].ewm(span=12, adjust=False).mean()
        exp2 = df['Close'].ewm(span=26, adjust=False).mean()
        df['MACD'] = exp1 - exp2
        df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
        df['Histogram'] = df['MACD'] - df['MACD_signal']
        df = df.fillna(method='bfill').fillna(method='ffill')
        return df

    def analyze_stock(self, stock_info: Dict[str, Any]) -> Dict[str, Any]:
        try:
            df = stock_info['data'].copy()
            if len(df) < 60:
                return {'success': False, 'message': f"數據量不足 ({len(df)} < 60)"}

            df_with_indicators = self.calculate_all_indicators(df)
            if df_with_indicators.empty:
                return {'success': False, 'message': "指標計算後數據為空"}

            trend_analysis = self.analyze_trend(df_with_indicators)
            momentum_analysis = self.analyze_momentum(df_with_indicators)
            volume_analysis = self.analyze_volume(df_with_indicators)
            pattern_analysis = self.analyze_patterns(df_with_indicators)

            weights = {'trend': 0.3, 'momentum': 0.3, 'volume': 0.2, 'pattern': 0.2}
            combined_score = (
                trend_analysis.get('score', 50) * weights['trend'] +
                momentum_analysis.get('score', 50) * weights['momentum'] +
                volume_analysis.get('score', 50) * weights['volume'] +
                pattern_analysis.get('score', 50) * weights['pattern']
            )

            recommendation = self.generate_recommendation(combined_score, trend_analysis, momentum_analysis, volume_analysis, pattern_analysis)

            return {
                'success': True, 'stock_id': stock_info['stock_id'], 'stock_name': stock_info['stock_name'],
                'industry': stock_info['industry'], 'data_df': df_with_indicators,
                'combined_score': round(combined_score, 2), 'recommendation': recommendation,
                'last_price': df_with_indicators['Close'].iloc[-1]
            }
        except Exception as e:
            logger.error(f"分析股票 {stock_info.get('stock_id', 'N/A')} 時發生頂層錯誤: {e}")
            return {'success': False, 'message': str(e)}

    def analyze_trend(self, df: pd.DataFrame) -> Dict[str, Any]:
        try:
            if len(df) < 60: return {'score': 50}
            recent_close = float(df['Close'].iloc[-1])
            recent_ma20 = float(df['MA20'].iloc[-1])
            recent_ma60 = float(df['MA60'].iloc[-1])
            score = 50
            if recent_close > recent_ma20: score += 15
            if recent_close > recent_ma60: score += 15
            if recent_ma20 > recent_ma60: score += 10
            return {'score': max(0, min(100, score))}
        except Exception as e:
            logger.error(f"趨勢分析時發生錯誤: {e}"); return {'score': 50}

    def analyze_volume(self, df: pd.DataFrame) -> Dict[str, Any]:
        try:
            if len(df) < 20 or 'Volume' not in df.columns or df['Volume'].isna().all():
                return {'score': 50}
            # Convert to numeric
            df['Volume'] = pd.to_numeric(df['Volume'], errors='coerce')
            recent_volume = float(df['Volume'].iloc[-1])
            avg_volume_20 = float(df['Volume'].iloc[-20:].mean())
            score = 50
            if pd.isna(avg_volume_20) or avg_volume_20 == 0:
                return {'score': 50}
            volume_ratio = recent_volume / avg_volume_20
            if volume_ratio > 1.5:
                score += 20
            if len(df) >= 2:
                price_change = (float(df['Close'].iloc[-1]) - float(df['Close'].iloc[-2])) / float(df['Close'].iloc[-2])
                if price_change > 0.02 and volume_ratio > 1.2:
                    score += 20
                elif price_change < -0.02 and volume_ratio > 1.2:
                    score -= 20
            return {'score': max(0, min(100, score))}
        except Exception as e:
            logger.error(f"成交量分析時發生錯誤: {e}")
            return {'score': 50}
    def analyze_patterns(self, df: pd.DataFrame) -> Dict[str, Any]:
        try:
            if len(df) < 21: return {'score': 50, 'patterns': []}
            patterns_found = []
            score = 50
            high_20 = float(df['High'].iloc[-21:-1].max())
            latest_high = float(df['High'].iloc[-1])
            if latest_high > high_20:
                score += 25; patterns_found.append("突破20日高點")
            return {'score': max(0, min(100, score)), 'patterns': patterns_found}
        except Exception as e:
            logger.error(f"形態分析時發生錯誤: {e}"); return {'score': 50, 'patterns': []}

    def analyze_momentum(self, df: pd.DataFrame) -> Dict[str, Any]:
        try:
            if len(df) < 30 or 'RSI' not in df.columns or 'Histogram' not in df.columns:
                return {'score': 50}
            score = 50
            recent_rsi = float(df['RSI'].iloc[-1])
            recent_macd_hist = float(df['Histogram'].iloc[-1])
            if recent_rsi < 30: score += 20
            elif recent_rsi > 70: score -= 20
            if recent_macd_hist > 0 and float(df['Histogram'].iloc[-2]) < 0:
                score += 20  # MACD 柱狀體由負轉正
            elif recent_macd_hist > 0:
                score += 10
            return {'score': max(0, min(100, score))}
        except Exception as e:
            logger.error(f"動能分析時發生錯誤: {e}"); return {'score': 50}

    def generate_recommendation(self, combined_score, trend_analysis, momentum_analysis, volume_analysis, pattern_analysis):
        buy_threshold = 65
        sell_threshold = 35
        action = "觀望"
        if combined_score >= buy_threshold: action = "買入"
        elif combined_score <= sell_threshold: action = "賣出"

        reasons = []
        if trend_analysis.get('score', 50) > 70: reasons.append("趨勢強勁向上")
        if momentum_analysis.get('score', 50) > 70: reasons.append("動能充沛")
        if volume_analysis.get('score', 50) > 70: reasons.append("價量配合良好")
        if pattern_analysis.get('patterns'): reasons.extend(pattern_analysis['patterns'])
        if not reasons: reasons.append("多空指標均衡")

        if combined_score > 50:
            confidence = (combined_score - 50) / 50 * 100
        else:
            confidence = (50 - combined_score) / 50 * 100

        return {'action': action, 'reasons': reasons, 'confidence': round(confidence, 2)}

    def generate_detailed_analysis_report(self, stock_id: str, stock_name: str, stock_data: Dict, save_path: str):
        df = stock_data.get('data_df')
        if df is None or df.empty:
            logger.error(f"無法為 {stock_id} 生成圖表，缺少 data_df。")
            return None

        df_chart = df.copy()

        cols_to_convert = ['Open', 'High', 'Low', 'Close', 'Volume']
        for col in cols_to_convert:
            if col in df_chart.columns:
                df_chart[col] = pd.to_numeric(df_chart[col], errors='coerce')

        df_chart.dropna(subset=cols_to_convert, inplace=True)

        if not isinstance(df_chart.index, pd.DatetimeIndex):
            if 'Date' in df_chart.columns:
                df_chart['Date'] = pd.to_datetime(df_chart['Date'])
                df_chart.set_index('Date', inplace=True)
            else:
                df_chart.index = pd.to_datetime(df_chart.index)

        apds = [
            mpf.make_addplot(df_chart['MA5'], color='orange', width=0.7),
            mpf.make_addplot(df_chart['MA20'], color='blue', width=0.7),
            mpf.make_addplot(df_chart['RSI'], panel=2, color='purple', ylabel='RSI'),
            mpf.make_addplot(df_chart[['MACD', 'MACD_signal']], panel=3, ylabel='MACD'),
            mpf.make_addplot(df_chart['Histogram'], type='bar', panel=3, color='gray', alpha=0.5)
        ]
        mc = mpf.make_marketcolors(up='r', down='g', inherit=True)
        s = mpf.make_mpf_style(marketcolors=mc, gridstyle='--', facecolor='#F0F0F0', rc={'font.family': 'SimHei'})
        title = f"#{stock_id} {stock_name} - Score: {stock_data.get('combined_score', 0):.0f}"

        try:
            mpf.plot(df_chart.tail(180), type='candle', style=s, title=title,
                     volume=True, addplot=apds, figsize=(16, 9),
                     panel_ratios=(3, 1, 1, 1), savefig=save_path)
            logger.info(f"成功生成圖表: {save_path}")
            return save_path
        except Exception as e:
            logger.error(f"為 {stock_id} 繪製圖表失敗: {e}")
            return None

    def print_advanced_filtered_stocks(self, results: Dict, top_n=20):
        if not results:
            logger.warning("沒有股票通過最終篩選。")
            print("\n" + "="*80)
            print("🎯 最終篩選優質股票清單 🎯".center(80))
            print("="*80)
            print("無股票通過篩選條件。")
            print("="*80)
            return

        sorted_stocks = sorted(results.values(), key=lambda x: x.get('combined_score', 0), reverse=True)[:top_n]

        # Print summary table
        print("\n" + "="*80)
        print("🎯 最終篩選優質股票清單 🎯".center(80))
        print("="*80)
        print(f"{'排名':<4}{'代碼':<7}{'名稱':<12}{'分數':<7}{'價格':<10}{'建議':<10}{'產業':<15}")
        print("-" * 80)
        for i, stock in enumerate(sorted_stocks, 1):
            # 安全處理 industry 欄位，確保是字串類型
            industry = stock.get('industry', '')
            if pd.isna(industry) or not isinstance(industry, str):
                industry = '未分類'

            print(
            f"{i:<4}"
            f"{stock.get('stock_id', ''):<7}"
            f"{stock.get('stock_name', '')[:10]:<12}"
            f"{stock.get('combined_score', 0):<7.0f}"
            f"{stock.get('last_price', 0):<10.2f}"
            f"{stock.get('recommendation', {}).get('action', ''):<10}"
            f"{industry[:12]:<15}"
            )
        print("=" * 80)

        # Print detailed report for each stock
        print("\n" + "="*80)
        print("📊 詳細分析報告 📊".center(80))
        print("="*80)
        for i, stock in enumerate(sorted_stocks, 1):
            # 同樣安全處理 industry 欄位
            industry = stock.get('industry', '')
            if pd.isna(industry) or not isinstance(industry, str):
                industry = '未分類'

        print(f"\n報告 #{i}: {stock.get('stock_id', 'N/A')} - {stock.get('stock_name', 'N/A')}")
        print("-" * 50)
        print(f"綜合評分: {stock.get('combined_score', 0):.2f}")
        print(f"最新收盤價: {stock.get('last_price', 0):.2f} 元")
        print(f"產業類別: {industry}")
        print(f"推薦行動: {stock.get('recommendation', {}).get('action', 'N/A')}")
        print(f"信心度: {stock.get('recommendation', {}).get('confidence', 0):.2f}%")
        print("推薦理由:")
        reasons = stock.get('recommendation', {}).get('reasons', ['無特定理由'])
        for reason in reasons:
            print(f"  - {reason}")
        print("-" * 50)
    print("=" * 80)
# ==============================================================================
# 通知函數 - 增強版本
# ==============================================================================
# 修改 send_notification 函數，使 session 成為可選參數
async def send_notification(message: str, files: List[str] = None, session: aiohttp.ClientSession = None):
    """發送通知到 Telegram 和 Discord"""
    # 如果沒有提供 session，則創建一個新的
    session_created = False
    if session is None:
        session = aiohttp.ClientSession()
        session_created = True

    try:
        # 如果沒有提供檔案，嘗試從預設目錄獲取
        if not files:
            chart_dir = "/content/results/charts/"
            if os.path.exists(chart_dir):
                # 獲取目錄中的所有 PNG 檔案
                files = [os.path.join(chart_dir, f) for f in os.listdir(chart_dir)
                        if f.endswith('.png') and os.path.isfile(os.path.join(chart_dir, f))]
                if files:
                    logger.info(f"找到 {len(files)} 個圖表檔案在 {chart_dir}")
                else:
                    logger.warning(f"在 {chart_dir} 中找不到任何 PNG 檔案")

        # Telegram
        try:
            # 遍歷所有 Telegram Chat ID
            for chat_id in TELEGRAM_CHAT_ID:
                # 發送文字訊息
                url_msg = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage"
                payload = {"chat_id": chat_id, "text": message, "parse_mode": "Markdown"}
                async with session.post(url_msg, json=payload, timeout=20) as response:
                    if response.status == 200:
                        logger.info(f"Telegram 摘要已成功發送到 chat_id: {chat_id}。")
                    else:
                        response_text = await response.text()
                        logger.error(f"Telegram 摘要發送失敗: {response.status} - {response_text}")

                # 如果有檔案，也遍歷發送
                if files:
                    url_photo = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendPhoto"
                    sent_count = 0
                    for file_path in files:
                        if os.path.exists(file_path):
                            try:
                                data = aiohttp.FormData()
                                data.add_field('chat_id', str(chat_id))  # 確保 chat_id 是字串
                                # 添加可選的圖片說明
                                caption = os.path.basename(file_path).replace('_report.png', '').replace('_', ' ')
                                data.add_field('caption', caption)

                                with open(file_path, 'rb') as f:
                                    data.add_field('photo', f, filename=os.path.basename(file_path))
                                    async with session.post(url_photo, data=data, timeout=60) as response:
                                        if response.status == 200:
                                            sent_count += 1
                                        else:
                                            response_text = await response.text()
                                            logger.error(f"Telegram 圖檔發送失敗: {response.status} - {response_text}")

                                # 添加小延遲以避免 Telegram API 限制
                                await asyncio.sleep(0.5)
                            except Exception as file_error:
                                logger.error(f"發送檔案 {file_path} 時出錯: {file_error}")

                    logger.info(f"Telegram 圖檔已成功發送到 chat_id: {chat_id} ({sent_count}/{len(files)}個)。")
        except Exception as e:
            logger.error(f"發送 Telegram 通知時異常: {e}")
            traceback.print_exc()

        # Discord
        try:
            # 將訊息分段發送，避免超過 Discord 的限制
            message_chunks = [message[i:i+1900] for i in range(0, len(message), 1900)]

            for chunk in message_chunks:
                webhook = DiscordWebhook(url=DISCORD_WEBHOOK_URL, content=f"```\n{chunk}\n```")
                response = webhook.execute()
                if response and response.ok:
                    logger.info("Discord 文字通知發送成功。")
                else:
                    status_code = response.status_code if response else "未知"
                    content = response.content if response else "未知"
                    logger.error(f"Discord 文字通知失敗: {status_code} {content}")

            # 分批發送檔案，每批最多 10 個檔案
            if files:
                batch_size = 10
                for i in range(0, len(files), batch_size):
                    batch_files = files[i:i+batch_size]
                    webhook = DiscordWebhook(url=DISCORD_WEBHOOK_URL, content=f"圖表批次 {i//batch_size + 1}/{(len(files)-1)//batch_size + 1}")

                    for file_path in batch_files:
                        if os.path.exists(file_path):
                            try:
                                with open(file_path, 'rb') as f:
                                    webhook.add_file(file=f.read(), filename=os.path.basename(file_path))
                            except Exception as file_error:
                                logger.error(f"添加檔案 {file_path} 到 Discord webhook 時出錯: {file_error}")

                    response = webhook.execute()
                    if response and response.ok:
                        logger.info(f"Discord 圖檔批次 {i//batch_size + 1} 發送成功。")
                    else:
                        status_code = response.status_code if response else "未知"
                        content = response.content if response else "未知"
                        logger.error(f"Discord 圖檔批次 {i//batch_size + 1} 發送失敗: {status_code} {content}")
        except Exception as e:
            logger.error(f"發送 Discord 通知時異常: {e}")
            traceback.print_exc()

    finally:
        # 如果我們創建了 session，則關閉它
        if session_created:
            await session.close()
async def send_chart_files(chart_path, caption=""):
    """
    發送圖表文件到 Telegram 和 Discord
    """
    if not os.path.exists(chart_path):
        logger.error(f"圖表文件不存在: {chart_path}")
        return

    try:
        # 使用 send_notification 函數發送單個文件
        await send_notification(caption, files=[chart_path])
        logger.info(f"已發送圖表: {os.path.basename(chart_path)}")
    except Exception as e:
        logger.error(f"發送圖表時出錯: {e}")
        logger.error(traceback.format_exc())

def format_notification_message(sorted_results):
    """
    格式化通知消息
    """
    if not sorted_results:
        return "⚠️ 分析完成，但沒有符合條件的股票。"

    # 取前 10 名
    top_n = min(10, len(sorted_results))
    top_stocks = sorted_results[:top_n]

    # 計算動作統計
    action_counts = {}
    for stock in sorted_results:
        action = stock.get('recommendation', {}).get('action', '持有')
        action_counts[action] = action_counts.get(action, 0) + 1

    # 格式化消息
    message = f"📊 台股技術分析結果 ({datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M')})\n"
    message += f"共分析 {len(sorted_results)} 支符合條件的股票\n"
    message += f"買入: {action_counts.get('買入', 0)} | 持有: {action_counts.get('持有', 0)} | 賣出: {action_counts.get('賣出', 0)}\n\n"

    message += "🏆 評分最高的股票:\n"
    for i, stock in enumerate(top_stocks, 1):
        stock_id = stock['stock_id']
        stock_name = stock['stock_name']
        score = stock.get('combined_score', 0)
        price = stock.get('last_price', 0)
        action = stock.get('recommendation', {}).get('action', '持有')
        confidence = stock.get('recommendation', {}).get('confidence', 0)

        message += f"{i}. {stock_name} ({stock_id}) - {price:.2f} 元\n"
        message += f"   評分: {score:.1f}/100 | 建議: {action} ({confidence:.0f}%)\n"

    message += "\n📈 詳細分析圖表已生成，請查看附件。"
    return message


async def generate_and_send_summary_chart(results, report_time):
    """
    生成並發送摘要圖表
    """
    if not results:
        logger.warning("沒有結果可供生成摘要圖表")
        return None

    try:
        # 創建摘要圖表
        logger.info("生成摘要圖表...")

        # 設定圖表大小 - 4K 解析度
        plt.figure(figsize=(32, 18), dpi=150)  # 4K 等效 (4800x2700)

        # 設定標題
        plt.suptitle(f'台股技術分析摘要 - {report_time}', fontsize=24, y=0.98)

        # 取前 20 名進行展示
        top_n = min(20, len(results))
        top_stocks = results[:top_n]

        # 準備數據
        stock_ids = [f"{r['stock_id']} {r['stock_name']}" for r in top_stocks]
        scores = [r.get('combined_score', 0) for r in top_stocks]
        actions = [r.get('recommendation', {}).get('action', '持有') for r in top_stocks]

        # 設定顏色映射
        colors = []
        for action in actions:
            if action == '買入':
                colors.append('#FF5252')  # 紅色
            elif action == '賣出':
                colors.append('#4CAF50')  # 綠色
            else:
                colors.append('#FFC107')  # 黃色

        # 繪製評分條形圖
        ax1 = plt.subplot2grid((2, 2), (0, 0), colspan=2)
        bars = ax1.barh(stock_ids, scores, color=colors, alpha=0.8)
        ax1.set_title('股票綜合評分 (越高越看好)', fontsize=18)
        ax1.set_xlim(0, 100)
        ax1.axvline(x=50, color='gray', linestyle='--', alpha=0.5)
        ax1.set_xlabel('評分', fontsize=14)
        ax1.grid(axis='x', linestyle='--', alpha=0.3)

        # 添加數值標籤
        for bar, score in zip(bars, scores):
            ax1.text(min(score + 2, 95), bar.get_y() + bar.get_height()/2,
                    f'{score:.1f}', va='center', fontsize=12)

        # 繪製評分分佈直方圖
        ax2 = plt.subplot2grid((2, 2), (1, 0))
        all_scores = [r.get('combined_score', 0) for r in results]
        ax2.hist(all_scores, bins=20, range=(0, 100), color='skyblue', edgecolor='black', alpha=0.7)
        ax2.set_title('所有股票評分分佈', fontsize=18)
        ax2.set_xlabel('評分', fontsize=14)
        ax2.set_ylabel('股票數量', fontsize=14)
        ax2.grid(linestyle='--', alpha=0.3)

        # 繪製建議動作餅圖
        ax3 = plt.subplot2grid((2, 2), (1, 1))
        action_counts = {}
        for r in results:
            action = r.get('recommendation', {}).get('action', '持有')
            action_counts[action] = action_counts.get(action, 0) + 1

        labels = list(action_counts.keys())
        sizes = list(action_counts.values())
        colors_pie = ['#FF5252', '#FFC107', '#4CAF50']  # 紅(買入), 黃(持有), 綠(賣出)
        explode = [0.1 if label == '買入' else 0 for label in labels]  # 突出"買入"

        ax3.pie(sizes, explode=explode, labels=labels, colors=colors_pie, autopct='%1.1f%%',
               shadow=True, startangle=90, textprops={'fontsize': 14})
        ax3.set_title('建議動作分佈', fontsize=18)

        # 添加圖例
        from matplotlib.lines import Line2D
        legend_elements = [
            Line2D([0], [0], color='#FF5252', lw=4, label='買入'),
            Line2D([0], [0], color='#FFC107', lw=4, label='持有'),
            Line2D([0], [0], color='#4CAF50', lw=4, label='賣出')
        ]
        ax1.legend(handles=legend_elements, loc='lower right', fontsize=14)

        # 添加註腳
        plt.figtext(0.5, 0.01,
                   f'分析時間: {report_time} | 共分析 {len(results)} 支股票 | 使用指標: 趨勢、動能、成交量、形態',
                   ha='center', fontsize=12, bbox=dict(facecolor='#f0f0f0', alpha=0.5))

        # 調整佈局
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])

        # 保存圖表
        summary_chart_path = os.path.join(CHARTS_DIR, f'summary_report_{datetime.now(taipei_tz).strftime("%Y%m%d_%H%M%S")}.png')
        plt.savefig(summary_chart_path, dpi=150, bbox_inches='tight')
        logger.info(f"摘要圖表已保存到: {summary_chart_path}")

        # 關閉圖表，釋放記憶體
        plt.close()

        return summary_chart_path

    except Exception as e:
        logger.error(f"生成摘要圖表時發生錯誤: {e}")
        logger.error(traceback.format_exc())
        return None


# 如果你還需要一個純統計圖表的版本，可以額外添加這個函數
async def generate_and_send_statistics_chart(final_results: list, report_time: str):
    """
    生成並發送統計分析圖表（行業分布 + 評分分布）
    """
    try:
        if not final_results:
            logger.info("無結果數據，跳過統計圖表生成")
            return

        # 創建圖表 - 1x2 布局
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
        fig.suptitle(f'📊 台股分析統計圖表 - {report_time}', fontsize=16, fontweight='bold')

        # 1. 行業分布圓餅圖
        industry_counts = {}
        for stock in final_results:
            industry = stock.get('industry', '其他')
            industry_counts[industry] = industry_counts.get(industry, 0) + 1

        if industry_counts:
            industries = list(industry_counts.keys())
            counts = list(industry_counts.values())
            colors = plt.cm.Set3(np.linspace(0, 1, len(industries)))

            wedges, texts, autotexts = ax1.pie(counts, labels=industries, autopct='%1.0f%%',
                                              colors=colors, startangle=90)
            ax1.set_title('🏭 行業分布', fontweight='bold', fontsize=14)

            # 調整文字大小
            for text in texts:
                text.set_fontsize(10)
            for autotext in autotexts:
                autotext.set_fontsize(9)
                autotext.set_fontweight('bold')

        # 2. 評分分布直方圖
        all_scores = [s.get('combined_score', 0) for s in final_results]
        n, bins, patches = ax2.hist(all_scores, bins=10, color='#4682B4', alpha=0.7, edgecolor='black')
        ax2.set_xlabel('綜合評分', fontsize=12)
        ax2.set_ylabel('股票數量', fontsize=12)
        ax2.set_title('📈 評分分布', fontweight='bold', fontsize=14)
        ax2.grid(axis='y', alpha=0.3)

        # 在直方圖柱子上顯示數量
        for i, (count, patch) in enumerate(zip(n, patches)):
            if count > 0:
                ax2.text(patch.get_x() + patch.get_width()/2, patch.get_height() + 0.1,
                        f'{int(count)}', ha='center', va='bottom', fontsize=9, fontweight='bold')

        # 添加統計線
        mean_score = np.mean(all_scores)
        ax2.axvline(mean_score, color='red', linestyle='--', linewidth=2, label=f'平均分: {mean_score:.1f}')
        ax2.legend(fontsize=11)

        # 調整布局
        plt.tight_layout()

        # 儲存圖表
        stats_chart_path = os.path.join(CHARTS_DIR, f"statistics_report_{datetime.now(taipei_tz).strftime('%Y%m%d_%H%M%S')}.png")
        plt.savefig(stats_chart_path, dpi=150, bbox_inches='tight',
                   facecolor='white', edgecolor='none')
        plt.close()

        # 發送統計圖表
        caption = f"📊 台股分析統計圖表\n⏰ {report_time}\n📈 共 {len(final_results)} 支符合條件"
        await send_chart_files(stats_chart_path, caption)

        logger.info(f"已生成並發送統計圖表: {stats_chart_path}")

    except Exception as e:
        logger.error(f"生成統計圖表時發生錯誤: {e}")
        logger.error(traceback.format_exc())

# ==============================================================================
# 主執行流程 (main) - 增強版本
# ==============================================================================
async def main():
    start_time = time.time()
    report_time = format_timestamp()  # 使用台北時區時間

    logger.info(f"==== 📈 開始執行台股分析工具 (時間: {report_time}) ====")

    analyzer = StockAnalyzer()

    # --- 階段 1: 獲取股票清單 ---
    logger.info("\n--- [階段 1/5] 獲取股票清單 ---")
    stocks_df = analyzer.get_taiwan_stocks()
    if stocks_df.empty:
        logger.error("無法獲取股票清單，程序終止。")
        return
    logger.info(f"共獲取 {len(stocks_df)} 支股票。")

    # --- 階段 2: 逐一處理與分析 ---
    logger.info("\n--- [階段 2/5] 進行篩選、分析與評分 ---")
    all_results = []
    failed_stocks = []
    MIN_VOLUME = 1000 * 1000  # 1000張

    for index, stock in stocks_df.iterrows():
        stock_id = stock['stock_id']
        yahoo_symbol = stock['yahoo_symbol']
        logger.info(f"--- 處理中 ({index + 1}/{len(stocks_df)}): {stock_id} {stock['stock_name']} ---")

        try:
            data_df = analyzer.fetch_yfinance_data_single_robust(yahoo_symbol, period='1y', interval='1d')
            if data_df.empty or len(data_df) < 60:
                logger.warning(f"[{stock_id}] 數據不足或獲取失敗，跳過。")
                failed_stocks.append(stock_id)
                continue

            data_df = analyzer.standardize_columns(data_df.reset_index())

            # Ensure Volume is numeric and handle missing/invalid data
            if 'Volume' not in data_df.columns:
                logger.info(f"[{stock_id}] 缺少成交量欄位，跳過。")
                failed_stocks.append(stock_id)
                continue

            # Convert Volume to numeric, coercing errors to NaN
            data_df['Volume'] = pd.to_numeric(data_df['Volume'], errors='coerce')

            # Check if recent volume data is sufficient and valid
            recent_volume = data_df['Volume'].tail(15)
            if recent_volume.empty or recent_volume.isna().all() or pd.isna(recent_volume.mean()) or recent_volume.mean() < MIN_VOLUME:
                logger.info(f"[{stock_id}] 未通過成交量篩選，跳過。")
                failed_stocks.append(stock_id)
                continue

            stock_info = {
                'stock_id': stock_id, 'stock_name': stock['stock_name'],
                'industry': stock['industry'], 'data': data_df
            }
            analysis_result = analyzer.analyze_stock(stock_info)

            if analysis_result and analysis_result.get('success'):
                all_results.append(analysis_result)
                logger.info(f"[{stock_id}] 分析完成，綜合評分: {analysis_result.get('combined_score')}")
            else:
                logger.error(f"對 {stock_id} 的分析失敗: {analysis_result.get('message', '未知錯誤')}")
                failed_stocks.append(stock_id)
        except Exception as e:
            logger.error(f"處理 {stock_id} 時發生異常: {e}")
            logger.error(traceback.format_exc())
            failed_stocks.append(stock_id)
            continue

   # --- 階段 3: 進階篩選與報告 ---
    logger.info("\n--- [階段 3/5] 進行進階篩選與輸出結果 ---")
    # 這裡定義 final_results
    final_results = [res for res in all_results if res.get('combined_score', 0) > 65 and res.get('recommendation', {}).get('action') == '買入']

    # 按評分排序 - 確保即使沒有符合條件的股票，也有一個空列表
    sorted_results = sorted(final_results, key=lambda x: x.get('combined_score', 0), reverse=True)

    final_results_dict = {res['stock_id']: res for res in final_results}
    analyzer.print_advanced_filtered_stocks(final_results_dict, top_n=20)
    # --- 階段 4: 生成個股圖表與發送通知 ---
    logger.info("\n--- [階段 4/5] 生成個股圖表與發送通知 ---")
    top_stocks_for_report = sorted_results[:5]  # 取前5名

    if not top_stocks_for_report:
        logger.warning("沒有符合條件的股票可供報告。")
        summary_message = f"📈 台股分析本日精選 📈\n⏰ {report_time}\n\n今日無特別亮眼標的，建議持續觀察市場。"
        await send_notification(summary_message)
    else:
        # 發送文字摘要
        summary_lines = [f"📈 台股分析本日精選 📈", f"⏰ {report_time}", ""]
        for i, stock in enumerate(top_stocks_for_report, 1):
            rec = stock['recommendation']
            summary_lines.append(
                f"{i}. *{stock['stock_id']} {stock['stock_name']}*\n"
                f"   評分: {stock['combined_score']:.0f} | "
                f"建議: *{rec['action']}* | "
                f"信心度: {rec.get('confidence', 0):.0f}%"
            )

        summary_lines.append(f"\n📊 總計 {len(final_results)} 支符合條件股票")
        summary_message = "\n".join(summary_lines)

        logger.info("準備發送摘要通知...")
        await send_notification(summary_message)

        # 生成詳細報告文件
        with open('stock_report.txt', 'w', encoding='utf-8') as f:
            f.write(f"台股分析詳細報告\n")
            f.write(f"報告時間: {report_time}\n")
            f.write("=" * 50 + "\n\n")

            original_stdout = sys.stdout
            sys.stdout = f
            analyzer.print_advanced_filtered_stocks(final_results_dict, top_n=10)
            sys.stdout = original_stdout

        # 發送個股詳細圖表
        logger.info("準備發送個股詳細圖表...")
        for i, stock in enumerate(top_stocks_for_report, 1):
            stock_id = stock['stock_id']
            stock_name = stock['stock_name']
            save_path = os.path.join(CHARTS_DIR, f"{stock_id}_{stock_name}_report.png")

            chart_path = analyzer.generate_detailed_analysis_report(stock_id, stock_name, stock, save_path)

            if chart_path:
                caption = f"📊 {i}/5 - {stock_id} {stock_name}\n評分: {stock['combined_score']:.0f}"
                await send_chart_files(chart_path, caption)
                await asyncio.sleep(2)  # 避免發送過快

   # --- 階段 5: 生成並發送最終摘要圖表 ---
    logger.info("\n--- [階段 5/5] 生成最終摘要圖表 ---")
    if final_results:
        summary_chart_path = await generate_and_send_summary_chart(final_results, report_time)
        stats_chart_path = await generate_and_send_statistics_chart(final_results, report_time)

        # 收集所有圖表路徑
        all_chart_paths = []
        if summary_chart_path:
            all_chart_paths.append(summary_chart_path)
        if 'stats_chart_path' in locals() and stats_chart_path:
            all_chart_paths.append(stats_chart_path)

    # 發送執行完成通知
    end_time = time.time()
    execution_time = end_time - start_time
    completion_time = format_timestamp()

    # 使用 sorted_results 而不是 final_results，並確保它已定義
    # 這裡是關鍵修復部分
    if 'sorted_results' in locals() and sorted_results:
        completion_message = format_notification_message(sorted_results)
    else:
        completion_message = f"""
🎉 **分析完成通知** 🎉

⏰ 開始時間: {report_time}
⏰ 完成時間: {completion_time}
⌛ 執行時長: {execution_time:.1f} 秒

📊 **執行結果:**
• 總處理股票: {len(stocks_df)} 支
• 成功分析: {len(all_results)} 支
• 符合條件: 0 支
• 失敗股票: {len(failed_stocks)} 支

⚠️ 沒有符合篩選條件的股票。
"""

    # 發送最終通知
    await send_notification(completion_message)

    logger.info(f"\n🎉🎉🎉 全部流程執行完畢！")
    logger.info(f"📊 處理統計: 總計 {len(stocks_df)} 支，成功 {len(all_results)} 支，符合條件 {len(final_results)} 支")
    logger.info(f"⌛ 總耗時: {execution_time:.2f} 秒")
    logger.info(f"⏰ 完成時間: {completion_time}")
# ==============================================================================
# 主程式入口點
# ==============================================================================
if __name__ == "__main__":
    try:
        # 檢查是否在 Jupyter 環境中
        in_jupyter = 'ipykernel' in sys.modules

        if in_jupyter:
            # Jupyter 環境處理
            loop = asyncio.get_event_loop()
            if loop.is_running():
                # 如果事件循環正在運行，創建任務
                task = loop.create_task(main())
                logger.info("在 Jupyter 環境中創建異步任務")
            else:
                # 如果事件循環未運行，直接執行
                loop.run_until_complete(main())
        else:
            # 一般 Python 環境
            asyncio.run(main())

    except KeyboardInterrupt:
        logger.info("用戶中斷程式執行")
    except Exception as e:
        error_time = format_timestamp()
        logger.critical(f"程式執行時發生致命錯誤 ({error_time}): {e}")
        logger.critical(traceback.format_exc())

        # 發送錯誤通知（如果通訊功能可用）
        if 'MESSAGING_AVAILABLE' in globals() and MESSAGING_AVAILABLE:
            try:
                error_message = f"""
❌ **程式執行錯誤** ❌

⏰ 錯誤時間: {error_time}
🐞 錯誤訊息: {str(e)}

請檢查日誌檔案以獲取詳細資訊。
                """.strip()
                asyncio.run(send_notification(error_message))
            except:
                pass  # 避免通知發送失敗導致程式崩潰

可傳訊但終端沒有顯現出報告，傳訊有圖片但會延遲出現，軟體要跑很久。discord與telegram 都成功！

In [ ]:

# --- 基礎 Python 標準庫 ---
import os
import sys
import json
import time
import traceback
import platform
import re
from io import StringIO
from datetime import datetime
from typing import List, Dict
import logging

# --- 數據處理與科學計算 ---
import pandas as pd
import numpy as np

# --- 網路請求 ---
import requests
import aiohttp # 異步請求
import asyncio # 異步框架

# --- 金融數據源 ---
import yfinance as yf

# --- 數據可視化 ---
import matplotlib.pyplot as plt
from matplotlib import font_manager
import mplfinance as mpf
# import chineseize_matplotlib # 建議使用 set_chinese_font() 進行更可控的設定

# --- 通知相關 ---
import pytz
from discord_webhook import DiscordWebhook

# ==============================================
# 全域設定與初始化
# ==============================================

# --- 目錄設定 ---
CACHE_DIR = 'cache'
RESULTS_DIR = 'results'
CHARTS_DIR = os.path.join(RESULTS_DIR, 'charts')
STOCK_LIST_PATH = os.path.join(CACHE_DIR, 'stock_list.csv')
HISTORY_DATA_CACHE_DIR = os.path.join(CACHE_DIR, 'history_data')

# --- 初始化目錄 ---
for directory in [CACHE_DIR, RESULTS_DIR, CHARTS_DIR, HISTORY_DATA_CACHE_DIR]:
    os.makedirs(directory, exist_ok=True)

# --- 通知設定 (請填入您的金鑰) ---
# ==============================================================================
# 9. 通訊與通知設定
# ==============================================================================
# 建議將來使用環境變數或 .env 檔案管理密鑰，以策安全
TELEGRAM_TOKEN = "7902318521:AAEYoDMqwfHabI7L1SRiE4z33aFay42-VGE"
TELEGRAM_CHAT_ID = [879781796, 8113868436]  # 支援多用戶通知
DISCORD_WEBHOOK_URL = "https://discord.com/api/webhooks/1362715080734802102/Jma7A3VhEQrrRxIX_JW2l6rATjAZXsGXGfnJuAMqmS1QvqG_2ptg3vr_nsVnuV_PlnBl"
# --- 日誌與時區設定 ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
try:
    taipei_tz = pytz.timezone('Asia/Taipei')
except pytz.UnknownTimeZoneError:
    logger.error("時區 'Asia/Taipei' 找不到，請確保 pytz 庫已安裝且數據為最新。")
    taipei_tz = pytz.utc # 降級使用 UTC

# ==============================================
# 輔助函式與環境設定
# ==============================================

def set_chinese_font():
    """
    自動設定適合當前作業系統的中文字體，用於 Matplotlib 顯示。
    """
    try:
        system = platform.system()
        font_name = ""

        if system == 'Windows':
            font_name = 'Microsoft YaHei'
        elif system == 'Darwin': # macOS
            font_name = 'PingFang HK'
        else: # Linux
            font_paths = [
                '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc',
                '/usr/share/fonts/truetype/wqy/wqy-zenhei.ttc',
            ]
            found_font_path = next((path for path in font_paths if os.path.exists(path)), None)
            if found_font_path:
                font_manager.fontManager.addfont(found_font_path)
                font_name = font_manager.FontProperties(fname=found_font_path).get_name()
            else:
                logger.warning("⚠️ 未在常見路徑找到中文字體，圖表中的中文可能無法正常顯示。")
                # 使用 chineseize_matplotlib 作為備援
                import chineseize_matplotlib
                logger.info("啟用 chineseize_matplotlib 作為備援字體方案。")
                return

        plt.rcParams['font.sans-serif'] = [font_name]
        plt.rcParams['axes.unicode_minus'] = False
        if font_name:
            logger.info(f"✅ 已設定圖表字體為: {font_name}")

    except Exception as e:
        logger.error(f"❌ 字體配置錯誤: {e}。使用預設字體。")
        plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
        plt.rcParams['axes.unicode_minus'] = False

# ==============================================
# 核心功能函式 (資料獲取、分析、評分)
# ==============================================

def get_taiwan_stocks(cache_path=STOCK_LIST_PATH, force_update=False):
    if not force_update and os.path.exists(cache_path) and (time.time() - os.path.getmtime(cache_path)) < 86400:
        try:
            df = pd.read_csv(cache_path, dtype={'stock_id': str})
            if not df.empty and 'yahoo_symbol' in df.columns:
                print(f"✅ 從快取載入 {len(df)} 支股票清單。")
                return df
        except Exception as e:
            print(f"⚠️ 載入股票清單快取失敗: {e}。")

    print("🌐 正在獲取最新股票清單 (從證交所)...")
    try:
        urls = {
            "上市": "https://isin.twse.com.tw/isin/C_public.jsp?strMode=2",
            "上櫃": "https://isin.twse.com.tw/isin/C_public.jsp?strMode=4"
        }
        all_stocks_df = []
        for market, url in urls.items():
            response = requests.get(url, timeout=20)
            df = pd.read_html(StringIO(response.text))[0]
            df.columns = df.iloc[0]
            df = df.iloc[1:].dropna(thresh=3, axis=0)
            df['市場別'] = market
            all_stocks_df.append(df)

        df_combined = pd.concat(all_stocks_df, ignore_index=True)
        df_combined.columns = ['有價證券代號及名稱', '國際證券識別碼', '上市日', '市場別', '產業別', 'CFI', '備註']
        df_combined[['stock_id', 'stock_name']] = df_combined['有價證券代號及名稱'].str.split('　', n=1, expand=True)
        df_combined['stock_id'] = df_combined['stock_id'].str.strip()
        df_combined['stock_name'] = df_combined['stock_name'].str.strip()
        df_stocks = df_combined[df_combined['stock_id'].str.match(r'^\d{4}$')].copy()
        exclude_keywords = ['ETF', 'ETN', 'TDR', '受益', '指數', '購', '牛', '熊']
        df_stocks = df_stocks[~df_stocks['stock_name'].str.contains('|'.join(exclude_keywords), na=False)]
        df_stocks['yahoo_symbol'] = df_stocks.apply(
            lambda row: f"{row['stock_id']}.TW" if '上市' in row['市場別'] else f"{row['stock_id']}.TWO",
            axis=1
        )
        final_df = df_stocks[['stock_id', 'stock_name', '市場別', '產業別', 'yahoo_symbol']].drop_duplicates(subset=['stock_id']).reset_index(drop=True)
        final_df.rename(columns={'市場別': 'market', '產業別': 'industry'}, inplace=True)
        final_df.to_csv(cache_path, index=False)
        print(f"✅ 成功從證交所獲取 {len(final_df)} 支股票清單並保存。")
        return final_df
    except Exception as e:
        print(f"❌ 從證交所獲取股票清單時發生錯誤: {e}")
        if os.path.exists(cache_path):
            print("❗ 降級使用舊的快取股票清單。")
            return pd.read_csv(cache_path, dtype={'stock_id': str})
        return pd.DataFrame()

def get_stock_basic_info(force_update=False):
    stocks_df = get_taiwan_stocks(force_update=force_update)
    if stocks_df.empty: return {}
    return {str(row['stock_id']): row.to_dict() for _, row in stocks_df.iterrows()}

# ***** 關鍵修正函式 *****
def fetch_stock_data(stock_id, yahoo_symbol, period='1y', force_update=False, retries=3, delay=2, cache_dir=HISTORY_DATA_CACHE_DIR):
    """下載股價數據，增加對 MultiIndex 欄位的處理。"""
    formatted_id = str(stock_id)
    os.makedirs(cache_dir, exist_ok=True)
    cache_path = os.path.join(cache_dir, f"{formatted_id}.json")

    if not force_update and os.path.exists(cache_path) and (time.time() - os.path.getmtime(cache_path)) < 86400:
        try:
            with open(cache_path, 'r', encoding='utf-8') as f:
                return json.load(f)
        except Exception:
            pass

    for attempt in range(retries):
        try:
            temp_df = yf.download(yahoo_symbol, period=period, progress=False, auto_adjust=True)

            if temp_df.empty:
                print(f"⚠️ {yahoo_symbol} 在此期間無數據。")
                return None

            # --- START OF THE FIX ---
            # 檢查並處理 yfinance 可能返回的多重索引 (MultiIndex) 列標頭
            if isinstance(temp_df.columns, pd.MultiIndex):
                # 將多重索引 'flatten' 為單層索引
                temp_df.columns = temp_df.columns.get_level_values(0)
            # --- END OF THE FIX ---

            df = temp_df.reset_index()
            df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')

            result = {
                'data': df.to_dict('records'),
                'stock_id': formatted_id,
                'yahoo_symbol': yahoo_symbol,
                'last_price': df['Close'].iloc[-1] if not df.empty else 0
            }
            with open(cache_path, 'w', encoding='utf-8') as f:
                json.dump(result, f, ensure_ascii=False, indent=2)
            return result
        except Exception as e:
            if attempt < retries - 1:
                time.sleep(delay)
            else:
                # 在此處印出錯誤，讓主流程知道發生了什麼
                print(f"下載 {yahoo_symbol} 失敗: {e}")
    return None

def fetch_multiple_stocks_data(stocks_info, period='1y', force_update=False):
    results = {}
    stocks_list = list(stocks_info.values())
    total_stocks = len(stocks_list)

    for idx, stock in enumerate(stocks_list):
        stock_id = str(stock.get('stock_id'))
        stock_name = stock.get('stock_name', 'N/A')
        yahoo_symbol = stock.get('yahoo_symbol')

        if not yahoo_symbol:
            print(f"⚠️ 股票 {stock_id} {stock_name} 缺少 'yahoo_symbol'，跳過。")
            continue

        print(f"[{idx+1}/{total_stocks}] 正在獲取: {stock_id} {stock_name}...")
        stock_data = fetch_stock_data(
            stock_id=stock_id,
            yahoo_symbol=yahoo_symbol,
            period=period,
            force_update=force_update
        )

        if stock_data and 'data' in stock_data and stock_data['data']:
            stock_data.update(stock) # 將股票基本資料合併進去
            results[stock_id] = stock_data

    print(f"\n處理完成，共獲取 {len(results)}/{total_stocks} 支股票的數據。")
    return results
import yfinance as yf
import pandas as pd
import time
import logging

# 假設您的程式中已經設定了 logger
# 如果沒有，可以取消下面這行的註解
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def fetch_stock_history(stock_id, period='1y', retries=3, delay=5):
    """
    使用 yfinance 獲取指定股票的歷史數據，並處理台灣市場的股票代號。

    參數:
    stock_id (str): 股票代號 (例如 '2330')
    period (str): 數據期間 (例如 '1y', '2y', 'max')
    retries (int): 下載失敗時的重試次數
    delay (int): 重試前的延遲秒數

    返回:
    pandas.DataFrame or None: 包含 OHLCV 數據的 DataFrame，如果失敗則返回 None
    """
    # 根據股票代號格式判斷市場
    # 對於數字代號，優先嘗試 .TW (上市)，失敗則嘗試 .TWO (上櫃)
    # 對於非數字代號 (如 ETF)，直接使用 .TW
    ticker_symbol_tw = f"{stock_id}.TW"
    ticker_symbol_two = f"{stock_id}.TWO"

    # 確定要嘗試的股票代號列表
    symbols_to_try = [ticker_symbol_tw]
    if stock_id.isdigit():
        symbols_to_try.append(ticker_symbol_two)

    for symbol in symbols_to_try:
        for attempt in range(retries):
            try:
                stock = yf.Ticker(symbol)
                data = stock.history(period=period, auto_adjust=False) # 使用 auto_adjust=False 保留原始OHLC

                if not data.empty:
                    # 轉換為台灣時區並重設索引
                    if data.index.tz is None:
                        data = data.tz_localize('UTC').tz_convert('Asia/Taipei')
                    else:
                        data = data.tz_convert('Asia/Taipei')

                    data = data.reset_index()

                    # 將 'Date' 或 'Datetime' 列的時區資訊移除，以方便後續處理
                    date_col = 'Date' if 'Date' in data.columns else 'Datetime'
                    if date_col in data.columns:
                        data[date_col] = data[date_col].dt.tz_localize(None)

                    # 更改欄位名稱以符合習慣
                    data.rename(columns={
                        date_col: 'date',
                        'Open': 'open',
                        'High': 'high',
                        'Low': 'low',
                        'Close': 'close',
                        'Volume': 'volume'
                    }, inplace=True)

                    # 移除不需要的欄位
                    cols_to_drop = ['Dividends', 'Stock Splits']
                    data = data.drop(columns=[col for col in cols_to_drop if col in data.columns])

                    logger.info(f"✅ 成功獲取 {symbol} 的歷史數據。")
                    return data

            except Exception as e:
                logger.error(f"下載 {symbol} 數據時發生錯誤 (第 {attempt + 1}/{retries} 次嘗試): {e}")
                if attempt < retries - 1:
                    time.sleep(delay)
                else:
                    logger.error(f"重試 {retries} 次後，仍無法下載 {symbol} 的數據。")

    logger.warning(f"最終無法獲取股票 {stock_id} 的任何歷史數據。")
    return None

def add_technical_indicators(df):
    if df.empty or 'Close' not in df.columns: return df
    result = df.copy()
    for days in [5, 10, 20, 60]: result[f'MA{days}'] = result['Close'].rolling(window=days).mean()
    delta = result['Close'].diff()
    gain = delta.where(delta > 0, 0); loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean(); avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / (avg_loss + 1e-9); result['RSI'] = 100 - (100 / (1 + rs))
    exp1 = result['Close'].ewm(span=12, adjust=False).mean(); exp2 = result['Close'].ewm(span=26, adjust=False).mean()
    result['MACD'] = exp1 - exp2; result['Signal'] = result['MACD'].ewm(span=9, adjust=False).mean()
    result['Histogram'] = result['MACD'] - result['Signal']
    low_min = result['Low'].rolling(window=14).min(); high_max = result['High'].rolling(window=14).max()
    result['K'] = 100 * ((result['Close'] - low_min) / (high_max - low_min + 1e-9))
    result['D'] = result['K'].rolling(window=3).mean()
    return result.fillna(0)

def analyze_stock(stock_id, stock_data):
    if not stock_data or not stock_data.get('data'):
        return {'success': False, 'message': "無效數據"}
    df = pd.DataFrame(stock_data['data'])
    if len(df) < 60: return {'success': False, 'message': "數據不足"}

    df['Date'] = pd.to_datetime(df['Date']); df.set_index('Date', inplace=True)
    df_with_indicators = add_technical_indicators(df)

    price_change_1m = ((df['Close'].iloc[-1] / df['Close'].iloc[-22]) - 1) * 100 if len(df) > 21 else 0

    return {
        'stock_id': stock_id,
        'stock_name': stock_data.get('stock_name', 'N/A'),
        'last_price': stock_data.get('last_price', 0),
        'price_change': {'1m': price_change_1m},
        'data_df': df_with_indicators,
        'success': True
    }

def calculate_comprehensive_score(analysis_result):
    scores = {'trend_score': 50, 'momentum_score': 50, 'volume_score': 50}
    df = analysis_result.get('data_df')
    if df is None or df.empty or len(df) < 60: return scores, 50

    if df['MA5'].iloc[-1] > df['MA10'].iloc[-1] > df['MA20'].iloc[-1] > df['MA60'].iloc[-1]:
        scores['trend_score'] = 85
    else: scores['trend_score'] = 20

    momentum_points = 50
    if df['RSI'].iloc[-1] > 60: momentum_points += 15
    if df['MACD'].iloc[-1] > df['Signal'].iloc[-1]: momentum_points += 20
    if df['K'].iloc[-1] > df['D'].iloc[-1]: momentum_points += 15
    scores['momentum_score'] = max(0, min(100, momentum_points))

    if df['Volume'].iloc[-1] > df['Volume'].rolling(window=20).mean().iloc[-1] * 1.5:
        scores['volume_score'] = 80

    final_score = scores['trend_score'] * 0.4 + scores['momentum_score'] * 0.45 + scores['volume_score'] * 0.15
    return scores, final_score

def generate_recommendation(score):
    if score >= 75: return '強力買入'
    if score >= 60: return '買入'
    if score >= 40: return '持有'
    if score >= 25: return '觀望'
    return '賣出'



import mplfinance as mpf
print(f"mplfinance 版本: {mpf.__version__}")

def generate_detailed_analysis_report(stock_id, stock_name, stock_data, save_path=None):
    """
    生成單一股票的詳細分析圖表 (修正版)
    """
    try:
        # 1. 獲取包含完整指標的 DataFrame 並輸出詳細診斷資訊
        logger.info(f"開始為 {stock_id} {stock_name} 生成圖表...")

        # 檢查 stock_data 的結構
        if not isinstance(stock_data, dict):
            logger.error(f"繪製圖表失敗({stock_id}): stock_data 不是字典類型，而是 {type(stock_data)}")
            return

        # 檢查 data_df 是否存在
        if 'data_df' not in stock_data:
            logger.error(f"繪製圖表失敗({stock_id}): stock_data 中缺少 'data_df' 鍵。可用的鍵: {list(stock_data.keys())}")
            return

        df_with_indicators = stock_data.get('data_df')

        # 檢查 df_with_indicators 是否為 DataFrame
        if not isinstance(df_with_indicators, pd.DataFrame):
            logger.error(f"繪製圖表失敗({stock_id}): data_df 不是 DataFrame，而是 {type(df_with_indicators)}")
            return

        if df_with_indicators is None or df_with_indicators.empty:
            logger.error(f"繪製圖表失敗({stock_id}): data_df 為空或 None")
            return

        # 輸出 DataFrame 的基本資訊
        logger.info(f"DataFrame 資訊: 行數={len(df_with_indicators)}, 列數={len(df_with_indicators.columns)}")
        logger.info(f"DataFrame 列名: {list(df_with_indicators.columns)}")

        # 檢查必要的列是否存在
        required_cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'MA5', 'MA20', 'MACD', 'Signal', 'Histogram', 'RSI']
        missing_cols = [col for col in required_cols if col not in df_with_indicators.columns]
        if missing_cols:
            logger.error(f"繪製圖表失敗({stock_id}): 缺少必要的列: {missing_cols}")
            return

        # 2. 截取最近 120 天的數據用於繪圖，使用 .copy() 避免警告
        df_plot = df_with_indicators.tail(120).copy()
        logger.info(f"截取後的 DataFrame 行數: {len(df_plot)}")

        if len(df_plot) < 20: # 確保有足夠數據繪製指標
            logger.warning(f"繪製圖表失敗({stock_id}): 有效數據不足 (<20)。")
            return

        # 3. 檢查是否有 NaN 值
        nan_counts = df_plot.isna().sum()
        if nan_counts.sum() > 0:
            logger.warning(f"DataFrame 中存在 NaN 值: {nan_counts[nan_counts > 0]}")
            # 填充 NaN 值，避免繪圖錯誤
            df_plot = df_plot.fillna(method='ffill').fillna(method='bfill')

        # 4. 檢查 DataFrame 的索引類型 - 移到這裡，確保 df_plot 已經定義
        if not isinstance(df_plot.index, pd.DatetimeIndex):
            logger.warning(f"DataFrame 索引不是 DatetimeIndex，嘗試轉換...")
            try:
                # 如果 'Date' 是列而不是索引
                if 'Date' in df_plot.columns:
                    df_plot.set_index('Date', inplace=True)
                # 嘗試將現有索引轉換為 DatetimeIndex
                df_plot.index = pd.to_datetime(df_plot.index)
                logger.info("索引轉換成功")
            except Exception as e:
                logger.error(f"轉換索引失敗: {e}")
                # 如果轉換失敗，嘗試創建一個假的日期索引
                logger.info("嘗試創建假的日期索引...")
                df_plot.index = pd.date_range(start='2023-01-01', periods=len(df_plot))
                logger.info("創建假的日期索引成功")

        # 5. 修正樣式設定：使用 'gridstyle' 而不是 'gridwidth'
        mc = mpf.make_marketcolors(up='r', down='g', inherit=True)
        s = mpf.make_mpf_style(base_mpf_style='yahoo', marketcolors=mc, gridstyle='--')

        # 6. 從截取後的 df_plot 創建附加圖，確保所有數據長度一致
        logger.info("開始創建附加圖...")
        try:
            apds = [
                mpf.make_addplot(df_plot['MA5'], color='orange', width=0.7),
                mpf.make_addplot(df_plot['MA20'], color='blue', width=0.7),
                mpf.make_addplot(df_plot['Histogram'], type='bar', panel=1, color='grey', alpha=0.5),
                mpf.make_addplot(df_plot[['MACD', 'Signal']], panel=1),
                mpf.make_addplot(df_plot['RSI'], panel=2),
            ]
            logger.info("附加圖創建成功")
        except Exception as e:
            logger.error(f"創建附加圖時出錯: {e}")
            # 嘗試簡化附加圖
            try:
                logger.info("嘗試簡化附加圖...")
                apds = [mpf.make_addplot(df_plot['MA5'], color='orange')]
                logger.info("簡化附加圖創建成功")
            except Exception as e2:
                logger.error(f"創建簡化附加圖也失敗: {e2}")
                apds = []

        # 7. 設定圖表標題與儲存路徑
        title = f"{stock_id} {stock_name} (Score: {int(round(stock_data.get('combined_score', 0)))})"
        if not save_path:
            save_path = os.path.join(CHARTS_DIR, f"{stock_id}_report.png")

        logger.info(f"準備繪製圖表，儲存至: {save_path}")

        # 8. 繪製圖表
        try:
            mpf.plot(
                df_plot,
                type='candle',
                style=s,
                title=title,
                volume=True,
                addplot=apds,
                panel_ratios=(6, 2, 2), # (主圖, MACD面板, RSI面板)
                figsize=(16, 9),
                savefig=dict(fname=save_path, dpi=150)
            )
            logger.info(f"✅ 圖表繪製成功並儲存至: {save_path}")
        except Exception as plot_error:
            logger.error(f"繪製圖表時出錯: {plot_error}")
            # 嘗試最簡單的繪圖方式
            try:
                logger.info("嘗試最簡單的繪圖方式...")
                plt.figure(figsize=(16, 9))
                plt.title(title)
                plt.plot(df_plot['Close'], label='Close')
                plt.savefig(save_path, dpi=150)
                logger.info(f"✅ 簡化圖表繪製成功並儲存至: {save_path}")
            except Exception as simple_plot_error:
                logger.error(f"簡化繪圖也失敗: {simple_plot_error}")

    except Exception as e:
        logger.error(f"為 {stock_id} 繪製圖表時發生內部錯誤: {e}")
        logger.error(traceback.format_exc())  # 輸出完整的堆疊追蹤
    finally:
        # 9. 確保關閉圖表，釋放記憶體，避免在大量迴圈中耗盡資源
        plt.close('all')

# ==============================================
# 使用者介面 (UI) 相關函式
# ==============================================

def input_stock_ids():
    while True:
        user_input = input("\n請輸入欲分析的4位台股股票代碼（用逗號或空格分隔，例如 2330,2303 2603）：\n> ")
        stock_ids = re.findall(r'\b\d{4}\b', user_input)
        if stock_ids:
            print(f"已識別股票代碼: {', '.join(stock_ids)}")
            return list(dict.fromkeys(stock_ids))
        print("❗ 輸入格式錯誤或未包含有效的4位數代碼，請重新輸入！")

def filter_by_volume(stock_data, min_volume_shares=1000000, days=22):
    filtered = {}
    print(f"\n進行成交量過濾 (近{days}日成交量 > {min_volume_shares/1000:,.0f} 張)...")
    for stock_id, data in stock_data.items():
        if not data or 'data' not in data: continue
        df = pd.DataFrame(data['data'])
        if len(df) >= days and (df['Volume'].tail(days) >= min_volume_shares).all():
            filtered[stock_id] = data
    print(f"成交量過濾後，剩下 {len(filtered)} / {len(stock_data)} 支股票。")
    return filtered

def top_percentile_filter(results, score_key='combined_score', percentile=98):
    if not results: return {}
    all_scores = [r.get(score_key, 0) for r in results.values()]
    if not all_scores: return {}
    threshold = np.percentile(all_scores, percentile)
    print(f"\n進行頂尖篩選 (分數需 >= {threshold:.0f}，即前 {100-percentile}%)")
    filtered = {sid: r for sid, r in results.items() if r.get(score_key, 0) >= threshold}
    print(f"頂尖篩選後，剩下 {len(filtered)} / {len(results)} 支股票。")
    return filtered

def print_top_stocks(results, top_n=10):
    sorted_stocks = sorted(results.values(), key=lambda x: x.get('combined_score', 0), reverse=True)[:top_n]
    print("\n" + "="*60); print("🏆" * 8 + " 前十名績優股列表 " + "🏆" * 8); print("="*60)
    print(f"{'排名':<4}{'代碼':<6}{'名稱':<12}{'分數':<6}{'價格':<8}{'月漲幅(%)':<10}{'建議':<8}")
    print("-" * 60)
    for i, stock in enumerate(sorted_stocks, 1):
        print(f"{i:<4}{stock.get('stock_id',''):<6}{stock.get('stock_name',''):<12}"
              f"{int(round(stock.get('combined_score', 0))):<6}"
              f"{int(round(stock.get('last_price', 0))):<8}"
              f"{int(round(stock.get('price_change',{}).get('1m',0))):<10}"
              f"{stock.get('recommendation',''):<8}")
    print("="*60)

# ==============================================================================
# 通知函數
# ==============================================================================
async def send_notification(session: aiohttp.ClientSession, message: str, files: List[str] = None):
    """發送通知到 Telegram 和 Discord"""
    # 如果沒有提供檔案，嘗試從預設目錄獲取
    if not files:
        chart_dir = "/content/results/charts/"
        if os.path.exists(chart_dir):
            # 獲取目錄中的所有 PNG 檔案
            files = [os.path.join(chart_dir, f) for f in os.listdir(chart_dir)
                    if f.endswith('.png') and os.path.isfile(os.path.join(chart_dir, f))]
            if files:
                logger.info(f"找到 {len(files)} 個圖表檔案在 {chart_dir}")
            else:
                logger.warning(f"在 {chart_dir} 中找不到任何 PNG 檔案")

    # Telegram
    try:
        # 遍歷所有 Telegram Chat ID
        for chat_id in TELEGRAM_CHAT_ID:
            # 發送文字訊息
            url_msg = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage"
            payload = {"chat_id": chat_id, "text": message, "parse_mode": "Markdown"}
            async with session.post(url_msg, json=payload, timeout=20) as response:
                if response.status == 200:
                    logger.info(f"Telegram 摘要已成功發送到 chat_id: {chat_id}。")
                else:
                    response_text = await response.text()
                    logger.error(f"Telegram 摘要發送失敗: {response.status} - {response_text}")

            # 如果有檔案，也遍歷發送
            if files:
                url_photo = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendPhoto"
                sent_count = 0
                for file_path in files:
                    if os.path.exists(file_path):
                        try:
                            data = aiohttp.FormData()
                            data.add_field('chat_id', str(chat_id))  # 確保 chat_id 是字串
                            # 添加可選的圖片說明
                            caption = os.path.basename(file_path).replace('_report.png', '').replace('_', ' ')
                            data.add_field('caption', caption)

                            with open(file_path, 'rb') as f:
                                data.add_field('photo', f, filename=os.path.basename(file_path))
                                async with session.post(url_photo, data=data, timeout=60) as response:
                                    if response.status == 200:
                                        sent_count += 1
                                    else:
                                        response_text = await response.text()
                                        logger.error(f"Telegram 圖檔發送失敗: {response.status} - {response_text}")

                            # 添加小延遲以避免 Telegram API 限制
                            await asyncio.sleep(0.5)
                        except Exception as file_error:
                            logger.error(f"發送檔案 {file_path} 時出錯: {file_error}")

                logger.info(f"Telegram 圖檔已成功發送到 chat_id: {chat_id} ({sent_count}/{len(files)}個)。")
    except Exception as e:
        logger.error(f"發送 Telegram 通知時異常: {e}")
        traceback.print_exc()

    # Discord
    try:
        # 將訊息分段發送，避免超過 Discord 的限制
        message_chunks = [message[i:i+1900] for i in range(0, len(message), 1900)]

        for chunk in message_chunks:
            webhook = DiscordWebhook(url=DISCORD_WEBHOOK_URL, content=f"```\n{chunk}\n```")
            response = webhook.execute()
            if response and response.ok:
                logger.info("Discord 文字通知發送成功。")
            else:
                status_code = response.status_code if response else "未知"
                content = response.content if response else "未知"
                logger.error(f"Discord 文字通知失敗: {status_code} {content}")

        # 分批發送檔案，每批最多 10 個檔案
        if files:
            batch_size = 10
            for i in range(0, len(files), batch_size):
                batch_files = files[i:i+batch_size]
                webhook = DiscordWebhook(url=DISCORD_WEBHOOK_URL, content=f"圖表批次 {i//batch_size + 1}/{(len(files)-1)//batch_size + 1}")

                for file_path in batch_files:
                    if os.path.exists(file_path):
                        try:
                            with open(file_path, 'rb') as f:
                                webhook.add_file(file=f.read(), filename=os.path.basename(file_path))
                        except Exception as file_error:
                            logger.error(f"添加檔案 {file_path} 到 Discord webhook 時出錯: {file_error}")

                response = webhook.execute()
                if response and response.ok:
                    logger.info(f"Discord 圖檔批次 {i//batch_size + 1} 發送成功。")
                else:
                    status_code = response.status_code if response else "未知"
                    content = response.content if response else "未知"
                    logger.error(f"Discord 圖檔批次 {i//batch_size + 1} 發送失敗: {status_code} {content}")
    except Exception as e:
        logger.error(f"發送 Discord 通知時異常: {e}")
        traceback.print_exc()

def format_notification_message(filtered_results: Dict) -> str:
    """格式化通知訊息"""
    try:
        current_time = datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')

        if not filtered_results:
            return f"""
🤖 台股技術分析報告
📅 分析時間: {current_time}

❌ 本次分析未找到符合條件的優質股票
💡 建議調整篩選條件或關注市場變化
"""

        # 獲取圖表檔案數量
        chart_dir = "/content/results/charts/"
        chart_count = 0
        if os.path.exists(chart_dir):
            chart_files = [f for f in os.listdir(chart_dir) if f.endswith('.png') and os.path.isfile(os.path.join(chart_dir, f))]
            chart_count = len(chart_files)

        message = f"""
🤖 台股技術分析報告
📅 分析時間: {current_time}
🎯 找到 {len(filtered_results)} 支優質股票
📊 生成 {chart_count} 張技術分析圖表

📈 TOP {min(5, len(filtered_results))} 推薦股票:
"""

        for rank, (stock_id, result) in enumerate(list(filtered_results.items())[:5], 1):
            trend = result.get('trend_analysis', {})
            tech = result.get('technical_analysis', {})
            recommendation = result.get('recommendation', {})

            # 獲取更多技術指標數據
            macd = tech.get('macd_value', 0)
            signal = tech.get('signal_value', 0)
            macd_status = "多頭" if macd > signal else "空頭" if macd < signal else "中性"

            message += f"""
{rank}. {result.get('stock_name', '')} ({stock_id})
   💰 價格: {result.get('current_price', 0):.2f} TWD
   📈 評分: {result.get('combined_score', 0):.1f}/100
   🎯 建議: {recommendation.get('action', '觀望')}
   📊 趨勢: {trend.get('trend', '盤整')}
   🔍 RSI: {tech.get('rsi_value', 50):.1f} ({tech.get('rsi_status', '中性')})
   📉 MACD: {macd_status}
"""

        # 添加圖表資訊
        if chart_count > 0:
            message += f"""
📊 詳細分析圖表:
• 已生成 {chart_count} 張技術分析圖表
• 圖表包含K線、均線、成交量、MACD及RSI指標
• 圖表將在此訊息後發送
"""

        message += f"""
⚠️  投資提醒:
• 本分析僅供參考，投資有風險
• 建議結合基本面分析
• 請做好風險控制
"""

        return message.strip()

    except Exception as e:
        logger.error(f"格式化通知訊息時發生錯誤: {e}")
        traceback.print_exc()
        return f"台股分析完成，但訊息格式化失敗: {str(e)}"

#=======================================
# 主程式執行流程
# ==============================================

async def main_menu():
    print("="*40)
    print("==== 台股多股技術分析與篩選工具 ====")
    print("="*40)
    print("🚀 開始自動分析所有股票...")

    print("\n[階段1/5] 獲取所有股票基本資料...")
    stocks_info = get_stock_basic_info()
    stock_ids = list(stocks_info.keys())

    print("\n[階段2/5] 批次下載歷史資料 (一年期)...")
    all_stocks_data = fetch_multiple_stocks_data(stocks_info, period='1y')
    if not all_stocks_data:
        print("❌ 無法取得任何股票歷史資料，流程中止。")
        return

    print("\n[階段3/5] 過濾低成交量股票...")
    filtered_data = filter_by_volume(all_stocks_data, min_volume_shares=1000 * 1000)
    if not filtered_data:
        print("❌ 沒有任何個股符合成交量標準 (近一個月每日成交量 > 1000張)。")
        return

    print("\n[階段4/5] 進行核心分析與綜合評分...")
    all_analysis_results = {}
    for i, (stock_id, stock_data) in enumerate(filtered_data.items()):
        print(f"  分析中 ({i+1}/{len(filtered_data)}): {stock_id} {stock_data.get('stock_name', '')}")
        analysis_result = analyze_stock(stock_id, stock_data)
        if analysis_result.get('success'):
            scores, final_score = calculate_comprehensive_score(analysis_result)
            analysis_result['scores'] = {k: int(round(v)) for k, v in scores.items()}
            analysis_result['combined_score'] = int(round(final_score))
            analysis_result['recommendation'] = generate_recommendation(final_score)
            all_analysis_results[stock_id] = analysis_result

    if not all_analysis_results: # <- 修正了變數名稱並補上冒號
        print("❌ 核心分析後，沒有任何股票符合標準。")
        # 發送空結果通知
        async with aiohttp.ClientSession() as session:
            empty_message = format_notification_message({})
            await send_notification(session, empty_message)
        print("📱 空結果通知已發送")
        return # 結束函式

    print("\n[階段5/5] 篩選頂尖個股並產出報告...")
    elite_results = top_percentile_filter(all_analysis_results, percentile=98)
    results_to_show = elite_results if elite_results else all_analysis_results

    if not results_to_show:
        print("🤷 找不到任何可顯示的結果。")
        return

    if not elite_results:
        print("🤷 沒有個股分數達到前2%的門檻。將顯示原始清單中分數最高的股票。")

    print_top_stocks(results_to_show, top_n=10)

    print("\n--- 正在為頂尖個股生成詳細圖表報告 ---")
    top_stocks_to_report = sorted(results_to_show.values(), key=lambda x: x.get('combined_score', 0), reverse=True)[:10]
    for stock in top_stocks_to_report:
        generate_detailed_analysis_report(
            stock_id=stock['stock_id'],
            stock_name=stock['stock_name'],
            stock_data=stock
        )
    print("\n🎉 全部流程執行完畢！")
    # 即使沒有結果也發送通知            try:
    try:
        async with aiohttp.ClientSession() as session:
            message = format_notification_message({})
            await send_notification(session, message)
            print("📱 空結果通知已發送")
    except Exception as e:
        logger.error(f"發送空結果通知時發生錯誤: {e}")
if __name__ == "__main__":
    set_chinese_font()
    asyncio.run(main_menu())

In [ ]:

# -*- coding: utf-8 -*-
"""
台股多股技術分析與篩選工具
=======================
功能：
- 自動獲取台股清單
- 批次下載歷史數據
- 技術指標分析
- 綜合評分排名
- 自動通知推送
- 圖表生成與報告匯出

作者：股票分析系統
版本：v2.0
更新日期：2025-08-08
"""

# ==============================================================================
# 1. 基礎 Python 標準庫
# ==============================================================================
import os
import sys
import json
import time
import traceback
import platform
import re
from io import StringIO
from datetime import datetime
from typing import List, Dict
import logging
import re  # 用於正則表達式處理
# ==============================================================================
# 2. 第三方套件
# ==============================================================================
# 數據處理與科學計算
import pandas as pd
import numpy as np

# 網路請求
import requests

# 金融數據源
import yfinance as yf

# 數據可視化
import matplotlib.pyplot as plt
from matplotlib import font_manager
import mplfinance as mpf
import plotly.graph_objects as go
from pathlib import Path

# 通知套件 (修正：新增 discord_webhook 匯入)
from discord_webhook import DiscordWebhook
import telegram
# 中文字體支援（如果有安裝的話）
try:
    import chineseize_matplotlib
    chineseize_matplotlib.chineseize()
    print("✅ 已載入 chineseize_matplotlib 中文字體支援")
except ImportError:
    print("⚠️ 未安裝 chineseize_matplotlib，將使用自定義字體設定")

import pandas as pd
import numpy as np
import os
import logging
import traceback
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import json
import re
# ==============================================================================
# 3. 全域設定與目錄結構
# ==============================================================================
# 目錄結構設定
CACHE_DIR = 'cache'
RESULTS_DIR = 'results'
CHARTS_DIR = os.path.join(RESULTS_DIR, 'charts')
STOCK_LIST_PATH = os.path.join(CACHE_DIR, 'stock_list.csv')
HISTORY_DATA_CACHE_DIR = os.path.join(CACHE_DIR, 'history_data')

# 初始化目錄
for directory in [CACHE_DIR, RESULTS_DIR, CHARTS_DIR, HISTORY_DATA_CACHE_DIR]:
    os.makedirs(directory, exist_ok=True)

# ==============================================================================
# 4. 通訊與通知設定
# ==============================================================================
# 建議將來使用環境變數或 .env 檔案管理密鑰，以策安全
TELEGRAM_TOKEN = "7902318521:AAEYoDMqwfHabI7L1SRiE4z33aFay42-VGE"
TELEGRAM_CHAT_ID = [879781796, 8113868436]
DISCORD_WEBHOOK_URL = "https://discord.com/api/webhooks/1362715080734802102/Jma7A3VhEQrrRxIX_JW2l6rATjAZXsGXGfnJuAMqmS1QvqG_2ptg3vr_nsVnuV_PlnBl"

# ==============================================================================
# 5. 日誌系統設定
# ==============================================================================
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=[
        logging.FileHandler("stock_analysis.log", encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# ==============================================================================
# 6. 輔助函式與環境設定
# ==============================================================================
def set_chinese_font():
    """
    自動設定適合當前作業系統的中文字體，用於 Matplotlib 顯示。

    支援的作業系統：
    - Windows: Microsoft YaHei
    - macOS: PingFang HK
    - Linux: Noto Sans CJK / WenQuanYi Zen Hei
    """
    try:
        system = platform.system()
        font_name = ""

        if system == 'Windows':
            # Windows 系統字體設定
            font_candidates = ['Microsoft YaHei', 'SimHei', 'KaiTi']
            for font in font_candidates:
                try:
                    plt.rcParams['font.sans-serif'] = [font]
                    font_name = font
                    break
                except:
                    continue

        elif system == 'Darwin':  # macOS
            # macOS 系統字體設定
            font_candidates = ['PingFang HK', 'PingFang SC', 'Heiti TC', 'STHeiti']
            for font in font_candidates:
                try:
                    plt.rcParams['font.sans-serif'] = [font]
                    font_name = font
                    break
                except:
                    continue

        else:  # Linux 和其他系統
            # Linux 系統字體路徑
            font_paths = [
                '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc',
                '/usr/share/fonts/truetype/wqy/wqy-zenhei.ttc',
                '/usr/share/fonts/truetype/arphic/ukai.ttc',
                '/usr/share/fonts/truetype/arphic/uming.ttc',
            ]

            found_font_path = None
            for path in font_paths:
                if os.path.exists(path):
                    found_font_path = path
                    break

            if found_font_path:
                try:
                    font_manager.fontManager.addfont(found_font_path)
                    font_prop = font_manager.FontProperties(fname=found_font_path)
                    font_name = font_prop.get_name()
                    plt.rcParams['font.sans-serif'] = [font_name]
                except Exception as e:
                    logger.warning(f"載入字體失敗: {e}")
                    font_name = "DejaVu Sans"
            else:
                print("⚠️ 未在常見路徑找到中文字體，圖表中的中文可能無法正常顯示。")
                font_name = "DejaVu Sans"

        # 設定負號顯示
        plt.rcParams['axes.unicode_minus'] = False

        # 設定字體大小
        plt.rcParams['font.size'] = 10
        plt.rcParams['axes.titlesize'] = 12
        plt.rcParams['axes.labelsize'] = 10
        plt.rcParams['xtick.labelsize'] = 9
        plt.rcParams['ytick.labelsize'] = 9
        plt.rcParams['legend.fontsize'] = 9

        if font_name and font_name != "DejaVu Sans":
            print(f"✅ 已設定圖表字體為: {font_name} ({system})")
        else:
            print(f"⚠️ 使用預設字體: {font_name}")
            plt.rcParams['font.sans-serif'] = ['DejaVu Sans']

    except Exception as e:
        print(f"❌ 字體配置錯誤: {e}。使用預設字體。")
        plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
        plt.rcParams['axes.unicode_minus'] = False

def check_environment():
    """
    檢查執行環境與相依套件
    """
    print("🔍 檢查執行環境...")
    print(f"Python 版本: {sys.version}")
    print(f"作業系統: {platform.system()} {platform.release()}")

    # 檢查重要套件版本
    required_packages = {
        'pandas': pd.__version__,
        'numpy': np.__version__,
        'matplotlib': plt.matplotlib.__version__,
        'requests': requests.__version__,
        'yfinance': yf.__version__ if hasattr(yf, '__version__') else 'Unknown'
    }

    print("\n📦 套件版本:")
    for package, version in required_packages.items():
        print(f"  {package}: {version}")

    # 檢查目錄權限
    print(f"\n📁 工作目錄: {os.getcwd()}")
    print(f"快取目錄: {CACHE_DIR} {'✅' if os.access(CACHE_DIR, os.W_OK) else '❌'}")
    print(f"結果目錄: {RESULTS_DIR} {'✅' if os.access(RESULTS_DIR, os.W_OK) else '❌'}")

    print("=" * 50)
# ==============================================================================
# 1. 新增：K線型態中文翻譯字典與輔助函數
# ==============================================================================
import re

# 中文翻譯字典
PATTERN_TRANSLATIONS = {
    # 均線指標 (MA)
    "ma5": "5日均線",
    "ma10": "10日均線",
    "ma20": "20日均線 (月線)",
    "ma60": "60日均線 (季線)",
    "ma120": "120日均線 (半年線)",

    # K線基礎屬性
    "body size": "實體大小",
    "upper shadow": "上影線",
    "lower shadow": "下影線",
    "total range": "總波幅",
    "prev open": "昨日開盤價",
    "prev high": "昨日最高價",
    "prev low": "昨日最低價",
    "prev close": "昨日收盤價",
    "prev volume": "昨日成交量",

    # 常見K線型態 (可根據您的分析庫擴充)
    "hammer": "鎚子線",
    "hanging man": "吊人線",
    "inverted hammer": "倒鎚子線",
    "shooting star": "射擊之星",
    "doji": "十字星",
    "dragonfly doji": "蜻蜓十字",
    "gravestone doji": "墓碑十字",
    "bullish engulfing": "看漲吞噬",
    "bearish engulfing": "看跌吞噬",
    "morning star": "晨星",
    "evening star": "夜星",
    "marubozu": "光頭光腳K線"
}

def translate_pattern(pattern_name):
    """ 翻譯英文pattern為中文，如果找不到則返回原文 """
    # 移除 "看漲" 或 "看跌" 前綴來查找基礎模式
    clean_name = pattern_name.replace("看漲", "").replace("看跌", "").strip()

    # 處理帶有比較詞的模式 (例如 "body size > prev body size")
    parts = re.split(r'([<>=])', clean_name)
    if len(parts) > 1:
        translated_parts = [PATTERN_TRANSLATIONS.get(p.strip(), p.strip()) for p in parts]
        return " ".join(translated_parts)

    return PATTERN_TRANSLATIONS.get(clean_name, clean_name)

# 核心功能函式 (資料獲取、分析、評分)
def get_taiwan_stocks(cache_path=STOCK_LIST_PATH, force_update=False):
    if not force_update and os.path.exists(cache_path) and (time.time() - os.path.getmtime(cache_path)) < 86400:
        try:
            df = pd.read_csv(cache_path, dtype={'stock_id': str})
            if not df.empty and 'yahoo_symbol' in df.columns:
                print(f"✅ 從快取載入 {len(df)} 支股票清單。")
                return df
        except Exception as e:
            print(f"⚠️ 載入股票清單快取失敗: {e}。")

    print("🌐 正在獲取最新股票清單 (從證交所)...")
    try:
        urls = {
            "上市": "https://isin.twse.com.tw/isin/C_public.jsp?strMode=2",
            "上櫃": "https://isin.twse.com.tw/isin/C_public.jsp?strMode=4"
        }
        all_stocks_df = []
        for market, url in urls.items():
            response = requests.get(url, timeout=20)
            df = pd.read_html(StringIO(response.text))[0]
            df.columns = df.iloc[0]
            df = df.iloc[1:].dropna(thresh=3, axis=0)
            df['市場別'] = market
            all_stocks_df.append(df)

        df_combined = pd.concat(all_stocks_df, ignore_index=True)
        df_combined.columns = ['有價證券代號及名稱', '國際證券識別碼', '上市日', '市場別', '產業別', 'CFI', '備註']
        df_combined[['stock_id', 'stock_name']] = df_combined['有價證券代號及名稱'].str.split('　', n=1, expand=True)
        df_combined['stock_id'] = df_combined['stock_id'].str.strip()
        df_combined['stock_name'] = df_combined['stock_name'].str.strip()
        df_stocks = df_combined[df_combined['stock_id'].str.match(r'^\d{4}$')].copy()
        exclude_keywords = ['ETF', 'ETN', 'TDR', '受益', '指數', '購', '牛', '熊']
        df_stocks = df_stocks[~df_stocks['stock_name'].str.contains('|'.join(exclude_keywords), na=False)]
        df_stocks['yahoo_symbol'] = df_stocks.apply(
            lambda row: f"{row['stock_id']}.TW" if '上市' in row['市場別'] else f"{row['stock_id']}.TWO",
            axis=1
        )
        final_df = df_stocks[['stock_id', 'stock_name', '市場別', '產業別', 'yahoo_symbol']].drop_duplicates(subset=['stock_id']).reset_index(drop=True)
        final_df.rename(columns={'市場別': 'market', '產業別': 'industry'}, inplace=True)
        final_df.to_csv(cache_path, index=False)
        print(f"✅ 成功從證交所獲取 {len(final_df)} 支股票清單並保存。")
        return final_df
    except Exception as e:
        print(f"❌ 從證交所獲取股票清單時發生錯誤: {e}")
        if os.path.exists(cache_path):
            print("❗ 降級使用舊的快取股票清單。")
            return pd.read_csv(cache_path, dtype={'stock_id': str})
        return pd.DataFrame()

def get_stock_basic_info(force_update=False):
    stocks_df = get_taiwan_stocks(force_update=force_update)
    if stocks_df.empty: return {}
    return {str(row['stock_id']): row.to_dict() for _, row in stocks_df.iterrows()}

# ***** 關鍵修正函式 *****
# ==============================================================================
# 請用這兩個優化後的函數，完整替換您程式碼中對應的函數
# ==============================================================================

import traceback
import requests # 確保導入 requests

def fetch_stock_data(stock_id, yahoo_symbol, period='1y', force_update=False, retries=3, delay=3, cache_dir=HISTORY_DATA_CACHE_DIR):
    """
    【優化版】下載單一股票數據，具備更強的錯誤處理、日誌記錄和穩健性。
    """
    formatted_id = str(stock_id)
    os.makedirs(cache_dir, exist_ok=True)
    cache_path = os.path.join(cache_dir, f"{formatted_id}.json")

    if not force_update and os.path.exists(cache_path) and (time.time() - os.path.getmtime(cache_path)) < 86400:
        try:
            with open(cache_path, 'r', encoding='utf-8') as f:
                logger.info(f"從快取載入 {stock_id} ({yahoo_symbol}) 的數據。")
                return json.load(f)
        except Exception as e:
            logger.warning(f"讀取 {stock_id} 的快取檔案失敗: {e}，將重新從網路獲取。")

    for attempt in range(retries):
        try:
            time.sleep(0.5)
            logger.info(f"嘗試獲取 {stock_id} ({yahoo_symbol}) 數據... (第 {attempt + 1}/{retries} 次)")
            temp_df = yf.download(yahoo_symbol, period=period, progress=False, auto_adjust=True, timeout=20)

            if temp_df.empty:
                logger.warning(f"股票 {stock_id} ({yahoo_symbol}) 下載後數據為空。可能原因：代號錯誤、已下市或該時段無交易數據。")
                return None

            if isinstance(temp_df.columns, pd.MultiIndex):
                temp_df.columns = temp_df.columns.get_level_values(0)

            df = temp_df.reset_index()
            df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')
            result = {
                'data': df.to_dict('records'), 'stock_id': formatted_id, 'yahoo_symbol': yahoo_symbol,
                'last_price': df['Close'].iloc[-1] if not df.empty else 0,
                'fetch_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }
            with open(cache_path, 'w', encoding='utf-8') as f:
                json.dump(result, f, ensure_ascii=False, indent=2)
            return result
        except requests.exceptions.ConnectionError as e:
            logger.error(f"下載 {yahoo_symbol} 時發生網路連線錯誤: {e}。請檢查您的網路連線或防火牆設定。")
            if attempt < retries - 1: time.sleep(delay)
            else: return None
        except Exception as e:
            logger.error(f"下載 {yahoo_symbol} 時發生未預期錯誤: {e}")
            logger.debug(traceback.format_exc())
            if attempt < retries - 1: time.sleep(delay)
            else: return None
    return None

def fetch_multiple_stocks_data(stocks_info, period='1y', force_update=False):
    """
    【優化版】批次獲取多支股票數據，並提供詳細的成功/失敗摘要。
    """
    results = {}
    failed_stocks = []
    stocks_list = list(stocks_info.values())
    total_stocks = len(stocks_list)

    if total_stocks == 0:
        logger.warning("傳入的股票清單為空，無需獲取數據。")
        return {}

    for idx, stock in enumerate(stocks_list):
        stock_id = str(stock.get('stock_id'))
        stock_name = stock.get('stock_name', 'N/A')
        yahoo_symbol = stock.get('yahoo_symbol')

        if not yahoo_symbol:
            logger.warning(f"股票 {stock_id} {stock_name} 缺少 'yahoo_symbol'，已跳過。")
            failed_stocks.append(f"{stock_id} {stock_name} (缺少Symbol)")
            continue

        # 這裡不再印出日誌，交給 fetch_stock_data 內部處理
        print(f"[{idx + 1}/{total_stocks}] 正在處理: {stock_id} {stock_name}")

        stock_data = fetch_stock_data(
            stock_id=stock_id, yahoo_symbol=yahoo_symbol, period=period, force_update=force_update
        )

        if stock_data and 'data' in stock_data and stock_data['data']:
            stock_data.update(stock)
            results[stock_id] = stock_data
        else:
            failed_stocks.append(f"{stock_id} {stock_name}")

    logger.info("\n" + "="*20 + " 數據獲取摘要 " + "="*20)
    logger.info(f"任務完成。成功獲取 {len(results)} / {total_stocks} 支股票的數據。")
    if failed_stocks:
        logger.warning(f"未能獲取以下 {len(failed_stocks)} 支股票的數據: {', '.join(failed_stocks)}")
    logger.info("="*58 + "\n")

    return results

def add_technical_indicators(df):
    """
    為數據框添加技術指標。
    接受大寫欄位名稱 (Open, High, Low, Close, Volume)，
    內部使用小寫處理，然後返回保持原始大寫格式的DataFrame。

    參數:
    df - 包含OHLCV數據的DataFrame，使用大寫欄位名稱

    返回:
    DataFrame - 添加了技術指標的DataFrame，保持大寫欄位名稱
    """
    # 複製DataFrame以避免修改原始數據
    result = df.copy()

    # 檢查必要欄位是否存在
    required_cols = ['Close', 'High', 'Low']
    missing_cols = [col for col in required_cols if col not in result.columns]
    if missing_cols or result.empty:
        print(f"警告: 缺少計算技術指標所需的欄位: {missing_cols}")
        return df  # 返回原始DataFrame

    # 暫時將欄位名稱轉換為小寫以進行計算
    lowercase_mapping = {col: col.lower() for col in result.columns if col in ['Open', 'High', 'Low', 'Close', 'Volume']}
    result = result.rename(columns=lowercase_mapping)

    # 計算簡單移動平均線 (MA)
    for days in [5, 10, 20, 60, 120, 200]:
        result[f'ma{days}'] = result['close'].rolling(window=days).mean()

    # 計算相對強弱指標 (RSI)
    delta = result['close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / (avg_loss + 1e-9)  # 避免除以零
    result['rsi'] = 100 - (100 / (1 + rs))

    # 計算MACD
    exp1 = result['close'].ewm(span=12, adjust=False).mean()
    exp2 = result['close'].ewm(span=26, adjust=False).mean()
    result['macd'] = exp1 - exp2
    result['macdsignal'] = result['macd'].ewm(span=9, adjust=False).mean()
    result['histogram'] = result['macd'] - result['macdsignal']

    # 計算布林通道 (Bollinger Bands)
    result['middle_band'] = result['close'].rolling(window=20).mean()
    result['std'] = result['close'].rolling(window=20).std()
    result['bollinger_upper'] = result['middle_band'] + (result['std'] * 2)
    result['bollinger_lower'] = result['middle_band'] - (result['std'] * 2)

    # 計算KD指標
    low_min = result['low'].rolling(window=9).min()
    high_max = result['high'].rolling(window=9).max()
    result['rsv'] = 100 * ((result['close'] - low_min) / (high_max - low_min + 1e-9))
    result['k'] = result['rsv'].ewm(alpha=1/3, adjust=False).mean()
    result['d'] = result['k'].ewm(alpha=1/3, adjust=False).mean()

    # 填充NaN值
    result = result.fillna(0)

    # 將原始欄位名稱轉換回大寫
    uppercase_mapping = {col.lower(): col for col in df.columns if col in ['Open', 'High', 'Low', 'Close', 'Volume']}
    result = result.rename(columns=uppercase_mapping)

    return result

# ==============================================================================
# 請用這整個函數替換掉您原有的 analyze_stock 函數
# ==============================================================================
def analyze_stock(stock_id, stock_data, min_periods=60):
    """
    對單一股票進行完整的技術分析。

    此版本修正了 NameError 並重構了內部邏輯，使其更穩健、清晰。

    參數:
    stock_id (str): 股票代號
    stock_data (dict): 包含股票基本資料和歷史數據的字典
    min_periods (int): 進行分析所需的最少歷史數據點數量

    返回:
    dict or None: 包含分析結果的字典，如果數據無效或分析失敗則返回 None
    """
    try:
        # --- 步驟 1: 驗證傳入的數據是否有效 ---
        if not stock_data or not stock_data.get('data'):
            logger.warning(f"分析股票 {stock_id} 失敗：傳入的 stock_data 為空或缺少 'data' 鍵。")
            return None

        df = pd.DataFrame(stock_data['data'])

        if df.empty or len(df) < min_periods:
            logger.warning(f"跳過股票 {stock_id}：數據點不足 {min_periods} 個 (實際: {len(df)})。")
            return None

        # --- 步驟 2: 標準化欄位名稱與索引 ---
        # 統一日期欄位
        date_col_candidates = ['Date', 'date', '日期']
        date_col = next((col for col in date_col_candidates if col in df.columns), None)

        if date_col:
            df[date_col] = pd.to_datetime(df[date_col])
            df.set_index(date_col, inplace=True)
        else:
            logger.error(f"分析股票 {stock_id} 失敗：找不到可用的日期欄位。")
            return None

        # 統一價格欄位 (將中文欄位對應到標準英文欄位)
        column_mapping = {
            '開盤': 'Open', '最高': 'High', '最低': 'Low', '收盤': 'Close', '成交量': 'Volume',
            '開盤價': 'Open', '最高價': 'High', '最低價': 'Low', '收盤價': 'Close'
        }
        df.rename(columns=column_mapping, inplace=True)

        # 檢查必要的價格欄位是否存在
        required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
        missing_cols = [col for col in required_cols if col not in df.columns]
        if missing_cols:
            logger.error(f"分析股票 {stock_id} 失敗：缺少必要欄位: {missing_cols}。")
            return None

        # --- 步驟 3: 計算技術指標 ---
        df_with_indicators = add_technical_indicators(df.copy())

        # --- 步驟 4: 計算價格變化率 ---
        price_changes = {}
        periods = {'1d': 2, '1w': 5, '1m': 20, '3m': 60}
        for label, days in periods.items():
            if len(df_with_indicators) >= days:
                price_changes[label] = ((df_with_indicators['Close'].iloc[-1] / df_with_indicators['Close'].iloc[-days]) - 1) * 100
            else:
                price_changes[label] = 0

        # --- 步驟 5: 組合分析結果 ---
        last_row = df_with_indicators.iloc[-1]
        analysis_result = {
            'success': True,
            'stock_id': stock_id,
            'stock_name': stock_data.get('stock_name', 'N/A'),
            'market': stock_data.get('market', 'Unknown'),
            'industry': stock_data.get('industry', 'Unknown'),
            'last_price': last_row['Close'],
            'last_update': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'price_change': price_changes,
            'data_df': df_with_indicators # 保留 DataFrame 以供後續評分和繪圖使用
        }
        return analysis_result

    except Exception as e:
        # 捕捉所有未預期的錯誤，確保主程序不會因單一股票而中斷
        logger.error(f"分析股票 {stock_id} 時發生無法預期的異常: {e}")
        logger.debug(traceback.format_exc()) # 在日誌中記錄詳細的錯誤堆疊
        return None
def calculate_comprehensive_score(analysis_result):
    """
    計算綜合評分。
    """
    scores = {'trend_score': 50, 'momentum_score': 50, 'volume_score': 50}
    df = analysis_result.get('data_df')
    if df is None or df.empty or len(df) < 60:
        return scores, 50

    try:
        # 趨勢評分
        if all(col in df.columns for col in ['MA5', 'MA10', 'MA20', 'MA60']):
            if df['MA5'].iloc[-1] > df['MA10'].iloc[-1] > df['MA20'].iloc[-1] > df['MA60'].iloc[-1]:
                scores['trend_score'] = 85
            else:
                scores['trend_score'] = 20

        # 動量評分
        momentum_points = 50
        if 'RSI' in df.columns and df['RSI'].iloc[-1] > 60:
            momentum_points += 15
        if all(col in df.columns for col in ['MACD', 'Signal']) and df['MACD'].iloc[-1] > df['Signal'].iloc[-1]:
            momentum_points += 20
        if all(col in df.columns for col in ['K', 'D']) and df['K'].iloc[-1] > df['D'].iloc[-1]:
            momentum_points += 15
        scores['momentum_score'] = max(0, min(100, momentum_points))

        # 成交量評分
        if 'Volume' in df.columns:
            current_volume = df['Volume'].iloc[-1]
            avg_volume = df['Volume'].rolling(window=20).mean().iloc[-1]
            if current_volume > avg_volume * 1.5:
                scores['volume_score'] = 80

    except Exception as e:
        logger.error(f"計算評分時發生錯誤: {e}")

    final_score = scores['trend_score'] * 0.4 + scores['momentum_score'] * 0.45 + scores['volume_score'] * 0.15
    return scores, final_score

def generate_recommendation(score):
    """
    根據評分生成交易建議。
    """
    if score >= 75:
        return {
            'action': '強力買入',
            'reasons': ['技術指標顯示強勢', '成交量放大', '價格趨勢向上'],
            'confidence': int(score),
            'risk_level': '中等',
            'time_frame': '1-3個月',
            'stop_loss': None,
            'take_profit': None
        }
    elif score >= 60:
        return {
            'action': '買入',
            'reasons': ['技術指標正面', '成交量穩定'],
            'confidence': int(score),
            'risk_level': '低',
            'time_frame': '1-3個月',
            'stop_loss': None,
            'take_profit': None
        }
    elif score >= 40:
        return {
            'action': '持有',
            'reasons': ['技術指標中性', '無明顯趨勢'],
            'confidence': int(score),
            'risk_level': '低',
            'time_frame': '1個月',
            'stop_loss': None,
            'take_profit': None
        }
    elif score >= 25:
        return {
            'action': '觀望',
            'reasons': ['技術指標偏弱', '成交量不足'],
            'confidence': int(score),
            'risk_level': '高',
            'time_frame': '短期',
            'stop_loss': None,
            'take_profit': None
        }
    else:
        return {
            'action': '賣出',
            'reasons': ['技術指標看跌', '成交量萎縮'],
            'confidence': int(score),
            'risk_level': '高',
            'time_frame': '短期',
            'stop_loss': None,
            'take_profit': None
        }

def generate_detailed_analysis_report(stock_id, stock_name, stock_data, save_path=None):
    """
    生成詳細分析圖表（K線圖、技術指標） - 修正版本。
    """
    try:
        df_plot = stock_data.get('data_df')
        if df_plot is None or df_plot.empty:
            logger.error(f"無法生成圖表: {stock_id} 無有效數據")
            return None

        # 確保數據框有足夠的數據
        if len(df_plot) < 20:
            logger.error(f"數據不足以生成圖表: {stock_id}")
            return None

        df_plot = df_plot.tail(120).copy()

        # 確保必要的欄位存在
        required_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
        missing_columns = [col for col in required_columns if col not in df_plot.columns]
        if missing_columns:
            logger.error(f"圖表生成失敗，缺少欄位: {missing_columns}")
            return None

        # 確保索引是日期時間格式
        if not isinstance(df_plot.index, pd.DatetimeIndex):
            if 'Date' in df_plot.columns:
                df_plot['Date'] = pd.to_datetime(df_plot['Date'])
                df_plot.set_index('Date', inplace=True)
            else:
                df_plot.index = pd.date_range(start='2023-01-01', periods=len(df_plot), freq='D')

        # 設定圖表樣式
        mc = mpf.make_marketcolors(up='r', down='g', inherit=True)
        s = mpf.make_mpf_style(base_mpf_style='yahoo', marketcolors=mc)

        # 準備附加圖表
        apds = []

        # 添加移動平均線
        if all(col in df_plot.columns for col in ['MA5', 'MA20']):
            apds.append(mpf.make_addplot(df_plot[['MA5', 'MA20']], width=0.7))

        # 添加 MACD
        if all(col in df_plot.columns for col in ['MACD', 'Signal']):
            apds.append(mpf.make_addplot(df_plot[['MACD', 'Signal']], panel=2))

        # 添加 RSI
        if 'RSI' in df_plot.columns:
            apds.append(mpf.make_addplot(df_plot['RSI'], panel=3))

        title = f"{stock_id} {stock_name} (分數: {int(round(stock_data.get('combined_score', 0)))})"

        if not save_path:
            save_path = os.path.join(CHARTS_DIR, f"{stock_id}_report.png")

        # 生成圖表
        mpf.plot(
            df_plot,
            type='candle',
            style=s,
            title=title,
            volume=True,
            addplot=apds if apds else None,
            panel_ratios=(6, 2, 2, 2) if len(apds) >= 2 else (6, 2),
            figsize=(16, 9),
            savefig=dict(fname=save_path, dpi=150, bbox_inches='tight')
        )

        print(f"✅ 圖表報告已儲存至: {save_path}")
        plt.close('all')
        return save_path

    except Exception as e:
        logger.error(f"生成圖表時發生錯誤: {e}")
        traceback.print_exc()
        return None



def identify_candlestick_patterns(df):
    """
    識別常見的K線型態，不使用 talib、ta-lib 或 pandas_ta

    參數:
    df - 包含OHLCV數據的DataFrame

    返回:
    DataFrame - 包含各種K線型態的識別結果
    """
    # 複製數據以避免修改原始數據
    result_df = df.copy()

    # 計算實體長度和影線長度
    result_df['body_size'] = abs(result_df['close'] - result_df['open'])
    result_df['upper_shadow'] = result_df['high'] - result_df[['open', 'close']].max(axis=1)
    result_df['lower_shadow'] = result_df[['open', 'close']].min(axis=1) - result_df['low']
    result_df['total_range'] = result_df['high'] - result_df['low']

    # 計算前一天的數據
    result_df['prev_open'] = result_df['open'].shift(1)
    result_df['prev_high'] = result_df['high'].shift(1)
    result_df['prev_low'] = result_df['low'].shift(1)
    result_df['prev_close'] = result_df['close'].shift(1)
    result_df['prev_body_size'] = result_df['body_size'].shift(1)
    result_df['prev_total_range'] = result_df['total_range'].shift(1)

    # 計算前兩天的數據
    result_df['prev2_open'] = result_df['open'].shift(2)
    result_df['prev2_high'] = result_df['high'].shift(2)
    result_df['prev2_low'] = result_df['low'].shift(2)
    result_df['prev2_close'] = result_df['close'].shift(2)

    # 計算前三天的數據
    result_df['prev3_open'] = result_df['open'].shift(3)
    result_df['prev3_high'] = result_df['high'].shift(3)
    result_df['prev3_low'] = result_df['low'].shift(3)
    result_df['prev3_close'] = result_df['close'].shift(3)

    # 計算K線趨勢
    result_df['is_bullish'] = result_df['close'] > result_df['open']
    result_df['prev_is_bullish'] = result_df['prev_close'] > result_df['prev_open']
    result_df['prev2_is_bullish'] = result_df['prev2_close'] > result_df['prev2_open']
    result_df['prev3_is_bullish'] = result_df['prev3_close'] > result_df['prev3_open']

    # 計算簡單移動平均線用於判斷趨勢
    result_df['sma5'] = result_df['close'].rolling(window=5).mean()
    result_df['sma10'] = result_df['close'].rolling(window=10).mean()
    result_df['sma20'] = result_df['close'].rolling(window=20).mean()

    # 判斷趨勢
    result_df['uptrend'] = (result_df['sma5'] > result_df['sma20']) & (result_df['close'] > result_df['sma20'])
    result_df['downtrend'] = (result_df['sma5'] < result_df['sma20']) & (result_df['close'] < result_df['sma20'])

    # 初始化所有型態為0
    pattern_columns = [
        '十字星_中性', '錘子線_看漲', '錘子線_看跌', '流星線_看跌',
        '吞噬_看漲', '吞噬_看跌', '母子線_看漲', '母子線_看跌',
        '晨星_看漲', '暮星_看跌', '三白兵_看漲', '三黑鴉_看跌',
        '穿刺線_看漲', '烏雲蓋頂_看跌', '長腳十字線_中性', '上吊線_看跌',
        '陀螺_中性', '反轉_看漲', '反轉_看跌', '島形反轉_看漲', '島形反轉_看跌',
        '頭肩頂_看跌', '頭肩底_看漲', '雙頂_看跌', '雙底_看漲'
    ]

    for col in pattern_columns:
        result_df[col] = 0

    # 1. 識別十字星 (Doji)
    # 十字星: 開盤價和收盤價幾乎相同
    doji_condition = result_df['body_size'] <= 0.1 * result_df['total_range']
    result_df.loc[doji_condition, '十字星_中性'] = 1

    # 2. 識別錘子線 (Hammer) 和流星線 (Shooting Star)
    hammer_condition = (
        (result_df['lower_shadow'] >= 2 * result_df['body_size']) &
        (result_df['upper_shadow'] <= 0.1 * result_df['total_range']) &
        (result_df['body_size'] <= 0.3 * result_df['total_range'])
    )

    shooting_star_condition = (
        (result_df['upper_shadow'] >= 2 * result_df['body_size']) &
        (result_df['lower_shadow'] <= 0.1 * result_df['total_range']) &
        (result_df['body_size'] <= 0.3 * result_df['total_range'])
    )

    # 在下跌趨勢中的錘子線是看漲信號
    result_df.loc[hammer_condition & result_df['downtrend'], '錘子線_看漲'] = 1

    # 在上漲趨勢中的錘子線是看跌信號
    result_df.loc[hammer_condition & result_df['uptrend'], '錘子線_看跌'] = -1

    # 流星線通常是看跌信號
    result_df.loc[shooting_star_condition & result_df['uptrend'], '流星線_看跌'] = -1

    # 3. 識別吞噬型態 (Engulfing)
    bullish_engulfing = (
        (~result_df['prev_is_bullish']) &  # 前一天是下跌
        result_df['is_bullish'] &          # 當天是上漲
        (result_df['open'] < result_df['prev_close']) &  # 開盤低於前一天收盤
        (result_df['close'] > result_df['prev_open'])    # 收盤高於前一天開盤
    )

    bearish_engulfing = (
        result_df['prev_is_bullish'] &     # 前一天是上漲
        (~result_df['is_bullish']) &       # 當天是下跌
        (result_df['open'] > result_df['prev_close']) &  # 開盤高於前一天收盤
        (result_df['close'] < result_df['prev_open'])    # 收盤低於前一天開盤
    )

    result_df.loc[bullish_engulfing & result_df['downtrend'], '吞噬_看漲'] = 1
    result_df.loc[bearish_engulfing & result_df['uptrend'], '吞噬_看跌'] = -1

    # 4. 識別母子線 (Harami)
    bullish_harami = (
        (~result_df['prev_is_bullish']) &  # 前一天是下跌
        result_df['is_bullish'] &          # 當天是上漲
        (result_df['high'] < result_df['prev_high']) &   # 最高價低於前一天最高價
        (result_df['low'] > result_df['prev_low']) &     # 最低價高於前一天最低價
        (result_df['body_size'] < result_df['prev_body_size'])  # 實體小於前一天
    )

    bearish_harami = (
        result_df['prev_is_bullish'] &     # 前一天是上漲
        (~result_df['is_bullish']) &       # 當天是下跌
        (result_df['high'] < result_df['prev_high']) &   # 最高價低於前一天最高價
        (result_df['low'] > result_df['prev_low']) &     # 最低價高於前一天最低價
        (result_df['body_size'] < result_df['prev_body_size'])  # 實體小於前一天
    )

    result_df.loc[bullish_harami & result_df['downtrend'], '母子線_看漲'] = 1
    result_df.loc[bearish_harami & result_df['uptrend'], '母子線_看跌'] = -1

    # 5. 識別晨星 (Morning Star) 和暮星 (Evening Star)
    morning_star = (
        (~result_df['prev2_is_bullish']) &  # 前兩天是下跌
        (result_df['prev_body_size'] <= 0.3 * result_df['prev_total_range']) &  # 前一天是小實體
        result_df['is_bullish'] &           # 當天是上漲
        (result_df['close'] > (result_df['prev2_open'] + result_df['prev2_close']) / 2)  # 收盤價高於前兩天實體中點
    )

    evening_star = (
        result_df['prev2_is_bullish'] &     # 前兩天是上漲
        (result_df['prev_body_size'] <= 0.3 * result_df['prev_total_range']) &  # 前一天是小實體
        (~result_df['is_bullish']) &        # 當天是下跌
        (result_df['close'] < (result_df['prev2_open'] + result_df['prev2_close']) / 2)  # 收盤價低於前兩天實體中點
    )

    result_df.loc[morning_star & result_df['downtrend'], '晨星_看漲'] = 1
    result_df.loc[evening_star & result_df['uptrend'], '暮星_看跌'] = -1

    # 6. 識別三白兵 (Three White Soldiers) 和三黑鴉 (Three Black Crows)
    # 需要檢查連續三天的K線
    three_white_soldiers = (
        result_df['is_bullish'] &
        result_df['prev_is_bullish'] &
        result_df['prev2_is_bullish'] &
        (result_df['close'] > result_df['prev_close']) &
        (result_df['prev_close'] > result_df['prev2_close']) &
        (result_df['open'] > result_df['prev_open']) &
        (result_df['prev_open'] > result_df['prev2_open'])
    )

    three_black_crows = (
        (~result_df['is_bullish']) &
        (~result_df['prev_is_bullish']) &
        (~result_df['prev2_is_bullish']) &
        (result_df['close'] < result_df['prev_close']) &
        (result_df['prev_close'] < result_df['prev2_close']) &
        (result_df['open'] < result_df['prev_open']) &
        (result_df['prev_open'] < result_df['prev2_open'])
    )

    result_df.loc[three_white_soldiers, '三白兵_看漲'] = 1
    result_df.loc[three_black_crows, '三黑鴉_看跌'] = -1

    # 7. 識別穿刺線 (Piercing) 和烏雲蓋頂 (Dark Cloud Cover)
    piercing = (
        (~result_df['prev_is_bullish']) &  # 前一天是下跌
        result_df['is_bullish'] &          # 當天是上漲
        (result_df['open'] < result_df['prev_low']) &  # 開盤低於前一天最低價
        (result_df['close'] > (result_df['prev_open'] + result_df['prev_close']) / 2) &  # 收盤價高於前一天實體中點
        (result_df['close'] < result_df['prev_open'])  # 收盤價低於前一天開盤價
    )

    dark_cloud_cover = (
        result_df['prev_is_bullish'] &     # 前一天是上漲
        (~result_df['is_bullish']) &       # 當天是下跌
        (result_df['open'] > result_df['prev_high']) &  # 開盤高於前一天最高價
        (result_df['close'] < (result_df['prev_open'] + result_df['prev_close']) / 2) &  # 收盤價低於前一天實體中點
        (result_df['close'] > result_df['prev_close'])  # 收盤價高於前一天收盤價
    )

    result_df.loc[piercing & result_df['downtrend'], '穿刺線_看漲'] = 1
    result_df.loc[dark_cloud_cover & result_df['uptrend'], '烏雲蓋頂_看跌'] = -1

    # 8. 識別長腳十字線 (Long-Legged Doji)
    long_legged_doji = (
        doji_condition &
        (result_df['upper_shadow'] >= 0.3 * result_df['total_range']) &
        (result_df['lower_shadow'] >= 0.3 * result_df['total_range'])
    )

    result_df.loc[long_legged_doji, '長腳十字線_中性'] = 1

    # 9. 識別上吊線 (Hanging Man)
    hanging_man = (
        (result_df['lower_shadow'] >= 2 * result_df['body_size']) &
        (result_df['upper_shadow'] <= 0.1 * result_df['total_range']) &
        (result_df['body_size'] <= 0.3 * result_df['total_range']) &
        result_df['uptrend']
    )

    result_df.loc[hanging_man, '上吊線_看跌'] = -1

    # 10. 識別陀螺 (Spinning Top)
    spinning_top = (
        (result_df['body_size'] <= 0.3 * result_df['total_range']) &
        (result_df['upper_shadow'] >= 0.2 * result_df['total_range']) &
        (result_df['lower_shadow'] >= 0.2 * result_df['total_range'])
    )

    result_df.loc[spinning_top, '陀螺_中性'] = 1

    # 11. 識別反轉型態 (Reversal)
    # 看漲反轉: 在下跌趨勢中，出現一根大陽線，收盤價高於前幾天的最高價
    bullish_reversal = (
        result_df['is_bullish'] &
        result_df['downtrend'] &
        (result_df['close'] > result_df[['prev_high', 'prev2_high']].max(axis=1))
    )

    # 看跌反轉: 在上漲趨勢中，出現一根大陰線，收盤價低於前幾天的最低價
    bearish_reversal = (
        (~result_df['is_bullish']) &
        result_df['uptrend'] &
        (result_df['close'] < result_df[['prev_low', 'prev2_low']].min(axis=1))
    )

    result_df.loc[bullish_reversal, '反轉_看漲'] = 1
    result_df.loc[bearish_reversal, '反轉_看跌'] = -1

    # 12. 識別島形反轉 (Island Reversal)
    # 看漲島形反轉: 在下跌趨勢中，出現向下跳空，然後價格在低位盤整，最後向上跳空
    bullish_island_reversal = (
        result_df['downtrend'] &
        (result_df['low'] > result_df['prev_high']) &  # 向上跳空
        (result_df['prev_low'] > result_df['prev2_high'])  # 前一天向下跳空
    )

    # 看跌島形反轉: 在上漲趨勢中，出現向上跳空，然後價格在高位盤整，最後向下跳空
    bearish_island_reversal = (
        result_df['uptrend'] &
        (result_df['high'] < result_df['prev_low']) &  # 向下跳空
        (result_df['prev_high'] < result_df['prev2_low'])  # 前一天向上跳空
    )

    result_df.loc[bullish_island_reversal, '島形反轉_看漲'] = 1
    result_df.loc[bearish_island_reversal, '島形反轉_看跌'] = -1

    # 13. 識別頭肩頂和頭肩底
    # 這些型態需要更多的數據點和複雜的邏輯，這裡提供一個簡化版本

    # 頭肩頂: 三個高點，中間的高點最高，兩側高點大致相等
    head_shoulders_top = (
        result_df['uptrend'] &
        (result_df['prev2_high'] < result_df['prev_high']) &  # 左肩低於頭部
        (result_df['high'] < result_df['prev_high']) &        # 右肩低於頭部
        (abs(result_df['high'] - result_df['prev2_high']) / result_df['prev2_high'] < 0.05)  # 左右肩大致相等
    )

    # 頭肩底: 三個低點，中間的低點最低，兩側低點大致相等
    head_shoulders_bottom = (
        result_df['downtrend'] &
        (result_df['prev2_low'] > result_df['prev_low']) &  # 左肩高於頭部
        (result_df['low'] > result_df['prev_low']) &        # 右肩高於頭部
        (abs(result_df['low'] - result_df['prev2_low']) / result_df['prev2_low'] < 0.05)  # 左右肩大致相等
    )

    result_df.loc[head_shoulders_top, '頭肩頂_看跌'] = -1
    result_df.loc[head_shoulders_bottom, '頭肩底_看漲'] = 1

    # 14. 識別雙頂和雙底
    # 雙頂: 兩個相近的高點，中間有一個低點
    double_top = (
        result_df['uptrend'] &
        (abs(result_df['high'] - result_df['prev2_high']) / result_df['prev2_high'] < 0.03) &  # 兩個高點相近
        (result_df['prev_high'] < result_df['high']) &  # 中間有一個低點
        (result_df['prev_high'] < result_df['prev2_high'])
    )

    # 雙底: 兩個相近的低點，中間有一個高點
    double_bottom = (
        result_df['downtrend'] &
        (abs(result_df['low'] - result_df['prev2_low']) / result_df['prev2_low'] < 0.03) &  # 兩個低點相近
        (result_df['prev_low'] > result_df['low']) &  # 中間有一個高點
        (result_df['prev_low'] > result_df['prev2_low'])
    )

    result_df.loc[double_top, '雙頂_看跌'] = -1
    result_df.loc[double_bottom, '雙底_看漲'] = 1

    return result_df
# ==============================================================================
# K線型態優先級定義 (數字越大優先級越高)
PATTERN_PRIORITY = {
    # 頂部反轉型態 (高優先級)
    "頭肩頂": 10,
    "雙頂": 9,
    "三頂": 8,
    "圓頂": 7,
    "上升楔形": 7,
    "上升三角形": 6,

    # 底部反轉型態 (高優先級)
    "頭肩底": 10,
    "雙底": 9,
    "三底": 8,
    "圓底": 7,
    "下降楔形": 7,
    "下降三角形": 6,

    # 持續型態 (中等優先級)
    "對稱三角形": 5,
    "矩形": 5,
    "旗形": 4,
    "三角旗": 4,

    # 單日型態 (較低優先級)
    "吞噬": 3,
    "十字星": 3,
    "錘子": 3,
    "上吊線": 3,
    "流星": 3,
    "多頭吞噬": 3,
    "空頭吞噬": 3,
    "多頭刺透": 2,
    "空頭烏雲蓋頂": 2,
    "多頭反轉": 2,
    "空頭反轉": 2,

    # 缺口型態
    "突破缺口": 3,
    "逃逸缺口": 3,
    "消耗缺口": 2,

    # 其他常見型態
    "島狀反轉": 6,
    "V型反轉": 5,
    "W型反轉": 5,
    "箱體突破": 4,
    "黃昏之星": 4,
    "曙光之星": 4,
    "多頭三星": 3,
    "空頭三星": 3
}

# ==============================================================================

def generate_condensed_pattern_report_and_summary(full_pattern_report):
    """
    將完整的K線型態報告簡化為精華摘要，並提取結構化數據

    Args:
        full_pattern_report (str): 完整的K線型態報告

    Returns:
        tuple: (condensed_report, report_summary)
            - condensed_report (str): 簡化後的報告文本
            - report_summary (dict): 結構化的報告摘要數據
    """
    # 初始化返回值
    condensed_report = ""
    report_summary = {
        "daily": {"bullish": [], "bearish": [], "neutral": []},
        "weekly": {"bullish": [], "bearish": [], "neutral": []},
        "monthly": {"bullish": [], "bearish": [], "neutral": []}
    }

    # 檢查報告是否為空或無效
    if not full_pattern_report or not isinstance(full_pattern_report, str):
        return "無法生成摘要：報告為空或格式錯誤", report_summary

    try:
        # 將報告分割為不同時間週期的部分
        periods = ["日線", "週線", "月線"]
        period_keys = ["daily", "weekly", "monthly"]
        pattern_types = ["看漲", "看跌", "中性"]
        type_keys = ["bullish", "bearish", "neutral"]

        # 初始化結果字符串
        result_parts = []

        # 檢查報告格式 - 嘗試尋找時間週期標記
        has_valid_format = False
        for period in periods:
            if period in full_pattern_report:
                has_valid_format = True
                break

        if not has_valid_format:
            # 如果報告沒有預期的格式，嘗試直接解析文本
            lines = full_pattern_report.strip().split('\n')
            if lines:
                condensed_report = "K線型態摘要:\n"
                for line in lines[:10]:  # 只取前10行
                    if line.strip():
                        condensed_report += f"• {line.strip()}\n"

                # 簡單檢測報告中的關鍵詞來填充摘要
                for i, period_key in enumerate(period_keys):
                    for j, type_key in enumerate(type_keys):
                        for line in lines:
                            if pattern_types[j] in line:
                                pattern = line.strip()
                                report_summary[period_key][type_key].append(pattern)

                return condensed_report, report_summary

        # 正常解析格式化報告
        for i, period in enumerate(periods):
            period_key = period_keys[i]
            period_section = ""

            # 尋找該時間週期的部分
            start_idx = full_pattern_report.find(f"{period}分析:")
            if start_idx == -1:
                continue

            end_idx = -1
            for next_period in periods[i+1:]:
                end_idx = full_pattern_report.find(f"{next_period}分析:", start_idx)
                if end_idx != -1:
                    break

            if end_idx == -1:
                period_content = full_pattern_report[start_idx:]
            else:
                period_content = full_pattern_report[start_idx:end_idx]

            # 解析該時間週期下的型態
            period_lines = []
            for j, pattern_type in enumerate(pattern_types):
                type_key = type_keys[j]
                type_start = period_content.find(f"{pattern_type}型態:")
                if type_start == -1:
                    continue

                type_end = -1
                for next_type in pattern_types[j+1:]:
                    type_end = period_content.find(f"{next_type}型態:", type_start)
                    if type_end != -1:
                        break

                if type_end == -1:
                    type_content = period_content[type_start:]
                else:
                    type_content = period_content[type_start:type_end]

                # 提取型態列表
                pattern_list_start = type_content.find(":")
                if pattern_list_start != -1:
                    pattern_list = type_content[pattern_list_start+1:].strip()
                    patterns = [p.strip() for p in pattern_list.split(',') if p.strip() and p.strip() != '無']

                    # 添加到摘要數據
                    report_summary[period_key][type_key].extend(patterns)

                    # 只有當有型態時才添加到報告中
                    if patterns:
                        pattern_line = f"• {pattern_type}: {', '.join(patterns)}"
                        period_lines.append(pattern_line)

            # 只有當該時間週期有型態時才添加到最終報告
            if period_lines:
                period_section = f"{period}分析:\n" + "\n".join(period_lines)
                result_parts.append(period_section)

        # 組合最終報告
        if result_parts:
            condensed_report = "\n\n".join(result_parts)
        else:
            condensed_report = "未檢測到明顯K線型態"

        return condensed_report, report_summary

    except Exception as e:
        # 如果解析過程中出現任何錯誤，返回一個簡單的錯誤信息
        error_msg = f"無法解析K線報告: {str(e)}"
        logger.error(error_msg)
        return "無法生成摘要：解析過程出錯", report_summary

# ==============================================================================
# 3. 新增：智能交易建議引擎
# ==============================================================================
# ==============================================================================
# 升級：智能建議引擎 V2 (替換舊版本)
# 功能：結合綜合評分與K線結構化報告，生成更深入的建議
# ==============================================================================
import math
import logging

def generate_recommendation(combined_score, report_summary=None):
    """
    根據綜合評分和詳細的K線型態報告生成投資建議、原因、信心度及型態詳情。
    """
    # 🔍 調試信息：確認函數被調用且參數正確
    #print(f"🔍 [DEBUG] generate_recommendation 被調用:")
    print(f"    combined_score: {combined_score} (type: {type(combined_score)})")
    print(f"    report_summary: {report_summary is not None}")

    # 確保 combined_score 是數字類型
    try:
        combined_score = float(combined_score)
    except (ValueError, TypeError):
        print(f"⚠️  [WARNING] combined_score 無法轉換為數字，使用預設值 50")
        combined_score = 50.0

    # --- 第1步：根據綜合評分設定基本建議 ---
    recommendation = {
        "action": "中立觀察",
        "reason": f"綜合評分中性 ({combined_score:.2f})。",
        "pattern_details": []
    }

    if combined_score >= 70:
        recommendation["action"] = "買入"
        recommendation["reason"] = f"綜合評分強勁 ({combined_score:.2f})。"
    elif combined_score >= 60:
        recommendation["action"] = "買入觀望"
        recommendation["reason"] = f"綜合評分偏強 ({combined_score:.2f})。"
    elif combined_score <= 30:
        recommendation["action"] = "賣出"
        recommendation["reason"] = f"綜合評分疲弱 ({combined_score:.2f})。"
    elif combined_score <= 40:
        recommendation["action"] = "賣出觀望"
        recommendation["reason"] = f"綜合評分偏弱 ({combined_score:.2f})。"

    # --- 第2步：根據K線型態微調建議 ---
    pattern_boost = 0  # K線型態對信心度的加成

    if report_summary:
        try:
            # 計算加權後的K線型態數量
            bullish_count = len(report_summary.get('daily', {}).get('bullish', [])) \
                          + len(report_summary.get('weekly', {}).get('bullish', [])) * 2 \
                          + len(report_summary.get('monthly', {}).get('monthly', [])) * 3
            bearish_count = len(report_summary.get('daily', {}).get('bearish', [])) \
                          + len(report_summary.get('weekly', {}).get('bearish', [])) * 2 \
                          + len(report_summary.get('monthly', {}).get('bearish', [])) * 3

            #print(f"🔍 [DEBUG] K線型態統計: 看漲={bullish_count}, 看跌={bearish_count}")

            # 檢查是否有強烈的K線信號
            if bullish_count >= 3 and bullish_count > bearish_count * 1.5:
                if combined_score >= 50:
                    recommendation["action"] = "強烈買入"
                    recommendation["reason"] += " 多個看漲K線型態提供強力支撐。"
                    pattern_boost = 20  # 強烈信號加成
                else:
                    recommendation["action"] = "買入觀望"
                    recommendation["reason"] += " K線型態看漲，但需注意技術指標較弱。"
                    pattern_boost = 10
            elif bearish_count >= 3 and bearish_count > bullish_count * 1.5:
                if combined_score <= 50:
                    recommendation["action"] = "強烈賣出"
                    recommendation["reason"] += " 多個看跌K線型態構成強力壓力。"
                    pattern_boost = 20  # 強烈信號加成
                else:
                    recommendation["action"] = "賣出觀望"
                    recommendation["reason"] += " K線型態看跌，但需注意技術指標較強。"
                    pattern_boost = 10
            elif bullish_count > bearish_count:
                recommendation["reason"] += " K線型態整體偏向看漲。"
                pattern_boost = 5
            elif bearish_count > bullish_count:
                recommendation["reason"] += " K線型態整體偏向看跌。"
                pattern_boost = 5

            # 整理K線型態詳情
            pattern_details = []
            for period in ['daily', 'weekly', 'monthly']:
                period_data = report_summary.get(period, {})
                period_patterns = []
                if period_data.get('bullish'):
                    period_patterns.append(f"看漲: {', '.join(period_data['bullish'])}")
                if period_data.get('bearish'):
                    period_patterns.append(f"看跌: {', '.join(period_data['bearish'])}")

                if period_patterns:
                    period_name = {'daily': '日線', 'weekly': '週線', 'monthly': '月線'}[period]
                    pattern_details.append(f"{period_name}: {'; '.join(period_patterns)}")

            if pattern_details:
                recommendation["pattern_details"] = pattern_details

        except Exception as e:
            logger.error(f"處理K線報告摘要時出錯: {str(e)}")

    # --- 第3步：計算信心度（修正版） ---
    # 使用線性函數而非平方函數，讓信心度更合理
    distance_from_center = abs(combined_score - 50)

    # 基礎信心度：距離中心越遠，信心度越高
    base_confidence = min(distance_from_center * 2, 100)  # 每偏離1分增加2%信心度，最高100%

    # 加上K線型態加成
    total_confidence = min(base_confidence + pattern_boost, 100)

    recommendation['confidence'] = int(total_confidence)

    # 🔍 調試信息：確認信心度計算
    #print(f"🔍 [DEBUG] 信心度計算:")
    #print(f"    distance_from_center: {distance_from_center}")
    #print(f"    base_confidence: {base_confidence}")
    #print(f"    pattern_boost: {pattern_boost}")
    #print(f"    total_confidence: {total_confidence}")
    #print(f"🔍 [DEBUG] 最終建議: {recommendation}")
    #print("-" * 50)

    return recommendation


def get_pattern_explanation(pattern_name):
    """
    獲取K線型態的解釋和交易建議

    參數:
    pattern_name - K線型態名稱

    返回:
    dict - 包含解釋和建議的字典
    """
    pattern_dict = {
        "看漲錘子線": {
            "explanation": "在下跌趨勢中出現，具有較長的下影線，表示買方力量正在增強。",
            "suggestion": "可能是底部反轉信號，考慮做多，止損設在錘子線的最低點下方。"
        },
        "看跌錘子線": {
            "explanation": "在上漲趨勢中出現，具有較長的下影線，表示賣方力量正在增強。",
            "suggestion": "可能是頂部反轉信號，考慮做空或減持，止損設在錘子線的最高點上方。"
        },
        "看跌流星線": {
            "explanation": "在上漲趨勢中出現，具有較長的上影線，表示賣方力量正在增強。",
            "suggestion": "可能是頂部反轉信號，考慮做空或減持，止損設在流星線的最高點上方。"
        },
        "中性十字星": {
            "explanation": "開盤價和收盤價幾乎相同，表示市場猶豫不決，多空力量均衡。",
            "suggestion": "通常表示趨勢可能即將反轉，建議等待確認信號。"
        },
        "看漲吞噬": {
            "explanation": "小陰線後出現大陽線，且陽線完全「吞噬」了前一天的陰線實體，表示買方力量強勁。",
            "suggestion": "強烈的看漲信號，特別是在下跌趨勢末期，可考慮做多，止損設在吞噬K線的最低點下方。"
        },
        "看跌吞噬": {
            "explanation": "小陽線後出現大陰線，且陰線完全「吞噬」了前一天的陽線實體，表示賣方力量強勁。",
            "suggestion": "強烈的看跌信號，特別是在上漲趨勢末期，可考慮做空或減持，止損設在吞噬K線的最高點上方。"
        },
        "看漲母子線": {
            "explanation": "大陰線後出現小陽線，且小陽線完全在前一天大陰線實體範圍內，表示下跌動能減弱。",
            "suggestion": "可能的底部反轉信號，建議等待進一步確認後再做多，止損設在母子線組合的最低點下方。"
        },
        "看跌母子線": {
            "explanation": "大陽線後出現小陰線，且小陰線完全在前一天大陽線實體範圍內，表示上漲動能減弱。",
            "suggestion": "可能的頂部反轉信號，建議等待進一步確認後再做空或減持，止損設在母子線組合的最高點上方。"
        },
        "看漲晨星": {
            "explanation": "由三根K線組成：大陰線、小實體線（通常是十字星）和大陽線，表示市場由空頭轉為多頭。",
            "suggestion": "強烈的底部反轉信號，可考慮做多，止損設在晨星組合的最低點下方。"
        },
        "看跌暮星": {
            "explanation": "由三根K線組成：大陽線、小實體線（通常是十字星）和大陰線，表示市場由多頭轉為空頭。",
            "suggestion": "強烈的頂部反轉信號，可考慮做空或減持，止損設在暮星組合的最高點上方。"
        },
        "看漲三白兵": {
            "explanation": "連續三根上漲的陽線，每根的收盤價都高於前一根，表示買方持續控制市場。",
            "suggestion": "強烈的上漲趨勢確認信號，可考慮做多，止損設在第一根K線的最低點下方。"
        },
        "看跌三黑鴉": {
            "explanation": "連續三根下跌的陰線，每根的收盤價都低於前一根，表示賣方持續控制市場。",
            "suggestion": "強烈的下跌趨勢確認信號，可考慮做空或減持，止損設在第一根K線的最高點上方。"
        },
        "看漲穿刺線": {
            "explanation": "大陰線後出現大陽線，且陽線開盤價低於前一天最低價，收盤價高於前一天實體中點，表示買方力量強勁。",
            "suggestion": "看漲反轉信號，可考慮做多，止損設在穿刺線的最低點下方。"
        },
        "看跌烏雲蓋頂": {
            "explanation": "大陽線後出現大陰線，且陰線開盤價高於前一天最高價，收盤價低於前一天實體中點，表示賣方力量強勁。",
            "suggestion": "看跌反轉信號，可考慮做空或減持，止損設在烏雲蓋頂的最高點上方。"
        },
        "中性長腳十字線": {
            "explanation": "開盤價和收盤價幾乎相同，但有很長的上下影線，表示市場波動劇烈但未形成明確方向。",
            "suggestion": "市場猶豫不決的信號，可能預示著劇烈波動，建議等待更明確的方向信號。"
        },
        "看跌上吊線": {
            "explanation": "在上漲趨勢中出現，形狀類似錘子線但出現在上漲趨勢頂部，具有較長的下影線，表示賣方力量正在增強。",
            "suggestion": "頂部反轉信號，可考慮做空或減持，止損設在上吊線的最高點上方。"
        },
        "中性陀螺": {
            "explanation": "小實體K線，上下影線長度相近，表示市場猶豫不決，多空力量相當。",
            "suggestion": "通常表示市場不確定性增加，建議等待更明確的方向信號。"
        }
    }

    # 從完整名稱中提取基本型態名稱
    for key in pattern_dict.keys():
        if key in pattern_name:
            return pattern_dict[key]

    return {
        "explanation": "這是一種技術形態，可能表示市場趨勢的變化。",
        "suggestion": "建議結合其他技術指標進行確認。"
    }



def generate_pattern_report(df, periods=['daily']):
    """
    生成K線型態分析報告（不依賴talib）

    Args:
        df (DataFrame): 包含OHLC數據的DataFrame
        periods (list): 要分析的時間週期列表，可以是 'daily', 'weekly', 'monthly'

    Returns:
        str: 格式化的K線型態報告
    """
    if df is None or df.empty:
        return "無法生成報告：數據為空"

    # 確保必要的列存在
    required_cols = ['open', 'high', 'low', 'close']
    if not all(col in df.columns for col in required_cols):
        missing = [col for col in required_cols if col not in df.columns]
        return f"無法生成報告：缺少必要列 {', '.join(missing)}"

    # 定義簡單的K線型態檢測函數
    def detect_patterns(ohlc_data):
        """使用簡單規則檢測常見K線型態"""
        patterns = {
            "看漲": [],
            "看跌": [],
            "中性": []
        }

        # 確保有足夠的數據
        if len(ohlc_data) < 5:
            return patterns

        # 取最近的K線數據進行分析
        recent = ohlc_data.tail(10).copy()

        # 計算每根K線的實體大小和影線長度
        recent['body_size'] = abs(recent['close'] - recent['open'])
        recent['upper_shadow'] = recent['high'] - recent[['open', 'close']].max(axis=1)
        recent['lower_shadow'] = recent[['open', 'close']].min(axis=1) - recent['low']
        recent['is_bullish'] = recent['close'] > recent['open']
        recent['is_bearish'] = recent['close'] < recent['open']

        # 計算移動平均線
        recent['ma5'] = recent['close'].rolling(window=5).mean()
        recent['ma10'] = recent['close'].rolling(window=10).mean()

        # 最近3根K線
        last3 = recent.tail(3)

        # 檢測看漲型態

        # 三白兵: 連續三根上漲K線，每根收盤價都高於前一根
        if len(last3) == 3 and all(last3['is_bullish']) and \
           last3['close'].iloc[0] < last3['close'].iloc[1] < last3['close'].iloc[2]:
            patterns["看漲"].append("三白兵")

        # 多頭吞噬: 一根看跌K線後跟一根較大的看漲K線，完全吞噬前一根
        if len(last3) >= 2 and last3['is_bearish'].iloc[-2] and last3['is_bullish'].iloc[-1] and \
           last3['open'].iloc[-1] <= last3['close'].iloc[-2] and \
           last3['close'].iloc[-1] >= last3['open'].iloc[-2]:
            patterns["看漲"].append("多頭吞噬")

        # 看漲錘頭: 下影線長，幾乎沒有上影線，小實體
        if any(last3['lower_shadow'] > 2 * last3['body_size']) and \
           any(last3['upper_shadow'] < 0.2 * last3['body_size']) and \
           any(last3['is_bullish']):
            patterns["看漲"].append("看漲錘頭")

        # 檢測看跌型態

        # 三黑鴉: 連續三根下跌K線，每根收盤價都低於前一根
        if len(last3) == 3 and all(last3['is_bearish']) and \
           last3['close'].iloc[0] > last3['close'].iloc[1] > last3['close'].iloc[2]:
            patterns["看跌"].append("三黑鴉")

        # 空頭吞噬: 一根看漲K線後跟一根較大的看跌K線，完全吞噬前一根
        if len(last3) >= 2 and last3['is_bullish'].iloc[-2] and last3['is_bearish'].iloc[-1] and \
           last3['open'].iloc[-1] >= last3['close'].iloc[-2] and \
           last3['close'].iloc[-1] <= last3['open'].iloc[-2]:
            patterns["看跌"].append("空頭吞噬")

        # 看跌吊錘: 上影線長，幾乎沒有下影線，小實體
        if any(last3['upper_shadow'] > 2 * last3['body_size']) and \
           any(last3['lower_shadow'] < 0.2 * last3['body_size']) and \
           any(last3['is_bearish']):
            patterns["看跌"].append("看跌吊錘")

        # 檢測中性型態

        # 十字星: 開盤價和收盤價幾乎相同
        if any(last3['body_size'] < 0.1 * (last3['high'] - last3['low'])):
            patterns["中性"].append("十字星")

        # 長腿十字: 開盤價和收盤價幾乎相同，上下影線都很長
        if any((last3['body_size'] < 0.1 * (last3['high'] - last3['low'])) &
               (last3['upper_shadow'] > last3['body_size']) &
               (last3['lower_shadow'] > last3['body_size'])):
            patterns["中性"].append("長腿十字")

        # 紡錘頂: 小實體，上下影線差不多長
        if any((last3['body_size'] < 0.3 * (last3['high'] - last3['low'])) &
               (abs(last3['upper_shadow'] - last3['lower_shadow']) < 0.2 * (last3['high'] - last3['low']))):
            patterns["中性"].append("紡錘")

        return patterns

    report_parts = []

    # 為每個時間週期生成報告
    for period in periods:
        period_df = df.copy()

        # 確保日期列是日期類型
        date_col = None
        if 'date' in period_df.columns:
            date_col = 'date'
        elif 'datetime' in period_df.columns:
            date_col = 'datetime'
        elif 'time' in period_df.columns:
            date_col = 'time'

        # 如果找到日期列，設置為索引
        if date_col:
            try:
                # 嘗試將日期列轉換為datetime類型
                period_df[date_col] = pd.to_datetime(period_df[date_col])
                period_df.set_index(date_col, inplace=True)
            except Exception as e:
                logger.warning(f"無法將 {date_col} 設置為索引: {str(e)}")
                # 如果無法設置日期索引，就使用原始數據

        # 根據時間週期重新取樣數據 (只有當索引是DatetimeIndex時才執行)
        try:
            if period != 'daily' and isinstance(period_df.index, pd.DatetimeIndex):
                if period == 'weekly':
                    period_df = period_df.resample('W').agg({
                        'open': 'first',
                        'high': 'max',
                        'low': 'min',
                        'close': 'last'
                    })
                elif period == 'monthly':
                    period_df = period_df.resample('M').agg({
                        'open': 'first',
                        'high': 'max',
                        'low': 'min',
                        'close': 'last'
                    })
        except Exception as e:
            logger.warning(f"重採樣到 {period} 時出錯: {str(e)}")
            # 如果重採樣失敗，繼續使用原始數據

        # 移除NaN值
        period_df = period_df.dropna()

        if period_df.empty:
            continue

        # 中文時間週期名稱
        period_name = {
            'daily': '日線',
            'weekly': '週線',
            'monthly': '月線'
        }.get(period, period)

        # 檢測K線型態
        try:
            patterns = detect_patterns(period_df)

            period_report = [f"{period_name}分析:"]

            # 添加檢測到的型態到報告中
            for pattern_type, detected_patterns in patterns.items():
                if detected_patterns:
                    pattern_str = ", ".join(detected_patterns)
                    period_report.append(f"{pattern_type}型態: {pattern_str}")
                else:
                    period_report.append(f"{pattern_type}型態: 無")

            # 添加該時間週期的報告
            report_parts.append("\n".join(period_report))

        except Exception as e:
            logger.error(f"分析 {period_name} 型態時出錯: {str(e)}")
            report_parts.append(f"{period_name}分析:\n分析時發生錯誤: {str(e)}")

    # 如果沒有生成任何報告部分，返回一個默認消息
    if not report_parts:
        return "未檢測到明顯K線型態"

    # 組合最終報告
    return "\n\n".join(report_parts)


def analyze_multi_timeframe(df):
    """
    多時間框架分析。
    """
    if df.empty:
        return {'error': '無有效數據'}

    try:
        result = {
            'D': {'quadrant': 'Q1', 'description': '看漲' if 'MA20' in df.columns and df['Close'].iloc[-1] > df['MA20'].iloc[-1] else '中性'},
            'W': {'quadrant': 'Q2', 'description': '中性'},
            'M': {'quadrant': 'Q1', 'description': '看漲' if 'MA60' in df.columns and df['Close'].iloc[-1] > df['MA60'].iloc[-1] else '中性'},
            'combined': {
                'trend_consistency': '看漲' if 'MA20' in df.columns and df['Close'].iloc[-1] > df['MA20'].iloc[-1] else '中性',
                'strength': '強勢' if 'RSI' in df.columns and df['RSI'].iloc[-1] > 60 else '一般',
                'suggestion': '買入' if 'MA20' in df.columns and df['Close'].iloc[-1] > df['MA20'].iloc[-1] else '持有'
            }
        }
        return result
    except Exception as e:
        logger.error(f"多時間框架分析時發生錯誤: {e}")
        return {'error': f'分析失敗: {str(e)}'}

def analyze_market_structure(df):
    """
    市場結構分析。
    """
    if df.empty:
        return {'error': '無有效數據'}

    try:
        last_price = df['Close'].iloc[-1]
        support_levels = df['Low'].rolling(window=20).min().tail(5).tolist()
        resistance_levels = df['High'].rolling(window=20).max().tail(5).tolist()
        recent_volatility = df['Close'].pct_change().tail(20).std() * 100
        long_term_volatility = df['Close'].pct_change().std() * 100

        return {
            'price_levels': {
                'position': '高於支撐位' if last_price > max(support_levels) else '低於阻力位',
                'resistance_levels': resistance_levels,
                'support_levels': support_levels
            },
            'trend': '看漲' if 'MA20' in df.columns and df['Close'].iloc[-1] > df['MA20'].iloc[-1] else '看跌',
            'volatility': {
                'status': '高' if recent_volatility > long_term_volatility else '低',
                'recent': round(recent_volatility, 2),
                'long_term': round(long_term_volatility, 2)
            },
            'volume': {
                'status': '增加' if 'Volume' in df.columns and df['Volume'].iloc[-1] > df['Volume'].rolling(window=20).mean().iloc[-1] else '減少',
                'recent_avg': int(df['Volume'].tail(20).mean()) if 'Volume' in df.columns else 0,
                'long_term_avg': int(df['Volume'].mean()) if 'Volume' in df.columns else 0
            },
            'sentiment': {
                'status': '超買' if 'RSI' in df.columns and df['RSI'].iloc[-1] > 70 else '超賣' if 'RSI' in df.columns and df['RSI'].iloc[-1] < 30 else '中性',
                'rsi': round(df['RSI'].iloc[-1], 2) if 'RSI' in df.columns else 50
            }
        }
    except Exception as e:
        logger.error(f"市場結構分析時發生錯誤: {e}")
        return {'error': f'分析失敗: {str(e)}'}

def generate_indicator_summary(status):
    """
    生成技術指標摘要。
    """
    try:
        if isinstance(status, dict) and 'error' in status:
            return "無有效技術指標"

        summary = []
        df = status.get('data_df')
        if df is None or df.empty:
            return "無有效技術指標數據"

        if 'MA5' in df.columns and 'MA20' in df.columns:
            summary.append("移動平均線: 看漲" if df['MA5'].iloc[-1] > df['MA20'].iloc[-1] else "移動平均線: 中性")

        if 'RSI' in df.columns:
            rsi = df['RSI'].iloc[-1]
            summary.append(f"RSI: {'超買' if rsi > 70 else '超賣' if rsi < 30 else '中性'} ({round(rsi, 2)})")

        if 'MACD' in df.columns and 'Signal' in df.columns:
            summary.append("MACD: 正向" if df['MACD'].iloc[-1] > df['Signal'].iloc[-1] else "MACD: 負向")

        if 'K' in df.columns and 'D' in df.columns:
            summary.append("KD: 看漲" if df['K'].iloc[-1] > df['D'].iloc[-1] else "KD: 看跌")

        return ", ".join(summary) if summary else "無可用技術指標"
    except Exception as e:
        logger.error(f"生成技術指標摘要時發生錯誤: {e}")
        return "技術指標摘要生成失敗"


def get_summary_report_html(data: dict):
    """
    生成 HTML 格式的統計報告。
    """
    rows = []
    for stock_id, stock_data in data.items():
        try:
            # 檢查歷史資料欄位
            hist_data = stock_data.get("歷史資料")
            if not hist_data:
                hist_data = stock_data.get("data", [])

            if not hist_data:
                raise ValueError("無歷史資料")

            df = pd.DataFrame(hist_data)
            if df.empty:
                raise ValueError("歷史資料為空")

            # 檢查欄位名稱（中文或英文）
            close_col = None
            volume_col = None

            for col in ['收盤', 'Close', 'close']:
                if col in df.columns:
                    close_col = col
                    break

            for col in ['成交量', 'Volume', 'volume']:
                if col in df.columns:
                    volume_col = col
                    break

            if not close_col or not volume_col:
                raise ValueError("缺少必要欄位")

            summary = df[[close_col, volume_col]].describe().T

            row = {
                "股票代碼": stock_id,
                "收盤平均": round(summary.loc[close_col, "mean"], 2),
                "收盤最低": round(summary.loc[close_col, "min"], 2),
                "收盤最高": round(summary.loc[close_col, "max"], 2),
                "成交量平均": round(summary.loc[volume_col, "mean"], 0),
                "成交量最低": round(summary.loc[volume_col, "min"], 0),
                "成交量最高": round(summary.loc[volume_col, "max"], 0),
            }
            rows.append(row)
        except Exception as e:
            rows.append({
                "股票代碼": stock_id,
                "收盤平均": "錯誤",
                "收盤最低": "",
                "收盤最高": "",
                "成交量平均": str(e),
                "成交量最低": "",
                "成交量最高": ""
            })

    result_df = pd.DataFrame(rows)
    return result_df.to_html(index=False, border=1, justify="center", escape=False)


# ==============================================================================
# 通知函數 (***重大修正區***)
# ==============================================================================
def send_notifications(message: str, files: List[str] = None):
    """
    簡化版通知函數 - 僅輸出到控制台，不需要外部配置
    """
    print("\n=== 通知訊息 ===")
    print(message)
    if files:
        print(f"附加檔案: {', '.join(files)}")
    print("===============\n")

    # 如果有日誌記錄器，也記錄一下
    try:
        logger.info(f"通知訊息已輸出到控制台 (含 {len(files) if files else 0} 個檔案)")
    except:
        pass  # 如果沒有 logger 或出錯，就忽略

    """
    發送通知到 Telegram 和 Discord (同步版本)
    """
    # --- Telegram 發送 ---
    for chat_id in TELEGRAM_CHAT_ID:
        try:
            # 發送文字訊息
            url_msg = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage"
            payload = {"chat_id": chat_id, "text": message, "parse_mode": "Markdown"}
            requests.post(url_msg, json=payload, timeout=20)
            logger.info(f"Telegram 摘要成功發送至 chat_id: {chat_id}")

            # 發送圖片檔案
            if files:
                url_photo = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendPhoto"
                for file_path in files:
                    if os.path.exists(file_path):
                        with open(file_path, 'rb') as f:
                            photo_files = {'photo': (os.path.basename(file_path), f)}
                            data = {'chat_id': chat_id}
                            requests.post(url_photo, data=data, files=photo_files, timeout=60)
                logger.info(f"Telegram 圖檔成功發送至 chat_id: {chat_id} ({len(files)}個)")
        except Exception as e:
            logger.error(f"發送 Telegram 通知至 {chat_id} 時異常: {e}")

    # --- Discord 發送 ---
    try:
        # 清理訊息中的 Markdown 符號，使其在 Discord 中正常顯示
        discord_message = message.replace('*', '').replace('_', '')
        webhook = DiscordWebhook(url=DISCORD_WEBHOOK_URL, content=f"```\n{discord_message}\n```")
        if files:
            for file_path in files:
                if os.path.exists(file_path):
                    with open(file_path, 'rb') as f:
                        webhook.add_file(file=f.read(), filename=os.path.basename(file_path))
        response = webhook.execute()
        if response.ok:
            logger.info("Discord 通知發送成功。")
        else:
            logger.error(f"Discord 通知失敗: {response.status_code} {response.content}")
    except Exception as e:
        logger.error(f"發送 Discord 通知時異常: {e}")

def format_notification_message(
    results_to_show: Dict,
    successful_analysis: int,
    failed_analysis: int,
    total_after_volume_filter: int,
    top_n=5
) -> str:
    """
    格式化通知訊息 (升級版)，包含分析統計數據。
    """
    try:
        current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        if not results_to_show:
            return f"""
🤖 *台股技術分析報告*
📅 分析時間: {current_time}

❌ 本次分析未找到符合顯示條件的股票。
💡 建議調整篩選條件或關注市場變化。
"""

        # 按分數排序，確保顯示的是最高分的股票
        top_stocks = sorted(results_to_show.values(), key=lambda x: x.get('combined_score', 0), reverse=True)

        message = f"""
🤖 *台股技術分析報告*
📅 分析時間: {current_time}

*📈 分析統計:*
> • 符合成交量標準: {total_after_volume_filter} 支
> • 成功分析: {successful_analysis} 支
> • 分析失敗: {failed_analysis} 支

🏆 *TOP {min(top_n, len(top_stocks))} 推薦股票:*
"""
        for rank, result in enumerate(top_stocks[:top_n], 1):
            recommendation = result.get('recommendation', {})
            message += f"""
*{rank}. {result.get('stock_name', '')} ({result.get('stock_id', '')})*
   💰 最新價格: {result.get('last_price', 0):.2f} TWD
   📈 綜合評分: *{result.get('combined_score', 0):.1f} / 100*
   🎯 投資建議: *{recommendation.get('action', '觀望')}*
"""

        message += """
⚠️ *投資提醒:*
• 本分析僅為技術指標參考，不構成任何投資建議。
• 投資有風險，請結合基本面分析並做好風險控管。
"""
        return message.strip()
    except Exception as e:
        logger.error(f"格式化通知訊息時發生錯誤: {e}")
        return f"台股分析完成，但訊息格式化失敗: {str(e)}"


# 缺少的函數定義
def filter_by_volume(stocks_data, min_volume_shares=1000000):
    """
    根據成交量過濾股票
    """
    filtered_data = {}

    for stock_id, stock_data in stocks_data.items():
        try:
            if 'data' not in stock_data or not stock_data['data']:
                continue

            df = pd.DataFrame(stock_data['data'])
            if 'Volume' not in df.columns:
                continue

            # 計算近30天平均成交量
            recent_volume = df['Volume'].tail(30).mean()

            # 轉換為張數（1張 = 1000股）
            avg_volume_lots = recent_volume / 1000

            if avg_volume_lots >= (min_volume_shares / 1000):  # min_volume_shares已經是股數
                filtered_data[stock_id] = stock_data

        except Exception as e:
            logger.warning(f"過濾股票 {stock_id} 時發生錯誤: {str(e)}")
            continue

    logger.info(f"成交量過濾完成: {len(filtered_data)}/{len(stocks_data)} 支股票符合標準")
    return filtered_data

def top_percentile_filter(analysis_results, percentile=98):
    """
    篩選前百分位的股票
    """
    if not analysis_results:
        return {}

    scores = [result.get('combined_score', 0) for result in analysis_results.values()]
    threshold = np.percentile(scores, percentile)

    elite_results = {
        stock_id: result for stock_id, result in analysis_results.items()
        if result.get('combined_score', 0) >= threshold
    }

    logger.info(f"前{100-percentile}%篩選完成: {len(elite_results)}/{len(analysis_results)} 支股票")
    return elite_results

def print_top_stocks(results, top_n=10):
    """
    打印最佳股票清單
    """
    sorted_results = sorted(results.values(), key=lambda x: x.get('combined_score', 0), reverse=True)

    print(f"\n🏆 前 {min(top_n, len(sorted_results))} 名推薦股票:")
    print("=" * 80)

    for i, result in enumerate(sorted_results[:top_n], 1):
        stock_id = result.get('stock_id', 'N/A')
        stock_name = result.get('stock_name', 'N/A')
        score = result.get('combined_score', 0)
        recommendation = result.get('recommendation', {})

        print(f"{i:2d}. {stock_id} {stock_name}")
        print(f"    綜合評分: {score}/100")
        print(f"    投資建議: {recommendation.get('action', 'N/A')}")
        print(f"    信心度: {recommendation.get('confidence', 0)}%")
        print("-" * 80)

def export_html_report(analysis_results):
    """
    匯出HTML報告
    """
    try:
        html_content = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <meta charset="UTF-8">
            <title>台股技術分析報告</title>
            <style>
                body {{ font-family: Arial, sans-serif; margin: 20px; }}
                table {{ border-collapse: collapse; width: 100%; }}
                th, td {{ border: 1px solid #ddd; padding: 8px; text-align: left; }}
                th {{ background-color: #f2f2f2; }}
                .high-score {{ background-color: #d4edda; }}
                .medium-score {{ background-color: #fff3cd; }}
                .low-score {{ background-color: #f8d7da; }}
            </style>
        </head>
        <body>
            <h1>台股技術分析報告</h1>
            <p>生成時間: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
            <table>
                <tr>
                    <th>排名</th>
                    <th>股票代號</th>
                    <th>股票名稱</th>
                    <th>綜合評分</th>
                    <th>投資建議</th>
                    <th>信心度</th>
                </tr>
        """

        sorted_results = sorted(analysis_results.values(), key=lambda x: x.get('combined_score', 0), reverse=True)

        for i, result in enumerate(sorted_results[:50], 1):  # 只顯示前50名
            score = result.get('combined_score', 0)
            score_class = 'high-score' if score >= 70 else 'medium-score' if score >= 50 else 'low-score'

            html_content += f"""
                <tr class="{score_class}">
                    <td>{i}</td>
                    <td>{result.get('stock_id', 'N/A')}</td>
                    <td>{result.get('stock_name', 'N/A')}</td>
                    <td>{score}</td>
                    <td>{result.get('recommendation', {}).get('action', 'N/A')}</td>
                    <td>{result.get('recommendation', {}).get('confidence', 0)}%</td>
                </tr>
            """

        html_content += """
            </table>
        </body>
        </html>
        """

        html_path = os.path.join(RESULTS_DIR, f"stock_analysis_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.html")
        with open(html_path, 'w', encoding='utf-8') as f:
            f.write(html_content)

        logger.info(f"✅ HTML報告已匯出: {html_path}")
        return html_path

    except Exception as e:
        logger.error(f"匯出HTML報告時發生錯誤: {str(e)}")
        return None

# 修正的圖表生成函數
def generate_detailed_analysis_report(stock_id, analysis_result, output_dir='reports'):
    """
    為單一股票生成詳細的技術分析報告，包含圖表和關鍵指標。

    修正了 addplot 參數的處理邏輯，避免傳遞 None 值。

    參數:
    stock_id (str): 股票代號
    analysis_result (dict): 包含分析結果的字典
    output_dir (str): 輸出目錄路徑

    返回:
    str: 報告文件的路徑，如果生成失敗則返回 None
    """
    try:
        if not analysis_result or not analysis_result.get('success'):
            logger.warning(f"無法為股票 {stock_id} 生成報告：分析結果無效")
            return None

        # 確保輸出目錄存在
        os.makedirs(output_dir, exist_ok=True)

        # 獲取分析數據
        df = analysis_result['data_df'].copy()

        # 確保索引是日期時間類型
        if not isinstance(df.index, pd.DatetimeIndex):
            logger.warning(f"股票 {stock_id} 的數據索引不是日期類型，嘗試轉換")
            df.index = pd.to_datetime(df.index)

        # 確保數據按日期排序
        df.sort_index(inplace=True)

        # 將 DataFrame 轉換為 mplfinance 可用的格式
        df_mpf = df[['Open', 'High', 'Low', 'Close', 'Volume']].copy()

        # 創建附加圖表
        ap = []  # 初始化為空列表

        # 添加移動平均線
        if 'ma20' in df.columns and 'ma60' in df.columns and 'ma120' in df.columns:
            ap.append(mpf.make_addplot(df['ma20'], color='blue', width=0.7))
            ap.append(mpf.make_addplot(df['ma60'], color='red', width=0.7))
            ap.append(mpf.make_addplot(df['ma120'], color='green', width=0.7))

        # 添加 RSI 指標（如果存在）
        if 'rsi' in df.columns:
            # 創建一個新的子圖用於 RSI
            ap.append(mpf.make_addplot(df['rsi'], panel=1, color='purple',
                                       ylabel='RSI'))

        # 添加 MACD 指標（如果存在）
        if all(col in df.columns for col in ['macd', 'macdsignal']):
            # 創建一個新的子圖用於 MACD
            ap.append(mpf.make_addplot(df['macd'], panel=2, color='blue',
                                       ylabel='MACD'))
            ap.append(mpf.make_addplot(df['macdsignal'], panel=2, color='red'))
            ap.append(mpf.make_addplot(df['histogram'], panel=2, type='bar', color='dimgray'))

        # 設置圖表樣式和佈局
        style = mpf.make_mpf_style(base_mpf_style='charles', rc={'figure.figsize': (12, 10)})

        # 設置子圖比例
        fig_kwargs = {
            'figratio': (12, 8),
            'panel_ratios': (6, 2, 2) if 'macd' in df.columns else (6, 2)
        }

        # 設置圖表標題
        title = f"{stock_id} {analysis_result.get('stock_name', '')} 技術分析 ({df.index[0].strftime('%Y-%m-%d')} 至 {df.index[-1].strftime('%Y-%m-%d')})"

        # 生成圖表
        plot_kwargs = {
            'type': 'candle',
            'style': style,
            'volume': True,
            'title': title,
            'figscale': 1.5,
            'tight_layout': True,
            'savefig': f"{output_dir}/{stock_id}_technical_analysis.png",
            'returnfig': True,
            **fig_kwargs
        }

        # 只有在 ap 列表非空時才添加 addplot 參數
        if ap:
            plot_kwargs['addplot'] = ap

        fig, axes = mpf.plot(df_mpf, **plot_kwargs)

        # 關閉圖表以釋放記憶體
        plt.close(fig)

        # 返回報告文件路徑
        return f"{output_dir}/{stock_id}_technical_analysis.png"

    except Exception as e:
        logger.error(f"生成圖表時發生錯誤 ({stock_id}): {str(e)}")
        logger.debug(traceback.format_exc())
        return None


# 您原本的其他函數（set_chinese_font, get_taiwan_stocks, etc.）保持不變
def set_chinese_font():
    """
    自動設定適合當前作業系統的中文字體，用於 Matplotlib 顯示。
    """
    try:
        system = platform.system()
        font_name = ""

        if system == 'Windows':
            font_name = 'Microsoft YaHei'
            plt.rcParams['font.sans-serif'] = [font_name]
        elif system == 'Darwin':
            font_name = 'PingFang HK'
            plt.rcParams['font.sans-serif'] = [font_name]
        else:
            font_paths = [
                '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc',
                '/usr/share/fonts/truetype/wqy/wqy-zenhei.ttc',
            ]
            found_font_path = next((path for path in font_paths if os.path.exists(path)), None)

            if found_font_path:
                font_manager.fontManager.addfont(found_font_path)
                font_name = font_manager.FontProperties(fname=found_font_path).get_name()
                plt.rcParams['font.sans-serif'] = [font_name]
            else:
                print("⚠️ 未在常見路徑找到中文字體，圖表中的中文可能無法正常顯示。")
                plt.rcParams['font.sans-serif'] = ['DejaVu Sans']

        plt.rcParams['axes.unicode_minus'] = False
        if font_name:
            print(f"✅ 已設定圖表字體為: {font_name}")

    except Exception as e:
        print(f"❌ 字體配置錯誤: {e}。使用預設字體。")
        plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
        plt.rcParams['axes.unicode_minus'] = False


# ==============================================================================
# 5. 升級：最終報告列印函數 (替換舊的 print_top_stocks_with_patterns)
# ==============================================================================
def print_final_report(results):
    """
    列印最終的頂尖股票分析報告 (V3.0 智能版)
    - 顯示中文K線趨勢摘要
    - 顯示智能交易建議
    """
    report_lines = []
    header = "🏆" * 20
    title = " " * 28 + "最終頂尖個股分析報告"

    report_lines.append("\n" + "="*80)
    report_lines.append(header)
    report_lines.append(title)
    report_lines.append(header)
    report_lines.append("="*80)

    if not results:
        report_lines.append("\n🤷 未能篩選出任何符合條件的股票。")
        final_report = "\n".join(report_lines)
        print(final_report)
        return final_report

    # 根據綜合評分從高到低排序
    sorted_results = sorted(results.values(), key=lambda x: x.get('combined_score', 0), reverse=True)

    for i, result in enumerate(sorted_results):
        report_lines.append(f"\n--- 排名 {i+1} -----------------------------------------------------------")

        stock_id = result.get('stock_id', 'N/A')
        stock_name = result.get('stock_name', 'N/A')
        industry = result.get('industry', 'N/A')
        last_price = result.get('last_price', 0.0)
        combined_score = result.get('combined_score', 0)

        # 獲取已生成的智能建議
        recommendation = result.get('recommendation', {})
        action = recommendation.get('action', '分析不完整')
        reason = recommendation.get('reason', '無法生成建議')

        # 獲取已生成的K線精華摘要
        pattern_report = result.get('pattern_report', 'K線型態報告未生成。')

        report_lines.append(f"📈 股票: {stock_id} {stock_name} ({industry})")
        report_lines.append(f"💰 最新股價: {last_price:.2f}")
        report_lines.append(f"⭐ 綜合評分: {combined_score}/100")
        report_lines.append(f"💡 交易建議: {action} - {reason}")
        report_lines.append(pattern_report) # 直接使用已生成的報告

        report_lines.append("-" * 70)

    report_lines.append("\n" + "="*80)
    report_lines.append("分析報告結束。祝您投資順利！")

    final_report = "\n".join(report_lines)
    print(final_report)
    return final_report




    # (可選) 在此處加入將報告發送到 Telegram 的邏輯
    final_report_str = print_final_report(results_to_show)

    send_notifications(final_report_str)

# 修正並優化後的主選單函數
def main_menu():
    """
    主程式執行流程 (最終優化版)
    """
    print("="*40)
    print("==== 台股多股技術分析與篩選工具 ====")
    print("="*40)
    print("🚀 開始自動分析所有股票...")

    try:
        # [階段1/7] 獲取股票基本資料
        print("\n[階段1/7] 獲取所有股票基本資料...")
        stocks_info = get_stock_basic_info()
        if not stocks_info:
            error_msg = "❌ 無法獲取股票基本資料，流程中止。"
            print(error_msg)
            send_notifications(f"🚨 *股票分析失敗*\n\n{error_msg}")
            return
        print(f"✅ 成功獲取 {len(stocks_info)} 支股票資料")

        # [階段2/7] 批次下載歷史資料
        print("\n[階段2/7] 批次下載歷史資料...")
        all_stocks_data = fetch_multiple_stocks_data(stocks_info, period='1y')
        if not all_stocks_data:
            error_msg = "❌ 無法取得任何股票歷史資料，流程中止。"
            print(error_msg)
            send_notifications(f"🚨 *股票分析失敗*\n\n{error_msg}")
            return

        # [階段3/7] 過濾低成交量股票
        print("\n[階段3/7] 過濾低成交量股票...")
        filtered_data = filter_by_volume(all_stocks_data, min_volume_shares=1000000)
        if not filtered_data:
            error_msg = "❌ 沒有任何個股符合成交量標準 (近一個月每日成交量 > 1000張)。"
            print(error_msg)
            send_notifications(f"🚨 *股票分析結果*\n\n{error_msg}")
            return

        progress_msg = f"📊 *股票分析進度報告*\n\n✅ 已獲取 {len(stocks_info)} 支股票資料\n✅ 符合成交量標準: {len(filtered_data)} 支\n🔄 開始進行技術分析..."
        send_notifications(progress_msg)

        # [階段4/7] 核心分析與評分
        print(f"\n[階段4/7] 進行核心分析與綜合評分 ({len(filtered_data)} 支)...")
        all_analysis_results = {}
        successful_analysis, failed_analysis = 0, 0

        for i, (stock_id, stock_data) in enumerate(filtered_data.items()):
            print(f"  分析中 ({i+1}/{len(filtered_data)}): {stock_id} {stock_data.get('stock_name', '')}")
            try:
                analysis_result = analyze_stock(stock_id, stock_data)
                if analysis_result.get('success'):
                    scores, final_score = calculate_comprehensive_score(analysis_result)
                    analysis_result.update({
                        'scores': {k: int(round(v)) for k, v in scores.items()},
                        'combined_score': int(round(final_score))
                        # 建議先不在此產生，待K線分析後統一產生
                    })
                    all_analysis_results[stock_id] = analysis_result
                    successful_analysis += 1
                else:
                    failed_analysis += 1
                    logger.warning(f"跳過股票 {stock_id}，分析失敗: {analysis_result.get('message', '未知錯誤')}")
            except Exception as e:
                failed_analysis += 1
                logger.error(f"分析股票 {stock_id} 時發生無法預期的異常: {str(e)}")

        print(f"\n分析完成: 成功 {successful_analysis} 支，失敗 {failed_analysis} 支")

        if not all_analysis_results:
            error_msg = "❌ 分析完成，但沒有任何有效的分析結果。"
            print(error_msg)
            send_notifications(f"🚨 *股票分析結果*\n\n{error_msg}")
            return

        # [階段5/7] 篩選頂尖個股並生成K線報告與建議 (合併原5, 7, 8階段)
        print("\n[階段5/7] 篩選頂尖個股並生成報告與建議...")

        # 直接從所有分析結果中，按分數排序，取前10名作為頂尖個股
        top_10_stocks = sorted(
            all_analysis_results.items(),
            key=lambda item: item[1].get('combined_score', 0),
            reverse=True
        )[:10]
        results_to_show = dict(top_10_stocks)

        if not results_to_show:
             print("🤷 未篩選出任何頂尖個股。")
        else:
            print(f"✅ 已篩選出分數最高的 {len(results_to_show)} 支股票進行最終報告。")

        success_count, error_count = 0, 0
        for stock_id, result in results_to_show.items():
            print(f"  處理中 ({success_count + error_count + 1}/{len(results_to_show)}): {stock_id}")
            df_for_patterns = result.get('data_df')

            if df_for_patterns is not None and not df_for_patterns.empty:
                try:
                    df_copy = df_for_patterns.copy()
                    df_copy.columns = [col.lower() for col in df_copy.columns]

                    date_col = next((col for col in ['date', 'datetime', 'time'] if col in df_copy.columns), None)

                    if date_col:
                        try:
                            df_copy[date_col] = pd.to_datetime(df_copy[date_col])
                            df_copy.set_index(date_col, inplace=True)
                        except Exception as e:
                            #print(f"    警告: 無法將 {date_col} 設為索引: {e}。創建臨時索引。")
                            df_copy.set_index(pd.date_range(start='2023-01-01', periods=len(df_copy)), inplace=True)
                    else:
                        #print("    警告: 未找到日期列，創建臨時日期索引。")
                        df_copy.set_index(pd.date_range(start='2023-01-01', periods=len(df_copy)), inplace=True)

                    required_cols = ['open', 'high', 'low', 'close']
                    if all(col in df_copy.columns for col in required_cols):
                        full_pattern_report = generate_pattern_report(df_copy, periods=['daily', 'weekly', 'monthly'])
                        condensed_report, report_summary = generate_condensed_pattern_report_and_summary(full_pattern_report)

                        # *** 修正點：加入調試信息 ***
                        result['pattern_report'] = condensed_report
                        combined_score = result.get('combined_score', 0)

                        # 🔍 調試信息：在調用前檢查參數
                        #print(f"    🔍 調用 generate_recommendation:")
                        #print(f"        stock_id: {stock_id}")
                        #print(f"        combined_score: {combined_score} (type: {type(combined_score)})")
                        #print(f"        report_summary 存在: {report_summary is not None}")
                        #if report_summary:
                            #print(f"        report_summary keys: {list(report_summary.keys())}")

                        recommendation = generate_recommendation(combined_score, report_summary)
                        result['recommendation'] = recommendation

                        # 🔍 調試信息：檢查返回結果
                        #print(f"    🔍 generate_recommendation 返回:")
                        #print(f"        action: {recommendation.get('action', 'N/A')}")
                        #print(f"        confidence: {recommendation.get('confidence', 'N/A')}")

                        #print(f"    報告已生成: 摘要長度 {len(condensed_report)} 字符")
                        #success_count += 1
                    else:
                        missing_cols = [col for col in required_cols if col not in df_copy.columns]
                        logger.warning(f"無法為股票 {stock_id} 生成報告，缺少欄位: {missing_cols}")
                        result['pattern_report'] = f"K線報告生成失敗：缺少欄位。"
                        result['recommendation'] = {"action": "錯誤", "reason": "數據不完整", "confidence": 0}
                        error_count += 1
                except Exception as e:
                    logger.error(f"為股票 {stock_id} 生成K線報告時發生錯誤: {e}")
                    result['pattern_report'] = "K線報告生成時發生錯誤。"
                    result['recommendation'] = {"action": "錯誤", "reason": "分析過程異常", "confidence": 0}
                    error_count += 1
            else:
                logger.warning(f"股票 {stock_id} 的數據為空，無法生成報告。")
                result['pattern_report'] = "數據不足，無法生成K線報告。"
                result['recommendation'] = {"action": "錯誤", "reason": "數據不足", "confidence": 0}
                error_count += 1

        print_top_stocks(results_to_show, top_n=10)
        print(f"\n報告生成結果: 成功 {success_count} 支，失敗 {error_count} 支")
        print("✅ 所有頂尖個股報告與建議已生成。")

        # [階段6/7] 生成報告與發送通知
        print("\n[階段6/7] 生成報告、圖表與發送通知...")

        summary_msg = format_notification_message(
            results_to_show,
            successful_analysis,
            failed_analysis,
            len(filtered_data)
        )

        report_dir = 'reports'
        os.makedirs(report_dir, exist_ok=True)
        generated_images = []

        # 只為分數最高的前5名生成圖表
        for stock_id, stock_info in list(results_to_show.items())[:5]:
            print(f"  生成圖表: {stock_id}")
            image_path = generate_detailed_analysis_report(
                stock_id=stock_id,
                analysis_result=stock_info,
                output_dir=report_dir
            )
            if image_path and os.path.exists(image_path):
                generated_images.append(image_path)

        export_html_report(all_analysis_results)

        final_message = summary_msg + f"\n\n📊 已生成 {len(generated_images)} 張技術分析圖表\n📋 HTML 詳細報告已匯出"
        send_notifications(final_message, generated_images)

        # [階段7/7] 打印最終報告
        if results_to_show:
            print("\n\n========== 🏆 頂尖個股分析報告 🏆 ==========\n")
            for stock_id, result in results_to_show.items():
                try:
                    stock_name = result.get('stock_name', '') or get_stock_name(stock_id)
                    print(f"【{stock_id} {stock_name}】")
                    print(f"綜合評分: {result.get('combined_score', 0)}")

                    pattern_report = result.get('pattern_report', '無K線型態報告')
                    print(f"K線分析:\n{pattern_report}")

                    recommendation = result.get('recommendation', {})
                    action = recommendation.get('action', '無建議')
                    reason = recommendation.get('reason', '無理由')
                    confidence = recommendation.get('confidence', 0)
                    print(f"\n投資建議: {action}")
                    print(f"建議理由: {reason}")
                    print(f"信心度: {confidence}%")

                    if 'pattern_details' in recommendation:
                        print("型態詳情:")
                        for detail in recommendation['pattern_details']:
                            print(f"  • {detail}")

                    print("\n關鍵指標:")
                    indicators = result.get('indicators', {})
                    for key in ['rsi', 'macd', 'bb_width', 'atr']:
                        if key in indicators:
                            print(f"  {key.upper()}: {indicators[key]:.2f}")

                    print("-" * 50 + "\n")
                except Exception as e:
                    print(f"❌ 打印股票 {stock_id} 的報告時發生錯誤: {e}")
        else:
            print("\n⚠️ 沒有頂尖個股可供顯示。")

        print("\n🎉 全部流程執行完畢！")

    except Exception as e:
        error_msg = f"❌ 主程序發生未預期錯誤: {e}"
        print(error_msg)
        logger.error(error_msg)
        logger.debug(traceback.format_exc())
        send_notifications(f"🚨 *股票分析嚴重失敗*\n\n{error_msg}")

if __name__ == "__main__":
    check_environment()
    set_chinese_font()
    main_menu()

In [ ]:
# -*- coding: utf-8 -*-
"""
台股技術 + 情緒 + 機器學習強化 評分系統 (完整整合版)
=================================================
功能包含：
1. 取得台股清單與備援
2. yfinance 抓取歷史日線
3. 技術指標：RSI / MACD / KD / 布林帶 / 均線
4. 新聞標題情緒 + 熱度統計（多來源；簡易字典情緒）
5. 台灣市場 Fear & Greed 指數（價動能、波動比、集中市場成交量、選擇權 Put/Call Ratio）
6. 綜合評分：趨勢 / 成交量 / 技術 / 情緒 / ML預測（可選）
7. Telegram / Discord 通知
8. 終端/文字輸出格式化
9. (更新) 使用 TWSE 公開 API 真實取得：本益比(PE)、三大法人買賣超(籌碼)，替換過去隨機特徵
=================================================
安裝需求：
pip install pandas numpy yfinance requests beautifulsoup4 jieba aiohttp nest_asyncio scikit-learn discord-webhook
"""

import os
import re
import io
import time
import json
import math
import random
import logging
import traceback
import asyncio
import aiohttp
import requests
import pandas as pd
import numpy as np
import yfinance as yf
import jieba
from bs4 import BeautifulSoup
from typing import Dict, Any, List, Optional, Tuple
from datetime import datetime, timezone, timedelta
from collections import Counter
from io import StringIO
from functools import lru_cache

# ML
try:
    from sklearn.ensemble import RandomForestRegressor
    SKLEARN_AVAILABLE = True
except ImportError:
    SKLEARN_AVAILABLE = False

# Discord
try:
    from discord_webhook import DiscordWebhook
    MESSAGING_AVAILABLE = True
except ImportError:
    MESSAGING_AVAILABLE = False

# nest_asyncio（在 Jupyter 等需要）
try:
    import nest_asyncio
    nest_asyncio.apply()
except Exception:
    pass

# ========= 日誌設定 =========
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger("StockSentimentSystem")

# ========= 時區 =========
taipei_tz = timezone(timedelta(hours=8))

# ========= 通訊設定 (請自行更換) =========
TELEGRAM_BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN", "7902318521:AAEYoDMqwfHabI7L1SRiE4z33aFay42-VGE")
TELEGRAM_CHAT_IDS = [int(x) for x in os.getenv("TELEGRAM_CHAT_IDS", [879781796, 8113868436]).split(",")]
DISCORD_WEBHOOK_URL = os.getenv("DISCORD_WEBHOOK_URL", "https://discord.com/api/webhooks/1362715080734802102/Jma7A3VhEQrrRxIX_JW2l6rATjAZXsGXGfnJuAMqmS1QvqG_2ptg3vr_nsVnuV_PlnBl")
TELEGRAM_TOKEN = "7902318521:AAEYoDMqwfHabI7L1SRiE4z33aFay42-VGE"
TELEGRAM_CHAT_ID = [879781796, 8113868436]
DISCORD_WEBHOOK_URL = "https://discord.com/api/webhooks/1362715080734802102/Jma7A3VhEQrrRxIX_JW2l6rATjAZXsGXGfnJuAMqmS1QvqG_2ptg3vr_nsVnuV_PlnBl"


# ========= 機器學習相關（可選） =========
def adaptive_weights(market_condition: str) -> Dict[str, float]:
    if market_condition == "bull":
        return {"pe": 0.2, "volume": 0.3, "ma": 0.3, "chips": 0.2}
    elif market_condition == "bear":
        return {"pe": 0.4, "volume": 0.2, "ma": 0.2, "chips": 0.2}
    else:
        return {"pe": 0.3, "volume": 0.25, "ma": 0.25, "chips": 0.2}

def build_ml_model(historical_data: pd.DataFrame) -> Optional[RandomForestRegressor]:
    if not SKLEARN_AVAILABLE:
        logger.warning("scikit-learn 未安裝，跳過 ML 模型建立")
        return None
    required_cols = ['pe_ratio', 'volume', 'ma_signal', 'chips_score', 'future_return']
    for c in required_cols:
        if c not in historical_data.columns:
            logger.warning(f"歷史資料缺少欄位: {c}，跳過 ML")
            return None
    try:
        model = RandomForestRegressor(n_estimators=200, random_state=42)
        model.fit(historical_data[['pe_ratio', 'volume', 'ma_signal', 'chips_score']], historical_data['future_return'])
        logger.info("ML 模型訓練完成")
        return model
    except Exception as e:
        logger.error(f"ML 模型訓練失敗: {e}")
        return None

def ml_predict_current(model: RandomForestRegressor,
                       pe_ratio: float,
                       volume: float,
                       ma_signal: int,
                       chips_score: int) -> float:
    if model is None:
        return 0.0
    try:
        df_feat = pd.DataFrame([{
            'pe_ratio': pe_ratio,
            'volume': volume,
            'ma_signal': ma_signal,
            'chips_score': chips_score
        }])
        pred = float(model.predict(df_feat)[0])
        return pred
    except Exception as e:
        logger.error(f"ML 預測失敗: {e}")
        return 0.0

# ========= 真實特徵抓取器 =========
class RealFeatureFetcher:
    """
    從台灣證交所公開 API 取得：
    1. 本益比、殖利率、股價淨值比 (BWIBBU_d)
    2. 三大法人買賣超 (T86)
    (具備日期回退、快取、轉換/清理)
    """
    def __init__(self):
        self.session = requests.Session()
        self.headers = {'User-Agent': 'Mozilla/5.0'}
        self._fundamental_cache: Dict[str, Dict[str, Any]] = {}
        self._chips_cache: Dict[str, Dict[str, Any]] = {}
        self._last_date_used = None

    @staticmethod
    def _date_str_list(days_back=7) -> List[str]:
        today = datetime.now(taipei_tz).date()
        dates = []
        for i in range(days_back):
            d = today - timedelta(days=i)
            dates.append(d.strftime("%Y%m%d"))
        return dates

    def _fetch_json(self, url: str, params: Dict[str, Any] = None, retry=3) -> Optional[Dict[str, Any]]:
        for attempt in range(retry):
            try:
                r = self.session.get(url, params=params, headers=self.headers, timeout=15)
                if r.status_code == 200:
                    return r.json()
                else:
                    logger.debug(f"URL {url} 狀態碼 {r.status_code}")
            except Exception as e:
                logger.debug(f"請求失敗 {url}: {e}")
            time.sleep(1 + attempt)
        return None

    def _load_fundamental_for_date(self, date_str: str) -> bool:
        """
        取得單一日期本益比資料
        API: https://www.twse.com.tw/rwd/zh/afterTrading/BWIBBU_d?date=YYYYMMDD&selectType=ALL
        """
        url = "https://www.twse.com.tw/rwd/zh/afterTrading/BWIBBU_d"
        params = {"date": date_str, "selectType": "ALL", "response": "json"}
        js = self._fetch_json(url, params=params)
        if not js or js.get("data") is None:
            return False
        fields = js.get("fields", [])
        data = js.get("data", [])
        # 典型欄位（可能變動）：['證券代號','證券名稱','殖利率(%)','股利年度','本益比','股價淨值比','財報年/季']
        try:
            col_index = {name: idx for idx, name in enumerate(fields)}
            for row in data:
                try:
                    code = row[col_index['證券代號']].strip()
                    pe_raw = row[col_index.get('本益比', -1)]
                    pb_raw = row[col_index.get('股價淨值比', -1)]
                    dy_raw = row[col_index.get('殖利率(%)', -1)]
                    # 清理
                    def to_float(v):
                        try:
                            v = str(v).replace(',', '').strip()
                            if v in ['-', 'N/A', 'nan', '']:
                                return float('nan')
                            return float(v)
                        except:
                            return float('nan')
                    pe = to_float(pe_raw)
                    pb = to_float(pb_raw)
                    dy = to_float(dy_raw)
                    self._fundamental_cache[code] = {
                        'pe_ratio': pe,
                        'pb_ratio': pb,
                        'div_yield': dy,
                        'date': date_str
                    }
                except Exception:
                    continue
            logger.info(f"取得本益比資料日期 {date_str} 共 {len(self._fundamental_cache)} 筆 (累積)")
            return True
        except Exception as e:
            logger.debug(f"解析本益比資料失敗 {date_str}: {e}")
            return False

    def _load_chips_for_date(self, date_str: str) -> bool:
        """
        取得三大法人資料
        API: https://www.twse.com.tw/rwd/zh/fund/T86?date=YYYYMMDD&selectType=ALL
        重點欄位：'證券代號','證券名稱','三大法人買賣超股數'
        """
        url = "https://www.twse.com.tw/rwd/zh/fund/T86"
        params = {"date": date_str, "selectType": "ALL", "response": "json"}
        js = self._fetch_json(url, params=params)
        if not js or js.get("data") is None:
            return False
        fields = js.get("fields", [])
        data = js.get("data", [])
        try:
            col_index = {name: idx for idx, name in enumerate(fields)}
            if '三大法人買賣超股數' not in col_index:
                # 結構變動 fallback
                return False
            for row in data:
                try:
                    code = row[col_index['證券代號']].strip()
                    net_raw = row[col_index['三大法人買賣超股數']]
                    net_raw = str(net_raw).replace(',', '').strip()
                    if net_raw in ['-', '']:
                        net_val = 0
                    else:
                        net_val = int(net_raw)
                    self._chips_cache[code] = {
                        'chips_net': net_val,
                        'date': date_str
                    }
                except Exception:
                    continue
            logger.info(f"取得三大法人資料日期 {date_str} 共 {len(self._chips_cache)} 筆 (累積)")
            return True
        except Exception as e:
            logger.debug(f"解析三大法人資料失敗 {date_str}: {e}")
            return False

    def prepare_latest(self, max_back_days: int = 7):
        """
        依序往前回退日期直到抓到資料或達上限
        """
        if self._last_date_used is not None:
            return  # 已經抓過當日/最近一次
        for d in self._date_str_list(max_back_days):
            f_ok = self._load_fundamental_for_date(d)
            c_ok = self._load_chips_for_date(d)
            if f_ok or c_ok:
                self._last_date_used = d
                break
        if self._last_date_used is None:
            logger.warning("無法取得最近 7 日內任何本益比 / 籌碼資料，將使用預設值")

    def get_features(self, stock_id: str) -> Dict[str, Any]:
        """
        回傳指定股票的特徵：
        pe_ratio, pb_ratio, div_yield, chips_net, chips_score
        chips_score 計算邏輯：
            - 依所有有資料股票的 chips_net 排名 (分位數)
            - 分位數 > 0.66 ⇒ 2
            - 0.33 ~ 0.66 ⇒ 1
            - <= 0.33 ⇒ 0
            - 若為顯著負值且小於 20% 分位可給 -1（供 ML 使用，不進入原本的組合評分）
        """
        # 確保資料已載入
        if self._last_date_used is None:
            self.prepare_latest()

        feat = {
            'pe_ratio': float('nan'),
            'pb_ratio': float('nan'),
            'div_yield': float('nan'),
            'chips_net': 0,
            'chips_score': 0
        }
        if stock_id in self._fundamental_cache:
            f = self._fundamental_cache[stock_id]
            feat['pe_ratio'] = f.get('pe_ratio', float('nan'))
            feat['pb_ratio'] = f.get('pb_ratio', float('nan'))
            feat['div_yield'] = f.get('div_yield', float('nan'))
        if stock_id in self._chips_cache:
            feat['chips_net'] = self._chips_cache[stock_id]['chips_net']

        # 計算籌碼分位數（只做一次全域排序）
        if not hasattr(self, '_chips_rank_prepared'):
            all_nets = [v['chips_net'] for v in self._chips_cache.values() if isinstance(v.get('chips_net'), int)]
            if len(all_nets) > 10:
                arr = np.array(all_nets)
                self._chips_p33 = np.percentile(arr, 33)
                self._chips_p66 = np.percentile(arr, 66)
                self._chips_p20 = np.percentile(arr, 20)
            else:
                self._chips_p33 = self._chips_p66 = self._chips_p20 = 0
            self._chips_rank_prepared = True

        try:
            cn = feat['chips_net']
            if ' _chips_p33' in dir(self):
                # 直接用屬性
                pass
            if cn <= self._chips_p20:
                feat['chips_score'] = -1  # 顯著偏空
            if cn > self._chips_p66:
                feat['chips_score'] = 2
            elif cn > self._chips_p33:
                feat['chips_score'] = 1
            # <= p33 且尚未標記 -1 時保持 0
        except Exception:
            feat['chips_score'] = 0

        # fallback: pe_ratio 若為 nan 給平均值 15~20
        if math.isnan(feat['pe_ratio']):
            feat['pe_ratio'] = 18.0
        return feat

# ========= 市場情緒分析器 =========
class MarketSentimentAnalyzer:
    def __init__(self):
        self.config = {
            'news_weight': 0.4,
            'fear_greed_weight': 0.6,
            'news_sources': [
                'https://news.cnyes.com/news/cat/tw_stock',
                'https://www.moneydj.com/kmdj/news/newsreallist.aspx?a=5',
                'https://www.cnbc.com/world/?region=world',
            ],
            'positive_keywords': ['上漲', '突破', '利多', '成長', '獲利', '看好', '強勢', '漲停', '突圍', '買進', '創高', '超預期'],
            'negative_keywords': ['下跌', '跌破', '利空', '衰退', '虧損', '看淡', '弱勢', '跌停', '重挫', '賣出', '調降', '裁員'],
            'cache_duration': 1800
        }
        self.news_cache: Dict[str, Any] = {}
        self.fear_greed_cache: Dict[str, Any] = {'timestamp': 0, 'value': 50, 'components': {}}

    def get_stock_news_sentiment(self, stock_id: str, stock_name: str) -> Tuple[float, int]:
        cache_key = f"{stock_id}_{stock_name}"
        now_ts = time.time()
        if cache_key in self.news_cache and (now_ts - self.news_cache[cache_key]['timestamp'] < self.config['cache_duration']):
            return self.news_cache[cache_key]['sentiment_score'], self.news_cache[cache_key]['news_count']
        search_terms = [stock_id, stock_name]
        news_count = 0
        sentiment_scores = []
        for url in self.config['news_sources']:
            try:
                headers = {'User-Agent': 'Mozilla/5.0'}
                resp = requests.get(url, headers=headers, timeout=12)
                soup = BeautifulSoup(resp.text, 'html.parser')
                candidates = soup.select('a, h2, h3, div')
                for c in candidates:
                    text = c.get_text(strip=True)
                    if not text or len(text) > 60:
                        continue
                    if any(term in text for term in search_terms):
                        news_count += 1
                        words = jieba.lcut(text)
                        pos = sum(1 for w in words if w in self.config['positive_keywords'])
                        neg = sum(1 for w in words if w in self.config['negative_keywords'])
                        if pos + neg > 0:
                            s = (pos - neg) / (pos + neg)
                            sentiment_scores.append(s)
            except Exception as e:
                logger.debug(f"新聞來源失敗 {url}: {e}")
        if sentiment_scores:
            avg = float(np.mean(sentiment_scores))
            score_0_100 = (avg + 1) * 50
        else:
            score_0_100 = 50.0
        self.news_cache[cache_key] = {
            'timestamp': now_ts,
            'sentiment_score': score_0_100,
            'news_count': news_count
        }
        return score_0_100, news_count

    def get_fear_greed_index(self) -> Tuple[float, Dict[str, Any]]:
        now_ts = time.time()
        if now_ts - self.fear_greed_cache['timestamp'] < self.config['cache_duration']:
            return self.fear_greed_cache['value'], self.fear_greed_cache['components']
        headers = {'User-Agent': 'Mozilla/5.0'}
        components = {
            'price_momentum': None,
            'price_score': 50,
            'volatility_ratio': None,
            'volatility_score': 50,
            'volume_change_pct': None,
            'volume_score': 50,
            'pc_ratio': None,
            'pc_ratio_score': 50
        }
        try:
            taiex_url = "https://query1.finance.yahoo.com/v8/finance/chart/%5ETWII?interval=1d&range=60d"
            r = requests.get(taiex_url, headers=headers, timeout=12).json()
            closes = r['chart']['result'][0]['indicators']['quote'][0]['close']
            closes = [c for c in closes if c is not None]
            if len(closes) >= 20:
                ma10 = np.mean(closes[-10:])
                last = closes[-1]
                price_momentum = (last / ma10 - 1) * 100 if ma10 else 0
                components['price_momentum'] = price_momentum
                price_score = 50 + price_momentum * 5
                components['price_score'] = max(0, min(100, price_score))
                ret = np.diff(closes) / closes[:-1]
                if len(ret) >= 21:
                    vol_5 = np.std(ret[-5:]) * 100
                    vol_20 = np.std(ret[-20:]) * 100
                    vol_ratio = vol_5 / vol_20 if vol_20 else 1
                    components['volatility_ratio'] = vol_ratio
                    volatility_score = 100 - (vol_ratio - 0.5) * 100
                    components['volatility_score'] = max(0, min(100, volatility_score))
            volume_url = "https://www.twse.com.tw/rwd/zh/afterTrading/FMTQIK?date=20230101&response=json"
            try:
                rv = requests.get(volume_url, headers=headers, timeout=12).json()
                vol_list = []
                if 'data' in rv:
                    for row in rv['data'][-8:]:
                        try:
                            val = row[1].replace(',', '')
                            vol_list.append(float(val))
                        except:
                            pass
                if len(vol_list) >= 5:
                    recent = np.mean(vol_list[-5:])
                    prev = np.mean(vol_list[-10:-5]) if len(vol_list) >= 10 else recent
                    volume_change = (recent / prev - 1) * 100 if prev else 0
                    components['volume_change_pct'] = volume_change
                    components['volume_score'] = max(0, min(100, 50 + volume_change))
            except Exception as e:
                logger.debug(f"FearGreed 成交量抓取失敗: {e}")
            try:
                pc_url = "https://www.taifex.com.tw/cht/3/pcRatio"
                pr = requests.get(pc_url, headers=headers, timeout=12)
                soup = BeautifulSoup(pr.text, 'html.parser')
                tables = soup.find_all('table')
                pc_ratio = None
                for tbl in tables:
                    tds = tbl.find_all('td')
                    for td in tds:
                        txt = td.get_text(strip=True)
                        if re.match(r'^\d+(\.\d+)?$', txt):
                            val = float(txt)
                            if 0.3 < val < 5:
                                pc_ratio = val
                                break
                    if pc_ratio:
                        break
                if pc_ratio:
                    components['pc_ratio'] = pc_ratio
                    pc_score = 50 + (pc_ratio - 1) * 40
                    components['pc_ratio_score'] = max(0, min(100, pc_score))
            except Exception as e:
                logger.debug(f"FearGreed P/C 失敗: {e}")
            final_value = (
                components['price_score'] * 0.5 +
                components['volatility_score'] * 0.2 +
                components['volume_score'] * 0.15 +
                components['pc_ratio_score'] * 0.15
            )
            final_value = max(0, min(100, final_value))
            market_mood = "極度恐慌" if final_value < 20 else \
                          "恐慌" if final_value < 40 else \
                          "中性" if final_value < 60 else \
                          "貪婪" if final_value < 80 else "極度貪婪"
            components['market_mood'] = market_mood
            self.fear_greed_cache = {
                'timestamp': now_ts,
                'value': final_value,
                'components': components
            }
            return final_value, components
        except Exception as e:
            logger.error(f"Fear & Greed 計算失敗: {e}")
            return self.fear_greed_cache['value'], self.fear_greed_cache['components']

    def calculate_sentiment_score(self, stock_id: str, stock_name: str) -> Dict[str, Any]:
        news_sentiment, news_count = self.get_stock_news_sentiment(stock_id, stock_name)
        fear_greed_value, fg_components = self.get_fear_greed_index()
        if fear_greed_value < 20 or fear_greed_value > 80:
            adj_fg_w = self.config['fear_greed_weight'] * 1.3
        else:
            adj_fg_w = self.config['fear_greed_weight']
        adj_news_w = self.config['news_weight'] * min(1.0, news_count / 5)
        total_w = adj_news_w + adj_fg_w
        if total_w == 0:
            total_w = 1
        adj_news_w /= total_w
        adj_fg_w /= total_w
        sentiment_score = news_sentiment * adj_news_w + fear_greed_value * adj_fg_w
        return {
            'sentiment_score': sentiment_score,
            'news_sentiment': news_sentiment,
            'news_count': news_count,
            'fear_greed_index': fear_greed_value,
            'market_mood': fg_components.get('market_mood', ''),
            'fear_greed_components': fg_components,
            'weight_news': adj_news_w,
            'weight_fear_greed': adj_fg_w
        }

# ========= 股票分析器 =========
class StockAnalyzer:
    def __init__(self, ml_model: Optional[RandomForestRegressor] = None):
        self.config = {
            'rsi_period': 14,
            'macd_fast': 12,
            'macd_slow': 26,
            'macd_signal': 9,
            'bb_period': 20,
            'bb_std': 2,
            'kd_period': 14,
            'volume_ma_period': 20,
            'min_volume_lots': 1000,
            'sentiment_weights': {
                'trend': 0.25,
                'volume': 0.10,
                'technical': 0.35,
                'sentiment': 0.20,
                'ml': 0.10
            }
        }
        self.stock_list_path = "taiwan_stocks_cache.csv"
        self.taiwan_stocks: Optional[pd.DataFrame] = None
        self.sentiment_analyzer = MarketSentimentAnalyzer()
        self.ml_model = ml_model
        self.feature_fetcher = RealFeatureFetcher()  # 新增真實特徵抓取器（本益比 / 籌碼）

    # --------- 股票列表取得 ---------
    def get_taiwan_stocks(self, force_update: bool = True) -> pd.DataFrame:
        if (not force_update) and os.path.exists(self.stock_list_path):
            if time.time() - os.path.getmtime(self.stock_list_path) < 86400:
                try:
                    return pd.read_csv(self.stock_list_path, dtype={'stock_id': str})
                except Exception:
                    pass
        urls = {
            "上市": "https://isin.twse.com.tw/isin/C_public.jsp?strMode=2",
            "上櫃": "https://isin.twse.com.tw/isin/C_public.jsp?strMode=4"
        }
        all_dfs = []
        headers = {'User-Agent': 'Mozilla/5.0'}
        for market_name, url in urls.items():
            try:
                resp = requests.get(url, headers=headers, timeout=30)
                resp.encoding = 'big5'
                tables = pd.read_html(StringIO(resp.text))
                df = tables[0].copy()
                df.columns = df.iloc[0]
                df = df.iloc[1:].copy()
                df['market'] = market_name
                all_dfs.append(df)
            except Exception as e:
                logger.error(f"取得 {market_name} 清單失敗: {e}")
        if not all_dfs:
            return self._get_backup_stock_list()
        try:
            df = pd.concat(all_dfs, ignore_index=True)
            df[['stock_id', 'stock_name']] = df['有價證券代號及名稱'].str.split(r'\s+', n=1, expand=True)
            df = df[df['stock_id'].str.match(r'^\d{4}$', na=False)]
            exclude = ['ETF', 'ETN', 'TDR', '受益', '指數', '購', '牛', '熊', '存託憑證']
            df = df[~df['stock_name'].str.contains('|'.join(exclude), na=False)]
            df['yahoo_symbol'] = df.apply(
                lambda r: f"{r['stock_id']}.TW" if '上市' in r['market'] else f"{r['stock_id']}.TWO", axis=1
            )
            final = df[['stock_id', 'stock_name', 'market', '產業別', 'yahoo_symbol']].rename(columns={'產業別': 'industry'})
            final = final.drop_duplicates(subset=['stock_id']).reset_index(drop=True)
            final.to_csv(self.stock_list_path, index=False)
            logger.info(f"取得股票清單 {len(final)} 支")
            return final
        except Exception as e:
            logger.error(f"處理清單錯誤: {e}")
            return self._get_backup_stock_list()

    def _get_backup_stock_list(self) -> pd.DataFrame:
        data = [
            {'stock_id': '2330', 'stock_name': '台積電', 'market': '上市', 'yahoo_symbol': '2330.TW', 'industry': '半導體'},
            {'stock_id': '2317', 'stock_name': '鴻海', 'market': '上市', 'yahoo_symbol': '2317.TW', 'industry': '電子'},
            {'stock_id': '2454', 'stock_name': '聯發科', 'market': '上市', 'yahoo_symbol': '2454.TW', 'industry': '半導體'},
            {'stock_id': '2881', 'stock_name': '富邦金', 'market': '上市', 'yahoo_symbol': '2881.TW', 'industry': '金融'},
            {'stock_id': '2882', 'stock_name': '國泰金', 'market': '上市', 'yahoo_symbol': '2882.TW', 'industry': '金融'},
            {'stock_id': '2303', 'stock_name': '聯電', 'market': '上市', 'yahoo_symbol': '2303.TW', 'industry': '半導體'},
            {'stock_id': '3008', 'stock_name': '大立光', 'market': '上市', 'yahoo_symbol': '3008.TW', 'industry': '光學'},
            {'stock_id': '3711', 'stock_name': '日月光投控', 'market': '上市', 'yahoo_symbol': '3711.TW', 'industry': '半導體'},
            {'stock_id': '2884', 'stock_name': '玉山金', 'market': '上市', 'yahoo_symbol': '2884.TW', 'industry': '金融'},
            {'stock_id': '2002', 'stock_name': '中鋼', 'market': '上市', 'yahoo_symbol': '2002.TW', 'industry': '鋼鐵'}
        ]
        return pd.DataFrame(data)

    # --------- 數據取得與指標 ---------
    def fetch_yfinance_data(self, symbol: str,
                            period: str = "1y",
                            interval: str = "1d",
                            retries: int = 3) -> pd.DataFrame:
        for attempt in range(retries):
            try:
                df = yf.download(symbol, period=period, interval=interval, progress=False, auto_adjust=True)
                if df.empty:
                    logger.warning(f"{symbol} 無資料")
                    return pd.DataFrame()
                if isinstance(df.columns, pd.MultiIndex):
                    df.columns = df.columns.get_level_values(0)
                df = df[['Open', 'High', 'Low', 'Close', 'Volume']].dropna()
                df.reset_index(inplace=True)
                return df
            except Exception as e:
                if attempt < retries - 1:
                    wait = 0.5 * (2 ** attempt) + random.random()
                    time.sleep(wait)
                else:
                    logger.error(f"{symbol} 抓取失敗: {e}")
        return pd.DataFrame()

    def calculate_rsi(self, prices: pd.Series, period: int = 14) -> pd.Series:
        delta = prices.diff()
        gain = delta.where(delta > 0, 0).rolling(period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(period).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        return rsi

    def calculate_macd(self, prices: pd.Series, fast=12, slow=26, signal=9):
        ema_fast = prices.ewm(span=fast).mean()
        ema_slow = prices.ewm(span=slow).mean()
        macd_line = ema_fast - ema_slow
        macd_signal = macd_line.ewm(span=signal).mean()
        macd_hist = macd_line - macd_signal
        return macd_line, macd_signal, macd_hist

    def calculate_bollinger_bands(self, prices: pd.Series, period=20, std_dev=2):
        mid = prices.rolling(period).mean()
        std = prices.rolling(period).std()
        upper = mid + std_dev * std
        lower = mid - std_dev * std
        return upper, mid, lower

    def calculate_kd(self, high: pd.Series, low: pd.Series, close: pd.Series, period=14):
        ll = low.rolling(period).min()
        hh = high.rolling(period).max()
        rsv = (close - ll) / (hh - ll) * 100
        k = rsv.ewm(alpha=1/3).mean()
        d = k.ewm(alpha=1/3).mean()
        return k, d

    def calculate_technical_indicators(self, df: pd.DataFrame) -> pd.DataFrame:
        if df.empty:
            return df
        df = df.copy()
        df['RSI'] = self.calculate_rsi(df['Close'], self.config['rsi_period'])
        macd_line, macd_signal, macd_hist = self.calculate_macd(
            df['Close'], self.config['macd_fast'], self.config['macd_slow'], self.config['macd_signal']
        )
        df['MACD'] = macd_line
        df['MACD_Signal'] = macd_signal
        df['MACD_Hist'] = macd_hist
        upper, mid, lower = self.calculate_bollinger_bands(df['Close'], self.config['bb_period'], self.config['bb_std'])
        df['BB_Upper'] = upper
        df['BB_Middle'] = mid
        df['BB_Lower'] = lower
        k, d = self.calculate_kd(df['High'], df['Low'], df['Close'], self.config['kd_period'])
        df['K_Percent'] = k
        df['D_Percent'] = d
        df['MA5'] = df['Close'].rolling(5).mean()
        df['MA10'] = df['Close'].rolling(10).mean()
        df['MA20'] = df['Close'].rolling(20).mean()
        df['Volume_MA'] = df['Volume'].rolling(self.config['volume_ma_period']).mean()
        return df

    # --------- 基礎分析 ---------
    def check_volume_filter(self, df: pd.DataFrame) -> bool:
        if df.empty:
            return False
        if 'Volume' not in df.columns:
            return False
        avg20 = df['Volume'].tail(20).mean()
        lots = avg20 / 1000.0
        return lots >= self.config['min_volume_lots']

    def analyze_trend(self, df: pd.DataFrame) -> Dict[str, Any]:
        if df.empty or len(df) < 20:
            return {'trend': '盤整', 'strength': 0, 'price_change_5d': 0, 'price_change_20d': 0}
        cp = float(df['Close'].iloc[-1])
        p5 = float(df['Close'].iloc[-6]) if len(df) >= 6 else cp
        p20 = float(df['Close'].iloc[-21]) if len(df) >= 21 else cp
        ch5 = (cp / p5 - 1) * 100 if p5 else 0
        ch20 = (cp / p20 - 1) * 100 if p20 else 0
        ma5 = df['MA5'].iloc[-1]
        ma20 = df['MA20'].iloc[-1]
        trend = '盤整'
        strength = 0
        if ch5 > 3 and ch20 > 5 and cp > ma5 > ma20:
            trend = '強勢上漲'; strength = 3
        elif ch5 > 1 and ch20 > 2 and cp > ma5:
            trend = '上漲'; strength = 2
        elif ch5 < -3 and ch20 < -5 and cp < ma5 < ma20:
            trend = '強勢下跌'; strength = -3
        elif ch5 < -1 and ch20 < -2 and cp < ma5:
            trend = '下跌'; strength = -2
        return {
            'trend': trend,
            'strength': strength,
            'price_change_5d': ch5,
            'price_change_20d': ch20,
            'ma5_position': ma5,
            'ma20_position': ma20
        }

    def analyze_volume(self, df: pd.DataFrame) -> Dict[str, Any]:
        if df.empty or len(df) < 20:
            return {'volume_trend': '正常', 'volume_ratio': 1.0, 'volume_signal': '中性', 'avg_volume_lots': 0}
        curr_vol = df['Volume'].iloc[-1]
        avg20 = df['Volume'].rolling(20).mean().iloc[-1]
        ratio = curr_vol / avg20 if avg20 else 1.0
        avg_lots = avg20 / 1000
        if ratio > 2:
            vt, vs = '爆量', '強烈'
        elif ratio > 1.5:
            vt, vs = '放量', '積極'
        elif ratio < 0.5:
            vt, vs = '縮量', '消極'
        else:
            vt, vs = '正常', '中性'
        return {
            'volume_trend': vt,
            'volume_ratio': ratio,
            'volume_signal': vs,
            'avg_volume_lots': avg_lots
        }

    def analyze_technical_indicators(self, df: pd.DataFrame) -> Dict[str, Any]:
        if df.empty:
            return {
                'rsi_signal': '中性', 'rsi_value': 50,
                'macd_signal': '中性', 'macd_value': 0,
                'kd_signal': '中性', 'k_value': 50, 'd_value': 50,
                'bb_signal': '中性', 'bb_position': 0.5,
                'score': 50
            }
        score = 50
        out = {}
        rsi_v = df['RSI'].iloc[-1]
        if math.isnan(rsi_v):
            rsi_v = 50
        out['rsi_value'] = rsi_v
        if rsi_v > 80:
            out['rsi_signal'] = '超買'; score -= 15
        elif rsi_v > 70:
            out['rsi_signal'] = '偏高'; score -= 5
        elif rsi_v < 20:
            out['rsi_signal'] = '超賣'; score += 15
        elif rsi_v < 30:
            out['rsi_signal'] = '偏低'; score += 5
        else:
            out['rsi_signal'] = '中性'
        macd_c = df['MACD'].iloc[-1]; macd_s = df['MACD_Signal'].iloc[-1]
        macd_p = df['MACD'].iloc[-2] if len(df) >= 2 else macd_c
        macd_sp = df['MACD_Signal'].iloc[-2] if len(df) >= 2 else macd_s
        out['macd_value'] = macd_c
        if macd_p <= macd_sp and macd_c > macd_s:
            out['macd_signal'] = '黃金交叉'; score += 20
        elif macd_p >= macd_sp and macd_c < macd_s:
            out['macd_signal'] = '死亡交叉'; score -= 20
        elif macd_c > macd_s:
            out['macd_signal'] = '多頭'; score += 5
        elif macd_c < macd_s:
            out['macd_signal'] = '空頭'; score -= 5
        else:
            out['macd_signal'] = '中性'
        k_v = df['K_Percent'].iloc[-1]; d_v = df['D_Percent'].iloc[-1]
        if math.isnan(k_v): k_v = 50
        if math.isnan(d_v): d_v = 50
        out['k_value'] = k_v; out['d_value'] = d_v
        if k_v > 80 and d_v > 80:
            out['kd_signal'] = '超買'; score -= 10
        elif k_v < 20 and d_v < 20:
            out['kd_signal'] = '超賣'; score += 10
        elif k_v > d_v:
            out['kd_signal'] = '偏多'; score += 3
        elif k_v < d_v:
            out['kd_signal'] = '偏空'; score -= 3
        else:
            out['kd_signal'] = '中性'
        upper = df['BB_Upper'].iloc[-1]; lower = df['BB_Lower'].iloc[-1]; cp = df['Close'].iloc[-1]
        if not (math.isnan(upper) or math.isnan(lower)) and upper != lower:
            pos = (cp - lower) / (upper - lower)
            out['bb_position'] = pos
            if pos > 0.8:
                out['bb_signal'] = '接近上軌'; score -= 5
            elif pos < 0.2:
                out['bb_signal'] = '接近下軌'; score += 5
            else:
                out['bb_signal'] = '中性'
        else:
            out['bb_signal'] = '中性'; out['bb_position'] = 0.5
        out['score'] = max(0, min(100, score))
        return out

    # --------- 綜合評分含情緒 & ML ---------
    def calculate_combined_score(self,
                                 trend_analysis: Dict[str, Any],
                                 volume_analysis: Dict[str, Any],
                                 technical_analysis: Dict[str, Any],
                                 sentiment_analysis: Optional[Dict[str, Any]] = None,
                                 ml_pred: Optional[float] = None) -> Dict[str, Any]:
        w = self.config['sentiment_weights']
        base = 50
        trend_strength = trend_analysis.get('strength', 0)
        trend_component = max(-30, min(30, trend_strength * 10))
        vr = volume_analysis.get('volume_ratio', 1.0)
        if vr > 1.8:
            vol_comp = 12
        elif vr > 1.5:
            vol_comp = 8
        elif vr > 1.2:
            vol_comp = 5
        elif vr < 0.6:
            vol_comp = -8
        elif vr < 0.8:
            vol_comp = -4
        else:
            vol_comp = 0
        tech_score = technical_analysis.get('score', 50) - 50
        tech_score = max(-50, min(50, tech_score))
        sent_score = (sentiment_analysis.get('sentiment_score', 50) - 50) if sentiment_analysis else 0
        if ml_pred is not None:
            ml_scaled = max(-30, min(30, ml_pred))
        else:
            ml_scaled = 0
        composite = (base +
                     trend_component * w['trend'] +
                     vol_comp * w['volume'] +
                     tech_score * w['technical'] +
                     sent_score * w['sentiment'] +
                     ml_scaled * w['ml'])
        composite = max(0, min(100, composite))
        return {
            'combined_score': composite,
            'components': {
                'trend_component': trend_component,
                'volume_component': vol_comp,
                'technical_component': tech_score,
                'sentiment_component': sent_score,
                'ml_component': ml_scaled
            },
            'weights': w
        }

    def generate_recommendation(self,
                                combined_score: float,
                                trend_analysis: Dict[str, Any],
                                technical_analysis: Dict[str, Any],
                                sentiment_analysis: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        if combined_score >= 80:
            rec, conf = "強力買進", "高"
        elif combined_score >= 65:
            rec, conf = "買進", "中高"
        elif combined_score >= 45:
            rec, conf = "持有", "中等"
        elif combined_score >= 30:
            rec, conf = "賣出", "中"
        else:
            rec, conf = "強力賣出", "高"
        reasons = []
        trend = trend_analysis.get('trend', '盤整')
        if '上漲' in trend:
            reasons.append(f"趨勢：{trend}")
        elif '下跌' in trend:
            reasons.append(f"趨勢：{trend}")
        rsi_sig = technical_analysis.get('rsi_signal', '中性')
        if rsi_sig in ['超賣', '偏低']:
            reasons.append(f"RSI {rsi_sig}")
        elif rsi_sig in ['超買', '偏高']:
            reasons.append(f"RSI {rsi_sig}")
        macd_sig = technical_analysis.get('macd_signal', '中性')
        if macd_sig == '黃金交叉':
            reasons.append("MACD 黃金交叉")
        elif macd_sig == '死亡交叉':
            reasons.append("MACD 死亡交叉")
        if sentiment_analysis:
            mood = sentiment_analysis.get('market_mood', '')
            if mood:
                reasons.append(f"市場情緒：{mood}")
        return {
            'recommendation': rec,
            'confidence': conf,
            'score': combined_score,
            'reasons': reasons[:5]
        }

    # --------- 單支股票分析 (含情緒 + 真實特徵 + ML) ---------
    async def analyze_stock_async(self,
                                  session: aiohttp.ClientSession,
                                  stock_info: Dict[str, Any]) -> Dict[str, Any]:
        symbol = stock_info['yahoo_symbol']
        stock_name = stock_info['stock_name']
        stock_id = stock_info['stock_id']
        try:
            df = self.fetch_yfinance_data(symbol)
            if df.empty or len(df) < 60:
                return {
                    'symbol': symbol,
                    'stock_name': stock_name,
                    'stock_id': stock_id,
                    'success': False,
                    'error': '數據不足'
                }
            if not self.check_volume_filter(df):
                return {
                    'symbol': symbol,
                    'stock_name': stock_name,
                    'stock_id': stock_id,
                    'success': False,
                    'error': '成交量不符合條件'
                }
            df_ind = self.calculate_technical_indicators(df)
            trend_analysis = self.analyze_trend(df_ind)
            volume_analysis = self.analyze_volume(df_ind)
            technical_analysis = self.analyze_technical_indicators(df_ind)
            sentiment_analysis = self.sentiment_analyzer.calculate_sentiment_score(stock_id, stock_name)

            # ====== (更新) 真實特徵取得 (取代原模擬隨機) ======
            real_feats = self.feature_fetcher.get_features(stock_id)
            pe_ratio = real_feats['pe_ratio']
            # 當下成交量使用最後一根 Volume（與 model 同量級）
            volume_value = float(df_ind['Volume'].iloc[-1])
            # ma_signal：利用均線多空結構
            cp = df_ind['Close'].iloc[-1]
            ma5 = df_ind['MA5'].iloc[-1]
            ma20 = df_ind['MA20'].iloc[-1]
            if cp > ma5 > ma20:
                ma_signal = 2
            elif cp > ma5:
                ma_signal = 1
            else:
                ma_signal = 0
            chips_score = real_feats['chips_score']
            # ====== ML 預測 ======
            ml_pred = None
            if self.ml_model:
                ml_pred = ml_predict_current(self.ml_model, pe_ratio, volume_value, ma_signal, chips_score)

            combined = self.calculate_combined_score(
                trend_analysis, volume_analysis, technical_analysis, sentiment_analysis, ml_pred
            )
            combined_score = combined['combined_score']
            recommendation = self.generate_recommendation(
                combined_score, trend_analysis, technical_analysis, sentiment_analysis
            )
            result = {
                'symbol': symbol,
                'stock_name': stock_name,
                'stock_id': stock_id,
                'market': stock_info.get('market', ''),
                'industry': stock_info.get('industry', ''),
                'success': True,
                'current_price': float(df_ind['Close'].iloc[-1]),
                'price_change_5d': trend_analysis.get('price_change_5d', 0),
                'combined_score': combined_score,
                'combined_components': combined.get('components', {}),
                'trend_analysis': trend_analysis,
                'volume_analysis': volume_analysis,
                'technical_analysis': technical_analysis,
                'sentiment_analysis': sentiment_analysis,
                'ml_predicted_return': ml_pred,
                'real_features': {
                    'pe_ratio': pe_ratio,
                    'chips_net': real_feats['chips_net'],
                    'chips_score': chips_score,
                    'pb_ratio': real_feats['pb_ratio'],
                    'div_yield': real_feats['div_yield'],
                    'ma_signal': ma_signal,
                    'volume_feature': volume_value
                },
                'recommendation': recommendation,
                'analysis_time': datetime.now(taipei_tz).isoformat()
            }
            return result
        except Exception as e:
            logger.error(f"{symbol} 分析錯誤: {e}")
            return {
                'symbol': symbol,
                'stock_name': stock_name,
                'stock_id': stock_id,
                'success': False,
                'error': str(e)
            }

    # --------- 批量分析 ---------
    async def analyze_all_stocks(self, limit: Optional[int] = None) -> Dict[str, Any]:
        if self.taiwan_stocks is None:
            self.taiwan_stocks = self.get_taiwan_stocks()
        df_list = self.taiwan_stocks
        if df_list.empty:
            logger.error("無股票清單")
            return {}
        if limit:
            df_list = df_list.head(limit)

        # 預先載入最新本益比 / 籌碼資料，避免每檔重複回退
        self.feature_fetcher.prepare_latest()

        results = {}
        tasks = []
        async with aiohttp.ClientSession() as session:
            for _, row in df_list.iterrows():
                tasks.append(self.analyze_stock_async(session, row.to_dict()))
            batch_size = 12
            for i in range(0, len(tasks), batch_size):
                batch = tasks[i:i+batch_size]
                batch_results = await asyncio.gather(*batch, return_exceptions=True)
                for r in batch_results:
                    if isinstance(r, dict) and 'symbol' in r:
                        results[r['symbol']] = r
                await asyncio.sleep(1)
        return results

# ========= 輸出格式化 =========
def format_analysis_message(results: Dict[str, Any], limit: int = 10) -> str:
    ok = [r for r in results.values() if r.get('success')]
    ok.sort(key=lambda x: x.get('combined_score', 0), reverse=True)
    lines = []
    now_str = datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')
    lines.append("🏆 台股綜合技術 + 情緒 + ML (真實特徵) 分析")
    lines.append(f"🕒 分析時間: {now_str}")
    lines.append(f"✅ 成功分析: {len(ok)} 支")
    lines.append(f"📊 評分組成: 趨勢/成交量/技術/情緒/ML")
    lines.append("=" * 60)
    if not ok:
        lines.append("❌ 無可用結果")
        return "\n".join(lines)
    top = ok[:limit]
    for idx, r in enumerate(top, 1):
        sa = r.get('sentiment_analysis', {})
        news_cnt = sa.get('news_count', 0)
        fear_greed = sa.get('fear_greed_index', 50)
        mood = sa.get('market_mood', '')
        fg_emoji = "😨" if fear_greed < 30 else "😐" if fear_greed < 70 else "🤑"
        rec = r.get('recommendation', {}).get('recommendation', '')
        feats = r.get('real_features', {})
        lines.append(f"{idx}. {r.get('stock_name')} ({r.get('stock_id')})")
        lines.append(f"   💰 價格: {r.get('current_price', 0):.2f} | 5日: {r.get('price_change_5d', 0):+.2f}% | 建議: {rec}")
        lines.append(f"   ⭐ 綜合評分: {r.get('combined_score', 0):.1f}  | PE:{feats.get('pe_ratio','-'):.2f} | 籌碼分數:{feats.get('chips_score','-')}")
        lines.append(f"   📰 新聞熱度: {news_cnt} | 新聞情緒: {sa.get('news_sentiment',50):.1f}")
        lines.append(f"   {fg_emoji} 市場情緒(F&G): {fear_greed:.1f} ({mood})")
        tech = r.get('technical_analysis', {})
        lines.append(f"   🔍 RSI {tech.get('rsi_value',50):.1f}({tech.get('rsi_signal','中性')}), MACD:{tech.get('macd_signal','中性')}, KD:{tech.get('k_value',50):.1f}/{tech.get('d_value',50):.1f}")
        lines.append("-" * 60)
    avg_score = np.mean([x.get('combined_score', 0) for x in ok]) if ok else 0
    high_cnt = sum(1 for x in ok if x.get('combined_score', 0) >= 70)
    lines.append(f"📈 平均分數: {avg_score:.1f} | 高分(>=70): {high_cnt}")
    lines.append("⚠️ 風險聲明：僅供參考，投資有風險。")
    return "\n".join(lines)

def display_terminal_results(results: Dict[str, Any], limit: int = 10):
    msg = format_analysis_message(results, limit)
    print(msg)

# ========= 通知 =========
async def send_notification(session: aiohttp.ClientSession, message: str, files: List[str] = None):
    """發送通知到 Telegram 和 Discord"""
    # 如果沒有提供檔案，嘗試從預設目錄獲取
    if not files:
        chart_dir = "/content/results/charts/"
        if os.path.exists(chart_dir):
            # 獲取目錄中的所有 PNG 檔案
            files = [os.path.join(chart_dir, f) for f in os.listdir(chart_dir)
                    if f.endswith('.png') and os.path.isfile(os.path.join(chart_dir, f))]
            if files:
                logger.info(f"找到 {len(files)} 個圖表檔案在 {chart_dir}")
            else:
                logger.warning(f"在 {chart_dir} 中找不到任何 PNG 檔案")

    # Telegram
    try:
        # 遍歷所有 Telegram Chat ID
        for chat_id in TELEGRAM_CHAT_ID:
            # 發送文字訊息
            url_msg = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage"
            payload = {"chat_id": chat_id, "text": message, "parse_mode": "Markdown"}
            async with session.post(url_msg, json=payload, timeout=20) as response:
                if response.status == 200:
                    logger.info(f"Telegram 摘要已成功發送到 chat_id: {chat_id}。")
                else:
                    response_text = await response.text()
                    logger.error(f"Telegram 摘要發送失敗: {response.status} - {response_text}")

            # 如果有檔案，也遍歷發送
            if files:
                url_photo = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendPhoto"
                sent_count = 0
                for file_path in files:
                    if os.path.exists(file_path):
                        try:
                            data = aiohttp.FormData()
                            data.add_field('chat_id', str(chat_id))  # 確保 chat_id 是字串
                            # 添加可選的圖片說明
                            caption = os.path.basename(file_path).replace('_report.png', '').replace('_', ' ')
                            data.add_field('caption', caption)

                            with open(file_path, 'rb') as f:
                                data.add_field('photo', f, filename=os.path.basename(file_path))
                                async with session.post(url_photo, data=data, timeout=60) as response:
                                    if response.status == 200:
                                        sent_count += 1
                                    else:
                                        response_text = await response.text()
                                        logger.error(f"Telegram 圖檔發送失敗: {response.status} - {response_text}")

                            # 添加小延遲以避免 Telegram API 限制
                            await asyncio.sleep(0.5)
                        except Exception as file_error:
                            logger.error(f"發送檔案 {file_path} 時出錯: {file_error}")

                logger.info(f"Telegram 圖檔已成功發送到 chat_id: {chat_id} ({sent_count}/{len(files)}個)。")
    except Exception as e:
        logger.error(f"發送 Telegram 通知時異常: {e}")
        traceback.print_exc()

    # Discord
    try:
        # 將訊息分段發送，避免超過 Discord 的限制
        message_chunks = [message[i:i+1900] for i in range(0, len(message), 1900)]

        for chunk in message_chunks:
            webhook = DiscordWebhook(url=DISCORD_WEBHOOK_URL, content=f"```\n{chunk}\n```")
            response = webhook.execute()
            if response and response.ok:
                logger.info("Discord 文字通知發送成功。")
            else:
                status_code = response.status_code if response else "未知"
                content = response.content if response else "未知"
                logger.error(f"Discord 文字通知失敗: {status_code} {content}")

        # 分批發送檔案，每批最多 10 個檔案
        if files:
            batch_size = 10
            for i in range(0, len(files), batch_size):
                batch_files = files[i:i+batch_size]
                webhook = DiscordWebhook(url=DISCORD_WEBHOOK_URL, content=f"圖表批次 {i//batch_size + 1}/{(len(files)-1)//batch_size + 1}")

                for file_path in batch_files:
                    if os.path.exists(file_path):
                        try:
                            with open(file_path, 'rb') as f:
                                webhook.add_file(file=f.read(), filename=os.path.basename(file_path))
                        except Exception as file_error:
                            logger.error(f"添加檔案 {file_path} 到 Discord webhook 時出錯: {file_error}")

                response = webhook.execute()
                if response and response.ok:
                    logger.info(f"Discord 圖檔批次 {i//batch_size + 1} 發送成功。")
                else:
                    status_code = response.status_code if response else "未知"
                    content = response.content if response else "未知"
                    logger.error(f"Discord 圖檔批次 {i//batch_size + 1} 發送失敗: {status_code} {content}")
    except Exception as e:
        logger.error(f"發送 Discord 通知時異常: {e}")
        traceback.print_exc()

def format_notification_message(filtered_results: Dict) -> str:
    """格式化通知訊息"""
    try:
        current_time = datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')

        if not filtered_results:
            return f"""
🤖 台股技術分析報告
📅 分析時間: {current_time}

❌ 本次分析未找到符合條件的優質股票
💡 建議調整篩選條件或關注市場變化
"""

        # 獲取圖表檔案數量
        chart_dir = "/content/results/charts/"
        chart_count = 0
        if os.path.exists(chart_dir):
            chart_files = [f for f in os.listdir(chart_dir) if f.endswith('.png') and os.path.isfile(os.path.join(chart_dir, f))]
            chart_count = len(chart_files)

        message = f"""
🤖 台股技術分析報告
📅 分析時間: {current_time}
🎯 找到 {len(filtered_results)} 支優質股票
📊 生成 {chart_count} 張技術分析圖表

📈 TOP {min(5, len(filtered_results))} 推薦股票:
"""

        for rank, (stock_id, result) in enumerate(list(filtered_results.items())[:5], 1):
            trend = result.get('trend_analysis', {})
            tech = result.get('technical_analysis', {})
            recommendation = result.get('recommendation', {})

            # 獲取更多技術指標數據
            macd = tech.get('macd_value', 0)
            signal = tech.get('signal_value', 0)
            macd_status = "多頭" if macd > signal else "空頭" if macd < signal else "中性"

            message += f"""
{rank}. {result.get('stock_name', '')} ({stock_id})
   💰 價格: {result.get('current_price', 0):.2f} TWD
   📈 評分: {result.get('combined_score', 0):.1f}/100
   🎯 建議: {recommendation.get('action', '觀望')}
   📊 趨勢: {trend.get('trend', '盤整')}
   🔍 RSI: {tech.get('rsi_value', 50):.1f} ({tech.get('rsi_status', '中性')})
   📉 MACD: {macd_status}
"""

        # 添加圖表資訊
        if chart_count > 0:
            message += f"""
📊 詳細分析圖表:
• 已生成 {chart_count} 張技術分析圖表
• 圖表包含K線、均線、成交量、MACD及RSI指標
• 圖表將在此訊息後發送
"""

        message += f"""
⚠️  投資提醒:
• 本分析僅供參考，投資有風險
• 建議結合基本面分析
• 請做好風險控制
"""

        return message.strip()

    except Exception as e:
        logger.error(f"格式化通知訊息時發生錯誤: {e}")
        traceback.print_exc()
        return f"台股分析完成，但訊息格式化失敗: {str(e)}"
# ========= 主程式入口 =========
async def main():
    print("🚀 啟動 台股技術 + 情緒 + ML (真實特徵) 分析系統")
    ml_model = None
    # 若要啟用 ML，請提供真實 historical_data：
    # historical_data = pd.read_csv('historical_features.csv')  # 需包含 pe_ratio, volume, ma_signal, chips_score, future_return
    # ml_model = build_ml_model(historical_data)

    analyzer = StockAnalyzer(ml_model=ml_model)
    print("📊 正在分析股票（可能需數分鐘）...")
    results = await analyzer.analyze_all_stocks(limit=2000)  # 可調整分析數量
    if not results:
        print("❌ 無分析結果")
        return
    display_terminal_results(results, limit=15)
    message = format_analysis_message(results, limit=15)

    async with aiohttp.ClientSession() as session:
        await send_notification(session, message)

    print("✅ 完成。")

if __name__ == "__main__":
    try:
        asyncio.run(main())
    except RuntimeError:
        loop = asyncio.get_event_loop()
        loop.run_until_complete(main())

In [ ]:
# ===============================
#  台股綜合分析系統 (融合版)
#  功能：
#   1. 股票清單抓取 + 快取 (含備援)
#   2. yfinance 抓 OHLCV
#   3. 技術指標：v1 (簡版) / v2 (完整擴充)
#   4. 新聞情緒 + 台灣 Fear & Greed 指數
#   5. 綜合評分 (趨勢 / 成交量 / 技術 / 情緒)
#   6. 背離偵測 (RSI vs Price)：常規 / 隱藏
#   7. 成交量條件過濾 (日/週)
#   8. 圖表輸出 (mplfinance)
#   9. Telegram / Discord 通知 (多段訊息 + Retry/Backoff)
#  作者：整合版（環境變數強化 + 改良重構）
# ===============================

# ===============================
# (可選) 安裝需求 (Jupyter 測試)
# ===============================
# !pip install --upgrade pip
# !pip install pandas numpy yfinance requests lxml html5lib seaborn matplotlib mplfinance chineseize-matplotlib -q
# !pip install twstock shioaji aiohttp nest-asyncio discord-webhook python-dotenv jieba -q

# ===============================
# Imports
# ===============================
import os
import sys
import re
import gc
import io
import time
import json
import math
import pickle
import random
import signal
import warnings
import logging
import traceback
import asyncio
import aiohttp
import requests
import platform
import dateutil.parser
import pandas as pd
import numpy as np
import yfinance as yf
import jieba

from io import StringIO
from functools import wraps
from datetime import datetime, timezone, timedelta
from typing import Dict, Any, List, Optional, Tuple, Union
from collections import Counter
from concurrent.futures import ThreadPoolExecutor, TimeoutError as FuturesTimeoutError

# 圖表
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import mplfinance as mpf

# BeautifulSoup
from bs4 import BeautifulSoup

# nest_asyncio（在 Jupyter 重入）
try:
    import nest_asyncio
    nest_asyncio.apply()
except Exception:
    pass

# Discord Webhook (可選)
try:
    from discord_webhook import DiscordWebhook
    MESSAGING_AVAILABLE = True
except ImportError:
    MESSAGING_AVAILABLE = False

# dotenv (可選)
try:
    from dotenv import load_dotenv
    load_dotenv()
except Exception:
    pass

# ===============================
# 全域設定
# ===============================
warnings.filterwarnings('ignore')
plt.switch_backend('Agg')

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("stock_analysis.log", encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger("FusionAnalyzer")

taipei_tz = timezone(timedelta(hours=8))

# ===============================
# 環境變數設定（無則留空 -> 不發送）
# ===============================

TELEGRAM_TOKEN = "7902318521:AAEYoDMqwfHabI7L1SRiE4z33aFay42-VGE"
TELEGRAM_CHAT_ID = [879781796, 8113868436]
DISCORD_WEBHOOK_URL = "https://discord.com/api/webhooks/1362715080734802102/Jma7A3VhEQrrRxIX_JW2l6rATjAZXsGXGfnJuAMqmS1QvqG_2ptg3vr_nsVnuV_PlnBl"

# ===============================
# 目錄結構
# ===============================
for d in ['data', 'data/stocks', 'cache', 'analysis_charts', 'analysis_reports', 'stock_reports']:
    os.makedirs(d, exist_ok=True)

# ===============================
# 字體設定（避免中文字亂碼）
# ===============================
def set_chinese_font():
    system = platform.system()
    if system == 'Windows':
        plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
    elif system == 'Darwin':
        plt.rcParams['font.sans-serif'] = ['PingFang TC', 'PingFang HK', 'Heiti TC']
    else:
        plt.rcParams['font.sans-serif'] = ['Noto Sans CJK TC', 'WenQuanYi Micro Hei', 'SimHei']
    plt.rcParams['axes.unicode_minus'] = False

set_chinese_font()

# ===============================
# 通用工具
# ===============================
def parse_date(date_str):
    try:
        return dateutil.parser.parse(date_str)
    except:
        return None

def ensure_date_column(df: pd.DataFrame):
    if df is None or df.empty:
        return None
    date_candidates = ['Date', 'date', '日期', 'TradeDate', 'time', 'datetime']
    for c in date_candidates:
        if c in df.columns:
            df = df.rename(columns={c: 'Date'})
            try:
                df['Date'] = pd.to_datetime(df['Date'])
            except:
                return None
            return df
    return None

# ===============================
# Timeout 機制
# ===============================
class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException("執行時間過長，已中止")

def timeout(seconds):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            if platform.system() == "Windows":
                with ThreadPoolExecutor(max_workers=1) as executor:
                    future = executor.submit(func, *args, **kwargs)
                    try:
                        return future.result(timeout=seconds)
                    except FuturesTimeoutError:
                        raise TimeoutException(f"操作超時 ({seconds} 秒)")
            else:
                try:
                    signal.signal(signal.SIGALRM, timeout_handler)
                    signal.alarm(seconds)
                    try:
                        return func(*args, **kwargs)
                    finally:
                        signal.alarm(0)
                except Exception:
                    with ThreadPoolExecutor(max_workers=1) as executor:
                        future = executor.submit(func, *args, **kwargs)
                        try:
                            return future.result(timeout=seconds)
                        except FuturesTimeoutError:
                            raise TimeoutException(f"操作超時 ({seconds} 秒)")
        return wrapper
    return decorator

# ===============================
# 快取工具
# ===============================
def _save_cache(cache_file, data):
    try:
        os.makedirs(os.path.dirname(cache_file), exist_ok=True)
        with open(cache_file, 'wb') as f:
            pickle.dump(data, f)
    except Exception as e:
        logger.error(f"保存快取失敗: {e}")

def _load_backup_cache(cache_file):
    try:
        if os.path.exists(cache_file):
            with open(cache_file, 'rb') as f:
                return pickle.load(f)
    except Exception as e:
        logger.error(f"載入備份快取失敗: {e}")
    return {}

# ===============================
# 驗證股票資料
# ===============================
def validate_stock_data(df: pd.DataFrame, min_points=20):
    if df is None or df.empty:
        return False
    if 'Date' not in df.columns:
        return False
    close_col = 'Close' if 'Close' in df.columns else ('close' if 'close' in df.columns else None)
    if close_col is None:
        return False
    if df[close_col].isnull().all():
        return False
    if len(df) < min_points:
        return False
    return True

# ===============================
# 市場情緒分析器
# ===============================
class MarketSentimentAnalyzer:
    def __init__(self):
        self.config = {
            'news_weight': 0.4,
            'fear_greed_weight': 0.6,
            'news_sources': [
                'https://news.cnyes.com/news/cat/tw_stock',
                'https://www.moneydj.com/kmdj/news/newsreallist.aspx?a=5',
                'https://www.cnbc.com/world/?region=world',
            ],
            'positive_keywords': ['上漲', '突破', '利多', '成長', '獲利', '看好', '強勢', '漲停', '突圍', '買進', '創高', '超預期'],
            'negative_keywords': ['下跌', '跌破', '利空', '衰退', '虧損', '看淡', '弱勢', '跌停', '重挫', '賣出', '調降', '裁員'],
            'cache_duration': 1800
        }
        self.news_cache = {}
        self.fear_greed_cache = {'timestamp': 0, 'value': 50, 'components': {}}

    def get_stock_news_sentiment(self, stock_id: str, stock_name: str) -> Tuple[float, int]:
        cache_key = f"{stock_id}_{stock_name}"
        now_ts = time.time()
        if cache_key in self.news_cache and (now_ts - self.news_cache[cache_key]['timestamp'] < self.config['cache_duration']):
            return self.news_cache[cache_key]['sentiment_score'], self.news_cache[cache_key]['news_count']
        search_terms = [stock_id, stock_name]
        news_count = 0
        sentiment_scores = []
        for url in self.config['news_sources']:
            try:
                headers = {'User-Agent': 'Mozilla/5.0'}
                resp = requests.get(url, headers=headers, timeout=12)
                soup = BeautifulSoup(resp.text, 'html.parser')
                candidates = soup.select('a, h2, h3, div')
                for c in candidates:
                    text = c.get_text(strip=True)
                    if not text or len(text) > 60:
                        continue
                    if any(term in text for term in search_terms):
                        news_count += 1
                        words = jieba.lcut(text)
                        pos = sum(1 for w in words if w in self.config['positive_keywords'])
                        neg = sum(1 for w in words if w in self.config['negative_keywords'])
                        if pos + neg > 0:
                            s = (pos - neg) / (pos + neg)
                            sentiment_scores.append(s)
            except Exception as e:
                logger.debug(f"新聞來源失敗 {url}: {e}")
        if sentiment_scores:
            avg = float(np.mean(sentiment_scores))
            score_0_100 = (avg + 1) * 50
        else:
            score_0_100 = 50.0
        self.news_cache[cache_key] = {
            'timestamp': now_ts,
            'sentiment_score': score_0_100,
            'news_count': news_count
        }
        return score_0_100, news_count

    def get_fear_greed_index(self) -> Tuple[float, Dict[str, Any]]:
        now_ts = time.time()
        if now_ts - self.fear_greed_cache['timestamp'] < self.config['cache_duration']:
            return self.fear_greed_cache['value'], self.fear_greed_cache['components']
        headers = {'User-Agent': 'Mozilla/5.0'}
        components = {
            'price_momentum': None,
            'price_score': 50,
            'volatility_ratio': None,
            'volatility_score': 50,
            'volume_change_pct': None,
            'volume_score': 50,
            'pc_ratio': None,
            'pc_ratio_score': 50
        }
        try:
            taiex_url = "https://query1.finance.yahoo.com/v8/finance/chart/%5ETWII?interval=1d&range=60d"
            r = requests.get(taiex_url, headers=headers, timeout=12).json()
            closes = r['chart']['result'][0]['indicators']['quote'][0]['close']
            closes = [c for c in closes if c is not None]
            if len(closes) >= 20:
                ma10 = np.mean(closes[-10:])
                last = closes[-1]
                price_momentum = (last / ma10 - 1) * 100 if ma10 else 0
                components['price_momentum'] = price_momentum
                price_score = 50 + price_momentum * 5
                components['price_score'] = max(0, min(100, price_score))
                ret = np.diff(closes) / closes[:-1]
                if len(ret) >= 21:
                    vol_5 = np.std(ret[-5:]) * 100
                    vol_20 = np.std(ret[-20:]) * 100
                    vol_ratio = vol_5 / vol_20 if vol_20 else 1
                    components['volatility_ratio'] = vol_ratio
                    volatility_score = 100 - (vol_ratio - 0.5) * 100
                    components['volatility_score'] = max(0, min(100, volatility_score))
            volume_url = "https://www.twse.com.tw/rwd/zh/afterTrading/FMTQIK?date=20230101&response=json"
            try:
                rv = requests.get(volume_url, headers=headers, timeout=12).json()
                vol_list = []
                if 'data' in rv:
                    for row in rv['data'][-8:]:
                        try:
                            val = row[1].replace(',', '')
                            vol_list.append(float(val))
                        except:
                            pass
                if len(vol_list) >= 5:
                    recent = np.mean(vol_list[-5:])
                    prev = np.mean(vol_list[-10:-5]) if len(vol_list) >= 10 else recent
                    volume_change = (recent / prev - 1) * 100 if prev else 0
                    components['volume_change_pct'] = volume_change
                    components['volume_score'] = max(0, min(100, 50 + volume_change))
            except Exception as e:
                logger.debug(f"成交量抓取失敗: {e}")
            try:
                pc_url = "https://www.taifex.com.tw/cht/3/pcRatio"
                pr = requests.get(pc_url, headers=headers, timeout=12)
                soup = BeautifulSoup(pr.text, 'html.parser')
                tables = soup.find_all('table')
                pc_ratio = None
                for tbl in tables:
                    tds = tbl.find_all('td')
                    for td in tds:
                        txt = td.get_text(strip=True)
                        if re.match(r'^\d+(\.\d+)?$', txt):
                            val = float(txt)
                            if 0.3 < val < 5:
                                pc_ratio = val
                                break
                    if pc_ratio:
                        break
                if pc_ratio:
                    components['pc_ratio'] = pc_ratio
                    pc_score = 50 + (pc_ratio - 1) * 40
                    components['pc_ratio_score'] = max(0, min(100, pc_score))
            except Exception as e:
                logger.debug(f"P/C 抓取失敗: {e}")
            final_value = (
                components['price_score'] * 0.5 +
                components['volatility_score'] * 0.2 +
                components['volume_score'] * 0.15 +
                components['pc_ratio_score'] * 0.15
            )
            final_value = max(0, min(100, final_value))
            mood = "極度恐慌" if final_value < 20 else \
                   "恐慌" if final_value < 40 else \
                   "中性" if final_value < 60 else \
                   "貪婪" if final_value < 80 else "極度貪婪"
            components['market_mood'] = mood
            self.fear_greed_cache = {
                'timestamp': now_ts,
                'value': final_value,
                'components': components
            }
            return final_value, components
        except Exception as e:
            logger.error(f"Fear & Greed 計算失敗: {e}")
            return self.fear_greed_cache['value'], self.fear_greed_cache['components']

    def calculate_sentiment_score(self, stock_id: str, stock_name: str) -> Dict[str, Any]:
        news_sentiment, news_count = self.get_stock_news_sentiment(stock_id, stock_name)
        fg_value, fg_components = self.get_fear_greed_index()
        if fg_value < 20 or fg_value > 80:
            adj_fg_w = self.config['fear_greed_weight'] * 1.3
        else:
            adj_fg_w = self.config['fear_greed_weight']
        adj_news_w = self.config['news_weight'] * min(1.0, news_count / 5)
        total_w = adj_news_w + adj_fg_w
        if total_w == 0:
            total_w = 1
        adj_news_w /= total_w
        adj_fg_w /= total_w
        sentiment_score = news_sentiment * adj_news_w + fg_value * adj_fg_w
        return {
            'sentiment_score': sentiment_score,
            'news_sentiment': news_sentiment,
            'news_count': news_count,
            'fear_greed_index': fg_value,
            'market_mood': fg_components.get('market_mood', ''),
            'fear_greed_components': fg_components,
            'weight_news': adj_news_w,
            'weight_fear_greed': adj_fg_w
        }

# ===============================
# 技術分析主類別
# ===============================
class StockAnalyzer:
    def __init__(self):
        self.config = {
            'rsi_period': 14,
            'macd_fast': 12,
            'macd_slow': 26,
            'macd_signal': 9,
            'bb_period': 20,
            'bb_std': 2,
            'kd_period': 14,
            'volume_ma_period': 20,
            'min_volume_lots': 1000,
            'sentiment_weights': {
                'trend': 0.25,
                'volume': 0.10,
                'technical': 0.40,
                'sentiment': 0.25
            }
        }
        self.stock_list_path = "taiwan_stocks_cache.csv"
        self.taiwan_stocks: Optional[pd.DataFrame] = None
        self.sentiment_analyzer = MarketSentimentAnalyzer()

    # 股票清單
    def get_taiwan_stocks(self, force_update=True):
        if not force_update and os.path.exists(self.stock_list_path):
            if time.time() - os.path.getmtime(self.stock_list_path) < 86400:
                try:
                    return pd.read_csv(self.stock_list_path, dtype={'stock_id': str})
                except:
                    pass
        urls = {
            "上市": "https://isin.twse.com.tw/isin/C_public.jsp?strMode=2",
            "上櫃": "https://isin.twse.com.tw/isin/C_public.jsp?strMode=4"
        }
        all_df = []
        headers = {'User-Agent': 'Mozilla/5.0'}
        for market_name, url in urls.items():
            try:
                res = requests.get(url, headers=headers, timeout=30)
                res.encoding = 'big5'
                html_dfs = pd.read_html(StringIO(res.text))
                df = html_dfs[0].copy()
                df.columns = df.iloc[0]
                df = df.iloc[1:].copy()
                df['market'] = market_name
                all_df.append(df)
            except Exception as e:
                logger.error(f"獲取 {market_name} 清單失敗: {e}")
        if not all_df:
            return self._backup_list()
        try:
            df = pd.concat(all_df, ignore_index=True)
            df[['stock_id', 'stock_name']] = df['有價證券代號及名稱'].str.split(r'\s+', n=1, expand=True)
            df = df[df['stock_id'].str.match(r'^\d{4}$', na=False)]
            exclude = ['ETF', 'ETN', 'TDR', '受益', '指數', '購', '牛', '熊', '存託憑證']
            df = df[~df['stock_name'].str.contains('|'.join(exclude), na=False)]
            df['yahoo_symbol'] = df.apply(
                lambda r: f"{r['stock_id']}.TW" if r['market'] == '上市' else f"{r['stock_id']}.TWO",
                axis=1
            )
            final = df[['stock_id', 'stock_name', 'market', '產業別', 'yahoo_symbol']].rename(columns={'產業別': 'industry'})
            final = final.drop_duplicates(subset=['stock_id'])
            final.to_csv(self.stock_list_path, index=False)
            return final
        except Exception as e:
            logger.error(f"處理股票清單錯誤: {e}")
            return self._backup_list()

    def _backup_list(self):
        data = [
            {'stock_id': '2330', 'stock_name': '台積電', 'market': '上市', 'industry': '半導體', 'yahoo_symbol': '2330.TW'},
            {'stock_id': '2317', 'stock_name': '鴻海', 'market': '上市', 'industry': '電子', 'yahoo_symbol': '2317.TW'},
            {'stock_id': '2454', 'stock_name': '聯發科', 'market': '上市', 'industry': '半導體', 'yahoo_symbol': '2454.TW'},
            {'stock_id': '2881', 'stock_name': '富邦金', 'market': '上市', 'industry': '金融', 'yahoo_symbol': '2881.TW'},
            {'stock_id': '2882', 'stock_name': '國泰金', 'market': '上市', 'industry': '金融', 'yahoo_symbol': '2882.TW'},
        ]
        return pd.DataFrame(data)

    # 抓資料
    def fetch_yfinance_data(self, symbol: str, period="1y", interval="1d", retries=3):
        for attempt in range(retries):
            try:
                df = yf.download(symbol, period=period, interval=interval, progress=False, auto_adjust=True)
                if df.empty:
                    return pd.DataFrame()
                if isinstance(df.columns, pd.MultiIndex):
                    df.columns = df.columns.get_level_values(0)
                df = df[['Open', 'High', 'Low', 'Close', 'Volume']].dropna()
                df.reset_index(inplace=True)
                return df
            except Exception as e:
                if attempt < retries - 1:
                    time.sleep(0.5 * (2 ** attempt))
                else:
                    logger.error(f"{symbol} 數據抓取失敗: {e}")
        return pd.DataFrame()

    # 簡版指標 v1
    def calculate_technical_indicators_v1(self, df: pd.DataFrame) -> pd.DataFrame:
        if df.empty:
            return df
        df = df.copy()
        delta = df['Close'].diff()
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        avg_gain = gain.rolling(self.config['rsi_period']).mean()
        avg_loss = loss.rolling(self.config['rsi_period']).mean()
        rs = avg_gain / avg_loss
        df['RSI'] = 100 - (100 / (1 + rs))
        ema_fast = df['Close'].ewm(span=self.config['macd_fast']).mean()
        ema_slow = df['Close'].ewm(span=self.config['macd_slow']).mean()
        macd_line = ema_fast - ema_slow
        macd_signal = macd_line.ewm(span=self.config['macd_signal']).mean()
        df['MACD'] = macd_line
        df['MACD_Signal'] = macd_signal
        df['MACD_Histogram'] = macd_line - macd_signal
        mid = df['Close'].rolling(self.config['bb_period']).mean()
        std = df['Close'].rolling(self.config['bb_period']).std()
        df['BB_Upper'] = mid + self.config['bb_std'] * std
        df['BB_Middle'] = mid
        df['BB_Lower'] = mid - self.config['bb_std'] * std
        ll = df['Low'].rolling(self.config['kd_period']).min()
        hh = df['High'].rolling(self.config['kd_period']).max()
        k = (df['Close'] - ll) / (hh - ll + 1e-9) * 100
        k = k.rolling(3).mean()
        d = k.rolling(3).mean()
        df['K_Percent'] = k
        df['D_Percent'] = d
        df['MA5'] = df['Close'].rolling(5).mean()
        df['MA10'] = df['Close'].rolling(10).mean()
        df['MA20'] = df['Close'].rolling(20).mean()
        df['Volume_MA'] = df['Volume'].rolling(self.config['volume_ma_period']).mean()
        return df

    def check_volume_filter(self, df: pd.DataFrame) -> bool:
        if df.empty:
            return False
        avg20 = df['Volume'].tail(20).mean()
        lots = avg20 / 1000
        return lots >= self.config['min_volume_lots']

    def analyze_trend(self, df: pd.DataFrame) -> Dict[str, Any]:
        if df.empty or len(df) < 20:
            return {'trend': '盤整', 'strength': 0, 'price_change_5d': 0, 'price_change_20d': 0}
        cp = df['Close'].iloc[-1]
        p5 = df['Close'].iloc[-6] if len(df) >= 6 else cp
        p20 = df['Close'].iloc[-21] if len(df) >= 21 else cp
        ch5 = (cp / p5 - 1) * 100 if p5 else 0
        ch20 = (cp / p20 - 1) * 100 if p20 else 0
        ma5 = df['MA5'].iloc[-1]
        ma20 = df['MA20'].iloc[-1]
        trend = '盤整'
        strength = 0
        if ch5 > 3 and ch20 > 5 and cp > ma5 > ma20:
            trend = '強勢上漲'; strength = 3
        elif ch5 > 1 and ch20 > 2 and cp > ma5:
            trend = '上漲'; strength = 2
        elif ch5 < -3 and ch20 < -5 and cp < ma5 < ma20:
            trend = '強勢下跌'; strength = -3
        elif ch5 < -1 and ch20 < -2 and cp < ma5:
            trend = '下跌'; strength = -2
        return {
            'trend': trend,
            'strength': strength,
            'price_change_5d': ch5,
            'price_change_20d': ch20
        }

    def analyze_volume(self, df: pd.DataFrame) -> Dict[str, Any]:
        if df.empty or len(df) < 20:
            return {'volume_trend': '正常', 'volume_ratio': 1.0, 'volume_signal': '中性', 'avg_volume_lots': 0}
        curr = df['Volume'].iloc[-1]
        avg20 = df['Volume'].rolling(20).mean().iloc[-1]
        ratio = curr / avg20 if avg20 else 1
        lots = avg20 / 1000
        if ratio > 2: vt, vs = '爆量', '強烈'
        elif ratio > 1.5: vt, vs = '放量', '積極'
        elif ratio < 0.5: vt, vs = '縮量', '消極'
        else: vt, vs = '正常', '中性'
        return {
            'volume_trend': vt, 'volume_ratio': ratio, 'volume_signal': vs, 'avg_volume_lots': lots
        }

    def analyze_technical_indicators(self, df: pd.DataFrame) -> Dict[str, Any]:
        if df.empty:
            return {
                'rsi_signal': '中性', 'rsi_value': 50,
                'macd_signal': '中性', 'macd_value': 0,
                'kd_signal': '中性', 'k_value': 50, 'd_value': 50,
                'bb_signal': '中性', 'bb_position': 0.5,
                'score': 50
            }
        score = 50
        out = {}
        rsi_v = df['RSI'].iloc[-1] if 'RSI' in df.columns else 50
        if math.isnan(rsi_v): rsi_v = 50
        out['rsi_value'] = rsi_v
        if rsi_v > 80: out['rsi_signal'] = '超買'; score -= 15
        elif rsi_v > 70: out['rsi_signal'] = '偏高'; score -= 5
        elif rsi_v < 20: out['rsi_signal'] = '超賣'; score += 15
        elif rsi_v < 30: out['rsi_signal'] = '偏低'; score += 5
        else: out['rsi_signal'] = '中性'
        macd_c = df['MACD'].iloc[-1] if 'MACD' in df.columns else 0
        macd_s = df['MACD_Signal'].iloc[-1] if 'MACD_Signal' in df.columns else 0
        macd_p = df['MACD'].iloc[-2] if 'MACD' in df.columns and len(df) >= 2 else macd_c
        macd_sp = df['MACD_Signal'].iloc[-2] if 'MACD_Signal' in df.columns and len(df) >= 2 else macd_s
        out['macd_value'] = macd_c
        if macd_p <= macd_sp and macd_c > macd_s:
            out['macd_signal'] = '黃金交叉'; score += 20
        elif macd_p >= macd_sp and macd_c < macd_s:
            out['macd_signal'] = '死亡交叉'; score -= 20
        elif macd_c > macd_s:
            out['macd_signal'] = '多頭'; score += 5
        elif macd_c < macd_s:
            out['macd_signal'] = '空頭'; score -= 5
        else:
            out['macd_signal'] = '中性'
        k_col = 'K_Percent' if 'K_Percent' in df.columns else ('K' if 'K' in df.columns else None)
        d_col = 'D_Percent' if 'D_Percent' in df.columns else ('D' if 'D' in df.columns else None)
        k_v = df[k_col].iloc[-1] if k_col else 50
        d_v = df[d_col].iloc[-1] if d_col else 50
        if math.isnan(k_v): k_v = 50
        if math.isnan(d_v): d_v = 50
        out['k_value'] = k_v; out['d_value'] = d_v
        if k_v > 80 and d_v > 80: out['kd_signal'] = '超買'; score -= 10
        elif k_v < 20 and d_v < 20: out['kd_signal'] = '超賣'; score += 10
        elif k_v > d_v: out['kd_signal'] = '偏多'; score += 3
        elif k_v < d_v: out['kd_signal'] = '偏空'; score -= 3
        else: out['kd_signal'] = '中性'
        if 'BB_Upper' in df.columns and 'BB_Lower' in df.columns and 'Close' in df.columns:
            bb_u = df['BB_Upper'].iloc[-1]; bb_l = df['BB_Lower'].iloc[-1]; cp = df['Close'].iloc[-1]
            if not (math.isnan(bb_u) or math.isnan(bb_l)) and bb_u != bb_l:
                pos = (cp - bb_l) / (bb_u - bb_l)
                out['bb_position'] = pos
                if pos > 0.8: out['bb_signal'] = '接近上軌'; score -= 5
                elif pos < 0.2: out['bb_signal'] = '接近下軌'; score += 5
                else: out['bb_signal'] = '中性'
            else:
                out['bb_signal'] = '中性'; out['bb_position'] = 0.5
        else:
            out['bb_signal'] = '中性'; out['bb_position'] = 0.5
        out['score'] = max(0, min(100, score))
        return out

    def calculate_combined_score(self,
                                 trend_analysis: Dict[str, Any],
                                 volume_analysis: Dict[str, Any],
                                 technical_analysis: Dict[str, Any],
                                 sentiment_analysis: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        w = self.config['sentiment_weights']
        base = 50
        trend_strength = trend_analysis.get('strength', 0)
        trend_component = max(-30, min(30, trend_strength * 10))
        vr = volume_analysis.get('volume_ratio', 1.0)
        if vr > 1.8: vol_comp = 12
        elif vr > 1.5: vol_comp = 8
        elif vr > 1.2: vol_comp = 5
        elif vr < 0.6: vol_comp = -8
        elif vr < 0.8: vol_comp = -4
        else: vol_comp = 0
        tech_score = max(-50, min(50, technical_analysis.get('score', 50) - 50))
        sent_score = (sentiment_analysis.get('sentiment_score', 50) - 50) if sentiment_analysis else 0
        composite = (base +
                     trend_component * w['trend'] +
                     vol_comp * w['volume'] +
                     tech_score * w['technical'] +
                     sent_score * w['sentiment'])
        composite = max(0, min(100, composite))
        return {
            'combined_score': composite,
            'components': {
                'trend_component': trend_component,
                'volume_component': vol_comp,
                'technical_component': tech_score,
                'sentiment_component': sent_score
            },
            'weights': w
        }

    def generate_recommendation(self,
                                combined_score: float,
                                trend_analysis: Dict[str, Any],
                                technical_analysis: Dict[str, Any],
                                sentiment_analysis: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        if combined_score >= 80: rec, conf = "強力買進", "高"
        elif combined_score >= 65: rec, conf = "買進", "中高"
        elif combined_score >= 45: rec, conf = "持有", "中等"
        elif combined_score >= 30: rec, conf = "賣出", "中"
        else: rec, conf = "強力賣出", "高"
        reasons = []
        trend = trend_analysis.get('trend', '盤整')
        if '上漲' in trend or '下跌' in trend:
            reasons.append(f"趨勢：{trend}")
        rsi_sig = technical_analysis.get('rsi_signal', '中性')
        if rsi_sig in ['超賣', '偏低', '超買', '偏高']:
            reasons.append(f"RSI {rsi_sig}")
        macd_sig = technical_analysis.get('macd_signal', '中性')
        if macd_sig in ['黃金交叉', '死亡交叉']:
            reasons.append(f"MACD {macd_sig}")
        if sentiment_analysis:
            mood = sentiment_analysis.get('market_mood', '')
            if mood:
                reasons.append(f"市場情緒：{mood}")
        return {
            'recommendation': rec,
            'confidence': conf,
            'score': combined_score,
            'reasons': reasons[:5]
        }

    async def analyze_stock_async(self, session: aiohttp.ClientSession, stock_info: Dict[str, Any]) -> Dict[str, Any]:
        symbol = stock_info['yahoo_symbol']
        stock_name = stock_info['stock_name']
        stock_id = stock_info['stock_id']
        try:
            df = self.fetch_yfinance_data(symbol)
            if df.empty or len(df) < 60:
                return {'symbol': symbol, 'stock_name': stock_name, 'stock_id': stock_id, 'success': False, 'error': '數據不足'}
            if not self.check_volume_filter(df):
                return {'symbol': symbol, 'stock_name': stock_name, 'stock_id': stock_id, 'success': False, 'error': '成交量不符合條件'}
            df_ind = self.calculate_technical_indicators_v1(df)
            trend_analysis = self.analyze_trend(df_ind)
            volume_analysis = self.analyze_volume(df_ind)
            technical_analysis = self.analyze_technical_indicators(df_ind)
            sentiment_analysis = self.sentiment_analyzer.calculate_sentiment_score(stock_id, stock_name)
            combined = self.calculate_combined_score(trend_analysis, volume_analysis, technical_analysis, sentiment_analysis)
            recommendation = self.generate_recommendation(combined['combined_score'], trend_analysis, technical_analysis, sentiment_analysis)
            return {
                'symbol': symbol,
                'stock_name': stock_name,
                'stock_id': stock_id,
                'market': stock_info.get('market', ''),
                'industry': stock_info.get('industry', ''),
                'success': True,
                'current_price': float(df_ind['Close'].iloc[-1]),
                'price_change_5d': trend_analysis.get('price_change_5d', 0),
                'combined_score': combined['combined_score'],
                'combined_components': combined.get('components', {}),
                'trend_analysis': trend_analysis,
                'volume_analysis': volume_analysis,
                'technical_analysis': technical_analysis,
                'sentiment_analysis': sentiment_analysis,
                'recommendation': recommendation,
                'analysis_time': datetime.now(taipei_tz).isoformat()
            }
        except Exception as e:
            logger.error(f"{symbol} 分析錯誤: {e}")
            return {'symbol': symbol, 'stock_name': stock_name, 'stock_id': stock_id, 'success': False, 'error': str(e)}

    async def analyze_all_stocks(self, limit: Optional[int] = None) -> Dict[str, Any]:
        if self.taiwan_stocks is None:
            self.taiwan_stocks = self.get_taiwan_stocks()
        if self.taiwan_stocks.empty:
            return {}
        df_list = self.taiwan_stocks if not limit else self.taiwan_stocks.head(limit)
        tasks = []
        results = {}
        async with aiohttp.ClientSession() as session:
            for _, row in df_list.iterrows():
                tasks.append(self.analyze_stock_async(session, row.to_dict()))
            batch_size = 12
            for i in range(0, len(tasks), batch_size):
                batch = tasks[i:i+batch_size]
                batch_results = await asyncio.gather(*batch, return_exceptions=True)
                for r in batch_results:
                    if isinstance(r, dict) and 'symbol' in r:
                        results[r['symbol']] = r
                await asyncio.sleep(1)
        return results

# ===============================
# 格式化輸出
# ===============================
def format_analysis_message(results: Dict[str, Any], limit: int = 10) -> str:
    ok = [r for r in results.values() if r.get('success')]
    ok.sort(key=lambda x: x.get('combined_score', 0), reverse=True)
    lines = []
    now_str = datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')
    lines.append("🏆 台股綜合技術 + 情緒 分析")
    lines.append(f"🕒 分析時間: {now_str}")
    lines.append(f"✅ 成功分析: {len(ok)} 支")
    lines.append("=" * 50)
    if not ok:
        lines.append("❌ 無可用結果")
        return "\n".join(lines)
    top = ok[:limit]
    for idx, r in enumerate(top, 1):
        sa = r.get('sentiment_analysis', {})
        news_cnt = sa.get('news_count', 0)
        fear_greed = sa.get('fear_greed_index', 50)
        mood = sa.get('market_mood', '')
        fg_emoji = "😨" if fear_greed < 30 else "😐" if fear_greed < 70 else "🤑"
        rec = r.get('recommendation', {}).get('recommendation', '')
        lines.append(f"{idx}. {r.get('stock_name')} ({r.get('stock_id')})")
        lines.append(f"   💰 價格: {r.get('current_price', 0):.2f} | 5日: {r.get('price_change_5d', 0):+.2f}%")
        lines.append(f"   ⭐ 綜合評分: {r.get('combined_score', 0):.1f} | 建議: {rec}")
        lines.append(f"   📰 新聞熱度: {news_cnt} | 新聞情緒: {sa.get('news_sentiment',50):.1f}")
        lines.append(f"   {fg_emoji} 市場情緒(F&G): {fear_greed:.1f} ({mood})")
        tech = r.get('technical_analysis', {})
        lines.append(f"   🔍 RSI {tech.get('rsi_value',50):.1f}({tech.get('rsi_signal','中性')}), MACD:{tech.get('macd_signal','中性')}, KD:{tech.get('k_value',50):.1f}/{tech.get('d_value',50):.1f}")
        lines.append("-" * 50)
    avg_score = np.mean([x.get('combined_score', 0) for x in ok]) if ok else 0
    high_cnt = sum(1 for x in ok if x.get('combined_score', 0) >= 70)
    lines.append(f"📈 平均分數: {avg_score:.1f} | 高分(>=70): {high_cnt}")
    lines.append("⚠️ 風險聲明：僅供參考，投資有風險。")
    return "\n".join(lines)

# ===============================
# 通知（含 Retry / Backoff）
# ===============================
async def send_notification(session: aiohttp.ClientSession, message: str, files: List[str] = None):
    """發送通知到 Telegram 和 Discord"""
    # 如果沒有提供檔案，嘗試從預設目錄獲取
    if not files:
        chart_dir = "/content/results/charts/"
        if os.path.exists(chart_dir):
            # 獲取目錄中的所有 PNG 檔案
            files = [os.path.join(chart_dir, f) for f in os.listdir(chart_dir)
                    if f.endswith('.png') and os.path.isfile(os.path.join(chart_dir, f))]
            if files:
                logger.info(f"找到 {len(files)} 個圖表檔案在 {chart_dir}")
            else:
                logger.warning(f"在 {chart_dir} 中找不到任何 PNG 檔案")

    # Telegram
    try:
        # 遍歷所有 Telegram Chat ID
        for chat_id in TELEGRAM_CHAT_ID:
            # 發送文字訊息
            url_msg = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage"
            payload = {"chat_id": chat_id, "text": message, "parse_mode": "Markdown"}
            async with session.post(url_msg, json=payload, timeout=20) as response:
                if response.status == 200:
                    logger.info(f"Telegram 摘要已成功發送到 chat_id: {chat_id}。")
                else:
                    response_text = await response.text()
                    logger.error(f"Telegram 摘要發送失敗: {response.status} - {response_text}")

            # 如果有檔案，也遍歷發送
            if files:
                url_photo = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendPhoto"
                sent_count = 0
                for file_path in files:
                    if os.path.exists(file_path):
                        try:
                            data = aiohttp.FormData()
                            data.add_field('chat_id', str(chat_id))  # 確保 chat_id 是字串
                            # 添加可選的圖片說明
                            caption = os.path.basename(file_path).replace('_report.png', '').replace('_', ' ')
                            data.add_field('caption', caption)

                            with open(file_path, 'rb') as f:
                                data.add_field('photo', f, filename=os.path.basename(file_path))
                                async with session.post(url_photo, data=data, timeout=60) as response:
                                    if response.status == 200:
                                        sent_count += 1
                                    else:
                                        response_text = await response.text()
                                        logger.error(f"Telegram 圖檔發送失敗: {response.status} - {response_text}")

                            # 添加小延遲以避免 Telegram API 限制
                            await asyncio.sleep(0.5)
                        except Exception as file_error:
                            logger.error(f"發送檔案 {file_path} 時出錯: {file_error}")

                logger.info(f"Telegram 圖檔已成功發送到 chat_id: {chat_id} ({sent_count}/{len(files)}個)。")
    except Exception as e:
        logger.error(f"發送 Telegram 通知時異常: {e}")
        traceback.print_exc()

    # Discord
    try:
        # 將訊息分段發送，避免超過 Discord 的限制
        message_chunks = [message[i:i+1900] for i in range(0, len(message), 1900)]

        for chunk in message_chunks:
            webhook = DiscordWebhook(url=DISCORD_WEBHOOK_URL, content=f"```\n{chunk}\n```")
            response = webhook.execute()
            if response and response.ok:
                logger.info("Discord 文字通知發送成功。")
            else:
                status_code = response.status_code if response else "未知"
                content = response.content if response else "未知"
                logger.error(f"Discord 文字通知失敗: {status_code} {content}")

        # 分批發送檔案，每批最多 10 個檔案
        if files:
            batch_size = 10
            for i in range(0, len(files), batch_size):
                batch_files = files[i:i+batch_size]
                webhook = DiscordWebhook(url=DISCORD_WEBHOOK_URL, content=f"圖表批次 {i//batch_size + 1}/{(len(files)-1)//batch_size + 1}")

                for file_path in batch_files:
                    if os.path.exists(file_path):
                        try:
                            with open(file_path, 'rb') as f:
                                webhook.add_file(file=f.read(), filename=os.path.basename(file_path))
                        except Exception as file_error:
                            logger.error(f"添加檔案 {file_path} 到 Discord webhook 時出錯: {file_error}")

                response = webhook.execute()
                if response and response.ok:
                    logger.info(f"Discord 圖檔批次 {i//batch_size + 1} 發送成功。")
                else:
                    status_code = response.status_code if response else "未知"
                    content = response.content if response else "未知"
                    logger.error(f"Discord 圖檔批次 {i//batch_size + 1} 發送失敗: {status_code} {content}")
    except Exception as e:
        logger.error(f"發送 Discord 通知時異常: {e}")
        traceback.print_exc()

def format_notification_message(filtered_results: Dict) -> str:
    """格式化通知訊息"""
    try:
        current_time = datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')

        if not filtered_results:
            return f"""
🤖 台股技術分析報告
📅 分析時間: {current_time}

❌ 本次分析未找到符合條件的優質股票
💡 建議調整篩選條件或關注市場變化
"""

        # 獲取圖表檔案數量
        chart_dir = "/content/results/charts/"
        chart_count = 0
        if os.path.exists(chart_dir):
            chart_files = [f for f in os.listdir(chart_dir) if f.endswith('.png') and os.path.isfile(os.path.join(chart_dir, f))]
            chart_count = len(chart_files)

        message = f"""
🤖 台股技術分析報告
📅 分析時間: {current_time}
🎯 找到 {len(filtered_results)} 支優質股票
📊 生成 {chart_count} 張技術分析圖表

📈 TOP {min(5, len(filtered_results))} 推薦股票:
"""

        for rank, (stock_id, result) in enumerate(list(filtered_results.items())[:5], 1):
            trend = result.get('trend_analysis', {})
            tech = result.get('technical_analysis', {})
            recommendation = result.get('recommendation', {})

            # 獲取更多技術指標數據
            macd = tech.get('macd_value', 0)
            signal = tech.get('signal_value', 0)
            macd_status = "多頭" if macd > signal else "空頭" if macd < signal else "中性"

            message += f"""
{rank}. {result.get('stock_name', '')} ({stock_id})
   💰 價格: {result.get('current_price', 0):.2f} TWD
   📈 評分: {result.get('combined_score', 0):.1f}/100
   🎯 建議: {recommendation.get('action', '觀望')}
   📊 趨勢: {trend.get('trend', '盤整')}
   🔍 RSI: {tech.get('rsi_value', 50):.1f} ({tech.get('rsi_status', '中性')})
   📉 MACD: {macd_status}
"""

        # 添加圖表資訊
        if chart_count > 0:
            message += f"""
📊 詳細分析圖表:
• 已生成 {chart_count} 張技術分析圖表
• 圖表包含K線、均線、成交量、MACD及RSI指標
• 圖表將在此訊息後發送
"""

        message += f"""
⚠️  投資提醒:
• 本分析僅供參考，投資有風險
• 建議結合基本面分析
• 請做好風險控制
"""

        return message.strip()

    except Exception as e:
        logger.error(f"格式化通知訊息時發生錯誤: {e}")
        traceback.print_exc()
        return f"台股分析完成，但訊息格式化失敗: {str(e)}"





def detect_divergence(df, rsi_threshold=30, rsi_overbought=70, window=20, min_divergence_points=2):
    """
    檢測 RSI 與價格之間的背離

    Args:
        df (pd.DataFrame): 包含價格和 RSI 的 DataFrame
        rsi_threshold (int): RSI 超賣閾值
        rsi_overbought (int): RSI 超買閾值
        window (int): 檢測窗口大小
        min_divergence_points (int): 最小背離點數量

    Returns:
        dict: 包含背離類型和分數的字典
    """
    try:
        # 確保必要欄位存在
        required_cols = ['Close', 'RSI']
        for col in required_cols:
            if col not in df.columns:
                # 嘗試查找小寫版本
                if col.lower() in df.columns:
                    df[col] = df[col.lower()]
                    logger.info(f"已將 {col.lower()} 欄位複製為 {col}")
                else:
                    logger.warning(f"detect_divergence: 缺少必要欄位 {col}")
                    return {
                        'type': '無',                        'bullish_regular': 0,
                        'bullish_hidden': 0,
                        'bearish_regular': 0,
                        'bearish_hidden': 0,
                        'details': f"缺少必要欄位: {col}"
                        }

        # 初始化返回結果
        result = {
        'type': '無',
        'bullish_regular': 0,
        'bullish_hidden': 0,
        'bearish_regular': 0,
        'bearish_hidden': 0,
        'details': None
        }

        if df is None or df.empty:
            return result
        if 'Close' not in df.columns or 'RSI' not in df.columns:
            return result
        lookback = 30
        if len(df) < lookback:
            return result
        recent = df.tail(lookback)
        price_highs, price_lows, rsi_highs, rsi_lows = [], [], [], []
        for i in range(1, len(recent)-1):
            if recent['Close'].iloc[i] > recent['Close'].iloc[i-1] and recent['Close'].iloc[i] > recent['Close'].iloc[i+1]:
                price_highs.append((i, recent['Close'].iloc[i]))
            if recent['Close'].iloc[i] < recent['Close'].iloc[i-1] and recent['Close'].iloc[i] < recent['Close'].iloc[i+1]:
                price_lows.append((i, recent['Close'].iloc[i]))
            if recent['RSI'].iloc[i] > recent['RSI'].iloc[i-1] and recent['RSI'].iloc[i] > recent['RSI'].iloc[i+1]:
                rsi_highs.append((i, recent['RSI'].iloc[i]))
            if recent['RSI'].iloc[i] < recent['RSI'].iloc[i-1] and recent['RSI'].iloc[i] < recent['RSI'].iloc[i+1]:
                rsi_lows.append((i, recent['RSI'].iloc[i]))
        if len(price_highs) >= 2 and len(rsi_highs) >= 2:
            lp = price_highs[-1]; pp = price_highs[-2]
            lr = rsi_highs[-1]; pr = rsi_highs[-2]
            if lp[1] > pp[1] and lr[1] < pr[1]:
                result['bearish_regular'] += 1
                result['type'] = '頂背離'
                result['details'] = {'price_high1': pp[1], 'price_high2': lp[1], 'rsi_high1': pr[1], 'rsi_high2': lr[1]}
            if lp[1] < pp[1] and lr[1] > pr[1]:
                result['bearish_hidden'] += 1
        if len(price_lows) >= 2 and len(rsi_lows) >= 2:
            lp = price_lows[-1]; pp = price_lows[-2]
            lr = rsi_lows[-1]; pr = rsi_lows[-2]
            if lp[1] < pp[1] and lr[1] > pr[1]:
                result['bullish_regular'] += 1
                result['type'] = '底背離'
                result['details'] = {'price_low1': pp[1], 'price_low2': lp[1], 'rsi_low1': pr[1], 'rsi_low2': lr[1]}
            if lp[1] > pp[1] and lr[1] < pr[1]:
                result['bullish_hidden'] += 1
        return result
    except Exception as e:
        logger.error(f"背離檢測錯誤: {e}")
        return result

def check_volume_condition(df, min_volume=1000, weeks=3):
    try:
        if 'Volume' not in df.columns or df.empty:
            return False
        if not isinstance(df.index, pd.DatetimeIndex):
            df.index = pd.to_datetime(df.index)
        if len(df) < weeks * 5:
            return False
        weekly_volume = df['Volume'].resample('W').mean() / 1000
        if len(weekly_volume) < weeks:
            return False
        is_high_volume = weekly_volume.tail(weeks) > min_volume
        return all(is_high_volume)
    except Exception as e:
        logger.error(f"檢查成交量時出錯: {e}")
        return False

# ===============================
# v2 技術指標（完整）
# ===============================
def calculate_technical_indicators_v2(df: pd.DataFrame, debug: bool = False):
    if df is None or df.empty:
        return None
    if debug:
        print("=== DEBUG BEFORE INDICATOR ===")
        print(df.columns.tolist())
        print(df.head(3))
        print(df.dtypes)
        for i, c in enumerate(df.columns):
            print(i, repr(c), "-> stripped:", repr(str(c).strip()))
    try:
        df = df.copy()
        if not isinstance(df.index, pd.DatetimeIndex):
            if 'Date' in df.columns:
                try:
                    df['Date'] = pd.to_datetime(df['Date'])
                    df.set_index('Date', inplace=True)
                except:
                    pass
        cols = []
        for c in df.columns:
            if isinstance(c, tuple):
                parts = [str(x) for x in c if x is not None and str(x) != ""]
                cols.append("_".join(parts))
            else:
                cols.append(str(c))
        df.columns = cols
        def normalize(col: str):
            col2 = str(col).strip().replace('\u3000', ' ')
            col2 = re.sub(r'\s+', '', col2)
            return col2.lower()
        norm_map = {}
        for original in df.columns:
            n = normalize(original)
            if n not in norm_map:
                norm_map[n] = original
        if debug:
            print("== Column Normalization Map ==")
            for k, v in norm_map.items():
                print(f"{k} -> {v}")
        def find_first_by_keywords(keywords):
            for kw in keywords:
                kw_n = normalize(kw)
                if kw_n in norm_map:
                    return norm_map[kw_n]
            for norm_key, orig in norm_map.items():
                if any(kw in norm_key for kw in keywords):
                    return orig
            return None
        col_close = find_first_by_keywords(['close', 'adjclose', 'closeprice', 'price'])
        col_high  = find_first_by_keywords(['high', 'max'])
        col_low   = find_first_by_keywords(['low', 'min'])
        col_open  = find_first_by_keywords(['open'])
        col_vol   = find_first_by_keywords(['volume', 'vol'])
        missing = [n for n, v in [('Close', col_close), ('High', col_high), ('Low', col_low)] if v is None]
        if missing:
            if debug:
                print("原始欄位列表:", df.columns.tolist())
            return None
        rename_map = {}
        if col_close and col_close != 'Close': rename_map[col_close] = 'Close'
        if col_high  and col_high  != 'High':  rename_map[col_high]  = 'High'
        if col_low   and col_low   != 'Low':   rename_map[col_low]   = 'Low'
        if col_open  and col_open  != 'Open':  rename_map[col_open]  = 'Open'
        if col_vol   and col_vol   != 'Volume':rename_map[col_vol]   = 'Volume'
        if rename_map:
            df.rename(columns=rename_map, inplace=True)
        def series_only(frame, col):
            s = frame[col]
            if isinstance(s, pd.DataFrame):
                for sub in s.columns:
                    if not s[sub].isna().all():
                        return s[sub]
                return s.iloc[:, 0]
            return s
        close = series_only(df, 'Close')
        high  = series_only(df, 'High')
        low   = series_only(df, 'Low')
        volume = series_only(df, 'Volume') if 'Volume' in df.columns else pd.Series(0, index=df.index)
        core = pd.concat([close, high, low], axis=1)
        df = df.loc[~core.isna().all(axis=1)].copy()
        for win in [5, 10, 20, 60, 120]:
            df[f"MA{win}"] = close.rolling(win, min_periods=1).mean()
        ema12 = close.ewm(span=12, adjust=False).mean()
        ema26 = close.ewm(span=26, adjust=False).mean()
        macd_line = ema12 - ema26
        macd_signal = macd_line.ewm(span=9, adjust=False).mean()
        df['MACD'] = macd_line
        df['MACD_Signal'] = macd_signal
        df['MACD_Hist'] = macd_line - macd_signal
        delta = close.diff()
        gain = delta.clip(lower=0)
        loss = (-delta.clip(upper=0)).abs()
        avg_gain = gain.rolling(14, min_periods=14).mean()
        avg_loss = loss.rolling(14, min_periods=14).mean()
        rs = avg_gain / (avg_loss.replace(0, np.nan))
        df['RSI'] = 100 - (100 / (1 + rs))
        mid = close.rolling(20, min_periods=15).mean()
        std = close.rolling(20, min_periods=15).std()
        df['BB_Middle'] = mid
        df['BB_Upper'] = mid + 2 * std
        df['BB_Lower'] = mid - 2 * std
        n = 14
        ll = low.rolling(n, min_periods=5).min()
        hh = high.rolling(n, min_periods=5).max()
        rng = (hh - ll).replace(0, np.nan)
        k_raw = 100 * (close - ll) / (rng + 1e-9)
        df['%K_raw'] = k_raw
        df['K'] = k_raw.rolling(3, min_periods=1).mean()
        df['D'] = df['K'].rolling(3, min_periods=1).mean()
        pc = close.diff()
        obv_step = np.where(pc > 0, volume,
                            np.where(pc < 0, -volume, 0))
        df['OBV'] = pd.Series(obv_step, index=df.index).cumsum()
        tr1 = (high - low).abs()
        tr2 = (high - close.shift()).abs()
        tr3 = (low - close.shift()).abs()
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        df['ATR'] = tr.rolling(14, min_periods=5).mean()
        plus_dm_raw = (high - high.shift())
        minus_dm_raw = (low.shift() - low)
        plus_dm = plus_dm_raw.where((plus_dm_raw > minus_dm_raw) & (plus_dm_raw > 0), 0.0)
        minus_dm = minus_dm_raw.where((minus_dm_raw > plus_dm_raw) & (minus_dm_raw > 0), 0.0)
        tr_14 = tr.rolling(14, min_periods=5).sum()
        plus_di = 100 * (plus_dm.rolling(14, min_periods=5).sum() / (tr_14 + 1e-9))
        minus_di = 100 * (minus_dm.rolling(14, min_periods=5).sum() / (tr_14 + 1e-9))
        dx = 100 * ((plus_di - minus_di).abs() / (plus_di + minus_di + 1e-9))
        df['Plus_DI'] = plus_di
        df['Minus_DI'] = minus_di
        df['ADX'] = dx.rolling(14, min_periods=5).mean()
        tp = (high + low + close) / 3
        tp_ma = tp.rolling(20, min_periods=15).mean()
        mean_dev = (tp - tp_ma).abs().rolling(20, min_periods=15).mean()
        df['CCI'] = (tp - tp_ma) / (0.015 * (mean_dev + 1e-9))
        df['ROC'] = (close / close.shift(10) - 1) * 100
        typical_price = tp
        money_flow = typical_price * volume
        pos_mf = money_flow.where(typical_price > typical_price.shift(), 0.0)
        neg_mf = money_flow.where(typical_price < typical_price.shift(), 0.0)
        pmf_sum = pos_mf.rolling(14, min_periods=5).sum()
        nmf_sum = neg_mf.rolling(14, min_periods=5).sum()
        mfr = pmf_sum / (nmf_sum + 1e-9)
        df['MFI'] = 100 - (100 / (1 + mfr))
        return df
    except Exception as e:
        logger.error(f"[calculate_technical_indicators_v2] 例外: {e}", exc_info=True)
        return None

# ===============================
# 背離分析流程
# ===============================
async def run_divergence_analysis(
        stocks: List[Dict[str, Any]],
        min_score=3,
        period='1y',
        volume_weeks=3,
        require_volume=1000,
        include_sentiment=True):
    results = []
    total = len(stocks)
    logger.info(f"背離分析開始: 共 {total} 支股票")
    sent_analyzer = MarketSentimentAnalyzer() if include_sentiment else None
    for idx, s in enumerate(stocks, 1):
        try:
            stock_id = s.get('stock_id')
            yahoo_symbol = s.get('yahoo_symbol', f"{stock_id}.TW")
            raw = yf.download(yahoo_symbol, period=period, interval="1d", auto_adjust=True, progress=False)
            if raw.empty or len(raw) < 200:
                continue
            raw.reset_index(inplace=True)
            if 'Date' not in raw.columns or 'Close' not in raw.columns:
                continue
            raw['Date'] = pd.to_datetime(raw['Date'])
            raw.set_index('Date', inplace=True)
            ind_df = calculate_technical_indicators_v2(raw)
            if ind_df is None or ind_df.empty:
                continue
            div = detect_divergence(ind_df)
            bull_score = div['bullish_regular'] + div['bullish_hidden']
            bear_score = div['bearish_regular'] + div['bearish_hidden']
            if bull_score == 0 and bear_score == 0:
                continue
            direction = None
            final_score = 0
            if bull_score > bear_score and bull_score >= min_score:
                direction = 'bullish'; final_score = bull_score
            elif bear_score > bull_score and bear_score >= min_score:
                direction = 'bearish'; final_score = bear_score
            else:
                continue
            if not check_volume_condition(ind_df[['Volume']].copy(), min_volume=require_volume, weeks=volume_weeks):
                continue
            sentiment_block = {}
            if include_sentiment and sent_analyzer:
                sentiment_block = sent_analyzer.calculate_sentiment_score(stock_id, s.get('stock_name', ''))
            results.append({
                'stock_id': stock_id,
                'stock_name': s.get('stock_name', ''),
                'type': div['type'],
                'score': final_score,
                'direction': direction,
                'close': ind_df['Close'].iloc[-1],
                'date': ind_df.index[-1].strftime('%Y-%m-%d'),
                'rsi': ind_df['RSI'].iloc[-1] if 'RSI' in ind_df.columns else None,
                'volume_condition': True,
                'sentiment': sentiment_block,
                'data': ind_df
            })
            logger.info(f"[{idx}/{total}] {stock_id} 背離符合: {direction} 分數={final_score}")
        except Exception as e:
            logger.error(f"背離分析錯誤 {s}: {e}")
    if results:
        results.sort(key=lambda x: x['score'], reverse=True)
    return results

# ===============================
# 背離圖表
# ===============================
def plot_divergence_chart(entry: Dict[str, Any], save_dir='analysis_charts'):
    try:
        df = entry.get('data')
        if df is None or df.empty:
            return None
        os.makedirs(save_dir, exist_ok=True)
        title = f"{entry['stock_id']} {entry['stock_name']} {entry['type']} 分數:{entry['score']}"
        ap = []
        if 'MA20' in df.columns:
            ap.append(mpf.make_addplot(df['MA20'], color='blue', width=1))
        if 'MA60' in df.columns:
            ap.append(mpf.make_addplot(df['MA60'], color='red', width=1))
        if 'RSI' in df.columns:
            ap.append(mpf.make_addplot(df['RSI'], panel=1, color='purple', ylabel='RSI'))
            ap.append(mpf.make_addplot([70]*len(df), panel=1, color='red', linestyle='--'))
            ap.append(mpf.make_addplot([30]*len(df), panel=1, color='green', linestyle='--'))
        path = os.path.join(save_dir, f"{entry['stock_id']}_{datetime.now().strftime('%Y%m%d')}_div.png")
        mpf.plot(df, type='candle', title=title, addplot=ap, volume=True, figsize=(14,8),
                 panel_ratios=(6,2), style='yahoo', savefig=path)
        return path
    except Exception as e:
        logger.error(f"繪製背離圖錯誤: {e}")
        return None

# ===============================
# 背離通知
# ===============================
async def notify_divergence_results(entries: List[Dict[str, Any]], limit=10):
    if not entries:
        await send_notification_text("📉 背離分析：沒有符合條件的股票")
        return
    text_lines = []
    text_lines.append("🔍 背離分析結果")
    text_lines.append(f"🕒 {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}")
    text_lines.append("="*40)
    top = entries[:limit]
    for i, e in enumerate(top, 1):
        sent = e.get('sentiment', {})
        fg = sent.get('fear_greed_index', 50)
        mood = sent.get('market_mood', '')
        text_lines.append(f"{i}. {e['stock_id']} {e['stock_name']} {e['type']} 分數:{e['score']} 價:{e['close']:.2f}")
        if sent:
            text_lines.append(f"   情緒:{sent.get('sentiment_score',50):.1f} F&G:{fg:.1f}({mood}) 新聞:{sent.get('news_count',0)}")
    await send_notification_text("\n".join(text_lines))
    for e in top:
        try:
            chart = plot_divergence_chart(e)
            if chart:
                logger.info(f"背離圖已生成: {chart}")
        except Exception as ex:
            logger.error(f"生成背離圖失敗: {ex}")

# ===============================
# 主流程
# ===============================
async def main_all():
    logger.info("===== 綜合分析啟動 (技術+情緒 + 背離) =====")
    analyzer = StockAnalyzer()
    results = await analyzer.analyze_all_stocks(limit=2000)
    msg = format_analysis_message(results, limit=10)
    print(msg)
    await send_notification_text(msg)
    if analyzer.taiwan_stocks is None or analyzer.taiwan_stocks.empty:
        logger.warning("無法取得股票清單進行背離分析")
        return
    stocks_list = analyzer.taiwan_stocks.to_dict('records')
    divergence_entries = await run_divergence_analysis(
        stocks=stocks_list,
        min_score=3,
        period='1y',
        volume_weeks=3,
        require_volume=1000,
        include_sentiment=True
    )
    await notify_divergence_results(divergence_entries, limit=10)
    logger.info("===== 綜合分析完成 =====")

# ===============================
# Debug 測試：指標 v2
# ===============================
def debug_indicator_test(symbol="2330.TW"):
    t = yf.download(symbol, period="3mo", interval="1d", auto_adjust=True, progress=False)
    if t.empty:
        print("下載測試資料失敗")
        return
    t.columns = [c + ' ' if c == 'Close' else c for c in t.columns]  # 模擬 Close 有空白
    ind = calculate_technical_indicators_v2(t, debug=True)
    if ind is None:
        print("指標計算失敗，請貼 debug 輸出。")
        return
    cols_show = [c for c in ['Close', 'K', 'D', 'RSI', 'MACD', 'MFI'] if c in ind.columns]
    print(ind.tail()[cols_show])

# ===============================
# 執行入口
# ===============================
if __name__ == "__main__":
    RUN_MAIN = True
    RUN_DEBUG_INDICATOR = False
    if RUN_MAIN:
        try:
            asyncio.run(main_all())
        except RuntimeError:
            loop = asyncio.get_event_loop()
            loop.run_until_complete(main_all())
    if RUN_DEBUG_INDICATOR:
        debug_indicator_test("2330.TW")

In [ ]:

import asyncio
import aiohttp
import pandas as pd
import numpy as np
import yfinance as yf
import logging
import traceback
from datetime import datetime, timezone, timedelta
from typing import Dict, List, Any, Optional
import json
import os
import time
import requests
import random
from io import StringIO

# 設定日誌
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# 台北時區
taipei_tz = timezone(timedelta(hours=8))

class StrongMomentumStockAnalyzer:
    def __init__(self):
        """初始化強勢股分析器"""
        self.config = {
            'rsi_period': 14,
            'volume_ma_period': 20,
            'min_volume_lots': 500,  # 最小成交量（張）
            'price_change_threshold': 5.0,  # 價格變化閾值(%)
            'volume_ratio_threshold': 1.5,  # 成交量比率閾值
            'trend_weights': {
                'price_trend': 0.5,
                'volume_trend': 0.3,
                'technical_score': 0.2
            }
        }
        self.taiwan_stocks = None
        self.stock_list_path = "taiwan_stocks_cache.csv"

    def get_taiwan_stocks(self, force_update=True):
        """獲取台灣股票清單"""
        if not force_update and os.path.exists(self.stock_list_path):
            if (time.time() - os.path.getmtime(self.stock_list_path)) < 86400:
                logger.info("從快取載入股票列表。")
                return pd.read_csv(self.stock_list_path, dtype={'stock_id': str})

        logger.info("從台灣證交所網站獲取最新股票清單...")
        urls = {
            "上市": "https://isin.twse.com.tw/isin/C_public.jsp?strMode=2",
            "上櫃": "https://isin.twse.com.tw/isin/C_public.jsp?strMode=4"
        }
        all_stocks_df = []
        headers = {'User-Agent': 'Mozilla/5.0'}

        for market_name, url in urls.items():
            try:
                res = requests.get(url, headers=headers, timeout=30)
                res.encoding = 'big5'
                html_dfs = pd.read_html(StringIO(res.text))
                df = html_dfs[0].copy()
                df.columns = df.iloc[0]
                df = df.iloc[1:].copy()
                df.loc[:, 'market'] = market_name
                all_stocks_df.append(df)
            except Exception as e:
                logger.error(f"獲取 {market_name} 股票列表失敗: {e}")
                continue

        if not all_stocks_df:
            logger.error("無法從網站獲取任何股票數據，使用備用清單。")
            return self._get_backup_stock_list()

        try:
            df = pd.concat(all_stocks_df, ignore_index=True)
            df[['stock_id', 'stock_name']] = df['有價證券代號及名稱'].str.split(r'\s+', n=1, expand=True)
            df = df[df['stock_id'].str.match(r'^\d{4}$', na=False)].copy()
            exclude = ['ETF', 'ETN', 'TDR', '受益', '指數', '購', '牛', '熊', '存託憑證']
            df = df[~df['stock_name'].str.contains('|'.join(exclude), na=False)].copy()

            df.loc[:, 'yahoo_symbol'] = df.apply(
                lambda row: f"{row['stock_id']}.TW" if '上市' in row['market'] else f"{row['stock_id']}.TWO", axis=1)

            final_df = df[['stock_id', 'stock_name', 'market', '產業別', 'yahoo_symbol']].rename(columns={'產業別': 'industry'})
            final_df = final_df.drop_duplicates(subset=['stock_id']).reset_index(drop=True)
            final_df.to_csv(self.stock_list_path, index=False)
            logger.info(f"成功獲取 {len(final_df)} 支股票清單並保存快取。")
            return final_df
        except Exception as e:
            logger.error(f"處理股票清單時發生錯誤: {e}")
            return self._get_backup_stock_list()

    def _get_backup_stock_list(self) -> pd.DataFrame:
        """備用股票清單"""
        try:
            logger.info("使用備用股票清單")

            # 從圖片中識別的股票加入備用列表
            image_stocks = [
                {'stock_id': '6134', 'stock_name': '萬旭', 'market': '上市', 'yahoo_symbol': '6134.TW', 'industry': '電子'},
                {'stock_id': '6133', 'stock_name': '金橋', 'market': '上市', 'yahoo_symbol': '6133.TW', 'industry': '電子'},
                {'stock_id': '3163', 'stock_name': '波若威', 'market': '上市', 'yahoo_symbol': '3163.TW', 'industry': '電子'},
                {'stock_id': '5475', 'stock_name': '德宏', 'market': '上市', 'yahoo_symbol': '5475.TW', 'industry': '電子'},
                {'stock_id': '3605', 'stock_name': '宏致', 'market': '上市', 'yahoo_symbol': '3605.TW', 'industry': '電子'},
                {'stock_id': '1815', 'stock_name': '富香', 'market': '上市', 'yahoo_symbol': '1815.TW', 'industry': '食品'},
                {'stock_id': '1802', 'stock_name': '台玻', 'market': '上市', 'yahoo_symbol': '1802.TW', 'industry': '玻璃'},
                {'stock_id': '3297', 'stock_name': '杭特', 'market': '上市', 'yahoo_symbol': '3297.TW', 'industry': '電子'},
                {'stock_id': '6895', 'stock_name': '宏碩系統', 'market': '上市', 'yahoo_symbol': '6895.TW', 'industry': '資訊服務'},
                {'stock_id': '4989', 'stock_name': '榮科', 'market': '上市', 'yahoo_symbol': '4989.TW', 'industry': '生技醫療'},
                {'stock_id': '3332', 'stock_name': '圭康', 'market': '上市', 'yahoo_symbol': '3332.TW', 'industry': '生技醫療'},
                {'stock_id': '6197', 'stock_name': '佳必琪', 'market': '上市', 'yahoo_symbol': '6197.TW', 'industry': '電子'},
            ]

            # 添加其他知名股票
            famous_stocks = [
                {'stock_id': '2330', 'stock_name': '台積電', 'market': '上市', 'yahoo_symbol': '2330.TW', 'industry': '半導體'},
                {'stock_id': '2317', 'stock_name': '鴻海', 'market': '上市', 'yahoo_symbol': '2317.TW', 'industry': '電子'},
                {'stock_id': '2454', 'stock_name': '聯發科', 'market': '上市', 'yahoo_symbol': '2454.TW', 'industry': '半導體'},
                {'stock_id': '2881', 'stock_name': '富邦金', 'market': '上市', 'yahoo_symbol': '2881.TW', 'industry': '金融'},
                {'stock_id': '2882', 'stock_name': '國泰金', 'market': '上市', 'yahoo_symbol': '2882.TW', 'industry': '金融'},
                {'stock_id': '2412', 'stock_name': '中華電', 'market': '上市', 'yahoo_symbol': '2412.TW', 'industry': '通信'},
                {'stock_id': '1301', 'stock_name': '台塑', 'market': '上市', 'yahoo_symbol': '1301.TW', 'industry': '塑膠'},
                {'stock_id': '2303', 'stock_name': '聯電', 'market': '上市', 'yahoo_symbol': '2303.TW', 'industry': '半導體'},
            ]

            # 合併兩個列表
            all_stocks = image_stocks + famous_stocks

            df = pd.DataFrame(all_stocks)
            logger.info(f"備用股票清單包含 {len(df)} 支股票")
            return df

        except Exception as e:
            logger.error(f"生成備用股票清單時發生錯誤: {e}")
            return pd.DataFrame()

    def fetch_yfinance_data(self, stock_id: str, period: str = "1mo", interval: str = "1d", retries: int = 3):
        """獲取股票數據"""
        for attempt in range(retries):
            try:
                df = yf.download(tickers=stock_id, period=period, interval=interval, progress=False, auto_adjust=True)
                if df.empty:
                    logger.warning(f"[{stock_id}] 無數據返回")
                    return pd.DataFrame()

                if isinstance(df.columns, pd.MultiIndex):
                    df.columns = df.columns.get_level_values(0)

                required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
                for col in required_cols:
                    if col in df.columns:
                        df[col] = pd.to_numeric(df[col], errors='coerce')

                df = df.dropna(subset=required_cols)
                return df.reset_index()

            except Exception as e:
                if attempt < retries - 1:
                    sleep_time = 0.5 * (2 ** attempt) + random.uniform(0, 1)
                    logger.warning(f"[{stock_id}] 第 {attempt + 1}/{retries} 次嘗試失敗: {e}。等待 {sleep_time:.2f} 秒後重試...")
                    time.sleep(sleep_time)
                else:
                    logger.error(f"[{stock_id}] 獲取數據失敗。")
                    return pd.DataFrame()
        return pd.DataFrame()

    def calculate_technical_indicators(self, df: pd.DataFrame) -> pd.DataFrame:
        """計算技術指標"""
        try:
            if df.empty:
                return df

            df = df.copy()

            # RSI
            df['RSI'] = self.calculate_rsi(df['Close'], self.config['rsi_period'])

            # 移動平均線
            df['MA5'] = df['Close'].rolling(window=5).mean()
            df['MA10'] = df['Close'].rolling(window=10).mean()
            df['MA20'] = df['Close'].rolling(window=20).mean()

            # 成交量移動平均
            if 'Volume' in df.columns:
                df['Volume_MA'] = df['Volume'].rolling(window=self.config['volume_ma_period']).mean()
                # 計算成交量比率 (當日成交量/20日平均)
                df['Volume_Ratio'] = df['Volume'] / df['Volume_MA']

            # 計算日漲幅
            df['Daily_Return'] = df['Close'].pct_change() * 100

            # 計算5日漲幅
            df['5D_Return'] = df['Close'].pct_change(5) * 100

            # 計算10日漲幅
            df['10D_Return'] = df['Close'].pct_change(10) * 100

            return df

        except Exception as e:
            logger.error(f"計算技術指標時發生錯誤: {e}")
            return df

    def calculate_rsi(self, prices: pd.Series, period: int = 14) -> pd.Series:
        """計算RSI"""
        try:
            delta = prices.diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
            rs = gain / loss
            rsi = 100 - (100 / (1 + rs))
            return rsi
        except Exception as e:
            logger.error(f"計算RSI時發生錯誤: {e}")
            return pd.Series(index=prices.index, data=50)

    def check_volume_surge(self, df: pd.DataFrame) -> Dict[str, Any]:
        """檢查成交量突增"""
        try:
            if df.empty or 'Volume' not in df.columns or 'Volume_Ratio' not in df.columns:
                return {'volume_surge': False, 'volume_ratio': 1.0}

            # 獲取最近一日的成交量比率
            latest_volume_ratio = df['Volume_Ratio'].iloc[-1]

            # 檢查是否有成交量突增
            volume_surge = latest_volume_ratio >= self.config['volume_ratio_threshold']

            return {
                'volume_surge': volume_surge,
                'volume_ratio': latest_volume_ratio
            }
        except Exception as e:
            logger.error(f"檢查成交量突增時發生錯誤: {e}")
            return {'volume_surge': False, 'volume_ratio': 1.0}

    def check_price_momentum(self, df: pd.DataFrame) -> Dict[str, Any]:
        """檢查價格動能"""
        try:
            if df.empty or 'Daily_Return' not in df.columns:
                return {'strong_momentum': False, 'daily_return': 0, '5d_return': 0}

            # 獲取最近一日的漲幅
            daily_return = df['Daily_Return'].iloc[-1]

            # 獲取5日漲幅
            five_day_return = df['5D_Return'].iloc[-1] if '5D_Return' in df.columns else 0

            # 檢查是否有強勢動能 (日漲幅超過閾值)
            strong_momentum = daily_return >= self.config['price_change_threshold']

            # 檢查是否接近漲停 (台灣股市漲停通常為10%)
            near_limit_up = daily_return >= 8.0

            return {
                'strong_momentum': strong_momentum,
                'near_limit_up': near_limit_up,
                'daily_return': daily_return,
                '5d_return': five_day_return
            }
        except Exception as e:
            logger.error(f"檢查價格動能時發生錯誤: {e}")
            return {'strong_momentum': False, 'daily_return': 0, '5d_return': 0}

    def check_ma_crossover(self, df: pd.DataFrame) -> Dict[str, Any]:
        """檢查均線交叉"""
        try:
            if df.empty or 'MA5' not in df.columns or 'MA20' not in df.columns or len(df) < 2:
                return {'ma_crossover': False, 'ma_support': False}

            # 檢查5日均線是否上穿20日均線
            prev_ma5 = df['MA5'].iloc[-2]
            prev_ma20 = df['MA20'].iloc[-2]
            curr_ma5 = df['MA5'].iloc[-1]
            curr_ma20 = df['MA20'].iloc[-1]

            ma_crossover = (prev_ma5 <= prev_ma20) and (curr_ma5 > curr_ma20)

            # 檢查價格是否站上均線 (價格 > 5日均線 > 20日均線)
            current_price = df['Close'].iloc[-1]
            ma_support = (current_price > curr_ma5) and (curr_ma5 > curr_ma20)

            return {
                'ma_crossover': ma_crossover,
                'ma_support': ma_support,
                'price_above_ma5': current_price > curr_ma5,
                'price_above_ma20': current_price > curr_ma20
            }
        except Exception as e:
            logger.error(f"檢查均線交叉時發生錯誤: {e}")
            return {'ma_crossover': False, 'ma_support': False}

    def calculate_strong_momentum_score(self, price_momentum: Dict, volume_surge: Dict, ma_analysis: Dict) -> float:
        """計算強勢動能分數"""
        try:
            base_score = 50

            # 價格動能分數 (權重: 50%)
            price_score = 0
            if price_momentum.get('near_limit_up', False):
                price_score = 30  # 接近漲停給予高分
            elif price_momentum.get('strong_momentum', False):
                price_score = 20  # 強勢動能給予中高分

            # 成交量分數 (權重: 30%)
            volume_score = 0
            volume_ratio = volume_surge.get('volume_ratio', 1.0)
            if volume_ratio > 2.5:
                volume_score = 20  # 爆量
            elif volume_ratio > 1.5:
                volume_score = 15  # 放量
            elif volume_ratio > 1.2:
                volume_score = 10  # 小幅放量

            # 均線分數 (權重: 20%)
            ma_score = 0
            if ma_analysis.get('ma_crossover', False):
                ma_score = 15  # 均線交叉
            elif ma_analysis.get('ma_support', False):
                ma_score = 10  # 均線支撐
            elif ma_analysis.get('price_above_ma5', False):
                ma_score = 5   # 價格站上5日均線

            # 計算加權總分
            weights = self.config['trend_weights']
            total_score = base_score + (price_score * weights['price_trend']) + (volume_score * weights['volume_trend']) + (ma_score * weights['technical_score'])

            return max(0, min(100, total_score))

        except Exception as e:
            logger.error(f"計算強勢動能分數時發生錯誤: {e}")
            return 50.0

    def generate_momentum_label(self, price_momentum: Dict, volume_surge: Dict, ma_analysis: Dict, score: float) -> str:
        """生成動能標籤"""
        try:
            # 根據各項指標生成標籤
            labels = []

            # 價格動能標籤
            if price_momentum.get('near_limit_up', False):
                labels.append("漲停股")
            elif price_momentum.get('daily_return', 0) >= 7.0:
                labels.append("強漲股")
            elif price_momentum.get('strong_momentum', False):
                labels.append("漲勢股")

            # 成交量標籤
            if volume_surge.get('volume_ratio', 1.0) > 2.5:
                labels.append("爆量")
            elif volume_surge.get('volume_ratio', 1.0) > 1.5:
                labels.append("放量")

            # 均線標籤
            if ma_analysis.get('ma_crossover', False):
                labels.append("均線交叉")
            elif ma_analysis.get('ma_support', False):
                labels.append("均線多頭")

            # 如果沒有特定標籤，根據分數給予一般標籤
            if not labels:
                if score >= 80:
                    labels.append("強勢股")
                elif score >= 65:
                    labels.append("熱門股")
                elif score >= 50:
                    labels.append("活躍股")

            # 返回最多兩個標籤
            return " + ".join(labels[:2]) if labels else "一般股"

        except Exception as e:
            logger.error(f"生成動能標籤時發生錯誤: {e}")
            return "一般股"

    async def analyze_stock_async(self, session: aiohttp.ClientSession, stock_info: Dict) -> Dict[str, Any]:
        """異步分析單支股票"""
        symbol = stock_info['yahoo_symbol']
        stock_name = stock_info['stock_name']
        stock_id = stock_info['stock_id']

        try:
            logger.info(f"開始分析股票: {stock_name} ({symbol})")

            # 獲取股票數據
            df = self.fetch_yfinance_data(symbol, period="1mo")
            if df is None or df.empty or len(df) < 5:
                return {
                    'symbol': symbol,
                    'stock_id': stock_id,
                    'stock_name': stock_name,
                    'success': False,
                    'error': '無法獲取足夠的股票數據'
                }

            # 計算技術指標
            df_with_indicators = self.calculate_technical_indicators(df)

            # 檢查成交量突增
            volume_surge = self.check_volume_surge(df_with_indicators)

            # 檢查價格動能
            price_momentum = self.check_price_momentum(df_with_indicators)

            # 檢查均線交叉
            ma_analysis = self.check_ma_crossover(df_with_indicators)

            # 計算強勢動能分數
            momentum_score = self.calculate_strong_momentum_score(price_momentum, volume_surge, ma_analysis)

            # 生成動能標籤
            momentum_label = self.generate_momentum_label(price_momentum, volume_surge, ma_analysis, momentum_score)

            # 獲取當前價格資訊
            current_price = float(df['Close'].iloc[-1])
            daily_return = price_momentum.get('daily_return', 0)
            five_day_return = price_momentum.get('5d_return', 0)

            # 獲取成交量資訊 (轉換為張數)
            current_volume = float(df['Volume'].iloc[-1]) if 'Volume' in df.columns else 0
            volume_lots = current_volume / 1000  # 轉換為張數

            result = {
                'symbol': symbol,
                'stock_id': stock_id,
                'stock_name': stock_name,
                'market': stock_info.get('market', ''),
                'industry': stock_info.get('industry', ''),
                'success': True,
                'current_price': current_price,
                'daily_return': daily_return,
                '5d_return': five_day_return,
                'volume_lots': volume_lots,
                'volume_ratio': volume_surge.get('volume_ratio', 1.0),
                'momentum_score': momentum_score,
                'momentum_label': momentum_label,
                'price_momentum': price_momentum,
                'volume_surge': volume_surge,
                'ma_analysis': ma_analysis,
                'analysis_time': datetime.now(taipei_tz).isoformat()
            }

            logger.info(f"完成分析: {stock_name} - 分數: {momentum_score:.1f} - 標籤: {momentum_label} - 日漲幅: {daily_return:.2f}%")
            return result

        except Exception as e:
            logger.error(f"分析股票 {symbol} 時發生錯誤: {e}")
            return {
                'symbol': symbol,
                'stock_id': stock_id,
                'stock_name': stock_name,
                'success': False,
                'error': str(e)
            }

    async def analyze_all_stocks(self) -> Dict[str, Any]:
        """分析所有股票"""
        try:
            # 獲取完整股票清單
            if self.taiwan_stocks is None:
                self.taiwan_stocks = self.get_taiwan_stocks()

            if self.taiwan_stocks.empty:
                logger.error("無法獲取股票清單")
                return {}

            logger.info(f"準備分析 {len(self.taiwan_stocks)} 支股票")

            results = {}
            failed_count = 0

            # 使用 aiohttp 進行異步分析
            async with aiohttp.ClientSession() as session:
                # 創建分析任務
                tasks = []
                for _, stock_info in self.taiwan_stocks.iterrows():
                    task = self.analyze_stock_async(session, stock_info.to_dict())
                    tasks.append(task)

                # 分批執行任務以避免過載
                batch_size = 10
                for i in range(0, len(tasks), batch_size):
                    batch_tasks = tasks[i:i+batch_size]
                    batch_results = await asyncio.gather(*batch_tasks, return_exceptions=True)

                    # 處理批次結果
                    for result in batch_results:
                        if isinstance(result, Exception):
                            logger.error(f"分析任務異常: {result}")
                            failed_count += 1
                            continue

                        if isinstance(result, dict) and 'symbol' in result:
                            if result.get('success', False):
                                results[result['symbol']] = result
                            else:
                                failed_count += 1

                    # 批次間暫停避免過載
                    if i + batch_size < len(tasks):
                        await asyncio.sleep(2)

            logger.info(f"分析完成統計：")
            logger.info(f"  - 成功分析: {len(results)} 支")
            logger.info(f"  - 失敗: {failed_count} 支")

            return results

        except Exception as e:
            logger.error(f"批量分析股票時發生錯誤: {e}")
            return {}

def format_momentum_stocks_message(results: Dict[str, Any], limit: int = 15) -> str:
    """格式化強勢股分析結果為訊息"""
    try:
        if not results:
            return "❌ 未能獲取任何分析結果"

        # 過濾成功的結果並按動能分數排序
        successful_results = [
            result for result in results.values()
            if result.get('success', False) and result.get('momentum_score', 0) > 0
        ]

        # 先按日漲幅排序
        successful_results.sort(key=lambda x: x.get('daily_return', 0), reverse=True)

        # 統計資訊
        total_analyzed = len(results)
        successful_count = len(successful_results)

        # 建立訊息
        message_parts = []
        message_parts.append("🚀 台股強勢股分析")
        message_parts.append("=" * 40)
        message_parts.append(f"📈 分析時間: {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}")
        message_parts.append(f"📊 成功分析: {successful_count} 支股票")
        message_parts.append("")

        if not successful_results:
            message_parts.append("❌ 沒有符合條件的強勢股")
            return "\n".join(message_parts)

        # 顯示漲幅前N名的強勢股
        top_stocks = successful_results[:limit]
        message_parts.append(f"🔥 今日強勢股 TOP {len(top_stocks)}:")
        message_parts.append("-" * 40)

        for i, result in enumerate(top_stocks, 1):
            stock_name = result.get('stock_name', 'Unknown')
            stock_id = result.get('stock_id', 'Unknown')
            daily_return = result.get('daily_return', 0)
            five_day_return = result.get('5d_return', 0)
            momentum_score = result.get('momentum_score', 0)
            momentum_label = result.get('momentum_label', '一般股')
            current_price = result.get('current_price', 0)
            volume_lots = result.get('volume_lots', 0)
            volume_ratio = result.get('volume_ratio', 1.0)

            # 漲幅符號
            change_symbol = "🔴" if daily_return > 0 else "🔵" if daily_return < 0 else "⚪"

            # 成交量符號
            volume_symbol = "💥" if volume_ratio > 2.5 else "📈" if volume_ratio > 1.5 else "📊"

            message_parts.append(f"{i}. {stock_name} ({stock_id}) - {momentum_label}")
            message_parts.append(f"   {change_symbol} 漲幅: {daily_return:+.2f}% | 5日: {five_day_return:+.2f}%")
            message_parts.append(f"   💰 價格: {current_price:.2f} | {volume_symbol} 成交量: {volume_lots:.0f}張 (x{volume_ratio:.1f})")
            message_parts.append(f"   ⭐ 動能評分: {momentum_score:.1f}/100")
            message_parts.append("")

        # 添加漲停股統計
        limit_up_stocks = [r for r in successful_results if r.get('daily_return', 0) >= 9.5]
        strong_up_stocks = [r for r in successful_results if 7.0 <= r.get('daily_return', 0) < 9.5]

        message_parts.append("📊 市場強勢股統計:")
        message_parts.append(f"   🔥 漲停股數: {len(limit_up_stocks)} 支")
        message_parts.append(f"   📈 強漲股數 (7-9.5%): {len(strong_up_stocks)} 支")

        # 添加產業分布
        industry_counts = {}
        for result in limit_up_stocks + strong_up_stocks:
            industry = result.get('industry', '其他')
            industry_counts[industry] = industry_counts.get(industry, 0) + 1

        if industry_counts:
            message_parts.append("   🏭 強勢產業分布:")
            for industry, count in sorted(industry_counts.items(), key=lambda x: x[1], reverse=True)[:5]:
                message_parts.append(f"      {industry}: {count} 支")

        message_parts.append("")
        message_parts.append("⚠️ 投資提醒:")
        message_parts.append("• 本分析僅供參考，投資有風險")
        message_parts.append("• 強勢股常有大幅波動，請謹慎交易")
        message_parts.append("• 建議結合基本面分析做最終決策")

        return "\n".join(message_parts)

    except Exception as e:
        logger.error(f"格式化訊息時發生錯誤: {e}")
        return f"❌ 格式化分析結果時發生錯誤: {str(e)}"

def display_terminal_results(results: Dict[str, Any], limit: int = 15):
    """在終端顯示分析結果"""
    try:
        if not results:
            print("❌ 未能獲取任何分析結果")
            return

        # 過濾成功的結果並按動能分數排序
        successful_results = [
            result for result in results.values()
            if result.get('success', False) and result.get('momentum_score', 0) > 0
        ]

        # 先按日漲幅排序
        successful_results.sort(key=lambda x: x.get('daily_return', 0), reverse=True)

        print("\n" + "=" * 80)
        print("🚀 台股強勢股分析結果".center(80))
        print("=" * 80)
        print(f"📈 分析時間: {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"📊 成功分析: {len(successful_results)} 支股票")
        print("=" * 80)

        if not successful_results:
            print("❌ 沒有符合條件的強勢股")
            return

        # 表格標題
        print(f"{'排名':<4}{'代碼':<8}{'股票名稱':<12}{'漲幅%':<8}{'5日%':<8}{'價格':<10}{'成交量(張)':<12}{'量比':<6}{'動能評分':<8}{'標籤':<12}")
        print("-" * 80)

        # 顯示前N名
        top_stocks = successful_results[:limit]
        for i, result in enumerate(top_stocks, 1):
            stock_name = result.get('stock_name', 'Unknown')[:10]  # 限制長度
            stock_id = result.get('stock_id', 'Unknown')
            daily_return = result.get('daily_return', 0)
            five_day_return = result.get('5d_return', 0)
            momentum_score = result.get('momentum_score', 0)
            momentum_label = result.get('momentum_label', '一般股')[:10]  # 限制長度
            current_price = result.get('current_price', 0)
            volume_lots = result.get('volume_lots', 0)
            volume_ratio = result.get('volume_ratio', 1.0)

            print(f"{i:<4}{stock_id:<8}{stock_name:<12}{daily_return:<+8.2f}{five_day_return:<+8.2f}{current_price:<10.2f}{volume_lots:<12.0f}{volume_ratio:<6.1f}{momentum_score:<8.1f}{momentum_label:<12}")

        print("=" * 80)

        # 統計資訊
        limit_up_stocks = [r for r in successful_results if r.get('daily_return', 0) >= 9.5]
        strong_up_stocks = [r for r in successful_results if 7.0 <= r.get('daily_return', 0) < 9.5]

        print(f"\n📊 市場強勢股統計:")
        print(f"   🔥 漲停股數: {len(limit_up_stocks)} 支")
        print(f"   📈 強漲股數 (7-9.5%): {len(strong_up_stocks)} 支")

        # 產業分布
        industry_counts = {}
        for result in limit_up_stocks + strong_up_stocks:
            industry = result.get('industry', '其他')
            industry_counts[industry] = industry_counts.get(industry, 0) + 1

        if industry_counts:
            print("   🏭 強勢產業分布:")
            for industry, count in sorted(industry_counts.items(), key=lambda x: x[1], reverse=True)[:5]:
                print(f"      {industry}: {count} 支")

        print("\n" + "=" * 80)
        print("⚠️  投資提醒:")
        print("• 本分析僅供參考，投資一定有風險")
        print("• 強勢股常有大幅波動，請謹慎交易")
        print("• 請務必結合基本面分析做最終投資決策")
        print("=" * 80)

    except Exception as e:
        logger.error(f"顯示終端結果時發生錯誤: {e}")
        print(f"❌ 顯示結果時發生錯誤: {str(e)}")


async def send_notification(session: aiohttp.ClientSession, message: str, files: List[str] = None):
    """發送通知到 Telegram 和 Discord"""
    # 如果沒有提供檔案，嘗試從預設目錄獲取
    if not files:
        chart_dir = "/content/results/charts/"
        if os.path.exists(chart_dir):
            # 獲取目錄中的所有 PNG 檔案
            files = [os.path.join(chart_dir, f) for f in os.listdir(chart_dir)
                    if f.endswith('.png') and os.path.isfile(os.path.join(chart_dir, f))]
            if files:
                logger.info(f"找到 {len(files)} 個圖表檔案在 {chart_dir}")
            else:
                logger.warning(f"在 {chart_dir} 中找不到任何 PNG 檔案")

    # Telegram
    try:
        # 遍歷所有 Telegram Chat ID
        for chat_id in TELEGRAM_CHAT_ID:
            # 發送文字訊息
            url_msg = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage"
            payload = {"chat_id": chat_id, "text": message, "parse_mode": "Markdown"}
            async with session.post(url_msg, json=payload, timeout=20) as response:
                if response.status == 200:
                    logger.info(f"Telegram 摘要已成功發送到 chat_id: {chat_id}。")
                else:
                    response_text = await response.text()
                    logger.error(f"Telegram 摘要發送失敗: {response.status} - {response_text}")

            # 如果有檔案，也遍歷發送
            if files:
                url_photo = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendPhoto"
                sent_count = 0
                for file_path in files:
                    if os.path.exists(file_path):
                        try:
                            data = aiohttp.FormData()
                            data.add_field('chat_id', str(chat_id))  # 確保 chat_id 是字串
                            # 添加可選的圖片說明
                            caption = os.path.basename(file_path).replace('_report.png', '').replace('_', ' ')
                            data.add_field('caption', caption)

                            with open(file_path, 'rb') as f:
                                data.add_field('photo', f, filename=os.path.basename(file_path))
                                async with session.post(url_photo, data=data, timeout=60) as response:
                                    if response.status == 200:
                                        sent_count += 1
                                    else:
                                        response_text = await response.text()
                                        logger.error(f"Telegram 圖檔發送失敗: {response.status} - {response_text}")

                            # 添加小延遲以避免 Telegram API 限制
                            await asyncio.sleep(0.5)
                        except Exception as file_error:
                            logger.error(f"發送檔案 {file_path} 時出錯: {file_error}")

                logger.info(f"Telegram 圖檔已成功發送到 chat_id: {chat_id} ({sent_count}/{len(files)}個)。")
    except Exception as e:
        logger.error(f"發送 Telegram 通知時異常: {e}")
        traceback.print_exc()

    # Discord
    try:
        # 將訊息分段發送，避免超過 Discord 的限制
        message_chunks = [message[i:i+1900] for i in range(0, len(message), 1900)]

        for chunk in message_chunks:
            webhook = DiscordWebhook(url=DISCORD_WEBHOOK_URL, content=f"```\n{chunk}\n```")
            response = webhook.execute()
            if response and response.ok:
                logger.info("Discord 文字通知發送成功。")
            else:
                status_code = response.status_code if response else "未知"
                content = response.content if response else "未知"
                logger.error(f"Discord 文字通知失敗: {status_code} {content}")

        # 分批發送檔案，每批最多 10 個檔案
        if files:
            batch_size = 10
            for i in range(0, len(files), batch_size):
                batch_files = files[i:i+batch_size]
                webhook = DiscordWebhook(url=DISCORD_WEBHOOK_URL, content=f"圖表批次 {i//batch_size + 1}/{(len(files)-1)//batch_size + 1}")

                for file_path in batch_files:
                    if os.path.exists(file_path):
                        try:
                            with open(file_path, 'rb') as f:
                                webhook.add_file(file=f.read(), filename=os.path.basename(file_path))
                        except Exception as file_error:
                            logger.error(f"添加檔案 {file_path} 到 Discord webhook 時出錯: {file_error}")

                response = webhook.execute()
                if response and response.ok:
                    logger.info(f"Discord 圖檔批次 {i//batch_size + 1} 發送成功。")
                else:
                    status_code = response.status_code if response else "未知"
                    content = response.content if response else "未知"
                    logger.error(f"Discord 圖檔批次 {i//batch_size + 1} 發送失敗: {status_code} {content}")
    except Exception as e:
        logger.error(f"發送 Discord 通知時異常: {e}")
        traceback.print_exc()

def format_notification_message(filtered_results: Dict) -> str:
    """格式化通知訊息"""
    try:
        current_time = datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')

        if not filtered_results:
            return f"""
🤖 台股技術分析報告
📅 分析時間: {current_time}

❌ 本次分析未找到符合條件的優質股票
💡 建議調整篩選條件或關注市場變化
"""

        # 獲取圖表檔案數量
        chart_dir = "/content/results/charts/"
        chart_count = 0
        if os.path.exists(chart_dir):
            chart_files = [f for f in os.listdir(chart_dir) if f.endswith('.png') and os.path.isfile(os.path.join(chart_dir, f))]
            chart_count = len(chart_files)

        message = f"""
🤖 台股技術分析報告
📅 分析時間: {current_time}
🎯 找到 {len(filtered_results)} 支優質股票
📊 生成 {chart_count} 張技術分析圖表

📈 TOP {min(5, len(filtered_results))} 推薦股票:
"""

        for rank, (stock_id, result) in enumerate(list(filtered_results.items())[:5], 1):
            trend = result.get('trend_analysis', {})
            tech = result.get('technical_analysis', {})
            recommendation = result.get('recommendation', {})

            # 獲取更多技術指標數據
            macd = tech.get('macd_value', 0)
            signal = tech.get('signal_value', 0)
            macd_status = "多頭" if macd > signal else "空頭" if macd < signal else "中性"

            message += f"""
{rank}. {result.get('stock_name', '')} ({stock_id})
   💰 價格: {result.get('current_price', 0):.2f} TWD
   📈 評分: {result.get('combined_score', 0):.1f}/100
   🎯 建議: {recommendation.get('action', '觀望')}
   📊 趨勢: {trend.get('trend', '盤整')}
   🔍 RSI: {tech.get('rsi_value', 50):.1f} ({tech.get('rsi_status', '中性')})
   📉 MACD: {macd_status}
"""

        # 添加圖表資訊
        if chart_count > 0:
            message += f"""
📊 詳細分析圖表:
• 已生成 {chart_count} 張技術分析圖表
• 圖表包含K線、均線、成交量、MACD及RSI指標
• 圖表將在此訊息後發送
"""

        message += f"""
⚠️  投資提醒:
• 本分析僅供參考，投資有風險
• 建議結合基本面分析
• 請做好風險控制
"""

        return message.strip()

    except Exception as e:
        logger.error(f"格式化通知訊息時發生錯誤: {e}")
        traceback.print_exc()
        return f"台股分析完成，但訊息格式化失敗: {str(e)}"


async def main():
    """主程式"""
    try:
        print("🚀 台股強勢股分析程式啟動")
        print(f"⏰ 開始時間: {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}")
        print("🔍 分析目標: 尋找市場強勢股")
        print("=" * 60)

        # 初始化分析器
        analyzer = StrongMomentumStockAnalyzer()

        # 執行全部股票分析
        print("📊 開始分析所有台股...")
        results = await analyzer.analyze_all_stocks()

        if not results:
            error_message = "❌ 分析失敗，未能獲取任何結果"
            print(error_message)
            return

        # 在終端顯示結果
        display_terminal_results(results, limit=15)

        # 格式化通知訊息
        print("\n📱 正在準備發送通知...")
        message = format_momentum_stocks_message(results, limit=15)

        # 發送通知
        print("📱 正在發送通知...")
        async with aiohttp.ClientSession() as session:
            await send_notification(session, message)

        print(f"\n⏰ 程式執行完成: {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}")
        print("=" * 60)
        print("✅ 分析報告已完成")

    except Exception as e:
        error_message = f"❌ 主程式執行錯誤: {str(e)}\n\n詳細錯誤:\n{traceback.format_exc()}"
        logger.error(error_message)
        print(error_message)

if __name__ == "__main__":
    # 執行主程式
    asyncio.run(main())

In [ ]:
import asyncio
import aiohttp
import pandas as pd
import numpy as np
import yfinance as yf
import logging
import traceback
from datetime import datetime, timezone, timedelta
from typing import Dict, List, Any, Optional
import json
import os
import time
import requests
import random
from io import StringIO
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.font_manager import FontProperties
import seaborn as sns
import warnings

# 處理中文字體和警告
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False
warnings.filterwarnings("ignore", category=FutureWarning)

# Jupyter Notebook 支援
try:
    import nest_asyncio
    nest_asyncio.apply()
except ImportError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "nest_asyncio"])
    import nest_asyncio
    nest_asyncio.apply()

# 通知設定 (請根據需要修改)
TELEGRAM_TOKEN = "7902318521:AAEYoDMqwfHabI7L1SRiE4z33aFay42-VGE"
TELEGRAM_CHAT_ID = [879781796, 8113868436]  # 支援多用戶通知
DISCORD_WEBHOOK_URL = "https://discord.com/api/webhooks/1362715080734802102/Jma7A3VhEQrrRxIX_JW2l6rATjAZXsGXGfnJuAMqmS1QvqG_2ptg3vr_nsVnuV_PlnBl"
# 嘗試導入通知相關模組
try:
    from discord_webhook import DiscordWebhook
    MESSAGING_AVAILABLE = True
except ImportError:
    MESSAGING_AVAILABLE = False

# 設定日誌
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# 台北時區
taipei_tz = timezone(timedelta(hours=8))

class StockAnalyzer:
    def __init__(self):
        """初始化股票分析器"""
        self.config = {
            'rsi_period': 14,
            'macd_fast': 12,
            'macd_slow': 26,
            'macd_signal': 9,
            'bb_period': 20,
            'bb_std': 2,
            'kd_period': 14,
            'volume_ma_period': 20,
            'min_volume_lots': 1000,  # 最小成交量（張）
            'screen_conditions': {
                'min_price_change': 5.0,      # 最小漲幅5%
                'min_volume_lots': 5000,      # 最小成交量5000張
                'max_volume_ratio': 3.0,      # 最大成交量比率3倍
                'macd_diff_min': 0.0,         # MACD差值最小值
                'macd_diff_max': 1.5,         # MACD差值最大值
            },
            'trend_weights': {
                'price_trend': 0.3,
                'volume_trend': 0.2,
                'technical_score': 0.5
            }
        }
        self.taiwan_stocks = None
        self.stock_list_path = "taiwan_stocks_cache.csv"

    def get_taiwan_stocks(self, force_update=True):
        """獲取台灣股票清單"""
        if not force_update and os.path.exists(self.stock_list_path):
            if (time.time() - os.path.getmtime(self.stock_list_path)) < 86400:
                logger.info("從快取載入股票列表。")
                return pd.read_csv(self.stock_list_path, dtype={'stock_id': str})

        logger.info("從台灣證交所網站獲取最新股票清單...")
        urls = {
            "上市": "https://isin.twse.com.tw/isin/C_public.jsp?strMode=2",
            "上櫃": "https://isin.twse.com.tw/isin/C_public.jsp?strMode=4"
        }
        all_stocks_df = []
        headers = {'User-Agent': 'Mozilla/5.0'}

        for market_name, url in urls.items():
            try:
                res = requests.get(url, headers=headers, timeout=30)
                res.encoding = 'big5'
                html_dfs = pd.read_html(StringIO(res.text))
                df = html_dfs[0].copy()
                df.columns = df.iloc[0]
                df = df.iloc[1:].copy()
                df.loc[:, 'market'] = market_name
                all_stocks_df.append(df)
            except Exception as e:
                logger.error(f"獲取 {market_name} 股票列表失敗: {e}")
                continue

        if not all_stocks_df:
            logger.error("無法從網站獲取任何股票數據，使用備用清單。")
            return self._get_backup_stock_list()

        try:
            df = pd.concat(all_stocks_df, ignore_index=True)
            df[['stock_id', 'stock_name']] = df['有價證券代號及名稱'].str.split(r'\s+', n=1, expand=True)
            df = df[df['stock_id'].str.match(r'^\d{4}$', na=False)].copy()
            exclude = ['ETF', 'ETN', 'TDR', '受益', '指數', '購', '牛', '熊', '存託憑證']
            df = df[~df['stock_name'].str.contains('|'.join(exclude), na=False)].copy()

            df.loc[:, 'yahoo_symbol'] = df.apply(
                lambda row: f"{row['stock_id']}.TW" if '上市' in row['market'] else f"{row['stock_id']}.TWO", axis=1)

            final_df = df[['stock_id', 'stock_name', 'market', '產業別', 'yahoo_symbol']].rename(columns={'產業別': 'industry'})
            final_df = final_df.drop_duplicates(subset=['stock_id']).reset_index(drop=True)
            final_df.to_csv(self.stock_list_path, index=False)
            logger.info(f"成功獲取 {len(final_df)} 支股票清單並保存快取。")
            return final_df
        except Exception as e:
            logger.error(f"處理股票清單時發生錯誤: {e}")
            return self._get_backup_stock_list()

    def _get_backup_stock_list(self) -> pd.DataFrame:
        """備用股票清單"""
        try:
            logger.info("使用備用股票清單")
            famous_stocks = [
                {'stock_id': '2330', 'stock_name': '台積電', 'market': '上市', 'yahoo_symbol': '2330.TW', 'industry': '半導體'},
                {'stock_id': '2317', 'stock_name': '鴻海', 'market': '上市', 'yahoo_symbol': '2317.TW', 'industry': '電子'},
                {'stock_id': '2454', 'stock_name': '聯發科', 'market': '上市', 'yahoo_symbol': '2454.TW', 'industry': '半導體'},
                {'stock_id': '2881', 'stock_name': '富邦金', 'market': '上市', 'yahoo_symbol': '2881.TW', 'industry': '金融'},
                {'stock_id': '2882', 'stock_name': '國泰金', 'market': '上市', 'yahoo_symbol': '2882.TW', 'industry': '金融'},
                {'stock_id': '2412', 'stock_name': '中華電', 'market': '上市', 'yahoo_symbol': '2412.TW', 'industry': '通信'},
                {'stock_id': '1301', 'stock_name': '台塑', 'market': '上市', 'yahoo_symbol': '1301.TW', 'industry': '塑膠'},
                {'stock_id': '1303', 'stock_name': '南亞', 'market': '上市', 'yahoo_symbol': '1303.TW', 'industry': '塑膠'},
                {'stock_id': '2308', 'stock_name': '台達電', 'market': '上市', 'yahoo_symbol': '2308.TW', 'industry': '電子'},
                {'stock_id': '2303', 'stock_name': '聯電', 'market': '上市', 'yahoo_symbol': '2303.TW', 'industry': '半導體'},
                {'stock_id': '2002', 'stock_name': '中鋼', 'market': '上市', 'yahoo_symbol': '2002.TW', 'industry': '鋼鐵'},
                {'stock_id': '2886', 'stock_name': '兆豐金', 'market': '上市', 'yahoo_symbol': '2886.TW', 'industry': '金融'},
                {'stock_id': '2891', 'stock_name': '中信金', 'market': '上市', 'yahoo_symbol': '2891.TW', 'industry': '金融'},
                {'stock_id': '3008', 'stock_name': '大立光', 'market': '上市', 'yahoo_symbol': '3008.TW', 'industry': '光學'},
                {'stock_id': '2357', 'stock_name': '華碩', 'market': '上市', 'yahoo_symbol': '2357.TW', 'industry': '電腦'},
                {'stock_id': '2382', 'stock_name': '廣達', 'market': '上市', 'yahoo_symbol': '2382.TW', 'industry': '電腦'},
                {'stock_id': '2395', 'stock_name': '研華', 'market': '上市', 'yahoo_symbol': '2395.TW', 'industry': '電腦'},
                {'stock_id': '3711', 'stock_name': '日月光投控', 'market': '上市', 'yahoo_symbol': '3711.TW', 'industry': '半導體'},
                {'stock_id': '2409', 'stock_name': '友達', 'market': '上市', 'yahoo_symbol': '2409.TW', 'industry': '面板'},
                {'stock_id': '2884', 'stock_name': '玉山金', 'market': '上市', 'yahoo_symbol': '2884.TW', 'industry': '金融'},
            ]
            df = pd.DataFrame(famous_stocks)
            logger.info(f"備用股票清單包含 {len(df)} 支股票")
            return df
        except Exception as e:
            logger.error(f"生成備用股票清單時發生錯誤: {e}")
            return pd.DataFrame()

    def check_volume_filter(self, df: pd.DataFrame) -> bool:
        """檢查成交量是否符合篩選條件（每日至少1000張）"""
        try:
            if df.empty or 'Volume' not in df.columns:
                return False
            recent_volume = df['Volume'].tail(20).mean()
            volume_lots = recent_volume / 1000
            meets_volume = volume_lots >= self.config['min_volume_lots']
            return meets_volume
        except Exception as e:
            logger.error(f"檢查成交量篩選時發生錯誤: {e}")
            return False

    def fetch_yfinance_data(self, stock_id: str, period: str = "1y", interval: str = "1d", retries: int = 3):
        """獲取股票數據（同步版本）"""
        for attempt in range(retries):
            try:
                df = yf.download(tickers=stock_id, period=period, interval=interval, progress=False, auto_adjust=True)
                if df.empty:
                    logger.warning(f"[{stock_id}] 無數據返回")
                    return pd.DataFrame()

                if isinstance(df.columns, pd.MultiIndex):
                    df.columns = df.columns.get_level_values(0)

                required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
                for col in required_cols:
                    if col in df.columns:
                        df[col] = pd.to_numeric(df[col], errors='coerce')

                df = df.dropna(subset=required_cols)
                return df.reset_index()

            except Exception as e:
                if attempt < retries - 1:
                    sleep_time = 0.5 * (2 ** attempt) + random.uniform(0, 1)
                    logger.warning(f"[{stock_id}] 第 {attempt + 1}/{retries} 次嘗試失敗: {e}。等待 {sleep_time:.2f} 秒後重試...")
                    time.sleep(sleep_time)
                else:
                    logger.error(f"[{stock_id}] 獲取數據失敗。")
                    return pd.DataFrame()
        return pd.DataFrame()

    # 基礎技術指標計算函數
    def calculate_sma(self, data, period):
        """計算SMA (簡單移動平均線)"""
        return data.rolling(window=period).mean()

    def calculate_ema(self, data, period):
        """計算EMA (指數移動平均線)"""
        return data.ewm(span=period, adjust=False).mean()

    def calculate_rsi(self, prices: pd.Series, period: int = None) -> pd.Series:
        """計算RSI"""
        try:
            if period is None:
                period = self.config['rsi_period']
            delta = prices.diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
            rs = gain / loss
            rsi = 100 - (100 / (1 + rs))
            return rsi
        except Exception as e:
            logger.error(f"計算RSI時發生錯誤: {e}")
            return pd.Series(index=prices.index, data=50)

    def calculate_macd(self, prices: pd.Series, fast: int = None, slow: int = None, signal: int = None):
        """計算MACD"""
        try:
            if fast is None:
                fast = self.config['macd_fast']
            if slow is None:
                slow = self.config['macd_slow']
            if signal is None:
                signal = self.config['macd_signal']

            ema_fast = self.calculate_ema(prices, fast)
            ema_slow = self.calculate_ema(prices, slow)
            macd_line = ema_fast - ema_slow
            macd_signal = self.calculate_ema(macd_line, signal)
            macd_histogram = macd_line - macd_signal
            return macd_line, macd_signal, macd_histogram
        except Exception as e:
            logger.error(f"計算MACD時發生錯誤: {e}")
            zero_series = pd.Series(index=prices.index, data=0)
            return zero_series, zero_series, zero_series

    def calculate_stoch(self, high, low, close, k_period=None, d_period=3):
        """計算KD指標"""
        try:
            if k_period is None:
                k_period = self.config['kd_period']

            # 計算%K
            lowest_low = low.rolling(window=k_period).min()
            highest_high = high.rolling(window=k_period).max()

            # 防止除以零
            denom = highest_high - lowest_low
            denom = denom.replace(0, 0.000001)

            k = 100 * ((close - lowest_low) / denom)
            # 計算%D (K的移動平均)
            d = k.rolling(window=d_period).mean()
            return k, d
        except Exception as e:
            logger.error(f"計算KD時發生錯誤: {e}")
            fifty_series = pd.Series(index=high.index, data=50)
            return fifty_series, fifty_series

    def calculate_bollinger_bands(self, prices: pd.Series, period: int = None, std_dev: int = None):
        """計算布林帶"""
        try:
            if period is None:
                period = self.config['bb_period']
            if std_dev is None:
                std_dev = self.config['bb_std']

            middle = prices.rolling(window=period).mean()
            std = prices.rolling(window=period).std()
            upper = middle + (std * std_dev)
            lower = middle - (std * std_dev)
            return upper, middle, lower
        except Exception as e:
            logger.error(f"計算布林帶時發生錯誤: {e}")
            zero_series = pd.Series(index=prices.index, data=0)
            return zero_series, zero_series, zero_series

    def calculate_indicators(self, data):
        """計算技術指標（整合版）"""
        if data is None or len(data) < 30:
            return None

        # 複製數據以避免修改原始數據
        df = data.copy()

        try:
            # 計算MACD
            df['MACD'], df['MACD_Signal'], df['MACD_Hist'] = self.calculate_macd(df['Close'])

            # 計算均線
            df['MA5'] = self.calculate_sma(df['Close'], 5)
            df['MA10'] = self.calculate_sma(df['Close'], 10)
            df['MA20'] = self.calculate_sma(df['Close'], 20)
            df['MA30'] = self.calculate_sma(df['Close'], 30)

            # 計算RSI
            df['RSI'] = self.calculate_rsi(df['Close'])

            # 計算KD
            df['K'], df['D'] = self.calculate_stoch(df['High'], df['Low'], df['Close'])
            df['K_Percent'] = df['K']  # 保持一致性
            df['D_Percent'] = df['D']  # 保持一致性

            # 計算布林帶
            bb_upper, bb_middle, bb_lower = self.calculate_bollinger_bands(df['Close'])
            df['BB_Upper'] = bb_upper
            df['BB_Middle'] = bb_middle
            df['BB_Lower'] = bb_lower

            # 計算漲幅
            df['Change'] = df['Close'].pct_change() * 100

            # 計算成交量均線
            if 'Volume' in df.columns:
                df['Volume_MA'] = df['Volume'].rolling(window=self.config['volume_ma_period']).mean()
                df['Volume_MA30'] = self.calculate_sma(df['Volume'], 30)

            return df

        except Exception as e:
            logger.error(f"計算技術指標時發生錯誤: {e}")
            return df

    def calculate_technical_indicators(self, df: pd.DataFrame) -> pd.DataFrame:
        """計算技術指標（與calculate_indicators保持一致）"""
        return self.calculate_indicators(df)

    def perform_quadrant_analysis(self, data):
        """四象限分析"""
        if data is None or len(data) < 30:
            return None

        try:
            # 獲取最新數據
            latest = data.iloc[-1]

            # 四象限判斷
            macd_above_signal = latest['MACD'] > latest['MACD_Signal']
            macd_rising = data['MACD'].iloc[-1] > data['MACD'].iloc[-2] if len(data) >= 2 else False

            rsi_above_50 = latest['RSI'] > 50
            rsi_rising = data['RSI'].iloc[-1] > data['RSI'].iloc[-2] if len(data) >= 2 else False

            # 判斷象限
            if macd_above_signal and rsi_above_50:
                quadrant = 1  # 強勢多頭
            elif not macd_above_signal and rsi_above_50:
                quadrant = 2  # 多頭警戒
            elif not macd_above_signal and not rsi_above_50:
                quadrant = 3  # 強勢空頭
            else:  # macd_above_signal and not rsi_above_50
                quadrant = 4  # 空頭反轉

            # 趨勢方向
            if macd_rising and rsi_rising:
                trend = "上升"
            elif not macd_rising and not rsi_rising:
                trend = "下降"
            else:
                trend = "盤整"

            # 返回分析結果
            return {
                'quadrant': quadrant,
                'trend': trend,
                'macd_above_signal': macd_above_signal,
                'macd_rising': macd_rising,
                'rsi_above_50': rsi_above_50,
                'rsi_rising': rsi_rising,
                'macd': latest['MACD'],
                'macd_signal': latest['MACD_Signal'],
                'macd_hist': latest['MACD_Hist'],
                'rsi': latest['RSI'],
                'k': latest['K'],
                'd': latest['D']
            }

        except Exception as e:
            logger.error(f"四象限分析時發生錯誤: {e}")
            return None

    def get_resample_frequency(self, freq_type: str) -> str:
        """獲取適當的重採樣頻率字符串"""
        try:
            import pandas as pd

            # 檢查 pandas 版本
            pd_version = pd.__version__
            major_version = int(pd_version.split('.')[0])
            minor_version = int(pd_version.split('.')[1])

            # pandas 2.2.0+ 使用新的頻率字符串
            if major_version > 2 or (major_version == 2 and minor_version >= 2):
                freq_mapping = {
                    'M': 'ME',  # Month End
                    'Q': 'QE',  # Quarter End
                    'Y': 'YE',  # Year End
                    'W': 'W',   # Week (unchanged)
                    'D': 'D',   # Day (unchanged)
                }
                return freq_mapping.get(freq_type, freq_type)
            else:
                return freq_type

        except Exception as e:
            logger.warning(f"檢查 pandas 版本時發生錯誤: {e}")
            return freq_type

    def perform_multi_timeframe_analysis(self, stock_id, start_date=None, end_date=None):
        """多時間週期分析（修正版）"""
        try:
            if start_date is None:
                start_date = (datetime.now() - timedelta(days=365)).strftime("%Y-%m-%d")
            if end_date is None:
                end_date = datetime.now().strftime("%Y-%m-%d")

            # 獲取較長時間的數據以便重採樣
            extended_start = (datetime.strptime(start_date, "%Y-%m-%d") - timedelta(days=365)).strftime("%Y-%m-%d")

            # 如果是yahoo symbol格式，直接使用；否則轉換
            if not stock_id.endswith(('.TW', '.TWO')):
                yahoo_symbol = f"{stock_id}.TW"
            else:
                yahoo_symbol = stock_id

            data = self.fetch_yfinance_data(yahoo_symbol, period="2y")

            if data is None or data.empty:
                return None

            # 設定日期為索引
            if 'Date' in data.columns:
                data.set_index('Date', inplace=True)
            elif data.index.name != 'Date':
                data.index = pd.to_datetime(data.index)

            # 不同時間週期的數據
            daily_data = self.calculate_indicators(data)

            # 週線數據
            try:
                weekly_freq = self.get_resample_frequency('W')
                weekly_data = data.resample(weekly_freq).agg({
                    'Open': 'first',
                    'High': 'max',
                    'Low': 'min',
                    'Close': 'last',
                    'Volume': 'sum'
                }).dropna()
                weekly_data = self.calculate_indicators(weekly_data)
            except Exception as e:
                logger.warning(f"週線數據重採樣失敗: {e}")
                weekly_data = None

            # 月線數據
            try:
                monthly_freq = self.get_resample_frequency('M')

                # 使用警告抑制來處理舊版本的棄用警告
                import warnings
                with warnings.catch_warnings():
                    warnings.filterwarnings("ignore", category=FutureWarning, message=".*'M' is deprecated.*")

                    monthly_data = data.resample(monthly_freq).agg({
                        'Open': 'first',
                        'High': 'max',
                        'Low': 'min',
                        'Close': 'last',
                        'Volume': 'sum'
                    }).dropna()

                monthly_data = self.calculate_indicators(monthly_data)
            except Exception as e:
                logger.warning(f"月線數據重採樣失敗: {e}")
                monthly_data = None

            # 進行四象限分析
            daily_analysis = self.perform_quadrant_analysis(daily_data) if daily_data is not None else None
            weekly_analysis = self.perform_quadrant_analysis(weekly_data) if weekly_data is not None else None
            monthly_analysis = self.perform_quadrant_analysis(monthly_data) if monthly_data is not None else None

            result = {
                'daily': {
                    'data': daily_data,
                    'analysis': daily_analysis
                }
            }

            if weekly_data is not None:
                result['weekly'] = {
                    'data': weekly_data,
                    'analysis': weekly_analysis
                }

            if monthly_data is not None:
                result['monthly'] = {
                    'data': monthly_data,
                    'analysis': monthly_analysis
                }

            return result

        except Exception as e:
            logger.error(f"多時間週期分析時發生錯誤: {e}")
            return None

    def generate_multi_timeframe_summary(self, results, stock_id):
        """生成多時間週期分析報告"""
        if not results:
            print("無分析結果可顯示")
            return

        print("\n" + "=" * 80)
        print(f"股票代碼: {stock_id} 多時間週期四象限分析結果")
        print("=" * 80)

        timeframes = {
            'daily': '日線',
            'weekly': '週線',
            'monthly': '月線'
        }

        quadrant_names = {
            1: "強勢多頭 (第一象限)",
            2: "多頭警戒 (第二象限)",
            3: "強勢空頭 (第三象限)",
            4: "空頭反轉 (第四象限)"
        }

        for tf, tf_name in timeframes.items():
            if tf in results and results[tf]['analysis']:
                analysis = results[tf]['analysis']
                data = results[tf]['data']
                latest = data.iloc[-1]

                print(f"\n【{tf_name}分析】")
                print(f"象限: {quadrant_names[analysis['quadrant']]}")
                print(f"趨勢: {analysis['trend']}")
                print(f"MACD: {analysis['macd']:.4f}, 訊號線: {analysis['macd_signal']:.4f}, 柱狀: {analysis['macd_hist']:.4f}")
                print(f"RSI: {analysis['rsi']:.2f}")
                print(f"KD指標: K={analysis['k']:.2f}, D={analysis['d']:.2f}")
                print(f"收盤價: {latest['Close']:.2f}, 10日均線: {latest['MA10']:.2f}, 30日均線: {latest['MA30']:.2f}")

                # 顯示相對位置
                print("技術指標相對位置:")
                if latest['Close'] > latest['MA10']:
                    print("- 價格位於10日均線之上")
                else:
                    print("- 價格位於10日均線之下")

                if latest['Close'] > latest['MA30']:
                    print("- 價格位於30日均線之上")
                else:
                    print("- 價格位於30日均線之下")

                if analysis['macd'] > analysis['macd_signal']:
                    print("- MACD位於訊號線之上 (多頭)")
                else:
                    print("- MACD位於訊號線之下 (空頭)")

                if analysis['k'] > analysis['d']:
                    print("- K線位於D線之上 (多頭)")
                else:
                    print("- K線位於D線之下 (空頭)")

                print(f"\n【{tf_name}分析】: 數據不足，無法分析")

        # 綜合建議
        print("\n【綜合分析】")

        # 檢查是否所有時間週期都有分析結果
        if all(tf in results and results[tf]['analysis'] for tf in timeframes):
            daily_q = results['daily']['analysis']['quadrant']
            weekly_q = results['weekly']['analysis']['quadrant']
            monthly_q = results['monthly']['analysis']['quadrant']

            # 多頭排列
            if monthly_q in [1, 4] and weekly_q in [1, 4] and daily_q in [1, 4]:
                print("多頭排列: 月線、週線和日線均呈現多頭趨勢，可考慮逢低買入")
            # 空頭排列
            elif monthly_q in [2, 3] and weekly_q in [2, 3] and daily_q in [2, 3]:
                print("空頭排列: 月線、週線和日線均呈現空頭趨勢，建議觀望或減倉")
            # 多空轉換
            else:
                if daily_q in [1, 4] and (weekly_q in [2, 3] or monthly_q in [2, 3]):
                    print("短線反彈: 日線呈現多頭，但中長期仍在空頭，可能是反彈行情")
                elif daily_q in [2, 3] and (weekly_q in [1, 4] or monthly_q in [1, 4]):
                    print("短線回調: 日線呈現空頭，但中長期仍在多頭，可能是回調行情")
                else:
                    print("多空交錯: 各時間週期走勢不一致，建議觀望或依主要時間週期操作")
        else:
            print("部分時間週期數據不足，無法提供完整綜合分析")

    def analyze_trend(self, df: pd.DataFrame) -> Dict[str, Any]:
        """趨勢分析"""
        try:
            if df.empty or len(df) < 20:
                return {'trend': '盤整', 'strength': 0, 'price_change_5d': 0, 'price_change_20d': 0}

            current_price = float(df['Close'].iloc[-1])
            price_5d_ago = float(df['Close'].iloc[-6]) if len(df) >= 6 else current_price
            price_20d_ago = float(df['Close'].iloc[-21]) if len(df) >= 21 else current_price

            change_5d = ((current_price - price_5d_ago) / price_5d_ago * 100) if price_5d_ago != 0 else 0
            change_20d = ((current_price - price_20d_ago) / price_20d_ago * 100) if price_20d_ago != 0 else 0

            # 移動平均線趨勢
            ma5_current = float(df['MA5'].iloc[-1]) if 'MA5' in df.columns and not pd.isna(df['MA5'].iloc[-1]) else current_price
            ma20_current = float(df['MA20'].iloc[-1]) if 'MA20' in df.columns and not pd.isna(df['MA20'].iloc[-1]) else current_price

            # 判斷趨勢
            if change_5d > 3 and change_20d > 5 and current_price > ma5_current > ma20_current:
                trend = '強勢上漲'
                strength = 3
            elif change_5d > 1 and change_20d > 2 and current_price > ma5_current:
                trend = '上漲'
                strength = 2
            elif change_5d < -3 and change_20d < -5 and current_price < ma5_current < ma20_current:
                trend = '強勢下跌'
                strength = -3
            elif change_5d < -1 and change_20d < -2 and current_price < ma5_current:
                trend = '下跌'
                strength = -2
            else:
                trend = '盤整'
                strength = 0

            return {
                'trend': trend,
                'strength': strength,
                'price_change_5d': change_5d,
                'price_change_20d': change_20d,
                'ma5_position': ma5_current,
                'ma20_position': ma20_current
            }

        except Exception as e:
            logger.error(f"趨勢分析時發生錯誤: {e}")
            return {'trend': '盤整', 'strength': 0, 'price_change_5d': 0, 'price_change_20d': 0}

    def analyze_volume(self, df: pd.DataFrame) -> Dict[str, Any]:
        """成交量分析"""
        try:
            if df.empty or 'Volume' not in df.columns or len(df) < 20:
                return {'volume_trend': '正常', 'volume_ratio': 1.0, 'volume_signal': '中性', 'avg_volume_lots': 0}

            current_volume = float(df['Volume'].iloc[-1])
            avg_volume_20d = float(df['Volume'].rolling(window=20).mean().iloc[-1])

            volume_ratio = current_volume / avg_volume_20d if avg_volume_20d > 0 else 1.0
            avg_volume_lots = avg_volume_20d / 1000  # 轉換為張數

            if volume_ratio > 2.0:
                volume_trend = '爆量'
                volume_signal = '強烈'
            elif volume_ratio > 1.5:
                volume_trend = '放量'
                volume_signal = '積極'
            elif volume_ratio < 0.5:
                volume_trend = '縮量'
                volume_signal = '消極'
            else:
                volume_trend = '正常'
                volume_signal = '中性'

            return {
                'volume_trend': volume_trend,
                'volume_ratio': volume_ratio,
                'volume_signal': volume_signal,
                'avg_volume_lots': avg_volume_lots
            }

        except Exception as e:
            logger.error(f"成交量分析時發生錯誤: {e}")
            return {'volume_trend': '正常', 'volume_ratio': 1.0, 'volume_signal': '中性', 'avg_volume_lots': 0}

    def analyze_technical_indicators(self, df: pd.DataFrame) -> Dict[str, Any]:
        """技術指標分析"""
        try:
            if df.empty:
                return {
                    'rsi_signal': '中性', 'rsi_value': 50,
                    'macd_signal': '中性', 'macd_value': 0,
                    'kd_signal': '中性', 'k_value': 50, 'd_value': 50,
                    'bb_signal': '中性', 'bb_position': 0.5,
                    'score': 50
                }

            signals = {}
            score = 50

            # RSI 分析
            if 'RSI' in df.columns and not df['RSI'].empty:
                rsi_value = float(df['RSI'].iloc[-1]) if not pd.isna(df['RSI'].iloc[-1]) else 50
                signals['rsi_value'] = rsi_value

                if rsi_value > 80:
                    signals['rsi_signal'] = '超買'
                    score -= 15
                elif rsi_value > 70:
                    signals['rsi_signal'] = '偏高'
                    score -= 5
                elif rsi_value < 20:
                    signals['rsi_signal'] = '超賣'
                    score += 15
                elif rsi_value < 30:
                    signals['rsi_signal'] = '偏低'
                    score += 5
                else:
                    signals['rsi_signal'] = '中性'
            else:
                signals['rsi_signal'] = '中性'
                signals['rsi_value'] = 50

            # MACD 分析
            if all(col in df.columns for col in ['MACD', 'MACD_Signal']) and len(df) >= 2:
                macd_current = float(df['MACD'].iloc[-1]) if not pd.isna(df['MACD'].iloc[-1]) else 0
                macd_signal_current = float(df['MACD_Signal'].iloc[-1]) if not pd.isna(df['MACD_Signal'].iloc[-1]) else 0
                macd_prev = float(df['MACD'].iloc[-2]) if not pd.isna(df['MACD'].iloc[-2]) else 0
                macd_signal_prev = float(df['MACD_Signal'].iloc[-2]) if not pd.isna(df['MACD_Signal'].iloc[-2]) else 0

                signals['macd_value'] = macd_current

                if macd_prev <= macd_signal_prev and macd_current > macd_signal_current:
                    signals['macd_signal'] = '黃金交叉'
                    score += 20
                elif macd_prev >= macd_signal_prev and macd_current < macd_signal_current:
                    signals['macd_signal'] = '死亡交叉'
                    score -= 20
                elif macd_current > macd_signal_current:
                    signals['macd_signal'] = '多頭'
                    score += 5
                elif macd_current < macd_signal_current:
                    signals['macd_signal'] = '空頭'
                    score -= 5
                else:
                    signals['macd_signal'] = '中性'
            else:
                signals['macd_signal'] = '中性'
                signals['macd_value'] = 0

            # KD 分析
            if all(col in df.columns for col in ['K_Percent', 'D_Percent']):
                k_value = float(df['K_Percent'].iloc[-1]) if not pd.isna(df['K_Percent'].iloc[-1]) else 50
                d_value = float(df['D_Percent'].iloc[-1]) if not pd.isna(df['D_Percent'].iloc[-1]) else 50

                signals['k_value'] = k_value
                signals['d_value'] = d_value

                if k_value > 80 and d_value > 80:
                    signals['kd_signal'] = '超買'
                    score -= 10
                elif k_value < 20 and d_value < 20:
                    signals['kd_signal'] = '超賣'
                    score += 10
                elif k_value > d_value:
                    signals['kd_signal'] = '偏多'
                    score += 3
                elif k_value < d_value:
                    signals['kd_signal'] = '偏空'
                    score -= 3
                else:
                    signals['kd_signal'] = '中性'
            else:
                signals['kd_signal'] = '中性'
                signals['k_value'] = 50
                signals['d_value'] = 50

            # 布林帶分析
            if all(col in df.columns for col in ['BB_Upper', 'BB_Lower', 'Close']):
                current_price = float(df['Close'].iloc[-1])
                bb_upper = float(df['BB_Upper'].iloc[-1]) if not pd.isna(df['BB_Upper'].iloc[-1]) else current_price
                bb_lower = float(df['BB_Lower'].iloc[-1]) if not pd.isna(df['BB_Lower'].iloc[-1]) else current_price

                if bb_upper != bb_lower:
                    bb_position = (current_price - bb_lower) / (bb_upper - bb_lower)
                    signals['bb_position'] = bb_position

                    if bb_position > 0.8:
                        signals['bb_signal'] = '接近上軌'
                        score -= 5
                    elif bb_position < 0.2:
                        signals['bb_signal'] = '接近下軌'
                        score += 5
                    else:
                        signals['bb_signal'] = '中性'
                else:
                    signals['bb_signal'] = '中性'
                    signals['bb_position'] = 0.5
            else:
                signals['bb_signal'] = '中性'
                signals['bb_position'] = 0.5

            score = max(0, min(100, score))
            signals['score'] = score

            return signals

        except Exception as e:
            logger.error(f"技術指標分析時發生錯誤: {e}")
            return {
                'rsi_signal': '中性', 'rsi_value': 50,
                'macd_signal': '中性', 'macd_value': 0,
                'kd_signal': '中性', 'k_value': 50, 'd_value': 50,
                'bb_signal': '中性', 'bb_position': 0.5,
                'score': 50
            }

    def calculate_combined_score(self, trend_analysis: Dict, volume_analysis: Dict, technical_analysis: Dict) -> float:
        """計算綜合分數"""
        try:
            base_score = 50

            # 趨勢分數 (權重: 30%)
            trend_score = 0
            trend_strength = trend_analysis.get('strength', 0)
            if trend_strength > 0:
                trend_score = min(30, trend_strength * 10)
            elif trend_strength < 0:
                trend_score = max(-30, trend_strength * 10)

            # 成交量分數 (權重: 20%)
            volume_score = 0
            volume_ratio = volume_analysis.get('volume_ratio', 1.0)
            if volume_ratio > 1.5:
                volume_score = 10
            elif volume_ratio > 1.2:
                volume_score = 5
            elif volume_ratio < 0.7:
                volume_score = -5

            # 技術指標分數 (權重: 50%)
            tech_score = technical_analysis.get('score', 50) - 50

            # 計算加權總分
            total_score = base_score + (trend_score * 0.3) + (volume_score * 0.2) + (tech_score * 0.5)

            return max(0, min(100, total_score))

        except Exception as e:
            logger.error(f"計算綜合分數時發生錯誤: {e}")
            return 50.0

    def generate_recommendation(self, combined_score: float, trend_analysis: Dict, technical_analysis: Dict) -> Dict[str, Any]:
        """生成投資建議"""
        try:
            recommendation = "持有"
            confidence = "中等"
            reasons = []

            if combined_score >= 75:
                recommendation = "強力買進"
                confidence = "高"
            elif combined_score >= 60:
                recommendation = "買進"
                confidence = "中高"
            elif combined_score >= 40:
                recommendation = "持有"
                confidence = "中等"
            elif combined_score >= 25:
                recommendation = "賣出"
                confidence = "中高"
            else:
                recommendation = "強力賣出"
                confidence = "高"

            # 生成建議原因
            trend = trend_analysis.get('trend', '盤整')
            if '上漲' in trend:
                reasons.append(f"價格趨勢：{trend}")
            elif '下跌' in trend:
                reasons.append(f"價格趨勢：{trend}")

            rsi_signal = technical_analysis.get('rsi_signal', '中性')
            if rsi_signal in ['超賣', '偏低']:
                reasons.append(f"RSI指標：{rsi_signal}，可能反彈")
            elif rsi_signal in ['超買', '偏高']:
                reasons.append(f"RSI指標：{rsi_signal}，注意回調")

            macd_signal = technical_analysis.get('macd_signal', '中性')
            if macd_signal == '黃金交叉':
                reasons.append("MACD出現黃金交叉")
            elif macd_signal == '死亡交叉':
                reasons.append("MACD出現死亡交叉")

            return {
                'recommendation': recommendation,
                'confidence': confidence,
                'score': combined_score,
                'reasons': reasons
            }

        except Exception as e:
            logger.error(f"生成投資建議時發生錯誤: {e}")
            return {
                'recommendation': '持有',
                'confidence': '低',
                'score': 50,
                'reasons': ['分析過程出現錯誤']
            }

    def screen_stocks(self, start_date=None, end_date=None) -> List[Dict[str, Any]]:
        """
        股票篩選功能
        篩選條件：
        1. 技術面
           - MACD: 0 < D-M < 1.5
           - 本日收盤價高於10日均價且高於30日均價
           - 本日股價漲幅5%以上
        2. 籌碼面
           - 本日成交量5000張以上但不超過30日均量的3倍
           - 由符合篩選條件中取成交量前三大為標的
        """
        print("\n" + "=" * 80)
        print("開始股票篩選...")
        print("=" * 80)

        if start_date is None:
            start_date = (datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d")
        if end_date is None:
            end_date = datetime.now().strftime("%Y-%m-%d")

        # 延長開始日期以確保有足夠的數據計算指標
        extended_start = (datetime.strptime(start_date, "%Y-%m-%d") - timedelta(days=60)).strftime("%Y-%m-%d")

        # 獲取股票清單
        if self.taiwan_stocks is None:
            self.taiwan_stocks = self.get_taiwan_stocks()

        if self.taiwan_stocks.empty:
            print("無法獲取股票清單")
            return []

        results = []
        stock_list = [(row['stock_id'], row['stock_name'], row['yahoo_symbol']) for _, row in self.taiwan_stocks.iterrows()]

        # 遍歷股票列表
        for i, (stock_id, stock_name, yahoo_symbol) in enumerate(stock_list):
            print(f"處理 {i+1}/{len(stock_list)}: {stock_id} - {stock_name}")

            # 獲取股票數據
            data = self.fetch_yfinance_data(yahoo_symbol, period="3mo")
            if data is None or len(data) < 30:
                continue

            # 計算技術指標
            df = self.calculate_indicators(data)
            if df is None:
                continue

            # 獲取最新一天的數據
            latest = df.iloc[-1]

            try:
                # 篩選條件檢查
                # 1. MACD: 0 < D-M < 1.5
                macd_diff = latest['MACD_Signal'] - latest['MACD']
                macd_condition = (self.config['screen_conditions']['macd_diff_min'] <
                                macd_diff < self.config['screen_conditions']['macd_diff_max'])

                # 2. 本日收盤價高於10日均價且高於30日均價
                price_above_ma = (latest['Close'] > latest['MA10']) and (latest['Close'] > latest['MA30'])

                # 3. 本日股價漲幅5%以上
                price_change = latest['Change']
                price_change_condition = price_change >= self.config['screen_conditions']['min_price_change']

                # 4. 本日成交量5000張以上但不超過30日均量的3倍
                # 台股一張是1000股，所以5000張是5,000,000股
                min_volume = self.config['screen_conditions']['min_volume_lots'] * 1000
                max_volume_ratio = self.config['screen_conditions']['max_volume_ratio']
                volume_condition = (latest['Volume'] >= min_volume and
                                  latest['Volume'] <= latest['Volume_MA30'] * max_volume_ratio)

                # 檢查是否符合所有條件
                if macd_condition and price_above_ma and price_change_condition and volume_condition:
                    # 進行四象限分析
                    quadrant_analysis = self.perform_quadrant_analysis(df)

                    results.append({
                        'stock_id': stock_id,
                        'stock_name': stock_name,
                        'yahoo_symbol': yahoo_symbol,
                        'close': latest['Close'],
                        'change': price_change,
                        'volume': latest['Volume'],
                        'volume_lots': latest['Volume'] / 1000,
                        'macd_diff': macd_diff,
                        'quadrant_analysis': quadrant_analysis,
                        'data': df
                    })

            except Exception as e:
                print(f"  篩選 {stock_id} 時發生錯誤: {e}")
                continue

        # 按成交量排序
        if results:
            results.sort(key=lambda x: x['volume'], reverse=True)

            # 取前三大成交量
            top_results = results[:min(3, len(results))]

            print("\n" + "=" * 80)
            print(f"篩選結果: 符合條件的股票有 {len(results)} 檔，取成交量前 {len(top_results)} 大")
            print("=" * 80)

            for i, result in enumerate(top_results):
                print(f"\n排名 {i+1}: {result['stock_id']} - {result['stock_name']}")
                print(f"收盤價: {result['close']:.2f}, 漲幅: {result['change']:.2f}%")
                print(f"成交量: {result['volume']:,.0f} 股 (約 {result['volume_lots']:.0f} 張)")
                print(f"MACD差值 (D-M): {result['macd_diff']:.4f}")

                # 四象限分析
                if result['quadrant_analysis']:
                    analysis = result['quadrant_analysis']
                    print(f"四象限分析: 第 {analysis['quadrant']} 象限 ({analysis['trend']}趨勢)")

            return top_results
        else:
            print("\n沒有符合篩選條件的股票")
            return []

    def create_stock_chart(self, stock_data: Dict[str, Any], save_path: str = None) -> str:
        """創建股票技術分析圖表"""
        try:
            df = stock_data.get('data')
            if df is None or df.empty:
                return None

            stock_name = stock_data.get('stock_name', 'Unknown')
            stock_id = stock_data.get('stock_id', 'Unknown')

            # 創建圖表
            fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
            fig.suptitle(f'{stock_name} ({stock_id}) 技術分析圖表', fontsize=16, fontweight='bold')

            # 確保日期格式正確
            if 'Date' not in df.columns:
                df = df.reset_index()
            df['Date'] = pd.to_datetime(df['Date'])

            # 取最近60天的數據用於繪圖
            plot_df = df.tail(60).copy()

            # 圖1: 價格和移動平均線
            ax1.plot(plot_df['Date'], plot_df['Close'], label='收盤價', linewidth=2, color='black')
            ax1.plot(plot_df['Date'], plot_df['MA5'], label='MA5', color='red', alpha=0.7)
            ax1.plot(plot_df['Date'], plot_df['MA10'], label='MA10', color='blue', alpha=0.7)
            ax1.plot(plot_df['Date'], plot_df['MA20'], label='MA20', color='green', alpha=0.7)
            ax1.plot(plot_df['Date'], plot_df['MA30'], label='MA30', color='purple', alpha=0.7)

            # 布林帶
            ax1.fill_between(plot_df['Date'], plot_df['BB_Upper'], plot_df['BB_Lower'],
                           alpha=0.1, color='gray', label='布林帶')

            ax1.set_title('價格走勢與移動平均線')
            ax1.legend()
            ax1.grid(True, alpha=0.3)
            ax1.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))

            # 圖2: MACD
            ax2.plot(plot_df['Date'], plot_df['MACD'], label='MACD', color='blue')
            ax2.plot(plot_df['Date'], plot_df['MACD_Signal'], label='Signal', color='red')
            ax2.bar(plot_df['Date'], plot_df['MACD_Hist'], label='Histogram', alpha=0.3, color='green')
            ax2.axhline(y=0, color='black', linestyle='-', alpha=0.5)
            ax2.set_title('MACD指標')
            ax2.legend()
            ax2.grid(True, alpha=0.3)
            ax2.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))

            # 圖3: RSI和KD
            ax3_twin = ax3.twinx()
            ax3.plot(plot_df['Date'], plot_df['RSI'], label='RSI', color='purple', linewidth=2)
            ax3.axhline(y=70, color='red', linestyle='--', alpha=0.5, label='超買線')
            ax3.axhline(y=30, color='green', linestyle='--', alpha=0.5, label='超賣線')
            ax3.set_ylim(0, 100)
            ax3.set_ylabel('RSI', color='purple')

            ax3_twin.plot(plot_df['Date'], plot_df['K'], label='K', color='orange')
            ax3_twin.plot(plot_df['Date'], plot_df['D'], label='D', color='brown')
            ax3_twin.set_ylim(0, 100)
            ax3_twin.set_ylabel('KD', color='orange')

            ax3.set_title('RSI與KD指標')
            ax3.legend(loc='upper left')
            ax3_twin.legend(loc='upper right')
            ax3.grid(True, alpha=0.3)
            ax3.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))

            # 圖4: 成交量
            ax4.bar(plot_df['Date'], plot_df['Volume']/1000, alpha=0.6, color='skyblue', label='成交量(千股)')
            ax4.plot(plot_df['Date'], plot_df['Volume_MA']/1000, color='red', label='20日均量', linewidth=2)
            ax4.set_title('成交量分析')
            ax4.legend()
            ax4.grid(True, alpha=0.3)
            ax4.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))

            # 調整日期標籤
            for ax in [ax1, ax2, ax3, ax4]:
                ax.tick_params(axis='x', rotation=45)

            plt.tight_layout()

            # 保存圖表
            if save_path is None:
                save_path = f"charts/{stock_id}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.png"

            # 確保目錄存在
            os.makedirs(os.path.dirname(save_path), exist_ok=True)
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
            plt.close()

            logger.info(f"圖表已保存: {save_path}")
            return save_path

        except Exception as e:
            logger.error(f"創建圖表時發生錯誤: {e}")
            return None

    async def analyze_stock_async(self, session: aiohttp.ClientSession, stock_info: Dict) -> Dict[str, Any]:
        """異步分析單支股票"""
        symbol = stock_info['yahoo_symbol']
        stock_name = stock_info['stock_name']

        try:
            logger.info(f"開始分析股票: {stock_name} ({symbol})")

            # 獲取股票數據
            df = self.fetch_yfinance_data(symbol)
            if df is None or df.empty or len(df) < 60:
                return {
                    'symbol': symbol,
                    'stock_name': stock_name,
                    'success': False,
                    'error': '無法獲取足夠的股票數據'
                }

            # 檢查成交量篩選條件
            if not self.check_volume_filter(df):
                return {
                    'symbol': symbol,
                    'stock_name': stock_name,
                    'success': False,
                    'error': f'成交量不符合條件（需≥{self.config["min_volume_lots"]}張/日）'
                }

                        # 計算技術指標
            df_with_indicators = self.calculate_technical_indicators(df)

            # 進行各項分析
            trend_analysis = self.analyze_trend(df_with_indicators)
            volume_analysis = self.analyze_volume(df_with_indicators)
            technical_analysis = self.analyze_technical_indicators(df_with_indicators)

            # 四象限分析
            quadrant_analysis = self.perform_quadrant_analysis(df_with_indicators)

            # 計算綜合分數
            combined_score = self.calculate_combined_score(trend_analysis, volume_analysis, technical_analysis)

            # 生成投資建議
            recommendation = self.generate_recommendation(combined_score, trend_analysis, technical_analysis)

            # 獲取當前價格資訊
            current_price = float(df['Close'].iloc[-1])
            price_change = trend_analysis.get('price_change_5d', 0)

            result = {
                'symbol': symbol,
                'stock_name': stock_name,
                'stock_id': stock_info.get('stock_id', ''),
                'market': stock_info.get('market', ''),
                'industry': stock_info.get('industry', ''),
                'success': True,
                'current_price': current_price,
                'price_change_5d': price_change,
                'combined_score': combined_score,
                'trend_analysis': trend_analysis,
                'volume_analysis': volume_analysis,
                'technical_analysis': technical_analysis,
                'quadrant_analysis': quadrant_analysis,
                'recommendation': recommendation,
                'data': df_with_indicators,  # 添加數據用於圖表生成
                'analysis_time': datetime.now(taipei_tz).isoformat()
            }

            logger.info(f"完成分析: {stock_name} - 分數: {combined_score:.1f} - 建議: {recommendation['recommendation']} - 象限: {quadrant_analysis.get('quadrant', 'N/A') if quadrant_analysis else 'N/A'}")
            return result

        except Exception as e:
            logger.error(f"分析股票 {symbol} 時發生錯誤: {e}")
            return {
                'symbol': symbol,
                'stock_name': stock_name,
                'success': False,
                'error': str(e)
            }

    async def analyze_all_stocks(self) -> Dict[str, Any]:
        """分析所有股票（加入成交量篩選）"""
        try:
            # 獲取完整股票清單
            if self.taiwan_stocks is None:
                self.taiwan_stocks = self.get_taiwan_stocks()

            if self.taiwan_stocks.empty:
                logger.error("無法獲取股票清單")
                return {}

            logger.info(f"準備分析 {len(self.taiwan_stocks)} 支股票（包含成交量篩選）")

            results = {}
            failed_count = 0
            volume_filtered_count = 0

            # 使用 aiohttp 進行異步分析
            async with aiohttp.ClientSession() as session:
                # 創建分析任務
                tasks = []
                for _, stock_info in self.taiwan_stocks.iterrows():
                    task = self.analyze_stock_async(session, stock_info.to_dict())
                    tasks.append(task)

                # 分批執行任務以避免過載
                batch_size = 10
                for i in range(0, len(tasks), batch_size):
                    batch_tasks = tasks[i:i+batch_size]
                    batch_results = await asyncio.gather(*batch_tasks, return_exceptions=True)

                    # 處理批次結果
                    for result in batch_results:
                        if isinstance(result, Exception):
                            logger.error(f"分析任務異常: {result}")
                            failed_count += 1
                            continue

                        if isinstance(result, dict) and 'symbol' in result:
                            if result.get('success', False):
                                results[result['symbol']] = result
                            else:
                                error_msg = result.get('error', '')
                                if '成交量不符合條件' in error_msg:
                                    volume_filtered_count += 1
                                else:
                                    failed_count += 1

                    # 批次間暫停避免過載
                    if i + batch_size < len(tasks):
                        await asyncio.sleep(2)

            logger.info(f"分析完成統計：")
            logger.info(f"  - 成功分析: {len(results)} 支")
            logger.info(f"  - 成交量篩選淘汰: {volume_filtered_count} 支")
            logger.info(f"  - 其他失敗: {failed_count} 支")

            return results

        except Exception as e:
            logger.error(f"批量分析股票時發生錯誤: {e}")
            return {}

    def analyze_single_stock_with_multi_timeframe(self, stock_id: str) -> Dict[str, Any]:
        """分析單支股票並包含多時間週期分析"""
        try:
            logger.info(f"開始分析股票: {stock_id}")

            # 如果不是yahoo symbol格式，轉換為yahoo symbol
            if not stock_id.endswith(('.TW', '.TWO')):
                yahoo_symbol = f"{stock_id}.TW"
            else:
                yahoo_symbol = stock_id
                stock_id = stock_id.split('.')[0]

            # 獲取股票數據
            df = self.fetch_yfinance_data(yahoo_symbol)
            if df is None or df.empty or len(df) < 60:
                return {
                    'symbol': yahoo_symbol,
                    'stock_id': stock_id,
                    'success': False,
                    'error': '無法獲取足夠的股票數據'
                }

            # 檢查成交量篩選條件
            if not self.check_volume_filter(df):
                return {
                    'symbol': yahoo_symbol,
                    'stock_id': stock_id,
                    'success': False,
                    'error': f'成交量不符合條件（需≥{self.config["min_volume_lots"]}張/日）'
                }

            # 計算技術指標
            df_with_indicators = self.calculate_technical_indicators(df)

            # 進行各項分析
            trend_analysis = self.analyze_trend(df_with_indicators)
            volume_analysis = self.analyze_volume(df_with_indicators)
            technical_analysis = self.analyze_technical_indicators(df_with_indicators)

            # 四象限分析
            quadrant_analysis = self.perform_quadrant_analysis(df_with_indicators)

            # 多時間週期分析
            multi_timeframe_analysis = self.perform_multi_timeframe_analysis(yahoo_symbol)

            # 計算綜合分數
            combined_score = self.calculate_combined_score(trend_analysis, volume_analysis, technical_analysis)

            # 生成投資建議
            recommendation = self.generate_recommendation(combined_score, trend_analysis, technical_analysis)

            # 獲取當前價格資訊
            current_price = float(df['Close'].iloc[-1])
            price_change = trend_analysis.get('price_change_5d', 0)

            result = {
                'symbol': yahoo_symbol,
                'stock_id': stock_id,
                'success': True,
                'current_price': current_price,
                'price_change_5d': price_change,
                'combined_score': combined_score,
                'trend_analysis': trend_analysis,
                'volume_analysis': volume_analysis,
                'technical_analysis': technical_analysis,
                'quadrant_analysis': quadrant_analysis,
                'multi_timeframe_analysis': multi_timeframe_analysis,
                'recommendation': recommendation,
                'data': df_with_indicators,
                'analysis_time': datetime.now(taipei_tz).isoformat()
            }

            logger.info(f"完成分析: {stock_id} - 分數: {combined_score:.1f} - 建議: {recommendation['recommendation']}")
            return result

        except Exception as e:
            logger.error(f"分析股票 {stock_id} 時發生錯誤: {e}")
            return {
                'symbol': yahoo_symbol if 'yahoo_symbol' in locals() else stock_id,
                'stock_id': stock_id,
                'success': False,
                'error': str(e)
            }

# 顯示和通知功能
def display_terminal_results(results: Dict[str, Any], limit: int = 10):
    """在終端顯示分析結果"""
    try:
        if not results:
            print("❌ 未能獲取任何分析結果")
            return

        # 過濾成功的結果並按分數排序
        successful_results = [
            result for result in results.values()
            if result.get('success', False) and result.get('combined_score', 0) > 0
        ]

        successful_results.sort(key=lambda x: x.get('combined_score', 0), reverse=True)

        print("\n" + "=" * 80)
        print("🏆 台股技術分析結果 - 前十名優質股票".center(80))
        print("=" * 80)
        print(f"📈 分析時間: {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"📊 成功分析: {len(successful_results)} 支股票")
        print(f"🔍 成交量篩選條件: ≥1000張/日")
        print("=" * 80)

        if not successful_results:
            print("❌ 沒有符合條件的優質股票")
            return

        # 表格標題
        print(f"{'排名':<4}{'代碼':<8}{'股票名稱':<12}{'評分':<8}{'價格':<10}{'漲跌%':<8}{'建議':<10}{'成交量(張)':<12}")
        print("-" * 80)

        # 顯示前十名
        top_stocks = successful_results[:limit]
        for i, result in enumerate(top_stocks, 1):
            stock_name = result.get('stock_name', 'Unknown')[:10]  # 限制長度
            stock_id = result.get('stock_id', 'Unknown')
            score = result.get('combined_score', 0)
            recommendation = result.get('recommendation', {})
            rec_text = recommendation.get('recommendation', '持有')[:8]  # 限制長度
            current_price = result.get('current_price', 0)
            price_change = result.get('price_change_5d', 0)
            volume_info = result.get('volume_analysis', {})
            avg_volume_lots = volume_info.get('avg_volume_lots', 0)

            print(f"{i:<4}{stock_id:<8}{stock_name:<12}{score:<8.1f}{current_price:<10.2f}{price_change:<+8.2f}{rec_text:<10}{avg_volume_lots:<12.0f}")

        print("=" * 80)

        # 詳細分析前五名
        print("\n📊 詳細分析報告 (前五名):")
        print("=" * 80)

        for i, result in enumerate(top_stocks[:5], 1):
            stock_name = result.get('stock_name', 'Unknown')
            stock_id = result.get('stock_id', 'Unknown')
            score = result.get('combined_score', 0)
            recommendation = result.get('recommendation', {})
            trend_analysis = result.get('trend_analysis', {})
            technical_analysis = result.get('technical_analysis', {})
            volume_analysis = result.get('volume_analysis', {})

            print(f"\n{i}. {stock_name} ({stock_id}) - 評分: {score:.1f}")
            print("-" * 50)
            print(f"💰 當前價格: {result.get('current_price', 0):.2f}")
            print(f"📈 5日漲跌: {result.get('price_change_5d', 0):+.2f}%")
            print(f"🎯 投資建議: {recommendation.get('recommendation', '持有')}")
            print(f"📊 信心度: {recommendation.get('confidence', '中等')}")

            # 技術指標詳情
            print(f"🔍 技術指標:")
            print(f"   RSI: {technical_analysis.get('rsi_value', 50):.1f} ({technical_analysis.get('rsi_signal', '中性')})")
            print(f"   MACD: {technical_analysis.get('macd_signal', '中性')}")
            print(f"   KD: K={technical_analysis.get('k_value', 50):.1f}, D={technical_analysis.get('d_value', 50):.1f}")

            # 趨勢和成交量
            print(f"📊 趨勢分析: {trend_analysis.get('trend', '盤整')}")
            print(f"📈 成交量: {volume_analysis.get('avg_volume_lots', 0):.0f}張/日 ({volume_analysis.get('volume_trend', '正常')})")

            # 建議原因
            reasons = recommendation.get('reasons', [])
            if reasons:
                print(f"💡 建議原因: {', '.join(reasons[:2])}")  # 顯示前兩個原因

        print("\n" + "=" * 80)
        print("⚠️  投資提醒:")
        print("• 本分析僅供參考，投資一定有風險")
        print("• 已篩選成交量≥1000張/日的活躍股票")
        print("• 請務必結合基本面分析做最終投資決策")
        print("• 建議分散投資，控制風險")
        print("=" * 80)

    except Exception as e:
        logger.error(f"顯示終端結果時發生錯誤: {e}")
        print(f"❌ 顯示結果時發生錯誤: {str(e)}")

def format_analysis_message(results: Dict[str, Any], limit: int = 10) -> str:
    """格式化分析結果為通知訊息"""
    try:
        if not results:
            return "❌ 未能獲取任何分析結果"

        # 過濾成功的結果並按分數排序
        successful_results = [
            result for result in results.values()
            if result.get('success', False) and result.get('combined_score', 0) > 0
        ]

        successful_results.sort(key=lambda x: x.get('combined_score', 0), reverse=True)

        if not successful_results:
            return "❌ 沒有符合條件的優質股票"

        # 建立訊息
        message = f"🏆 台股技術分析報告\n"
        message += f"📈 分析時間: {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}\n"
        message += f"📊 成功分析: {len(successful_results)} 支股票\n"
        message += f"🔍 成交量篩選: ≥1000張/日\n\n"

        message += "🏅 前十名優質股票:\n"
        message += "=" * 40 + "\n"

        top_stocks = successful_results[:limit]
        for i, result in enumerate(top_stocks, 1):
            stock_name = result.get('stock_name', 'Unknown')
            stock_id = result.get('stock_id', 'Unknown')
            score = result.get('combined_score', 0)
            current_price = result.get('current_price', 0)
            price_change = result.get('price_change_5d', 0)
            recommendation = result.get('recommendation', {}).get('recommendation', '持有')

            message += f"{i:2d}. {stock_name} ({stock_id})\n"
            message += f"    💰 價格: {current_price:.2f} ({price_change:+.1f}%)\n"
            message += f"    📊 評分: {score:.1f} | 🎯 {recommendation}\n\n"

        # 詳細分析前三名
        message += "\n📊 詳細分析 (前三名):\n"
        message += "=" * 40 + "\n"

        for i, result in enumerate(top_stocks[:3], 1):
            stock_name = result.get('stock_name', 'Unknown')
            stock_id = result.get('stock_id', 'Unknown')
            technical_analysis = result.get('technical_analysis', {})
            quadrant_analysis = result.get('quadrant_analysis', {})

            message += f"\n{i}. {stock_name} ({stock_id})\n"
            message += f"🔍 RSI: {technical_analysis.get('rsi_value', 50):.1f}\n"
            message += f"📈 MACD: {technical_analysis.get('macd_signal', '中性')}\n"
            if quadrant_analysis:
                message += f"🎯 四象限: 第{quadrant_analysis.get('quadrant', 'N/A')}象限\n"

        message += "\n⚠️ 投資提醒:\n"
        message += "• 本分析僅供參考，投資有風險\n"
        message += "• 請結合基本面分析做決策\n"
        message += "• 建議分散投資，控制風險"

        return message

    except Exception as e:
        logger.error(f"格式化訊息時發生錯誤: {e}")
        return f"❌ 格式化訊息時發生錯誤: {str(e)}"

async def send_notification(session: aiohttp.ClientSession, message: str, chart_files: List[str] = None):
    """發送通知到 Telegram 和 Discord"""
    # Telegram 通知
    try:
        # 檢查 Telegram 設定
        if not TELEGRAM_BOT_TOKEN or not TELEGRAM_CHAT_IDS or TELEGRAM_BOT_TOKEN == "YOUR_BOT_TOKEN_HERE":
            logger.warning("Telegram 設定未完成，跳過發送通知")
            print("📱 Telegram 設定未完成，請設定 TELEGRAM_BOT_TOKEN 和 TELEGRAM_CHAT_IDS")
        else:
            # 分割長訊息
            max_length = 4000
            if len(message) > max_length:
                parts = []
                current_part = ""

                for line in message.split('\n'):
                    if len(current_part + line + '\n') > max_length:
                        if current_part:
                            parts.append(current_part.strip())
                            current_part = line + '\n'
                        else:
                            # 如果單行太長，直接截斷
                            parts.append(line[:max_length])
                    else:
                        current_part += line + '\n'

                if current_part:
                    parts.append(current_part.strip())
            else:
                parts = [message]

            # 發送文字訊息給所有 chat_id
            telegram_url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"

            for chat_id in TELEGRAM_CHAT_IDS:
                for i, part in enumerate(parts):
                    if i > 0:
                        part = f"🏆 台股分析報告 (續 {i+1}/{len(parts)})\n\n" + part

                    payload = {
                        'chat_id': chat_id,
                        'text': part
                    }

                    try:
                        async with session.post(telegram_url, json=payload, timeout=20) as response:
                            if response.status == 200:
                                logger.info(f"成功發送 Telegram 通知到 {chat_id} (第 {i+1}/{len(parts)} 部分)")
                            else:
                                error_text = await response.text()
                                logger.error(f"發送 Telegram 通知失敗到 {chat_id} (第 {i+1} 部分): {response.status} - {error_text}")
                    except Exception as e:
                        logger.error(f"發送 Telegram 通知時發生網路錯誤到 {chat_id} (第 {i+1} 部分): {e}")

                    # 避免發送太快
                    if i < len(parts) - 1:
                        await asyncio.sleep(1)

                # 發送圖表文件
                if chart_files:
                    telegram_photo_url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendPhoto"
                    for chart_file in chart_files:
                        if os.path.exists(chart_file):
                            try:
                                with open(chart_file, 'rb') as photo:
                                    files = {'photo': photo}
                                    data = {'chat_id': chat_id}

                                    # 使用 requests 同步發送圖片（因為 aiohttp 處理文件上傳較複雜）
                                    import requests
                                    response = requests.post(telegram_photo_url, files=files, data=data, timeout=30)

                                    if response.status_code == 200:
                                        logger.info(f"成功發送圖表到 Telegram: {chart_file}")
                                    else:
                                        logger.error(f"發送圖表失敗: {response.status_code} - {response.text}")
                            except Exception as e:
                                logger.error(f"發送圖表時發生錯誤: {e}")

    except Exception as e:
        logger.error(f"發送 Telegram 通知時異常: {e}")

    # Discord 通知 (如果可用)
    if MESSAGING_AVAILABLE and DISCORD_WEBHOOK_URL and DISCORD_WEBHOOK_URL != "YOUR_DISCORD_WEBHOOK_URL":
        try:
            webhook = DiscordWebhook(url=DISCORD_WEBHOOK_URL, content=f"```\n{message}\n```")

            # 添加圖表文件
            if chart_files:
                for chart_file in chart_files:
                    if os.path.exists(chart_file):
                        with open(chart_file, "rb") as f:
                            webhook.add_file(file=f.read(), filename=os.path.basename(chart_file))

            response = webhook.execute()
            if response.ok:
                logger.info("Discord 通知發送成功")
            else:
                logger.error(f"Discord 通知失敗: {response.status_code} {response.content}")
        except Exception as e:
            logger.error(f"發送 Discord 通知時異常: {e}")

def run_stock_screening():
    """執行股票篩選功能"""
    print("=" * 80)
    print("台股篩選系統")
    print("=" * 80)
    print("\n篩選條件:")
    print("1. 技術面")
    print("   - MACD: 0 < D-M < 1.5")
    print("   - 本日收盤價高於10日均價且高於30日均價")
    print("   - 本日股價漲幅5%以上")
    print("2. 籌碼面")
    print("   - 本日成交量5000張以上但不超過30日均量的3倍")
    print("   - 由符合篩選條件中取成交量前三大為標的")
    print("=" * 80)

    analyzer = StockAnalyzer()

    # 執行篩選
    screened_stocks = analyzer.screen_stocks()

    if screened_stocks:
        print(f"\n🎯 篩選完成，共找到 {len(screened_stocks)} 支符合條件的股票")

        # 為篩選出的股票生成圖表
        chart_files = []
        for stock_data in screened_stocks:
            chart_path = analyzer.create_stock_chart(stock_data)
            if chart_path:
                chart_files.append(chart_path)

        return screened_stocks, chart_files
    else:
        print("\n❌ 未找到符合篩選條件的股票")
        return [], []

async def main():
    """主程式"""
    try:
        print("🚀 台股技術分析程式啟動")
        print(f"⏰ 開始時間: {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}")
        print("🔍 分析條件: 成交量≥1000張/日")
        print("🏆 目標: 找出前十名優質股票")
        print("=" * 60)

        # 初始化分析器
        analyzer = StockAnalyzer()

        # 執行全部股票分析
        print("📊 開始分析所有台股（包含成交量篩選）...")
        results = await analyzer.analyze_all_stocks()

        if not results:
            error_message = "❌ 分析失敗，未能獲取任何結果"
            print(error_message)

            # 發送錯誤通知
            async with aiohttp.ClientSession() as session:
                await send_notification(session, error_message)
            return

        # 在終端顯示結果
        display_terminal_results(results, limit=10)

        # 為前三名股票生成圖表
        print("\n📊 正在為前三名股票生成技術分析圖表...")
        successful_results = [
            result for result in results.values()
            if result.get('success', False) and result.get('combined_score', 0) > 0
        ]
        successful_results.sort(key=lambda x: x.get('combined_score', 0), reverse=True)

        chart_files = []
        for i, result in enumerate(successful_results[:3]):
            print(f"生成圖表 {i+1}/3: {result.get('stock_name', 'Unknown')}")
            chart_path = analyzer.create_stock_chart(result)
            if chart_path:
                chart_files.append(chart_path)

        # 格式化通知訊息
        print("\n📱 正在準備發送通知...")
        message = format_analysis_message(results, limit=10)

        # 發送通知
        print("📱 正在發送通知到 Telegram 和 Discord...")
        async with aiohttp.ClientSession() as session:
            await send_notification(session, message, chart_files)

        print(f"\n⏰ 程式執行完成: {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}")
        print("=" * 60)
        print("✅ 分析報告已完成並發送")
        print("🏆 前十名優質股票已顯示在上方")
        print("📊 技術分析圖表已生成並發送")

    except Exception as e:
        error_message = f"❌ 主程式執行錯誤: {str(e)}\n\n詳細錯誤:\n{traceback.format_exc()}"
        logger.error(error_message)
        print(error_message)

        # 嘗試發送錯誤通知
        try:
            async with aiohttp.ClientSession() as session:
                await send_notification(session, f"❌ 程式執行錯誤: {str(e)}")
        except Exception as notify_error:
            logger.error(f"發送錯誤通知失敗: {notify_error}")

# 使用範例
def example_usage():
    """使用範例"""
    analyzer = StockAnalyzer()

    print("=" * 80)
    print("台股技術分析系統 - 使用範例")
    print("=" * 80)

    # 範例1: 分析單支股票（包含多時間週期分析）
    print("\n=== 範例1: 單支股票完整分析 ===")
    result = analyzer.analyze_single_stock_with_multi_timeframe("2330")
    if result['success']:
        print(f"股票: {result['stock_id']} - {result.get('stock_name', 'Unknown')}")
        print(f"當前價格: {result['current_price']:.2f}")
        print(f"綜合分數: {result['combined_score']:.1f}")
        print(f"投資建議: {result['recommendation']['recommendation']}")

        # 顯示四象限分析
        if result['quadrant_analysis']:
            quad = result['quadrant_analysis']
            print(f"四象限: 第{quad['quadrant']}象限 ({quad['trend']})")

        # 顯示多時間週期分析
        if result['multi_timeframe_analysis']:
            analyzer.generate_multi_timeframe_summary(result['multi_timeframe_analysis'], result['stock_id'])

                # 生成圖表
        chart_path = analyzer.create_stock_chart(result)
        if chart_path:
            print(f"技術分析圖表已生成: {chart_path}")
    else:
        print(f"分析失敗: {result['error']}")

    # 範例2: 股票篩選功能
    print("\n=== 範例2: 股票篩選功能 ===")
    screened_stocks, chart_files = run_stock_screening()

    # 範例3: 批量分析（注意：這會分析所有股票，可能需要較長時間）
    print("\n=== 範例3: 批量分析 ===")
    print("注意：批量分析會分析所有股票，可能需要較長時間")
    user_input = input("是否執行批量分析？(y/N): ")
    if user_input.lower() == 'y':
        results = asyncio.run(analyzer.analyze_all_stocks())
        print(f"成功分析 {len(results)} 支股票")
        display_terminal_results(results, limit=5)
    else:
        print("跳過批量分析")

# 定時執行功能
def schedule_analysis():
    """定時執行分析"""
    import schedule
    import time

    def job():
        """定時任務"""
        print(f"\n🕐 定時分析開始: {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}")
        asyncio.run(main())
        print(f"🕐 定時分析完成: {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}\n")

    # 設定排程：每天早上9點和下午2點執行
    schedule.every().day.at("09:00").do(job)
    schedule.every().day.at("14:00").do(job)

    print("📅 定時分析已啟動")
    print("⏰ 執行時間: 每天 09:00 和 14:00")
    print("按 Ctrl+C 停止定時執行")

    try:
        while True:
            schedule.run_pending()
            time.sleep(60)  # 每分鐘檢查一次
    except KeyboardInterrupt:
        print("\n📅 定時分析已停止")

# 互動式選單
def interactive_menu():
    """互動式選單"""
    while True:
        print("\n" + "=" * 60)
        print("🏆 台股技術分析系統".center(60))
        print("=" * 60)
        print("1. 📊 分析單支股票（含多時間週期）")
        print("2. 🔍 股票篩選功能")
        print("3. 📈 批量分析所有股票")
        print("4. 📱 發送測試通知")
        print("5. 📅 啟動定時分析")
        print("6. 🔧 設定管理")
        print("7. ❓ 使用說明")
        print("0. 🚪 退出程式")
        print("=" * 60)

        choice = input("請選擇功能 (0-7): ").strip()

        if choice == "1":
            stock_id = input("請輸入股票代碼 (例: 2330): ").strip()
            if stock_id:
                analyzer = StockAnalyzer()
                result = analyzer.analyze_single_stock_with_multi_timeframe(stock_id)
                if result['success']:
                    print(f"\n✅ 分析完成: {result.get('stock_name', 'Unknown')} ({result['stock_id']})")
                    print(f"📊 綜合評分: {result['combined_score']:.1f}")
                    print(f"🎯 投資建議: {result['recommendation']['recommendation']}")

                    # 生成圖表
                    chart_path = analyzer.create_stock_chart(result)
                    if chart_path:
                        print(f"📊 圖表已生成: {chart_path}")

                    # 顯示多時間週期分析
                    if result.get('multi_timeframe_analysis'):
                        show_detail = input("是否顯示詳細多時間週期分析？(y/N): ")
                        if show_detail.lower() == 'y':
                            analyzer.generate_multi_timeframe_summary(
                                result['multi_timeframe_analysis'], result['stock_id'])
                else:
                    print(f"❌ 分析失敗: {result['error']}")
            else:
                print("❌ 請輸入有效的股票代碼")

        elif choice == "2":
            screened_stocks, chart_files = run_stock_screening()
            if chart_files:
                print(f"📊 已生成 {len(chart_files)} 個技術分析圖表")

        elif choice == "3":
            confirm = input("⚠️  批量分析需要較長時間，確定執行？(y/N): ")
            if confirm.lower() == 'y':
                asyncio.run(main())
            else:
                print("❌ 已取消批量分析")

        elif choice == "4":
            test_message = f"🧪 測試通知\n⏰ 時間: {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}\n✅ 通知系統運作正常"
            asyncio.run(send_test_notification(test_message))

        elif choice == "5":
            confirm = input("⚠️  這將啟動定時分析（每天09:00和14:00執行），確定啟動？(y/N): ")
            if confirm.lower() == 'y':
                schedule_analysis()
            else:
                print("❌ 已取消定時分析")

        elif choice == "6":
            settings_menu()

        elif choice == "7":
            show_help()

        elif choice == "0":
            print("👋 感謝使用台股技術分析系統！")
            break

        else:
            print("❌ 無效選擇，請重新輸入")

async def send_test_notification(message: str):
    """發送測試通知"""
    async with aiohttp.ClientSession() as session:
        await send_notification(session, message)
    print("📱 測試通知已發送")

def settings_menu():
    """設定選單"""
    print("\n" + "=" * 50)
    print("🔧 設定管理".center(50))
    print("=" * 50)
    print("1. 📱 Telegram 通知設定")
    print("2. 💬 Discord 通知設定")
    print("3. 📊 分析參數設定")
    print("4. 🔍 篩選條件設定")
    print("0. 🔙 返回主選單")
    print("=" * 50)

    choice = input("請選擇設定項目 (0-4): ").strip()

    if choice == "1":
        print(f"\n📱 當前 Telegram 設定:")
        print(f"Bot Token: {'已設定' if TELEGRAM_BOT_TOKEN != 'YOUR_BOT_TOKEN_HERE' else '未設定'}")
        print(f"Chat IDs: {TELEGRAM_CHAT_IDS if TELEGRAM_CHAT_IDS != ['YOUR_CHAT_ID_HERE'] else '未設定'}")
        print("\n💡 請在程式碼中修改 TELEGRAM_BOT_TOKEN 和 TELEGRAM_CHAT_IDS 變數")

    elif choice == "2":
        print(f"\n💬 當前 Discord 設定:")
        print(f"Webhook URL: {'已設定' if DISCORD_WEBHOOK_URL != 'YOUR_DISCORD_WEBHOOK_URL' else '未設定'}")
        print(f"Discord 模組: {'已安裝' if MESSAGING_AVAILABLE else '未安裝'}")
        print("\n💡 請在程式碼中修改 DISCORD_WEBHOOK_URL 變數")

    elif choice == "3":
        analyzer = StockAnalyzer()
        print(f"\n📊 當前分析參數:")
        print(f"RSI 週期: {analyzer.config['rsi_period']}")
        print(f"MACD 快線: {analyzer.config['macd_fast']}")
        print(f"MACD 慢線: {analyzer.config['macd_slow']}")
        print(f"MACD 信號線: {analyzer.config['macd_signal']}")
        print(f"最小成交量: {analyzer.config['min_volume_lots']} 張/日")
        print("\n💡 如需修改請編輯 StockAnalyzer.__init__ 中的 config")

    elif choice == "4":
        analyzer = StockAnalyzer()
        print(f"\n🔍 當前篩選條件:")
        conditions = analyzer.config['screen_conditions']
        print(f"最小漲幅: {conditions['min_price_change']}%")
        print(f"最小成交量: {conditions['min_volume_lots']} 張")
        print(f"最大成交量倍數: {conditions['max_volume_ratio']} 倍")
        print(f"MACD差值範圍: {conditions['macd_diff_min']} ~ {conditions['macd_diff_max']}")
        print("\n💡 如需修改請編輯 StockAnalyzer.__init__ 中的 screen_conditions")

def show_help():
    """顯示使用說明"""
    help_text = """
📖 台股技術分析系統 - 使用說明

🎯 系統功能:
1. 📊 單股分析: 提供完整技術指標分析，包含多時間週期分析
2. 🔍 股票篩選: 根據技術面和籌碼面條件篩選優質股票
3. 📈 批量分析: 分析所有台股，找出評分最高的前十名
4. 📱 通知功能: 自動發送分析結果到 Telegram 和 Discord
5. 📊 圖表生成: 自動生成技術分析圖表

🔧 技術指標:
• RSI: 相對強弱指標，判斷超買超賣
• MACD: 指數平滑移動平均線，判斷趨勢轉折
• KD: 隨機指標，短期買賣時機
• 布林帶: 價格通道，判斷價格相對位置
• 移動平均線: MA5, MA10, MA20, MA30

🎯 四象限分析:
• 第一象限: 強勢多頭 (MACD>Signal, RSI>50)
• 第二象限: 多頭警戒 (MACD<Signal, RSI>50)
• 第三象限: 強勢空頭 (MACD<Signal, RSI<50)
• 第四象限: 空頭反轉 (MACD>Signal, RSI<50)

🔍 篩選條件:
• 技術面: MACD差值0-1.5, 價格高於均線, 漲幅≥5%
• 籌碼面: 成交量≥5000張且≤30日均量3倍

📱 通知設定:
1. 申請 Telegram Bot Token
2. 獲取 Chat ID
3. 在程式碼中設定相關變數
4. 可選: 設定 Discord Webhook

⚠️  風險提醒:
• 本系統僅供參考，不構成投資建議
• 投資有風險，請謹慎評估
• 建議結合基本面分析
• 請做好風險控制和資金管理

💡 技術支援:
• 確保網路連線正常
• 股票數據來源: Yahoo Finance
• 建議在交易時間外執行批量分析
• 如遇問題請檢查日誌輸出
"""
    print(help_text)

if __name__ == "__main__":
    print("🚀 台股技術分析系統")
    print("=" * 50)
    print("選擇執行模式:")
    print("1. 🖥️  互動式選單")
    print("2. 🔄 直接執行主程式")
    print("3. 📚 查看使用範例")
    print("=" * 50)

    mode = input("請選擇模式 (1-3): ").strip()

    if mode == "1":
        interactive_menu()
    elif mode == "2":
        asyncio.run(main())
    elif mode == "3":
        example_usage()
    else:
        print("❌ 無效選擇，啟動互動式選單")
        interactive_menu()

In [ ]:

import os
import time
import pandas as pd
import numpy as np
import yfinance as yf
import requests
from bs4 import BeautifulSoup
import re
import warnings
import logging
import traceback
from datetime import datetime, timedelta, timezone
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.font_manager import FontProperties
import seaborn as sns
from io import StringIO
import asyncio
import aiohttp
import json
import pickle
from typing import Dict, List, Any, Optional
import random

# 機器學習相關
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# 通知相關
try:
    from discord_webhook import DiscordWebhook
    MESSAGING_AVAILABLE = True
except ImportError:
    MESSAGING_AVAILABLE = False

# Jupyter Notebook 支援
try:
    import nest_asyncio
    nest_asyncio.apply()
    import ipywidgets as widgets
    from ipywidgets import Layout, HTML, Button, HBox, VBox, Output
    from IPython.display import display, clear_output
    JUPYTER_AVAILABLE = True
except ImportError:
    JUPYTER_AVAILABLE = False

# Shioaji 支援
try:
    import shioaji as sj
    SHIOAJI_AVAILABLE = True
except ImportError:
    SHIOAJI_AVAILABLE = False

# =============================================================================
# 全域設定
# =============================================================================

# 禁止中文錯誤警告
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# 設定日誌
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# 中文字體設定
plt.rcParams['font.sans-serif'] = [
    'Microsoft JhengHei',  # 繁體中文
    'SimHei',              # 簡體中文
    'Arial Unicode MS',    # Unicode 字體
    'DejaVu Sans'         # 基礎字體
]
plt.rcParams['axes.unicode_minus'] = False
plt.style.use('seaborn-v0_8-whitegrid')

# 通知設定
TELEGRAM_TOKEN = "7902318521:AAEYoDMqwfHabI7L1SRiE4z33aFay42-VGE"
TELEGRAM_CHAT_ID = [879781796, 8113868436]
DISCORD_WEBHOOK_URL = "https://discord.com/api/webhooks/1362715080734802102/Jma7A3VhEQrrRxIX_JW2l6rATjAZXsGXGfnJuAMqmS1QvqG_2ptg3vr_nsVnuV_PlnBl"

# 台北時區
taipei_tz = timezone(timedelta(hours=8))

# 圖表儲存目錄
CHARTS_DIR = 'charts'
if not os.path.exists(CHARTS_DIR):
    os.makedirs(CHARTS_DIR)

print("🚀 環境設定完成，開始建立分析系統...")

# =============================================================================
# 1. 數據獲取模組
# =============================================================================

def fetch_from_cnn():
    """從CNN網站獲取恐懼與貪婪指數"""
    url = "https://money.cnn.com/data/fear-and-greed/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')

        for element in soup.find_all(['div', 'span']):
            if "Fear & Greed Now:" in element.get_text():
                number_match = re.search(r'(\d+)', element.get_text())
                if number_match:
                    value = int(number_match.group(1))
                    logger.info(f"成功從CNN獲取恐懼與貪婪指數: {value}")
                    return value
        logger.warning("無法從CNN提取恐懼與貪婪指數")
        return None
    except Exception as e:
        logger.error(f"從CNN獲取恐懼與貪婪指數時出錯: {e}")
        return None

def fetch_from_alternative_me(limit=1):
    """從alternative.me API獲取恐懼與貪婪指數"""
    try:
        url = f"https://api.alternative.me/fng/?limit={limit}&format=json"
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()

        if 'data' in data and len(data['data']) > 0:
            if limit == 1:
                value = int(data['data'][0]['value'])
                logger.info(f"成功從Alternative.me API獲取恐懼與貪婪指數: {value}")
                return value
            else:
                df = pd.DataFrame(data['data'])
                df['value'] = pd.to_numeric(df['value'])
                df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s').dt.date
                df = df.rename(columns={'timestamp': 'Date', 'value': 'FearGreedIndex'})
                df['Date'] = pd.to_datetime(df['Date'])
                logger.info(f"成功獲取 {len(df)} 天的歷史恐懼與貪婪指數數據")
                return df[['Date', 'FearGreedIndex']]
        else:
            logger.warning("Alternative.me API返回的數據格式不正確")
            return None
    except Exception as e:
        logger.error(f"從Alternative.me獲取恐懼與貪婪指數時出錯: {e}")
        return None

def get_fear_and_greed_data(days):
    """獲取即時與歷史恐懼與貪婪指數數據"""
    logger.info("正在獲取恐懼與貪婪指數...")

    current_value = fetch_from_cnn()
    if current_value is None:
        current_value = fetch_from_alternative_me(limit=1)

    historical_df = fetch_from_alternative_me(limit=days)

    if historical_df is not None:
        logger.info("已成功獲取真實歷史恐懼與貪婪指數數據。")
        if current_value is not None:
            today = pd.to_datetime(datetime.now().date())
            if today not in historical_df['Date'].values:
                new_row = pd.DataFrame([{'Date': today, 'FearGreedIndex': current_value}])
                historical_df = pd.concat([historical_df, new_row], ignore_index=True)
        return historical_df.drop_duplicates(subset=['Date'], keep='last')

    logger.warning("無法獲取真實歷史數據，將使用模擬數據作為備案。")
    if current_value is None:
        current_value = 50
        logger.warning(f"所有即時來源均失敗，使用預設值 {current_value}")

    dates = pd.to_datetime([datetime.now() - timedelta(days=i) for i in range(days)]).sort_values()
    scores = [current_value]
    np.random.seed(42)
    for _ in range(1, days):
        mean_reversion = 0.1 * (50 - scores[-1])
        random_change = np.random.normal(0, 3)
        new_score = scores[-1] + mean_reversion + random_change
        scores.append(max(0, min(100, new_score)))

    return pd.DataFrame({'Date': dates, 'FearGreedIndex': scores})

def fetch_stock_data(symbol, period='1y'):
    """從Yahoo Finance獲取股票數據，自動判斷台股/美股"""
    original_symbol = symbol

    # 如果輸入的是純數字，判斷為台股代碼
    if re.match(r'^\d{4,5}$', symbol):
        symbol += ".TW"
        logger.info(f"檢測到台股代碼，轉換為 {symbol}")

    logger.info(f"正在從Yahoo Finance獲取 {symbol} ({period}) 的數據...")
    try:
        stock = yf.Ticker(symbol)
        df = stock.history(period=period, interval='1d')
        if df.empty:
            logger.error(f"無法獲取 {symbol} 的數據，請檢查代碼是否正確。")
            return None

        df.reset_index(inplace=True)
        df['Date'] = pd.to_datetime(df['Date'].dt.date)

        logger.info(f"成功獲取 {original_symbol} 的 {len(df)} 筆數據")
        return df
    except Exception as e:
        logger.error(f"從Yahoo Finance獲取 {symbol} 的數據時出錯: {e}")
        return None

# =============================================================================
# 2. 技術指標計算模組
# =============================================================================

def calculate_technical_indicators(df):
    """計算完整的技術指標"""
    # 移動平均線
    df['MA5'] = df['Close'].rolling(window=5).mean()
    df['MA10'] = df['Close'].rolling(window=10).mean()
    df['MA20'] = df['Close'].rolling(window=20).mean()
    df['MA60'] = df['Close'].rolling(window=60).mean()

    # EMA
    df['EMA12'] = df['Close'].ewm(span=12, adjust=False).mean()
    df['EMA26'] = df['Close'].ewm(span=26, adjust=False).mean()

    # MACD
    df['MACD'] = df['EMA12'] - df['EMA26']
    df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_Hist'] = df['MACD'] - df['MACD_Signal']

    # RSI
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean().replace(0, np.nan)
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))

    # 布林帶
    df['BB_Middle'] = df['MA20']
    std_dev = df['Close'].rolling(window=20).std()
    df['BB_Upper'] = df['BB_Middle'] + (std_dev * 2)
    df['BB_Lower'] = df['BB_Middle'] - (std_dev * 2)

    # KD指標
    low_min = df['Low'].rolling(window=9).min()
    high_max = df['High'].rolling(window=9).max()
    rsv = (df['Close'] - low_min) / (high_max - low_min) * 100
    df['K'] = rsv.ewm(com=2).mean()
    df['D'] = df['K'].ewm(com=2).mean()

    # 威廉指標
    df['Williams_R'] = -100 * (high_max - df['Close']) / (high_max - low_min)

    # 成交量指標
    df['Volume_MA'] = df['Volume'].rolling(window=20).mean()
    df['Volume_Ratio'] = df['Volume'] / df['Volume_MA']

    # OBV
    df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).fillna(0).cumsum()

    # ADX (簡化版)
    high_diff = df['High'].diff()
    low_diff = df['Low'].diff()
    plus_dm = np.where((high_diff > low_diff) & (high_diff > 0), high_diff, 0)
    minus_dm = np.where((low_diff > high_diff) & (low_diff > 0), low_diff, 0)
    tr1 = df['High'] - df['Low']
    tr2 = abs(df['High'] - df['Close'].shift(1))
    tr3 = abs(df['Low'] - df['Close'].shift(1))
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = tr.ewm(alpha=1/14, adjust=False).mean()
    plus_di = 100 * (pd.Series(plus_dm).ewm(alpha=1/14, adjust=False).mean() / atr)
    minus_di = 100 * (pd.Series(minus_dm).ewm(alpha=1/14, adjust=False).mean() / atr)
    dx = 100 * (abs(plus_di - minus_di) / (plus_di + minus_di).replace(0, np.nan))
    df['ADX'] = dx.ewm(alpha=1/14, adjust=False).mean()

    # CCI
    typical_price = (df['High'] + df['Low'] + df['Close']) / 3
    sma_tp = typical_price.rolling(window=20).mean()
    mad = typical_price.rolling(window=20).apply(lambda x: np.mean(np.abs(x - np.mean(x))))
    df['CCI'] = (typical_price - sma_tp) / (0.015 * mad)

    # MFI
    typical_price = (df['High'] + df['Low'] + df['Close']) / 3
    money_flow = typical_price * df['Volume']
    mf_sign = np.where(typical_price.diff() > 0, 1, -1)
    signed_mf = money_flow * mf_sign
    positive_mf = np.where(signed_mf > 0, signed_mf, 0)
    negative_mf = np.where(signed_mf < 0, -signed_mf, 0)
    mfr = pd.Series(positive_mf).rolling(14).sum() / pd.Series(negative_mf).rolling(14).sum().replace(0, np.nan)
    df['MFI'] = 100 - (100 / (1 + mfr))

    return df

def feature_engineering(df):
    """創建衍生特徵"""
    # 價格變化
    df['Price_Change_1d'] = df['Close'].pct_change(1)
    df['Price_Change_5d'] = df['Close'].pct_change(5)
    df['Price_Change_20d'] = df['Close'].pct_change(20)

    # 波動率
    df['Volatility_5d'] = df['Close'].rolling(5).std()
    df['Volatility_20d'] = df['Close'].rolling(20).std()

    # 布林帶相關
    df['BB_Width'] = (df['BB_Upper'] - df['BB_Lower']) / df['BB_Middle']
    df['BB_Position'] = (df['Close'] - df['BB_Lower']) / (df['BB_Upper'] - df['BB_Lower']).replace(0, np.nan)

    # 動量指標
    df['Momentum_14'] = df['Close'] - df['Close'].shift(14)

    # 相對強度
    df['Relative_Volume'] = df['Volume'] / df['Volume_MA']

    # 填充NaN值
    df.bfill(inplace=True)
    df.ffill(inplace=True)
    df.fillna(0, inplace=True)
    return df

# =============================================================================
# 3. K線型態識別模組
# =============================================================================

def identify_candlestick_patterns(df):
    """識別常見的K線型態"""
    result_df = df.copy()

    # 基本計算
    result_df['body_size'] = abs(result_df['Close'] - result_df['Open'])
    result_df['upper_shadow'] = result_df['High'] - result_df[['Open', 'Close']].max(axis=1)
    result_df['lower_shadow'] = result_df[['Open', 'Close']].min(axis=1) - result_df['Low']
    result_df['total_range'] = result_df['High'] - result_df['Low']

    # 前幾天數據
    for i in range(1, 4):
        result_df[f'prev{i}_open'] = result_df['Open'].shift(i)
        result_df[f'prev{i}_high'] = result_df['High'].shift(i)
        result_df[f'prev{i}_low'] = result_df['Low'].shift(i)
        result_df[f'prev{i}_close'] = result_df['Close'].shift(i)
        result_df[f'prev{i}_body_size'] = result_df['body_size'].shift(i)

    # K線趨勢
    result_df['is_bullish'] = result_df['Close'] > result_df['Open']
    result_df['is_bearish'] = result_df['Close'] < result_df['Open']

    for i in range(1, 4):
        result_df[f'prev{i}_is_bullish'] = result_df[f'prev{i}_close'] > result_df[f'prev{i}_open']
        result_df[f'prev{i}_is_bearish'] = result_df[f'prev{i}_close'] < result_df[f'prev{i}_open']

    # 移動平均線趨勢判斷
    result_df['uptrend'] = (result_df['MA5'] > result_df['MA20']) & (result_df['Close'] > result_df['MA20'])
    result_df['downtrend'] = (result_df['MA5'] < result_df['MA20']) & (result_df['Close'] < result_df['MA20'])

    # 初始化型態欄位
    pattern_columns = [
        '十字星_中性', '錘子線_看漲', '錘子線_看跌', '流星線_看跌',
        '吞噬_看漲', '吞噬_看跌', '母子線_看漲', '母子線_看跌',
        '晨星_看漲', '暮星_看跌', '三白兵_看漲', '三黑鴉_看跌',
        '穿刺線_看漲', '烏雲蓋頂_看跌', '長腿十字線_中性', '上吊線_看跌',
        '紡錘_中性', '反轉_看漲', '反轉_看跌'
    ]

    for col in pattern_columns:
        result_df[col] = 0

    # 十字星
    doji_condition = result_df['body_size'] <= 0.1 * result_df['total_range']
    result_df.loc[doji_condition, '十字星_中性'] = 1

    # 錘子線
    hammer_condition = (
        (result_df['lower_shadow'] >= 2 * result_df['body_size']) &
        (result_df['upper_shadow'] <= 0.1 * result_df['total_range']) &
        (result_df['body_size'] <= 0.3 * result_df['total_range'])
    )
    result_df.loc[hammer_condition & result_df['downtrend'], '錘子線_看漲'] = 1
    result_df.loc[hammer_condition & result_df['uptrend'], '錘子線_看跌'] = -1

    # 流星線
    shooting_star_condition = (
        (result_df['upper_shadow'] >= 2 * result_df['body_size']) &
        (result_df['lower_shadow'] <= 0.1 * result_df['total_range']) &
        (result_df['body_size'] <= 0.3 * result_df['total_range'])
    )
    result_df.loc[shooting_star_condition & result_df['uptrend'], '流星線_看跌'] = -1

    # 看漲吞噬
    bullish_engulfing = (
        (~result_df['prev1_is_bullish']) &
        result_df['is_bullish'] &
        (result_df['Open'] < result_df['prev1_close']) &
        (result_df['Close'] > result_df['prev1_open'])
    )
    result_df.loc[bullish_engulfing & result_df['downtrend'], '吞噬_看漲'] = 1

    # 看跌吞噬
    bearish_engulfing = (
        result_df['prev1_is_bullish'] &
        (~result_df['is_bullish']) &
        (result_df['Open'] > result_df['prev1_close']) &
        (result_df['Close'] < result_df['prev1_open'])
    )
    result_df.loc[bearish_engulfing & result_df['uptrend'], '吞噬_看跌'] = -1

    # 看漲母子線
    bullish_harami = (
        (~result_df['prev1_is_bullish']) &
        result_df['is_bullish'] &
        (result_df['High'] < result_df['prev1_high']) &
        (result_df['Low'] > result_df['prev1_low']) &
        (result_df['body_size'] < result_df['prev1_body_size'])
    )
    result_df.loc[bullish_harami & result_df['downtrend'], '母子線_看漲'] = 1

    # 看跌母子線
    bearish_harami = (
        result_df['prev1_is_bullish'] &
        (~result_df['is_bullish']) &
        (result_df['High'] < result_df['prev1_high']) &
        (result_df['Low'] > result_df['prev1_low']) &
        (result_df['body_size'] < result_df['prev1_body_size'])
    )
    result_df.loc[bearish_harami & result_df['uptrend'], '母子線_看跌'] = -1

    # 晨星
    morning_star = (
        (~result_df['prev2_is_bullish']) &
        (result_df['prev1_body_size'] <= 0.3 * result_df['prev1_total_range']) &
        result_df['is_bullish'] &
        (result_df['Close'] > (result_df['prev2_open'] + result_df['prev2_close']) / 2)
    )
    result_df.loc[morning_star & result_df['downtrend'], '晨星_看漲'] = 1

    # 暮星
    evening_star = (
        result_df['prev2_is_bullish'] &
        (result_df['prev1_body_size'] <= 0.3 * result_df['prev1_total_range']) &
        (~result_df['is_bullish']) &
        (result_df['Close'] < (result_df['prev2_open'] + result_df['prev2_close']) / 2)
    )
    result_df.loc[evening_star & result_df['uptrend'], '暮星_看跌'] = -1

    # 三白兵
    three_white_soldiers = (
        result_df['is_bullish'] &
        result_df['prev1_is_bullish'] &
        result_df['prev2_is_bullish'] &
        (result_df['Close'] > result_df['prev1_close']) &
        (result_df['prev1_close'] > result_df['prev2_close']) &
        (result_df['Open'] > result_df['prev1_open']) &
        (result_df['prev1_open'] > result_df['prev2_open'])
    )
    result_df.loc[three_white_soldiers, '三白兵_看漲'] = 1

    # 三黑鴉
    three_black_crows = (
        (~result_df['is_bullish']) &
        (~result_df['prev1_is_bullish']) &
        (~result_df['prev2_is_bullish']) &
        (result_df['Close'] < result_df['prev1_close']) &
        (result_df['prev1_close'] < result_df['prev2_close']) &
        (result_df['Open'] < result_df['prev1_open']) &
        (result_df['prev1_open'] < result_df['prev2_open'])
    )
    result_df.loc[three_black_crows, '三黑鴉_看跌'] = -1

    return result_df

def generate_pattern_report(df, periods=['daily']):
    """生成K線型態分析報告"""
    if df is None or df.empty:
        return "無法生成報告：數據為空"

    required_cols = ['Open', 'High', 'Low', 'Close']
    if not all(col in df.columns for col in required_cols):
        return "無法生成報告：缺少必要欄位"

    def detect_patterns(ohlc_data):
        patterns = {"看漲": [], "看跌": [], "中性": []}

        if len(ohlc_data) < 5:
            return patterns

        recent = ohlc_data.tail(10).copy()
        recent['body_size'] = abs(recent['Close'] - recent['Open'])
        recent['upper_shadow'] = recent['High'] - recent[['Open', 'Close']].max(axis=1)
        recent['lower_shadow'] = recent[['Open', 'Close']].min(axis=1) - recent['Low']
        recent['total_range'] = recent['High'] - recent['Low']
        recent['is_bullish'] = recent['Close'] > recent['Open']
        recent['is_bearish'] = recent['Close'] < recent['Open']

        # 計算移動平均線
        recent['ma5'] = recent['Close'].rolling(window=5).mean()
        recent['ma10'] = recent['Close'].rolling(window=10).mean()

        last3 = recent.tail(3)

        # 三白兵
        if len(last3) == 3 and all(last3['is_bullish']) and \
           last3['Close'].iloc[0] < last3['Close'].iloc[1] < last3['Close'].iloc[2]:
            patterns["看漲"].append("三白兵")

        # 三黑鴉
        if len(last3) == 3 and all(last3['is_bearish']) and \
           last3['Close'].iloc[0] > last3['Close'].iloc[1] > last3['Close'].iloc[2]:
            patterns["看跌"].append("三黑鴉")

        # 多頭吞噬
        if len(last3) >= 2 and last3['is_bearish'].iloc[-2] and last3['is_bullish'].iloc[-1] and \
           last3['Open'].iloc[-1] <= last3['Close'].iloc[-2] and \
           last3['Close'].iloc[-1] >= last3['Open'].iloc[-2]:
            patterns["看漲"].append("多頭吞噬")

        # 空頭吞噬
        if len(last3) >= 2 and last3['is_bullish'].iloc[-2] and last3['is_bearish'].iloc[-1] and \
           last3['Open'].iloc[-1] >= last3['Close'].iloc[-2] and \
           last3['Close'].iloc[-1] <= last3['Open'].iloc[-2]:
            patterns["看跌"].append("空頭吞噬")

        # 看漲錘頭
        if any(last3['lower_shadow'] > 2 * last3['body_size']) and \
           any(last3['upper_shadow'] < 0.2 * last3['body_size']) and \
           any(last3['is_bullish']):
            patterns["看漲"].append("看漲錘頭")

        # 看跌吊錘
        if any(last3['upper_shadow'] > 2 * last3['body_size']) and \
           any(last3['lower_shadow'] < 0.2 * last3['body_size']) and \
           any(last3['is_bearish']):
            patterns["看跌"].append("看跌吊錘")

        # 十字星
        if any(last3['body_size'] < 0.1 * last3['total_range']):
            patterns["中性"].append("十字星")

        # 長腿十字
        if any((last3['body_size'] < 0.1 * last3['total_range']) &
               (last3['upper_shadow'] > last3['body_size']) &
               (last3['lower_shadow'] > last3['body_size'])):
            patterns["中性"].append("長腿十字")

        # 紡錘
        if any((last3['body_size'] < 0.3 * last3['total_range']) &
               (abs(last3['upper_shadow'] - last3['lower_shadow']) < 0.2 * last3['total_range'])):
            patterns["中性"].append("紡錘")

        return patterns

    report_parts = []

    for period in periods:
        period_name = {'daily': '日線', 'weekly': '週線', 'monthly': '月線'}.get(period, period)

        try:
            patterns = detect_patterns(df)
            period_report = [f"{period_name}分析:"]

            for pattern_type, detected_patterns in patterns.items():
                if detected_patterns:
                    pattern_str = ", ".join(detected_patterns)
                    period_report.append(f"{pattern_type}型態: {pattern_str}")
                else:
                    period_report.append(f"{pattern_type}型態: 無")

            report_parts.append("\n".join(period_report))

        except Exception as e:
            logger.error(f"分析 {period_name} 型態時出錯: {e}")
            report_parts.append(f"{period_name}分析:\n分析時發生錯誤")

    return "\n\n".join(report_parts) if report_parts else "未檢測到明顯K線型態"

# =============================================================================
# 4. 波浪理論分析模組
# =============================================================================

def wave_theory_analysis(df, stock_id):
    """完整的波浪理論分析"""
    close = df['Close'].values
    high = df['High'].values
    low = df['Low'].values

    if len(close) < 30:
        return False, "數據不足", None

    # 尋找最近的顯著高點作為可能的3浪頂點
    window = min(20, len(high) - 1)
    peaks = []
    troughs = []

    # 找出所有局部高點和低點
    for i in range(window, len(high) - window):
        # 檢查是否為局部高點
        if high[i] == max(high[i-window:i+window+1]):
            peaks.append((i, high[i]))
        # 檢查是否為局部低點
        if low[i] == min(low[i-window:i+window+1]):
            troughs.append((i, low[i]))

    # 如果沒有找到足夠的波峰波谷
    if len(peaks) < 2 or len(troughs) < 2:
        return False, "無法識別足夠的波峰波谷", None

    # 按時間排序
    peaks.sort(key=lambda x: x[0])
    troughs.sort(key=lambda x: x[0])

    # 尋找可能的3浪高點（最後或倒數第二個高點）
    potential_wave3_peaks = []

    # 檢查最後一個高點
    if peaks[-1][0] > troughs[-1][0]:  # 最後一個高點在最後一個低點之後
        wave3_peak_idx, wave3_peak = peaks[-1]

        # 尋找3浪起點（前一個低點）
        for i in range(len(troughs) - 1, -1, -1):
            if troughs[i][0] < wave3_peak_idx:
                wave3_start_idx, wave3_start = troughs[i]
                break
        else:
            wave3_start_idx, wave3_start = 0, low[0]

        wave3_rise = wave3_peak - wave3_start
        current_price = close[-1]
        retracement = (wave3_peak - current_price) / wave3_rise if wave3_rise > 0 else 0

        if 0.10 <= retracement <= 0.618:  # 回撤比例
            potential_wave3_peaks.append((wave3_peak_idx, wave3_peak, wave3_start_idx, wave3_start, retracement))

    # 檢查倒數第二個高點
    if len(peaks) >= 2:
        wave3_peak_idx, wave3_peak = peaks[-2]

        # 尋找3浪起點（前一個低點）
        for i in range(len(troughs) - 1, -1, -1):
            if troughs[i][0] < wave3_peak_idx:
                wave3_start_idx, wave3_start = troughs[i]
                break
        else:
            wave3_start_idx, wave3_start = 0, low[0]

        wave3_rise = wave3_peak - wave3_start
        current_price = close[-1]
        retracement = (wave3_peak - current_price) / wave3_rise if wave3_rise > 0 else 0

        if 0.10 <= retracement <= 0.618:
            potential_wave3_peaks.append((wave3_peak_idx, wave3_peak, wave3_start_idx, wave3_start, retracement))

    # 如果沒有找到可能的3浪高點
    if not potential_wave3_peaks:
        return False, "無法識別符合回撤比例的3浪高點", None

    # 選擇最佳的3浪高點（選擇回撤比例最接近0.382的）
    best_match = min(potential_wave3_peaks, key=lambda x: abs(x[4] - 0.382))
    wave3_peak_idx, wave3_peak, wave3_start_idx, wave3_start, retracement = best_match

    # 檢查是否有1浪高點
    if wave3_start_idx > window:
        # 尋找1浪高點（在3浪起點之前的高點）
        wave1_candidates = [(i, h) for i, h in peaks if i < wave3_start_idx]
        if wave1_candidates:
            wave1_peak_idx, wave1_peak = max(wave1_candidates, key=lambda x: x[1])

            # 檢查4浪是否與1浪重疊
            if close[-1] > wave1_peak:
                wave_data = {
                    'wave3_peak': wave3_peak,
                    'wave3_start': wave3_start,
                    'retracement': retracement,
                    'current_price': close[-1],
                    'wave1_peak': wave1_peak
                }
                return True, f"4浪回撤比例: {retracement:.2%}, 未與1浪重疊", wave_data
            else:
                return False, f"4浪與1浪重疊，回撤比例: {retracement:.2%}", None
        else:
            # 如果找不到1浪高點，僅依據回撤比例判斷
            wave_data = {
                'wave3_peak': wave3_peak,
                'wave3_start': wave3_start,
                'retracement': retracement,
                'current_price': close[-1]
            }
            return True, f"4浪回撤比例: {retracement:.2%}（無法確認與1浪關係）", wave_data
    else:
        wave_data = {
            'wave3_peak': wave3_peak,
            'wave3_start': wave3_start,
            'retracement': retracement,
            'current_price': close[-1]
        }
        return True, f"4浪回撤比例: {retracement:.2%}（無法確認與1浪關係）", wave_data

# =============================================================================
# 5. 機器學習預測模組
# =============================================================================

class StockPredictor:
    def __init__(self, target_days=7):
        self.models = {
            '線性迴歸': LinearRegression(),
            '隨機森林': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
            'XGBoost': XGBRegressor(n_estimators=100, random_state=42, n_jobs=-1, verbosity=0),
            'LightGBM': LGBMRegressor(n_estimators=100, random_state=42, n_jobs=-1, verbose=-1)
        }
        self.scalers = {}
        self.feature_names = None
        self.target_days = target_days
        self.backtest_results = {}
        self.future_predictions = {}
        self.feature_importances = None

    def _prepare_data(self, df, predict_day):
        """準備特徵和目標變量"""
        feature_columns = [
            'MA5', 'MA10', 'MA20', 'MA60', 'MACD', 'MACD_Signal', 'RSI',
            'BB_Width', 'BB_Position', 'K', 'D', 'Williams_R', 'ADX', 'CCI', 'MFI',
            'Price_Change_1d', 'Price_Change_5d', 'Price_Change_20d',
            'Volatility_5d', 'Volatility_20d', 'Momentum_14', 'Relative_Volume',
            'FearGreedIndex'
        ]
        self.feature_names = [col for col in feature_columns if col in df.columns]

        X = df[self.feature_names].copy()
        y = df['Close'].shift(-predict_day)

        valid_idx = ~y.isnull()
        X = X[valid_idx]
        y = y[valid_idx]

        return X, y

    def run_backtest(self, df):
        """執行回測"""
        logger.info("執行回測...")
        X, y = self._prepare_data(df, predict_day=1)

        if len(X) < 50:
            logger.warning("數據量不足，跳過回測")
            return

        split_index = int(len(X) * 0.8)
        X_train, X_test = X[:split_index], X[split_index:]
        y_train, y_test = y[:split_index], y[split_index:]

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        for name, model in self.models.items():
            try:
                model.fit(X_train_scaled, y_train)
                y_pred = model.predict(X_test_scaled)
                mse = mean_squared_error(y_test, y_pred)
                r2 = r2_score(y_test, y_pred)
                self.backtest_results[name] = {
                    'y_test': y_test, 'y_pred': y_pred, 'mse': mse, 'r2': r2
                }
                logger.info(f"[回測] {name} - R²: {r2:.4f}, MSE: {mse:.4f}")
            except Exception as e:
                logger.error(f"模型 {name} 訓練失敗: {e}")

        # 特徵重要性
        if '隨機森林' in self.models and hasattr(self.models['隨機森林'], 'feature_importances_'):
            self.feature_importances = pd.DataFrame({
                '特徵': self.feature_names,
                '重要性': self.models['隨機森林'].feature_importances_
            }).sort_values('重要性', ascending=False)

    def predict_future(self, df):
        """預測未來價格"""
        logger.info("訓練完整數據集並預測未來價格...")

        for day in range(1, self.target_days + 1):
            X, y = self._prepare_data(df, predict_day=day)

            if X.empty or len(X) < 20:
                logger.warning(f"無法為預測第 {day} 天準備數據，跳過。")
                continue

            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X)
            X_last = df[self.feature_names].iloc[-1:].copy()
            X_last_scaled = scaler.transform(X_last)

            daily_predictions = {}
            for name, model in self.models.items():
                try:
                    model.fit(X_scaled, y)
                    prediction = model.predict(X_last_scaled)[0]
                    daily_predictions[name] = prediction
                except Exception as e:
                    logger.error(f"模型 {name} 預測第 {day} 天失敗: {e}")

            if daily_predictions:
                self.future_predictions[day] = daily_predictions

# =============================================================================
# 6. 綜合評分系統
# =============================================================================

def calculate_combined_score(df):
    """計算綜合評分"""
    if df.empty:
        return 50

    scores = []
    latest = df.iloc[-1]

    # 技術指標評分
    if 'RSI' in df.columns and not pd.isna(latest['RSI']):
        rsi = latest['RSI']
        if 30 <= rsi <= 50:
            scores.append(70)  # 潛在反彈區
        elif 50 < rsi <= 70:
            scores.append(60)  # 強勢區
        elif rsi > 70:
            scores.append(30)  # 超買
        elif rsi < 30:
            scores.append(80)  # 超賣反彈
        else:
            scores.append(50)

    # 移動平均線評分
    if all(col in df.columns for col in ['MA5', 'MA20', 'MA60']):
        ma5, ma20, ma60 = latest['MA5'], latest['MA20'], latest['MA60']
        if not any(pd.isna([ma5, ma20, ma60])):
            if ma5 > ma20 > ma60:
                scores.append(80)  # 多頭排列
            elif ma5 > ma20:
                scores.append(65)  # 短期多頭
            elif ma5 < ma20 < ma60:
                scores.append(20)  # 空頭排列
            else:
                scores.append(45)

    # MACD評分
    if all(col in df.columns for col in ['MACD', 'MACD_Signal', 'MACD_Hist']):
        macd, signal, hist = latest['MACD'], latest['MACD_Signal'], latest['MACD_Hist']
        if not any(pd.isna([macd, signal, hist])):
            if macd > signal and hist > 0:
                scores.append(75)  # 多頭信號
            elif macd > signal:
                scores.append(60)  # 潛在多頭
            elif macd < signal and hist < 0:
                scores.append(25)  # 空頭信號
            else:
                scores.append(45)

    # KD指標評分
    if all(col in df.columns for col in ['K', 'D']):
        k, d = latest['K'], latest['D']
        if not any(pd.isna([k, d])):
            if k < 20 and d < 20:
                scores.append(75)  # 超賣
            elif k > 80 and d > 80:
                scores.append(25)  # 超買
            elif k > d:
                scores.append(60)  # 黃金交叉
            else:
                scores.append(40)  # 死亡交叉

    # 布林帶評分
    if 'BB_Position' in df.columns and not pd.isna(latest['BB_Position']):
        bb_pos = latest['BB_Position']
        if bb_pos < 0.2:
            scores.append(75)  # 接近下軌
        elif bb_pos > 0.8:
            scores.append(25)  # 接近上軌
        else:
            scores.append(50)

    # 價格趨勢評分
    if 'Price_Change_5d' in df.columns and not pd.isna(latest['Price_Change_5d']):
        change = latest['Price_Change_5d']
        if change > 0.1:
            scores.append(80)  # 強勢上漲
        elif change > 0.05:
            scores.append(65)  # 溫和上漲
        elif change > 0:
            scores.append(55)  # 微幅上漲
        elif change > -0.05:
            scores.append(45)  # 微幅下跌
        elif change > -0.1:
            scores.append(35)  # 溫和下跌
        else:
            scores.append(20)  # 大幅下跌

    # 成交量評分
    if 'Relative_Volume' in df.columns and not pd.isna(latest['Relative_Volume']):
        vol_ratio = latest['Relative_Volume']
        if vol_ratio > 2:
            scores.append(70)  # 爆量
        elif vol_ratio > 1.5:
            scores.append(60)  # 放量
        elif vol_ratio < 0.5:
            scores.append(40)  # 縮量
        else:
            scores.append(50)

    # 恐懼貪婪指數評分
    if 'FearGreedIndex' in df.columns and not pd.isna(latest['FearGreedIndex']):
        fgi = latest['FearGreedIndex']
        if fgi < 25:
            scores.append(75)  # 極度恐懼，反彈機會
        elif fgi < 45:
            scores.append(65)  # 恐懼，潛在機會
        elif fgi > 75:
            scores.append(30)  # 極度貪婪，風險高
        elif fgi > 55:
            scores.append(40)  # 貪婪，謹慎
        else:
            scores.append(50)  # 中性

    return np.mean(scores) if scores else 50

# =============================================================================
# 7. 智能建議系統
# =============================================================================

def generate_recommendation(combined_score, pattern_report=None, wave_analysis=None):
    """生成投資建議"""
    try:
        combined_score = float(combined_score)
    except (ValueError, TypeError):
        combined_score = 50.0

    recommendation = {
        "action": "中立觀察",
        "reason": f"綜合評分中性 ({combined_score:.2f})。",
        "pattern_details": [],
        "confidence": 50,
        "risk_level": "中等"
    }

    # 基於評分的建議
    if combined_score >= 80:
        recommendation["action"] = "強烈買入"
        recommendation["reason"] = f"綜合評分極佳 ({combined_score:.2f})，多項指標顯示強勢。"
        recommendation["risk_level"] = "低"
    elif combined_score >= 70:
        recommendation["action"] = "買入"
        recommendation["reason"] = f"綜合評分良好 ({combined_score:.2f})，技術面偏多。"
        recommendation["risk_level"] = "低-中等"
    elif combined_score >= 60:
        recommendation["action"] = "買入觀望"
        recommendation["reason"] = f"綜合評分偏強 ({combined_score:.2f})，可考慮分批進場。"
        recommendation["risk_level"] = "中等"
    elif combined_score <= 20:
        recommendation["action"] = "強烈賣出"
        recommendation["reason"] = f"綜合評分極差 ({combined_score:.2f})，多項指標顯示弱勢。"
        recommendation["risk_level"] = "高"
    elif combined_score <= 30:
        recommendation["action"] = "賣出"
        recommendation["reason"] = f"綜合評分疲弱 ({combined_score:.2f})，技術面偏空。"
        recommendation["risk_level"] = "中等-高"
    elif combined_score <= 40:
        recommendation["action"] = "賣出觀望"
        recommendation["reason"] = f"綜合評分偏弱 ({combined_score:.2f})，建議減倉。"
        recommendation["risk_level"] = "中等"

    # 整合K線型態分析
    if pattern_report and "看漲" in pattern_report:
        if combined_score >= 60:
            recommendation["pattern_details"].append("K線型態支持看漲觀點")
        else:
            recommendation["pattern_details"].append("K線型態看漲但其他指標偏弱")

    if pattern_report and "看跌" in pattern_report:
        if combined_score <= 40:
            recommendation["pattern_details"].append("K線型態支持看跌觀點")
        else:
            recommendation["pattern_details"].append("K線型態看跌但其他指標偏強")

    # 整合波浪理論分析
    if wave_analysis and wave_analysis[0]:  # 如果波浪分析為True
        recommendation["pattern_details"].append(f"波浪理論: {wave_analysis[1]}")
        if "4浪回撤" in wave_analysis[1]:
            if combined_score >= 60:
                recommendation["action"] = "買入"
                recommendation["reason"] += " 波浪理論支持5浪上漲。"

    # 計算信心度
    distance_from_center = abs(combined_score - 50)
    base_confidence = min(distance_from_center * 2, 100)

    # 根據型態分析調整信心度
    if recommendation["pattern_details"]:
        base_confidence = min(base_confidence + 10, 100)

    recommendation['confidence'] = int(base_confidence)

    return recommendation

# =============================================================================
# 8. 可視化模組
# =============================================================================

class StockVisualizer:
    def __init__(self, symbol):
        self.symbol = symbol

    def plot_stock_analysis(self, df):
        """繪製股票技術分析圖"""
        fig, axes = plt.subplots(4, 1, figsize=(16, 16), sharex=True)
        fig.suptitle(f'{self.symbol} 完整技術分析總覽', fontsize=20, fontweight='bold')

        # 價格與移動平均線、布林帶
        axes[0].plot(df['Date'], df['Close'], label='收盤價', linewidth=2, color='black')
        if 'MA5' in df.columns:
            axes[0].plot(df['Date'], df['MA5'], label='MA5', alpha=0.8, color='blue')
        if 'MA20' in df.columns:
            axes[0].plot(df['Date'], df['MA20'], label='MA20', alpha=0.8, color='red')
        if 'MA60' in df.columns:
            axes[0].plot(df['Date'], df['MA60'], label='MA60', alpha=0.8, color='green')
        if 'BB_Upper' in df.columns:
            axes[0].plot(df['Date'], df['BB_Upper'], label='布林上軌', linestyle='--', alpha=0.7, color='gray')
            axes[0].plot(df['Date'], df['BB_Lower'], label='布林下軌', linestyle='--', alpha=0.7, color='gray')
            axes[0].fill_between(df['Date'], df['BB_Lower'], df['BB_Upper'], alpha=0.1, color='gray')
        axes[0].set_title('價格走勢與移動平均線')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)

        # MACD
        if 'MACD' in df.columns:
            axes[1].plot(df['Date'], df['MACD'], label='MACD', color='blue')
            axes[1].plot(df['Date'], df['MACD_Signal'], label='訊號線', color='red', linestyle='--')
            if 'MACD_Hist' in df.columns:
                colors = ['green' if x >= 0 else 'red' for x in df['MACD_Hist']]
                axes[1].bar(df['Date'], df['MACD_Hist'], label='MACD柱狀圖', color=colors, alpha=0.6)
        axes[1].set_title('MACD')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        axes[1].axhline(0, color='black', linestyle='-', alpha=0.3)

        # RSI 和 KD
        if 'RSI' in df.columns:
            axes[2].plot(df['Date'], df['RSI'], label='RSI', color='purple', linewidth=2)
            axes[2].axhline(70, color='red', linestyle='--', alpha=0.7, label='超買區 (70)')
            axes[2].axhline(30, color='green', linestyle='--', alpha=0.7, label='超賣區 (30)')
            axes[2].axhline(50, color='black', linestyle='-', alpha=0.3)
        if 'K' in df.columns and 'D' in df.columns:
            axes[2].plot(df['Date'], df['K'], label='K值', color='orange', alpha=0.8)
            axes[2].plot(df['Date'], df['D'], label='D值', color='brown', alpha=0.8)
        axes[2].set_title('RSI 與 KD 指標')
        axes[2].legend()
        axes[2].grid(True, alpha=0.3)
        axes[2].set_ylim(0, 100)

        # 成交量
        volume_colors = ['green' if df['Close'].iloc[i] >= df['Open'].iloc[i] else 'red'
                        for i in range(len(df))]
        axes[3].bar(df['Date'], df['Volume'], color=volume_colors, alpha=0.6, label='成交量')
        if 'Volume_MA' in df.columns:
            axes[3].plot(df['Date'], df['Volume_MA'], label='成交量均線', color='blue', linewidth=2)
        axes[3].set_title('成交量')
        axes[3].legend()
        axes[3].grid(True, alpha=0.3)

        plt.tight_layout(rect=[0, 0, 1, 0.96])
        return fig

    def plot_future_prediction(self, last_price, last_date, predictions):
        """繪製未來預測圖"""
        fig, ax = plt.subplots(figsize=(12, 8))
        pred_dates, pred_prices_avg, pred_prices_std = [], [], []

        current_date = last_date
        for day, preds in predictions.items():
            while True:
                current_date += timedelta(days=1)
                if current_date.weekday() < 5:  # 工作日
                    break

            pred_dates.append(current_date)
            prices = list(preds.values())
            avg_price = np.mean(prices)
            std_price = np.std(prices)
            pred_prices_avg.append(avg_price)
            pred_prices_std.append(std_price)

        # 繪製平均預測價格
        ax.plot(pred_dates, pred_prices_avg, 'ro-', label='平均預測價格', linewidth=2, markersize=8)

       # 繪製信心區間
        upper_bound = np.array(pred_prices_avg) + np.array(pred_prices_std)
        lower_bound = np.array(pred_prices_avg) - np.array(pred_prices_std)
        ax.fill_between(pred_dates, lower_bound, upper_bound, alpha=0.3, color='blue', label='預測區間')

        # 當前價格線
        ax.axhline(last_price, color='gray', linestyle='--', label=f'當前價格 ({last_price:.2f})', linewidth=2)

        # 標註預測價格
        for date, price, std in zip(pred_dates, pred_prices_avg, pred_prices_std):
            change_pct = (price / last_price - 1) * 100
            ax.text(date, price, f'{price:.2f}\n({change_pct:+.1f}%)',
                   ha='center', va='bottom', fontsize=10, fontweight='bold')

        ax.set_title(f'{self.symbol} 未來一週股價預測', fontsize=16, fontweight='bold')
        ax.set_xlabel('日期')
        ax.set_ylabel('預測價格')
        ax.legend()
        ax.grid(True, alpha=0.3)

        # 格式化日期軸
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))
        ax.xaxis.set_major_locator(mdates.DayLocator(interval=1))
        plt.xticks(rotation=45)

        plt.tight_layout()
        return fig

    def plot_feature_importance(self, importances):
        """繪製特徵重要性圖"""
        fig, ax = plt.subplots(figsize=(12, 10))
        top_15 = importances.head(15)

        bars = ax.barh(top_15['特徵'], top_15['重要性'], color='skyblue', edgecolor='navy', alpha=0.7)
        ax.invert_yaxis()
        ax.set_title('特徵重要性排名 (隨機森林)', fontsize=16, fontweight='bold')
        ax.set_xlabel('重要性')

        # 添加數值標籤
        for bar, value in zip(bars, top_15['重要性']):
            ax.text(bar.get_width(), bar.get_y() + bar.get_height()/2,
                   f'{value:.3f}', ha='left', va='center', fontweight='bold')

        ax.grid(True, alpha=0.3, axis='x')
        plt.tight_layout()
        return fig

# =============================================================================
# 9. 通知系統
# =============================================================================

async def send_notification(session: aiohttp.ClientSession, message: str, chart_files: List[str] = None):
    """發送通知到 Telegram 和 Discord"""
    # Telegram 通知
    try:
        if TELEGRAM_TOKEN and TELEGRAM_CHAT_ID:
            max_length = 4000
            if len(message) > max_length:
                parts = []
                current_part = ""
                for line in message.split('\n'):
                    if len(current_part + line + '\n') > max_length:
                        if current_part:
                            parts.append(current_part.strip())
                            current_part = line + '\n'
                        else:
                            parts.append(line[:max_length])
                    else:
                        current_part += line + '\n'
                if current_part:
                    parts.append(current_part.strip())
            else:
                parts = [message]

            telegram_url = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage"

            for chat_id in TELEGRAM_CHAT_ID:
                for i, part in enumerate(parts):
                    if i > 0:
                        part = f"🏆 股票分析報告 (續 {i+1}/{len(parts)})\n\n" + part

                    payload = {'chat_id': chat_id, 'text': part}

                    try:
                        async with session.post(telegram_url, json=payload, timeout=20) as response:
                            if response.status == 200:
                                logger.info(f"成功發送 Telegram 通知到 {chat_id}")
                            else:
                                error_text = await response.text()
                                logger.error(f"發送 Telegram 通知失敗: {response.status} - {error_text}")
                    except Exception as e:
                        logger.error(f"發送 Telegram 通知時發生網路錯誤: {e}")

                    if i < len(parts) - 1:
                        await asyncio.sleep(1)

                # 發送圖表文件
                if chart_files:
                    telegram_photo_url = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendPhoto"
                    for chart_file in chart_files:
                        if os.path.exists(chart_file):
                            try:
                                with open(chart_file, 'rb') as photo:
                                    files = {'photo': photo}
                                    data = {'chat_id': chat_id}

                                    import requests
                                    response = requests.post(telegram_photo_url, files=files, data=data, timeout=30)

                                    if response.status_code == 200:
                                        logger.info(f"成功發送圖表到 Telegram: {chart_file}")
                                    else:
                                        logger.error(f"發送圖表失敗: {response.status_code}")
                            except Exception as e:
                                logger.error(f"發送圖表時發生錯誤: {e}")

    except Exception as e:
        logger.error(f"發送 Telegram 通知時異常: {e}")

    # Discord 通知
    if MESSAGING_AVAILABLE and DISCORD_WEBHOOK_URL:
        try:
            webhook = DiscordWebhook(url=DISCORD_WEBHOOK_URL, content=f"```\n{message}\n```")

            if chart_files:
                for chart_file in chart_files:
                    if os.path.exists(chart_file):
                        with open(chart_file, "rb") as f:
                            webhook.add_file(file=f.read(), filename=os.path.basename(chart_file))

            response = webhook.execute()
            if response.ok:
                logger.info("Discord 通知發送成功")
            else:
                logger.error(f"Discord 通知失敗: {response.status_code}")
        except Exception as e:
            logger.error(f"發送 Discord 通知時異常: {e}")

def format_analysis_message(results: Dict[str, Any], limit: int = 10) -> str:
    """格式化分析結果為通知訊息"""
    try:
        if not results:
            return "❌ 未能獲取任何分析結果"

        # 過濾成功的結果並按分數排序
        successful_results = [
            result for result in results.values()
            if result.get('success', False) and result.get('combined_score', 0) > 0
        ]

        successful_results.sort(key=lambda x: x.get('combined_score', 0), reverse=True)

        if not successful_results:
            return "❌ 沒有符合條件的優質股票"

        # 建立訊息
        message = f"🏆 多股票技術分析報告\n"
        message += f"📈 分析時間: {datetime.now(taipei_tz).strftime('%Y-%m-%d %H:%M:%S')}\n"
        message += f"📊 成功分析: {len(successful_results)} 支股票\n\n"

        message += "🏅 前十名優質股票:\n"
        message += "=" * 40 + "\n"

        top_stocks = successful_results[:limit]
        for i, result in enumerate(top_stocks, 1):
            symbol = result.get('symbol', 'Unknown')
            score = result.get('combined_score', 0)
            current_price = result.get('last_price', 0)
            recommendation = result.get('recommendation', {}).get('action', '持有')

            message += f"{i:2d}. {symbol}\n"
            message += f"    💰 價格: {current_price:.2f}\n"
            message += f"    📊 評分: {score:.1f} | 🎯 {recommendation}\n\n"

        # 詳細分析前三名
        message += "\n📊 詳細分析 (前三名):\n"
        message += "=" * 40 + "\n"

        for i, result in enumerate(top_stocks[:3], 1):
            symbol = result.get('symbol', 'Unknown')

            message += f"\n{i}. {symbol}\n"
            if 'data_df' in result and not result['data_df'].empty:
                latest = result['data_df'].iloc[-1]
                if 'RSI' in result['data_df'].columns:
                    message += f"🔍 RSI: {latest['RSI']:.1f}\n"
                if 'MACD' in result['data_df'].columns:
                    macd_signal = "多頭" if latest['MACD'] > latest.get('MACD_Signal', 0) else "空頭"
                    message += f"📈 MACD: {macd_signal}\n"

        message += "\n⚠️ 投資提醒:\n"
        message += "• 本分析僅供參考，投資有風險\n"
        message += "• 請結合基本面分析做決策\n"
        message += "• 建議分散投資，控制風險"

        return message

    except Exception as e:
        logger.error(f"格式化訊息時發生錯誤: {e}")
        return f"❌ 格式化訊息時發生錯誤: {str(e)}"

# =============================================================================
# 10. 股票篩選器
# =============================================================================

def screen_stocks(stock_list, days_back, condition):
    """篩選符合條件的股票"""
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days_back * 2)
    results = pd.DataFrame(columns=['ticker', 'close', 'volume', 'change_pct'])

    for ticker in stock_list:
        try:
            data = yf.download(ticker, start=start_date, end=end_date, progress=False)
            if data.empty or len(data) < 20:
                continue

            data = data.tail(days_back)
            data['change_pct'] = data['Close'].pct_change() * 100

            if condition == "突破整理區間":
                high_20d = data['High'].rolling(window=20).max().shift(1)
                if data['Close'].iloc[-1] > high_20d.iloc[-1]:
                    results = pd.concat([results, pd.DataFrame({
                        'ticker': [ticker],
                        'close': [data['Close'].iloc[-1]],
                        'volume': [data['Volume'].iloc[-1]],
                        'change_pct': [data['change_pct'].iloc[-1]]
                    })], ignore_index=True)

            elif condition == "爆量長紅":
                vol_5d_avg = data['Volume'].rolling(window=5).mean().shift(1)
                if (data['Volume'].iloc[-1] > vol_5d_avg.iloc[-1] * 2 and
                    data['change_pct'].iloc[-1] > 3):
                    results = pd.concat([results, pd.DataFrame({
                        'ticker': [ticker],
                        'close': [data['Close'].iloc[-1]],
                        'volume': [data['Volume'].iloc[-1]],
                        'change_pct': [data['change_pct'].iloc[-1]]
                    })], ignore_index=True)

            elif condition == "突破季線":
                ma60 = data['Close'].rolling(window=60).mean()
                if (data['Close'].iloc[-1] > ma60.iloc[-1] and
                    data['Close'].iloc[-2] <= ma60.iloc[-2]):
                    results = pd.concat([results, pd.DataFrame({
                        'ticker': [ticker],
                        'close': [data['Close'].iloc[-1]],
                        'volume': [data['Volume'].iloc[-1]],
                        'change_pct': [data['change_pct'].iloc[-1]]
                    })], ignore_index=True)

            elif condition == "多頭吞噬":
                if len(data) >= 2:
                    prev_candle = data.iloc[-2]
                    curr_candle = data.iloc[-1]
                    if (prev_candle['Close'] < prev_candle['Open'] and  # 前一根為陰線
                        curr_candle['Close'] > curr_candle['Open'] and  # 當前為陽線
                        curr_candle['Open'] < prev_candle['Close'] and  # 低開
                        curr_candle['Close'] > prev_candle['Open']):    # 高收
                        results = pd.concat([results, pd.DataFrame({
                            'ticker': [ticker],
                            'close': [data['Close'].iloc[-1]],
                            'volume': [data['Volume'].iloc[-1]],
                            'change_pct': [data['change_pct'].iloc[-1]]
                        })], ignore_index=True)

            elif condition == "5與20日均線黃金交叉":
                ma5 = data['Close'].rolling(window=5).mean()
                ma20 = data['Close'].rolling(window=20).mean()
                if (ma5.iloc[-1] > ma20.iloc[-1] and
                    ma5.iloc[-2] <= ma20.iloc[-2]):
                    results = pd.concat([results, pd.DataFrame({
                        'ticker': [ticker],
                        'close': [data['Close'].iloc[-1]],
                        'volume': [data['Volume'].iloc[-1]],
                        'change_pct': [data['change_pct'].iloc[-1]]
                    })], ignore_index=True)

        except Exception:
            continue

    return results

# =============================================================================
# 11. 主要分析系統
# =============================================================================

class MultiStockAnalysisSystem:
    def __init__(self, symbols, period='1y'):
        self.symbols = symbols if isinstance(symbols, list) else [symbols]
        self.period = period
        self.results = {}

    async def analyze_stock(self, symbol):
        """分析單一股票"""
        try:
            logger.info(f"開始分析 {symbol}...")

            # 1. 獲取股票數據
            stock_df = fetch_stock_data(symbol, self.period)
            if stock_df is None:
                return None

            # 2. 獲取恐懼貪婪指數
            days_needed = len(stock_df)
            sentiment_df = get_fear_and_greed_data(days=days_needed + 60)

            # 3. 合併數據
            df = pd.merge(stock_df, sentiment_df, on='Date', how='left')

            # 4. 技術指標計算
            df = calculate_technical_indicators(df)
            df = feature_engineering(df)

            # 5. K線型態分析
            pattern_report = generate_pattern_report(df)

            # 6. 波浪理論分析
            wave_analysis = wave_theory_analysis(df, symbol)

            # 7. 綜合評分
            combined_score = calculate_combined_score(df)

            # 8. 機器學習預測
            predictor = StockPredictor()
            predictor.run_backtest(df)
            predictor.predict_future(df)

            # 9. 生成建議
            recommendation = generate_recommendation(combined_score, pattern_report, wave_analysis)

            # 10. 生成圖表
            visualizer = StockVisualizer(symbol)

            # 保存圖表
            chart_files = []

            # 技術分析圖
            fig1 = visualizer.plot_stock_analysis(df)
            chart_path1 = os.path.join(CHARTS_DIR, f"{symbol}_technical.png")
            fig1.savefig(chart_path1, dpi=150, bbox_inches='tight')
            chart_files.append(chart_path1)
            plt.close(fig1)

            # 預測圖
            if predictor.future_predictions:
                fig2 = visualizer.plot_future_prediction(
                    df['Close'].iloc[-1], df['Date'].iloc[-1], predictor.future_predictions
                )
                chart_path2 = os.path.join(CHARTS_DIR, f"{symbol}_prediction.png")
                fig2.savefig(chart_path2, dpi=150, bbox_inches='tight')
                chart_files.append(chart_path2)
                plt.close(fig2)

            # 特徵重要性圖
            if predictor.feature_importances is not None:
                fig3 = visualizer.plot_feature_importance(predictor.feature_importances)
                chart_path3 = os.path.join(CHARTS_DIR, f"{symbol}_features.png")
                fig3.savefig(chart_path3, dpi=150, bbox_inches='tight')
                chart_files.append(chart_path3)
                plt.close(fig3)

            result = {
                'symbol': symbol,
                'data_df': df,
                'combined_score': combined_score,
                'pattern_report': pattern_report,
                'wave_analysis': wave_analysis,
                'recommendation': recommendation,
                'predictor': predictor,
                'chart_files': chart_files,
                'last_price': df['Close'].iloc[-1],
                'last_date': df['Date'].iloc[-1],
                'success': True
            }

            logger.info(f"完成分析 {symbol}")
            return result

        except Exception as e:
            logger.error(f"分析 {symbol} 時發生錯誤: {e}")
            traceback.print_exc()
            return {'symbol': symbol, 'success': False, 'error': str(e)}

    async def run_analysis(self):
        """執行多股票分析"""
        logger.info(f"開始分析 {len(self.symbols)} 支股票...")

        # 並行分析所有股票
        tasks = [self.analyze_stock(symbol) for symbol in self.symbols]
        results = await asyncio.gather(*tasks, return_exceptions=True)

        # 整理結果
        for i, result in enumerate(results):
            if isinstance(result, Exception):
                logger.error(f"分析 {self.symbols[i]} 時發生異常: {result}")
            elif result is not None:
                self.results[self.symbols[i]] = result

        # 生成總結報告
        await self.generate_summary_report()

    async def generate_summary_report(self):
        """生成總結報告並發送通知"""
        if not self.results:
            logger.warning("沒有成功分析的股票")
            return

        # 生成報告文本
        report_text = format_analysis_message(self.results, limit=10)

        # 收集所有圖表文件
        all_chart_files = []
        for result in self.results.values():
            if result.get('success', False):
                all_chart_files.extend(result.get('chart_files', []))

        # 發送通知
        async with aiohttp.ClientSession() as session:
            await send_notification(session, report_text, all_chart_files)

        # 終端顯示
        print("\n" + report_text)

# =============================================================================
# 12. Shioaji 整合模組
# =============================================================================

if SHIOAJI_AVAILABLE:
    class ShioajiStockScanner:
        def __init__(self, api_key, secret_key, max_retries=3):
            self.api_key = api_key
            self.secret_key = secret_key
            self.max_retries = max_retries
            self.api = None
            self.connect()

        def connect(self):
            """建立連接並處理重試邏輯"""
            retries = 0
            while retries < self.max_retries:
                try:
                    if self.api is not None:
                        try:
                            self.api.logout()
                        except:
                            pass

                    self.api = sj.Shioaji()
                    self.api.login(self.api_key, self.secret_key)
                    logger.info("Shioaji 登入成功")
                    return True
                except Exception as e:
                    retries += 1
                    logger.error(f"Shioaji 登入嘗試 {retries} 失敗: {str(e)}")
                    if "Too Many Connections" in str(e):
                        logger.info("等待連接釋放...")
                        time.sleep(10 + random.randint(1, 5))
                    else:
                        time.sleep(2)

            logger.error("超過最大重試次數，無法建立連接")
            return False

        def get_stock_list(self, max_stocks=50):
            """獲取台股清單"""
            if not self.api:
                if not self.connect():
                    return pd.DataFrame()

            tse_stocks = self.api.Contracts.Stocks.TSE
            otc_stocks = self.api.Contracts.Stocks.OTC

            stock_list = []
            count = 0

            # 先取上市股票
            for stock in tse_stocks:
                try:
                    stock_code = stock.code
                    if len(stock_code) == 4 and stock_code.isdigit():
                        stock_list.append({
                            'code': stock_code,
                            'name': stock.name,
                            'exchange': stock.exchange
                        })
                        count += 1
                        if count >= max_stocks:
                            break
                except AttributeError:
                    continue

            # 如果上市股票不足，再取上櫃股票
            if count < max_stocks:
                for stock in otc_stocks:
                    try:
                        stock_code = stock.code
                        if len(stock_code) == 4 and stock_code.isdigit():
                            stock_list.append({
                                'code': stock_code,
                                'name': stock.name,
                                'exchange': stock.exchange
                            })
                            count += 1
                            if count >= max_stocks:
                                break
                    except AttributeError:
                        continue

            df = pd.DataFrame(stock_list)
            logger.info(f"獲取到 {len(df)} 支台股資料")
            return df

        def __del__(self):
            """清理資源"""
            if hasattr(self, 'api') and self.api:
                try:
                    self.api.logout()
                    logger.info("Shioaji 成功登出")
                except:
                    logger.error("Shioaji 登出時發生錯誤")

# =============================================================================
# 13. UI 介面 (Jupyter Notebook)
# =============================================================================

if JUPYTER_AVAILABLE:
    def create_stock_analysis_ui():
        """創建股票分析UI介面"""
        style = {'description_width': '120px'}
        layout = Layout(width='300px')

        # 控制元件
        symbol_input = widgets.Text(
            value='AAPL,TSLA,2330,2454',
            placeholder='輸入股票代碼，用逗號分隔',
            description='股票代碼:',
            style=style,
            layout=Layout(width='400px')
        )

        period_dropdown = widgets.Dropdown(
            options=[('1年', '1y'), ('2年', '2y'), ('5年', '5y'), ('最大', 'max')],
            value='1y',
            description='時間範圍:',
            style=style,
            layout=layout
        )

        analyze_button = widgets.Button(
            description='開始分析',
            button_style='primary',
            layout=Layout(width='150px', height='40px')
        )

        output_area = widgets.Output()

        def on_analyze_click(b):
            with output_area:
                clear_output(wait=True)
                print("🚀 開始分析...")

                symbols = [s.strip().upper() for s in symbol_input.value.split(',') if s.strip()]
                if not symbols:
                    print("❌ 請輸入有效的股票代碼")
                    return

                try:
                    system = MultiStockAnalysisSystem(symbols, period_dropdown.value)

                    # 在 Jupyter 中運行異步代碼
                    loop = asyncio.get_event_loop()
                    loop.run_until_complete(system.run_analysis())

                    print("✅ 分析完成！請查看生成的圖表和通知。")

                except Exception as e:
                    print(f"❌ 分析過程中發生錯誤: {e}")
                    traceback.print_exc()

        analyze_button.on_click(on_analyze_click)

        # 組合UI
        ui = VBox([
            HTML('<h2>📈 多股票分析系統</h2>'),
            HTML('<p>支援台股(輸入數字代碼如2330)和美股(輸入字母代碼如AAPL)</p>'),
            symbol_input,
            period_dropdown,
            analyze_button,
            output_area
        ])

        return ui

    def create_stock_screener_ui():
        """創建股票篩選器UI"""
        tw_stocks = [
            '2330', '2317', '2454', '2881', '2882', '2883', '2884', '2885',
            '2886', '2887', '2888', '2889', '2890', '2891', '2892', '2912'
        ]

        us_stocks = [
            'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA', 'NFLX',
            'JPM', 'JNJ', 'V', 'PG', 'UNH', 'HD', 'MA', 'DIS'
        ]

        style = {'description_width': '120px'}
        layout = Layout(width='300px')

        market_dropdown = widgets.Dropdown(
            options=[('台股', 'tw'), ('美股', 'us')],
            value='tw',
            description='市場:',
            style=style,
            layout=layout
        )

        condition_dropdown = widgets.Dropdown(
            options=[
                ('突破整理區間', '突破整理區間'),
                ('爆量長紅', '爆量長紅'),
                ('突破季線', '突破季線'),
                ('多頭吞噬', '多頭吞噬'),
                ('5與20日均線黃金交叉', '5與20日均線黃金交叉')
            ],
            value='突破整理區間',
            description='篩選條件:',
            style=style,
            layout=Layout(width='400px')
        )

        days_slider = widgets.IntSlider(
            value=30,
            min=10,
            max=100,
            step=10,
            description='回看天數:',
            style=style,
            layout=layout
        )

        screen_button = widgets.Button(
            description='開始篩選',
            button_style='success',
            layout=Layout(width='150px', height='40px')
        )

        output_area = widgets.Output()

        def on_screen_click(b):
            with output_area:
                clear_output(wait=True)
                print("🔍 開始篩選股票...")

                stock_list = tw_stocks if market_dropdown.value == 'tw' else us_stocks

                try:
                    results = screen_stocks(stock_list, days_slider.value, condition_dropdown.value)

                    if results.empty:
                        print("❌ 沒有找到符合條件的股票")
                    else:
                        print(f"✅ 找到 {len(results)} 支符合條件的股票:")
                        print("\n" + "="*60)
                        for _, row in results.iterrows():
                            print(f"股票: {row['ticker']}")
                            print(f"收盤價: {row['close']:.2f}")
                            print(f"成交量: {row['volume']:,.0f}")
                            print(f"漲跌幅: {row['change_pct']:.2f}%")
                            print("-" * 30)

                except Exception as e:
                    print(f"❌ 篩選過程中發生錯誤: {e}")
                    traceback.print_exc()

        screen_button.on_click(on_screen_click)

        ui = VBox([
            HTML('<h2>🔍 股票篩選器</h2>'),
            market_dropdown,
            condition_dropdown,
            days_slider,
            screen_button,
            output_area
        ])

        return ui

# =============================================================================
# 14. 命令行介面
# =============================================================================

def main():
    """主函數，運行互動式界面"""
    while True:
        print("\n\n🚀 歡迎使用多股票分析預測系統")
        print("="*50)
        print("1. 多股票分析與預測")
        print("2. 股票篩選器")
        print("3. 單股票詳細分析")
        if SHIOAJI_AVAILABLE:
            print("4. Shioaji 台股掃描")
        if JUPYTER_AVAILABLE:
            print("5. 啟動 Jupyter UI 介面")
        print("0. 退出程序")
        print("-"*50)

        try:
            choice = input("請選擇功能 (0-5): ").strip()

            if choice == '0':
                print("感謝使用，程序退出。")
                break

            elif choice == '1':
                symbols_input = input("請輸入股票代碼 (用逗號分隔，如: AAPL,TSLA,2330,2454): ").strip()
                if not symbols_input:
                    print("❌ 請輸入有效的股票代碼")
                    continue

                symbols = [s.strip().upper() for s in symbols_input.split(',') if s.strip()]
                period = input("請輸入歷史數據區間 (1y, 2y, 5y, max) [預設: 1y]: ").strip().lower()
                if not period:
                    period = '1y'

                print(f"🚀 開始分析 {len(symbols)} 支股票...")

                try:
                    system = MultiStockAnalysisSystem(symbols, period)
                    asyncio.run(system.run_analysis())
                    print("✅ 分析完成！")
                except Exception as e:
                    print(f"❌ 分析過程中發生錯誤: {e}")
                    traceback.print_exc()

            elif choice == '2':
                print("\n股票篩選器")
                print("1. 台股篩選")
                print("2. 美股篩選")

                market_choice = input("請選擇市場 (1-2): ").strip()
                if market_choice not in ['1', '2']:
                    print("❌ 無效選擇")
                    continue

                conditions = [
                    "突破整理區間", "爆量長紅", "突破季線", "多頭吞噬", "5與20日均線黃金交叉"
                ]

                print("\n篩選條件:")
                for i, condition in enumerate(conditions, 1):
                    print(f"{i}. {condition}")

                condition_choice = input("請選擇篩選條件 (1-5): ").strip()
                try:
                    condition_idx = int(condition_choice) - 1
                    if condition_idx < 0 or condition_idx >= len(conditions):
                        print("❌ 無效選擇")
                        continue
                    condition = conditions[condition_idx]
                except ValueError:
                    print("❌ 請輸入數字")
                    continue

                days_input = input("請輸入回看天數 [預設: 30]: ").strip()
                try:
                    days_back = int(days_input) if days_input else 30
                except ValueError:
                    days_back = 30

                # 股票清單
                tw_stocks = [
                    '2330.TW', '2317.TW', '2454.TW', '2881.TW', '2882.TW', '2883.TW',
                    '2884.TW', '2885.TW', '2886.TW', '2887.TW', '2888.TW', '2889.TW',
                    '2890.TW', '2891.TW', '2892.TW', '2912.TW', '2002.TW', '1303.TW',
                    '3711.TW', '1301.TW', '2207.TW', '2357.TW', '2382.TW', '5871.TW'
                ]

                us_stocks = [
                    'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA', 'NFLX',
                    'JPM', 'JNJ', 'V', 'PG', 'UNH', 'HD', 'MA', 'DIS', 'PYPL', 'ADBE',
                    'CRM', 'INTC', 'CSCO', 'PFE', 'KO', 'PEP', 'WMT', 'MRK'
                ]

                stock_list = tw_stocks if market_choice == '1' else us_stocks
                market_name = "台股" if market_choice == '1' else "美股"

                print(f"🔍 開始篩選 {market_name}，條件: {condition}，回看: {days_back} 天...")

                try:
                    results = screen_stocks(stock_list, days_back, condition)

                    if results.empty:
                        print("❌ 沒有找到符合條件的股票")
                    else:
                        print(f"✅ 找到 {len(results)} 支符合條件的股票:")
                        print("\n" + "="*70)
                        for _, row in results.iterrows():
                            print(f"股票: {row['ticker']:<10} | 收盤價: {row['close']:>8.2f} | "
                                  f"成交量: {row['volume']:>12,.0f} | 漲跌幅: {row['change_pct']:>6.2f}%")
                        print("="*70)

                        # 詢問是否對篩選結果進行詳細分析
                        analyze_choice = input("\n是否對篩選結果進行詳細分析？(y/n): ").strip().lower()
                        if analyze_choice == 'y':
                            selected_symbols = results['ticker'].tolist()
                            print(f"🚀 開始詳細分析 {len(selected_symbols)} 支股票...")

                            system = MultiStockAnalysisSystem(selected_symbols, '1y')
                            asyncio.run(system.run_analysis())
                            print("✅ 詳細分析完成！")

                except Exception as e:
                    print(f"❌ 篩選過程中發生錯誤: {e}")
                    traceback.print_exc()

            elif choice == '3':
                symbol = input("請輸入股票代碼 (如: AAPL 或 2330): ").strip().upper()
                if not symbol:
                    print("❌ 請輸入有效的股票代碼")
                    continue

                period = input("請輸入歷史數據區間 (1y, 2y, 5y, max) [預設: 1y]: ").strip().lower()
                if not period:
                    period = '1y'

                print(f"🚀 開始詳細分析 {symbol}...")

                try:
                    system = MultiStockAnalysisSystem([symbol], period)
                    asyncio.run(system.run_analysis())

                    # 顯示詳細結果
                    if symbol in system.results:
                        result = system.results[symbol]
                        if result.get('success', False):
                            print("\n📊 詳細分析結果:")
                            print("="*50)
                            print(f"股票代碼: {symbol}")
                            print(f"最新價格: {result['last_price']:.2f}")
                            print(f"綜合評分: {result['combined_score']:.2f}")
                            print(f"投資建議: {result['recommendation']['action']}")
                            print(f"建議理由: {result['recommendation']['reason']}")
                            print(f"信心度: {result['recommendation']['confidence']}%")
                            print(f"風險等級: {result['recommendation']['risk_level']}")

                            print("\nK線型態分析:")
                            print(result['pattern_report'])

                            if result['wave_analysis'][0]:
                                print(f"\n波浪理論分析: {result['wave_analysis'][1]}")

                            if result['predictor'].future_predictions:
                                print("\n未來一週預測:")
                                for day, preds in result['predictor'].future_predictions.items():
                                    avg_pred = np.mean(list(preds.values()))
                                    change_pct = (avg_pred / result['last_price'] - 1) * 100
                                    print(f"第 {day} 天: {avg_pred:.2f} ({change_pct:+.1f}%)")

                            print(f"\n📈 圖表已保存到: {', '.join(result['chart_files'])}")
                        else:
                            print(f"❌ 分析 {symbol} 失敗: {result.get('error', '未知錯誤')}")

                    print("✅ 詳細分析完成！")

                except Exception as e:
                    print(f"❌ 分析過程中發生錯誤: {e}")
                    traceback.print_exc()

            elif choice == '4' and SHIOAJI_AVAILABLE:
                api_key = input("請輸入 Shioaji API Key: ").strip()
                secret_key = input("請輸入 Shioaji Secret Key: ").strip()

                if not api_key or not secret_key:
                    print("❌ API Key 和 Secret Key 不能為空")
                    continue

                max_stocks = input("請輸入要掃描的股票數量 [預設: 20]: ").strip()
                try:
                    max_stocks = int(max_stocks) if max_stocks else 20
                except ValueError:
                    max_stocks = 20

                print(f"🚀 開始掃描台股，數量: {max_stocks}...")

                try:
                    scanner = ShioajiStockScanner(api_key, secret_key)
                    stock_list_df = scanner.get_stock_list(max_stocks)

                    if stock_list_df.empty:
                        print("❌ 無法獲取股票清單")
                        continue

                    # 轉換為分析系統可用的格式
                    symbols = [f"{code}.TW" for code in stock_list_df['code'].tolist()]

                    print(f"📊 獲取到 {len(symbols)} 支台股，開始技術分析...")

                    system = MultiStockAnalysisSystem(symbols, '1y')
                    asyncio.run(system.run_analysis())

                    print("✅ Shioaji 台股掃描完成！")

                except Exception as e:
                    print(f"❌ Shioaji 掃描過程中發生錯誤: {e}")
                    traceback.print_exc()

            elif choice == '5' and JUPYTER_AVAILABLE:
                print("🎯 啟動 Jupyter UI 介面...")
                print("請在 Jupyter Notebook 中執行以下代碼:")
                print("-"*50)
                print("# 股票分析UI")
                print("analysis_ui = create_stock_analysis_ui()")
                print("display(analysis_ui)")
                print()
                print("# 股票篩選UI")
                print("screener_ui = create_stock_screener_ui()")
                print("display(screener_ui)")
                print("-"*50)

                # 如果在 Jupyter 環境中，直接顯示 UI
                try:
                    from IPython.display import display
                    print("檢測到 Jupyter 環境，正在顯示 UI...")
                    analysis_ui = create_stock_analysis_ui()
                    screener_ui = create_stock_screener_ui()
                    display(analysis_ui)
                    display(screener_ui)
                except:
                    print("請在 Jupyter Notebook 中手動執行上述代碼")

            else:
                print("❌ 無效選擇，請重新輸入")

        except KeyboardInterrupt:
            print("\n\n程序被用戶中斷")
            break
        except Exception as e:
            print(f"❌ 發生未預期的錯誤: {e}")
            traceback.print_exc()

# =============================================================================
# 15. 示例和測試
# =============================================================================

def run_example():
    """運行示例分析"""
    print("🚀 運行示例分析...")

    # 示例股票組合
    example_symbols = ['AAPL', 'TSLA', '2330.TW', '2454.TW']

    try:
        system = MultiStockAnalysisSystem(example_symbols, '1y')
        asyncio.run(system.run_analysis())
        print("✅ 示例分析完成！")

        # 顯示簡要結果
        print("\n📊 示例分析結果摘要:")
        print("="*50)

        successful_results = [
            result for result in system.results.values()
            if result.get('success', False)
        ]

        successful_results.sort(key=lambda x: x.get('combined_score', 0), reverse=True)

        for i, result in enumerate(successful_results, 1):
            symbol = result.get('symbol', 'Unknown')
            score = result.get('combined_score', 0)
            recommendation = result.get('recommendation', {}).get('action', '持有')

            print(f"{i}. {symbol:<8} | 評分: {score:>5.1f} | 建議: {recommendation}")

        print("="*50)

    except Exception as e:
        print(f"❌ 示例分析失敗: {e}")
        traceback.print_exc()

def test_individual_components():
    """測試各個組件"""
    print("🧪 測試系統組件...")

    # 測試數據獲取
    print("1. 測試股票數據獲取...")
    test_df = fetch_stock_data('AAPL', '3mo')
    if test_df is not None:
        print(f"✅ 成功獲取 AAPL 數據，共 {len(test_df)} 筆")
    else:
        print("❌ 股票數據獲取失敗")

    # 測試恐懼貪婪指數
    print("2. 測試恐懼貪婪指數獲取...")
    sentiment_df = get_fear_and_greed_data(30)
    if sentiment_df is not None:
        print(f"✅ 成功獲取恐懼貪婪指數，共 {len(sentiment_df)} 筆")
    else:
        print("❌ 恐懼貪婪指數獲取失敗")

    # 測試技術指標計算
    if test_df is not None:
        print("3. 測試技術指標計算...")
        try:
            test_df = calculate_technical_indicators(test_df)
            test_df = feature_engineering(test_df)
            print("✅ 技術指標計算成功")
        except Exception as e:
            print(f"❌ 技術指標計算失敗: {e}")

    # 測試K線型態識別
    if test_df is not None:
        print("4. 測試K線型態識別...")
        try:
            pattern_report = generate_pattern_report(test_df)
            print("✅ K線型態識別成功")
            print(f"型態報告: {pattern_report[:100]}...")
        except Exception as e:
            print(f"❌ K線型態識別失敗: {e}")

    # 測試評分系統
    if test_df is not None:
        print("5. 測試評分系統...")
        try:
            score = calculate_combined_score(test_df)
            print(f"✅ 評分系統成功，綜合評分: {score:.2f}")
        except Exception as e:
            print(f"❌ 評分系統失敗: {e}")

    print("🧪 組件測試完成")

# =============================================================================
# 16. 程序入口點
# =============================================================================

if __name__ == "__main__":
    print("🎯 多股票分析預測系統 v2.0")
    print("支援台股、美股技術分析、機器學習預測、型態識別")
    print("="*60)

    # 檢查環境
    print("🔍 檢查系統環境...")
    print(f"Jupyter 支援: {'✅' if JUPYTER_AVAILABLE else '❌'}")
    print(f"Shioaji 支援: {'✅' if SHIOAJI_AVAILABLE else '❌'}")
    print(f"通知功能: {'✅' if MESSAGING_AVAILABLE else '❌'}")

    # 詢問運行模式
    print("\n選擇運行模式:")
    print("1. 互動式命令行介面")
    print("2. 運行示例分析")
    print("3. 測試系統組件")
    print("0. 退出")

    try:
        mode = input("請選擇 (0-3): ").strip()

        if mode == '0':
            print("程序退出")
        elif mode == '1':
            main()
        elif mode == '2':
            run_example()
        elif mode == '3':
            test_individual_components()
        else:
            print("無效選擇，啟動互動式介面...")
            main()

    except KeyboardInterrupt:
        print("\n程序被用戶中斷")
    except Exception as e:
        print(f"程序運行時發生錯誤: {e}")
        traceback.print_exc()

    print("\n感謝使用多股票分析預測系統！")

In [ ]:

import os
import time
import pandas as pd
import numpy as np
import yfinance as yf
import requests
from bs4 import BeautifulSoup
import re
import warnings
import logging
import traceback
from datetime import datetime, timedelta, timezone
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.font_manager import FontProperties
import seaborn as sns
from io import StringIO
import asyncio
import aiohttp
import json
import pickle
from typing import Dict, List, Any, Optional
import random

# 機器學習相關
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# 通知相關
try:
    from discord_webhook import DiscordWebhook
    MESSAGING_AVAILABLE = True
except ImportError:
    MESSAGING_AVAILABLE = False

# Jupyter Notebook 支援
try:
    import nest_asyncio
    nest_asyncio.apply()
    import ipywidgets as widgets
    from ipywidgets import Layout, HTML, Button, HBox, VBox, Output
    from IPython.display import display, clear_output
    JUPYTER_AVAILABLE = True
except ImportError:
    JUPYTER_AVAILABLE = False

# =============================================================================
# 全域設定
# =============================================================================

# 禁止中文錯誤警告
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# 設定日誌
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# 中文字體設定
plt.rcParams['font.sans-serif'] = [
    'Microsoft JhengHei',  # 繁體中文
    'SimHei',              # 簡體中文
    'Arial Unicode MS',    # Unicode 字體
    'DejaVu Sans'         # 基礎字體
]
plt.rcParams['axes.unicode_minus'] = False
plt.style.use('seaborn-v0_8-whitegrid')

# 通知設定
TELEGRAM_TOKEN = "7902318521:AAEYoDMqwfHabI7L1SRiE4z33aFay42-VGE"
TELEGRAM_CHAT_ID = [879781796, 8113868436]
DISCORD_WEBHOOK_URL = "https://discord.com/api/webhooks/1362715080734802102/Jma7A3VhEQrrRxIX_JW2l6rATjAZXsGXGfnJuAMqmS1QvqG_2ptg3vr_nsVnuV_PlnBl"

# 台北時區
taipei_tz = timezone(timedelta(hours=8))

# 圖表儲存目錄
CHARTS_DIR = 'charts'
if not os.path.exists(CHARTS_DIR):
    os.makedirs(CHARTS_DIR)

print("🚀 環境設定完成，開始建立分析系統...")

# =============================================================================
# 1. 數據獲取模組
# =============================================================================

def fetch_from_cnn():
    """從CNN網站獲取恐懼與貪婪指數"""
    url = "https://money.cnn.com/data/fear-and-greed/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')

        for element in soup.find_all(['div', 'span']):
            if "Fear & Greed Now:" in element.get_text():
                number_match = re.search(r'(\d+)', element.get_text())
                if number_match:
                    value = int(number_match.group(1))
                    logger.info(f"成功從CNN獲取恐懼與貪婪指數: {value}")
                    return value
        logger.warning("無法從CNN提取恐懼與貪婪指數")
        return None
    except Exception as e:
        logger.error(f"從CNN獲取恐懼與貪婪指數時出錯: {e}")
        return None

def fetch_from_alternative_me(limit=1):
    """從alternative.me API獲取恐懼與貪婪指數"""
    try:
        url = f"https://api.alternative.me/fng/?limit={limit}&format=json"
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()

        if 'data' in data and len(data['data']) > 0:
            if limit == 1:
                value = int(data['data'][0]['value'])
                logger.info(f"成功從Alternative.me API獲取恐懼與貪婪指數: {value}")
                return value
            else:
                df = pd.DataFrame(data['data'])
                df['value'] = pd.to_numeric(df['value'])
                df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s').dt.date
                df = df.rename(columns={'timestamp': 'Date', 'value': 'FearGreedIndex'})
                df['Date'] = pd.to_datetime(df['Date'])
                logger.info(f"成功獲取 {len(df)} 天的歷史恐懼與貪婪指數數據")
                return df[['Date', 'FearGreedIndex']]
        else:
            logger.warning("Alternative.me API返回的數據格式不正確")
            return None
    except Exception as e:
        logger.error(f"從Alternative.me獲取恐懼與貪婪指數時出錯: {e}")
        return None

def get_fear_and_greed_data(days):
    """獲取即時與歷史恐懼與貪婪指數數據"""
    logger.info("正在獲取恐懼與貪婪指數...")

    current_value = fetch_from_cnn()
    if current_value is None:
        current_value = fetch_from_alternative_me(limit=1)

    historical_df = fetch_from_alternative_me(limit=days)

    if historical_df is not None:
        logger.info("已成功獲取真實歷史恐懼與貪婪指數數據。")
        if current_value is not None:
            today = pd.to_datetime(datetime.now().date())
            if today not in historical_df['Date'].values:
                new_row = pd.DataFrame([{'Date': today, 'FearGreedIndex': current_value}])
                historical_df = pd.concat([historical_df, new_row], ignore_index=True)
        return historical_df.drop_duplicates(subset=['Date'], keep='last')

    logger.warning("無法獲取真實歷史數據，將使用模擬數據作為備案。")
    if current_value is None:
        current_value = 50
        logger.warning(f"所有即時來源均失敗，使用預設值 {current_value}")

    dates = pd.to_datetime([datetime.now() - timedelta(days=i) for i in range(days)]).sort_values()
    scores = [current_value]
    np.random.seed(42)
    for _ in range(1, days):
        mean_reversion = 0.1 * (50 - scores[-1])
        random_change = np.random.normal(0, 3)
        new_score = scores[-1] + mean_reversion + random_change
        scores.append(max(0, min(100, new_score)))

    return pd.DataFrame({'Date': dates, 'FearGreedIndex': scores})

def fetch_stock_data(symbol, period='1y'):
    """從Yahoo Finance獲取股票數據，自動判斷台股/美股"""
    original_symbol = symbol

    # 如果輸入的是純數字，判斷為台股代碼
    if re.match(r'^\d{4,5}$', symbol):
        symbol += ".TW"
        logger.info(f"檢測到台股代碼，轉換為 {symbol}")

    logger.info(f"正在從Yahoo Finance獲取 {symbol} ({period}) 的數據...")
    try:
        stock = yf.Ticker(symbol)
        df = stock.history(period=period, interval='1d')
        if df.empty:
            logger.error(f"無法獲取 {symbol} 的數據，請檢查代碼是否正確。")
            return None

        df.reset_index(inplace=True)
        df['Date'] = pd.to_datetime(df['Date'].dt.date)

        logger.info(f"成功獲取 {original_symbol} 的 {len(df)} 筆數據")
        return df
    except Exception as e:
        logger.error(f"從Yahoo Finance獲取 {symbol} 的數據時出錯: {e}")
        return None

# =============================================================================
# 2. 技術指標計算模組
# =============================================================================

def calculate_technical_indicators(df):
    """計算所有需要的技術指標"""
    # 移動平均線
    df['MA5'] = df['Close'].rolling(window=5).mean()
    df['MA10'] = df['Close'].rolling(window=10).mean()
    df['MA20'] = df['Close'].rolling(window=20).mean()
    df['MA60'] = df['Close'].rolling(window=60).mean()

    # MACD
    df['EMA12'] = df['Close'].ewm(span=12, adjust=False).mean()
    df['EMA26'] = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = df['EMA12'] - df['EMA26']
    df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

    # RSI
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean().replace(0, np.nan)
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))

    # 布林帶
    df['BB_Middle'] = df['MA20']
    std_dev = df['Close'].rolling(window=20).std()
    df['BB_Upper'] = df['BB_Middle'] + (std_dev * 2)
    df['BB_Lower'] = df['BB_Middle'] - (std_dev * 2)

    # KD指標
    low_min = df['Low'].rolling(window=9).min()
    high_max = df['High'].rolling(window=9).max()
    rsv = (df['Close'] - low_min) / (high_max - low_min) * 100
    df['K'] = rsv.ewm(com=2).mean()
    df['D'] = df['K'].ewm(com=2).mean()

    # 成交量指標
    df['Volume_MA'] = df['Volume'].rolling(window=20).mean()

    return df

def feature_engineering(df):
    """創建衍生特徵"""
    df['Price_Change_1d'] = df['Close'].pct_change(1)
    df['Price_Change_5d'] = df['Close'].pct_change(5)
    df['Volatility_5d'] = df['Close'].rolling(5).std()
    df['Volatility_20d'] = df['Close'].rolling(20).std()
    df['BB_Width'] = (df['BB_Upper'] - df['BB_Lower']) / df['BB_Middle']
    df['BB_Position'] = (df['Close'] - df['BB_Lower']) / (df['BB_Upper'] - df['BB_Lower']).replace(0, np.nan)

    # 填充NaN值
    df.bfill(inplace=True)
    df.ffill(inplace=True)
    df.fillna(0, inplace=True)
    return df

# =============================================================================
# 3. K線型態識別模組
# =============================================================================

def identify_candlestick_patterns(df):
    """識別常見的K線型態"""
    result_df = df.copy()

    # 基本計算
    result_df['body_size'] = abs(result_df['Close'] - result_df['Open'])
    result_df['upper_shadow'] = result_df['High'] - result_df[['Open', 'Close']].max(axis=1)
    result_df['lower_shadow'] = result_df[['Open', 'Close']].min(axis=1) - result_df['Low']
    result_df['total_range'] = result_df['High'] - result_df['Low']

    # 前一天數據
    result_df['prev_open'] = result_df['Open'].shift(1)
    result_df['prev_high'] = result_df['High'].shift(1)
    result_df['prev_low'] = result_df['Low'].shift(1)
    result_df['prev_close'] = result_df['Close'].shift(1)

    # K線趨勢
    result_df['is_bullish'] = result_df['Close'] > result_df['Open']
    result_df['prev_is_bullish'] = result_df['prev_close'] > result_df['prev_open']

    # 初始化型態欄位
    pattern_columns = [
        '十字星_中性', '錘子線_看漲', '錘子線_看跌', '流星線_看跌',
        '吞噬_看漲', '吞噬_看跌', '母子線_看漲', '母子線_看跌'
    ]

    for col in pattern_columns:
        result_df[col] = 0

    # 十字星
    doji_condition = result_df['body_size'] <= 0.1 * result_df['total_range']
    result_df.loc[doji_condition, '十字星_中性'] = 1

    # 錘子線
    hammer_condition = (
        (result_df['lower_shadow'] >= 2 * result_df['body_size']) &
        (result_df['upper_shadow'] <= 0.1 * result_df['total_range'])
    )
    result_df.loc[hammer_condition, '錘子線_看漲'] = 1

    # 流星線
    shooting_star_condition = (
        (result_df['upper_shadow'] >= 2 * result_df['body_size']) &
        (result_df['lower_shadow'] <= 0.1 * result_df['total_range'])
    )
    result_df.loc[shooting_star_condition, '流星線_看跌'] = -1

    # 看漲吞噬
    bullish_engulfing = (
        (~result_df['prev_is_bullish']) &
        result_df['is_bullish'] &
        (result_df['Open'] < result_df['prev_close']) &
        (result_df['Close'] > result_df['prev_open'])
    )
    result_df.loc[bullish_engulfing, '吞噬_看漲'] = 1

    # 看跌吞噬
    bearish_engulfing = (
        result_df['prev_is_bullish'] &
        (~result_df['is_bullish']) &
        (result_df['Open'] > result_df['prev_close']) &
        (result_df['Close'] < result_df['prev_open'])
    )
    result_df.loc[bearish_engulfing, '吞噬_看跌'] = -1

    return result_df

def generate_pattern_report(df, periods=['daily']):
    """生成K線型態分析報告"""
    if df is None or df.empty:
        return "無法生成報告：數據為空"

    required_cols = ['Open', 'High', 'Low', 'Close']
    if not all(col in df.columns for col in required_cols):
        return "無法生成報告：缺少必要欄位"

    def detect_patterns(ohlc_data):
        patterns = {"看漲": [], "看跌": [], "中性": []}

        if len(ohlc_data) < 5:
            return patterns

        recent = ohlc_data.tail(10).copy()
        recent['body_size'] = abs(recent['Close'] - recent['Open'])
        recent['is_bullish'] = recent['Close'] > recent['Open']
        recent['is_bearish'] = recent['Close'] < recent['Open']

        last3 = recent.tail(3)

        # 三白兵
        if len(last3) == 3 and all(last3['is_bullish']) and \
           last3['Close'].iloc[0] < last3['Close'].iloc[1] < last3['Close'].iloc[2]:
            patterns["看漲"].append("三白兵")

        # 三黑鴉
        if len(last3) == 3 and all(last3['is_bearish']) and \
           last3['Close'].iloc[0] > last3['Close'].iloc[1] > last3['Close'].iloc[2]:
            patterns["看跌"].append("三黑鴉")

        # 十字星
        if any(last3['body_size'] < 0.1 * (last3['High'] - last3['Low'])):
            patterns["中性"].append("十字星")

        return patterns

    report_parts = []

    for period in periods:
        period_name = {'daily': '日線', 'weekly': '週線', 'monthly': '月線'}.get(period, period)

        try:
            patterns = detect_patterns(df)
            period_report = [f"{period_name}分析:"]

            for pattern_type, detected_patterns in patterns.items():
                if detected_patterns:
                    pattern_str = ", ".join(detected_patterns)
                    period_report.append(f"{pattern_type}型態: {pattern_str}")
                else:
                    period_report.append(f"{pattern_type}型態: 無")

            report_parts.append("\n".join(period_report))

        except Exception as e:
            logger.error(f"分析 {period_name} 型態時出錯: {e}")
            report_parts.append(f"{period_name}分析:\n分析時發生錯誤")

    return "\n\n".join(report_parts) if report_parts else "未檢測到明顯K線型態"

# =============================================================================
# 4. 機器學習預測模組
# =============================================================================

class StockPredictor:
    def __init__(self, target_days=7):
        self.models = {
            '線性迴歸': LinearRegression(),
            '隨機森林': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
            'XGBoost': XGBRegressor(n_estimators=100, random_state=42, n_jobs=-1),
            'LightGBM': LGBMRegressor(n_estimators=100, random_state=42, n_jobs=-1)
        }
        self.scalers = {}
        self.feature_names = None
        self.target_days = target_days
        self.backtest_results = {}
        self.future_predictions = {}
        self.feature_importances = None

    def _prepare_data(self, df, predict_day):
        """準備特徵和目標變量"""
        feature_columns = [
            'MA5', 'MA10', 'MA20', 'MACD', 'MACD_Signal', 'RSI', 'BB_Width', 'BB_Position',
            'Price_Change_1d', 'Price_Change_5d', 'Volatility_5d', 'Volatility_20d',
            'FearGreedIndex'
        ]
        self.feature_names = [col for col in feature_columns if col in df.columns]

        X = df[self.feature_names].copy()
        y = df['Close'].shift(-predict_day)

        valid_idx = ~y.isnull()
        X = X[valid_idx]
        y = y[valid_idx]

        return X, y

    def run_backtest(self, df):
        """執行回測"""
        logger.info("執行回測...")
        X, y = self._prepare_data(df, predict_day=1)

        split_index = int(len(X) * 0.8)
        X_train, X_test = X[:split_index], X[split_index:]
        y_train, y_test = y[:split_index], y[split_index:]

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        for name, model in self.models.items():
            model.fit(X_train_scaled, y_train)
            y_pred = model.predict(X_test_scaled)
            mse = mean_squared_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            self.backtest_results[name] = {
                'y_test': y_test, 'y_pred': y_pred, 'mse': mse, 'r2': r2
            }
            logger.info(f"[回測] {name} - R²: {r2:.4f}, MSE: {mse:.4f}")

        if '隨機森林' in self.models:
            self.feature_importances = pd.DataFrame({
                '特徵': self.feature_names,
                '重要性': self.models['隨機森林'].feature_importances_
            }).sort_values('重要性', ascending=False)

    def predict_future(self, df):
        """預測未來價格"""
        logger.info("訓練完整數據集並預測未來價格...")

        for day in range(1, self.target_days + 1):
            X, y = self._prepare_data(df, predict_day=day)

            if X.empty:
                logger.warning(f"無法為預測第 {day} 天準備數據，跳過。")
                continue

            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X)
            X_last = df[self.feature_names].iloc[-1:].copy()
            X_last_scaled = scaler.transform(X_last)

            daily_predictions = {}
            for name, model in self.models.items():
                model.fit(X_scaled, y)
                prediction = model.predict(X_last_scaled)[0]
                daily_predictions[name] = prediction

            self.future_predictions[day] = daily_predictions

# =============================================================================
# 5. 綜合評分系統
# =============================================================================

def calculate_combined_score(df):
    """計算綜合評分"""
    if df.empty:
        return 50

    scores = []

    # 技術指標評分
    if 'RSI' in df.columns:
        rsi = df['RSI'].iloc[-1]
        if 30 <= rsi <= 70:
            scores.append(60 + (rsi - 50) * 0.8)
        elif rsi > 70:
            scores.append(80)
        else:
            scores.append(30)

    # 移動平均線評分
    if all(col in df.columns for col in ['MA5', 'MA20']):
        if df['MA5'].iloc[-1] > df['MA20'].iloc[-1]:
            scores.append(70)
        else:
            scores.append(40)

    # MACD評分
    if all(col in df.columns for col in ['MACD', 'MACD_Signal']):
        if df['MACD'].iloc[-1] > df['MACD_Signal'].iloc[-1]:
            scores.append(65)
        else:
            scores.append(45)

    # 價格趨勢評分
    if 'Price_Change_5d' in df.columns:
        change = df['Price_Change_5d'].iloc[-1]
        if change > 0.05:
            scores.append(75)
        elif change > 0:
            scores.append(60)
        elif change > -0.05:
            scores.append(40)
        else:
            scores.append(25)

    return np.mean(scores) if scores else 50

# =============================================================================
# 6. 智能建議系統
# =============================================================================

def generate_recommendation(combined_score, report_summary=None):
    """生成投資建議"""
    try:
        combined_score = float(combined_score)
    except (ValueError, TypeError):
        combined_score = 50.0

    recommendation = {
        "action": "中立觀察",
        "reason": f"綜合評分中性 ({combined_score:.2f})。",
        "pattern_details": [],
        "confidence": 50
    }

    if combined_score >= 70:
        recommendation["action"] = "買入"
        recommendation["reason"] = f"綜合評分強勁 ({combined_score:.2f})。"
    elif combined_score >= 60:
        recommendation["action"] = "買入觀望"
        recommendation["reason"] = f"綜合評分偏強 ({combined_score:.2f})。"
    elif combined_score <= 30:
        recommendation["action"] = "賣出"
        recommendation["reason"] = f"綜合評分疲弱 ({combined_score:.2f})。"
    elif combined_score <= 40:
        recommendation["action"] = "賣出觀望"
        recommendation["reason"] = f"綜合評分偏弱 ({combined_score:.2f})。"

    # 計算信心度
    distance_from_center = abs(combined_score - 50)
    base_confidence = min(distance_from_center * 2, 100)
    recommendation['confidence'] = int(base_confidence)

    return recommendation

# =============================================================================
# 7. 可視化模組
# =============================================================================

class StockVisualizer:
    def __init__(self, symbol):
        self.symbol = symbol

    def plot_stock_analysis(self, df):
        """繪製股票技術分析圖"""
        fig, axes = plt.subplots(3, 1, figsize=(16, 12), sharex=True)
        fig.suptitle(f'{self.symbol} 技術分析總覽', fontsize=20, fontweight='bold')

        # 價格與布林帶
        axes[0].plot(df['Date'], df['Close'], label='收盤價', linewidth=2)
        if 'BB_Upper' in df.columns:
            axes[0].plot(df['Date'], df['BB_Upper'], label='布林上軌', linestyle='--', alpha=0.7)
            axes[0].plot(df['Date'], df['BB_Lower'], label='布林下軌', linestyle='--', alpha=0.7)
            axes[0].fill_between(df['Date'], df['BB_Lower'], df['BB_Upper'], alpha=0.1)
        axes[0].set_title('價格與布林通道')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)

        # MACD
        if 'MACD' in df.columns:
            axes[1].plot(df['Date'], df['MACD'], label='MACD', color='blue')
            axes[1].plot(df['Date'], df['MACD_Signal'], label='訊號線', color='red', linestyle='--')
            axes[1].bar(df['Date'], df['MACD'] - df['MACD_Signal'], label='柱狀圖', color='gray', alpha=0.5)
        axes[1].set_title('MACD')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)

        # RSI
        if 'RSI' in df.columns:
            axes[2].plot(df['Date'], df['RSI'], label='RSI', color='purple')
            axes[2].axhline(70, color='red', linestyle='--', alpha=0.7, label='超買區 (70)')
            axes[2].axhline(30, color='green', linestyle='--', alpha=0.7, label='超賣區 (30)')
        axes[2].set_title('RSI 相對強弱指數')
        axes[2].legend()
        axes[2].grid(True, alpha=0.3)

        plt.tight_layout(rect=[0, 0, 1, 0.96])
        return fig

    def plot_future_prediction(self, last_price, last_date, predictions):
        """繪製未來預測圖"""
        fig, ax = plt.subplots(figsize=(12, 7))
        pred_dates, pred_prices_avg = [], []

        current_date = last_date
        for day, preds in predictions.items():
            while True:
                current_date += timedelta(days=1)
                if current_date.weekday() < 5:
                    break

            pred_dates.append(current_date)
            avg_price = np.mean(list(preds.values()))
            pred_prices_avg.append(avg_price)

        ax.plot(pred_dates, pred_prices_avg, 'ro-', label='平均預測價格')
        ax.axhline(last_price, color='gray', linestyle='--', label=f'當前價格 ({last_price:.2f})')

        for date, price in zip(pred_dates, pred_prices_avg):
            ax.text(date, price, f'{price:.2f}', ha='center', va='bottom')

        ax.set_title(f'{self.symbol} 未來一週每日股價預測', fontsize=16)
        ax.set_xlabel('日期')
        ax.set_ylabel('預測價格')
        ax.legend()
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        return fig

    def plot_feature_importance(self, importances):
        """繪製特徵重要性圖"""
        fig, ax = plt.subplots(figsize=(12, 8))
        top_10 = importances.head(10)
        ax.barh(top_10['特徵'], top_10['重要性'], color='skyblue')
        ax.invert_yaxis()
        ax.set_title('特徵重要性排名 (隨機森林)', fontsize=16)
        ax.set_xlabel('重要性')
        for index, value in enumerate(top_10['重要性']):
            ax.text(value, index, f'{value:.3f}')
        plt.tight_layout()
        return fig

# =============================================================================
# 8. 通知系統
# =============================================================================

async def send_notification(session: aiohttp.ClientSession, message: str, chart_files: List[str] = None):
    """發送通知到 Telegram 和 Discord"""
    # Telegram 通知
    try:
        if TELEGRAM_TOKEN and TELEGRAM_CHAT_ID:
            max_length = 4000
            if len(message) > max_length:
                parts = []
                current_part = ""
                for line in message.split('\n'):
                    if len(current_part + line + '\n') > max_length:
                        if current_part:
                            parts.append(current_part.strip())
                            current_part = line + '\n'
                        else:
                            parts.append(line[:max_length])
                    else:
                        current_part += line + '\n'
                if current_part:
                    parts.append(current_part.strip())
            else:
                parts = [message]

            telegram_url = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendMessage"

            for chat_id in TELEGRAM_CHAT_ID:
                for i, part in enumerate(parts):
                    if i > 0:
                        part = f"🏆 股票分析報告 (續 {i+1}/{len(parts)})\n\n" + part

                    payload = {'chat_id': chat_id, 'text': part}

                    try:
                        async with session.post(telegram_url, json=payload, timeout=20) as response:
                            if response.status == 200:
                                logger.info(f"成功發送 Telegram 通知到 {chat_id}")
                            else:
                                error_text = await response.text()
                                logger.error(f"發送 Telegram 通知失敗: {response.status} - {error_text}")
                    except Exception as e:
                        logger.error(f"發送 Telegram 通知時發生網路錯誤: {e}")

                    if i < len(parts) - 1:
                        await asyncio.sleep(1)

                # 發送圖表文件
                if chart_files:
                    telegram_photo_url = f"https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendPhoto"
                    for chart_file in chart_files:
                        if os.path.exists(chart_file):
                            try:
                                with open(chart_file, 'rb') as photo:
                                    files = {'photo': photo}
                                    data = {'chat_id': chat_id}

                                    import requests
                                    response = requests.post(telegram_photo_url, files=files, data=data, timeout=30)

                                    if response.status_code == 200:
                                        logger.info(f"成功發送圖表到 Telegram: {chart_file}")
                                    else:
                                        logger.error(f"發送圖表失敗: {response.status_code}")
                            except Exception as e:
                                logger.error(f"發送圖表時發生錯誤: {e}")

    except Exception as e:
        logger.error(f"發送 Telegram 通知時異常: {e}")

    # Discord 通知
    if MESSAGING_AVAILABLE and DISCORD_WEBHOOK_URL:
        try:
            webhook = DiscordWebhook(url=DISCORD_WEBHOOK_URL, content=f"```\n{message}\n```")

            if chart_files:
                for chart_file in chart_files:
                    if os.path.exists(chart_file):
                        with open(chart_file, "rb") as f:
                            webhook.add_file(file=f.read(), filename=os.path.basename(chart_file))

            response = webhook.execute()
            if response.ok:
                logger.info("Discord 通知發送成功")
            else:
                logger.error(f"Discord 通知失敗: {response.status_code}")
        except Exception as e:
            logger.error(f"發送 Discord 通知時異常: {e}")

# =============================================================================
# 9. 股票篩選器
# =============================================================================

def screen_stocks(stock_list, days_back, condition):
    """篩選符合條件的股票"""
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days_back * 2)
    results = pd.DataFrame(columns=['ticker', 'close', 'volume', 'change_pct'])

    for ticker in stock_list:
        try:
            data = yf.download(ticker, start=start_date, end=end_date, progress=False)
            if data.empty or len(data) < 20:
                continue

            data = data.tail(days_back)
            data['change_pct'] = data['Close'].pct_change() * 100

            if condition == "突破整理區間":
                high_20d = data['High'].rolling(window=20).max().shift(1)
                if data['Close'].iloc[-1] > high_20d.iloc[-1]:
                    results = pd.concat([results, pd.DataFrame({
                        'ticker': [ticker],
                        'close': [data['Close'].iloc[-1]],
                        'volume': [data['Volume'].iloc[-1]],
                        'change_pct': [data['change_pct'].iloc[-1]]
                    })], ignore_index=True)

            elif condition == "爆量長紅":
                vol_5d_avg = data['Volume'].rolling(window=5).mean().shift(1)
                if (data['Volume'].iloc[-1] > vol_5d_avg.iloc[-1] * 2 and
                    data['change_pct'].iloc[-1] > 3):
                    results = pd.concat([results, pd.DataFrame({
                        'ticker': [ticker],
                        'close': [data['Close'].iloc[-1]],
                        'volume': [data['Volume'].iloc[-1]],
                        'change_pct': [data['change_pct'].iloc[-1]]
                    })], ignore_index=True)

        except Exception:
            continue

    return results

# =============================================================================
# 10. 主要分析系統
# =============================================================================

class MultiStockAnalysisSystem:
    def __init__(self, symbols, period='1y'):
        self.symbols = symbols if isinstance(symbols, list) else [symbols]
        self.period = period
        self.results = {}

    async def analyze_stock(self, symbol):
        """分析單一股票"""
        try:
            logger.info(f"開始分析 {symbol}...")

            # 1. 獲取股票數據
            stock_df = fetch_stock_data(symbol, self.period)
            if stock_df is None:
                return None

            # 2. 獲取恐懼貪婪指數
            days_needed = len(stock_df)
            sentiment_df = get_fear_and_greed_data(days=days_needed + 60)

            # 3. 合併數據
            df = pd.merge(stock_df, sentiment_df, on='Date', how='left')

            # 4. 技術指標計算
            df = calculate_technical_indicators(df)
            df = feature_engineering(df)

            # 5. K線型態分析
            pattern_report = generate_pattern_report(df)

            # 6. 綜合評分
            combined_score = calculate_combined_score(df)

            # 7. 機器學習預測
            predictor = StockPredictor()
            predictor.run_backtest(df)
            predictor.predict_future(df)

            # 8. 生成建議
            recommendation = generate_recommendation(combined_score)

            # 9. 生成圖表
            visualizer = StockVisualizer(symbol)

            # 保存圖表
            chart_files = []

            # 技術分析圖
            fig1 = visualizer.plot_stock_analysis(df)
            chart_path1 = os.path.join(CHARTS_DIR, f"{symbol}_technical.png")
            fig1.savefig(chart_path1, dpi=150, bbox_inches='tight')
            chart_files.append(chart_path1)
            plt.close(fig1)

            # 預測圖
            if predictor.future_predictions:
                fig2 = visualizer.plot_future_prediction(
                    df['Close'].iloc[-1], df['Date'].iloc[-1], predictor.future_predictions
                )
                chart_path2 = os.path.join(CHARTS_DIR, f"{symbol}_prediction.png")
                fig2.savefig(chart_path2, dpi=150, bbox_inches='tight')
                chart_files.append(chart_path2)
                plt.close(fig2)

            # 特徵重要性圖
            if predictor.feature_importances is not None:
                fig3 = visualizer.plot_feature_importance(predictor.feature_importances)
                chart_path3 = os.path.join(CHARTS_DIR, f"{symbol}_features.png")
                fig3.savefig(chart_path3, dpi=150, bbox_inches='tight')
                chart_files.append(chart_path3)
                plt.close(fig3)

            result = {
                'symbol': symbol,
                'data_df': df,
                'combined_score': combined_score,
                'pattern_report': pattern_report,
                'recommendation': recommendation,
                'predictor': predictor,
                'chart_files': chart_files,
                'last_price': df['Close'].iloc[-1],
                'last_date': df['Date'].iloc[-1]
            }

            logger.info(f"完成分析 {symbol}")
            return result

        except Exception as e:
            logger.error(f"分析 {symbol} 時發生錯誤: {e}")
            traceback.print_exc()
            return None

    async def run_analysis(self):
        """執行多股票分析"""
        logger.info(f"開始分析 {len(self.symbols)} 支股票...")

        # 並行分析所有股票
        tasks = [self.analyze_stock(symbol) for symbol in self.symbols]
        results = await asyncio.gather(*tasks, return_exceptions=True)

        # 整理結果
        for i, result in enumerate(results):
            if isinstance(result, Exception):
                logger.error(f"分析 {self.symbols[i]} 時發生異常: {result}")
            elif result is not None:
                self.results[self.symbols[i]] = result

        # 生成總結報告
        await self.generate_summary_report()

    async def generate_summary_report(self):
        """生成總結報告並發送通知"""
        if not self.results:
            logger.warning("沒有成功分析的股票")
            return

        # 生成報告文本
        report_lines = []
        report_lines.append("📊 多股票分析報告")
        report_lines.append("=" * 50)
        report_lines.append(f"分析時間: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        report_lines.append(f"分析股票數量: {len(self.results)}")
        report_lines.append("")

        # 按評分排序
        sorted_results = sorted(self.results.items(),
                              key=lambda x: x[1]['combined_score'], reverse=True)

        for symbol, result in sorted_results:
            report_lines.append(f"🏢 {symbol}")
            report_lines.append(f"   綜合評分: {result['combined_score']:.2f}")
            report_lines.append(f"   當前價格: {result['last_price']:.2f}")
            report_lines.append(f"   投資建議: {result['recommendation']['action']}")
            report_lines.append(f"   信心度: {result['recommendation']['confidence']}%")
            report_lines.append(f"   建議原因: {result['recommendation']['reason']}")

            # 預測價格
            if result['predictor'].future_predictions:
                pred_1d = result['predictor'].future_predictions.get(1, {})
                if pred_1d:
                    avg_pred = np.mean(list(pred_1d.values()))
                    change_pct = (avg_pred / result['last_price'] - 1) * 100
                    report_lines.append(f"   明日預測: {avg_pred:.2f} ({change_pct:+.2f}%)")

            report_lines.append("")

        # 市場整體分析
        avg_score = np.mean([r['combined_score'] for r in self.results.values()])
        report_lines.append("📈 市場整體分析")
        report_lines.append(f"   平均評分: {avg_score:.2f}")

        if avg_score >= 60:
            market_sentiment = "樂觀"
        elif avg_score >= 40:
            market_sentiment = "中性"
        else:
            market_sentiment = "謹慎"

        report_lines.append(f"   市場情緒: {market_sentiment}")
        report_lines.append("")

        # 推薦股票
        buy_stocks = [symbol for symbol, result in sorted_results
                     if result['recommendation']['action'] in ['買入', '強烈買入']]

        if buy_stocks:
            report_lines.append("💰 推薦買入:")
            for stock in buy_stocks[:5]:  # 最多推薦5支
                score = self.results[stock]['combined_score']
                report_lines.append(f"   • {stock} (評分: {score:.2f})")

        report_lines.append("")
        report_lines.append("⚠️ 免責聲明: 本分析僅供參考，投資有風險，請謹慎決策。")

        report_text = "\n".join(report_lines)

        # 收集所有圖表文件
        all_chart_files = []
        for result in self.results.values():
            all_chart_files.extend(result.get('chart_files', []))

        # 發送通知
        async with aiohttp.ClientSession() as session:
            await send_notification(session, report_text, all_chart_files)

        # 終端顯示
        print("\n" + report_text)

# =============================================================================
# 11. UI 介面 (Jupyter Notebook)
# =============================================================================

if JUPYTER_AVAILABLE:
    def create_stock_analysis_ui():
        """創建股票分析UI介面"""
        # 樣式設定
        style = {'description_width': '120px'}
        layout = Layout(width='300px')

        # 控制元件
        symbol_input = widgets.Text(
            value='AAPL,TSLA,2330,2454',
            placeholder='輸入股票代碼，用逗號分隔',
            description='股票代碼:',
            style=style,
            layout=Layout(width='400px')
        )

        period_dropdown = widgets.Dropdown(
            options=[('1年', '1y'), ('2年', '2y'), ('5年', '5y'), ('最大', 'max')],
            value='1y',
            description='時間範圍:',
            style=style,
            layout=layout
        )

        analyze_button = widgets.Button(
            description='開始分析',
            button_style='primary',
            layout=Layout(width='150px', height='40px')
        )

        output_area = widgets.Output()

        def on_analyze_click(b):
            with output_area:
                clear_output(wait=True)
                print("🚀 開始分析...")

                symbols = [s.strip().upper() for s in symbol_input.value.split(',') if s.strip()]
                if not symbols:
                    print("❌ 請輸入有效的股票代碼")
                    return

                try:
                    system = MultiStockAnalysisSystem(symbols, period_dropdown.value)

                    # 在 Jupyter 中運行異步代碼
                    import asyncio
                    loop = asyncio.get_event_loop()
                    loop.run_until_complete(system.run_analysis())

                    print("✅ 分析完成！請查看生成的圖表和通知。")

                except Exception as e:
                    print(f"❌ 分析過程中發生錯誤: {e}")
                    traceback.print_exc()

        analyze_button.on_click(on_analyze_click)

        # 組合UI
        ui = VBox([
            HTML('<h2>📈 多股票分析系統</h2>'),
            HTML('<p>支援台股(輸入數字代碼如2330)和美股(輸入字母代碼如AAPL)</p>'),
            symbol_input,
            period_dropdown,
            analyze_button,
            output_area
        ])

        return ui

    def create_stock_screener_ui():
        """創建股票篩選器UI"""
        # 台股代碼列表 (簡化版)
        tw_stocks = [
            '2330', '2317', '2454', '2881', '2882', '2883', '2884', '2885',
            '2886', '2887', '2888', '2889', '2890', '2891', '2892', '2912'
        ]

        # 美股代碼列表 (簡化版)
        us_stocks = [
            'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA', 'NFLX',
            'JPM', 'JNJ', 'V', 'PG', 'UNH', 'HD', 'MA', 'DIS'
        ]

        style = {'description_width': '120px'}
        layout = Layout(width='300px')

        market_dropdown = widgets.Dropdown(
            options=[('台股', 'tw'), ('美股', 'us')],
            value='tw',
            description='市場:',
            style=style,
            layout=layout
        )

        condition_dropdown = widgets.Dropdown(
            options=[
                ('突破整理區間', '突破整理區間'),
                ('爆量長紅', '爆量長紅'),
                ('突破季線', '突破季線'),
                ('多頭吞噬', '多頭吞噬'),
                ('5與20日均線黃金交叉', '5與20日均線黃金交叉')
            ],
            value='突破整理區間',
            description='篩選條件:',
            style=style,
            layout=Layout(width='400px')
        )

        days_slider = widgets.IntSlider(
            value=30,
            min=10,
            max=100,
            step=10,
            description='回看天數:',
            style=style,
            layout=layout
        )

        screen_button = widgets.Button(
            description='開始篩選',
            button_style='success',
            layout=Layout(width='150px', height='40px')
        )

        output_area = widgets.Output()

        def on_screen_click(b):
            with output_area:
                clear_output(wait=True)
                print("🔍 開始篩選股票...")

                stock_list = tw_stocks if market_dropdown.value == 'tw' else us_stocks

                try:
                    results = screen_stocks(stock_list, days_slider.value, condition_dropdown.value)

                    if results.empty:
                        print("❌ 沒有找到符合條件的股票")
                    else:
                        print(f"✅ 找到 {len(results)} 支符合條件的股票:")
                        print("\n" + "="*60)
                        for _, row in results.iterrows():
                            print(f"股票: {row['ticker']}")
                            print(f"收盤價: {row['close']:.2f}")
                            print(f"成交量: {row['volume']:,.0f}")
                            print(f"漲跌幅: {row['change_pct']:.2f}%")
                            print("-" * 30)

                except Exception as e:
                    print(f"❌ 篩選過程中發生錯誤: {e}")
                    traceback.print_exc()

        screen_button.on_click(on_screen_click)

        ui = VBox([
            HTML('<h2>🔍 股票篩選器</h2>'),
            market_dropdown,
            condition_dropdown,
            days_slider,
            screen_button,
            output_area
        ])

        return ui

# =============================================================================
# 12. 命令行介面
# =============================================================================

def main():
    """主函數，運行互動式界面"""
    while True:
        print("\n\n🚀 歡迎使用多股票分析預測系統")
        print("="*50)
        print("1. 多股票分析與預測")
        print("2. 股票篩選器")
        print("3. 單股票詳細分析")
        if JUPYTER_AVAILABLE:
            print("4. 啟動 Jupyter UI 介面")
        print("0. 退出程序")
        print("-"*50)

        try:
            choice = input("請選擇功能 (0-4): ").strip()

            if choice == '0':
                print("感謝使用，程序退出。")
                break

            elif choice == '1':
                symbols_input = input("請輸入股票代碼 (用逗號分隔，如: AAPL,TSLA,2330,2454): ").strip()
                if not symbols_input:
                    print("❌ 請輸入有效的股票代碼")
                    continue

                symbols = [s.strip().upper() for s in symbols_input.split(',') if s.strip()]
                period = input("請輸入歷史數據區間 (1y, 2y, 5y, max) [預設: 1y]: ").strip().lower()
                if not period:
                    period = '1y'

                print(f"🚀 開始分析 {len(symbols)} 支股票...")

                try:
                    system = MultiStockAnalysisSystem(symbols, period)
                    asyncio.run(system.run_analysis())
                    print("✅ 分析完成！")
                except Exception as e:
                    print(f"❌ 分析過程中發生錯誤: {e}")
                    traceback.print_exc()

            elif choice == '2':
                print("\n股票篩選器")
                print("1. 台股篩選")
                print("2. 美股篩選")

                market_choice = input("請選擇市場 (1-2): ").strip()
                if market_choice not in ['1', '2']:
                    print("❌ 無效選擇")
                    continue

                conditions = [
                    "突破整理區間", "爆量長紅", "突破季線", "多頭吞噬", "5與20日均線黃金交叉"
                ]

                print("\n篩選條件:")
                for i, condition in enumerate(conditions, 1):
                    print(f"{i}. {condition}")

                condition_choice = input("請選擇條件 (1-5): ").strip()
                try:
                    condition_idx = int(condition_choice) - 1
                    if condition_idx < 0 or condition_idx >= len(conditions):
                        print("❌ 無效選擇")
                        continue
                    selected_condition = conditions[condition_idx]
                except ValueError:
                    print("❌ 請輸入數字")
                    continue

                # 簡化的股票列表
                tw_stocks = ['2330.TW', '2317.TW', '2454.TW', '2881.TW', '2882.TW']
                us_stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']

                stock_list = tw_stocks if market_choice == '1' else us_stocks

                try:
                    results = screen_stocks(stock_list, 30, selected_condition)
                    if results.empty:
                        print("❌ 沒有找到符合條件的股票")
                    else:
                        print(f"✅ 找到 {len(results)} 支符合條件的股票:")
                        print(results.to_string(index=False))
                except Exception as e:
                    print(f"❌ 篩選過程中發生錯誤: {e}")

            elif choice == '3':
                symbol = input("請輸入股票代碼: ").strip().upper()
                if not symbol:
                    print("❌ 請輸入有效的股票代碼")
                    continue

                period = input("請輸入歷史數據區間 (1y, 2y, 5y, max) [預設: 1y]: ").strip().lower()
                if not period:
                    period = '1y'

                try:
                    system = MultiStockAnalysisSystem([symbol], period)
                    asyncio.run(system.run_analysis())
                    print("✅ 分析完成！")
                except Exception as e:
                    print(f"❌ 分析過程中發生錯誤: {e}")

            elif choice == '4' and JUPYTER_AVAILABLE:
                print("🚀 啟動 Jupyter UI 介面...")
                print("請在 Jupyter Notebook 中運行以下代碼:")
                print("ui = create_stock_analysis_ui()")
                print("display(ui)")
                break

            else:
                print("❌ 無效的選擇，請重新輸入")

        except KeyboardInterrupt:
            print("\n\n操作被中斷。返回主選單...")
            continue
        except Exception as e:
            print(f"\n❌ 發生錯誤: {str(e)}")
            input("\n按Enter鍵返回主選單...")

if __name__ == "__main__":
    main()
    # 創建分析介面
    ui = create_stock_analysis_ui()
    display(ui)

    # 創建篩選介面
    screener_ui = create_stock_screener_ui()
    display(screener_ui)